<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/Dislocation_ALIGNNFF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
!pip install  -q dgl -f https://data.dgl.ai/wheels/cu118/repo.html
!pip install  -q dglgo -f https://data.dgl.ai/wheels-test/repo.html
!pip install -q alignn phonopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.3/91.3 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 39.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 44.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wh

In [2]:
from jarvis.core.atoms import Atoms
from jarvis.db.figshare import data,get_jid_data
atoms_si=Atoms.from_dict(get_jid_data(jid='JVASP-1002',dataset='dft_3d')['atoms']).get_conventional_atoms
print(atoms_si)

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699


100%|██████████| 40.8M/40.8M [00:01<00:00, 23.6MiB/s]


Loading the zipfile...
Loading completed.
System
1.0
5.49363 0.0 0.0
0.0 5.49363 0.0
0.0 0.0 5.49363
Si 
8 
direct
0.25 0.75 0.25 Si
0.0 0.0 0.5 Si
0.25 0.25 0.75 Si
0.0 0.5 0.0 Si
0.75 0.75 0.75 Si
0.5 0.0 0.0 Si
0.75 0.25 0.25 Si
0.5 0.5 0.5 Si



In [4]:
from alignn.graphs import Graph
from alignn.pretrained import get_figshare_model
import torch

device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")

def atom_to_energy(model='',atoms=''):
    g, lg = Graph.atom_dgl_multigraph(atoms)
    out_data = (
        model([g.to(device), lg.to(device)])
        .detach()
        .cpu()
        .numpy()
        .flatten()
        .tolist()[0]
    )
    return out_data


model = get_figshare_model("jv_optb88vdw_total_energy_alignn")
model.to(device)
atom_to_energy(model=model,atoms=atoms_si)

100%|██████████| 47.5M/47.5M [00:02<00:00, 22.1MiB/s]


Using chk file jv_optb88vdw_total_energy_alignn/checkpoint_300.pt from  ['jv_optb88vdw_total_energy_alignn/checkpoint_300.pt']
Path /usr/local/lib/python3.10/dist-packages/alignn/jv_optb88vdw_total_energy_alignn.zip
Config /content/jv_optb88vdw_total_energy_alignn/config.json


-4.167013168334961

In [13]:
from alignn.pretrained import get_figshare_model,get_multiple_predictions
from jarvis.db.jsonutils import loadjson
pr=get_multiple_predictions(atoms_array=[atoms_si],model_name="jv_optb88vdw_total_energy_alignn",workers=0,pin_memory=False)
d=loadjson('pred_data.json')
print()
print('Energy per atom eV',d[0]['pred'])

Using chk file jv_optb88vdw_total_energy_alignn/checkpoint_300.pt from  ['jv_optb88vdw_total_energy_alignn/checkpoint_300.pt']
Path /usr/local/lib/python3.10/dist-packages/alignn/jv_optb88vdw_total_energy_alignn.zip
Config /content/jv_optb88vdw_total_energy_alignn/config.json
data range -9999 -9999


100%|██████████| 1/1 [00:00<00:00, 23.24it/s]


df                                                atoms  prop jid
0  {'lattice_mat': [[5.49363, 0.0, 0.0], [0.0, 5.... -9999   0
building line graphs


100%|██████████| 1/1 [00:00<00:00, 307.73it/s]


Energy per atom eV -4.167519569396973


#Mo231, Mo924, Mo3696

In [ ]:
from ase import Atom, Atoms
import numpy as np
import matplotlib.pyplot as plt
from jarvis.io.vasp.inputs import Poscar
from alignn.ff.ff import AlignnAtomwiseCalculator,default_path,wt10_path,alignnff_fmult,fd_path,ForceField
# Using the path for newly trained model
model_path = alignnff_fmult()
calc = AlignnAtomwiseCalculator(path=model_path)

In [ ]:
Mo231="""Mo
1.0000000000000e+00
2.7387360574360e+00 0.0000000000000e+00 0.0000000000000e+00
0.0000000000000e+00 5.4224207466607e+01 0.0000000000000e+00
-1.3693680287180e+00 2.7112103733303e+01 2.4597854896560e+01
Mo
231
direct
8.1083670055224e-01 1.8925215481829e-01 6.2148872685933e-01
1.3105592873447e-01 1.9393803172113e-01 8.0403029755124e-01
7.5111746742262e-01 2.1527023462658e-01 7.1353329431684e-01
3.6245445495090e-01 2.3665260697414e-01 6.2306135279863e-01
5.2183912310785e-01 1.9928676601412e-01 9.8489144712776e-01
1.4647319745291e-01 2.2060679297492e-01 8.9440434991627e-01
7.6536050948354e-01 2.4185672468471e-01 8.0402563106911e-01
3.7679294203883e-01 2.6309231896030e-01 7.1367997066980e-01
9.7576278764558e-01 2.8433015853751e-01 6.2330346508602e-01
9.2873690587348e-01 2.0345966685071e-01 1.6703013590599e-01
5.5728292216490e-01 2.2533632481101e-01 7.5832784098810e-02
1.7985947731343e-01 2.4704190741161e-01 9.8484788971688e-01
7.9900597429814e-01 2.6851817634110e-01 8.9415746634425e-01
4.1294310047424e-01 2.8983211713322e-01 8.0366520471992e-01
2.0347737691039e-02 3.1103951935684e-01 7.1326294715916e-01
6.1823621789062e-01 3.3217246953619e-01 6.2282793664829e-01
3.3924138968260e-01 2.0661036987807e-01 3.5031214500320e-01
9.7896669472002e-01 2.2868844245582e-01 2.5886953653345e-01
6.0899985908891e-01 2.5074629095100e-01 1.6741486367194e-01
2.3245779738101e-01 2.7274454694564e-01 7.6038677993038e-02
8.4928955627816e-01 2.9464680983199e-01 9.8480145475670e-01
4.6282151526486e-01 3.1632474218560e-01 8.9385186549830e-01
7.1775126500826e-02 3.3777662844252e-01 8.0315703834365e-01
6.7565578908045e-01 3.5902157224280e-01 7.1263354480886e-01
2.7314240447168e-01 3.8010817053254e-01 6.2215318797687e-01
5.9892813922550e-01 2.0912457487302e-01 5.3289071938415e-01
3.8628821110475e-01 2.3207546282834e-01 4.4188411118973e-01
4.4381442751425e-02 2.5404994024507e-01 3.5040230526997e-01
6.7742271373375e-01 2.7601913864082e-01 2.5898044056095e-01
3.0131560613251e-01 2.9802762423375e-01 1.6757278727673e-01
9.1916420587141e-01 3.2005960047053e-01 7.6163043316263e-02
5.3020257603142e-01 3.4209934006103e-01 9.8476471582133e-01
1.3776947546078e-01 3.6398665340638e-01 8.9356490464898e-01
7.4092914329240e-01 3.8563939453573e-01 8.0262496873136e-01
3.4010476814267e-01 4.0699582039797e-01 7.1194177068513e-01
9.3533515602781e-01 4.2809571568105e-01 6.2140129288647e-01
9.3196461468834e-01 2.5797720363873e-01 5.3288877868711e-01
2.2006361248347e-01 2.8014912028214e-01 4.3969183157197e-01
7.8791117107443e-01 3.0179501893842e-01 3.4923673968287e-01
3.9507281916615e-01 3.2354532243975e-01 2.5839886859852e-01
6.8032440899568e-03 3.4540725924587e-01 1.6734063569680e-01
6.1737954657577e-01 3.6735860842526e-01 7.6128495711014e-02
2.2263734444082e-01 3.8942020604484e-01 9.8475345840953e-01
8.2425984847160e-01 4.1146435099719e-01 8.9338559521801e-01
4.2071532185976e-01 4.3336580371592e-01 8.0217068698561e-01
1.3248558147671e-02 4.5492584010817e-01 7.1126922676164e-01
6.0336656700993e-01 4.7610115460545e-01 6.2063568824319e-01
5.4566173873671e-01 3.0539575581449e-01 5.3273570506302e-01
5.3829818822242e-02 3.2603237149290e-01 4.4188055040860e-01
5.6145869762298e-01 3.4895782728748e-01 3.4923512462848e-01
1.2931605758595e-01 3.7109577855355e-01 2.5779601169676e-01
7.2348009987900e-01 3.9288750563426e-01 1.6690578706715e-01
3.2529243192126e-01 4.1471179142883e-01 7.5952162575169e-02
9.2546231738964e-01 4.3667292833721e-01 9.8477644394354e-01
5.2262148330590e-01 4.5873148601684e-01 8.9342457800237e-01
1.1267210319900e-01 4.8085692582160e-01 8.0195833996332e-01
6.9650566133911e-01 5.0274499184677e-01 7.1071858284509e-01
2.7768224341068e-01 5.2409737621899e-01 6.1991292638627e-01
2.0019458742248e-01 3.5315448423160e-01 5.3226097051134e-01
7.5890699694835e-01 3.7392534512624e-01 4.4161175890890e-01
3.0496772637416e-01 3.9553913710455e-01 3.5039750542561e-01
8.6354779963895e-01 4.1804491116722e-01 2.5839602408063e-01
4.4442459857823e-01 4.4019389004711e-01 1.6690456221410e-01
3.8600667132353e-02 4.6207199469926e-01 7.5841698720412e-02
6.3542039849126e-01 4.8394496532772e-01 9.8480924356884e-01
2.3144024651743e-01 5.0586893726024e-01 8.9367168003721e-01
8.1892214140564e-01 5.2800393937945e-01 8.0220433777136e-01
3.9406362053816e-01 5.5027889040564e-01 7.1052274470024e-01
9.6114292235276e-01 5.7204203935175e-01 6.1931387776456e-01
8.6184974396260e-01 4.0105691940528e-01 5.3162435416292e-01
4.3975314347826e-01 4.2191034013117e-01 4.4105108164414e-01
1.0066363908821e-02 4.4307080079482e-01 3.5030429640844e-01
5.8116340869607e-01 4.6499123363689e-01 2.5897484163531e-01
1.6107617607409e-01 4.8724073017323e-01 1.6733700585001e-01
7.5189512714319e-01 5.0932286331619e-01 7.5950161234439e-02
3.4784225031707e-01 5.3126095522504e-01 9.8480839166295e-01
9.4628261497170e-01 5.5309171395114e-01 8.9383062078259e-01
5.3847195934055e-01 5.7498463449336e-01 8.0274424244705e-01
1.1367612059366e-01 5.9722669136322e-01 7.1111674419037e-01
6.6297501005877e-01 6.1975005144777e-01 6.1906815036626e-01
5.2630565076793e-01 4.4903279087928e-01 5.3088643155250e-01
1.1280595866817e-01 4.6991357289534e-01 4.4035596733875e-01
6.9616438638287e-01 4.9094878457324e-01 3.4975267654182e-01
2.7956190954947e-01 5.1243467658196e-01 2.5886136357786e-01
8.6651324472997e-01 5.3438982869349e-01 1.6756688664790e-01
4.5976668141495e-01 5.5650105507178e-01 7.6124581692363e-02
5.7829393615583e-02 5.7856635346029e-01 9.8477393225826e-01
6.6113838021686e-01 6.0047438095012e-01 8.9366952953251e-01
2.6338371709081e-01 6.2228487633621e-01 8.0274294816832e-01
8.5556140503035e-01 6.4406475291923e-01 7.1188254695375e-01
4.1153054241650e-01 6.6636103694816e-01 6.2022460979498e-01
1.9251282530205e-01 4.9704064443248e-01 5.3010638944769e-01
7.8177583056972e-01 5.1792757664813e-01 4.3959095807562e-01
3.7189462691440e-01 5.3891913473689e-01 3.4905022610713e-01
9.6383112336983e-01 5.6018674147879e-01 2.5837148185943e-01
5.5875310614256e-01 5.8183082843620e-01 1.6740691619508e-01
1.5791311881399e-01 6.0376702546058e-01 7.6157394085657e-02
7.6071250072425e-01 6.2584228866987e-01 9.8474941910946e-01
3.6999733956884e-01 6.4785958798060e-01 8.9342036592830e-01
9.8295805326069e-01 6.6980651393676e-01 8.0220049976493e-01
5.9745780394738e-01 6.9166999548272e-01 7.1111377136693e-01
2.0884942449073e-01 7.1342598683876e-01 6.2022293592509e-01
8.6032138020085e-01 5.4505055209587e-01 5.2934028329643e-01
4.4760590977982e-01 5.6593323782243e-01 4.3881237243733e-01
4.0201663263737e-02 5.8691409158648e-01 3.4830030869294e-01
6.3746526659625e-01 6.0807841907375e-01 2.5774512896821e-01
2.3891481968996e-01 6.2950400572937e-01 1.6702045560637e-01
8.4452302470700e-01 6.5120807772795e-01 7.6031560098739e-02
4.5323449465893e-01 6.7315174645940e-01 9.8475936940460e-01
6.8425987137678e-02 6.9516601929894e-01 8.9337949958853e-01
6.8925714125612e-01 7.1720026180690e-01 8.0195209279448e-01
3.1710365502577e-01 7.3921291267703e-01 7.1051688935963e-01
9.5742488875846e-01 7.6119488469578e-01 6.1906317415310e-01
5.3023922059831e-01 5.9303992849123e-01 5.2864541218010e-01
1.1001442681719e-01 6.1390029999758e-01 4.3807487659268e-01
7.0155821904334e-01 6.3490157818397e-01 3.4756526427787e-01
3.0174685534887e-01 6.5603573143391e-01 2.5709724394328e-01
9.0815653955200e-01 6.7732763759324e-01 1.6655817731006e-01
5.1957376057690e-01 6.9882390546876e-01 7.5824553586151e-02
1.3426386008197e-01 7.2056696483953e-01 9.8479511038798e-01
7.5501021802172e-01 7.4246433624754e-01 8.9355719284149e-01
3.8128753047069e-01 7.6447942716853e-01 8.0216225518246e-01
1.4717224490203e-02 7.8655178308156e-01 7.1071002576020e-01
6.5929685729787e-01 8.0865835587490e-01 6.1930573548212e-01
2.0433737743023e-01 6.4099983969152e-01 5.2808242571976e-01
7.6707374338046e-01 6.6178279262424e-01 4.3743786538969e-01
3.5444273438092e-01 6.8284930479560e-01 3.4691918403234e-01
9.5626736568071e-01 7.0401793246952e-01 2.5651931386279e-01
5.6673292231189e-01 7.2524982759868e-01 1.6611933122866e-01
1.8334982096040e-01 7.4659044866202e-01 7.5603260906103e-02
8.0383944262475e-01 7.6812439320412e-01 9.8484094568830e-01
4.3007892077827e-01 7.8983878607637e-01 8.9384289467710e-01
6.1171914535618e-02 8.1175155358977e-01 8.0261466716089e-01
6.9805223486980e-01 8.3382074870988e-01 7.1125823949109e-01
3.4281739994612e-01 8.5600484472806e-01 6.1990184543860e-01
8.9306471495340e-01 6.8897878716525e-01 5.2771282086854e-01
4.1136705537047e-01 7.0950982703272e-01 4.3698051104991e-01
9.9374196196686e-01 7.3071707222652e-01 3.4652954714928e-01
5.9873269419489e-01 7.5197443447494e-01 2.5617566093046e-01
2.1390370472010e-01 7.7321727299275e-01 1.6582236419694e-01
8.3573966673880e-01 7.9447564791721e-01 7.5426604965273e-02
4.6203450453913e-01 8.1583442773400e-01 9.8488439144216e-01
9.4042139128636e-02 8.3733877642618e-01 8.9414768401749e-01
7.3044878666720e-01 8.5908181249527e-01 8.0314527273243e-01
3.7129628141530e-01 8.8107828445374e-01 7.1192871604758e-01
1.7212973655111e-02 9.0327882073151e-01 6.2062198758253e-01
7.6480913286529e-01 7.3500408178520e-01 5.3000129794349e-01
3.1246676739484e-04 7.5696451530116e-01 4.3742743593489e-01
6.0543466971612e-01 7.7840132603047e-01 3.4697370130518e-01
2.2445904323398e-01 7.9980263090023e-01 2.5638707635895e-01
8.4830353765880e-01 8.2113997159609e-01 1.6584541909558e-01
4.7646663910740e-01 8.4242006962636e-01 7.5371742530595e-02
1.0870047246095e-01 8.6368984026826e-01 9.8491146927211e-01
7.4674958836174e-01 8.8500178247869e-01 8.9439429462010e-01
3.8942838235205e-01 9.0651723776264e-01 8.0365247231364e-01
3.5867933222358e-02 9.2836037463081e-01 7.1261887750769e-01
6.8534438282029e-01 9.5051876687605e-01 6.2138538359258e-01
6.3656235979951e-01 7.8331959735075e-01 5.2770915335032e-01
4.3853628693739e-01 8.0561701594086e-01 4.3742544739396e-01
1.7403243748547e-01 8.2589243191691e-01 3.4822157194220e-01
8.3037155565808e-01 8.4743042477338e-01 2.5718048348622e-01
4.6987398024576e-01 8.6893341034526e-01 1.6624035479132e-01
1.0625431029752e-01 8.9033814483347e-01 7.5489152551084e-02
7.4421411837689e-01 9.1165035542872e-01 9.8490714526874e-01
3.8769160646691e-01 9.3286706244503e-01 8.9447740393333e-01
3.7183333508161e-02 9.5413075627687e-01 8.0401252142322e-01
6.9132115440941e-01 9.7571215118777e-01 7.1324721455028e-01
3.4765731693027e-01 9.9775407773189e-01 6.2213557399156e-01
3.2531613384048e-01 8.3093045841007e-01 5.2807518188162e-01
2.7496993694989e-02 8.5352043377598e-01 4.3697459114321e-01
7.4263668172627e-01 8.7463345262831e-01 3.4696947106178e-01
4.2690381035239e-01 8.9539500396654e-01 2.5717824740465e-01
8.3238014393396e-02 9.1673937456329e-01 1.6652438478903e-01
7.2822199141225e-01 9.3820712863990e-01 7.5662030613646e-02
3.7074140069167e-01 9.5963275783290e-01 9.8487677030659e-01
1.7159149613783e-02 9.8092261900402e-01 8.9431729380189e-01
6.7218329564704e-01 2.0236435373137e-03 8.0403939632655e-01
3.3563291480685e-01 2.3217371231083e-02 7.1369073955827e-01
3.3753115633360e-03 4.4987262624120e-02 6.2284000766624e-01
9.9945838300682e-01 8.7832863029394e-01 5.2863477459552e-01
6.7182091719128e-01 9.0079168873954e-01 4.3742815135817e-01
3.5434883023087e-01 9.2276371700905e-01 3.4652117824530e-01
3.2826923450228e-02 9.4381829566605e-01 2.5638041389100e-01
6.9660616142152e-01 9.6483164564393e-01 1.6623569827766e-01
3.4745127202639e-01 9.8613346639575e-01 7.5659616395237e-02
9.9243266561575e-01 7.5689592728427e-03 9.8486179571371e-01
6.3822811149223e-01 2.8996747622286e-02 8.9407768658405e-01
2.9194378645992e-01 5.0256939656592e-02 8.0356787689673e-01
9.6133118834813e-01 7.1187981646911e-02 7.1353984727922e-01
6.4588313331611e-01 9.2355609484867e-02 6.2331166514951e-01
6.6943801865247e-01 9.2562406528046e-01 5.2932652626625e-01
3.2892620292925e-01 9.4803846904334e-01 4.3806159747884e-01
9.9368049686359e-01 9.7024347000030e-01 3.4690685982602e-01
6.5857449840110e-01 9.9185980674311e-01 2.5616470950651e-01
3.1868852297405e-01 1.3003979267695e-02 1.6585706792763e-01
9.6982836232816e-01 3.4165120433859e-02 7.5496461490833e-02
6.1383214906156e-01 5.5491403411520e-02 9.8488032473123e-01
2.5583018982121e-01 7.6924772430243e-02 8.9407529713751e-01
9.0162863691284e-01 9.8399955231030e-02 8.0319639284248e-01
5.6233814305379e-01 1.1970388902634e-01 7.1260136109358e-01
2.5921207887438e-01 1.4027707795469e-01 6.2306549860782e-01
3.3732613742731e-01 9.7286843255896e-01 5.3008987952400e-01
9.9139610897478e-01 9.9526901162678e-01 4.3879584954380e-01
6.4737581826556e-01 1.7517801340165e-02 3.4758212098304e-01
3.0180001585846e-01 3.9448345188084e-02 2.5653415967066e-01
9.5329162250793e-01 6.0947600416749e-02 1.6583444942964e-01
5.9985128454679e-01 8.2197821581176e-02 7.5380270732521e-02
2.4009676014508e-01 1.0344488008119e-01 9.8491259781737e-01
8.7649425279724e-01 1.2475966920575e-01 8.9432591119303e-01
5.1130897823141e-01 1.4617383900615e-01 8.0356328175813e-01
1.5012190021103e-01 1.6769050775069e-01 7.1259916151112e-01
4.4260817797593e-03 2.0065933626121e-02 5.3090252495272e-01
6.5812106274042e-01 4.2465875138713e-02 4.3960693665236e-01
3.0885714050098e-01 6.4769811114118e-02 3.4831557539576e-01
9.5647063216021e-01 8.6851642215165e-02 2.5711114843355e-01
6.0064005114457e-01 1.0861652491665e-01 1.6613117204280e-01
2.4078562797119e-01 1.3008521060703e-01 7.5435631737706e-02
8.7537712104383e-01 1.5140263824612e-01 9.8491805820255e-01
5.0573291629778e-01 1.7265670302513e-01 8.9448710183182e-01
6.6894647757883e-01 6.7304786166094e-02 5.3163729887627e-01
3.2717396319210e-01 8.9715355004650e-02 4.4036929857765e-01
9.7726800078400e-01 1.1201487456794e-01 3.4906339979633e-01
6.2087907734758e-01 1.3416059077266e-01 2.5775754605990e-01
2.5936854254375e-01 1.5609884756508e-01 1.6656918580509e-01
8.9335503826457e-01 1.7779215228754e-01 7.5612157949072e-02
3.3064993050983e-01 1.1457183735977e-01 5.3227044550241e-01
2.9127134143552e-04 1.3702432859743e-01 4.4106139947650e-01
6.5308120301330e-01 1.5928320823743e-01 3.4976334687832e-01
2.9461690482054e-01 1.8142588574365e-01 2.5838195788606e-01
9.8521494323459e-01 1.6185734576541e-01 5.3274148147357e-01
6.8118343872188e-01 1.8444982484825e-01 4.4161878938696e-01"""


In [ ]:
%%time

atoms = Poscar.from_string(Mo231).atoms
ase_atoms = atoms.ase_converter()
ase_atoms.calc=calc
ase_atoms.get_potential_energy()

CPU times: user 563 ms, sys: 33.2 ms, total: 596 ms
Wall time: 1.54 s


-1830.3267517089844

In [ ]:
Mo924="""Mo
1.0000000000000e+00
2.7387360574360e+00 0.0000000000000e+00 0.0000000000000e+00
0.0000000000000e+00 1.0844841493321e+02 0.0000000000000e+00
-1.3693680287180e+00 5.4224207466607e+01 4.9195709793119e+01
Mo
924
direct
7.8057657886396e-01 2.1962113233225e-01 5.6075373882164e-01
6.1231996950554e-02 2.2173007291550e-01 6.5227072510499e-01
7.0066181089979e-01 2.3250150167680e-01 6.0690621527480e-01
3.3302086656594e-01 2.4327098910267e-01 5.6158931233558e-01
4.1928630986633e-01 2.2405597634310e-01 7.4311804053435e-01
5.8979566518167e-02 2.3479392783437e-01 6.9779027441301e-01
6.9552622517128e-01 2.4553662526408e-01 6.5246798102419e-01
3.2668041927576e-01 2.5628133262532e-01 6.0715609079884e-01
9.4670995730360e-01 2.6701450698260e-01 5.6184800658571e-01
7.8609629527990e-01 2.2648660159886e-01 8.3375548456820e-01
4.2797618928321e-01 2.3720818580528e-01 7.8843883648362e-01
6.7933972471773e-02 2.4793053168028e-01 7.4312429074689e-01
7.0545591910899e-01 2.5865510887867e-01 6.9780981868407e-01
3.3959137136648e-01 2.6938172362700e-01 6.5249280218294e-01
9.6852269527273e-01 2.8010582617829e-01 6.0716642214905e-01
5.8863294462954e-01 2.9080785704641e-01 5.6181277477808e-01
1.5567613305108e-01 2.2891698340390e-01 9.2436869807941e-01
7.9931330699702e-01 2.3965604057918e-01 8.7902405550226e-01
4.4150824786490e-01 2.5038378659391e-01 8.3369514211044e-01
8.2081625043345e-02 2.6110419247754e-01 7.8837600113757e-01
7.2075600199363e-01 2.7182050509765e-01 7.4306108925135e-01
3.5708104663907e-01 2.8253479479875e-01 6.9774495133417e-01
9.9030537467465e-01 2.9324687791013e-01 6.5242121959022e-01
6.1915862757457e-01 3.0395152277677e-01 6.0708109052432e-01
2.4151875562681e-01 3.1463183611073e-01 5.6171353839132e-01
8.8361970589808e-01 2.2037075613436e-01 6.0615692613640e-02
5.2949389070859e-01 2.3119754365768e-01 1.5155284731607e-02
1.7307437868747e-01 2.4201091773789e-01 9.6971381502486e-01
8.1665017903960e-01 2.5278702675124e-01 9.2431958166816e-01
4.5897221162102e-01 2.6354213042262e-01 8.7895260061449e-01
9.9933744354777e-02 2.7427984810542e-01 8.3360700105234e-01
7.3937668817613e-01 2.8500404478061e-01 7.8827601023694e-01
3.7706596765263e-01 2.9571830649251e-01 7.4295246428073e-01
1.2643847010157e-02 3.0642521248873e-01 6.9762900234250e-01
6.4555726561908e-01 3.1712513115199e-01 6.5229798374141e-01
2.7495564165636e-01 3.2781411156691e-01 6.0695163399885e-01
8.9959292328620e-01 3.3848089806018e-01 5.6158409760818e-01
2.5908925645893e-01 2.2237671346511e-01 1.5173237544900e-01
9.0605186248384e-01 2.3327283761968e-01 1.0618384226924e-01
5.5172616193331e-01 2.4414859935323e-01 6.0660362252543e-02
1.9619509191302e-01 2.5500107496935e-01 1.5166962182016e-02
8.3828004358319e-01 2.6584156153526e-01 9.6969096743593e-01
4.8046626862450e-01 2.7664201318403e-01 9.2426509388786e-01
1.2146301975832e-01 2.8741742440771e-01 8.7887029051748e-01
7.6119818098385e-01 2.9816991746190e-01 8.3350177931292e-01
3.9955285855889e-01 3.0890258536356e-01 7.8815293884640e-01
3.6347814910723e-02 3.1961892297603e-01 7.4281601324074e-01
6.7131840258478e-01 3.3032198292991e-01 6.9748279896163e-01
3.0407636041347e-01 3.4101319704240e-01 6.5214530093595e-01
9.3406574417663e-01 3.5169088791743e-01 6.0679666961518e-01
5.6054819442686e-01 3.6234914136991e-01 5.6143314658591e-01
6.3638877174153e-01 2.2425729062936e-01 2.4299356771709e-01
2.8537913619598e-01 2.3518452883998e-01 1.9740372089362e-01
9.3261380738754e-01 2.4610484382321e-01 1.5182101542365e-01
5.7832311531162e-01 2.5701458512298e-01 1.0625170660956e-01
2.2266135946083e-01 2.6790963364409e-01 6.0702122744325e-02
8.6573613991803e-01 2.7878578164950e-01 1.5178202381766e-02
5.0632499532894e-01 2.8965320299074e-01 9.6966813844509e-01
1.4709290608977e-01 3.0047982215740e-01 9.2420893532814e-01
7.8671966886854e-01 3.1127890855969e-01 8.7878305465281e-01
4.2517679378133e-01 3.2205046691766e-01 8.3338757801164e-01
6.2394035934667e-02 3.3279616472331e-01 7.8801690608369e-01
6.9825314465114e-01 3.4351893081984e-01 7.4266349924048e-01
3.3257631953902e-01 3.5422214736651e-01 6.9731890784740e-01
9.6510844762332e-01 3.6490851826732e-01 6.5197494361864e-01
5.9550044638320e-01 3.7557887510641e-01 6.0662499366284e-01
2.2331890651615e-01 3.8623166818303e-01 5.6126559200669e-01
1.2213853694832e-02 2.2613565797800e-01 3.3425812936171e-01
6.6522080074580e-01 2.3705801148494e-01 2.8866690815756e-01
3.1526202311096e-01 2.4798155246479e-01 2.4307246531244e-01
9.6306968294675e-01 2.5890570355007e-01 1.9747850445174e-01
6.0905860088312e-01 2.6982837114067e-01 1.5188842931576e-01
2.5347214195218e-01 2.8074648062874e-01 1.0630640329456e-01
8.9646316485958e-01 2.9165609971340e-01 6.0737650136426e-02
5.3813918381020e-01 3.0255251852371e-01 1.5188220781948e-02
1.7722201150991e-01 3.1344531050548e-01 9.6964679734257e-01
8.1653028907338e-01 3.2429876506076e-01 9.2415448119507e-01
4.5471877765191e-01 3.3512412281045e-01 8.7869583383025e-01
9.1805727447623e-02 3.4591868909006e-01 8.3327061365900e-01
7.2777466053066e-01 3.5668198301043e-01 7.8787513138634e-01
3.6257071992295e-01 3.6741575193209e-01 7.4250277885415e-01
9.9609717771303e-01 3.7812337069003e-01 6.9714523041170e-01
6.2820954650998e-01 3.8880875287007e-01 6.5179401529008e-01
2.5871085812585e-01 3.9947515127610e-01 6.0644214988278e-01
8.8736377847085e-01 4.1012457367051e-01 5.6108555557546e-01
3.7848319691121e-01 2.2807883010461e-01 4.2545063748098e-01
4.2575111241541e-02 2.3899130224085e-01 3.7986235231739e-01
6.9835381021356e-01 2.4989678434554e-01 3.3426988452469e-01
3.4956301164090e-01 2.6080479907274e-01 2.8868094925958e-01
9.9787039655743e-01 2.7171716048589e-01 2.4309446526206e-01
6.4407619207722e-01 2.8263363533032e-01 1.9750880208761e-01
2.8857764421226e-01 2.9355301698772e-01 1.5192366692585e-01
9.3157401286232e-01 3.0447326740680e-01 1.0634055323604e-01
5.7316979443173e-01 3.1539134099155e-01 6.0762901388010e-02
2.1343217200774e-01 3.2630291464909e-01 1.5196072873459e-02
8.5100116765320e-01 3.3721775732696e-01 9.6962865652642e-01
4.8881030862633e-01 3.4809725983732e-01 9.2410543207184e-01
1.2548412342959e-01 3.5895058439596e-01 8.7861356885670e-01
7.6108518031004e-01 3.6977180665912e-01 8.3315643886348e-01
3.9565383698665e-01 3.8055742186696e-01 7.8773336814711e-01
2.9201400070421e-02 3.9130711080944e-01 7.4233955672802e-01
6.6170962275966e-01 4.0202362027328e-01 6.9696720008272e-01
2.9313167006960e-01 4.1271178537559e-01 6.5160742141111e-01
9.2339218959586e-01 4.2337711633646e-01 6.0625237082064e-01
5.5239409022019e-01 4.3402472308013e-01 5.6089684408436e-01
5.9125923056135e-01 2.2955933438160e-01 5.1645787866502e-01
4.0175401662232e-01 2.4100020699964e-01 4.7099142155969e-01
8.3613527475545e-02 2.5191769202883e-01 4.2532203465848e-01
7.4152095893683e-01 2.6280297750161e-01 3.7971688349792e-01
3.9214605022158e-01 2.7368747964852e-01 3.3414814916476e-01
3.9464627905753e-02 2.8457695218657e-01 2.8859194873859e-01
6.8480927222411e-01 2.9547308488225e-01 2.4303739746621e-01
3.2869018697893e-01 3.0637615963589e-01 1.9747941862124e-01
9.7129396876775e-01 3.1728578545159e-01 1.5191586735867e-01
6.1266359807706e-01 3.2820100801389e-01 1.0634661744605e-01
2.5278121251842e-01 3.3912004440337e-01 6.0773511881971e-02
8.9161643312403e-01 3.5003974763980e-01 1.5200675980478e-02
5.2768587722707e-01 3.6097109446815e-01 9.6961577228338e-01
1.6398289619621e-01 3.7187379116664e-01 9.2406632142859e-01
7.9908216488128e-01 3.8275537582949e-01 8.7854218916503e-01
4.3307940438178e-01 3.9360640846785e-01 8.3305127013047e-01
6.6072857897563e-02 4.0441934422094e-01 7.8759736975653e-01
6.9814538394485e-01 4.1519049503267e-01 7.4217885565660e-01
3.2935992123328e-01 4.2592089842625e-01 6.9678917760652e-01
9.5976354585819e-01 4.3661572699139e-01 6.5141904449491e-01
5.8938990245488e-01 4.4728257921859e-01 6.0605926324510e-01
2.1825831627279e-01 4.5792957886217e-01 5.6070303642864e-01
9.2450627869561e-01 2.5397865950338e-01 5.1645739928859e-01
2.3519086713471e-01 2.6507371668316e-01 4.6984783587674e-01
8.2628818119154e-01 2.7586978664591e-01 4.2464332993819e-01
4.5781724217593e-01 2.8668527497889e-01 3.7928247143139e-01
9.5868281353924e-02 2.9752726636488e-01 3.3384961243470e-01
7.3587552938595e-01 3.0838578147942e-01 2.8838250825593e-01
3.7647326132454e-01 3.1925726700068e-01 2.4289271372142e-01
1.7041946739309e-02 3.3014003307491e-01 1.9738459860238e-01
6.5720227111523e-01 3.4103314930359e-01 1.5186003867812e-01
2.9666932709783e-01 3.5193600687550e-01 1.0632012006615e-01
9.3520698121612e-01 3.6284787072998e-01 6.0766211167472e-02
5.7262083260909e-01 3.7376716735338e-01 1.5201048021028e-02
2.0725324032468e-01 3.8470707267644e-01 9.6961031585561e-01
8.4208688118601e-01 3.9562730978053e-01 9.2404251507838e-01
4.7560676169378e-01 4.0653509521540e-01 8.7848924506987e-01
1.0791356052411e-01 4.1741791911523e-01 8.3296320856913e-01
7.3915768038878e-01 4.2826332844888e-01 7.8747435564543e-01
3.6950220996351e-01 4.3906247341645e-01 7.4202638028240e-01
9.9910220469678e-01 4.4981284004484e-01 6.9661552372759e-01
6.2810537308561e-01 4.6051878760991e-01 6.5123246270401e-01
2.5665677300889e-01 4.7118973134574e-01 6.0586615990237e-01
8.8489083609695e-01 4.8183707747680e-01 5.6050755126522e-01
5.3836356419408e-01 2.7763414796648e-01 5.1647465817858e-01
6.8625515539250e-02 2.8800411178357e-01 4.7099050183123e-01
5.9871173061382e-01 2.9948202824350e-01 4.2464288990553e-01
1.8981126209119e-01 3.1053293509426e-01 3.7892871909855e-01
8.0882807508414e-01 3.2138160856323e-01 3.3352100753896e-01
4.3837808962779e-01 3.3221718575124e-01 2.8812312608979e-01
7.2545674715944e-02 3.4306393998913e-01 2.4269767650450e-01
7.0885412333229e-01 3.5392402272239e-01 1.9724430585363e-01
3.4604026131726e-01 3.6479692186879e-01 1.5176597383758e-01
9.8333856299340e-01 3.7568226254521e-01 1.0626462427640e-01
6.2021340122908e-01 3.8658008382572e-01 6.0741235265748e-02
2.5625332173034e-01 3.9749040215987e-01 1.5196809863486e-02
8.8958421435201e-01 4.0842924118243e-01 9.6961383382283e-01
5.2310391430904e-01 4.1935821385392e-01 9.2403919284197e-01
1.5514747526938e-01 4.3028651441271e-01 8.7846357298547e-01
7.8576264968784e-01 4.4120048732804e-01 8.3290297208292e-01
4.1512480230553e-01 4.5208290290211e-01 7.8737444492034e-01
4.3483513015568e-02 4.6291776543737e-01 7.4188998717205e-01
6.7110822573234e-01 4.7369604228105e-01 6.9645167186951e-01
2.9826882267444e-01 4.8441897145882e-01 6.5105155854924e-01
9.2524042362078e-01 4.9509704649344e-01 6.0567639353777e-01
5.5229184909558e-01 5.0574553796834e-01 5.6031374844823e-01
1.9288814271952e-01 3.0142035639511e-01 5.1640938404826e-01
7.7360195171327e-01 3.1189588780266e-01 4.7099207441422e-01
3.4138169058405e-01 3.2275588038218e-01 4.2532071740376e-01
9.2180279091684e-01 3.3402727373371e-01 3.7928163449645e-01
5.2541928509958e-01 3.4509185821878e-01 3.3352061094190e-01
1.4443859496965e-01 3.5599792657069e-01 2.8799812046252e-01
7.7166839426609e-01 3.6685740193514e-01 2.4255094520616e-01
4.0328294957586e-01 3.7771037568471e-01 1.9711337440069e-01
3.7208333756541e-02 3.8856921126078e-01 1.5166400429004e-01
6.7219495311647e-01 3.9943813420027e-01 1.0619553826405e-01
3.0739558810381e-01 4.1031932107232e-01 6.0704603335532e-02
9.4217060638442e-01 4.2121465876769e-01 1.5188791446626e-02
5.7441907321146e-01 4.3214320266714e-01 9.6962605721754e-01
2.0689998525171e-01 4.4306981878446e-01 9.2405892127503e-01
8.3773356005872e-01 4.5400838638454e-01 8.7847305296202e-01
4.6682008860337e-01 4.6494794279611e-01 8.3288322334272e-01
9.4283141712154e-02 4.7586914783381e-01 7.8731171263011e-01
7.2043809045053e-01 4.8674802967439e-01 7.4178143942319e-01
3.4569880297041e-01 4.9756493678521e-01 6.9630552343642e-01
9.7050228028379e-01 5.0831348650126e-01 6.5088123748666e-01
5.9529579141556e-01 5.1900310690725e-01 6.0549357091751e-01
2.2052737466517e-01 5.2965360316272e-01 5.6012504663366e-01
8.5423512844970e-01 3.2525179242903e-01 5.1631642927297e-01
4.5482114111710e-01 3.3580070456816e-01 4.7091400605253e-01
4.6502632533932e-02 3.4646662109394e-01 4.2544845153530e-01
6.3809160035480e-01 3.5747561194439e-01 3.7971521530193e-01
2.3837380873580e-01 3.6861801662003e-01 3.3384842547288e-01
8.5049632180998e-01 3.7965404829723e-01 2.8812238006280e-01
4.7184111944474e-01 3.9058552396473e-01 2.4255059606857e-01
9.9073694017676e-02 4.0146715817951e-01 1.9705911594373e-01
7.2983642553695e-01 4.1233176836619e-01 1.5159688751670e-01
3.6257322892519e-01 4.2319474604411e-01 1.0613772828725e-01
9.9619927586614e-01 4.3406367138337e-01 6.0668042777562e-02
6.2988715042038e-01 4.4494339982017e-01 1.5179296386013e-02
2.6134185497626e-01 4.5585592721286e-01 9.6964386791542e-01
8.9317294329265e-01 4.6676956208662e-01 9.2409833467130e-01
5.2325207666402e-01 4.7770442687077e-01 8.7851976268918e-01
1.5122399184889e-01 4.8865683911792e-01 8.3291479081043e-01
7.7700953208035e-01 4.9961174892514e-01 7.8730358634943e-01
4.0088239545114e-01 5.1054067957078e-01 7.4171833442616e-01
2.3402729058677e-02 5.2140991361120e-01 6.9618964507324e-01
6.4525012560188e-01 5.3219756914232e-01 6.5072860300967e-01
2.6712806522407e-01 5.4290627807686e-01 6.0532192458304e-01
8.8974933733843e-01 5.5356022965729e-01 5.5994502471481e-01
5.1792995866091e-01 3.4910765875565e-01 5.1619796606633e-01
1.2899595590310e-01 3.5969488534904e-01 4.7080727198860e-01
7.3411146632386e-01 3.7032491139013e-01 4.2539543778596e-01
3.3702498994124e-01 3.8114229029343e-01 3.7985986421081e-01
9.4208549867401e-01 3.9215970937788e-01 3.3414618032754e-01
5.5299054607301e-01 4.0322648115822e-01 2.8838102184421e-01
1.7095799556892e-01 4.1423263049974e-01 2.4269663190336e-01
7.9486137399279e-01 4.2517002041566e-01 1.9711272726133e-01
4.2295160072685e-01 4.3606468951948e-01 1.5159658979559e-01
5.3717385028299e-02 4.4693899150928e-01 1.0611499263901e-01
6.8598631021783e-01 4.5780681677554e-01 6.0645087636001e-02
3.1882441634960e-01 4.6867688169484e-01 1.5171561744285e-02
9.4980973949801e-01 4.7957384338720e-01 9.6966146752629e-01
5.8143820619014e-01 4.9046843847178e-01 9.2414613150399e-01
2.1136452473959e-01 5.0138596898315e-01 8.7859384663363e-01
8.3892382392592e-01 5.1233201586577e-01 8.3299816374166e-01
4.6364793254276e-01 5.2330371620834e-01 7.8736537677185e-01
8.5503257970606e-02 5.3427927495645e-01 7.4172470039447e-01
7.0504632092884e-01 5.4521452247901e-01 6.9612486401830e-01
3.2326696942452e-01 5.5606200219999e-01 6.5060542755336e-01
9.4128653457161e-01 5.6680393606397e-01 6.0516699826935e-01
5.6024685652174e-01 5.7746474335433e-01 5.5977748948096e-01
1.8272339969523e-01 3.7298018787134e-01 5.1605719120484e-01
8.0016904243498e-01 3.8358810636855e-01 4.7067667018759e-01
4.1353485756137e-01 3.9421561632726e-01 4.2528582931421e-01
2.4667759487275e-02 4.0494508515497e-01 3.7983641627525e-01
6.3586192437676e-01 4.1582913093608e-01 3.3426714796529e-01
2.4938755267516e-01 4.2682630212311e-01 2.8858973322917e-01
8.6701872121759e-01 4.3784466609593e-01 2.4289098209292e-01
4.8928100641923e-01 4.4882580657863e-01 1.9724301717803e-01
1.1557327851997e-01 4.5976045652383e-01 1.5166311391699e-01
7.4485819202393e-01 4.7066078723998e-01 1.0613718788268e-01
3.7609823712287e-01 4.8154100320252e-01 6.0644845222694e-02
8.3704918683850e-03 4.9241207781776e-01 1.5168458511920e-02
6.3921388801789e-01 5.0330040635320e-01 9.6967232414335e-01
2.7107385451068e-01 5.1417759449996e-01 9.2418534435533e-01
9.0148156369430e-01 5.2507086671398e-01 8.7867117145381e-01
5.2952505958408e-01 5.3599177474647e-01 8.3311265551089e-01
1.5427253154579e-01 5.4695167585667e-01 7.8749551865366e-01
7.7502902041771e-01 5.5795012878226e-01 7.4182429288547e-01
3.9184108056095e-01 5.6895320446871e-01 6.9614451023320e-01
5.8238954282594e-03 5.7988786363306e-01 6.5053391740492e-01
6.1876008805577e-01 5.9069025924495e-01 6.0503758825830e-01
2.3256161304549e-01 6.0136692531919e-01 5.5962656263312e-01
8.4813583780582e-01 3.9686501847396e-01 5.1589781943765e-01
4.6976077940169e-01 4.0748436475872e-01 4.7052577875844e-01
8.8556481589203e-02 4.1811512321337e-01 4.2514688397979e-01
7.0552374431391e-01 4.2880582015096e-01 3.7973700051166e-01
3.2198079046592e-01 4.3960249009407e-01 3.3425464490363e-01
9.3926971279517e-01 4.5050990544653e-01 2.8867802158927e-01
5.5866517647839e-01 4.6148458541713e-01 2.4303499287472e-01
1.8107785859437e-01 4.7246989178214e-01 1.9738267959917e-01
8.0672852220722e-01 4.8343112919929e-01 1.5176449918456e-01
4.3522642014923e-01 4.9435990366425e-01 1.0619446301170e-01
6.5878289452867e-02 5.0526146696979e-01 6.0667318303003e-02
6.9791293484208e-01 5.1614440045302e-01 1.5171135791338e-02
3.2893177112250e-01 5.2703470957613e-01 9.6967210101951e-01
9.6139933286765e-01 5.3790339658976e-01 9.2420036276101e-01
5.9283801889125e-01 5.4877639986957e-01 8.7872073595173e-01
2.2224206808844e-01 5.5966495973835e-01 8.3321438555962e-01
8.4837306741315e-01 5.7058626280510e-01 7.8765508637436e-01
4.6978553369439e-01 5.8156196000081e-01 7.4201410917181e-01
8.5274235154858e-02 5.9259839380408e-01 6.9628947297728e-01
6.9503227362585e-01 6.0363735785465e-01 6.5055882304477e-01
3.0139711945071e-01 6.1454933315130e-01 6.0495231158290e-01
9.0775966971832e-01 6.2526676466632e-01 5.5949715084858e-01
5.1394799203523e-01 4.2075899753759e-01 5.1572374422892e-01
1.3838584736857e-01 4.3138464583744e-01 4.7035838377553e-01
7.6089987257886e-01 4.4201853316779e-01 4.2498778429507e-01
3.8206843568362e-01 4.5269107177556e-01 3.7959875835866e-01
2.6871576051630e-03 4.6343616460806e-01 3.3416533322296e-01
6.2358690296905e-01 4.7427120061600e-01 2.8866329092049e-01
2.4558066954222e-01 4.8518388127035e-01 2.4309140740626e-01
8.6940816125512e-01 4.9613935435091e-01 1.9747688610930e-01
4.9554726733929e-01 5.0710121311792e-01 1.5185799370227e-01
1.2406606293107e-01 5.1804702972472e-01 1.0626302529206e-01
7.5466801250527e-01 5.2896955857405e-01 6.0703405096890e-02
3.8683930282983e-01 5.3987040821360e-01 1.5178449983642e-02
1.8343090489086e-02 5.5077231771735e-01 9.6966080091104e-01
6.5172825493905e-01 5.6164445257108e-01 9.2418484125026e-01
2.8461053011722e-01 5.7250995470834e-01 8.7872045623740e-01
9.1604607102734e-01 5.8337577861347e-01 8.3325517260447e-01
5.4477398923478e-01 5.9425410660180e-01 7.8776800397633e-01
1.6900144689893e-01 6.0516868356540e-01 7.4222132401107e-01
7.8624329014658e-01 6.1615970581065e-01 6.9655876189138e-01
3.9401456617698e-01 6.2725294189556e-01 6.5075617546419e-01
9.9283060441794e-01 6.3833392850386e-01 6.0496270630587e-01
5.8814868064263e-01 6.4916146670130e-01 5.5939794827857e-01
1.8004920479717e-01 4.4465954411772e-01 5.1553883437588e-01
8.0634528646582e-01 4.5528857676592e-01 4.7017834320027e-01
4.3143025391820e-01 4.6592450288059e-01 4.2481386284729e-01
5.5651089922653e-02 4.7658767552687e-01 3.7943747620081e-01
6.7949672736696e-01 4.8730228629868e-01 3.3403272961720e-01
3.0349433866089e-01 4.9808637853549e-01 2.8858010251685e-01
9.2815980807470e-01 5.0894194397660e-01 2.4306877953381e-01
5.5399456346766e-01 5.1985292237058e-01 1.9750567850319e-01
1.8142990597349e-01 5.3079314504597e-01 1.5191327160806e-01
8.1071185002743e-01 5.4173815309547e-01 1.0631801402912e-01
4.4182925065070e-01 5.5267249130266e-01 6.0739577095266e-02
7.4537682610361e-02 5.6358994275780e-01 1.5187535603988e-02
7.0682531733444e-01 5.7450798474158e-01 9.6964276607874e-01
3.4137423474541e-01 5.8539302203450e-01 9.2414513083363e-01
9.7597358891507e-01 5.9626529961967e-01 8.7867033509318e-01
6.0985323345434e-01 6.0712767575430e-01 8.3321377313309e-01
2.4195889207626e-01 6.1798453510752e-01 7.8776767154116e-01
8.7068394241405e-01 6.2884605011716e-01 7.4231411536374e-01
4.9333123299600e-01 6.3974005242175e-01 6.9680694470691e-01
1.0520796904369e-01 6.5073753657271e-01 6.5114001344334e-01
7.0009095623019e-01 6.6192362160184e-01 6.0521265361268e-01
2.7950351451110e-01 6.7302277529251e-01 5.5936275509873e-01
8.4638162067863e-01 4.6856442788479e-01 5.1534682945318e-01
4.7379631427464e-01 4.7919515394466e-01 4.6998943456602e-01
1.0060932354625e-01 4.8983219335183e-01 4.2462936079507e-01
7.2702917341863e-01 5.0049002983230e-01 3.7926145171047e-01
3.5334970506341e-01 5.1118600375242e-01 3.3387486267495e-01
9.7989816310609e-01 5.2193546981653e-01 2.8845493124890e-01
6.0699796779564e-01 5.3274557079743e-01 2.4298928501086e-01
2.3496734059071e-01 5.4361159418119e-01 1.9747481785792e-01
8.6411414507996e-01 5.5451852845573e-01 1.5192054554754e-01
4.9468742627727e-01 5.6544703834773e-01 1.0634402660584e-01
1.2680773592308e-01 5.7638007869072e-01 6.0764112437891e-02
7.6042953516909e-01 5.8730649476178e-01 1.5195161099579e-02
3.9376961415814e-01 5.9823863115345e-01 9.6962453394626e-01
2.9656717926770e-02 6.0913985816742e-01 9.2409684751740e-01
6.6610383581176e-01 6.2002773419109e-01 8.7859246197490e-01
3.0257972082121e-01 6.3090284923738e-01 8.3311143624630e-01
9.3836584780674e-01 6.4176559864758e-01 7.8765409187377e-01
5.7236931190711e-01 6.5261655110102e-01 7.4222061006658e-01
2.0266907132466e-01 6.6345911882120e-01 6.9680656348590e-01
8.2531377212033e-01 6.7431264191180e-01 6.5138033975476e-01
4.3083825095917e-01 6.8527856710004e-01 6.0577977488088e-01
9.9909564680188e-01 6.9663997007791e-01 5.5962149277937e-01
5.1291699180438e-01 4.9247166519099e-01 5.1515132341586e-01
1.4082050803129e-01 5.0310305440714e-01 4.6979529012464e-01
7.6868711780935e-01 5.1374065381667e-01 4.2443804094292e-01
3.9663343299007e-01 5.2439531014316e-01 3.7907609527235e-01
2.4816628049902e-02 5.3507954109373e-01 3.3370200765348e-01
6.5341810851900e-01 5.4580547821431e-01 2.8830478055857e-01
2.8262323528130e-01 5.5658127986477e-01 2.4287301146843e-01
9.1261801147358e-01 5.6740800723295e-01 1.9739950499205e-01
5.4359467737187e-01 5.7827884136918e-01 1.5188481306610e-01
1.7574015152841e-01 5.8918124441483e-01 1.0633750542130e-01
8.0919858122210e-01 6.0010097449253e-01 6.0770997514111e-02
4.4402932222585e-01 6.1102582771351e-01 1.5199028370995e-02
7.8633162694391e-02 6.2196488437958e-01 9.6961190842594e-01
7.1595378675756e-01 6.3287913734581e-01 9.2405696426571e-01
3.5423609764896e-01 6.4378353425046e-01 8.7851784364977e-01
9.9319675591005e-01 6.5467732275802e-01 8.3299634881827e-01
6.3247845234456e-01 6.6556002235939e-01 7.8749387050365e-01
2.7159384750052e-01 6.7643080198359e-01 7.4201268691286e-01
9.0976247570171e-01 6.8728800138896e-01 6.9655762105222e-01
5.4542216225890e-01 6.9812846455676e-01 6.5113920591614e-01
1.7441421937806e-01 7.0894716955146e-01 6.0577934887549e-01
7.7993638751653e-01 7.1977392470742e-01 5.6045711451705e-01
1.7964694023177e-01 5.1637944645480e-01 5.1495581798348e-01
8.0745311987359e-01 5.2701074794639e-01 4.6959941593323e-01
4.3578864321378e-01 5.3764869328423e-01 4.2424338315857e-01
6.4699358854629e-02 5.4830167890824e-01 3.7888553318378e-01
6.9423278786563e-01 5.5897860706096e-01 3.3352065657449e-01
3.2445059209820e-01 5.6968856464527e-01 2.8814031460370e-01
9.5542583722762e-01 5.8043899291140e-01 2.4273453118314e-01
5.8724135535187e-01 5.9123344901092e-01 1.9729477341304e-01
2.1999452963827e-01 6.0207022415666e-01 1.5181694068084e-01
8.5379845136245e-01 6.1294259705527e-01 1.0630293192415e-01
4.8876808395795e-01 6.2384067578931e-01 6.0760001860474e-02
1.2499374695181e-01 6.3475397548118e-01 1.5198313003979e-02
7.6100000351726e-01 6.4569059362707e-01 9.6960801317169e-01
3.9978086138053e-01 6.5661054998752e-01 9.2403678815829e-01
3.9781036048891e-02 6.6752641766731e-01 8.7847061903425e-01
6.8091870997266e-01 6.7843605694175e-01 8.3291239700295e-01
3.2312115648366e-01 6.8933835702751e-01 7.8736308900304e-01
9.6636473294140e-01 7.0023272855010e-01 7.4182217357461e-01
6.1074245471621e-01 7.1111892454555e-01 6.9628758099286e-01
2.5662332682423e-01 7.2199725983632e-01 6.5075456607400e-01
9.0516640031924e-01 7.3286963650611e-01 6.0521137843556e-01
5.6077942684834e-01 7.4374393597228e-01 5.5962059973873e-01
8.4658024279300e-01 5.4028607748012e-01 5.1476381487547e-01
4.7369434923208e-01 5.5091650499326e-01 4.6940526907224e-01
1.0195206927937e-01 5.6155482430581e-01 4.2404872235018e-01
7.3133151063886e-01 5.7220755341489e-01 3.7869329295071e-01
3.6177432827113e-01 5.8288053625481e-01 3.3333544503984e-01
9.9322856519067e-01 5.9358001187894e-01 2.8796850859683e-01
6.2566336262140e-01 6.0431198441446e-01 2.4258374037445e-01
2.5907085670761e-01 6.1508073563257e-01 1.9717236411917e-01
8.9346770380126e-01 6.2588744535764e-01 1.5172788422783e-01
5.2889719120589e-01 6.3672961701975e-01 1.0624785070582e-01
1.6542580229559e-01 6.4760157010761e-01 6.0734401497409e-02
8.0313075730191e-01 6.5849578451494e-01 1.5193399665093e-02
4.4061041364024e-01 6.6942108985757e-01 9.6961312269012e-01
8.0835794128594e-02 6.8033816513270e-01 9.2403969046308e-01
7.2240015360856e-01 6.9125785923720e-01 8.7846064924446e-01
3.6535106053416e-01 7.0217666407360e-01 8.3288027302830e-01
9.7837010996879e-03 7.1309230915809e-01 7.8730067861312e-01
6.5591061959959e-01 7.2400341769258e-01 7.4172190093165e-01
3.0419199776170e-01 7.3490936512768e-01 6.9614188122920e-01
9.5561855724858e-01 7.4581052771296e-01 6.5055642312259e-01
6.1243652563977e-01 7.5670964843993e-01 6.0496059047549e-01
2.8037845556192e-01 7.6762027724384e-01 5.5936097470816e-01
5.1374554637059e-01 5.6418994590138e-01 5.1457890804651e-01
1.3950934288882e-01 5.7481834763925e-01 4.6921635559923e-01
7.6713962526549e-01 5.8545719736487e-01 4.2385739342648e-01
3.9653008763535e-01 5.9611125932803e-01 3.7850272118867e-01
2.7496267264554e-02 6.0678332654024e-01 3.3315022685779e-01
6.5985970513472e-01 6.1747679382180e-01 2.8779444445114e-01
2.9348060541352e-01 6.2819588974644e-01 2.4242751161113e-01
9.2826196319654e-01 6.3894468298697e-01 1.9704076256116e-01
5.6414596252170e-01 6.4972582814183e-01 1.5162651409824e-01
2.0111143623035e-01 6.6053966566102e-01 1.0617964647335e-01
8.3917169601882e-01 6.7138397694075e-01 6.0698566625488e-02
4.7836921483778e-01 6.8225441979159e-01 1.5185275983695e-02
1.1734535418062e-01 6.9316146612763e-01 9.6962569595216e-01
7.5898457840376e-01 7.0406786196435e-01 9.2406311019696e-01
4.0198051459499e-01 7.1498364960413e-01 8.7848586218362e-01
4.6443275654261e-02 7.2590447018216e-01 8.3289949325044e-01
6.9254027803417e-01 7.3682693686907e-01 7.8730821020660e-01
3.4055736155186e-01 7.4774858281375e-01 7.4171487735616e-01
9.9100861317117e-01 7.5866794526979e-01 6.9612151777955e-01
6.4484505205498e-01 7.6958522494390e-01 6.5053074395375e-01
3.0388467357794e-01 7.8050497853894e-01 6.0494936930117e-01
9.7174478537225e-01 7.9144677251717e-01 5.5939529192225e-01
1.8120110562027e-01 5.8808954041436e-01 5.1440483707065e-01
8.0481692270407e-01 5.9871396453978e-01 4.6903630776703e-01
4.3122670356285e-01 6.0935352152647e-01 4.2367287625698e-01
6.0190928861654e-02 6.2001084979867e-01 3.7831734540551e-01
6.9134398518563e-01 6.3068512419188e-01 3.3296885583790e-01
3.2434421091956e-01 6.4137665976605e-01 2.8762262153496e-01
9.5892426690275e-01 6.5208771141839e-01 2.4227127258942e-01
5.9489154016584e-01 6.6282164248870e-01 1.9690632931303e-01
2.3211465731902e-01 6.7358172054549e-01 1.5151958539063e-01
8.7051181806847e-01 6.8437009643967e-01 1.0610437028116e-01
5.1004397626036e-01 6.9518719385201e-01 6.0656545884914e-02
1.5071044372184e-01 7.0603157343898e-01 1.5175030197408e-02
7.9118192953100e-01 7.1691564167048e-01 9.6964356727988e-01
4.3420886928566e-01 7.2780521570467e-01 9.2410187312469e-01
7.8551377836684e-02 7.3871041899231e-01 8.7853838340675e-01
7.2433347424443e-01 7.4962685452450e-01 8.3295923483070e-01
3.7173485065157e-01 7.6055055807020e-01 7.8737037872965e-01
2.1033305279979e-02 7.7147828959460e-01 7.4177735294540e-01
6.7267953200064e-01 7.8240798561803e-01 6.9618560705251e-01
3.2742527698408e-01 7.9333983670599e-01 6.5060150325597e-01
9.8654126065346e-01 8.0427908143589e-01 6.0503383941720e-01
6.5214989461146e-01 8.1524262004341e-01 5.5949363560955e-01
8.4905797772399e-01 6.1198357110599e-01 5.1424547074427e-01
4.6946027048737e-01 6.2260057970041e-01 4.6886890311044e-01
9.3963666957515e-02 6.3324098269130e-01 4.2349893364922e-01
7.2207894292084e-01 6.4390403402325e-01 3.7814129190239e-01
3.5314173672954e-01 6.5458395695331e-01 3.3279596758593e-01
9.8657480324417e-01 6.6527755184283e-01 2.8745812176479e-01
6.2194804335597e-01 6.7598501717977e-01 2.4212045100760e-01
2.5895978613586e-01 6.8670879348641e-01 1.9677470364137e-01
8.9740319028806e-01 6.9745224042455e-01 1.5141264283279e-01
5.3713928193027e-01 7.0821857632297e-01 1.0602687732343e-01
1.7808050363327e-01 7.1901013475581e-01 6.0611668845497e-02
8.2018238737411e-01 7.2982792718168e-01 1.5163612393505e-02
4.6214355803321e-01 7.4068622102608e-01 9.6966468596636e-01
1.0654750118189e-01 7.5155454012118e-01 9.2415061903380e-01
7.5220230981589e-01 7.6244377288144e-01 8.7860938211923e-01
3.9921507813387e-01 7.7335030665311e-01 8.3304684073526e-01
4.7744257484043e-02 7.8427028319596e-01 7.8746976226032e-01
6.9802528570141e-01 7.9520012324550e-01 7.4188530515268e-01
3.5041626103985e-01 8.0613725437091e-01 6.9630082476577e-01
5.4706066449306e-03 8.1708131504109e-01 6.5072395623893e-01
6.6403926685162e-01 8.2803626305560e-01 6.0516246846021e-01
3.2736865596408e-01 8.3901336100237e-01 5.5962221130151e-01
5.1753584370508e-01 6.3587129547613e-01 5.1410470253044e-01
1.3313874591086e-01 6.4647470593615e-01 4.6871799959282e-01
7.5488921699585e-01 6.5711617663916e-01 4.2333980676604e-01
3.8176378210938e-01 6.6778822410636e-01 3.7797997106979e-01
1.2554629048257e-02 6.7847762200835e-01 3.3263805413018e-01
6.4631892897563e-01 6.8917721001855e-01 2.8730792035982e-01
2.8240753850577e-01 6.9988529704362e-01 2.4198191944044e-01
9.2039023285910e-01 7.1060342195287e-01 1.9665224612489e-01
5.5998255579654e-01 7.2133472316489e-01 1.5131123115517e-01
2.0099402570186e-01 7.3208290317144e-01 1.0595156985818e-01
8.4329642847331e-01 7.4285153481616e-01 6.0566774389801e-02
4.8680718935324e-01 7.5364354845870e-01 1.5151812717316e-02
1.3026032064574e-01 7.6447478707617e-01 9.6968734521086e-01
7.7605041116843e-01 7.7531885058151e-01 9.2420482007181e-01
4.2302717598865e-01 7.8618785689171e-01 8.7869131364268e-01
7.1263094126527e-02 7.9707969038417e-01 8.3315159871367e-01
7.2087742247527e-01 8.0799126472064e-01 7.8759229141787e-01
3.7204976184613e-01 8.1891909176021e-01 7.4202114226909e-01
2.5045122976741e-02 8.2986012563641e-01 6.9644634937401e-01
6.8025213032004e-01 8.4081294133583e-01 6.5087590231841e-01
3.3822709134256e-01 8.5177922309588e-01 6.0531664510844e-01
9.9970872865857e-01 8.6276489197834e-01 5.5977233056478e-01
1.8711471796341e-01 6.5975335801380e-01 5.1398624717693e-01
7.9523806874028e-01 6.7033156979015e-01 4.6858738329101e-01
4.1313853024561e-01 6.8097511202882e-01 4.2320082820019e-01
3.8488379566364e-02 6.9166072188623e-01 3.7784168058784e-01
6.6901252383071e-01 7.0236361791651e-01 3.3250539073718e-01
3.0318056942375e-01 7.1307280716463e-01 2.8718268148790e-01
9.4004556128453e-01 7.2378537826473e-01 2.4186557411746e-01
5.7902782995628e-01 7.3450222450659e-01 1.9654744222371e-01
2.1976779890553e-01 7.4522606359819e-01 1.5122210546436e-01
8.6203550000450e-01 7.5596055279944e-01 1.0588330308561e-01
5.0567699547352e-01 7.6670981797282e-01 6.0524701403704e-02
1.5058504674584e-01 7.7747807358385e-01 1.5140356539244e-02
7.9554500092479e-01 7.8828227474376e-01 9.6971008133983e-01
4.4274949653207e-01 7.9910026282465e-01 9.2426078129223e-01
9.1092441478566e-02 8.0994568016444e-01 8.7877825419908e-01
7.4060268967815e-01 8.2081852860519e-01 8.3326541332795e-01
3.9135085015730e-01 8.3171714726719e-01 7.8772785278142e-01
4.3455537783901e-02 8.4263861396779e-01 7.4217310689529e-01
6.9709491715326e-01 8.5357955325470e-01 6.9660961994151e-01
3.5252446541356e-01 8.6453726797526e-01 6.5104557477765e-01
1.0093629186018e-02 8.7551093333911e-01 6.0548757880577e-01
6.7023618175897e-01 8.8650210588108e-01 5.5993909245350e-01
8.5904423551373e-01 6.8363432737978e-01 5.1389330145861e-01
4.5433826126329e-01 6.9416359395423e-01 4.6848063231698e-01
6.6988473078177e-02 7.0481350316185e-01 4.2309118047085e-01
6.9090387902109e-01 7.1551890341523e-01 3.7774220685093e-01
3.2157561527947e-01 7.2623876544079e-01 3.3241600600759e-01
9.5654039129451e-01 7.3696033402634e-01 2.8709940860450e-01
5.9447228326871e-01 7.4768076478169e-01 2.4178598732951e-01
2.3463949051647e-01 7.5840064198665e-01 1.9647203295030e-01
8.7662755377051e-01 7.6912206612637e-01 1.5115413616501e-01
5.2019346921376e-01 7.7984811866405e-01 1.0582812807217e-01
1.6518473916768e-01 7.9058273872413e-01 6.0488779466995e-02
8.1149421022549e-01 8.0133064296696e-01 1.5130034011879e-02
4.5798522225279e-01 8.1210921085035e-01 9.6973143069740e-01
1.0664496173543e-01 8.2290046065277e-01 9.2431513314668e-01
7.5642189070865e-01 8.3371969793098e-01 8.7886526865705e-01
4.0729812412292e-01 8.4456962383651e-01 8.3338207376412e-01
5.9290505136726e-02 8.5545072580000e-01 7.8786923262552e-01
7.1245425598576e-01 8.6636126305523e-01 7.4233334818217e-01
3.6688646982221e-01 8.7729786741241e-01 6.9678274079192e-01
2.2731999216005e-02 8.8825663198010e-01 6.5122587585941e-01
6.8018817742852e-01 8.9923431069594e-01 6.0566973155995e-01
3.3949270347458e-01 9.1022890258426e-01 5.6011838101421e-01
5.3779936125641e-01 7.0753282660384e-01 5.1382803758717e-01
1.0639707885469e-01 7.1795630216974e-01 4.6840254463942e-01
7.1265577139751e-01 7.2862770280050e-01 4.2303812144596e-01
3.3653518991234e-01 7.3935904869218e-01 3.7771869130966e-01
9.6872860192426e-01 7.5009741400110e-01 3.3240341665026e-01
6.0549265080194e-01 7.6083328306025e-01 2.8708457659724e-01
2.4516001662518e-01 7.7156484877560e-01 2.4176324672609e-01
8.8693083328730e-01 7.8229235135128e-01 1.9644105183122e-01
5.3040821563924e-01 7.9301700433265e-01 1.5111827911219e-01
1.7539326446108e-01 8.0374085267339e-01 1.0579342954312e-01
8.2178184380594e-01 8.1446694884487e-01 6.0463057236597e-02
4.6950713078157e-01 8.2519962278603e-01 1.5121795237075e-02
1.1755131068497e-01 8.3595571736856e-01 9.6974968653583e-01
7.6770531845550e-01 8.4672102131071e-01 9.2436418072602e-01
4.1899215012043e-01 8.5751238811421e-01 8.7894742190489e-01
7.1349603818959e-02 8.6833574853051e-01 8.3349603312741e-01
7.2473768826916e-01 8.7919461378591e-01 7.8801068328697e-01
3.7914546329928e-01 8.9008930961402e-01 7.4249616722618e-01
3.4591542104842e-02 9.0101709580720e-01 6.9696028425028e-01
6.9112312597281e-01 9.1197314388106e-01 6.5141190586403e-01
3.4881593347279e-01 9.2295195030019e-01 6.0585887659179e-01
7.7674217071162e-03 9.3394841187501e-01 5.6030639522391e-01
2.5722538412572e-01 7.3147949359856e-01 5.1384530793572e-01
7.3470205197833e-01 7.4168239385163e-01 4.6840409550493e-01
3.4081416166271e-01 7.5240869065613e-01 4.2316580429360e-01
9.7087887838244e-01 7.6316720300644e-01 3.7786326299463e-01
6.0822682390761e-01 7.7392652169124e-01 3.3252428476352e-01
2.4888253257738e-01 7.8468090000273e-01 2.8717274678098e-01
8.9151378496504e-01 7.9542865517113e-01 2.4181952808357e-01
5.3561073793219e-01 8.0616951227796e-01 1.9646969970923e-01
1.8098715536362e-01 8.1690386928365e-01 1.5112540084771e-01
8.2760041703557e-01 8.2763268744848e-01 1.0578675211440e-01
4.7546874534126e-01 8.3835775661459e-01 6.0451904936689e-02
1.2462287895618e-01 8.4908223066049e-01 1.5116728474485e-02
7.7422025504341e-01 8.5982123850206e-01 9.6976279823616e-01
4.2588103869548e-01 8.7056345530254e-01 9.2440339625236e-01
7.8737165533766e-02 8.8132666335424e-01 8.7901879022364e-01
7.3269328218877e-01 8.9212030897024e-01 8.3360108064527e-01
3.8765151321804e-01 9.0295194339845e-01 7.8814645712429e-01
4.3529367839799e-02 9.1382526114350e-01 7.4265654699535e-01
7.0026427500673e-01 9.2473923119055e-01 6.9713789534386e-01
3.5780952616570e-01 9.3568868751653e-01 6.5159978805970e-01
1.6131844056674e-02 9.4666603825407e-01 6.0605141291936e-01
6.7521227059293e-01 9.5766316632675e-01 5.6049956197685e-01
2.5726565262116e-01 7.5468004095059e-01 5.1384484121499e-01
2.3455261878298e-01 7.6522875466788e-01 4.6954673734510e-01
9.3142713701548e-01 7.7607645815390e-01 4.2384357451071e-01
5.8787054696148e-01 7.8690475285511e-01 3.7829675697455e-01
2.3779297823420e-01 7.9770656112012e-01 3.3282192688594e-01
8.8576056094275e-01 8.0849186929997e-01 2.8738132317638e-01
5.3313938065816e-01 8.1926423773576e-01 2.4196338527572e-01
1.8054932825169e-01 8.3002536329079e-01 1.9656373769084e-01
8.2837007845972e-01 8.4077618336091e-01 1.5118049894759e-01
4.7688689860117e-01 8.5151731374003e-01 1.0581257726567e-01
1.2633628181695e-01 8.6224949645440e-01 6.0458598618386e-02
7.7691298830516e-01 8.7297431196371e-01 1.5115821383120e-02
4.2801535832423e-01 8.8370401871592e-01 9.6976860024301e-01
8.1133697319074e-02 8.9442880707481e-01 9.2442741962454e-01
7.3556348259365e-01 9.0516592069380e-01 8.7907182903297e-01
3.9120440465455e-01 9.1592787567542e-01 8.3368912096238e-01
4.7906429978697e-02 9.2672713135438e-01 7.8826934047720e-01
7.0550683903292e-01 9.3757254389973e-01 7.4280878863162e-01
3.6385094541409e-01 9.4846663371789e-01 6.9731121912374e-01
2.2791409523676e-02 9.5940504802890e-01 6.5178595008776e-01
6.8218354218592e-01 9.7037837815573e-01 6.0624401360592e-01
3.4189328023098e-01 9.8137522449034e-01 5.6069446842834e-01
9.7669598320652e-01 7.7864442150651e-01 5.1382664027533e-01
7.3440519410784e-01 7.8991823204401e-01 4.6840312515641e-01
4.9229053007658e-01 8.0008408607719e-01 4.2384307121244e-01
1.8916329944776e-01 8.1067695289389e-01 3.7864957596427e-01
8.5811989800383e-01 8.2147206330546e-01 3.3314961550130e-01
5.1654476645564e-01 8.3228028725020e-01 2.8763981318383e-01
1.7035389636995e-01 8.4307735843436e-01 2.4215756149233e-01
8.2202435510193e-01 8.5386113244497e-01 1.9670320760578e-01
4.7281967689182e-01 8.6463212838973e-01 1.5127378609419e-01
1.2350574720277e-01 8.7539072816769e-01 1.0586734743379e-01
7.7461855262003e-01 8.8613689970868e-01 6.0482907856255e-02
4.2656990652160e-01 8.9687063355248e-01 1.5119458667073e-02
7.9056067098187e-02 9.0760050398733e-01 9.6976555168366e-01
7.3348204878655e-01 9.1831667362500e-01 9.2443107675495e-01
3.8938194615502e-01 9.2903338341384e-01 8.7909770767494e-01
4.6708377492082e-02 9.3976429502972e-01 8.3374944195185e-01
7.0528634878665e-01 9.5052664438629e-01 7.8836922003310e-01
3.6486665080821e-01 9.6133643346656e-01 7.4294504137679e-01
2.5180210184657e-02 9.7220270127712e-01 6.9747482798628e-01
6.8595751506778e-01 9.8312421621303e-01 6.5196651526606e-01
3.4692381582696e-01 9.9409049123786e-01 6.0643335174058e-01
8.6038910252314e-03 5.0711218922375e-03 5.6090510553113e-01
6.5545972558265e-01 8.0247807370879e-01 5.1389098210810e-01
3.6271619342231e-01 8.1364632554500e-01 4.6840060965174e-01
8.2907231221360e-02 8.2443010593515e-01 4.2316429725643e-01
7.9045776858153e-01 8.3480248231228e-01 3.7829571766228e-01
4.7481413414857e-01 8.4538167135613e-01 3.3314907983309e-01
1.4376926692977e-01 8.5611938738027e-01 2.8776390425884e-01
8.0451585380563e-01 8.6690371428910e-01 2.4230340300824e-01
4.6087998854748e-01 8.7769456776078e-01 1.9683328107933e-01
1.1493595835866e-01 8.8847959140286e-01 1.5137493646864e-01
7.6793371665796e-01 8.9925456672292e-01 1.0593566004655e-01
4.2072084235396e-01 9.1001732418944e-01 6.0518819085299e-02
7.3937326730180e-02 9.2076598393980e-01 1.5126815957800e-02
7.2760251241299e-01 9.3150508683777e-01 9.6975392725406e-01
3.8306050958033e-01 9.4222373472132e-01 9.2441180684352e-01
4.0163709040760e-02 9.5293029368340e-01 8.7908855187912e-01
6.9901219223402e-01 9.6363573244474e-01 8.3376938674373e-01
3.5948276071390e-01 9.7435939731472e-01 7.8843202656224e-01
2.1260415708694e-02 9.8512526631411e-01 7.4305354796738e-01
6.8393166121875e-01 9.9595299724546e-01 6.9762082637369e-01
3.4777929764496e-01 6.8351681416189e-03 6.5215240309526e-01
1.0943000883992e-02 1.7789393537206e-02 6.0663231120748e-01
6.7366682944585e-01 2.8782782244552e-02 5.6109302905324e-01
3.2740217108558e-01 8.2626648498225e-01 5.1398302004643e-01
1.4785056765419e-02 8.3736137040734e-01 4.6847774411170e-01
7.1107307438998e-01 8.4833923476250e-01 4.2303561924431e-01
4.0745458779229e-01 8.5897401513521e-01 3.7786119009003e-01
9.5144194111191e-02 8.6947537906385e-01 3.3282032274221e-01
7.7099519027955e-01 8.8008312042189e-01 2.8763871339261e-01
4.3762595723082e-01 8.9079542927889e-01 2.4230283924853e-01
9.8371363087172e-02 9.0155759814206e-01 1.9688665206361e-01
7.5558938800311e-01 9.1233681665439e-01 1.5144119822491e-01
4.1083647684546e-01 9.2311770036496e-01 1.0599265446155e-01
6.5197817310507e-02 9.3389267617040e-01 6.0554610498941e-02
7.1950118427178e-01 9.4465689569684e-01 1.5135595210143e-02
3.7407106220228e-01 9.5541079238187e-01 9.6973685082943e-01
3.0171637671848e-02 9.6614254800818e-01 9.2437298137580e-01
6.8802211902697e-01 9.7685293088640e-01 8.7904228976041e-01
3.4797814369262e-01 9.8754562941377e-01 8.3373813270285e-01
1.0119242574112e-02 9.9823569952130e-01 7.8844033833568e-01
6.7475883493224e-01 8.9402264985734e-03 7.4312976645887e-01
3.4020391038820e-01 1.9714767469293e-02 6.9775079447753e-01
6.3195031364115e-03 3.0570905692490e-02 6.5230414584457e-01
6.7240249619482e-01 4.1506016282939e-02 6.0680309843046e-01
3.3774046373743e-01 5.2495916993479e-02 5.6127223156361e-01
9.9699828821599e-01 8.5003044770022e-01 5.1410058759187e-01
6.7389931775006e-01 8.6108703842199e-01 4.6858355900764e-01
3.5675155536222e-01 8.7210080660987e-01 4.2308769740027e-01
4.1806862945857e-02 8.8292720586238e-01 3.7771559625561e-01
7.2471663026120e-01 8.9355355589232e-01 3.3252162072850e-01
4.0178366516755e-01 9.0413055123287e-01 2.8737912931628e-01
7.1790465339177e-02 9.1476817408039e-01 2.4215587307323e-01
7.3586386470878e-01 9.2547456849453e-01 1.9683212943645e-01
3.9575330428066e-01 9.3622370294333e-01 1.5144061072378e-01
5.2970434771032e-02 9.4699323081631e-01 1.0601453800487e-01
7.0868803238349e-01 9.5776926881412e-01 6.0576753650184e-02
3.6384041610295e-01 9.6854310765215e-01 1.5142564934553e-02
1.9004943758591e-02 9.7931118724315e-01 9.6972012657406e-01
6.7530042371736e-01 9.9006211392577e-01 9.2432590768247e-01
3.3363447793093e-01 7.8350126873231e-04 8.7897618229536e-01
9.9405268330135e-01 1.1481227025247e-02 8.3366424753590e-01
6.5730268648120e-01 2.2153290361202e-02 7.8838941209543e-01
3.2341844114286e-01 3.2821493438943e-02 7.4312245546559e-01
9.9184378811556e-01 4.3530012978111e-02 6.9781464240903e-01
6.6158924140050e-01 5.4326312621371e-02 6.5242640720205e-01
3.3153383231512e-01 6.5228177466316e-02 6.0695713678137e-01
5.3608352557561e-04 7.6211195244840e-02 5.6143891206411e-01
6.6549771678463e-01 8.7377772442161e-01 5.1424049369252e-01
3.3601673615685e-01 8.8481364841968e-01 4.6871326209728e-01
1.0616414552620e-02 8.9582995147627e-01 4.2319638428465e-01
6.8745020001723e-01 9.0674427317964e-01 3.7773810681918e-01
3.6422427857291e-01 9.1750400246505e-01 3.3239970703603e-01
3.8668811651881e-02 9.2815059792113e-01 2.8716947029979e-01
7.0901008395131e-01 9.3877599922818e-01 2.4196058077888e-01
3.7472288020675e-01 9.4943862973880e-01 1.9670091003648e-01
3.6408690911494e-02 9.6014776359103e-01 1.5137317682026e-01
6.9510522184111e-01 9.7089123657847e-01 1.0599145973828e-01
3.5185034259417e-01 9.8165485133648e-01 6.0576146829982e-02
7.5673343842564e-03 9.9242764157570e-01 1.5144859718905e-02
6.6308805128114e-01 3.2012453746603e-03 9.6971207981428e-01
3.1921539464028e-01 1.3967898836757e-02 9.2429140392359e-01
9.7679099633859e-01 2.4718438600044e-02 8.7889793512400e-01
6.3672717191182e-01 3.5441320035176e-02 8.3354881752784e-01
2.9995609207206e-01 4.6125192996032e-02 7.8825832318492e-01
9.6717307654978e-01 5.6770505846629e-02 7.4302191426662e-01
6.3833193200766e-01 6.7411194729150e-02 6.9779405292391e-01
3.1231783643581e-01 7.8120305997641e-02 6.5249698663492e-01
9.8734852862387e-01 8.8961693734382e-02 6.0708563574192e-01
6.6151241614206e-01 9.9928830954903e-02 5.6158895462395e-01
3.3338047717936e-01 8.9751267149666e-01 5.1439902933603e-01
9.9971733993313e-01 9.0853719831466e-01 4.6886328100649e-01
6.6888438313576e-01 9.1955027710106e-01 4.2333442776808e-01
3.3988116996963e-01 9.3050338851640e-01 3.7783659848792e-01
1.1389840361600e-02 9.4135050504082e-01 3.3241127086167e-01
6.8206866403030e-01 9.5208686129997e-01 2.8708023467438e-01
3.5064333788224e-01 9.6275594513888e-01 2.4181562181700e-01
1.6203546499953e-02 9.7341440185694e-01 1.9656030563126e-01
6.7852888802251e-01 9.8409693389134e-01 1.5127086286517e-01
3.3801047032663e-01 9.9481194235810e-01 1.0593327630224e-01
9.9555630214918e-01 5.5503646850629e-03 6.0557184634981e-02
6.5144365635732e-01 1.6313049023978e-02 1.5143710262567e-02
3.0664253941658e-01 2.7086696283434e-02 9.6971146754967e-01
9.6216373294809e-01 3.7861885887571e-02 9.2427551377736e-01
6.1870991698829e-01 4.8632723215473e-02 8.7884746955314e-01
2.7728721191309e-01 5.9387975044573e-02 8.3344616412392e-01
9.3913439052615e-01 7.0110461862629e-02 7.8809781690109e-01
6.0569729620651e-01 8.0778539829281e-02 7.4283115130126e-01
2.7818152135406e-01 9.1385873547837e-02 6.9764814248835e-01
9.5639432513878e-01 1.0199067644920e-01 6.5247113939956e-01
6.3799859856121e-01 1.1272247459140e-01 6.0716998402474e-01
3.1960380618281e-01 1.2364883004266e-01 5.6171745831792e-01
8.6625759863557e-04 9.2123843699067e-01 5.1457230680995e-01
6.6438685575451e-01 9.3225669782604e-01 4.6902983541035e-01
3.2983233406606e-01 9.4326667740247e-01 4.2349265108297e-01
9.9662468843667e-01 9.5423797041954e-01 3.7797393556082e-01
6.6396866151528e-01 9.6513668091085e-01 3.3249965585316e-01
3.3103375185982e-01 9.7594542693857e-01 2.8709402416026e-01
9.9700732446164e-01 9.8667651407430e-01 2.4175825873201e-01
6.6115003768630e-01 9.9736480160797e-01 1.9646515032157e-01
3.2343366234391e-01 8.0384453267408e-03 1.5118590779109e-01
9.8284085038622e-01 1.8737824697124e-02 1.0587165612936e-01
6.4016665558564e-01 2.9460632509444e-02 6.0521918082431e-02
2.9592526597500e-01 4.0205266105195e-02 1.5137372422462e-02
9.5050679793024e-01 5.0968809427876e-02 9.6972197184511e-01
6.0511134192110e-01 6.1740591524206e-02 9.2429019681439e-01
2.6021500311022e-01 7.2518963980062e-02 8.7884687511737e-01
9.1676198560661e-01 8.3296979385023e-02 8.3340447427215e-01
5.7601353740440e-01 9.4062462499396e-02 7.8798397488393e-01
2.3976285631428e-01 1.0479167550591e-01 7.4262299750980e-01
9.1049545655395e-01 1.1544442916950e-01 6.9737790717263e-01
5.9069665160707e-01 1.2599496199754e-01 6.5227284022669e-01
2.7985147303123e-01 1.3655774436309e-01 6.0715864929763e-01
9.7250300630502e-01 1.4737398224621e-01 5.6181573473142e-01
6.6806612358096e-01 9.4495759687714e-01 5.1475646308635e-01
3.2972465144854e-01 9.5597251481867e-01 4.6920907311435e-01
9.9259545180324e-01 9.6698049071649e-01 4.2366572740404e-01
6.5633190671389e-01 9.7796117842000e-01 3.7813433740787e-01
3.2044544774211e-01 9.8889039328942e-01 3.3263135106119e-01
9.8440926650081e-01 9.9975009897669e-01 2.8717628319577e-01
6.4833392790762e-01 1.0526662601373e-02 2.4179330636111e-01
3.1042982034638e-01 2.1260508889669e-02 1.9644757587586e-01
9.7092623716984e-01 3.1965233931325e-02 1.5113102844848e-01
6.2957753684176e-01 4.2665302776079e-02 1.0581720419980e-01
2.8639523731357e-01 5.3376176707694e-02 6.0486427211218e-02
9.4162413680706e-01 6.4104078627125e-02 1.5129118207398e-02
5.9530375600510e-01 7.4852826026534e-02 9.6973925602894e-01
2.4874500903098e-01 8.5611751313834e-02 9.2432910763714e-01
9.0213216563671e-01 9.6383387639361e-02 8.7889615466908e-01
5.5623573128652e-01 1.0716488378882e-01 8.3344499713918e-01
2.1211034175032e-01 1.1795186214293e-01 7.8798340272352e-01
8.7136297924899e-01 1.2873415731804e-01 7.4252928061849e-01
5.3669115544308e-01 1.3948394473161e-01 6.9712878494490e-01
2.1278802559839e-01 1.5013023688745e-01 6.5188805604619e-01
9.0587714479169e-01 1.6058792180576e-01 6.0690775609334e-01
6.1443555404361e-01 1.7113253874938e-01 5.6184998941285e-01
3.3503833931325e-01 9.6867237650272e-01 5.1494776438012e-01
9.9557391822025e-01 9.7968564790555e-01 4.6939722237212e-01
6.5671244709015e-01 9.9069255105599e-01 4.2384942135455e-01
3.1900934973641e-01 1.6640866751794e-03 3.7832615239816e-01
9.8062468215287e-01 1.2612520685315e-02 3.3280434906257e-01
6.4201217892394e-01 2.3507560769752e-02 2.8731578666425e-01
3.0284879053906e-01 3.4342070595221e-02 2.4187283253301e-01
9.6281666649185e-01 4.5120769047711e-02 1.9647858777324e-01
6.2160831430848e-01 5.5858673942537e-02 1.5112403172588e-01
2.7897506019976e-01 6.6575126786142e-02 1.0579160106364e-01
9.3479672370591e-01 7.7287178906796e-02 6.0462439684170e-02
5.8911921495645e-01 8.8005991116807e-02 1.5122184941386e-02
2.4164265111473e-01 9.8741820768491e-02 9.6975678826038e-01
8.9374568970249e-01 1.0948460723694e-01 9.2437663584567e-01
5.4528516608810e-01 1.2024068975049e-01 8.7897322438011e-01
1.9679268642092e-01 1.3100948530356e-01 8.3354648927101e-01
8.4898713537640e-01 1.4179060557422e-01 7.8809610327866e-01
5.0296177767527e-01 1.5258348891201e-01 7.4262187946901e-01
1.6063801527025e-01 1.6338473012931e-01 6.9712823943395e-01
8.2596756251454e-01 1.7417499616276e-01 6.5164678978648e-01
5.0841594337997e-01 1.8485284732748e-01 6.0633968874304e-01
2.2813037972928e-01 1.9513521497329e-01 5.6159030660605e-01
1.8115623760212e-03 9.9238475451892e-01 5.1514262254856e-01
6.6266144665164e-01 3.3820370227048e-03 4.6960827588349e-01
3.2273725570931e-01 1.4388600917530e-02 4.2405739999573e-01
9.8273281112768e-01 2.5378186903185e-02 3.7851113642316e-01
6.4249130715562e-01 3.6338283907164e-02 3.3297692528906e-01
3.0183170727184e-01 4.7256766958292e-02 2.8746575886431e-01
9.6056901983776e-01 5.8125484209982e-02 2.4198903449480e-01
6.1851761115024e-01 6.8943380924564e-02 1.9655394248579e-01
2.7548561050566e-01 7.9717282437303e-02 1.5115992590632e-01
9.3128646901221e-01 9.0459732926405e-02 1.0579841012665e-01
5.8577640908748e-01 1.0118498052144e-01 6.0455974890064e-02
2.3889644463777e-01 1.1190523529478e-01 1.5118876472081e-02
8.9006670118311e-01 1.2263516142967e-01 9.6976877020626e-01
5.4073570241093e-01 1.3336497809390e-01 9.2441581407818e-01
1.9043957935513e-01 1.4410458963834e-01 8.7904708310656e-01
8.3946203083310e-01 1.5485475536574e-01 8.3366076943553e-01
4.8816072573222e-01 1.6561596276946e-01 7.8825547667850e-01
1.3702336349003e-01 1.7638904991934e-01 7.4282892094100e-01
7.8683077002534e-01 1.8717568406807e-01 6.9737627350086e-01
4.3914548049007e-01 1.9797902825506e-01 6.5188699562451e-01
9.8126543324365e-02 2.0880411144371e-01 6.0633917418379e-01
6.6919834390517e-01 1.6081188929710e-02 5.1535512266677e-01
3.2934052923936e-01 2.7093922248660e-02 4.6980354508648e-01
9.8895255860570e-01 3.8100077807035e-02 4.2425152639487e-01
6.4798895051305e-01 4.9091265912623e-02 3.7870124759765e-01
3.0640255130038e-01 6.0058591017374e-02 3.3315791272810e-01
9.6413206677765e-01 7.0992972857757e-02 2.8762995518555e-01
6.2110479913845e-01 8.1886976765990e-02 2.4212734580828e-01
2.7723828431462e-01 9.2737052114353e-02 1.9665861232803e-01
9.3243550728181e-01 1.0354491415359e-01 1.5122785027528e-01
5.8658374497160e-01 1.1431729057883e-01 1.0583315572026e-01
2.3956840217171e-01 1.2506407964603e-01 6.0467269046886e-02
8.9129952753906e-01 1.3579577250656e-01 1.5120022939704e-02
5.4099226508973e-01 1.4652899401281e-01 9.6977208379081e-01
1.9020002755928e-01 1.5725316865002e-01 9.2443534085591e-01
8.3818514534484e-01 1.6798136459256e-01 8.7909359755361e-01
4.8502979567455e-01 1.7871572848767e-01 8.3374395711888e-01
1.3080706519092e-01 1.8945737806328e-01 7.8838544700706e-01
7.7554095676603e-01 2.0020690965811e-01 7.4301858303265e-01
4.1913881548044e-01 2.1096457721758e-01 6.9764542924049e-01
3.3556793547768e-01 3.9794238981810e-02 5.1554640070867e-01
9.9640673084115e-01 5.0807795503706e-02 4.6999702710938e-01
6.5610120588097e-01 6.1813520050893e-02 4.2444558954730e-01
3.1467361884076e-01 7.2804901424221e-02 3.7889296425735e-01
9.7218268097147e-01 8.3776102438995e-02 3.3334268161261e-01
6.2868071569111e-01 9.4720890741126e-02 2.8780140624036e-01
2.8419894838530e-01 1.0563326885048e-01 2.4227787606621e-01
9.3874560965121e-01 1.1650896158887e-01 1.9678086209828e-01
5.9230440996421e-01 1.2734679566910e-01 1.5131685477469e-01
2.4483242888666e-01 1.3814927414081e-01 1.0588829900949e-01
8.9626354990052e-01 1.4892208393591e-01 6.0493051866205e-02
5.4652091957085e-01 1.5967275152060e-01 1.5125245382273e-02
1.9467946517860e-01 1.7041798277183e-01 9.6976646008692e-01
8.4244121326604e-01 1.8114510505366e-01 9.2443185051308e-01
4.8886076289696e-01 1.9186953498160e-01 8.7910291222769e-01
1.3389090573274e-01 2.0259478752656e-01 8.3377536573381e-01
7.7743681595501e-01 2.1332314007270e-01 7.8844708867980e-01
1.7021653139355e-03 6.3510092767056e-02 5.1573053488933e-01
6.6389531468087e-01 7.4525629697432e-02 4.7018522171536e-01
3.2422137810097e-01 8.5530772175338e-02 4.2463626036404e-01
9.8278702634489e-01 9.6520762794418e-02 3.7908294560268e-01
6.3977708728135e-01 1.0749281534318e-01 3.3352738394370e-01
2.9537038525137e-01 1.1844354175581e-01 2.8797503591560e-01
9.4970708155507e-01 1.2936872016567e-01 2.4243375848768e-01
6.0288454666615e-01 1.4026428855756e-01 1.9691221211863e-01
2.5496096217290e-01 1.5112759893943e-01 1.5141807476833e-01
9.0595786087137e-01 1.6195831741462e-01 1.0595646102156e-01
5.5586229666704e-01 1.7275866908539e-01 6.0528958858116e-02
2.0463216212755e-01 1.8353300258321e-01 1.5133561854679e-02
8.5124773970106e-01 1.9429702987010e-01 9.6975344570692e-01
4.9759366659454e-01 2.0503490331412e-01 9.2440790963858e-01
1.4257976919786e-01 2.1576330532881e-01 8.7907710485160e-01
6.6754318785331e-01 8.7230256599599e-02 5.1590379143012e-01
3.3188786793955e-01 9.8249731189143e-02 4.7036450244874e-01
9.9343809174551e-01 1.0925412013790e-01 4.2482006477173e-01
6.5243374259086e-01 1.2024079190567e-01 3.7926766991834e-01
3.0924079806918e-01 1.3121057850260e-01 3.3370817169805e-01
9.6420128412785e-01 1.4216317167382e-01 2.8814635062011e-01
6.1758290437174e-01 1.5309632328453e-01 2.4258957114181e-01
2.6957863628405e-01 1.6400667671270e-01 1.9704630756595e-01
9.2032021779495e-01 1.7489097106094e-01 1.5152476089121e-01
5.6988981372075e-01 1.8574706210089e-01 1.0603159641557e-01
2.1832683467482e-01 1.9657453272396e-01 6.0570947073117e-02
8.6563233318009e-01 2.0737482879363e-01 1.5143889784424e-02
5.1072038374957e-01 2.1816221129726e-01 9.6973521191807e-01
3.3298035414573e-01 1.1095601541526e-01 5.1606230730905e-01
5.4161699318759e-04 1.2198287100021e-01 4.7053109755073e-01
6.6400139205171e-01 1.3298637307192e-01 4.2499324573789e-01
3.2384926672056e-01 1.4396727460567e-01 3.7944301667464e-01
9.8074996599089e-01 1.5493135923093e-01 3.3388041503517e-01
6.3528109846354e-01 1.6588183343727e-01 2.8831027419385e-01
2.8787312678644e-01 1.7681850554990e-01 2.4273989207531e-01
9.3882808546439e-01 1.8773894181983e-01 1.9717751490441e-01
5.8835317936358e-01 1.9863978931667e-01 1.5163137412951e-01
2.3658774588841e-01 2.0951783639904e-01 1.0610885569176e-01
9.9779439064756e-01 1.3468810674823e-01 5.1620219481910e-01
6.7015760960923e-01 1.4572853135321e-01 4.7068115481545e-01
3.3637298475750e-01 1.5673093024167e-01 4.2515156785612e-01
9.9746435345145e-01 1.6770279384332e-01 3.7960358123396e-01
6.5463989052323e-01 1.7865735541019e-01 3.3403762767346e-01
3.0884278752039e-01 1.8960178216090e-01 2.8845983715559e-01
9.6072255058183e-01 2.0053777204806e-01 2.4287785444005e-01
6.1070976724264e-01 2.1146379302662e-01 1.9729947926806e-01
6.6150568979807e-01 1.5842588115323e-01 5.1631974495283e-01
3.4135053282619e-01 1.6949148075167e-01 4.7081089390768e-01
1.1418099949955e-02 1.8049177804096e-01 4.2528970427408e-01
6.7403647603547e-01 1.9145004284177e-01 3.7974107165288e-01
3.3148111599032e-01 2.0239106355394e-01 3.3416954007691e-01
9.8528277550980e-01 2.1332584006095e-01 2.8858438105929e-01
3.2286502289341e-01 1.8216476502123e-01 5.1641176666282e-01
1.5540770636615e-02 1.9327929155798e-01 4.7091674244388e-01
6.9086027739334e-01 2.0427319722773e-01 4.2539847819958e-01
3.5491489662057e-01 2.1521164061638e-01 3.7983970726016e-01
9.7739783092843e-01 2.0588613038060e-01 5.1647609345462e-01
6.9677097353383e-01 2.1710643509086e-01 4.7099390817332e-01"""

In [ ]:
%%time

import numpy as np
import torch
from alignn.pretrained import get_figshare_model,get_multiple_predictions
import matplotlib.pyplot as plt
from jarvis.io.vasp.inputs import Poscar
from alignn.ff.ff import AlignnAtomwiseCalculator,default_path,wt10_path,alignnff_fmult,fd_path,ForceField
import time
from jarvis.core.graphs import Graph
import torch.multiprocessing as mp
from jarvis.core.atoms import Atoms
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
atoms=Poscar.from_string(Mo924).atoms
pr=get_multiple_predictions(atoms_array=[atoms],workers=0,pin_memory=False)
pr

Using chk file jv_formation_energy_peratom_alignn/checkpoint_300.pt from  ['jv_formation_energy_peratom_alignn/checkpoint_300.pt']
Path /usr/local/lib/python3.10/dist-packages/alignn/jv_formation_energy_peratom_alignn.zip
Config /content/jv_formation_energy_peratom_alignn/config.json
data range -9999 -9999


100%|██████████| 1/1 [00:04<00:00,  4.47s/it]


df                                                atoms  prop jid
0  {'lattice_mat': [[2.738736057436, 0.0, 0.0], [... -9999   0
building line graphs


100%|██████████| 1/1 [00:00<00:00,  1.78it/s]

CPU times: user 4.45 s, sys: 265 ms, total: 4.71 s
Wall time: 6.42 s


In [ ]:
Mo3696="""Mo
1.0000000000000e+00
2.7387360574360e+00 0.0000000000000e+00 0.0000000000000e+00
0.0000000000000e+00 2.1689682986643e+02 0.0000000000000e+00
-1.3693680287180e+00 1.0844841493321e+02 9.8391419586238e+01
Mo
3696
direct
7.6544738121702e-01 2.3480986509030e-01 5.3037814647897e-01
2.4511568730164e-02 2.3583307235945e-01 5.7616744146983e-01
6.7473284397147e-01 2.4123335641426e-01 5.5347067626533e-01
3.1809680197413e-01 2.4662839696911e-01 5.3080211004539e-01
3.6245445495089e-01 2.3692289200327e-01 6.2167442910875e-01
1.2212837151961e-02 2.4231855183406e-01 5.9898173060978e-01
6.5918583833729e-01 2.4771340358968e-01 5.7629588573199e-01
3.0103739941295e-01 2.5310555299478e-01 5.5361954577092e-01
-6.8035385311664e-02 2.5848780740319e-01 5.3094946739905e-01
7.1040222485241e-01 2.3803915247182e-01 6.6711186233073e-01
3.6148725964598e-01 2.4343162856745e-01 6.4441775389918e-01
1.1165254992446e-02 2.4882435993419e-01 6.2172558753079e-01
6.5882467315431e-01 2.5421686094748e-01 5.9903576148836e-01
3.0341861919511e-01 2.5960801735963e-01 5.7634790612439e-01
-5.6956857043865e-02 2.6499472119692e-01 5.5365918057121e-01
5.7401139018333e-01 2.7036654693128e-01 5.3096092536820e-01
6.1174015864728e-02 2.3917210305862e-01 7.1251831776199e-01
7.1332763730233e-01 2.4456144885467e-01 6.8982475372523e-01
3.6460473140908e-01 2.4995129299945e-01 6.6713162500820e-01
1.4767916976665e-02 2.5534149255013e-01 6.4443899243457e-01
6.6346481008766e-01 2.6073173106517e-01 6.2174671495123e-01
3.1015915413055e-01 2.6612131640996e-01 5.9905416868448e-01
-4.5989104730842e-02 2.7150865942226e-01 5.7635972715581e-01
5.9366696262131e-01 2.7688989879814e-01 5.5366004438745e-01
2.2693625447864e-01 2.8225561514890e-01 5.3095024283181e-01
7.5947414511976e-01 2.3493099512554e-01 7.8059741446054e-01
4.1301238337763e-01 2.4031750312923e-01 7.5790484088224e-01
6.6002580025339e-02 2.4570432922147e-01 7.3521236567690e-01
7.1836406938778e-01 2.5109153052140e-01 7.1251992384577e-01
3.6998598605368e-01 2.5647911666324e-01 6.8982743798537e-01
2.0714777123805e-02 2.6186702411907e-01 6.6713477205241e-01
6.7033557943574e-01 2.6725506421014e-01 6.4444165360515e-01
3.1854386971272e-01 2.7264281555732e-01 6.2174754736106e-01
-3.5098243085115e-02 2.7802939672687e-01 5.9905146801968e-01
6.0877125628279e-01 2.8341298592105e-01 5.7635177129857e-01
2.4922030845189e-01 2.8878986997052e-01 5.5364617018713e-01
-1.1506673417652e-01 2.9415302537315e-01 5.3093279339435e-01
1.1124271491907e-01 2.3608487357427e-01 8.2597188100248e-01
7.6536050948353e-01 2.4147067467767e-01 8.0327913504000e-01
4.1902565239255e-01 2.4685628974049e-01 7.8058687526854e-01
7.2194383397953e-02 2.5224186869141e-01 7.5789488512312e-01
7.2480901025220e-01 2.5762752760521e-01 7.3520296268522e-01
3.7679294203883e-01 2.6301333948034e-01 7.1251090175836e-01
2.8043886389366e-02 2.6839931678534e-01 6.8981846288464e-01
6.7842438472053e-01 2.7378537918437e-01 6.6712533032954e-01
3.2774847333895e-01 2.7917129394278e-01 6.4443105115707e-01
-2.4237212354426e-02 2.8455656560840e-01 6.2173495718060e-01
6.2212015133350e-01 2.8994023389554e-01 5.9903608921078e-01
2.6634486025516e-01 2.9532052114635e-01 5.7633319764336e-01
-9.2203994728005e-02 3.0069430706329e-01 5.5362501925344e-01
5.4566173873671e-01 3.0605673116141e-01 5.3091119899303e-01
4.6349110921735e-01 2.3723779976270e-01 8.7134558711449e-01
1.1809378757858e-01 2.4262522023975e-01 8.4865000963374e-01
7.7230686052808e-01 2.4801169781086e-01 8.2595578076901e-01
4.2610405400361e-01 2.5339746068445e-01 8.0326255801334e-01
7.9452350959887e-02 2.5878271214360e-01 7.8057001797123e-01
7.3230969559772e-01 2.6416762575164e-01 7.5787785148079e-01
3.8462198118661e-01 2.6955233778345e-01 7.3518575356174e-01
3.6319087003603e-02 2.7493693472995e-01 7.1249340725615e-01
6.8730963300276e-01 2.8032143242334e-01 6.8980046032635e-01
3.3747399950430e-01 2.8570574103695e-01 6.6710649437692e-01
-1.3344960999624e-02 2.9108960645410e-01 6.4441098853131e-01
6.3464589425026e-01 2.9647251322017e-01 6.2171328709196e-01
2.8117477956594e-01 3.0185352996827e-01 5.9901259803629e-01
-7.4111502312396e-02 3.0723108721431e-01 5.7630808435377e-01
5.6834246541175e-01 3.1260274324160e-01 5.5359916137502e-01
2.0802024311698e-01 3.1796523286726e-01 5.3088611307654e-01
8.1630789596556e-01 2.3838108284402e-01 9.1672990044234e-01
4.7129783332883e-01 2.4377297035353e-01 8.9402804542689e-01
1.2594502030505e-01 2.4916301163164e-01 8.7132866805315e-01
7.8023220109447e-01 2.5455147036541e-01 8.4863137151025e-01
4.3413859087291e-01 2.5993860143074e-01 8.2593575714847e-01
8.7638836710389e-02 2.6532464532246e-01 8.0324143040330e-01
7.4070161311621e-01 2.7070982209328e-01 7.8054800309769e-01
3.9328777204881e-01 2.7609432385465e-01 7.5785509172915e-01
4.5347941272160e-02 2.8147830451215e-01 7.3516231141255e-01
6.9681943057625e-01 2.8686186481595e-01 7.1246926531666e-01
3.4762226611261e-01 2.9224502987012e-01 6.8977552994539e-01
-2.3458571640385e-03 2.9762771485127e-01 6.6708063796474e-01
6.4678410562517e-01 3.0300967293790e-01 6.4438406338304e-01
2.9484495376697e-01 3.0839041827766e-01 6.2168522031214e-01
-5.8375007550642e-02 3.1376911973257e-01 5.9898349824247e-01
-4.1313853024561e-01 3.1914447861763e-01 5.7627837377869e-01
2.3024206840924e-01 3.2451466158586e-01 5.5356964941723e-01
-1.2857140207361e-01 3.2987750514822e-01 5.3085782350601e-01
1.6986867531244e-01 2.3950588947417e-01 9.6213590169650e-01
8.2515687471537e-01 2.4490502956129e-01 9.3942437303517e-01
4.8013638119172e-01 2.5030149336563e-01 9.1671637995629e-01
1.3479633620496e-01 2.5569550355298e-01 8.9401159086277e-01
7.8912343710871e-01 2.6108729845959e-01 8.7130963360646e-01
4.4310158397929e-01 2.6647712517062e-01 8.4861010713778e-01
9.6711325757234e-02 2.7186523251931e-01 8.2591259172238e-01
7.4992905975604e-01 2.7725186374207e-01 8.0321665698726e-01
4.0272593412251e-01 2.8263724830070e-01 7.8052186722412e-01
5.5066395861525e-02 2.8802159212823e-01 7.5782778356698e-01
-2.9309368238593e-01 2.9340506523700e-01 7.3513396291336e-01
3.5819062657633e-01 2.9878778520380e-01 7.1243995387518e-01
8.8503554769241e-03 3.0416979449107e-01 6.8974529082244e-01
6.5879905564996e-01 3.0955102894266e-01 6.6704948855651e-01
3.0792859671695e-01 3.1493127447227e-01 6.4435204283366e-01
-4.3895383116924e-02 3.2031011009660e-01 6.2165244644612e-01
-3.9683752981032e-01 3.2568684109003e-01 5.9895023693284e-01
2.4890578653723e-01 3.3106044308499e-01 5.7624509765678e-01
-1.0688780986220e-01 3.3642957935370e-01 5.5353702804219e-01
-4.6444559814728e-01 3.4179283388017e-01 5.3082655163506e-01
8.6973967935426e-01 2.3518684888616e-01 3.0307842546083e-02
5.2553450804946e-01 2.4059866068465e-01 7.5798038222456e-03
1.7985947731342e-01 2.4601372360615e-01 9.8484833606559e-01
8.3509562652107e-01 2.5141888934569e-01 9.6212895445108e-01
4.9003690486002e-01 2.5682098583867e-01 9.3941358250207e-01
1.4467669091467e-01 2.6222015136558e-01 9.1670199801804e-01
7.9900597429814e-01 2.6761655707229e-01 8.9399391204061e-01
4.5301321854514e-01 2.7301040119974e-01 8.7128897918601e-01
1.0668411087013e-01 2.7840190248682e-01 8.4858680866484e-01
7.6000115993685e-01 2.8379129275028e-01 8.2588697511539e-01
4.1294310047424e-01 2.8917880850408e-01 8.0318902845271e-01
6.5484062361478e-02 2.9456468131156e-01 7.8049250208418e-01
7.1759246149423e-01 2.9994912636997e-01 7.5779691906766e-01
3.6922957106902e-01 3.0533232860379e-01 7.3510179611876e-01
2.0347737691035e-02 3.1071442529376e-01 7.1240664588070e-01
6.7088822402088e-01 3.1609548403096e-01 6.8971097867828e-01
3.2077870428508e-01 3.2147547471011e-01 6.6701430626824e-01
-3.0069456569300e-02 3.2685423478105e-01 6.4431615207238e-01
-3.8176378210938e-01 3.3223142907236e-01 6.2161607507116e-01
2.6556769611575e-01 3.3760651133740e-01 5.9891371725700e-01
-8.8222550329174e-02 3.4297870821722e-01 5.7620888449577e-01
-4.4329486620254e-01 3.4834707306897e-01 5.5350166021830e-01
2.0019458742247e-01 3.5371069897998e-01 5.3079251535511e-01
2.2501589137200e-01 2.3624803068271e-01 7.5790146080597e-02
8.8104464759484e-01 2.4166929848662e-01 5.3049889332922e-02
5.3677715925830e-01 2.4708753042827e-01 3.0313556321148e-02
1.9221856214644e-01 2.5250262386366e-01 7.5813229458336e-03
8.4614992355533e-01 2.5792120756796e-01 9.8484538494730e-01
5.0103299389892e-01 2.6332962018102e-01 9.6212184268326e-01
1.5562539024572e-01 2.6873482659085e-01 9.3940244335538e-01
8.0992279492047e-01 2.7413689589270e-01 9.1668702884467e-01
4.6391843900291e-01 2.7953593828003e-01 8.9397536301667e-01
1.1760299835734e-01 2.8493210119282e-01 8.7126713957149e-01
7.7096441698276e-01 2.9032556390978e-01 8.4856199178315e-01
4.2398762413649e-01 2.9571653057624e-01 8.2585950334549e-01
7.6654108336338e-02 3.0110522160107e-01 8.0315921952616e-01
7.2894131104973e-01 3.0649186326836e-01 7.8046065791741e-01
3.8082180600721e-01 3.1187667530108e-01 7.5776331826803e-01
3.2262237704274e-02 3.1725985599903e-01 7.3506669120373e-01
6.8322200750753e-01 3.2264156447985e-01 7.1237026613300e-01
3.3365172391682e-01 3.2802189954897e-01 6.8967353935294e-01
-1.6508512805691e-02 3.3340087497688e-01 6.6697602438191e-01
6.3266918061797e-01 3.3877839184062e-01 6.4427726786128e-01
2.8109913006398e-01 3.4415421083717e-01 6.2157687576981e-01
-7.1318172068152e-02 3.4952793245173e-01 5.9887455534101e-01
-4.2469404395549e-01 3.5489900257363e-01 5.7617017574526e-01
2.2085467081084e-01 3.6026677740432e-01 5.5346384065126e-01
-1.3478126217499e-01 3.6563070220545e-01 5.3075594109099e-01
5.8113102351745e-01 2.3728460706577e-01 1.2130136954644e-01
2.3745380110286e-01 2.4271374278548e-01 9.8550894705623e-02
8.9345249881811e-01 2.4814047885716e-01 7.5803478361888e-02
5.4913908160446e-01 2.5356459526041e-01 5.3059470764656e-02
2.0452310863346e-01 2.5898589381597e-01 3.0319172168334e-02
8.5961186786730e-01 2.6440420468009e-01 7.5828213481565e-03
5.1315094595922e-01 2.6982629614942e-01 9.8484243622749e-01
1.6767908355441e-01 2.7523801439598e-01 9.6211467850611e-01
8.2191949636673e-01 2.8064644777361e-01 9.3939113527422e-01
4.7587035198191e-01 2.8605159014907e-01 9.1667171968797e-01
1.2952738473640e-01 2.9145348240128e-01 8.9395625830798e-01
7.8288380020823e-01 2.9685221122460e-01 8.7124449582692e-01
4.3593014304527e-01 3.0224790576175e-01 8.4853610282603e-01
8.8654101185312e-02 3.0764073196798e-01 8.2583068594158e-01
7.4104021386414e-01 3.1303088464208e-01 8.0312779969447e-01
3.9306944947767e-01 3.1841857706855e-01 7.8042695922418e-01
4.4718623310700e-02 3.2380402820665e-01 7.5772765328408e-01
6.9595963564138e-01 3.2918744735829e-01 7.3502935710936e-01
3.4675853008916e-01 3.3456901628638e-01 7.1233154517821e-01
-2.9255952230690e-03 3.3994886891217e-01 6.8963370446641e-01
6.4685826327047e-01 3.4532706914239e-01 6.6693534954037e-01
2.9605198262443e-01 3.5070358834634e-01 6.4423604165765e-01
-5.5412276254169e-02 3.5607828601868e-01 6.2153541465486e-01
-4.0761165985779e-01 3.6145090099292e-01 5.9883321006230e-01
2.3936966637582e-01 3.6682106709591e-01 5.7612932105418e-01
-1.1455406732529e-01 3.7218837655045e-01 5.5342383690874e-01
5.3053972951262e-01 3.7755252400261e-01 5.3071706359630e-01
9.3786153088129e-01 2.3830356622330e-01 1.6683318939777e-01
5.9458086697187e-01 2.4373799090572e-01 1.4407564390763e-01
2.5093106770683e-01 2.4917088387840e-01 1.2132001789661e-01
9.0693135265662e-01 2.5460198897908e-01 9.8566716826550e-02
5.6259772468557e-01 2.6003104670690e-01 7.5816135724865e-02
2.1794355887958e-01 2.6545780057910e-01 5.3068648860653e-02
8.7297997269750e-01 2.7088200395258e-01 3.0324597474902e-02
5.2771602184006e-01 2.7630342737255e-01 7.5842764450860e-03
1.8086337954053e-01 2.8172897317190e-01 9.8483953470475e-01
8.3503392014959e-01 2.8714401723066e-01 9.6210757131406e-01
4.8891825833095e-01 2.9255575935527e-01 9.3937982937012e-01
1.4251710711522e-01 2.9796411333862e-01 9.1665630055298e-01
7.9582885319704e-01 3.0336904324731e-01 8.9393688409123e-01
4.4884944485063e-01 3.0877056558407e-01 8.7122138763459e-01
1.0157227450450e-01 3.1416874881849e-01 8.4850953221404e-01
7.5398803745709e-01 3.1956371001035e-01 8.2580096091531e-01
4.0608453911613e-01 3.2495560838046e-01 8.0309525063014e-01
5.7846419017071e-02 3.3034463579689e-01 7.8039192610532e-01
7.0925476234662e-01 3.3573100424685e-01 7.5769047550283e-01
3.6028658031048e-01 3.4111493047267e-01 7.3499036682668e-01
1.0914161323305e-02 3.4649661810526e-01 7.1229106486421e-01
6.6110432858145e-01 3.5187623789537e-01 6.8959204872226e-01
3.1081769096337e-01 3.5725390713743e-01 6.6689283056655e-01
-3.9991949034236e-02 3.6262967029767e-01 6.4419297667781e-01
-3.9137775463564e-01 3.6800348450022e-01 6.2149213213718e-01
2.5660084912214e-01 3.7337521628427e-01 5.9879004977912e-01
-9.6119637830464e-02 3.7874466021447e-01 5.7608662150793e-01
5.5039766576385e-01 3.8411159512065e-01 5.5338190428252e-01
1.9609752484242e-01 3.8947589764015e-01 5.3067612294792e-01
2.9493181601510e-01 2.3931346555364e-01 2.1237541669578e-01
9.5218262913667e-01 2.4475036178148e-01 1.8961413708272e-01
6.0899985908890e-01 2.5018657273775e-01 1.6685371591798e-01
2.6541374164402e-01 2.5562188072211e-01 1.4409450442938e-01
9.2144862216331e-01 2.6105604591510e-01 1.2133686890269e-01
5.7712443531893e-01 2.6648881053788e-01 9.8581188851332e-02
2.3245779738101e-01 2.7191990297018e-01 7.5827851316441e-02
8.8746277790349e-01 2.7734904253026e-01 5.3077241683451e-02
5.4215140380233e-01 2.8277594539853e-01 3.0329731511574e-02
1.9653394228531e-01 2.8820033191870e-01 7.5856640785149e-03
8.4928955627816e-01 2.9362923492553e-01 9.8483672732554e-01
5.0309924690200e-01 2.9904758403079e-01 9.6210063321997e-01
1.5662269040631e-01 3.0446268013027e-01 9.3936869675451e-01
8.0986308503242e-01 3.0987435335411e-01 9.1664099552819e-01
4.6282151526486e-01 3.1528248424770e-01 8.9391751182912e-01
1.1549679691211e-01 3.2068700997013e-01 8.7119812878601e-01
7.6788534268099e-01 3.2608792760604e-01 8.4848263154711e-01
4.1998104090045e-01 3.3148529403771e-01 8.2577071346612e-01
7.1775126500825e-02 3.3687922204569e-01 8.0306198685864e-01
7.2325601467819e-01 3.4226987253330e-01 7.8035599757850e-01
3.7440906643059e-01 3.4765744299210e-01 7.5765224249317e-01
2.5216263417760e-02 3.5304215255282e-01 7.3495018892724e-01
6.7565578908044e-01 3.5842422422953e-01 7.1224929529312e-01
3.2570154533278e-01 3.6380386531757e-01 6.8954903241457e-01
-2.4677318518966e-02 3.6918124742787e-01 6.6684890541438e-01
-3.7551672302763e-01 3.7455648845397e-01 6.4414847636027e-01
2.7314240447168e-01 3.7992964003573e-01 6.2144738790331e-01
-7.8745097094249e-02 3.8530068597449e-01 5.9874538748873e-01
-4.3122670356285e-01 3.9066955958309e-01 5.7604234975030e-01
2.1565135885181e-01 3.9603619061277e-01 5.5333829072214e-01
-1.3815025603740e-01 4.0140059348768e-01 5.3063336126800e-01
9.9350862564684e-01 2.3488556951343e-01 2.8068058134699e-01
6.5206310235793e-01 2.4032245882288e-01 2.5791834799029e-01
3.1001809686975e-01 2.4575940353571e-01 2.3515600864440e-01
9.6744168678107e-01 2.5119630972277e-01 2.1239380446278e-01
6.2438571274974e-01 2.5663304203733e-01 1.8963197567846e-01
2.8088996662330e-01 2.6206943218515e-01 1.6687077754775e-01
9.3698533519111e-01 2.6750528340404e-01 1.4411048997577e-01
5.9269617801246e-01 2.7294037278512e-01 1.2135142213226e-01
2.4804212577635e-01 2.7837445270363e-01 9.8593912766699e-02
9.0303942362697e-01 2.8380725235695e-01 7.5838326532971e-02
5.5770190333984e-01 2.8923848025183e-01 5.3085046448317e-02
2.1204164360320e-01 2.9466782833270e-01 3.0334462632494e-02
8.6606936510166e-01 3.0009497823374e-01 7.5869576763502e-03
5.1843293115193e-01 3.0552709209213e-01 9.8483406508949e-01
1.7187828716587e-01 3.1094867956425e-01 9.6209398420799e-01
8.2503567647807e-01 3.1636713433813e-01 9.3935791710620e-01
4.7791059014319e-01 3.2178220086060e-01 9.1662603461846e-01
1.3050674953045e-01 3.2719367038390e-01 8.9389841298425e-01
7.8282588114950e-01 3.3260139182018e-01 8.7117502427981e-01
4.3486753027682e-01 3.3800527975044e-01 8.4845573261147e-01
8.6628921244462e-02 3.4340531863555e-01 8.2574029656226e-01
7.3810485353431e-01 3.4880156257711e-01 8.0302837779303e-01
3.8928760781399e-01 3.5419413032138e-01 7.8031955517459e-01
4.0166828654717e-02 3.5958319555634e-01 7.5761334344084e-01
-3.0927064600589e-01 3.6496897291257e-01 7.3490921514349e-01
3.4095897749614e-01 3.7035170046039e-01 7.1220662466398e-01
-9.1638402043542e-03 3.7573161993364e-01 6.8950503317482e-01
-3.5966225175960e-01 3.8110895645635e-01 6.6680393368290e-01
2.8943682126279e-01 3.8648390026575e-01 6.4410287552145e-01
-6.1897159595585e-02 3.9185659388331e-01 6.2140148769347e-01
-4.1369763624887e-01 3.9722712942508e-01 5.9869950001086e-01
2.3400063317873e-01 4.0259556216794e-01 5.7599675963548e-01
-1.1883563991404e-01 4.0796194764371e-01 5.5329323806786e-01
-4.7223406125848e-01 4.1332640926752e-01 5.3058901991233e-01
3.4924880788067e-01 2.3590200680468e-01 3.2621601474586e-01
1.0089913868288e+00 2.4133702303610e-01 3.0345439262538e-01
6.6790031421530e-01 2.4677232783233e-01 2.8069247588026e-01
3.2610930511481e-01 2.5220795967537e-01 2.5793044010980e-01
9.8371619575436e-01 2.5764388845993e-01 2.3516841034015e-01
6.4079305529916e-01 2.6308003805897e-01 2.1240650095731e-01
2.9739328701126e-01 2.6851629763733e-01 1.8964484044971e-01
9.5355672434574e-01 2.7395252681368e-01 1.6688358670812e-01
6.0931336453251e-01 2.7938855729608e-01 1.4412293629835e-01
2.6468615207167e-01 2.8482419251395e-01 1.2136312951425e-01
9.1969307678458e-01 2.9025920632577e-01 9.8604451963503e-02
5.7434875492957e-01 2.9569334174510e-01 7.5847232780310e-02
2.2866559939635e-01 3.0112631060722e-01 5.3091839204194e-02
8.8265464581939e-01 3.0655779507471e-01 3.0338667175434e-02
5.3632607965744e-01 3.1198745177249e-01 7.5881277570434e-03
1.8829745182889e-01 3.1742257365399e-01 9.8483160443240e-01
8.4137526163383e-01 3.2284727959203e-01 9.6208775638548e-01
4.9416163731452e-01 3.2826905049066e-01 9.3934768638423e-01
1.4666401310615e-01 3.3368754527839e-01 9.1661166507515e-01
7.9888856376965e-01 3.3910246154179e-01 8.9387987373302e-01
4.5083988245074e-01 3.4451355151732e-01 8.7115238790733e-01
1.0252070414496e-01 3.4992063571263e-01 8.4842916719331e-01
7.5393170328558e-01 3.5532361266635e-01 8.2571005235170e-01
4.0507137013679e-01 3.6072246370132e-01 8.0299477023457e-01
5.5935947710041e-02 3.6611725198636e-01 7.8028294619430e-01
-2.9348060541351e-01 3.7150811573284e-01 7.5757412298232e-01
3.5681333424926e-01 3.7689525587456e-01 7.3486778466563e-01
6.8069615319464e-03 3.8227891910641e-01 7.1216338383504e-01
-3.4351309732678e-01 3.8765937770611e-01 6.8946037033920e-01
3.0583706606878e-01 3.9303690815845e-01 6.6675821990444e-01
-4.5161427308818e-02 3.9841177126676e-01 6.4405646119809e-01
-3.9653008763534e-01 4.0378419717581e-01 6.2135470001173e-01
2.5170752521636e-01 4.0915437951512e-01 5.9865263906102e-01
-1.0047295859882e-01 4.1452248356865e-01 5.7595009116555e-01
-4.5309459824680e-01 4.1988867363667e-01 5.5324698210181e-01
1.9382399044452e-01 4.2525316384271e-01 5.3054333735190e-01
7.0379977916312e-01 2.3692881741105e-01 3.7174023777228e-01
3.6542959815714e-01 2.4236154327847e-01 3.4897872486784e-01
1.0257302446257e+00 2.4779436043155e-01 3.2621720520645e-01
6.8499878742350e-01 2.5322754019557e-01 3.0345588420971e-01
3.4343991844894e-01 2.5866119861862e-01 2.8069478482371e-01
1.0011969123160e+00 2.6409536793510e-01 2.5793386244790e-01
6.5837120413814e-01 2.6953002947975e-01 2.3517306450875e-01
3.1503514565187e-01 2.7496512932603e-01 2.1241236112658e-01
9.7124057026647e-01 2.8040058542579e-01 1.8965176122460e-01
6.2702471864457e-01 2.8583629003917e-01 1.6689132148790e-01
2.8241446023530e-01 2.9127210922916e-01 1.4413115180016e-01
9.3742938446980e-01 2.9670788041793e-01 1.2137141871343e-01
5.9208411633885e-01 3.0214340877566e-01 9.8612347320100e-02
2.4639007427210e-01 3.0757846324075e-01 7.5854221181033e-02
9.0035680070235e-01 3.1301277310387e-01 5.3097379541909e-02
5.5399293976727e-01 3.1844602622253e-01 3.0342210889078e-02
2.0730690312511e-01 3.2387786998041e-01 7.5891419699509e-03
8.5888666684254e-01 3.2931573261930e-01 9.8482940774219e-01
5.1159465195885e-01 3.3474337486043e-01 9.6208209658439e-01
1.6400596334676e-01 3.4016836282253e-01 9.3933822268212e-01
8.1612943357590e-01 3.4559027321578e-01 9.1659816111478e-01
4.6797335304703e-01 3.5100870785317e-01 8.9386220859200e-01
1.1954502958405e-01 3.5642331496389e-01 8.7113055935048e-01
7.7085038252363e-01 3.6183380923997e-01 8.4840328689227e-01
4.2189362688954e-01 3.6723998844942e-01 8.2568033380253e-01
7.2677076017237e-02 3.7264174476323e-01 8.0296151098447e-01
-2.7679894080273e-01 3.7803906949332e-01 7.8024650658000e-01
3.7346392261559e-01 3.8343205062014e-01 7.5753490380610e-01
2.3462064893162e-02 3.8882086321369e-01 7.3482620608510e-01
-3.2681004141129e-01 3.9420575357542e-01 7.1211986727018e-01
3.2264012858820e-01 3.9958701863817e-01 6.8941532448611e-01
-2.8196903968954e-02 4.0496498286186e-01 6.6671203108931e-01
6.2066734713692e-01 4.1033997554507e-01 6.4400948742518e-01
2.6921944125252e-01 4.1571231209945e-01 6.2130726730304e-01
-8.2555580339359e-02 4.2108228332898e-01 5.9860503824649e-01
-4.3467329899375e-01 4.2645015698337e-01 5.7590257341252e-01
2.1285163182886e-01 4.3181619556197e-01 5.5319975247253e-01
-1.3999222016483e-01 4.3718069310027e-01 5.3049654780616e-01
1.0556167823483e+00 2.3796689555581e-01 4.1725366050121e-01
7.2089680769357e-01 2.4339842253328e-01 3.9448940173195e-01
3.8354293464270e-01 2.4882888664747e-01 3.7172606822226e-01
1.0443814427514e+00 2.5425923830802e-01 3.4896421268113e-01
7.0391807256257e-01 2.5968989955122e-01 3.2620370423390e-01
3.6247466890497e-01 2.6512107501192e-01 3.0344421090055e-01
1.0202623555178e+00 2.7055286356509e-01 2.8068538510523e-01
6.7742271373375e-01 2.7598530414072e-01 2.5792692832206e-01
3.3405204706967e-01 2.8141839692224e-01 2.3516860942114e-01
9.9021629890031e-01 2.8685211381923e-01 2.1241026675405e-01
6.4596059764713e-01 2.9228640332162e-01 1.8965180579882e-01
3.0131560613251e-01 2.9772119177272e-01 1.6689319681918e-01
9.5630191158915e-01 3.0315638196794e-01 1.4413447397184e-01
6.1093317972478e-01 3.0859184960218e-01 1.2137573598383e-01
2.6521850387065e-01 3.1402743804523e-01 9.8617147859335e-02
9.1916420587141e-01 3.1946295206568e-01 7.5858942647248e-02
5.7277523650837e-01 3.2489815137112e-01 5.3101421972536e-02
2.2605625302642e-01 3.3033274511451e-01 3.0344953806453e-02
8.7901240664417e-01 3.3576638875061e-01 7.5899658935474e-03
5.3020257603142e-01 3.4120665319707e-01 9.8482754259170e-01
1.8254020401654e-01 3.4663697759843e-01 9.6207716637699e-01
8.3457419770441e-01 3.5206501581200e-01 9.3932976894827e-01
4.8631399958704e-01 3.5749027002464e-01 9.1658583061910e-01
1.3776947546078e-01 3.6291224776718e-01 8.9384577161679e-01
7.8895034841675e-01 3.6833048818375e-01 8.7110991967278e-01
4.3986562953207e-01 3.7374458900949e-01 8.4837848212851e-01
9.0523133708025e-02 3.7915423132409e-01 8.2565152611387e-01
-2.5907085670760e-01 3.8455919928328e-01 8.0292896945557e-01
3.9108824265387e-01 3.8995939258894e-01 7.8021058363228e-01
4.1003304045274e-02 3.9535483030359e-01 7.5749600860065e-01
-3.0932442921395e-01 4.0074564559373e-01 7.3478477794655e-01
3.4010476814266e-01 4.0613207198261e-01 7.1207635176219e-01
-1.0710818886845e-02 4.1151442264954e-01 6.8937015399992e-01
6.3822567172887e-01 4.1689306520593e-01 6.6666561081341e-01
2.8690966051405e-01 4.2226839517531e-01 6.4396218651278e-01
-6.4664843972186e-02 4.2764081203715e-01 6.2125941413071e-01
-4.1650503225341e-01 4.3301070204968e-01 5.9855691807104e-01
2.3138126405397e-01 4.3837843200294e-01 5.7585442670370e-01
-1.2101337747403e-01 4.4374435739693e-01 5.5315177292464e-01
-4.7369434923207e-01 4.4910884707047e-01 5.3044888061540e-01
3.9790802830459e-01 2.3901010432926e-01 4.6276381534777e-01
1.0738623399046e+00 2.4444782153853e-01 4.3998868446952e-01
7.4171495262681e-01 2.4987812508157e-01 4.1721574957719e-01
4.0528641532714e-01 2.5530630322967e-01 3.9444869796217e-01
1.0663381281807e+00 2.6073391688772e-01 3.7168661520428e-01
7.2577485485605e-01 2.6616163084952e-01 3.4892801553917e-01
3.8410096295562e-01 2.7158978924337e-01 3.2617166187287e-01
1.0416151150630e+00 2.7701858340546e-01 3.0341663987570e-01
6.9850192978317e-01 2.8244811881571e-01 2.8066229405822e-01
3.5487875609453e-01 2.8787844794207e-01 2.5790815843500e-01
1.0108212017068e+00 2.9330958825187e-01 2.3515390475831e-01
6.6637789623403e-01 2.9874153237532e-01 2.1239930782163e-01
3.2157948585814e-01 3.0417425336698e-01 1.8964422369772e-01
9.7644439287003e-01 3.0960770640257e-01 1.6688857729782e-01
6.3098269345285e-01 3.1504182753940e-01 1.4413235665072e-01
2.8519886836447e-01 3.2047652987791e-01 1.2137561203650e-01
9.3909386013982e-01 3.2591169741638e-01 9.8618458482265e-02
5.9266668774255e-01 3.3134717702542e-01 7.5861080194990e-02
2.4591575829795e-01 3.3678276924673e-01 5.3103735353223e-02
8.9883994280792e-01 3.4221821900092e-01 3.0346759410763e-02
5.5143943319917e-01 3.4765320770430e-01 7.5905648158378e-03
2.0224424686890e-01 3.5309545876799e-01 9.8482607923038e-01
8.5421367531306e-01 3.5852813043615e-01 9.6207313840065e-01
5.0587095624828e-01 3.6395897220946e-01 9.3932259097047e-01
1.5722505761132e-01 3.6938742529939e-01 9.1657501696262e-01
8.0828661403055e-01 3.7481290985537e-01 8.9383096334731e-01
4.5906725474901e-01 3.8023485493022e-01 8.7109090365318e-01
1.0957885401941e-01 3.8565273223909e-01 8.4835519923477e-01
-2.4016717302321e-01 3.9106609006698e-01 8.2562406729691e-01
4.0983957919399e-01 3.9647458337947e-01 8.0289756017295e-01
5.9608176396108e-02 4.0187799657144e-01 7.8017555888172e-01
-2.9085375699532e-01 4.0727625622591e-01 7.5745778200224e-01
3.5846004094033e-01 4.1266943252061e-01 7.3474380884130e-01
7.5546404639554e-03 4.1805772929207e-01 7.1203311410381e-01
6.5643408513245e-01 4.2344146409912e-01 6.8932511003474e-01
3.0510147750244e-01 4.2882104084639e-01 6.6661919136845e-01
-4.6441033998264e-02 4.3419691857648e-01 6.4391477837699e-01
-3.9819231732161e-01 4.3956958082182e-01 6.2121135377013e-01
2.4984774658290e-01 4.4493951027471e-01 5.9850849002016e-01
-1.0232151301438e-01 4.5030717332165e-01 5.7580586465692e-01
-4.5470096331699e-01 4.5567301806714e-01 5.5310326197686e-01
1.9270938853376e-01 4.6103748777298e-01 5.3040056029560e-01
5.8744783379918e-01 2.3977930533381e-01 5.0823049680450e-01
4.0940101404550e-01 2.4549547736020e-01 4.8550182181675e-01
1.1028650956663e+00 2.5094538709297e-01 4.6267599958958e-01
7.7255390648499e-01 2.5637478526061e-01 4.3988666982760e-01
4.3519216922231e-01 2.6179964507095e-01 4.1711997669292e-01
1.0948335996808e+00 2.6722319040495e-01 3.9436400775192e-01
7.5290905254674e-01 2.7264666369468e-01 3.7161329126137e-01
4.1004333984243e-01 2.7807063415110e-01 3.4886514120542e-01
1.0665477659392e+00 2.8349538417511e-01 3.2611811171273e-01
7.2259087861712e-01 2.8892105720774e-01 3.0337136101872e-01
3.7826835327543e-01 2.9434772316631e-01 2.8062436744539e-01
1.0336352670586e+00 2.9977541061673e-01 2.5787679200058e-01
6.8872246301057e-01 3.0520412378807e-01 2.3512840764538e-01
3.4354551018301e-01 3.1063385178928e-01 2.1237906155908e-01
9.9810995681616e-01 3.1606457318095e-01 1.8962865519124e-01
6.5241462336067e-01 3.2149625724341e-01 1.6687713483186e-01
3.0645382006138e-01 3.2692886245105e-01 1.4412448905570e-01
9.6021896395111e-01 3.3236233229189e-01 1.2137075108893e-01
6.1369986119338e-01 3.3779658846123e-01 9.8616004875391e-02
2.6688580528614e-01 3.4323152154304e-01 7.5860393799644e-02
9.1976657051287e-01 3.4866697955194e-01 5.3104130847102e-02
5.7233332981607e-01 3.5410275513451e-01 3.0347508565669e-02
2.2457948647328e-01 3.5953857278497e-01 7.5909066636090e-03
8.7500625304819e-01 3.6498231961872e-01 9.8482508592225e-01
5.2661335426335e-01 3.7041691735134e-01 9.6207018784252e-01
1.7789858270872e-01 3.7585022487591e-01 9.3931696876673e-01
8.2886900639098e-01 3.8128164322948e-01 9.1656609361887e-01
4.7953488197088e-01 3.8671051965418e-01 8.9381823100762e-01
1.2990893922375e-01 3.9213617871232e-01 8.7107400670239e-01
-2.1999452963826e-01 3.9755796151475e-01 8.4833395390200e-01
4.2983937348910e-01 4.0297526905166e-01 8.2559846687992e-01
7.9425311314803e-02 4.0838760449887e-01 8.0286776480863e-01
-2.7122288490780e-01 4.1379460921591e-01 7.8014187134560e-01
3.7790749610571e-01 4.1919608795988e-01 7.5742061319631e-01
2.6828042874107e-02 4.2459202035378e-01 7.3470363795932e-01
6.7554940790180e-01 4.2998255750421e-01 7.1199044875876e-01
3.2408154684414e-01 4.3536800455477e-01 6.8928045097483e-01
-2.7566121198966e-02 4.4074879170762e-01 6.6657300504609e-01
-3.7938471149669e-01 4.4612543774322e-01 6.4386747890022e-01
2.6863397447871e-01 4.5149851119523e-01 6.2116329401417e-01
-8.3502763925476e-02 4.5686859489925e-01 5.9845996025919e-01
-4.3578864321378e-01 4.6223625941849e-01 5.7575709629950e-01
2.1178175720799e-01 4.6760204972295e-01 5.5305443404529e-01
-1.4078645320692e-01 4.7296648749867e-01 5.3035180718490e-01
-7.9246191786092e-02 2.5198804231572e-01 5.0823037771828e-01
1.2427547508447e+00 2.5753678702015e-01 4.8492412001732e-01
8.4531645738783e-01 2.6293145466264e-01 4.6232467620788e-01
4.8848644382931e-01 2.6833137078456e-01 4.3965135798582e-01
1.1384083418727e+00 2.7374028969760e-01 4.1694639721292e-01
7.9059404349302e-01 2.7915348434381e-01 3.9422877476352e-01
4.4377863016186e-01 2.8456944202878e-01 3.7150442362188e-01
1.0974645965640e+00 2.8998753498985e-01 3.4877572652375e-01
7.5141243951696e-01 2.9540744455880e-01 3.2604381176947e-01
4.0548867161132e-01 3.0082898168940e-01 3.0330927531262e-01
1.0596094401804e+00 3.0625202000127e-01 2.8057245579536e-01
7.1371629967250e-01 3.1167646752376e-01 2.5783355500300e-01
3.6776464508290e-01 3.1710225364137e-01 2.3509269682668e-01
1.0217177532231e+00 3.2252932243390e-01 2.1234995928369e-01
6.7554355338230e-01 3.2795762857735e-01 1.8960539410997e-01
3.2921284091132e-01 3.3338713384364e-01 1.6685904073018e-01
9.8269831817992e-01 3.3881780299887e-01 1.4411093814537e-01
6.3597415350073e-01 3.4424959823996e-01 1.2136113674856e-01
2.8901589659220e-01 3.4968247150805e-01 9.8609711237413e-02
9.4180066198977e-01 3.5511635421322e-01 7.5856775180715e-02
5.9430752471393e-01 3.6055114418935e-01 5.3102497184924e-02
2.4651808101466e-01 3.6598669013787e-01 3.0347117882371e-02
8.9841711878111e-01 3.7142277445170e-01 7.5909660961124e-03
5.4847703643125e-01 3.7686745896817e-01 9.8482462191530e-01
1.9973241608201e-01 3.8230347575116e-01 9.6206847817333e-01
-1.4934443843342e-01 3.8773881236132e-01 9.3931317954213e-01
5.0124985634749e-01 3.9317285861687e-01 9.1655944884646e-01
1.5152362143908e-01 3.9860491307564e-01 8.9380805875380e-01
-1.9851052253120e-01 4.0403421115401e-01 8.7105978315794e-01
4.5116335353233e-01 4.0945996673007e-01 8.4831533827780e-01
1.0056329215922e-01 4.1488142368078e-01 8.2557532088010e-01
-2.5029174120096e-01 4.2029791141861e-01 8.0284015284878e-01
3.9861716706171e-01 4.2570889720955e-01 7.8011004112420e-01
4.7308269747765e-02 4.3111402823877e-01 7.5738495975130e-01
-3.0420104270643e-01 4.3651315791929e-01 7.3466465703272e-01
3.4410588525207e-01 4.4190635332435e-01 7.1194868658504e-01
-7.7547385983585e-03 4.4729388330690e-01 6.8923645739842e-01
-3.5976690818386e-01 4.5267618944618e-01 6.6652729542210e-01
2.8808522464815e-01 4.5805384421214e-01 6.4382050799848e-01
-6.4182872023465e-02 4.6342750249437e-01 6.2111544265565e-01
-4.1655653886139e-01 4.6879785365762e-01 5.9841153319099e-01
2.3097760446497e-01 4.7416558125729e-01 5.7570832832009e-01
-1.2156863828464e-01 4.7953133628201e-01 5.5300550092862e-01
-4.7418497677459e-01 4.8489572737686e-01 5.3030283856143e-01
5.3466802120335e-01 2.6380887047789e-01 5.0825109272987e-01
1.0761079492076e+00 2.6900051294098e-01 4.8550158845638e-01
6.1750138078027e-01 2.7474153179540e-01 4.6232456195386e-01
1.2200636124835e+00 2.8026455279781e-01 4.3946841243845e-01
8.5074992303557e-01 2.8568216507895e-01 4.1676964621824e-01
4.9225244728861e-01 2.9108918578704e-01 3.9408002253195e-01
1.1387565533064e+00 2.9649798657505e-01 3.7138102814191e-01
7.8791117107443e-01 3.0190979369564e-01 3.4867282128435e-01
4.3860280126952e-01 3.0732434999861e-01 3.2595738685875e-01
1.0902375242983e+00 3.1274128069080e-01 3.0323634098904e-01
7.4246755323509e-01 3.1816028372793e-01 2.8051080501174e-01
3.9507281916614e-01 3.2358113580474e-01 2.5778153533145e-01
1.0479043314543e+00 3.2900367424077e-01 2.3504904208394e-01
7.0085488098877e-01 3.3442778102810e-01 2.1231367080878e-01
3.5384291802542e-01 3.3985337151396e-01 1.8957565628268e-01
1.0068032440900e+00 3.4528038624004e-01 1.6683515892420e-01
6.5968145434321e-01 3.5070878418166e-01 1.4409229117935e-01
3.1243056901771e-01 3.5613853590507e-01 1.2134713878118e-01
9.6500902192610e-01 3.6156961542573e-01 9.8599780108390e-02
6.1737954657577e-01 3.6700198976865e-01 7.5850305745935e-02
2.6950869643342e-01 3.7243560547492e-01 5.3098839463230e-02
9.2136683730561e-01 3.7787037166302e-01 3.0345560938547e-02
5.7292849520527e-01 3.8330613983409e-01 7.5907295148898e-03
2.2263734444082e-01 3.8875115540731e-01 9.8482472823875e-01
8.7355722960176e-01 3.9418800809707e-01 9.6206814091641e-01
5.2413475211123e-01 3.9962483742906e-01 9.3931147093992e-01
1.7436750041881e-01 4.0506105889513e-01 9.1655545786213e-01
-1.7574015152840e-01 4.1049595787917e-01 8.9380094425905e-01
4.7382258089434e-01 4.1592870903350e-01 8.7104883172006e-01
1.2307198929480e-01 4.2135841481325e-01 8.4830001770938e-01
-2.2797146210940e-01 4.2678416152931e-01 8.2555531992114e-01
4.2071532185976e-01 4.3220508729262e-01 8.0281539901913e-01
6.9156612722189e-02 4.3762045302735e-01 7.8008069271719e-01
-2.8262323528129e-01 4.4302970638796e-01 7.5735137305366e-01
3.6539964224250e-01 4.4843252930732e-01 7.3462733463258e-01
1.3248558147672e-02 4.5382886262445e-01 7.1190821578132e-01
-3.3905345211289e-01 4.5921890492879e-01 6.8919344864143e-01
3.0851670528343e-01 4.6460308659599e-01 6.6648232950660e-01
-4.4017691663746e-02 4.6998202347029e-01 6.4377409798172e-01
-3.9663343299006e-01 4.7535645748253e-01 6.2106801296989e-01
2.5069199170629e-01 4.8072719336085e-01 5.9836341498084e-01
-1.0202045114268e-01 4.8609504105703e-01 5.7565976740733e-01
-4.5475212486914e-01 4.9146077223030e-01 5.5295667352633e-01
1.9251282530205e-01 4.9682509617305e-01 5.3025387008919e-01
1.8924036648524e-01 2.7568931388826e-01 5.0824200148081e-01
7.8101825313843e-01 2.8093904847505e-01 4.8552029677779e-01
1.3599516293847e+00 2.8637639298153e-01 4.6267565700549e-01
9.5164020649227e-01 2.9201490243548e-01 4.3965113454784e-01
5.6668719946400e-01 2.9754567634046e-01 4.1676953705394e-01
1.1973740702069e+00 3.0299313315554e-01 3.9401108289924e-01
8.3660776867086e-01 3.0841350882382e-01 3.7129426318683e-01
4.8068807847797e-01 3.1382704500578e-01 3.4858650699031e-01
1.1276944235855e+00 3.1923994749584e-01 3.2587765841669e-01
7.7656007760662e-01 3.2465408665464e-01 3.0316494569017e-01
4.2665054294641e-01 3.3006997875183e-01 2.8044789965235e-01
1.0775645828653e+00 3.3548770674528e-01 2.5772672282682e-01
7.2903416049289e-01 3.4090722058057e-01 2.3500176404497e-01
3.8087099589644e-01 3.4632843994677e-01 2.1227335358305e-01
1.0329364875339e+00 3.5175129015898e-01 1.8954175870192e-01
6.8512401842230e-01 3.5717571404492e-01 1.6680717959242e-01
3.3734816905162e-01 3.6260167448739e-01 1.4406975864621e-01
9.8953797068455e-01 3.6802915255769e-01 1.2132959423149e-01
6.4163263911892e-01 3.7345814259949e-01 9.8586756659359e-02
2.9357891364306e-01 3.7888864422474e-01 7.5841306337752e-02
9.4532949843028e-01 3.8432065064255e-01 5.3093314799148e-02
5.9684230991927e-01 3.8975413263458e-01 3.0342889212247e-02
2.4808034416459e-01 3.9518901773454e-01 7.5902003829136e-03
8.9745893968451e-01 4.0063373966194e-01 9.8482541715706e-01
5.4806578458920e-01 4.0607079066080e-01 9.6206925051351e-01
1.9832156429213e-01 4.1150848659070e-01 9.3931202472708e-01
8.4821574791731e-01 4.1694631142348e-01 9.1655444089910e-01
4.9774631855018e-01 4.2238358410095e-01 8.9379735807664e-01
1.4691989539417e-01 4.2781946244370e-01 8.7104176298829e-01
-2.0424881397791e-01 4.3325297067328e-01 8.4828871159890e-01
4.4426174056277e-01 4.3868305280367e-01 8.2553924558347e-01
9.2478172869806e-02 4.4410864874571e-01 8.0279429400618e-01
7.4043018858518e-01 4.4952878416008e-01 7.8005457636331e-01
3.8814883685647e-01 4.5494266087898e-01 7.5732052521870e-01
3.5665332057243e-02 4.6034973360814e-01 7.3459224374938e-01
-3.1698941149868e-01 4.6574976088155e-01 7.1186950644926e-01
3.3021535133404e-01 4.7114282290615e-01 6.8915180233494e-01
-2.2689405812861e-02 4.7652930456098e-01 6.6643841183163e-01
-3.7567222648851e-01 4.8190984726638e-01 6.4372850288470e-01
2.7129861245237e-01 4.8728527808516e-01 6.2102122952003e-01
-8.1745699489489e-02 4.9265652779751e-01 5.9831581711414e-01
-4.3477574717345e-01 4.9802455118289e-01 5.7561162258847e-01
2.1223476969342e-01 5.0339026172289e-01 5.5290816363706e-01
-1.4069200459159e-01 5.0875448932914e-01 5.3020511743218e-01
-1.4938146128122e-01 2.8758710515393e-01 5.0822956728292e-01
1.4622006387436e+00 2.9287860834310e-01 4.8551022363066e-01
1.0649064704388e+00 2.9822397955147e-01 4.6276324496759e-01
6.6757101986479e-01 3.0373629201732e-01 4.3988622330728e-01
1.2790275951964e+00 3.0931091278042e-01 4.1694606989767e-01
9.0249508237416e-01 3.1482824891751e-01 3.9407980952926e-01
5.3545193426969e-01 3.2028954798247e-01 3.7129415936765e-01
1.1746862278797e+00 3.2572246692289e-01 3.4855225436505e-01
8.1799188088981e-01 3.3114357779694e-01 3.2582893711396e-01
4.6399759534200e-01 3.3656048387944e-01 3.0311186977220e-01
1.1118419026070e+00 3.4197649016435e-01 2.8039553576162e-01
7.6096402142622e-01 3.4739302315079e-01 2.5767757375459e-01
4.1098403496722e-01 3.5281069348820e-01 2.3495702873169e-01
1.0616341521933e+00 3.5822975478877e-01 2.1223356012372e-01
7.1271819291699e-01 3.6365030622742e-01 1.8950708476856e-01
3.6408694584546e-01 3.6907238490512e-01 1.6677761521019e-01
1.0156227325681e+00 3.7449600902504e-01 1.4404518723539e-01
6.6722949752158e-01 3.7992119783952e-01 1.2130982961861e-01
3.1882633999381e-01 3.8534797948289e-01 9.8571555187436e-02
9.7034328207323e-01 3.9077639126261e-01 7.5830364120091e-02
6.2171856454354e-01 3.9620647395141e-01 5.3086255821252e-02
2.7289705250853e-01 4.0163826018085e-01 3.0339248178387e-02
9.2382950128033e-01 4.0707175643662e-01 7.5894038711590e-03
5.7290379816424e-01 4.1251558378794e-01 9.8482666187445e-01
2.2322645545128e-01 4.1795217627619e-01 9.6207179715412e-01
-1.2680773592308e-01 4.2339004723472e-01 9.3931491347802e-01
5.2278024428114e-01 4.2882879352651e-01 9.1655660787604e-01
1.7197924038940e-01 4.3426782281748e-01 8.9379768366678e-01
-1.7921122842244e-01 4.3970633582343e-01 8.7103914410085e-01
4.6921918825338e-01 4.4514333317473e-01 8.4828215146866e-01
1.1729089932930e-01 4.5057765500009e-01 8.2552794779690e-01
7.6503265940929e-01 4.5600805572326e-01 8.0277774258619e-01
4.1247919602011e-01 4.6143330820127e-01 7.8003258379525e-01
5.9668698871033e-02 4.6685232282639e-01 7.5729323628245e-01
-2.9335924159697e-01 4.7226426181751e-01 7.3456009332350e-01
3.5343517022724e-01 4.7766862872810e-01 7.1183314043616e-01
9.1671554783634e-05 4.8306531799794e-01 6.8911197877260e-01
-3.5334970506341e-01 4.8845461717141e-01 6.6639590199586e-01
2.9315178115584e-01 4.9383716290167e-01 6.4368400974386e-01
-6.0362506751962e-02 4.9921385954272e-01 6.2097533475082e-01
-4.1385148326051e-01 5.0458577515237e-01 5.9826896009453e-01
2.3272400978709e-01 5.0995403319422e-01 5.7556410748126e-01
-1.2060078764613e-01 5.1531971806477e-01 5.5286018567687e-01
-4.7379631427464e-01 5.2068380830905e-01 5.3015679786850e-01
5.1432017040306e-01 2.9949251684531e-01 5.0821353914323e-01
1.1363826409304e+00 3.0480794059653e-01 4.8549568640335e-01
7.5243876133010e-01 3.1014014451721e-01 4.6277097998391e-01
1.3662597130399e+00 3.1556135954733e-01 4.3998801557817e-01
9.8224139691615e-01 3.2107809269183e-01 4.1711943175963e-01
6.0415165388252e-01 3.2661530902759e-01 3.9422834911300e-01
1.2333018920930e+00 3.3211841212788e-01 3.7138071686159e-01
8.6868373052390e-01 3.3758373884027e-01 3.4858630493152e-01
5.0869390613274e-01 3.4302464544863e-01 3.2582883889315e-01
1.1520001891738e+00 3.4845246816946e-01 3.0309209932466e-01
7.9765070122634e-01 3.5387398468637e-01 2.8036556182279e-01
4.4498445671940e-01 3.5929286061929e-01 2.5764322897417e-01
1.0935374556036e+00 3.6471102429335e-01 2.3492184016345e-01
7.4297605348539e-01 3.7012948841773e-01 2.1219965272789e-01
3.9305337786235e-01 3.7554879109821e-01 1.8947573800068e-01
1.0435811370584e+00 3.8096922713805e-01 1.6674959469149e-01
6.9441120110852e-01 3.8639097010775e-01 1.4402093949858e-01
3.4542337667473e-01 3.9181413764715e-01 1.2128959435226e-01
9.9651717554179e-01 3.9723882639510e-01 9.8555426965614e-02
6.4760623099777e-01 4.0266512954276e-01 7.5818322142538e-02
2.9861454035997e-01 4.0809314310564e-01 5.3078172511850e-02
9.4947403660248e-01 4.1352296335211e-01 3.0334883385375e-02
6.0012319556831e-01 4.1895467583978e-01 7.5883895989340e-03
2.4892398590367e-01 4.2439708172971e-01 9.8482838875883e-01
-1.0100266738886e-01 4.2983258769587e-01 9.6207566241731e-01
5.4871265611118e-01 4.3526991745164e-01 9.3932005607259e-01
1.9803649181947e-01 4.4070882028984e-01 9.1656199430596e-01
-1.5305423889130e-01 4.4614884987887e-01 8.9380213942802e-01
4.9542949337283e-01 4.5158932152476e-01 8.7104141732942e-01
1.4349003678261e-01 4.5702928928109e-01 8.4828100894555e-01
7.9114327547848e-01 4.6246755758916e-01 8.2552229369746e-01
4.3841753225315e-01 4.6790273890988e-01 8.0276673987081e-01
8.5351189545639e-02 4.7333335973640e-01 7.8001575158810e-01
-2.6801049389995e-01 4.7875800423190e-01 7.5727049831730e-01
3.7838143502327e-01 4.8417547224638e-01 7.3453176352910e-01
2.4577259085753e-02 4.8958492196590e-01 7.1179984828747e-01
-3.2937250359587e-01 4.9498596959785e-01 6.8907455265739e-01
3.1658294616848e-01 5.0037872787936e-01 6.6635523795276e-01
-3.7504779496559e-02 5.0576377798102e-01 6.4364095337530e-01
-3.9158309994613e-01 5.1114208208996e-01 6.2093059716607e-01
2.5440065789772e-01 5.1651485460534e-01 5.9822307752906e-01
-9.9502782454393e-02 5.2188341704435e-01 5.7551744241287e-01
-4.5324740985820e-01 5.2724906368303e-01 5.5281295816441e-01
1.9320539246024e-01 5.3261296035517e-01 5.3010913173626e-01
1.7908774763065e-01 3.1140247373412e-01 5.0819391394703e-01
8.0762243115293e-01 3.1673261985989e-01 4.8547759169494e-01
1.4319129550146e+00 3.2206800751891e-01 4.6275953346082e-01
1.0538298188222e+00 3.2744965131478e-01 4.4001559214761e-01
6.7571505694179e-01 3.3290395709563e-01 4.1721498791413e-01
1.2999090437698e+00 3.3841048357137e-01 3.9436337016336e-01
9.2827730008918e-01 3.4392367090126e-01 3.7150390541210e-01
5.6145869762298e-01 3.4941478149720e-01 3.4867241752076e-01
1.1989900339802e+00 3.5487965584251e-01 3.2587736393100e-01
8.4000210033734e-01 3.6032477898393e-01 3.0311167916170e-01
4.8366488068284e-01 3.6575746285892e-01 2.8036546945149e-01
1.1293160575860e+00 3.7118303554748e-01 2.5763082110601e-01
7.7645532508982e-01 3.7660488218077e-01 2.3490238348647e-01
4.2470777459041e-01 3.8202504846356e-01 2.1217677029003e-01
1.0737892613838e+00 3.8744475434309e-01 1.8945188903240e-01
7.2348009987900e-01 3.9286473355142e-01 1.6672644676679e-01
3.7360630234063e-01 3.9828544108761e-01 1.4399962776619e-01
1.0240264110849e+00 4.0370717702999e-01 1.2127089305947e-01
6.7462225410945e-01 4.0913016012506e-01 9.8539858513177e-02
3.2529243192126e-01 4.1455457123383e-01 7.5806222461974e-02
9.7594773971180e-01 4.1998057783473e-01 5.3069725410969e-02
6.2650801599890e-01 4.2540834529155e-01 3.0330132112143e-02
2.7690010996560e-01 4.3083803720370e-01 7.5872312677929e-03
-7.4537682610358e-02 4.3627862169469e-01 9.8483047462225e-01
5.7532631327794e-01 4.4171249882075e-01 9.6208060404083e-01
2.2483741601597e-01 4.4714860283352e-01 9.3932718114934e-01
-1.2605172120884e-01 4.5258686254721e-01 9.1657039819927e-01
5.2262148330590e-01 4.5802702734837e-01 8.9381068995514e-01
1.7083208560032e-01 4.6346860153002e-01 8.7104879432319e-01
8.1857009402651e-01 4.6891078678438e-01 8.4828578843839e-01
4.6584220667835e-01 4.7435245219175e-01 8.2552307695485e-01
1.1267210319901e-01 4.7979215353332e-01 8.0276231226356e-01
-2.4090115332169e-01 4.8522821835090e-01 7.8000524026969e-01
4.0517352525864e-01 4.9065889814157e-01 7.5725348902097e-01
5.0954950899526e-02 4.9608256806678e-01 7.3450834241986e-01
-3.0349433866089e-01 5.0149793627338e-01 7.1177055480218e-01
3.4188932612880e-01 5.0690421809090e-01 6.8904025560866e-01
-1.2830060700288e-02 5.1230123780304e-01 6.6631696852473e-01
-3.6758774990318e-01 5.1768943817830e-01 6.4359973721708e-01
2.7768224341069e-01 5.2306979860020e-01 6.2088732250566e-01
-7.6953519692594e-02 5.2844368143337e-01 5.9817842113304e-01
-4.3143025391819e-01 5.3381264033460e-01 5.7547185645514e-01
2.1431138072515e-01 5.3917823072715e-01 5.5276670484420e-01
-1.3967861979915e-01 5.4454185880319e-01 5.3006234355138e-01
-1.5556718624742e-01 3.2331567977172e-01 5.0817080160623e-01
1.4773569353188e+00 3.2865547433738e-01 4.8545603238992e-01
1.1071527284450e+00 3.3399655221982e-01 4.6274086742440e-01
7.3484687125912e-01 3.3936285223625e-01 4.4001334530053e-01
1.3618084848025e+00 3.4477739535253e-01 4.1725268335209e-01
9.8945195239137e-01 3.5024279664732e-01 3.9444784950034e-01
6.1914310481700e-01 3.5573769398119e-01 3.7161256700832e-01
1.2519020381858e+00 3.6123424344280e-01 3.4877512176335e-01
8.8807899731994e-01 3.6671562929114e-01 3.2595689663845e-01
5.2743757003766e-01 3.7217820041976e-01 3.0316456482213e-01
1.1694722781168e+00 3.7762507912883e-01 2.8039525882395e-01
8.1364693996513e-01 3.8306089367513e-01 2.5764305031213e-01
4.5949396933978e-01 3.8848960198726e-01 2.3490229721374e-01
1.1066339364538e+00 3.9391406212146e-01 2.1216863421732e-01
7.5476563873667e-01 3.9933620048156e-01 1.8943895946786e-01
4.0364967757999e-01 4.0475728309460e-01 1.6671114441157e-01
1.0530933154027e+00 4.1017814288977e-01 1.4398373758210e-01
7.0293838766358e-01 4.1559934154192e-01 1.2125573311371e-01
3.5305213180726e-01 4.2102127800383e-01 9.8526406573807e-02
1.0033203579778e+00 4.2644425957763e-01 7.5795203858235e-02
6.5364240864391e-01 4.3186854767361e-01 5.3061668736784e-02
3.0392748884843e-01 4.3729438599158e-01 3.0325398629869e-02
-4.5907909542257e-02 4.4272201525245e-01 7.5860224109542e-03
6.0245375764859e-01 4.4816055284357e-01 9.8483275116060e-01
2.5215203440995e-01 4.5359240295859e-01 9.6208625620770e-01
-9.8489700231946e-02 4.5902669520814e-01 9.3933580969555e-01
5.5046687435206e-01 4.6446354593242e-01 9.1658133272868e-01
1.9896842574903e-01 4.6990293116800e-01 8.9382296152669e-01
8.4697303301181e-01 4.7534460686790e-01 8.7106113637471e-01
4.9445384302204e-01 4.8078802015857e-01 8.4829668583714e-01
1.4140264468885e-01 4.8623223038005e-01 8.2553087839081e-01
-2.1216763478447e-01 4.9167586891035e-01 8.0276540622351e-01
4.3377742861994e-01 4.9711717095101e-01 7.8000227045425e-01
7.9291354133440e-02 5.0255410310944e-01 7.5724356016614e-01
-2.7555807120391e-01 5.0798458502979e-01 7.3449115667043e-01
3.6930536417895e-01 5.1340676917355e-01 7.1174643292838e-01
1.3961358442928e-02 5.1881931644992e-01 6.8901003346568e-01
-3.4150953467002e-01 5.2422160077207e-01 6.6628180035401e-01
3.0297381831712e-01 5.2961379414953e-01 6.4356086448503e-01
-5.2506136334153e-02 5.3499681550461e-01 6.2084587045574e-01
-4.0786582838907e-01 5.4037215932677e-01 5.9813526772588e-01
2.3697697628860e-01 5.4574164733471e-01 5.7542758960995e-01
-1.1790151397393e-01 5.5110716244967e-01 5.5272165536462e-01
-4.7243664501375e-01 5.5647042415391e-01 5.3001666265409e-01
5.1012653051589e-01 3.3523146149645e-01 5.0814436351856e-01
1.1462186153699e+00 3.4057786644764e-01 4.8543111883578e-01
7.7992416527646e-01 3.4592416508399e-01 4.6271777761084e-01
1.4118513193574e+00 3.5128506751515e-01 4.3999799555768e-01
1.0428524564004e+00 3.5667742619177e-01 4.1725888465561e-01
6.7383606224873e-01 3.6211009202512e-01 3.9448834381676e-01
1.3057089983590e+00 3.6757723173165e-01 3.7168568614877e-01
9.3931876715635e-01 3.7306184074053e-01 3.4886433651033e-01
5.7526537045253e-01 3.7854621703497e-01 3.2604312669837e-01
1.2137567098534e+00 3.8401971419424e-01 3.0323577056946e-01
8.5466083534781e-01 3.8947932547655e-01 2.8044743867695e-01
4.9766521976773e-01 3.9492648072812e-01 2.5767721678234e-01
1.1424103660916e+00 4.0036409292513e-01 2.3492158152088e-01
7.8855934409855e-01 4.0579502462118e-01 2.1217660407244e-01
4.3582149237509e-01 4.1122157591043e-01 1.8943887954067e-01
1.0839539289727e+00 4.1664544402660e-01 1.6670574703110e-01
7.3275501479683e-01 4.2206783741414e-01 1.4397521949526e-01
3.8205613654944e-01 4.2748961139042e-01 1.2124583248736e-01
1.0317141457482e+00 4.3291138299124e-01 9.8516494957504e-02
6.8160507839429e-01 4.3833361787031e-01 7.5786367253659e-02
3.3161915758378e-01 4.4375669301094e-01 5.3054769221285e-02
-1.8343090489084e-02 4.4918094070842e-01 3.0321114695993e-02
6.3162622270698e-01 4.5460667800434e-01 7.5848682514268e-03
2.7982736383684e-01 4.6004315087156e-01 9.8483501743242e-01
-7.0594438024733e-02 4.6547277027945e-01 9.6209214978310e-01
5.7866532082806e-01 4.7090483110258e-01 9.3934526714853e-01
2.2753129965520e-01 4.7633962282824e-01 9.1659401199250e-01
8.7593393706893e-01 4.8177735093851e-01 8.9383818432367e-01
5.2381249469797e-01 4.8721805746162e-01 8.7107784295799e-01
1.7111939932738e-01 4.9266151465316e-01 8.4831342739129e-01
-1.8217451501782e-01 4.9810710281734e-01 8.2554587732565e-01
4.6392517207822e-01 5.0355369944792e-01 8.0277670846050e-01
1.0944006538927e-01 5.0899962419806e-01 7.8000798984669e-01
-2.4558066954221e-01 5.1444269139945e-01 7.5724217508475e-01
3.9893597017379e-01 5.1988040514579e-01 7.3448177938571e-01
4.3079763697620e-02 5.2531028713789e-01 7.1172896060036e-01
-3.1305067966092e-01 5.3073027108143e-01 6.8898512194838e-01
3.3064617572818e-01 5.3613906055311e-01 6.6625066672186e-01
-2.5727499256952e-02 5.4153635161379e-01 6.4352498663585e-01
-3.8206843568361e-01 5.4692286238062e-01 6.2080668142673e-01
2.6172823369859e-01 5.5230016751110e-01 5.9809393030449e-01
-9.4233205912186e-02 5.5767038709576e-01 5.7538489573125e-01
-4.4985477632460e-01 5.6303581551662e-01 5.5267804550767e-01
1.9494748596477e-01 5.6839858617219e-01 5.2997232326375e-01
1.7604765453056e-01 3.4714940474178e-01 5.0811479169970e-01
8.1452795642245e-01 3.5250047937762e-01 4.8540300818442e-01
1.4511382134448e+00 3.5785101377739e-01 4.6269112541356e-01
1.0862610501539e+00 3.6321052138959e-01 4.3997555835267e-01
7.2045183400676e-01 3.6859091281753e-01 4.1724832929003e-01
1.3543251764320e+00 3.7400130198763e-01 3.9449987637311e-01
9.8849790287282e-01 3.7944292616432e-01 3.7172493595987e-01
6.2358143036407e-01 3.8490808333353e-01 3.4892701232571e-01
1.2601247255797e+00 3.9038408733467e-01 3.2611723302831e-01
8.9850078343913e-01 3.9585918478563e-01 3.0330851640058e-01
5.3883962126813e-01 4.0132621692998e-01 2.8051016087977e-01
1.1810610657495e+00 4.0678274554765e-01 2.5772618824812e-01
8.2496084120420e-01 4.1222932816854e-01 2.3495659824614e-01
4.7028880252107e-01 4.1766779057977e-01 2.1219932064332e-01
1.1167963256166e+00 4.2310017600403e-01 1.8945164942579e-01
7.6425739049953e-01 4.2852828609280e-01 1.6671099113032e-01
4.1247428341867e-01 4.3395355772162e-01 1.4397514615852e-01
1.0612761382121e+00 4.3937708667145e-01 1.2124234937926e-01
7.1051485515442e-01 4.4479969869106e-01 9.8511211406738e-02
3.6006050543481e-01 4.5022202520357e-01 7.5780631573318e-02
9.7970627522954e-03 4.5564456895317e-01 5.3049713008023e-02
6.5961872906487e-01 4.6106775659031e-01 3.0317690601363e-02
3.0942689760070e-01 4.6649197907801e-01 7.5838745766434e-03
-4.2491575693183e-02 4.7192658845859e-01 9.8483705949290e-01
6.0701256419939e-01 4.7735400006233e-01 9.6209775301609e-01
2.5622876911957e-01 4.8278363259133e-01 9.3935473080189e-01
9.0506980020356e-01 4.8821591366027e-01 9.1660738476718e-01
5.5345101975826e-01 4.9365124927059e-01 8.9385518717554e-01
2.0129277637683e-01 4.9908996077234e-01 8.7109778235401e-01
-1.5147540506277e-01 5.0453218260337e-01 8.4833512148741e-01
4.9509127618389e-01 5.0997771869013e-01 8.2556763225200e-01
1.4096043859275e-01 5.1542587094590e-01 8.0279639202740e-01
-2.1386990249063e-01 5.2087527984395e-01 7.8002324199780e-01
4.3063389538081e-01 5.2632384720908e-01 7.5725075492199e-01
7.4541364258977e-02 5.3176882421434e-01 7.3448198046809e-01
-2.8204801645040e-01 5.3720711651540e-01 7.1171996279451e-01
3.6098469201188e-01 5.4263577741141e-01 6.8896714013647e-01
3.7671541104686e-03 5.4805256550304e-01 6.6622482774349e-01
-3.5357088752131e-01 5.5345639570436e-01 6.4349298043098e-01
2.8910102497426e-01 5.5884754235937e-01 6.2077032153091e-01
-6.8084899852977e-02 5.6422753817630e-01 5.9805477722675e-01
-4.2499600358441e-01 5.6959880919158e-01 5.7534404746253e-01
2.1849429546738e-01 5.7496416334218e-01 5.5263611711341e-01
-1.3750405819498e-01 5.8032628739528e-01 5.2992956385126e-01
-1.5787389269543e-01 3.5906921704250e-01 5.0808229908558e-01
1.4824641562949e+00 3.6442365962124e-01 4.8537189115902e-01
1.1213068562989e+00 3.6977750063559e-01 4.6266130331008e-01
7.5890699694835e-01 3.7513718043740e-01 4.3994839427268e-01
1.3956398806481e+00 3.8051114842890e-01 4.1722811554328e-01
1.0319362138826e+00 3.8590715160617e-01 3.9449354117825e-01
6.6823351079865e-01 3.9132894520789e-01 3.7173890425355e-01
1.3049677263742e+00 3.9677439466575e-01 3.4896301272004e-01
9.4256488003311e-01 4.0223624443332e-01 3.2617059116645e-01
5.8139243151181e-01 4.0770513210229e-01 3.0337041502651e-01
1.2216921289944e+00 4.1317293444869e-01 2.8057162974055e-01
8.6354779963895e-01 4.1863460441518e-01 2.5778082420198e-01
5.0690629725507e-01 4.2408816187276e-01 2.3500116259465e-01
1.1516269325868e+00 4.2953370842429e-01 2.1223306287343e-01
7.9752912086369e-01 4.3497237721778e-01 1.8947533923957e-01
4.4442459857823e-01 4.4040561536892e-01 1.6672614055352e-01
1.0921343674887e+00 4.4583481122463e-01 1.4398351774613e-01
7.4049531444209e-01 4.5126115071457e-01 1.2124569263020e-01
3.8936114661105e-01 4.5668559452364e-01 9.8511144903312e-02
3.8600667132353e-02 4.6210890776572e-01 7.5778606498285e-02
6.8809504658370e-01 4.6753170670291e-01 5.3047016550179e-02
3.3773489554417e-01 4.7295450668380e-01 3.0315464641763e-02
-1.2582507854205e-02 4.7837776536948e-01 7.5831353002909e-03
6.3542039849125e-01 4.8381091665661e-01 9.8483867452857e-01
2.8489555881198e-01 4.8923637598311e-01 9.6210252834754e-01
-6.5878289452866e-02 4.9466363880948e-01 9.3936331054547e-01
5.8300233801224e-01 5.0009322195448e-01 9.1662022244487e-01
2.3144024651743e-01 5.0552567587350e-01 8.9387246546385e-01
-1.2066124230433e-01 5.1096155026965e-01 8.7111930272140e-01
5.2660481112612e-01 5.1640131851724e-01 8.4836017465015e-01
1.7315460503842e-01 5.2184524577288e-01 8.2559487930711e-01
-1.8107785859436e-01 5.2729319263707e-01 8.0282381171557e-01
4.6387017639983e-01 5.3274436950101e-01 7.8004822013529e-01
1.0800149018828e-01 5.3819710123083e-01 7.5727038235052e-01
-2.4863339612038e-01 5.4364871610990e-01 7.3449355849834e-01
3.9406362053816e-01 5.4909569662738e-01 7.1172159526597e-01
3.6228483233742e-02 5.5453416938410e-01 6.8895818766380e-01
-3.2198079046591e-01 5.5996065971487e-01 6.6620601088152e-01
3.1960167235955e-01 5.6537287737988e-01 6.4346607105890e-01
-3.8857077647241e-02 5.7077025009768e-01 6.2073756035023e-01
-3.9718865356482e-01 5.7615401575651e-01 5.9801826736962e-01
2.4477685174414e-01 5.8152685397773e-01 5.7530534588889e-01
-1.1279573740237e-01 5.8689220035026e-01 5.5259611813118e-01
-4.6976077940168e-01 5.9225348861631e-01 5.2988862577230e-01
5.0831868355498e-01 3.7099066847030e-01 5.0804711335886e-01
1.1501349310673e+00 3.7634757335584e-01 4.8533798123047e-01
7.9073771602345e-01 3.8170395833489e-01 4.6262858269680e-01
1.4303008769772e+00 3.8706436110212e-01 4.3991757468320e-01
1.0690859625199e+00 3.9243487666290e-01 4.1720166851998e-01
7.0740252601987e-01 3.9782169945267e-01 3.9447596114045e-01
1.3455713118061e+00 4.0322908531404e-01 3.7173547003489e-01
9.8391116020593e-01 4.0865769557176e-01 3.4897733028443e-01
6.2273979150531e-01 4.1410420638247e-01 3.2620244345074e-01
1.2623606840523e+00 4.1956243803966e-01 3.0341550856898e-01
9.0302620879899e-01 4.2502542681971e-01 2.8062336105423e-01
5.4489785189315e-01 4.3048735622557e-01 2.5783266873054e-01
1.1880302348053e+00 4.3594452921114e-01 2.3504827089843e-01
8.3238480898022e-01 4.4139532450371e-01 2.1227269221900e-01
4.7785967326871e-01 4.4683960550024e-01 1.8950652772790e-01
1.1243196119453e+00 4.5227805934450e-01 1.6674913624476e-01
7.7161814985440e-01 4.5771170346258e-01 1.4399926195380e-01
4.1961058700654e-01 4.6314159551620e-01 1.2125545374716e-01
6.8160169637023e-02 4.6856869703952e-01 9.8516295620728e-02
7.1713996790664e-01 4.7399383023588e-01 7.5780505623945e-02
3.6643240100012e-01 4.7941768323855e-01 5.3046957120915e-02
1.5927618600336e-02 4.8484083706745e-01 3.0314659535471e-02
6.6552141055342e-01 4.9026380029227e-01 7.5827207518811e-03
3.1348632192809e-01 4.9569606181155e-01 9.8483969564328e-01
-3.7024200740800e-02 5.0112003342769e-01 9.6210599673157e-01
6.1226244780446e-01 5.0654524070146e-01 9.3937015231078e-01
2.6124359393477e-01 5.1197223654328e-01 9.1663125401310e-01
-9.0187017803821e-02 5.1740164463982e-01 8.9388832752249e-01
5.5786053020908e-01 5.2283415781713e-01 8.7114035217584e-01
2.0527364181058e-01 5.2827050408141e-01 8.4838632987565e-01
-1.4805898123627e-01 5.3371135844129e-01 8.2562542823452e-01
4.9776011423314e-01 5.3915717343891e-01 8.0285722169974e-01
1.4264912095545e-01 5.4460790778931e-01 7.8008203337937e-01
-2.1343689915985e-01 5.5006266690170e-01 7.5730132253818e-01
4.2951111957816e-01 5.5551934541809e-01 7.3451796473738e-01
7.1567879983978e-02 5.6097446323390e-01 7.1173618007997e-01
-2.8712755065534e-01 5.6642343176584e-01 6.8896089643251e-01
3.5361187067023e-01 5.7186136411578e-01 6.6619658990517e-01
-6.0026930889153e-03 5.7728424831253e-01 6.4344602564021e-01
-3.6575402781577e-01 5.8269003549443e-01 6.2070950746132e-01
2.7457586487683e-01 5.8807918202003e-01 5.9798501685868e-01
-8.4792950633990e-02 5.9345443073493e-01 5.7526914065887e-01
-4.4364357130821e-01 5.9881994650705e-01 5.5255830385249e-01
1.9821975218553e-01 6.0418017721475e-01 5.2984975114887e-01
1.7459740073485e-01 3.8291356390303e-01 5.0800947215816e-01
8.1760868894267e-01 3.8827228371107e-01 4.8530150705211e-01
1.4596256043610e+00 3.9363061262580e-01 4.6259320619165e-01
1.1007717202878e+00 3.9899186201573e-01 4.3988368838041e-01
7.4123417306960e-01 4.0436052145121e-01 4.1717074548329e-01
1.3812399119442e+00 4.0974145046411e-01 3.9445090284055e-01
1.0210294671055e+00 4.1513862441736e-01 3.7172018903502e-01
6.6084277123841e-01 4.2055390368131e-01 3.4897546553997e-01
1.3009183097576e+00 4.2598636126172e-01 3.2621575664606e-01
9.4149225277239e-01 4.3143248867473e-01 3.0344289639855e-01
5.8278422312594e-01 4.3688721417762e-01 2.8066110927010e-01
1.2249709795823e+00 4.4234528192922e-01 2.5787573234514e-01
8.6816255640343e-01 4.4780239685719e-01 2.3509175748712e-01
5.1239413512088e-01 4.5325576380327e-01 2.1231284673378e-01
1.1576352016909e+00 4.5870404088212e-01 1.8954104460674e-01
8.0380827374925e-01 4.6414696976962e-01 1.6677700557789e-01
4.5080840492197e-01 4.6958494670990e-01 1.4402042858118e-01
9.8518436305706e-02 4.7501867919712e-01 1.2127047487916e-01
7.4681881077299e-01 4.8044896446726e-01 9.8526074924210e-02
3.9559281215599e-01 4.8587657071539e-01 7.5786115701124e-02
4.4728627544369e-02 4.9130218795616e-01 5.3049534892923e-02
6.9411944531544e-01 4.9672642020002e-01 3.0315353079839e-02
3.4366243076936e-01 5.0214979986576e-01 7.5826686356133e-03
-8.3704918683838e-03 5.0758183987900e-01 9.8484001316828e-01
6.4117447997588e-01 5.1300494612034e-01 9.6210779868691e-01
2.9056842954380e-01 5.1842863259788e-01 9.3937454708562e-01
-6.0294223799924e-02 5.2385344056563e-01 9.1663932739816e-01
5.8847514130941e-01 5.2927999599040e-01 8.9390110302835e-01
2.3675774823006e-01 5.3470903571171e-01 8.7115871004884e-01
-1.1557327851997e-01 5.4014141675973e-01 8.4841086728393e-01
5.3134819039248e-01 5.4557808893712e-01 8.2565624633204e-01
1.7738566024745e-01 5.5101999714359e-01 8.0289364292614e-01
-1.7759034116791e-01 5.5646786625625e-01 7.8012229730284e-01
4.6631461211575e-01 5.6192182279619e-01 7.5734238774142e-01
1.0904305640541e-01 5.6738085666048e-01 7.3455563050483e-01
-2.4938755267516e-01 5.7284226019386e-01 7.1176573944602e-01
3.9113320011480e-01 5.7830137922984e-01 6.8897831128634e-01
3.0804742289907e-02 5.8375210119863e-01 6.6619975935069e-01
-3.3011276300065e-01 5.8918823952500e-01 6.4343544182918e-01
3.0866557384606e-01 5.9460537950044e-01 6.2068785246843e-01
-5.2571777800270e-02 6.0000231639190e-01 5.9795592884296e-01
-4.1353485756137e-01 6.0538136932555e-01 5.7523587322168e-01
2.2606571228837e-01 6.1074744992537e-01 5.5252294184478e-01
-1.3350237576831e-01 6.1610637971616e-01 5.2981318005772e-01
-1.5905656992644e-01 3.9483772927028e-01 5.0796961904748e-01
1.4849307667479e+00 4.0019779195645e-01 4.8526270682869e-01
1.1280991586826e+00 4.0555759930901e-01 4.6255541271172e-01
7.7053890176581e-01 4.1091964468728e-01 4.3984713277302e-01
1.4123869311385e+00 4.1628730809752e-01 4.1713633929943e-01
1.0538124061602e+00 4.2166439537736e-01 3.9442054186033e-01
6.9499855437251e-01 4.2705436070046e-01 3.7169666901593e-01
3.3612993253906e-01 4.3245944256316e-01 3.4896185477708e-01
9.7738910561941e-01 4.3788002401840e-01 3.2621438106220e-01
6.1895788999949e-01 4.4331446554587e-01 3.0345438898547e-01
1.2610139858098e+00 4.4875949025892e-01 2.8068402421267e-01
9.0371814762399e-01 4.5421098638067e-01 2.5790692750903e-01
5.4719590014240e-01 4.5966492025712e-01 2.3512730208463e-01
1.1915229650164e+00 4.6511803426346e-01 2.1234897425151e-01
8.3672104961200e-01 4.7056815808519e-01 1.8957478670827e-01
4.8276409163372e-01 4.7601416800756e-01 1.6680642017193e-01
1.2959041168570e-01 4.8145574726116e-01 1.4404453243308e-01
7.7711569236268e-01 4.8689309976311e-01 1.2128903840167e-01
4.2524362915196e-01 4.9232670797943e-01 9.8539395418363e-02
7.3873227849186e-02 4.9775716490037e-01 7.5794827394992e-02
7.2290301809746e-01 5.0318507493453e-01 5.3054472938433e-02
3.7223292275186e-01 5.0861100616534e-01 3.0317467822045e-02
2.1764521124534e-02 5.1403547648429e-01 7.5829791233099e-03
6.6977360246663e-01 5.1946798670552e-01 9.8483958898307e-01
3.1941316778100e-01 5.2489093567386e-01 9.6210774166887e-01
-3.1042710038886e-02 5.3031379132174e-01 9.3937602618323e-01
6.1830059779034e-01 5.3573704743455e-01 9.1664356790376e-01
2.6733047880068e-01 5.4116127519220e-01 8.9390937671871e-01
-8.4075531125580e-02 5.4658716577734e-01 8.7117229606358e-01
5.6394780082695e-01 5.5201557197674e-01 8.4843096111856e-01
2.1125171238381e-01 5.5744753711989e-01 8.2568378692278e-01
-1.4232627369344e-01 5.6288428559910e-01 8.0292902577389e-01
5.0304211091729e-01 5.6832712541112e-01 7.8016496004424e-01
1.4718798085130e-01 5.7377718229837e-01 7.5739032098405e-01
-2.1003088647092e-01 5.7923486959394e-01 7.3460500758403e-01
4.3130713315808e-01 5.8469906205020e-01 7.1181103998190e-01
7.1230971178762e-02 5.9016618213048e-01 6.8901334203078e-01
-2.9009414820714e-01 5.9562981061529e-01 6.6621954608310e-01
3.4762307242734e-01 6.0108162274183e-01 6.4343812306583e-01
-1.5249986793207e-02 6.0651383888510e-01 6.2067527858780e-01
-3.7832321170327e-01 6.1192213373835e-01 5.9793245042538e-01
2.5879517941443e-01 6.1730731351225e-01 5.7520617208461e-01
-1.0350160796143e-01 6.2267478109804e-01 5.5249032681343e-01
-4.6483998965669e-01 6.2803218079201e-01 5.2977914725228e-01
5.0734372206722e-01 4.0676300448483e-01 5.0792780009204e-01
1.1521322277131e+00 4.1212406096165e-01 4.8522182388892e-01
7.9624619783871e-01 4.1748498720770e-01 4.6251544302123e-01
1.4397531434783e+00 4.2284771490292e-01 4.3980822495649e-01
1.0827551752979e+00 4.2821484402873e-01 4.1709908121008e-01
7.2538013209506e-01 4.3358937395936e-01 3.9438619783502e-01
1.3677691706544e+00 4.3897421533988e-01 3.7166721943894e-01
1.0100663639088e+00 4.4437159630260e-01 3.4893971046575e-01
6.5241414482173e-01 4.4978254114074e-01 3.2620178830174e-01
1.2949536790712e+00 4.5520659157435e-01 3.0345271950600e-01
9.3782520847124e-01 4.6064186944557e-01 2.8069325047258e-01
5.8116340869606e-01 4.6608547074688e-01 2.5792552859064e-01
1.2250868492230e+00 4.7153406544566e-01 2.3515263526113e-01
8.6968540079058e-01 4.7698450553220e-01 2.1237791767468e-01
5.1501114778401e-01 4.8243426204999e-01 1.8960437098201e-01
1.6107617607408e-01 4.8788161111474e-01 1.6683425146250e-01
8.0785689762136e-01 4.9332560062421e-01 1.4406896152790e-01
4.5530214192708e-01 4.9876588996730e-01 1.2130913728924e-01
1.0334196175340e-01 5.0420255417203e-01 9.8554833640350e-02
7.5189512714319e-01 5.0963591063424e-01 7.5805722126792e-02
4.0087449501142e-01 5.1506639165651e-01 5.3061255149710e-02
5.0190260501990e-02 5.2049446358778e-01 3.0320781388373e-02
6.9975145952699e-01 5.2592058369947e-01 7.5836148544338e-03
3.4784225031707e-01 5.3135419984522e-01 9.8483846155210e-01
-2.3863706113644e-03 5.3677770312703e-01 9.6210582584987e-01
6.4734702311551e-01 5.4220048771613e-01 9.3937442587897e-01
2.9694012224897e-01 5.4762297827503e-01 9.1664350373173e-01
-5.3717385028295e-02 5.5304565576051e-01 8.9391220065019e-01
5.9525302252524e-01 5.5846910327037e-01 8.7117949490700e-01
2.4371506593982e-01 5.6389406216667e-01 8.4844412934463e-01
-1.0848622498132e-01 5.6932149608370e-01 8.2570453708095e-01
5.3847195934055e-01 5.7475265212983e-01 8.0295878788449e-01
1.8438854895654e-01 5.8018908901716e-01 7.8020460635924e-01
-1.7095799556892e-01 5.8563260249905e-01 7.5743956283620e-01
4.7220463483769e-01 5.9108491466929e-01 7.3466160215446e-01
1.1367612059366e-01 5.9654693258107e-01 7.1187009513850e-01
-2.4665787883724e-01 6.0201744624937e-01 6.8906739773652e-01
3.9124783465933e-01 6.0749157388249e-01 6.6626024316956e-01
2.7646637045798e-02 6.1296013369311e-01 6.4345938309525e-01
-3.3702498994123e-01 6.1841153883597e-01 6.2067612850066e-01
2.9776226247525e-01 6.2383629539686e-01 5.9791708535154e-01
-6.7444799657192e-02 6.2923148369888e-01 5.7518106898581e-01
-4.3209778514169e-01 6.3460200040389e-01 5.5246082172651e-01
2.0433737743023e-01 6.3995775213067e-01 5.2974787915721e-01
1.7378902896542e-01 4.1868923971773e-01 5.0788426103822e-01
8.1923490874810e-01 4.2405102930815e-01 4.8517910315517e-01
1.4641282713308e+00 4.2941279714432e-01 4.6247353939553e-01
1.1085205631165e+00 4.3477607975300e-01 4.3976724574201e-01
7.5248911080144e-01 4.4014291899363e-01 4.1705942391506e-01
1.3961316097975e+00 4.4551570257751e-01 3.9434872812899e-01
1.0395576842942e+00 4.5089685350941e-01 3.7163334787708e-01
6.8288083153721e-01 4.5628842448232e-01 3.4891127738581e-01
1.3262137131810e+00 4.6169169912819e-01 3.2618073093142e-01
9.6966737975432e-01 4.6710691125768e-01 3.0344059450772e-01
6.1335219500414e-01 4.7253316503817e-01 2.8069077107013e-01
1.2573769900337e+00 4.7796858534285e-01 2.5793229672906e-01
9.0184421736280e-01 4.8341066110151e-01 2.3516717862446e-01
5.4684169773244e-01 4.8885668396586e-01 2.1239800754136e-01
1.9243393268693e-01 4.9430415776174e-01 1.8962748078601e-01
8.3865630873433e-01 4.9975107903818e-01 1.6685798732406e-01
4.8551391870533e-01 5.0519605148914e-01 1.4409135366293e-01
1.3298458773191e-01 5.1063825931174e-01 1.2132876726298e-01
7.8102436450985e-01 5.1607735694588e-01 9.8570833193778e-02
4.2957357911532e-01 5.2151333196198e-01 7.5817699320601e-02
7.8562134383926e-02 5.2694637919906e-01 5.3069195728736e-02
7.2791340650125e-01 5.3237680363713e-01 3.0324955827666e-02
3.7754665011248e-01 5.3780495508227e-01 7.5845058431839e-03
2.5758765254648e-02 5.4324018541314e-01 9.8483674088809e-01
6.7569743165775e-01 5.4866487737564e-01 9.6210224411941e-01
3.2565562308693e-01 5.5408832872668e-01 9.3936991024314e-01
-2.4463346128699e-02 5.5951088574041e-01 9.1663913505530e-01
6.2523558841396e-01 5.6493292593375e-01 8.9390924144393e-01
2.7463595896009e-01 5.7035489622637e-01 8.7117942382114e-01
-7.6394413996989e-02 5.7577736640157e-01 8.4844867082261e-01
5.7199139712816e-01 5.8120110158438e-01 8.2571566428178e-01
2.1961326442068e-01 5.8662715622875e-01 8.0297865727724e-01
-1.3374267924344e-01 5.9205698441377e-01 7.8023534983528e-01
5.1167038757235e-01 5.9749253719458e-01 7.5748282069804e-01
1.5555984073912e-01 6.0293625758594e-01 7.3471765862824e-01
-2.0239154809659e-01 6.0839076088870e-01 7.1193658146748e-01
4.3752080994853e-01 6.1385781522256e-01 6.8913796521867e-01
7.5118308236275e-02 6.1933623287574e-01 6.6632447333246e-01
-2.8952967647971e-01 6.2481908423893e-01 6.4350566598864e-01
3.4398026083169e-01 6.3029265460629e-01 6.2069727462642e-01
-2.3675747250227e-02 6.3574053340786e-01 5.9791439939922e-01
-3.9171404577175e-01 6.4115214291493e-01 5.7516250627365e-01
2.4065985503608e-01 6.4652904579420e-01 5.5243497124579e-01
-1.2576715689091e-01 6.5188339579484e-01 5.2971959366373e-01
-1.5972730450691e-01 4.3061629336556e-01 5.0783924513427e-01
1.4862544536294e+00 4.3597862012290e-01 4.8513478840099e-01
1.1317893903613e+00 4.4134101594159e-01 4.6242994371737e-01
7.7691732932823e-01 4.4670473365621e-01 4.3972445733956e-01
4.2169753089377e-01 4.5207140886655e-01 4.1701772703682e-01
1.0662052681093e+00 4.5744296485768e-01 3.9430873467818e-01
7.1052610864440e-01 4.6282141282545e-01 3.7159607712767e-01
3.5474987439812e-01 4.6820857473620e-01 3.4887812273759e-01
9.9896638017674e-01 4.7360578725334e-01 3.2615328097608e-01
6.4326393760799e-01 4.7901365791302e-01 3.0342032897853e-01
2.8772986205301e-01 4.8443193548054e-01 2.8067870943140e-01
9.3245093788219e-01 4.8985953034448e-01 2.5792871157480e-01
5.7751179908407e-01 4.9529468449290e-01 2.3517147540197e-01
2.2299046791965e-01 5.0073525107855e-01 2.1240881288589e-01
8.6895204643024e-01 5.0617901283136e-01 1.8964290064234e-01
5.1544267481779e-01 5.1162396139342e-01 1.6687593792812e-01
1.6248574491331e-01 5.1706848147944e-01 1.4410986249791e-01
8.1008125666931e-01 5.2251142292825e-01 1.2134617926147e-01
4.5820794554571e-01 5.2795207943511e-01 9.8585907906803e-02
1.0682705670735e-01 5.3339011073158e-01 7.5829620543138e-02
7.5588654647831e-01 5.3882544450961e-01 5.3077528289169e-02
4.0532481169041e-01 5.4425818357972e-01 3.0329581193808e-02
5.5073469774830e-02 5.4968853127015e-01 7.5855585196663e-03
7.0344618291295e-01 5.5512570268732e-01 9.8483459423124e-01
3.5358576910310e-01 5.6055207249047e-01 9.6209735630365e-01
3.8007241338663e-03 5.6597682231697e-01 9.3936294830819e-01
6.5400225624414e-01 5.7140021898436e-01 9.1663093401914e-01
3.0409343774149e-01 5.7682254039786e-01 8.9390083282607e-01
-4.6033848362543e-02 5.8224409970087e-01 8.7117208298002e-01
6.0349690779639e-01 5.8766528464807e-01 8.4844398048407e-01
2.5254124072871e-01 5.9308661764719e-01 8.2571558652429e-01
-9.9073694017672e-02 5.9850884515267e-01 8.0298559354329e-01
5.4844128736865e-01 6.0393306754209e-01 7.8025214057558e-01
1.9482554722193e-01 6.0936091767132e-01 7.5751258075360e-01
-1.6024425517458e-01 6.1479477364391e-01 7.3476325006759e-01
4.8283701152336e-01 6.2023791091632e-01 7.1199941602920e-01
1.2361283743310e-01 6.2569428168638e-01 6.8921587650173e-01
-2.3837380873580e-01 6.3116718216618e-01 6.6640912059689e-01
3.9657772564676e-01 6.3665574767921e-01 6.4358190329438e-01
2.8575066311866e-02 6.4214963991525e-01 6.2074845295510e-01
-3.4170486045879e-01 6.4762728592833e-01 5.9793282228447e-01
2.8683070079941e-01 6.5306539075173e-01 5.7515456211114e-01
-8.4611016755004e-02 6.5845538119412e-01 5.5241382823624e-01
-4.5482114111710e-01 6.6380960442587e-01 5.2969451212525e-01
5.0678989373805e-01 4.4254403084648e-01 5.0779299162374e-01
1.1532022056424e+00 4.4790674672743e-01 4.8508912026257e-01
7.9926146567382e-01 4.5326960633177e-01 4.6238489548798e-01
4.4499889202643e-01 4.5863365536507e-01 4.3968011033999e-01
1.0904602680174e+00 4.6400022880268e-01 4.1697429809464e-01
7.3570385487742e-01 4.6937089499552e-01 3.9426667315762e-01
3.8079655026933e-01 4.7474732644407e-01 3.7155613553810e-01
1.0258094658880e+00 4.8013111010537e-01 3.4884136056579e-01
6.7081435591530e-01 4.8552353083279e-01 3.2612097316522e-01
1.3158818487385e+00 4.9092537319777e-01 3.0339377862138e-01
9.6108141865598e-01 4.9633678573852e-01 2.8065899958612e-01
6.0648204147663e-01 5.0175723943114e-01 2.5791646016395e-01
2.5215207105165e-01 5.0718559258878e-01 2.3516666626543e-01
8.9815728343149e-01 5.1262025071882e-01 2.1241075683051e-01
5.4455703146482e-01 5.1805938672745e-01 1.8965033707153e-01
1.9139947252086e-01 5.2350117197319e-01 1.6688723969353e-01
8.3871731641098e-01 5.2894396939659e-01 1.4412327789367e-01
4.8652528746000e-01 5.3438645680294e-01 1.2136004711414e-01
1.3481975558573e-01 5.3982767305758e-01 9.8598806854068e-02
7.8358023118909e-01 5.4526700099471e-01 7.5840444084247e-02
4.3277197713446e-01 5.5070411116790e-01 5.3085498958635e-02
8.2348930252375e-02 5.5613889025629e-01 3.0334226074217e-02
7.3225630925086e-01 5.6157137149675e-01 7.5866674403557e-03
3.8082796285852e-01 5.6701059973455e-01 9.8483222473916e-01
3.1199895290679e-02 5.7243894463937e-01 9.6209164179342e-01
6.8169977501271e-01 5.7786545363273e-01 9.3935424943209e-01
3.3224864243644e-01 5.8329031655045e-01 9.1661977566610e-01
-1.7240216676316e-02 5.8871372455122e-01 8.9388792308728e-01
6.3313615038177e-01 5.9413587879751e-01 8.7115835548966e-01
2.8326657812411e-01 5.9955701163000e-01 8.4843066374623e-01
-6.6979569294960e-02 6.0497742429339e-01 8.2570430398322e-01
5.8224011365414e-01 6.1039754949402e-01 8.0297849531732e-01
2.3073060773877e-01 6.1581805347818e-01 7.8025205639069e-01
-1.2175512205909e-01 6.2124000135191e-01 7.5752315078728e-01
5.2446387440194e-01 6.2666511857735e-01 7.3478890861406e-01
1.6897067474286e-01 6.3209617602055e-01 7.1204492400241e-01
-1.8877720613461e-01 6.3753744698243e-01 6.8928478715679e-01
4.5054236733591e-01 6.4299485184049e-01 6.6650032397580e-01
8.6185217479360e-02 6.4847445081361e-01 6.4368439468617e-01
-2.8239022447328e-01 6.5397650735897e-01 6.2083907414550e-01
3.4498949631911e-01 6.5948455850245e-01 5.9798686982798e-01
-3.0455667678023e-02 6.6496254369918e-01 5.7516639558834e-01
-4.0685876258078e-01 6.7037890352077e-01 5.5239996248607e-01
2.1775565603159e-01 6.7573711537591e-01 5.2967292240266e-01
1.7333712646395e-01 4.5447232375910e-01 5.0774573491879e-01
8.2008642387367e-01 4.5983531623829e-01 4.8504233498651e-01
4.6656779765849e-01 4.6519851379738e-01 4.6233863028941e-01
1.1128059586682e+00 4.7056280880825e-01 4.3963444638014e-01
7.5883640170140e-01 4.7592931019867e-01 4.1692941271305e-01
4.0470420858444e-01 4.8129931234724e-01 3.9422291151094e-01
1.0504615220742e+00 4.8667421189005e-01 3.7151406901781e-01
6.9616438638287e-01 4.9205537796149e-01 3.4880180549909e-01
3.4186990942016e-01 4.9744399519916e-01 3.2608494762217e-01
9.8763453036292e-01 5.0284090823077e-01 3.0336238979694e-01
6.3351361389100e-01 5.0824649827694e-01 2.8063327533947e-01
2.7956190954947e-01 5.1366061719744e-01 2.5789715907628e-01
9.2583395776711e-01 5.1908259381050e-01 2.3515411329702e-01
5.7238352495451e-01 5.2451131352898e-01 2.1240474974555e-01
2.1926160924648e-01 5.2994535716296e-01 1.8965015015509e-01
8.6651324472997e-01 5.3538317145909e-01 1.6689172166197e-01
5.1417389935114e-01 5.4082323732370e-01 1.4413101294027e-01
1.6226643994128e-01 5.4626420507613e-01 1.2136953412505e-01
8.1079938614281e-01 5.5170497838953e-01 9.8608616086315e-02
4.5976668141495e-01 5.5714474428743e-01 7.5849327241273e-02
1.0914873993899e-01 5.6258295926985e-01 5.3092447543093e-02
7.5891426012187e-01 5.6801930775734e-01 3.0338486542748e-02
4.0902225401481e-01 5.7345364883077e-01 7.5877277262981e-03
5.7829393615586e-02 5.7889483511832e-01 9.8482984670093e-01
7.0846232810974e-01 5.8432523976694e-01 9.6208563849294e-01
3.5927107856287e-01 5.8975375937726e-01 9.3934466802920e-01
1.0188020807214e-02 5.9518050933091e-01 9.1660681241649e-01
6.6113838021687e-01 6.0060560822454e-01 8.9387192783767e-01
3.1203828566981e-01 6.0602917328566e-01 8.7113985701119e-01
-3.7208333756537e-02 6.1145132510554e-01 8.4841042209740e-01
6.1328562912399e-01 6.1687220337253e-01 8.2568339900905e-01
2.6338371709081e-01 6.2229199830483e-01 8.0295846431481e-01
-8.7083929103718e-02 6.2771100785550e-01 7.8023509745601e-01
5.6166493906725e-01 6.3312974059087e-01 7.5751240618480e-01
2.0934308501284e-01 6.3854910210679e-01 7.3478881824805e-01
-1.4443859496965e-01 6.4397073368435e-01 7.1206154582935e-01
4.9978100984993e-01 6.4939761315374e-01 6.8932570001379e-01
1.4124738156709e-01 6.5483502327156e-01 6.6657302786238e-01
-2.2107568859684e-01 6.6029163172291e-01 6.4379094840514e-01
4.1153054241650e-01 6.6577857092535e-01 6.2096524335784e-01
3.7994737937562e-02 6.7129942575413e-01 5.9809485387214e-01
-3.4138169058405e-01 6.7682552301238e-01 5.7521842917691e-01
2.7582384305559e-01 6.8229247901246e-01 5.5240083527804e-01
-1.0693528504660e-01 6.8766677471339e-01 5.2965547794441e-01
-1.6008810050089e-01 4.6640104915364e-01 5.0769770442830e-01
4.8691308124499e-01 4.7176423177202e-01 4.8499466388348e-01
1.1337253676450e+00 4.7712767115745e-01 4.6229137888304e-01
7.8036844902348e-01 4.8249214491473e-01 4.3958769922555e-01
4.2686977240470e-01 4.8785858922774e-01 4.1688332501006e-01
1.0732639703702e+00 4.9322808425460e-01 3.9417776212254e-01
7.1959128336644e-01 4.9860180188802e-01 3.7147030706680e-01
3.6589538154555e-01 5.0398091765241e-01 3.4876007317961e-01
1.0122212145280e+00 5.0936649780621e-01 3.2604606290523e-01
6.5861348894250e-01 5.1475937990745e-01 3.0332728016985e-01
3.0511607775675e-01 5.2016006795302e-01 2.8060286609952e-01
9.5177222548146e-01 5.2556866135167e-01 2.5787223357202e-01
5.9862503021196e-01 5.3098483145846e-01 2.3513517358667e-01
2.4571753299474e-01 5.3640785141118e-01 2.1239191020025e-01
8.9309189596894e-01 5.4183667552919e-01 1.8964309071943e-01
5.4078753547946e-01 5.4727005489799e-01 1.6688971227630e-01
1.8883852006997e-01 5.5270666833985e-01 1.4413300102908e-01
8.3727084572695e-01 5.5814524552143e-01 1.2137427087747e-01
4.8610023878679e-01 5.6358466281252e-01 9.8614790780628e-02
1.3533092594740e-01 5.6902400128055e-01 7.5855683197459e-02
7.8495548019257e-01 5.7446256632815e-01 5.3097864406442e-02
4.3495555484187e-01 5.7989987633168e-01 3.0342025665692e-02
8.5303149827012e-02 5.8533563135671e-01 7.5886461885088e-03
7.3437953255024e-01 5.9077848342081e-01 9.8482766140246e-01
3.8529873598507e-01 5.9621082522916e-01 9.6207987849837e-01
3.6435700876038e-02 6.0164138840296e-01 9.3933509455569e-01
6.8773507238332e-01 6.0707021497261e-01 9.1659331562956e-01
3.3913514309371e-01 6.1249736134208e-01 8.9385451774784e-01
-9.4333852421171e-03 6.1792288453852e-01 8.7111866816966e-01
6.4195035836442e-01 6.2334683529971e-01 8.4838573792140e-01
2.9319385491734e-01 6.2876925794098e-01 8.2565570447611e-01
-5.5814219194074e-02 6.3419019825800e-01 8.0292854129483e-01
5.9478783584107e-01 6.3960972338523e-01 7.8020418631185e-01
2.4482228996676e-01 6.4502796295267e-01 7.5748247191189e-01
-1.0594736035599e-01 6.5044519241469e-01 7.3476297914558e-01
5.4215282993582e-01 6.5586200482650e-01 7.1204473731936e-01
1.8865702736395e-01 6.6127967409372e-01 6.8932560371502e-01
-1.6712400946094e-01 6.6670093537579e-01 6.6660099285696e-01
4.7375841761611e-01 6.7213162843346e-01 6.4386140498173e-01
1.0968321805655e-01 6.7758370525559e-01 6.2108853676742e-01
-2.6169482572640e-01 6.8307753581992e-01 5.9825439088860e-01
3.5725169852118e-01 6.8862635830037e-01 5.7534688738490e-01
-3.2874599508101e-02 6.9417329040037e-01 5.5244047957912e-01
-4.2655322851517e-01 6.9959800681434e-01 5.2964480068742e-01
-4.9348749479333e-01 4.7833008879115e-01 5.0764912498046e-01
1.1536863931016e+00 4.8369339365572e-01 4.8494633343250e-01
8.0074636260431e-01 4.8905700149263e-01 4.6224336697795e-01
4.4770815090442e-01 4.9442160331020e-01 4.3954009547285e-01
1.0945927366166e+00 4.9978800098627e-01 4.1683627337449e-01
7.4142624304589e-01 5.0515710445891e-01 3.9413150020081e-01
3.8823907438157e-01 5.1052990114002e-01 3.7142520234973e-01
1.0350645106589e+00 5.1590739738747e-01 3.4871664496780e-01
6.8193716914099e-01 5.2129053828424e-01 3.2600497821070e-01
3.2889177426767e-01 5.2668011738762e-01 3.0328931558040e-01
9.7596253448288e-01 5.3207669098676e-01 2.8056883473407e-01
6.2318315365110e-01 5.3748051122071e-01 2.5784288520186e-01
2.7058721949203e-01 5.4289148960685e-01 2.3511108435671e-01
9.1820853102780e-01 5.4830919789960e-01 2.1237338360043e-01
5.6608092620259e-01 5.5373290736427e-01 1.8963009132102e-01
2.1423735031216e-01 5.5916166109789e-01 1.6688184729187e-01
8.6270819232670e-01 5.6459436799573e-01 1.4412955329154e-01
5.1151919139729e-01 5.7002990296680e-01 1.2137427411333e-01
1.6068936871000e-01 5.7546719756061e-01 9.8617128745364e-02
8.1022941902430e-01 5.8090530865746e-01 7.5859191457802e-02
4.6014083140785e-01 5.8634345905651e-01 5.3101417266565e-02
1.1041578564800e-01 5.9178105040446e-01 3.0344598240101e-02
7.6103767936222e-01 5.9721765386273e-01 7.5893494698804e-03
4.1041334493633e-01 6.0266172532699e-01 9.8482583749250e-01
6.1640185123575e-02 6.0809570188919e-01 9.6207483128996e-01
7.1311993210426e-01 6.1352813990532e-01 9.3932635206410e-01
3.6480975939625e-01 6.1895900822064e-01 9.1658051426005e-01
1.6661437006608e-02 6.2438830595683e-01 8.9383738483150e-01
6.6862079214617e-01 6.2981604401062e-01 8.7109700998185e-01
3.2062640534942e-01 6.3524223170630e-01 8.4835943732364e-01
-2.7392701482932e-02 6.4066686799235e-01 8.2562473365982e-01
6.2447921000104e-01 6.4608993669076e-01 8.0289299858834e-01
2.7613876373223e-01 6.5151140593633e-01 7.8016437320585e-01
-7.2545674715938e-02 6.5693123346864e-01 7.5743904053563e-01
5.7825139389707e-01 6.6234938302592e-01 7.3471720767831e-01
2.2828857703888e-01 6.6776586599884e-01 7.1199904301571e-01
-1.2278340326465e-01 6.7318084602247e-01 6.8928449843705e-01
5.2450525824919e-01 6.7859491000410e-01 6.6657282956386e-01
1.6930691761368e-01 6.8400980270516e-01 6.4386130300064e-01
-1.8981126209119e-01 6.8943050214984e-01 6.2114170105678e-01
4.4459707321928e-01 6.9487105257099e-01 5.9839134959094e-01
6.7954936052851e-02 7.0036721098648e-01 5.7555734292602e-01
-3.2587092784707e-01 7.0596183605634e-01 5.5258935911006e-01
2.6470781779409e-01 7.1151742462770e-01 5.2965626423590e-01
1.7313788251703e-01 4.9025932834465e-01 5.0760021774691e-01
8.2040916441152e-01 4.9562269990950e-01 4.8489756595079e-01
4.6763918976048e-01 5.0098641983572e-01 4.6219481562426e-01
1.1148405115176e+00 5.0635111350061e-01 4.3949185538769e-01
7.6202873977104e-01 5.1171747630406e-01 4.1678848416697e-01
4.0922284270238e-01 5.1708628016580e-01 3.9408437495107e-01
1.0564448543968e+00 5.2245835892859e-01 3.7137905519018e-01
7.0371909832477e-01 5.2783457147451e-01 3.4867190994537e-01
3.5107118506991e-01 5.3321574597596e-01 3.2596221126276e-01
9.9852710367283e-01 5.3860261280636e-01 3.0324917306797e-01
6.4611266753360e-01 5.4399573620305e-01 2.8053202602292e-01
2.9385341651576e-01 5.4939545532862e-01 2.5781010243124e-01
9.4177487772608e-01 5.5480184406944e-01 2.3508291845148e-01
5.8990292547086e-01 5.6021469622838e-01 2.1235024023872e-01
2.3826391768151e-01 5.6563353916387e-01 1.8961212248434e-01
8.8688434527497e-01 5.7105767472557e-01 1.6686891212828e-01
5.3578989045779e-01 5.7648624197964e-01 1.4412121624100e-01
1.8500399003393e-01 5.8191829250093e-01 1.2136983996486e-01
8.3454616503259e-01 5.8735286698559e-01 9.8615706130563e-02
4.8443044896530e-01 5.9278906238384e-01 7.5859770962320e-02
1.3466420597738e-01 5.9822608163170e-01 5.3102949353079e-02
7.8524750586127e-01 6.0366326240510e-01 3.0346059136784e-02
4.3617307133712e-01 6.0910008569430e-01 7.5897882957954e-03
8.5873645545703e-02 6.1454482568481e-01 9.8482449850385e-01
7.3742531276163e-01 6.1997999706569e-01 9.6207086303032e-01
3.8925778597777e-01 6.2541397567616e-01 9.3931911546350e-01
4.1340327608386e-02 6.3084665810398e-01 9.1656945987862e-01
6.9363783490341e-01 6.3627798698607e-01 8.9382203405491e-01
3.4611090962306e-01 6.4170793150730e-01 8.7107693468057e-01
-1.2845580937484e-03 6.4713647173482e-01 8.4833424053323e-01
6.5140155258353e-01 6.5256358653453e-01 8.2559403358685e-01
3.0411165178855e-01 6.5798924439384e-01 8.0285641890424e-01
-4.3222797461877e-02 6.6341339626022e-01 7.8012154490157e-01
6.0931346210518e-01 6.6883596943338e-01 7.5738962622356e-01
2.6161087519236e-01 6.7425686164012e-01 7.3466097205681e-01
-8.6479390103108e-02 6.7967593495018e-01 7.1193602282881e-01
5.6482926000243e-01 6.8509301129908e-01 6.8921539589076e-01
2.1521224872631e-01 6.9050787943491e-01 6.6649992773240e-01
-1.3585769463899e-01 6.9592035471625e-01 6.4379064263893e-01
5.1069488052516e-01 7.0133056649030e-01 6.2108832735639e-01
1.5309198729163e-01 7.0674027434411e-01 5.9839124218008e-01
-2.1250024924862e-01 7.1215934183857e-01 5.7568385539413e-01
4.0454522801270e-01 7.1763874665223e-01 5.5288520730502e-01
-1.5851221655287e-02 7.2332026954937e-01 5.2980362746850e-01
-1.6021246696645e-01 5.0218865654800e-01 5.0755120154698e-01
4.8708300819562e-01 5.0755204619238e-01 4.8484858071631e-01
1.1344091357837e+00 5.1291583393871e-01 4.6214594213712e-01
7.8177583056972e-01 5.1828059546073e-01 4.3944319406436e-01
4.2919406797723e-01 5.2364693975511e-01 4.1674017458185e-01
1.0766765603244e+00 5.2901552393752e-01 3.9403661701851e-01
7.2423796143189e-01 5.3438705135311e-01 3.7133212952621e-01
3.7189462691440e-01 5.3976225121669e-01 3.4862619289042e-01
1.0196640871455e+00 5.4514184155392e-01 3.2591817898145e-01
6.6756446196989e-01 5.5052648041048e-01 3.0320738965136e-01
3.1561403582107e-01 5.5591671248181e-01 2.8049311272126e-01
9.6383112336983e-01 5.6131291913593e-01 2.5777468864668e-01
6.1223422226552e-01 5.6671527928768e-01 2.3505157916801e-01
2.6084231961122e-01 5.7212374703097e-01 2.1232342823789e-01
9.0967513835444e-01 5.7753804967667e-01 1.8959010603880e-01
5.5875310614256e-01 5.8295770710905e-01 1.6685172904877e-01
2.0809690318442e-01 5.8838207037065e-01 1.4410865273791e-01
8.5772657163956e-01 5.9381037447022e-01 1.2136143807112e-01
5.0766030331786e-01 5.9924179817067e-01 9.8610797598832e-02
1.5791311881399e-01 6.0467552259891e-01 7.5857530339596e-02
8.0849567902733e-01 6.1011078129169e-01 5.3102455916504e-02
4.5941342634199e-01 6.1554689656221e-01 3.0346357223711e-02
1.1066616103729e-01 6.2098330013240e-01 7.5899378402461e-03
7.6071250072425e-01 6.2642810463500e-01 9.8482371841373e-01
4.1260201642567e-01 6.3186394222526e-01 9.6206821632756e-01
6.4793018783885e-02 6.3729900835795e-01 9.3931386411323e-01
7.1726579847490e-01 6.4273313093065e-01 9.1656093873427e-01
3.6999733956884e-01 6.4816619569645e-01 8.9380963693662e-01
2.2961847116021e-02 6.5359812848280e-01 8.7106009445586e-01
6.7613106221811e-01 6.5902887989366e-01 8.4831240490336e-01
3.2947429040072e-01 6.6445841293232e-01 8.2556663730934e-01
-1.7041946739303e-02 6.6988669341925e-01 8.0282285221396e-01
6.3654522421965e-01 6.7531368270027e-01 7.8008111699448e-01
2.9019340534214e-01 6.8073933186762e-01 7.5734152192717e-01
-5.6147902705716e-02 6.8616357644341e-01 7.3460419957106e-01
5.9745780394738e-01 6.9158633003951e-01 7.1186935193264e-01
2.5092534789191e-01 6.9700747461983e-01 6.8913729359943e-01
-9.5868281353920e-02 7.0242684300803e-01 6.6640852711599e-01
5.5688396241830e-01 7.0784418463420e-01 6.4368388566607e-01
2.0884942449074e-01 7.1325909411229e-01 6.2096482489036e-01
-1.4061490427315e-01 7.1867085409388e-01 5.9825406883352e-01
5.0704515214180e-01 7.2407809369003e-01 5.7555712290969e-01
1.4776419244970e-01 7.2947855508740e-01 5.5288509471906e-01
-2.3519086713471e-01 7.3488738408266e-01 5.3022759721315e-01
-4.9353851831456e-01 5.1411796433270e-01 5.0750229438905e-01
1.1537084659255e+00 5.1948132535965e-01 4.8479959540621e-01
8.0105876329548e-01 5.2484514434772e-01 4.6209696142320e-01
4.4852001847984e-01 5.3020995966880e-01 4.3939432292472e-01
1.0960990058506e+00 5.3557630815389e-01 4.1669155525252e-01
7.4380270425012e-01 5.4094474830027e-01 3.9398844404986e-01
3.9163897725763e-01 5.4631586946298e-01 3.7128466407664e-01
1.0396168231395e+00 5.5169028484577e-01 3.4857977374258e-01
6.8774628131965e-01 5.5706860913645e-01 3.2587322660491e-01
3.3603816157394e-01 5.6245142424453e-01 3.0316439951988e-01
9.8450377947695e-01 5.6783923842979e-01 2.8045263518405e-01
6.3315483706970e-01 5.7323244495305e-01 2.5773729477377e-01
2.8200350025843e-01 5.7863128624970e-01 2.3501781470386e-01
9.3106262531598e-01 5.8403582871358e-01 2.1229376053402e-01
5.8034600885815e-01 5.8944595172456e-01 1.8956487098317e-01
2.2986850311499e-01 5.9486135274061e-01 1.6683108598483e-01
8.7964586045926e-01 6.0028156822617e-01 1.4409255473078e-01
5.2969424016265e-01 6.0570600805768e-01 1.2134962252861e-01
1.8002940379934e-01 6.1113399912307e-01 9.8602798598612e-02
8.3066571348594e-01 6.1656483252585e-01 7.5852709962710e-02
4.8161510167293e-01 6.2199780849502e-01 5.3100048573500e-02
1.3288618758421e-01 6.2743227393118e-01 3.0345519269554e-02
7.8448367650955e-01 6.3286764926858e-01 7.5897949855348e-03
4.3489188683315e-01 6.3831190726603e-01 9.8482352436336e-01
8.7128386894936e-02 6.4374784152746e-01 9.6206700994879e-01
7.3968030649494e-01 6.4918347200868e-01 9.3931086972162e-01
3.9253744149640e-01 6.5461856533406e-01 9.1655543800187e-01
4.5687299970211e-02 6.6005295169611e-01 8.9380096364388e-01
6.9911591066960e-01 6.6548651050523e-01 8.7104762137566e-01
3.5280864787430e-01 6.7091915704110e-01 8.4829552425891e-01
6.7510594879275e-03 6.7635083113013e-01 8.2554473543387e-01
6.6092971787575e-01 6.8178148833017e-01 8.0279527792199e-01
3.1533314993342e-01 6.8721109372879e-01 7.8004714169729e-01
-3.0047047370480e-02 6.9263961825639e-01 7.5730028742817e-01
6.2478528084045e-01 6.9806703737111e-01 7.3455464616130e-01
2.7983304383374e-01 7.0349333209391e-01 7.1181011361969e-01
-6.4890549164113e-02 7.0891849272069e-01 6.8906653634529e-01
5.9064421328140e-01 7.1434252631255e-01 6.6632368367518e-01
2.4649529911292e-01 7.1976547083858e-01 6.4358119190583e-01
-9.7226694632699e-02 7.2518742332110e-01 6.2083844733080e-01
5.5969795111033e-01 7.3060860218039e-01 5.9809431770537e-01
2.1775014978426e-01 7.3602950731701e-01 5.7534644770764e-01
-1.2181851444593e-01 7.4145142349375e-01 5.5258902153007e-01
5.4547003790361e-01 7.4687858201124e-01 5.2980339735834e-01
1.7316027672956e-01 5.2604714401693e-01 5.0745371509244e-01
8.2028504598804e-01 5.3141042675024e-01 4.8475082769765e-01
4.6758809722317e-01 5.3677424396073e-01 4.6204808755799e-01
1.1150750170605e+00 5.4213910667281e-01 4.3934545148265e-01
7.6274854134726e-01 5.4750548920595e-01 4.1664283284185e-01
4.1061009754512e-01 5.5287386190585e-01 3.9394006534715e-01
1.0586609877734e+00 5.5824471105286e-01 3.7123688086676e-01
7.0690312080848e-01 5.6361854343071e-01 3.4853290199497e-01
3.5533933119418e-01 5.6899587596454e-01 3.2582764934149e-01
1.0039734121000e+00 5.7437721312454e-01 3.0312056271189e-01
6.5281002311086e-01 5.7976301631895e-01 2.8041103431755e-01
3.0185461605515e-01 5.8515367023258e-01 2.5769845095958e-01
9.5111346466334e-01 5.9054945106635e-01 2.3498224100631e-01
6.0059380876911e-01 5.9595050104970e-01 2.1226192118234e-01
2.5030405458405e-01 6.0135681261005e-01 1.8953713790538e-01
9.0025392895156e-01 6.0676822433750e-01 1.6680769826810e-01
5.5045447854177e-01 6.1218442946543e-01 1.4407358682007e-01
2.0091783511872e-01 6.1760499606794e-01 1.2133496602355e-01
8.5165672533988e-01 6.2302939668153e-01 9.8592160456989e-02
5.0268377047047e-01 6.2845704381062e-01 7.5845627164493e-02
1.5401067787127e-01 6.3388732705093e-01 5.3095916508732e-02
8.0564745610627e-01 6.3931964757112e-01 3.0343629009825e-02
4.5760178242856e-01 6.4475344645980e-01 7.5893736757353e-03
1.0838356687597e-01 6.5019657639655e-01 9.8482390443400e-01
7.6097279897106e-01 6.5563203782344e-01 9.6206725405100e-01
4.1388545414826e-01 6.6106768342602e-01 9.3931021375598e-01
6.7119045318811e-02 6.6650322998514e-01 9.1655316001889e-01
7.2066973525627e-01 6.7193845703202e-01 8.9379638824667e-01
3.7453325974706e-01 6.7737319679776e-01 8.7104011631510e-01
2.8706031232259e-02 6.8280732372442e-01 8.4828449056309e-01
6.8318644694069e-01 6.8824074486475e-01 8.2552959217358e-01
3.3797647968774e-01 6.9367339204353e-01 8.0277544220453e-01
-6.9162923420837e-03 6.9910521629031e-01 7.8002200378873e-01
6.4852406110923e-01 7.0453618483160e-01 7.5726918005478e-01
3.0432577000634e-01 7.0996628087724e-01 7.3451680600057e-01
-3.9464627905749e-02 7.1539550659070e-01 7.1176463169131e-01
6.1722736897553e-01 7.2082389008562e-01 6.8901229245732e-01
2.7452061644509e-01 7.2625149827098e-01 6.6625925875095e-01
-6.7392171975990e-02 7.3167845947398e-01 6.4350475347138e-01
5.9181237439651e-01 7.3710500475569e-01 6.2074761885710e-01
2.5270682147086e-01 7.4253155101072e-01 5.9798612043711e-01
-8.3613527475539e-02 7.4795890290333e-01 5.7521777054955e-01
5.8518743177635e-01 7.5338895514774e-01 5.5243991753873e-01
2.6491265004435e-01 7.5882890082918e-01 5.2965580437167e-01
-1.6011497089870e-01 5.3797608862739e-01 5.0740568482878e-01
4.8681118522447e-01 5.4333923529854e-01 4.8470249686860e-01
1.1339946248022e+00 5.4870301719629e-01 4.6199953544817e-01
7.8143893434902e-01 5.5406792628230e-01 4.3929678925204e-01
4.2914269938790e-01 5.5943438018149e-01 4.1659421268095e-01
1.0771016377079e+00 5.6480276662407e-01 3.9389168611533e-01
7.2531036225950e-01 5.7017347479858e-01 3.7118899234998e-01
3.7376362556403e-01 5.7554691078731e-01 3.4848580814369e-01
1.0224570514809e+00 5.8092349746320e-01 3.2578170934444e-01
6.7138746310982e-01 5.8630366139123e-01 3.0307618792033e-01
3.2055296203930e-01 5.9168781053608e-01 2.8036867898604e-01
9.6995291064663e-01 5.9707630710544e-01 2.5765859544129e-01
6.1958792918556e-01 6.0246943981299e-01 2.3494536723130e-01
2.6945996016813e-01 6.0786739937993e-01 2.1222848169141e-01
9.1957239193704e-01 6.1327026035164e-01 1.8950752112621e-01
5.6993018672365e-01 6.1867797139581e-01 1.6678219382888e-01
2.2053993615264e-01 6.2409035523469e-01 1.4405235524778e-01
8.7140977266833e-01 6.2950711827864e-01 1.2131801697844e-01
5.2254909532422e-01 6.3492786891943e-01 9.8579342645993e-02
1.7396811341838e-01 6.4035214241120e-01 7.5836631382144e-02
8.2567725858249e-01 6.4577942947889e-01 5.3090291213263e-02
4.7768655212990e-01 6.5120920542349e-01 3.0340805927967e-02
1.3000503066534e-01 6.5664095664996e-01 7.5886995975823e-03
7.8116828870780e-01 6.6208243147504e-01 9.8482481794255e-01
4.3411324285364e-01 6.6751688165446e-01 9.6206887453926e-01
8.7384728143405e-02 6.7295200240012e-01 9.3931182398946e-01
7.4098611268047e-01 6.7838747530933e-01 9.1655406962051e-01
3.9492024844949e-01 6.8382303818846e-01 8.9379594649906e-01
4.9190379518632e-02 6.8925847950746e-01 8.7103771833112e-01
7.0380132826064e-01 6.9469363126801e-01 8.4827957791648e-01
3.5876100178535e-01 7.0012836164035e-01 8.2552164938648e-01
1.4082315732442e-02 7.0556256840808e-01 8.0276399062144e-01
6.6978571666394e-01 7.1099617394939e-01 7.8000659450030e-01
3.2590261592158e-01 7.1642912224811e-01 7.5724938790277e-01
-1.7519735696192e-02 7.2186137832561e-01 7.3449222765826e-01
6.3958897216437e-01 7.2729293058223e-01 7.1173489304979e-01
2.9733326249762e-01 7.3272379698882e-01 6.8897707547404e-01
-4.4130152839573e-02 7.3815403730236e-01 6.6621836867227e-01
6.1543792346440e-01 7.4358377683633e-01 6.4345827104350e-01
2.7641278575137e-01 7.4901325738655e-01 6.2069623466390e-01
-6.0593983109509e-02 7.5444296501522e-01 5.9793186091232e-01
6.0546455695305e-01 7.5987401440808e-01 5.7516551907724e-01
2.7649240123459e-01 7.6530951397724e-01 5.5240004966673e-01
-4.3823550873067e-02 7.7075988968949e-01 5.2964411178132e-01
-4.9336250368752e-01 5.4990469144618e-01 5.0735842842627e-01
1.1532841285362e+00 5.5526763053854e-01 4.8465482523630e-01
8.0027310922472e-01 5.6063133885863e-01 4.6195152240895e-01
4.4760590977982e-01 5.6599629646859e-01 4.3924854765479e-01
1.0952765418593e+00 5.7136286658351e-01 4.1654590143271e-01
7.4327447707081e-01 5.7673135525168e-01 3.9384351155938e-01
3.9158712562444e-01 5.8210205597141e-01 3.7114120785540e-01
1.0402016632637e+00 5.8747527614337e-01 3.4843871353687e-01
6.8910619598657e-01 5.9285134615786e-01 3.2573564990263e-01
3.3829037724034e-01 5.9823061410579e-01 3.0303155147379e-01
9.8774564736064e-01 6.0361343007990e-01 2.8032588930788e-01
6.3746526659625e-01 6.0900012424896e-01 2.5761810042387e-01
2.8744427946752e-01 6.1439098264917e-01 2.3490762115782e-01
9.3767949634668e-01 6.1978622413145e-01 2.1219392198705e-01
5.8816952273689e-01 6.2518598124945e-01 1.8947654110055e-01
2.3891481968997e-01 6.3059028714663e-01 1.6675511390159e-01
8.8991774849018e-01 6.3599906973760e-01 1.4402939580378e-01
5.4118254323708e-01 6.4141215368947e-01 1.2129927617771e-01
1.9271516585602e-01 6.4682926990033e-01 9.8564782125662e-02
8.4452302470701e-01 6.5225007138004e-01 7.5826071842866e-02
4.9661457249539e-01 6.5767415374351e-01 5.3083418524679e-02
1.4899883234681e-01 6.6310107804648e-01 3.0337186888305e-02
8.0168492442423e-01 6.6853039353452e-01 7.5878051706777e-03
4.5323449465893e-01 6.7396975479667e-01 9.8482620598751e-01
1.0653608784631e-01 6.7940270683301e-01 9.6207174023744e-01
7.6016353890161e-01 6.8483679703669e-01 9.3931551900297e-01
4.1412428949045e-01 6.9027168837655e-01 9.1655795723659e-01
6.8425987137680e-02 6.9570708924032e-01 8.9379942035168e-01
7.2307729526870e-01 7.0114275209159e-01 8.7104021612432e-01
3.7808902245441e-01 7.0657846986112e-01 8.4828059077950e-01
3.3475619944250e-02 7.1201407126538e-01 8.2552072810324e-01
6.8925714125612e-01 7.1744941610108e-01 8.0276075047135e-01
3.4546182880033e-01 7.2288439137019e-01 7.8000072095632e-01
2.1296034425743e-03 7.2831890893454e-01 7.5724064615685e-01
6.5931690918477e-01 7.3375290538087e-01 7.3448048016774e-01
3.1710365502578e-01 7.3918634505238e-01 7.1172013143082e-01
-2.4396509605853e-02 7.4461922810548e-01 6.8895947667863e-01
6.3497952311978e-01 7.5005160782723e-01 6.6619839107099e-01
2.9546917955324e-01 7.5548362759635e-01 6.4343681342844e-01
-4.2575111241534e-02 7.6091560429083e-01 6.2067488444737e-01
6.2138295249661e-01 7.6634823048222e-01 5.9791322764392e-01
2.8818346975547e-01 7.7178309577021e-01 5.7515346913830e-01
-4.0820443430381e-02 7.7722406189659e-01 5.5239895454927e-01
6.3656235979951e-01 7.8268054868834e-01 5.2965456106485e-01
1.7342022242261e-01 5.6183284588553e-01 5.0731217529375e-01
8.1969971045844e-01 5.6719548553546e-01 4.8460803927979e-01
4.6641519313817e-01 5.7255907273616e-01 4.6190426949049e-01
1.1135656935070e+00 5.7792408203396e-01 4.3920094178528e-01
7.6113983399455e-01 5.8329082083561e-01 4.1649810973060e-01
4.0911984240272e-01 5.8865950970636e-01 3.9379575098143e-01
1.0574849584068e+00 5.9403034331367e-01 3.7109373983649e-01
7.0621395785151e-01 5.9940352879465e-01 3.4839183952270e-01
3.5528678960249e-01 6.0477930340829e-01 3.2568970816835e-01
1.0046854766557e+00 6.1015793563076e-01 3.0298691404565e-01
6.5439449295286e-01 6.1553971444192e-01 2.8028295719551e-01
3.0440082398908e-01 6.2092493129560e-01 2.5757729549595e-01
9.5469388112128e-01 6.2631385867522e-01 2.3486937359750e-01
6.0526538621607e-01 6.3170672844940e-01 2.1215865312233e-01
2.5610928815249e-01 6.3710371255929e-01 1.8944464233647e-01
9.0722172475090e-01 6.4250490792034e-01 1.6672692333636e-01
5.5860100888629e-01 6.4791032680968e-01 1.4400517476483e-01
2.1024759905480e-01 6.5331989341999e-01 1.2127918825868e-01
8.6216401292878e-01 6.5873344667564e-01 9.8548877260674e-02
5.1435465519705e-01 6.6415074882423e-01 7.5814277491083e-02
1.6682555363964e-01 6.6957149876692e-01 5.3075539201290e-02
8.1958402378084e-01 6.7499534863577e-01 3.0332912205798e-02
4.7263830617617e-01 6.8042192184754e-01 7.5867254538383e-03
1.2457675892654e-01 6.8585878474028e-01 9.8482800065179e-01
7.7823450967213e-01 6.9128981406461e-01 9.6207568802243e-01
4.3221474622352e-01 6.9672241756948e-01 9.3932105150151e-01
8.6527385573495e-02 7.0215625660584e-01 9.1656450298288e-01
7.4118316054715e-01 7.0759102455039e-01 8.9380642669389e-01
3.9619429143496e-01 7.1302644899953e-01 8.7104716777321e-01
5.1575476735555e-02 7.1846229152976e-01 8.4828702509412e-01
7.0734525367672e-01 7.2389834595856e-01 8.2552624785341e-01
3.6352781052599e-01 7.2933443606368e-01 8.0276503539625e-01
2.0155385072367e-02 7.3477041367941e-01 7.8000353995835e-01
6.7727146542899e-01 7.4020615807975e-01 7.5724187249715e-01
3.3493513564964e-01 7.4564157771290e-01 7.3448011262138e-01
-6.7728934406192e-03 7.5107661589260e-01 7.1171832497856e-01
6.5225828872616e-01 7.5651126341821e-01 6.8895658661970e-01
3.1218228021133e-01 7.6194558425196e-01 6.6619503323448e-01
-2.6786010656285e-02 7.6737976750756e-01 6.4343393690988e-01
6.3565798456959e-01 7.7281423499975e-01 6.2067383257271e-01
2.9995220210025e-01 7.7824986859845e-01 5.9791570516675e-01
-3.3265328132662e-02 7.8368849053816e-01 5.7516119861690e-01
6.3693749840719e-01 7.8913381442548e-01 5.5241259957546e-01
3.1187637448877e-01 7.9459286420029e-01 5.2967177897441e-01
-1.5976324542754e-01 5.7376044580596e-01 5.0726715984338e-01
4.8605200796477e-01 5.7912266573337e-01 4.8456237030978e-01
1.1324087394559e+00 5.8448606991046e-01 4.6185800240195e-01
7.7930288819582e-01 5.8985113307817e-01 4.3915419190924e-01
4.2671633954212e-01 5.9521810104081e-01 4.1645105476911e-01
1.0746224752075e+00 6.0058709968663e-01 3.9374862202831e-01
7.2299082453153e-01 6.0595821687699e-01 3.7104681031865e-01
3.7179045349002e-01 6.1133155430089e-01 3.4834541659738e-01
1.0209921283063e+00 6.1670725327196e-01 3.2564412743055e-01
6.7056948895364e-01 6.2208550080217e-01 3.0294253630930e-01
3.2049952368483e-01 6.2746652221846e-01 2.8024016532839e-01
9.7076261341039e-01 6.3285056559240e-01 2.5753648936081e-01
6.2134235891484e-01 6.3823788210531e-01 2.3483096170405e-01
2.7222533946934e-01 6.4362870546448e-01 2.1212304036744e-01
9.2340089076286e-01 6.4902323267583e-01 1.8941221403419e-01
5.7486093921905e-01 6.5442160783592e-01 1.6669802650383e-01
2.2659989170047e-01 6.5982391008196e-01 1.4398009824871e-01
8.7861455568142e-01 6.6523014638698e-01 1.2125814369094e-01
5.3090405520313e-01 6.7064024946894e-01 9.8531982964515e-02
1.8346971092663e-01 6.7605408067308e-01 7.5801547281254e-02
8.3631486558588e-01 6.8147143728492e-01 5.3066877543800e-02
4.8944465515676e-01 6.8689206336198e-01 3.0328116480233e-02
1.4286574651267e-01 6.9231566288505e-01 7.5854952158751e-03
7.9519414952314e-01 6.9774971476412e-01 9.8483013203327e-01
4.4920679729165e-01 7.0317846234170e-01 9.6208054305514e-01
1.0353683514043e-01 7.0860917958053e-01 9.3932814462876e-01
7.5819529442106e-01 7.1404154247396e-01 9.1657333199200e-01
4.1319452471449e-01 7.1947524453411e-01 8.9381649532242e-01
6.8548719181094e-02 7.2491000163386e-01 8.7105800649012e-01
7.2427473245022e-01 7.3034555468941e-01 8.4829820958455e-01
3.8039324119907e-01 7.3578167083512e-01 8.2553741531896e-01
3.6930317053257e-02 7.4121814389596e-01 8.0277589928928e-01
6.9391951603386e-01 7.4665479506714e-01 7.8001390415415e-01
3.5140464355571e-01 7.5209147484986e-01 7.5725164613166e-01
9.4434370409826e-03 7.5752806760009e-01 7.3448932679199e-01
6.6811253258754e-01 7.6296450075104e-01 7.1172715198183e-01
3.2751426775076e-01 7.6840076226656e-01 6.8896536080795e-01
-1.2213853694828e-02 7.7383693291576e-01 6.6620426865248e-01
6.4911417713851e-01 7.7927324586723e-01 6.4344432809785e-01
3.1175177004610e-01 7.8471019700783e-01 6.2068620697598e-01
-2.3953853468118e-02 7.9014874705861e-01 5.9793086412050e-01
6.4247305594000e-01 7.9559067411341e-01 5.7517954877900e-01
3.1167345260400e-01 8.0103909890635e-01 5.5243352381881e-01
-1.5540770636610e-02 8.0649888235954e-01 5.2969314392957e-01
-4.9290802940122e-01 5.8568738641703e-01 5.0722362131867e-01
1.1523328118822e+00 5.9104902764037e-01 4.8451805457291e-01
7.9823681674933e-01 5.9641216667237e-01 4.6181295190479e-01
4.4479575592091e-01 6.0177728325016e-01 4.3910852462346e-01
1.0919826629388e+00 6.0714454987999e-01 4.1640496290904e-01
7.3975963435320e-01 6.1251398185645e-01 3.9370235534541e-01
3.8808419206157e-01 6.1788554676461e-01 3.7100065802091e-01
1.0369137661833e+00 6.2325923197163e-01 3.4829969425423e-01
6.8620844717595e-01 6.2863507804237e-01 3.2559916984265e-01
3.3593240126274e-01 6.3401318810209e-01 3.0289869459352e-01
9.8605437541814e-01 6.3939372187760e-01 2.8019780561909e-01
6.3654765398681e-01 6.4477688096127e-01 2.5749599072176e-01
2.8738973768949e-01 6.5016288986238e-01 2.3479271150408e-01
9.3856192858295e-01 6.5555197593273e-01 2.1208742618026e-01
5.9004893193147e-01 6.6094435023879e-01 1.8937961192050e-01
2.4183852958616e-01 6.6634019077049e-01 1.6666878624401e-01
8.9392133837875e-01 6.7173962890700e-01 1.4395452667387e-01
5.4629063986571e-01 6.7714273971267e-01 1.2123648767186e-01
1.9894225320804e-01 6.8254953635057e-01 9.8514413829478e-02
8.5187441957943e-01 6.8795996863432e-01 7.5788148417844e-02
5.0508767232373e-01 6.9337392547375e-01 5.3057636687129e-02
1.5858467956683e-01 6.9879124070604e-01 3.0322923722827e-02
8.1237006202429e-01 7.0421170156595e-01 7.5841471552108e-03
4.6508866686109e-01 7.0964269647801e-01 9.8483253290699e-01
1.1945470879342e-01 7.1506886675918e-01 9.6208613304672e-01
7.7413214997179e-01 7.2049735577571e-01 9.3933651871589e-01
4.2913202704882e-01 7.2592786886784e-01 9.1658405635518e-01
8.4467096273638e-02 7.3136011444997e-01 8.9382912891448e-01
7.4015222499997e-01 7.3679381051236e-01 8.7107212202656e-01
3.9620503473649e-01 7.4222868955181e-01 8.4831341308008e-01
5.2646844396501e-02 7.4766450232613e-01 8.2555336371600e-01
7.0950397322870e-01 7.5310102105327e-01 8.0279231609951e-01
3.6680948347422e-01 7.5853804287756e-01 7.8003059328259e-01
2.4605476136272e-02 7.6397539466410e-01 7.5726850408196e-01
6.8294610315623e-01 7.6941294056738e-01 7.3450635313116e-01
3.4190153047357e-01 7.7485059452681e-01 7.1174445703983e-01
1.5631833551635e-03 7.8028834114139e-01 6.8898316769595e-01
6.6205072670490e-01 7.8572627066986e-01 6.6622290314387e-01
3.2352135325670e-01 7.9116463766180e-01 6.4346418390777e-01
-1.3818034521880e-02 7.9660395800553e-01 6.2070766533167e-01
6.5030439428357e-01 8.0204516350237e-01 5.9795413908344e-01
3.1624188215765e-01 8.0748982407156e-01 5.7520444181788e-01
-1.5560942474180e-02 8.1294038186565e-01 5.5245915784765e-01
6.5541240522749e-01 8.1840010250399e-01 5.2971800283863e-01
1.7399257572520e-01 5.9761356594512e-01 5.0718180296786e-01
8.1853082886085e-01 6.0297441716994e-01 4.8447533270542e-01
4.6387617454928e-01 6.0833718182854e-01 4.6176935358119e-01
1.1100144268172e+00 6.1370234772667e-01 4.3906417369362e-01
7.5690649758236e-01 6.1906999373926e-01 4.1636007253940e-01
4.0449955361681e-01 6.2443999967623e-01 3.9365720013827e-01
1.0527357728235e+00 6.2981219281055e-01 3.7095554688158e-01
7.0155821904334e-01 6.3518643350703e-01 3.4825495243310e-01
3.5091423240898e-01 6.4056265514180e-01 3.2555513061327e-01
1.0007570137943e+00 6.4594087436870e-01 3.0285569759145e-01
6.5104604679950e-01 6.5132118417423e-01 2.8015619818618e-01
3.0174685534887e-01 6.5670373805328e-01 2.5745612916764e-01
9.5283044236163e-01 6.6208873045085e-01 2.3475496015144e-01
6.0427263073200e-01 6.6747637651280e-01 2.1205215301876e-01
2.5605343646685e-01 6.7286689291771e-01 1.8934718052911e-01
9.0815653955201e-01 6.7826048082688e-01 1.6663954432752e-01
5.6056887416611e-01 6.8365731156828e-01 1.4392879209879e-01
2.1328033113147e-01 6.8905751541713e-01 1.2121453327966e-01
8.6628354868129e-01 6.9446117366528e-01 9.8496452541906e-02
5.1957376057690e-01 6.9986831402953e-01 7.5774320214720e-02
1.7314867162601e-01 7.0527890930618e-01 5.3047998919502e-02
8.2700833800440e-01 7.1069287902411e-01 3.0317445928972e-02
4.8115504146416e-01 7.1611009368713e-01 7.5827111016968e-03
1.3426386008198e-01 7.2153784510599e-01 9.8483514123336e-01
7.8898199067160e-01 7.2696120094562e-01 9.6209229677394e-01
4.4400530720584e-01 7.3238717451995e-01 9.3934590832481e-01
9.9343982833474e-02 7.3781551327671e-01 9.1659630294917e-01
7.5501021802172e-01 7.4324595419175e-01 8.9384384366492e-01
4.1101852094514e-01 7.4867823093871e-01 8.7108891603376e-01
6.7386192612468e-02 7.5411208025290e-01 8.4833191653060e-01
7.2413406306779e-01 7.5954724768196e-01 8.2557324346722e-01
3.8128753047069e-01 7.6498349315577e-01 8.0281329106834e-01
3.8877965549834e-02 7.7042059704742e-01 7.8005244716934e-01
6.9694456159180e-01 7.7585836767233e-01 7.5729109492933e-01
3.5553673608346e-01 7.8129665155287e-01 7.3452961888835e-01
1.4717224490208e-02 7.8673534836779e-01 7.1176841553096e-01
6.7456604591917e-01 7.9217443344294e-01 6.8900790800503e-01
3.3518555067636e-01 7.9761399203255e-01 6.6624856329488e-01
-3.2932608555413e-03 8.0305427138988e-01 6.4349090704115e-01
6.5929685729787e-01 8.0849575779989e-01 6.2073552477962e-01
3.2316764520817e-01 8.1393928283404e-01 5.9798302666629e-01
-1.1418099949943e-02 8.1938614561155e-01 5.7523393574175e-01
6.5585198434912e-01 8.2483817991443e-01 5.5248844915379e-01
3.2531613384049e-01 8.3029754966745e-01 5.2974606819768e-01
-1.5905213232183e-01 6.0953888834397e-01 5.0714195053729e-01
4.8463046508586e-01 6.1489866729274e-01 4.8443444848091e-01
1.1292949520574e+00 6.2026091301662e-01 4.6172744693250e-01
7.7491824335468e-01 6.2562612081336e-01 4.3902138075679e-01
4.2144405292124e-01 6.3099424219861e-01 4.1631663777845e-01
1.0687991800730e+00 6.3636498406755e-01 3.9361343161560e-01
7.1690565740325e-01 6.4173800528007e-01 3.7091177722454e-01
3.6568837953724e-01 6.4711302269727e-01 3.4821151591248e-01
1.0150791278653e+00 6.5248985508541e-01 3.2551235498088e-01
6.6501816671728e-01 6.5786843061931e-01 3.0281390534181e-01
3.1545439226674e-01 6.6324877572919e-01 2.8011571189425e-01
9.6634472146740e-01 6.6863099585821e-01 2.5741727690369e-01
6.1765315700916e-01 6.7401525383108e-01 2.3471807845800e-01
2.6934978587873e-01 6.7940174867630e-01 2.1201758615334e-01
9.2140985262446e-01 6.8479069621884e-01 1.8931527558695e-01
5.7381297499772e-01 6.9018231200463e-01 1.6661064252506e-01
2.2654252383320e-01 6.9557679677078e-01 1.4390321698681e-01
8.7958516072922e-01 7.0097432452781e-01 1.2119257700716e-01
5.3293051041226e-01 7.0637503326240e-01 9.8478361519542e-02
1.8657093640721e-01 7.1177901824104e-01 7.5760281626301e-02
8.4050138698851e-01 7.1718632786412e-01 5.3038129490764e-02
4.9471928212513e-01 7.2259696197003e-01 3.0311784178060e-02
1.4922442008279e-01 7.2801087241493e-01 7.5812139762822e-03
8.0272367749459e-01 7.3343524589389e-01 9.8483790092550e-01
4.5779312523353e-01 7.3885560230646e-01 9.6209888775444e-01
1.1316185063738e-01 7.4427882306653e-01 9.3935607085733e-01
7.6883850892110e-01 7.4970470849553e-01 9.1660972875032e-01
4.2483400385497e-01 7.5513303634553e-01 8.9386019270110e-01
8.1161683331761e-02 7.6056356869948e-01 8.7110783452618e-01
7.3783769253291e-01 7.6599605889267e-01 8.4835305494138e-01
3.9488152778562e-01 7.7143025851360e-01 8.2559627334527e-01
5.2316837052743e-02 7.7686592474881e-01 8.0283791977304e-01
7.1017251747121e-01 7.8230282856062e-01 7.8007842959329e-01
3.6848416732586e-01 7.8774076444127e-01 7.5731824133047e-01
2.7295959270189e-02 7.9317956280484e-01 7.3455779774372e-01
6.8666301140596e-01 7.9861910650201e-01 7.1179754987609e-01
3.4665433630468e-01 8.0405935349888e-01 6.8903796301097e-01
7.3564279333184e-03 8.0950036838115e-01 6.6627952199876e-01
6.6887746197115e-01 8.1494236566567e-01 6.4352273073354e-01
3.3135184551222e-01 8.2038576676720e-01 6.2076809607202e-01
-5.0557301883149e-03 8.2583126684375e-01 5.9801608012282e-01
6.5985115026160e-01 8.3127989069619e-01 5.7526699916936e-01
3.2629490793727e-01 8.3673297548637e-01 5.5252085343259e-01
-5.4971327416418e-03 8.4219193766523e-01 5.2977711901059e-01
-4.9202905403110e-01 6.2146326750668e-01 5.0710431009215e-01
1.1506099336381e+00 6.2682159437602e-01 4.8439564682182e-01
7.9444918804374e-01 6.3218313140226e-01 4.6168747384164e-01
4.3945180557314e-01 6.3754837303799e-01 4.3898039640402e-01
1.0855362993577e+00 6.4291708810804e-01 4.1627493425102e-01
7.3260092249003e-01 6.4828875517549e-01 3.9357136216211e-01
3.8054065544842e-01 6.5366282667774e-01 3.7086970171486e-01
1.0292569729199e+00 6.5903885602425e-01 3.4816977377445e-01
6.7866235854284e-01 6.6441654015214e-01 3.2547125986594e-01
3.2868159741522e-01 6.6979571849176e-01 3.0277375203434e-01
9.7925133697741e-01 6.7517635323608e-01 2.8007678764998e-01
6.3031885175595e-01 6.8055850387995e-01 2.5737987216751e-01
2.8184054568877e-01 6.8594230193671e-01 2.3468249420315e-01
9.3378047477445e-01 6.9132792809744e-01 2.1198413663741e-01
5.8610902744970e-01 6.9671559241994e-01 1.8928428629454e-01
2.3880181901615e-01 7.0210551745953e-01 1.6658244367687e-01
8.9183881167806e-01 7.0749792404571e-01 1.4387813339657e-01
5.4520364690835e-01 7.1289301939788e-01 1.2117091534732e-01
1.9888316312846e-01 7.1829098733580e-01 9.8460396248448e-02
8.5286706470883e-01 7.2369198042097e-01 7.5746240934856e-02
5.0714770608018e-01 7.2909611393585e-01 5.3028182631920e-02
1.6171995641681e-01 7.3450346165661e-01 3.0306031433548e-02
8.1658111542053e-01 7.3991405339163e-01 7.5796802865124e-03
4.7047157475365e-01 7.4533496035082e-01 9.8484076133323e-01
1.2589233386139e-01 7.5075217853707e-01 9.6210577423098e-01
7.8160717140003e-01 7.5617245357014e-01 9.3936678857070e-01
4.3762275406000e-01 7.6159564758387e-01 9.1662402622827e-01
9.3948137516160e-02 7.6702158984817e-01 8.9387777583508e-01
7.5059490353352e-01 7.7245008253924e-01 8.7112838244565e-01
4.0757741020774e-01 7.7788090733671e-01 8.4837623651792e-01
6.4913194194182e-02 7.8331383284364e-01 8.2562176307999e-01
7.2262356609071e-01 7.8874862296721e-01 8.0286541188112e-01
3.8073444135122e-01 7.9418504656646e-01 7.8010764917624e-01
3.9277449418188e-02 7.9962288886775e-01 7.5734895156785e-01
6.9829136243688e-01 8.0506196537137e-01 7.3458980199888e-01
3.5782387914629e-01 8.1050213922258e-01 7.1183068748281e-01
1.7933782231161e-02 8.1594334325736e-01 6.8907209734984e-01
6.7869344281012e-01 8.2138560800919e-01 6.6631451949660e-01
3.4019154127984e-01 8.2682909645747e-01 6.4355843015308e-01
2.5356465485583e-03 8.3227414420388e-01 6.2080426998908e-01
6.6585383550256e-01 8.3772129792633e-01 5.9805239666108e-01
3.3029368190918e-01 8.4317133141596e-01 5.7530300395073e-01
-3.9844241474344e-03 8.4862519174877e-01 5.5255600807344e-01
6.6317572372411e-01 8.5408378627145e-01 5.2981093774379e-01
1.7508068941015e-01 6.3338663381073e-01 5.0706912519640e-01
8.1643822131126e-01 6.3874299209241e-01 4.8435917105694e-01
4.5927738301085e-01 6.4410357375868e-01 4.6164967658411e-01
1.1035389948071e+00 6.4946884764293e-01 4.3894148284555e-01
7.4910345440811e-01 6.5483830866415e-01 4.1623526947671e-01
3.9582799231934e-01 6.6021112557135e-01 3.9353135972184e-01
1.0435705575589e+00 6.6558649465020e-01 3.7082974985297e-01
6.9220198978721e-01 6.7096378378948e-01 3.4813020737997e-01
3.4161056247963e-01 6.7634256314653e-01 3.2543236299317e-01
9.9170237089338e-01 6.8172258659202e-01 3.0273577468836e-01
6.4239979058360e-01 6.8710375760494e-01 2.8003996580889e-01
2.9363923752346e-01 6.9248609439077e-01 2.5734444510670e-01
9.4536884440721e-01 6.9786969940855e-01 2.3464871654335e-01
5.9754633025920e-01 7.0325473429554e-01 2.1195228439728e-01
2.5013716940736e-01 7.0864139962968e-01 1.8925465716042e-01
9.0311308388667e-01 7.1402991857475e-01 1.6655535215402e-01
5.5645084580332e-01 7.1942052349514e-01 1.4385390174688e-01
2.1013135964865e-01 7.2481344480245e-01 1.2114986137859e-01
8.6413898749746e-01 7.3020890149208e-01 9.8442819146557e-02
5.1846107725187e-01 7.3560709300895e-01 7.5732406422751e-02
1.7308765358031e-01 7.4100819224246e-01 5.3018308775137e-02
8.2801123245907e-01 7.4641233958510e-01 3.0300276364250e-02
4.8322672339348e-01 7.5181963809401e-01 7.5781329665640e-03
1.3750988454545e-01 7.5723703154088e-01 9.8484367586018e-01
7.9328284063361e-01 7.6265101421641e-01 9.6211283657015e-01
4.4934569100132e-01 7.6806819032792e-01 9.3937786585724e-01
1.0570279865730e-01 7.7348849119228e-01 9.1663892151846e-01
7.6236097786757e-01 7.7891180702961e-01 8.9389623936378e-01
4.1932959812441e-01 7.8433799081499e-01 8.7115012536629e-01
7.6620760794852e-02 7.8976686368618e-01 8.4840094592286e-01
7.3424958199568e-01 7.9519822191571e-01 8.2564911700829e-01
3.9223461858990e-01 8.0063184551394e-01 8.0289509302143e-01
5.0598474496206e-02 8.0606750861735e-01 7.8013935603443e-01
7.0936862137420e-01 8.1150499192483e-01 7.5738240596088e-01
3.6857846012118e-01 8.1694409755971e-01 7.3462475183547e-01
2.8268634764694e-02 8.2238466682870e-01 7.1186690390826e-01
6.8848858219911e-01 8.2782660135197e-01 6.8910936553886e-01
3.4929824755525e-01 8.3326988778575e-01 6.6635262286391e-01
1.0769792945595e-02 8.3871462548089e-01 6.4359712889535e-01
6.7298893788066e-01 8.4416105416937e-01 6.2084327730616e-01
3.3605523534265e-01 8.4960957379606e-01 5.9809136051302e-01
8.0048884015905e-05 8.5506073890269e-01 5.7534150899435e-01
6.6518026746384e-01 8.6051519371990e-01 5.5259361872884e-01
3.3146517084718e-01 8.6597349341910e-01 5.2984729833064e-01
-1.5769486937040e-01 6.4530894461061e-01 5.0703663348862e-01
1.4820700413391e+00 6.5066262083862e-01 4.8432525939108e-01
1.1236917681468e+00 6.5602193036944e-01 4.6161429592434e-01
7.6707374338046e-01 6.6138725659762e-01 4.3890492089288e-01
4.1203677481778e-01 6.6675766824115e-01 4.1619800307836e-01
1.0583776916770e+00 6.7213190525550e-01 3.9349387997027e-01
7.0590290076398e-01 6.7750884563434e-01 3.7079246754884e-01
3.5444273438092e-01 6.8288765087422e-01 3.4809343237172e-01
1.0038548916461e+00 6.8826776511335e-01 3.2539632355945e-01
6.5402281965651e-01 6.9364886561711e-01 3.0270065034390e-01
3.0485224397714e-01 6.9903080707230e-01 2.8000591912681e-01
9.5626736568071e-01 7.0441357402647e-01 2.5731164664752e-01
6.0820734993454e-01 7.0979724406051e-01 2.3461736142650e-01
2.6062331182387e-01 7.1518196029368e-01 2.1192260094213e-01
9.1347582632420e-01 7.2056791096553e-01 1.8922690864005e-01
5.6673292231189e-01 7.2595531404253e-01 1.6652983280716e-01
2.2036850140728e-01 7.3134440524324e-01 1.4383092815601e-01
8.7436112171232e-01 7.3673542829518e-01 1.2112976029882e-01
5.2869309124392e-01 7.4212862657371e-01 9.8425912900594e-02
1.8334982096041e-01 7.4752423554213e-01 7.5718997044708e-02
8.3831939118036e-01 7.5292247563891e-01 5.3008662151629e-02
4.9359228797920e-01 7.5832354546006e-01 3.0294607639088e-02
1.4916126862751e-01 7.6372761526465e-01 7.5765944062905e-03
8.0383944262475e-01 7.6914148791142e-01 9.8484660005844e-01
4.5996638952596e-01 7.7455217662982e-01 9.6211996301043e-01
1.1638029410193e-01 7.7996613708434e-01 9.3938912339015e-01
7.7308303416251e-01 7.8538337581801e-01 9.1665416772211e-01
4.3007892077827e-01 7.9080385236325e-01 8.9391526912382e-01
8.7374793788127e-02 7.9622748043860e-01 8.7117268257832e-01
7.4498015396977e-01 8.0165413131569e-01 8.4842673717767e-01
4.0290735881805e-01 8.0708363945512e-01 8.2567782595660e-01
6.1171914535624e-02 8.1251581047536e-01 8.0292639406295e-01
7.1979289816935e-01 8.1795043151058e-01 7.8017292602380e-01
3.7879353988024e-01 8.2338728402159e-01 7.5741793275023e-01
3.8201984837621e-02 8.2882615912785e-01 7.3466193866944e-01
6.9805223486980e-01 8.3426687548916e-01 7.1190546896368e-01
3.5838523730072e-01 8.3970929960824e-01 6.8914903631569e-01
1.9250034009116e-02 8.4515336800358e-01 6.6639312581575e-01
6.8070479338123e-01 8.5059910973234e-01 6.4363817586122e-01
3.4281739994612e-01 8.5604666572747e-01 6.2088455226874e-01
5.6650456261580e-03 8.6149629758420e-01 5.9813251315966e-01
6.6933194128164e-01 8.6694837190560e-01 5.7538216500984e-01
3.3390390209842e-01 8.7240329689979e-01 5.5263341818970e-01
-5.4161699317978e-04 8.7786137835271e-01 5.2988596637615e-01
-4.9031247496699e-01 6.5723020202290e-01 5.0700706265140e-01
1.1474370356193e+00 6.6258018870252e-01 4.8429414047770e-01
7.8756375615800e-01 6.6793782638762e-01 4.6158157077962e-01
4.2990537927804e-01 6.7330327689133e-01 4.3887102764177e-01
1.0741861053581e+00 6.7867492426141e-01 4.1616358775621e-01
7.2011164171759e-01 6.8405090853714e-01 3.9345952484386e-01
3.6741562846560e-01 6.8942971799776e-01 3.7075858315043e-01
1.0158744762590e+00 6.9481029514930e-01 3.4806026337903e-01
6.6530722210591e-01 7.0019196991179e-01 3.2536400012379e-01
3.1556994377912e-01 7.0557436043831e-01 3.0266924484198e-01
9.6654906352174e-01 7.1095728790500e-01 2.7997549238430e-01
6.1815517672248e-01 7.1634071381174e-01 2.5728228017723e-01
2.7031786487153e-01 7.2172469657880e-01 2.3458917705097e-01
9.2298150299131e-01 7.2710936213525e-01 2.1189577032799e-01
5.7610193145845e-01 7.3249488396054e-01 1.8920165505499e-01
2.2964384065089e-01 7.3788146926992e-01 1.6650642687146e-01
8.8357873472877e-01 7.4326934905060e-01 1.4380967890142e-01
5.3788336706145e-01 7.4865877036287e-01 1.2111100257822e-01
1.9253856331052e-01 7.5404998979156e-01 9.8409992103426e-02
8.4752836583264e-01 7.5944326725884e-01 7.5706252115095e-02
5.0283944475048e-01 7.6483885966758e-01 5.2999408011666e-02
1.5846072779999e-01 7.7023701407969e-01 3.0289118221851e-02
8.1438320446010e-01 7.7563796036746e-01 7.5750875400985e-03
4.6945946401737e-01 7.8104834534321e-01 9.8484948948064e-01
1.2594300599217e-01 7.8645572023837e-01 9.6212704450941e-01
7.8271199578083e-01 7.9186638357125e-01 9.3940039041061e-01
4.3976576581389e-01 7.9728042195636e-01 9.1666953518760e-01
9.7105974868507e-02 8.0269787173619e-01 8.9393457929258e-01
7.5473672015131e-01 8.0811871681859e-01 8.7119571475412e-01
4.1266465472236e-01 8.1354288914622e-01 8.4845322021705e-01
7.0899128778003e-02 8.1897027207301e-01 8.2570745226904e-01
7.2945238241015e-01 8.2440070679340e-01 8.0295883368943e-01
3.8833982158641e-01 8.2983400185603e-01 7.8020783940021e-01
4.7580406625177e-02 8.3526994569186e-01 7.5745498090600e-01
7.0719717182652e-01 8.4070832197814e-01 7.3470078986727e-01
3.6721787428089e-01 8.4614892750492e-01 7.1194580115904e-01
2.7675736295045e-02 8.5159159194377e-01 6.8919053533489e-01
6.8861019449025e-01 8.5703619842388e-01 6.6643547988756e-01
3.5006749186494e-01 8.6248270290397e-01 6.4368106819295e-01
1.2100837132519e-02 8.6793114868607e-01 6.2092765482474e-01
6.7476970081194e-01 8.7338166965841e-01 5.9817548660163e-01
3.3813762844620e-01 8.7883447168495e-01 5.7542467127079e-01
2.2677842416715e-03 8.8428977636559e-01 5.5267515152328e-01
6.6721544871371e-01 8.8974770749387e-01 5.2992670216251e-01
1.7729800128109e-01 6.6915048544768e-01 5.0698062562066e-01
8.1243153821696e-01 6.7449531590670e-01 4.8426602778776e-01
4.5069858542113e-01 6.7985079331560e-01 4.6155174376812e-01
1.0918145777065e+00 6.8521654947710e-01 4.3884020050131e-01
7.3534055752123e-01 6.9058983792712e-01 4.1613265475240e-01
3.8084137242543e-01 6.9596796155534e-01 3.9342915170754e-01
1.0279467773118e+00 7.0134895119769e-01 3.7072911945572e-01
6.7636129199202e-01 7.0673153939991e-01 3.4803181444731e-01
3.2585533476704e-01 7.1211497195703e-01 3.2533653281031e-01
9.7625228669426e-01 7.1749883636686e-01 3.0264267584807e-01
6.2741657484321e-01 7.2288294088670e-01 2.7994974845004e-01
2.7924399800638e-01 7.2826723662867e-01 2.5725733416220e-01
9.3165422163955e-01 7.3365176909671e-01 2.3456506671469e-01
5.8458504162332e-01 7.3903664834997e-01 2.1187260549182e-01
2.3798800910208e-01 7.4442203059651e-01 1.8917961699407e-01
8.9182508546843e-01 7.4980810670305e-01 1.6648576227811e-01
5.4606608015680e-01 7.5519509479646e-01 1.4379068977384e-01
2.0068669300299e-01 7.6058323512166e-01 1.2109403285493e-01
8.5566703452121e-01 7.6597278588187e-01 9.8395411669303e-02
5.1099053361558e-01 7.7136401912056e-01 7.5694438839083e-02
1.6664316589304e-01 7.7675721594151e-01 5.2990728644101e-02
8.2261294958377e-01 7.8215266058514e-01 3.0283909192482e-02
4.7888966261005e-01 7.8755063312734e-01 7.5736368759925e-03
1.3436766681992e-01 7.9295760724821e-01 9.8485229753203e-01
7.9121099696505e-01 7.9836168939740e-01 9.6213396930671e-01
4.4833983199186e-01 8.0376901069430e-01 9.3941149616041e-01
1.0575101918922e-01 8.0917974141377e-01 9.1668480018033e-01
7.6344351961259e-01 8.1459400141709e-01 8.9395389876191e-01
4.2141856300178e-01 8.2001185395349e-01 8.7121890845736e-01
7.9679782205067e-02 8.2543330234832e-01 8.4848004379682e-01
7.3823333445398e-01 8.3085829013086e-01 8.2573761110246e-01
3.9708803038263e-01 8.3628670493369e-01 8.0299199773218e-01
5.6255500361849e-02 8.4171838626821e-01 7.8024365750323e-01
7.1575042896161e-01 8.4715313705904e-01 7.5749309321697e-01
3.7559088008993e-01 8.5259073859316e-01 7.3474083721541e-01
3.5798715872156e-02 8.5803096827564e-01 7.1198743075055e-01
6.9640007995720e-01 8.6347361923175e-01 6.8923340266101e-01
3.5742586859750e-01 8.6891852027251e-01 6.6647924748431e-01
1.8912047898888e-02 8.7436555392773e-01 6.4372540281113e-01
6.8089959074248e-01 8.7981466898223e-01 6.2097222564689e-01
3.4343370373918e-01 8.8526588206064e-01 5.9821996820108e-01
6.5619082545008e-03 8.9071926027737e-01 5.7546875549293e-01
6.7033047601975e-01 8.9617487430964e-01 5.5271857113961e-01
3.3477879872713e-01 9.0163271016300e-01 5.2996926392715e-01
-1.5474214701802e-01 6.8107001690656e-01 5.0695751441199e-01
4.7687440150591e-01 6.8640746510474e-01 4.8424111204456e-01
1.1127886839168e+00 6.9176022695132e-01 4.6152508628649e-01
7.5247235875485e-01 6.9712668679747e-01 4.3881302826806e-01
3.9519776257160e-01 7.0250219093912e-01 4.1610619693368e-01
1.0403068104466e+00 7.0788290477874e-01 3.9340408019715e-01
6.8728002899910e-01 7.1326637235817e-01 3.7070558403846e-01
3.3572669260978e-01 7.1865116866280e-01 3.4800965277771e-01
9.8535706948961e-01 7.2403651107407e-01 3.2531545634938e-01
6.3595648111229e-01 7.2942199579441e-01 3.0262238972378e-01
2.8736519782960e-01 7.3480744162343e-01 2.7993001673307e-01
9.3946371499827e-01 7.4019280147117e-01 2.5723801015415e-01
5.9216211792019e-01 7.4557811282785e-01 2.3454610326305e-01
2.4539237418729e-01 7.5096347005352e-01 2.1185405455749e-01
8.9910268832561e-01 7.5634900892492e-01 1.8916162345711e-01
5.5325331366352e-01 7.6173489816537e-01 1.6646855435980e-01
2.0781340857823e-01 7.6712133496519e-01 1.4377456701629e-01
8.6275865866652e-01 7.7250854266037e-01 1.2107935191438e-01
5.1806947838366e-01 7.7789676930178e-01 9.8382569963867e-02
1.7372966771658e-01 7.8328628611706e-01 7.5683856172039e-02
8.2972544000650e-01 7.8867738502337e-01 5.2982827184703e-02
4.8604476166624e-01 7.9407037449909e-01 3.0279092547145e-02
1.4267695709518e-01 7.9946557333164e-01 7.5722693356129e-03
7.9856064616526e-01 8.0486926264664e-01 9.8485497356515e-01
4.5576718927832e-01 8.1027011904001e-01 9.6214061777113e-01
1.1326096910954e-01 8.1567409391397e-01 9.3942226131444e-01
7.7103654020695e-01 8.2108144322419e-01 9.1669973304988e-01
4.2909022681299e-01 8.2649237612264e-01 8.9397295642430e-01
8.7420348037166e-02 8.3190704410317e-01 8.7124195904510e-01
7.4602740422822e-01 8.3732553299242e-01 8.4850687648701e-01
4.0491421668377e-01 8.4274785873885e-01 8.2576794984542e-01
6.4086031554716e-02 8.4817396765186e-01 8.0302551713545e-01
7.2355061380780e-01 8.5360374139777e-01 7.8027999915975e-01
3.8331836450639e-01 8.5903700670279e-01 7.5753188085556e-01
4.3402491346203e-02 8.6447354935208e-01 7.3478168933996e-01
7.0381924629397e-01 8.6991313169259e-01 7.1202996989807e-01
3.6458821434157e-01 8.7535551240927e-01 6.8927726102176e-01
2.5732593762575e-02 8.8080046679833e-01 6.6652406936640e-01
6.8727935173921e-01 8.8624780504422e-01 6.4377084525898e-01
3.4925907233646e-01 8.9169738504856e-01 6.2101795935477e-01
1.1705243065664e-02 8.9714911511977e-01 5.9826568149824e-01
6.7465266763756e-01 9.0260294040680e-01 5.7551416418207e-01
3.3813468022793e-01 9.0805880580494e-01 5.5276343555851e-01
2.1789321293889e-03 9.1351658832748e-01 5.3001341067436e-01
-4.8620403134022e-01 6.9298930833411e-01 5.0693789042309e-01
1.1404451906138e+00 6.9831579582842e-01 4.8421955040011e-01
7.7332211851088e-01 7.0366533235232e-01 4.6150199081255e-01
4.1136705537046e-01 7.0903330091402e-01 4.3879058230793e-01
1.0533136772131e+00 7.1441180197320e-01 4.1608597156377e-01
6.9814397139394e-01 7.1979557175934e-01 3.9338648589517e-01
3.4512483656183e-01 7.2518175071206e-01 3.7069028636076e-01
9.9374196196686e-01 7.3056887844624e-01 3.4799602315096e-01
6.4363390679899e-01 7.3595622583634e-01 3.2530284276682e-01
2.9454408089101e-01 7.4134344123166e-01 3.0261024217312e-01
9.4628823332460e-01 7.4673037371635e-01 2.7991793103969e-01
5.9873269419490e-01 7.5211698524211e-01 2.5722573341443e-01
2.5177967243925e-01 7.5750330634722e-01 2.3453352391956e-01
9.0535714405012e-01 7.6288941359087e-01 2.1184118781880e-01
5.5941175091522e-01 7.6827541823963e-01 1.8914859627476e-01
2.1390370472011e-01 7.7366146110533e-01 1.6645559104923e-01
8.6880305336512e-01 7.7904771090956e-01 1.4376197523045e-01
5.2408689746881e-01 7.8443436465161e-01 1.2106750818026e-01
1.7973729231561e-01 7.8982164890076e-01 9.8371903948311e-02
8.3573966673880e-01 7.9520982107021e-01 7.5674833059500e-02
4.9208165292490e-01 8.0059916975042e-01 5.2975927935235e-02
1.4875226029895e-01 8.0599001320348e-01 3.0274792298055e-02
8.0574134844366e-01 8.1138269522806e-01 7.5710147497345e-03
4.6203450453913e-01 8.1678328225818e-01 9.8485746149690e-01
1.1960741246782e-01 8.2218103310613e-01 9.6214685814880e-01
7.7747103666631e-01 8.2758170441835e-01 9.3943249026791e-01
4.3561798857796e-01 8.3298563766624e-01 9.1671408689980e-01
9.4042139128642e-02 8.3839313566499e-01 8.9399146645892e-01
7.5273894086619e-01 8.4380444658163e-01 8.7126455308224e-01
4.1170570449378e-01 8.4921975031383e-01 8.4853338685826e-01
7.0941801280635e-02 8.5463914873000e-01 8.2579812670732e-01
7.3044878666720e-01 8.6006266091603e-01 8.0305904545584e-01
3.9023046334745e-01 8.6549022411353e-01 7.8031651742739e-01
5.0292918444937e-02 8.7092170052323e-01 7.5757099954404e-01
7.1064457417909e-01 8.7635688962505e-01 7.3482300740986e-01
3.7129628141531e-01 8.8179554513941e-01 7.1207308810280e-01
3.2261459608951e-02 8.8723739520551e-01 6.8932179144069e-01
6.9355624605199e-01 8.9268216375698e-01 6.6656964136148e-01
3.5519956545183e-01 8.9812959041047e-01 6.4381710885998e-01
1.7212973655115e-02 9.0357944544262e-01 6.2106458780472e-01
6.7962007619513e-01 9.0903153564681e-01 5.9831237513812e-01
3.4244528856571e-01 9.1448569616341e-01 5.7556065769677e-01
5.7117155684206e-03 9.1994176311784e-01 5.5280950935852e-01
6.6943801865247e-01 9.2539952281362e-01 5.3005890429384e-01
1.8340085529129e-01 7.0490954573760e-01 5.0692186356578e-01
8.0251149389734e-01 7.1021883518210e-01 4.8420145320204e-01
4.3138799153515e-01 7.1556508032968e-01 4.6148331873999e-01
1.0676677844888e+00 7.2093606453221e-01 4.3877522320858e-01
7.0901898545113e-01 7.2631850215823e-01 4.1607540374789e-01
3.5383710633191e-01 7.3170569084989e-01 3.9338013538319e-01
1.0010906724516e+00 7.3709468018536e-01 3.7068683418549e-01
6.5011343035042e-01 7.4248417206568e-01 3.4799413803800e-01
3.0046561092588e-01 7.4787357704849e-01 3.2530144463045e-01
9.5185019047587e-01 7.5326262424448e-01 3.0260854818513e-01
6.0406252225028e-01 7.5865119804098e-01 2.7991542540086e-01
2.5695937677801e-01 7.6403926663833e-01 2.5722212052412e-01
9.1043936408174e-01 7.6942684913590e-01 2.3452868577549e-01
5.6443017755339e-01 7.7481399989363e-01 2.1183515068296e-01
2.1888002890301e-01 7.8020080139234e-01 1.8914150597287e-01
8.7375172257702e-01 7.8558736180377e-01 1.6644769461107e-01
5.2901843422390e-01 7.9097381549603e-01 1.4375360636623e-01
1.8466061949741e-01 7.9636032547715e-01 1.2105907434019e-01
8.4066369847773e-01 8.0174708700634e-01 9.8363873096022e-02
4.9701629779094e-01 8.0713433157362e-01 7.5667718729993e-02
1.5370892004353e-01 8.1252233031580e-01 5.2970271655287e-02
8.1073296611939e-01 8.1791139580187e-01 3.0271143046222e-02
4.6808006936278e-01 8.2330188107362e-01 7.5699058179379e-03
1.2478574360384e-01 8.2869961277646e-01 9.8485969929257e-01
7.8272723515252e-01 8.3409444055024e-01 9.6215254396178e-01
4.4096469457568e-01 8.3949190767252e-01 9.3944196528206e-01
9.9489335061507e-02 8.4489243783043e-01 9.1672758786908e-01
7.5829291197460e-01 8.5029642961014e-01 8.9400911470541e-01
4.1736816326243e-01 8.5570423518350e-01 8.7128635124367e-01
7.6709215364109e-02 8.6111614026133e-01 8.4855922366833e-01
7.3631189903458e-01 8.6653234748029e-01 8.2582778907801e-01
3.9617394671851e-01 8.7195296507584e-01 8.0309223625061e-01
5.6295074905342e-02 8.7737800214423e-01 7.8035287672271e-01
7.1667698318757e-01 8.8280737111498e-01 7.5761012695327e-01
3.7732331871962e-01 8.8824089732970e-01 7.3486448319339e-01
3.8239654713669e-02 8.9367833490023e-01 7.1211649126094e-01
6.9943351204877e-01 8.9911938730869e-01 6.8936671369979e-01
3.6091441534320e-01 9.0456373051173e-01 6.6661569681990e-01
2.2693924126252e-02 9.1001103562891e-01 6.4386393994001e-01
6.8478552479014e-01 9.1546098766831e-01 6.2111186891280e-01
3.4720422177951e-01 9.2091329624717e-01 5.9835981587459e-01
9.9656421640548e-03 9.2636769403853e-01 5.7560800732626e-01
6.7308448510480e-01 9.3182391896723e-01 5.5285656325239e-01
3.3657222858015e-01 9.3728167742342e-01 5.3010551092743e-01
-1.4466992282799e-01 7.1683380733048e-01 5.0690943072527e-01
4.6163643580592e-01 7.2211366194750e-01 4.8418691333495e-01
1.0852202696589e+00 7.2745828855355e-01 4.6147186580535e-01
7.1995670936308e-01 7.3283473785069e-01 4.3877296640412e-01
3.6128047451336e-01 7.3822189987756e-01 4.1608159177463e-01
1.0066529140576e+00 7.4361258044058e-01 3.9339165157028e-01
6.5465942563489e-01 7.4900432698286e-01 3.7070078324470e-01
3.0447377482873e-01 7.5439618953246e-01 3.4800843373033e-01
9.5559026623229e-01 7.5978774649038e-01 3.2531473354399e-01
6.0768710014850e-01 7.6517879365857e-01 3.0262001430865e-01
2.6055319796444e-01 7.7056923259835e-01 2.7992462324399e-01
9.1404702346701e-01 7.7595902481546e-01 2.5722885852071e-01
5.6807231825118e-01 7.8134817056167e-01 2.3453295091449e-01
2.2256318404501e-01 7.8673669836192e-01 2.1183706172071e-01
8.7747453751965e-01 7.9212466016015e-01 1.8914128510947e-01
5.3277576093394e-01 7.9751213004625e-01 1.6644565046643e-01
1.8844631212382e-01 8.0289920565727e-01 1.4375012328564e-01
8.4447257043638e-01 8.0828601172998e-01 1.2105460449222e-01
5.0084548758015e-01 8.1367270532547e-01 9.8358928735643e-02
1.5755878673397e-01 8.1905948210520e-01 7.5662862622249e-02
8.1460756212252e-01 8.2444658279075e-01 5.2966104183940e-02
4.7198720148808e-01 8.2983429865737e-01 3.0268285108057e-02
1.2969259857920e-01 8.3522297467762e-01 7.5689773104288e-03
7.8681241376250e-01 8.4061816969424e-01 9.8486161973311e-01
4.4512296177510e-01 8.4601032884401e-01 9.6215751399242e-01
1.0373645009950e-01 8.5140475889785e-01 9.3945044376387e-01
7.6264348208415e-01 8.5680195806348e-01 9.1673992843208e-01
4.2183423778002e-01 8.6220241920665e-01 8.9402554746495e-01
8.1299039948626e-02 8.6760660358167e-01 8.7130697282330e-01
7.4102892336914e-01 8.7301491385376e-01 8.4858399685508e-01
4.0101611899877e-01 8.7842766934547e-01 8.2585655211695e-01
6.1254390348801e-02 8.8384508630513e-01 8.0312472046601e-01
7.2173919890870e-01 8.8926726543678e-01 7.8038873010468e-01
3.8246771828138e-01 8.9469418808558e-01 7.5764894075477e-01
4.3438750999224e-02 9.0012572149450e-01 7.3490581850630e-01
7.0465261764223e-01 9.0556163255151e-01 7.1215990294043e-01
3.6611107905199e-01 9.1100160849022e-01 6.8941176977571e-01
2.7817319028546e-02 9.1644528211384e-01 6.6666199252703e-01
6.8977596286935e-01 9.2189225831511e-01 6.4391110655049e-01
3.5199304782592e-01 9.2734213803248e-01 6.2115957847739e-01
1.4475810219996e-02 9.3279453542722e-01 5.9840778356568e-01
6.7723212745237e-01 9.3824908413040e-01 5.7565599310514e-01
3.4026946771620e-01 9.4370542906464e-01 5.5290437385746e-01
3.5932691588581e-03 9.4916320181525e-01 5.3015300159683e-01
-4.6593394774517e-01 7.2877207140562e-01 5.0690034090856e-01
1.1137618953047e+00 7.3399379477126e-01 4.8417683739809e-01
7.3099532631406e-01 7.3934401120485e-01 4.6147959403503e-01
3.6569343133481e-01 7.4472849501556e-01 4.3880053241581e-01
1.0084933324389e+00 7.5012039433193e-01 4.1611927311006e-01
6.5557452590051e-01 7.5551442132155e-01 3.9343212846617e-01
3.0517565667211e-01 7.6090901551815e-01 3.7074001254450e-01
9.5639200620977e-01 7.6630351256621e-01 3.4804441075018e-01
6.0871925203120e-01 7.7169756877538e-01 3.2534655981853e-01
2.6185877666748e-01 7.7709099324702e-01 3.0264737372133e-01
9.1562600662159e-01 7.8248368093833e-01 2.7994750759086e-01
5.6990363801531e-01 7.8787557981802e-01 2.5724742705626e-01
2.2461610823009e-01 7.9326667285490e-01 2.3454746000993e-01
8.7971483273276e-01 7.9865696785437e-01 2.1184783130407e-01
5.3516921041114e-01 8.0404649219689e-01 1.8914868451267e-01
1.9096086403138e-01 8.0943529114118e-01 1.6645009437245e-01
8.4707976245373e-01 8.1482342906365e-01 1.4375207250380e-01
5.0352146994870e-01 8.2021099329693e-01 1.2105456827717e-01
1.6028508899365e-01 8.2559810027510e-01 9.8357466317381e-02
8.1737164561984e-01 8.3098490355909e-01 7.5660582070847e-02
4.7478277767828e-01 8.3637160303799e-01 5.2963657013980e-02
1.3251965912351e-01 8.4175845422112e-01 3.0266355349566e-02
7.9058214296415e-01 8.4714577612119e-01 7.5682642876591e-03
4.4811549783202e-01 8.5253882933249e-01 9.8486315292340e-01
1.0679288519687e-01 8.5792865505788e-01 9.6216159595849e-01
7.6578173781813e-01 8.6332029504342e-01 9.3945766028149e-01
4.2507313368551e-01 8.6871430846946e-01 9.1675076557098e-01
8.4656484103345e-02 8.7411127558045e-01 8.9404036455501e-01
7.4452020576587e-01 8.7951176771939e-01 8.7132598340023e-01
4.0465247036068e-01 8.8491631356892e-01 8.4860726044445e-01
6.5041908694437e-02 8.9032536524376e-01 8.2588397817219e-01
7.2567816113586e-01 8.9573926818000e-01 8.0315608393671e-01
3.8655220789969e-01 9.0115823838627e-01 7.8042369640295e-01
4.7656470500998e-02 9.0658234968494e-01 7.5768709667296e-01
7.0898473295104e-01 9.1201153230552e-01 7.3494670511832e-01
3.7053197091450e-01 9.1744558282055e-01 7.1220304670992e-01
3.2294186752422e-02 9.2288418408201e-01 6.8945670887606e-01
6.9426832381691e-01 9.2832693259787e-01 6.6670829655315e-01
3.5645227957825e-01 9.3377336973988e-01 6.4395838913752e-01
1.8844967874492e-02 9.3922301239163e-01 6.2120750358545e-01
6.8144631552776e-01 9.4467537827854e-01 5.9845606708842e-01
3.4425703912553e-01 9.5013000143347e-01 5.7570440177816e-01
7.2780517129386e-03 9.5558643417292e-01 5.5295272301766e-01
6.7050940625537e-01 9.6104423365764e-01 5.3020115214869e-01
2.5346113639987e-01 7.4074014191434e-01 5.0692105742735e-01
7.4216038797106e-01 7.4584616808381e-01 4.8419554277995e-01
3.5934875899318e-01 7.5121845710716e-01 4.6156717483581e-01
1.0003124667674e+00 7.5661125869542e-01 4.3890231352918e-01
6.4832683369678e-01 7.6200860008015e-01 4.1621481433703e-01
2.9933818910022e-01 7.6740725781961e-01 3.9351658933230e-01
9.5191572340949e-01 7.7280598992079e-01 3.7081310989793e-01
6.0543466971611e-01 7.7820422761151e-01 3.4810706168993e-01
2.5958376875683e-01 7.8360168892944e-01 3.2539989022031e-01
9.1419451785291e-01 7.8899823087173e-01 3.0269243689491e-01
5.6917128669864e-01 7.9439378395897e-01 2.7998522303370e-01
2.2445904323399e-01 7.9978832006272e-01 2.5727858727156e-01
8.8002698948928e-01 8.0518183539097e-01 2.3457275629095e-01
5.3585960147576e-01 8.1057434127139e-01 2.1186788255708e-01
1.9195137600358e-01 8.1596585957947e-01 1.8916406426587e-01
8.4830353765880e-01 8.2135642147045e-01 1.6646135477390e-01
5.0492182121836e-01 8.2674606890493e-01 1.4375976515410e-01
1.6181485465517e-01 8.3213485882926e-01 1.2105926182926e-01
8.1899287679419e-01 8.3752286998185e-01 9.8359760505812e-02
4.7646663910740e-01 8.4291021221179e-01 7.5661117450831e-02
1.3424641226160e-01 8.4829703793672e-01 5.2963119332017e-02
7.9234106824362e-01 8.5368355494165e-01 3.0265473256112e-02
4.5075724868306e-01 8.5907003916309e-01 7.5677991711080e-03
1.0870047246096e-01 8.6446142110603e-01 9.8486423095439e-01
7.6773876742237e-01 8.6984933491449e-01 9.6216461502868e-01
4.2709826449497e-01 8.7523852294755e-01 9.3946333517353e-01
8.6771941687865e-02 8.8062958455720e-01 9.1675972616958e-01
7.4674958836175e-01 8.8602317289240e-01 8.9405311910957e-01
4.0701852076667e-01 8.9141996378400e-01 8.7134288792247e-01
6.7564492718195e-02 8.9682061651628e-01 8.4862849910479e-01
7.2837265214047e-01 9.0222573049622e-01 8.2590955807537e-01
3.8942838235205e-01 9.0763580294715e-01 8.0318584535114e-01
5.0717898124663e-02 9.1305119291063e-01 7.8045733696118e-01
7.1222853419490e-01 9.1847209603259e-01 7.5772420588374e-01
3.7394874805947e-01 9.2389853309496e-01 7.3498680420124e-01
3.5867933222363e-02 9.2933035339796e-01 7.1224562846783e-01
6.9797618002802e-01 9.3476725220665e-01 6.8950127297899e-01
3.6026411536630e-01 9.4020879972941e-01 6.6675437696928e-01
2.2722900484556e-02 9.4565447759805e-01 6.4400557218162e-01
6.8534438282029e-01 9.5110371769304e-01 6.2125543680724e-01
3.4812130897703e-01 9.5655593759460e-01 5.9850446064072e-01
1.1047441394315e-02 9.6201056715709e-01 5.7575302468136e-01
6.7411740539576e-01 9.6746706182977e-01 5.5300139668027e-01
3.3732613742732e-01 9.7292490034753e-01 5.3024974260827e-01
2.5346548647983e-01 7.5234113501726e-01 5.0692093992312e-01
2.4211698825266e-01 7.5761458838530e-01 4.8477300812982e-01
9.5020776082464e-01 7.6304211897477e-01 4.6191826235181e-01
6.1769033333227e-01 7.6846440168389e-01 4.3913739053194e-01
2.7842111919238e-01 7.7387768264057e-01 4.1638816044727e-01
9.3688827154682e-01 7.7928668934227e-01 3.9365159086248e-01
5.9435675403638e-01 7.8469293492278e-01 3.7092174843258e-01
2.5132411708296e-01 7.9009704758401e-01 3.4819625006144e-01
9.0802990896880e-01 7.9549934643322e-01 3.2547396708878e-01
5.6460766219529e-01 8.0090002095547e-01 3.0275430320155e-01
2.2114127451188e-01 8.0629919796969e-01 2.8003691939875e-01
8.7768923654281e-01 8.1169696988479e-01 2.5732161353685e-01
5.3429619835395e-01 8.1709340775415e-01 2.3460826136728e-01
1.9099892818243e-01 8.2248856793550e-01 2.1189678451610e-01
8.4782954177389e-01 8.2788249618996e-01 1.8918713089166e-01
5.0481728879759e-01 8.3327523118223e-01 1.6647926071455e-01
1.6198951188772e-01 8.3866680858235e-01 1.4377313463005e-01
8.1937208771840e-01 8.4405726662903e-01 1.2106870189258e-01
4.7698951153949e-01 8.4944665381616e-01 9.8365887453162e-02
1.3486471376425e-01 8.5483503916807e-01 7.5664577392971e-02
7.9301866430009e-01 8.6022252528466e-01 5.2964602756887e-02
4.5146981180288e-01 8.6560926389760e-01 3.0265722565360e-02
1.1023341326603e-01 8.7099547304632e-01 7.5676076496366e-03
7.6857894618324e-01 8.7638572149734e-01 9.8486479493366e-01
4.2796748371126e-01 8.8177223070689e-01 9.6216640808864e-01
8.7687596198255e-02 8.8715940375941e-01 9.3946719159213e-01
7.4773594629016e-01 8.9254785122281e-01 9.1676642232728e-01
4.0810425908617e-01 8.9793827492756e-01 8.9406332733129e-01
6.8779960075701e-02 9.0333143984765e-01 8.7135713241189e-01
7.2974716628161e-01 9.0872813249744e-01 8.4864712104855e-01
3.9098788129289e-01 9.1412910939453e-01 8.2593269616988e-01
5.2483185776996e-02 9.1953504152299e-01 8.0321343558426e-01
7.1421421223776e-01 9.2494646201544e-01 7.8048913204072e-01
3.7616275398369e-01 9.3036372408729e-01 7.5775981118078e-01
3.8311466681039e-02 9.3578697472810e-01 7.3502572438545e-01
7.0064374087164e-01 9.4121614726913e-01 7.1228731771901e-01
3.6314341486244e-01 9.4665097326388e-01 6.8954518186948e-01
2.5794528653989e-02 9.5209101154145e-01 6.6679999056312e-01
6.8858128015567e-01 9.5753569004211e-01 6.4405243613045e-01
3.5148824733670e-01 9.6298435428836e-01 6.2130317071380e-01
1.4500816892487e-02 9.6843631532933e-01 5.9855276018372e-01
6.7760565450837e-01 9.7389089002531e-01 5.7580165549018e-01
3.4079100242235e-01 9.7934742780514e-01 5.5305018341077e-01
4.0466162769402e-03 9.8480532044294e-01 5.3029855606164e-01
-2.7138364583254e-02 7.6433003672398e-01 5.0689998857403e-01
7.4207410277198e-01 7.6996059254946e-01 4.8419530309174e-01
5.1133311515768e-01 7.7504177174227e-01 4.6191814018827e-01
2.1942342113604e-01 7.8034094898539e-01 4.3932010028615e-01
8.9938978710954e-01 7.8574553558778e-01 4.1656467671598e-01
5.6854012334231e-01 7.9116071484092e-01 3.9380010987780e-01
2.3268910688091e-01 7.9657411553151e-01 3.7104491264822e-01
8.9418785284327e-01 8.0198451184649e-01 3.4829892643017e-01
5.5414990564800e-01 8.0739216137557e-01 3.2556016596368e-01
2.1316922997467e-01 8.1279743935125e-01 3.0282701476472e-01
8.7159365782482e-01 8.1820064825694e-01 2.8009835113716e-01
5.2964330317665e-01 8.2360201183686e-01 2.5737341831738e-01
1.8746720171909e-01 8.2900169321015e-01 2.3465170581119e-01
8.4517260760112e-01 8.3439981082306e-01 2.1193286772136e-01
5.0284111558993e-01 8.3979644976642e-01 1.8921666891433e-01
1.6053796916691e-01 8.4519166993618e-01 1.6650294861557e-01
8.1831761816114e-01 8.5058551279424e-01 1.4379159402406e-01
4.7622708731155e-01 8.5597800821113e-01 1.2108251905278e-01
1.3430798775740e-01 8.6136918260768e-01 9.8375644970188e-02
7.9259763092242e-01 8.6675906939461e-01 7.5670880839698e-02
4.5112950824909e-01 8.7214772246602e-01 5.2968102541525e-02
1.0993329881690e-01 8.7753523313798e-01 3.0267130048385e-02
7.6903452147454e-01 8.8292175034327e-01 7.5677036696943e-03
4.2777022202891e-01 8.8831145161668e-01 9.8486480418820e-01
8.7492715762583e-02 8.9369713967373e-01 9.6216684397214e-01
7.4755692424586e-01 8.9908283441461e-01 9.3946898225194e-01
4.0796530523366e-01 9.0446912073065e-01 9.1677047918602e-01
6.8713530987799e-02 9.0985671362338e-01 8.9407049190637e-01
7.2979086309524e-01 9.1524643882802e-01 8.7136811852834e-01
3.9118105616102e-01 9.2063919427918e-01 8.4866246128926e-01
5.2863692179676e-02 9.2603589405778e-01 8.2595270219339e-01
7.1481572218310e-01 9.3143740044681e-01 8.0323818027226e-01
3.7701292085740e-01 9.3684445291813e-01 7.8051845750430e-01
3.9430980162249e-02 9.4225760421536e-01 7.5779336154048e-01
7.0204608189786e-01 9.4767717280576e-01 7.3506299746315e-01
3.6483495939484e-01 9.5310321825246e-01 7.1232772662845e-01
2.7774623374764e-02 9.5853554237014e-01 6.8958811657557e-01
6.9084202617773e-01 9.6397371518927e-01 6.6684487068881e-01
3.5401393595350e-01 9.6941712127369e-01 6.4409874903857e-01
1.7267188872332e-02 9.7486501910252e-01 6.2135049239448e-01
6.8057932065076e-01 9.8031660435950e-01 5.9860075991696e-01
3.4392941095738e-01 9.8577106750660e-01 5.7585008788093e-01
7.2988692057865e-03 9.9122763730020e-01 5.5309887267467e-01
6.7067188668171e-01 9.9668560486527e-01 5.3034737720977e-01
6.5159974015899e-01 7.7625932265827e-01 5.0690884409505e-01
3.7047413819032e-01 7.8183178691563e-01 4.8417635837831e-01
1.0219310911271e-01 7.8721664061115e-01 4.6156680852362e-01
8.2115698678783e-01 7.9240032915662e-01 4.3913714178578e-01
5.1676260133802e-01 7.9769175348826e-01 4.1656455015429e-01
1.9672853597546e-01 8.0306649558680e-01 3.9386881489229e-01
8.6814788586261e-01 8.0846831994902e-01 3.7113144453535e-01
5.3472091248128e-01 8.1387698535433e-01 3.4838500964868e-01
1.9836823696906e-01 8.1928628644815e-01 3.2563966576428e-01
8.6015663079699e-01 8.2469435330966e-01 3.0289818429731e-01
5.2072063663292e-01 8.3010067010888e-01 2.8016103404588e-01
1.8046153627640e-01 8.3550515432750e-01 2.5742801212659e-01
8.3964741164426e-01 8.4090785645928e-01 2.3469876934881e-01
4.9846658770041e-01 8.4630885725432e-01 2.1197297507766e-01
1.5705771100210e-01 8.5170823183638e-01 1.8925036169436e-01
8.1552744353034e-01 8.5710603781632e-01 1.6653072865095e-01
4.7396124977457e-01 8.6250231274960e-01 1.4381393319950e-01
1.3243014343204e-01 8.6789707600278e-01 1.2109987661651e-01
7.9099495364415e-01 8.7329033366964e-01 9.8388488236598e-02
4.4970898603981e-01 8.7868208657523e-01 7.5679707298650e-02
1.0861958133985e-01 8.8407234194694e-01 5.2973461918196e-02
7.6776886797550e-01 8.8946112943908e-01 3.0269644575804e-02
4.2719389473655e-01 8.9484852195335e-01 7.5680841162167e-03
8.6302587902625e-02 9.0023828043936e-01 9.8486424681039e-01
7.4633652440168e-01 9.0562378524319e-01 9.6216584859516e-01
4.0672088935647e-01 9.1100862810033e-01 9.3946852574456e-01
6.7466259946704e-02 9.1639332541915e-01 9.1677157687197e-01
7.2857469939465e-01 9.2177855864031e-01 8.9407414264168e-01
3.9003963480677e-01 9.2716517032788e-01 8.7137523603313e-01
5.1846404920697e-02 9.3255413664720e-01 8.4867380078625e-01
7.1397350591675e-01 9.3794651397335e-01 8.2596879492757e-01
3.7639437024765e-01 9.4334336278673e-01 8.0325928909128e-01
3.9079338573127e-02 9.4874565782004e-01 7.8054456347637e-01
7.0199740804114e-01 9.5415419763661e-01 7.5782418521120e-01
3.6511741056578e-01 9.5956952792831e-01 7.3509805080794e-01
2.8408478111463e-02 9.6499189056085e-01 7.1236638545894e-01
6.9183991180008e-01 9.7042120572908e-01 6.8962969983097e-01
3.5538078425450e-01 9.7585708895774e-01 6.6688871317936e-01
1.8999685264332e-02 9.8129889923234e-01 6.4414425723652e-01
6.8266493790914e-01 9.8674580989788e-01 6.2139717765166e-01
3.4634539889583e-01 9.9219689058393e-01 5.9864824872071e-01
1.0011699272786e-02 9.9765118692280e-01 5.7589811319210e-01
6.7443581052688e-01 3.1001762413649e-03 5.5315597102661e-01
3.3800368271927e-01 8.5581986788005e-03 5.3040476518333e-01
3.2353215671813e-01 7.8817125990468e-01 5.0692104353088e-01
2.2602169071583e-02 7.9370195645472e-01 4.8418619569693e-01
7.3054852611838e-01 7.9917878394264e-01 4.6147898410954e-01
4.3853628693740e-01 8.0448866956963e-01 4.3890181639394e-01
1.3773218848723e-01 8.0973624673564e-01 4.1638778094083e-01
8.2491738995337e-01 8.1504110880379e-01 3.9379985259818e-01
5.0261348358220e-01 8.2040200866022e-01 3.7113131383886e-01
1.7403243748547e-01 8.2579128979741e-01 3.4841902934919e-01
8.4138037907541e-01 8.3119238059735e-01 3.2568815617528e-01
5.0602865139643e-01 8.3659767822539e-01 3.0295103179818e-01
1.6883876815132e-01 8.4200387812772e-01 2.8021317243516e-01
8.3037155565808e-01 8.4740955424529e-01 2.5747693905337e-01
4.9100697579384e-01 8.5281409636865e-01 2.3474328631218e-01
1.5101286462361e-01 8.5821725132544e-01 2.1201255442128e-01
8.1058544734576e-01 8.6361892038041e-01 1.8928482618389e-01
4.6987398024576e-01 8.6901906687203e-01 1.6656008869783e-01
1.2899618670714e-01 8.7441767303631e-01 1.4383830581887e-01
7.8804816724524e-01 8.7981472005982e-01 1.2111944841600e-01
4.4711086750250e-01 8.8521018024675e-01 9.8403503305985e-02
1.0625431029752e-01 8.9060401672785e-01 7.5690469955953e-02
7.6554029972947e-01 8.9599618916812e-01 5.2980348607682e-02
4.2502401555196e-01 9.0138666537333e-01 3.0273121021328e-02
8.4754747283608e-02 9.0677543929459e-01 7.5687241660912e-03
7.4421411837689e-01 9.1216583561043e-01 9.8486314995355e-01
4.0453058587644e-01 9.1755181428115e-01 9.6216343212667e-01
6.5203276294106e-02 9.2293649712669e-01 9.3946574985489e-01
7.2625321579617e-01 9.2832028766122e-01 9.1676950582656e-01
3.8769160646691e-01 9.3370377859827e-01 8.9407389643788e-01
4.9518975541923e-02 9.3908776960376e-01 8.7137791814964e-01
7.1172501902487e-01 9.4447326043133e-01 8.4868040837949e-01
3.7428937441289e-01 9.4986141133850e-01 8.2598012480548e-01
3.7183333508165e-02 9.5525346829000e-01 8.0327585762853e-01
7.0037221518482e-01 9.6065065881946e-01 7.8056655858823e-01
3.6381787612762e-01 9.6605407292750e-01 7.5785146252138e-01
2.7480772281086e-02 9.7146454879027e-01 7.3513017584436e-01
6.9132115440941e-01 9.7688258325149e-01 7.1240271272848e-01
3.5529933166502e-01 9.8230828227070e-01 6.8966947170766e-01
1.9375317847147e-02 9.8774135870496e-01 6.6693115880205e-01
6.8350841850783e-01 9.9318117630463e-01 6.4418867405404e-01
3.4765731693028e-01 9.9862683112128e-01 6.2144298440698e-01
1.2553354586252e-02 4.0698805542839e-03 5.9870345524574e-01
6.7662088624320e-01 9.5238543421346e-03 5.7595407720152e-01
3.4058830049546e-01 1.4980407305062e-02 5.5320376128796e-01
4.4260817797673e-03 2.0438561328608e-02 5.3045290396545e-01
-6.8586967259018e-03 8.0007557558441e-01 5.0693683823113e-01
6.8173071118707e-01 8.0558235280692e-01 4.8420049801588e-01
3.7632655946183e-01 8.1107234841638e-01 4.6147101315883e-01
7.3157711813186e-02 8.1647333191211e-01 4.3879978760569e-01
7.6782831252999e-01 8.2177879672874e-01 4.1621418242171e-01
4.5657056582560e-01 8.2706377822924e-01 3.9365107666062e-01
1.3807310867342e-01 8.3238287328042e-01 3.7104452073748e-01
8.1334436721373e-01 8.3773974547489e-01 3.4838474436447e-01
4.8398764959411e-01 8.4312103896919e-01 3.2568802160954e-01
1.5133523065228e-01 8.4851541798727e-01 3.0297057153295e-01
8.1633903388701e-01 8.5391610528605e-01 2.8024291817659e-01
4.7966008993214e-01 8.5931943568613e-01 2.5751105859270e-01
1.4176244388357e-01 8.6472348130291e-01 2.3477825303591e-01
8.0297978516410e-01 8.7012722986768e-01 2.1204624380863e-01
4.6355903128667e-01 8.7553014371503e-01 1.8931595921786e-01
1.2368851891827e-01 8.8093192848193e-01 1.6658790019473e-01
7.8351642299408e-01 8.8633241103793e-01 1.4386234968243e-01
4.4316298180010e-01 8.9173147418302e-01 1.2113948539288e-01
1.0272872847658e-01 8.9712902171794e-01 9.8419439256074e-02
7.6230007463610e-01 9.0252496089082e-01 7.5702326112071e-02
4.2195306783663e-01 9.0791919612509e-01 5.2988252977028e-02
8.1755819952693e-02 9.1331163159096e-01 3.0277314185168e-02
7.4176989996211e-01 9.1870218216897e-01 7.5695745478630e-03
4.0155279806718e-01 9.2409372288626e-01 9.8486158760799e-01
6.2116860484001e-02 9.2948080373455e-01 9.6215971334672e-01
7.2303833121110e-01 9.3486604310154e-01 9.3946073606309e-01
3.8435072134482e-01 9.4024969206847e-01 9.1676423089762e-01
4.6077156027559e-02 9.4563219734330e-01 8.9406953525804e-01
7.0822865238166e-01 9.5101424399873e-01 8.7137572296853e-01
3.7080290875214e-01 9.5639677836222e-01 8.4868161280511e-01
3.3784087511962e-02 9.6178099637583e-01 8.2598582505480e-01
6.9714391237780e-01 9.6716828596427e-01 8.0328689113665e-01
3.6084404741416e-01 9.7256012102232e-01 7.8058340662960e-01
2.4839280372990e-02 9.7795791777696e-01 7.5787420176392e-01
6.8908070463423e-01 9.8336287677058e-01 7.3515849276399e-01
3.5351808390339e-01 9.8877584021173e-01 7.1243597825778e-01
1.8100946527307e-02 9.9419719228984e-01 6.8970685786918e-01
6.8277853976523e-01 9.9962682066677e-01 6.6697176997023e-01
3.4823107716500e-01 5.0571405163868e-03 6.4423968841783e-01
1.2954415938471e-02 1.0501053682731e-02 6.2149583177338e-01
6.7761519082470e-01 1.5950503181501e-02 5.9874913264903e-01
3.4216272391692e-01 2.1404167929818e-02 5.7600054273965e-01
6.5510497013568e-03 2.6860754084702e-02 5.5325079534313e-01
6.7074159706001e-01 3.2319096244098e-02 5.3050038315328e-01
6.6168474442511e-01 8.1197534461725e-01 5.0695623166731e-01
3.4380164377327e-01 8.1746740103917e-01 4.8421835909283e-01
3.0162806873812e-02 8.2295421408373e-01 4.6148222462180e-01
7.1889777928973e-01 8.2839476958635e-01 4.3877197499054e-01
4.0766457181627e-01 8.3376266213831e-01 4.1611838967910e-01
9.4122855208325e-02 8.3907833346941e-01 3.9351581892303e-01
7.7640715382375e-01 8.4438734386046e-01 3.7092109584386e-01
4.5387864106829e-01 8.4971843133982e-01 3.4829839621933e-01
1.2700056444879e-01 8.5507575586468e-01 3.2563926224586e-01
7.9664217789027e-01 8.6045283282900e-01 3.0295075904270e-01
4.6363354249079e-01 8.6584235073614e-01 2.8024278000938e-01
1.2863702075103e-01 8.7123898194704e-01 2.5752323847863e-01
7.9215296368027e-01 8.7663934177152e-01 2.3479748472785e-01
4.5455632491636e-01 8.8204138494585e-01 2.1206890469885e-01
1.1613129417717e-01 8.8744389196376e-01 1.8933959051525e-01
7.7709760204683e-01 8.9284612953416e-01 1.6661083476374e-01
4.3762928123167e-01 8.9824764309883e-01 1.4388345281175e-01
9.7867834363308e-02 9.0364813302015e-01 1.2115798327152e-01
7.5793147836768e-01 9.0904738099216e-01 9.8434809918343e-02
4.1792165763570e-01 9.1444520659417e-01 7.5714234060218e-02
7.7927621847310e-02 9.1984144278782e-01 5.2996514835598e-02
7.3802957732824e-01 9.2523592464907e-01 3.0281887137044e-02
3.9830071971005e-01 9.3062848901796e-01 7.5705619078609e-03
5.8375863192946e-02 9.3602155376003e-01 9.8485968332054e-01
7.1914750180350e-01 9.4141027940183e-01 9.6215493487741e-01
3.8027142935311e-01 9.4679676013100e-01 9.3945375609136e-01
4.1795039275382e-02 9.5218106750417e-01 9.1675595443636e-01
7.0375579649159e-01 9.5756345250806e-01 8.9406109486980e-01
3.6617869158606e-01 9.6294441122530e-01 8.7136843920377e-01
2.9073762830174e-02 9.6832474234170e-01 8.4867691001818e-01
6.9243435875064e-01 9.7370557715734e-01 8.2598510237033e-01
3.5623684632030e-01 9.7908836027378e-01 8.0329136357722e-01
2.0442209058951e-02 9.8447476453430e-01 7.8059394743834e-01
6.8499940542573e-01 9.8986653882918e-01 7.5789122560732e-01
3.4984967005278e-01 9.9526530838668e-01 7.3518191387981e-01
1.5590733499208e-02 6.6607605277933e-04 7.1247246446272e-01
6.8085475458372e-01 6.0819976143954e-03 6.8974856937999e-01
3.4622121892457e-01 1.1507184307671e-02 6.6701765780274e-01
1.1625422736943e-02 1.6941193802746e-02 6.4428068570339e-01
6.7700142661946e-01 2.2383047153634e-02 6.2153889190213e-01
3.4228271502310e-01 2.7831382432121e-02 5.9879357929771e-01
7.4045481967802e-03 3.3284646416645e-02 5.7604592417677e-01
6.7230763171844e-01 3.8741284117993e-02 5.5329684981095e-01
3.3694201521814e-01 4.4199889776810e-02 5.3054697859434e-01
3.2965098752624e-01 8.2387186274324e-01 5.0697911428449e-01
7.3780916283815e-03 8.2935427263865e-01 4.8423968640025e-01
6.8823364210365e-01 8.3483539697283e-01 4.6150065682927e-01
3.7119100601466e-01 8.4029129736726e-01 4.3877398662031e-01
5.4881107500807e-02 8.4569895329520e-01 4.1608045807869e-01
7.3788960841583e-01 8.5105575011756e-01 3.9343110292183e-01
4.1885072312630e-01 8.5638305041369e-01 3.7081219752506e-01
9.6744400670550e-02 8.6170869853520e-01 3.4819545563915e-01
7.7122044135696e-01 8.6704951066369e-01 3.2555949402901e-01
4.4251530248228e-01 8.7240913834643e-01 3.0289763914398e-01
1.1113450736726e-01 8.7778445972432e-01 2.8021275811233e-01
7.7761428268503e-01 8.8317084698180e-01 2.5751077890374e-01
4.4242225974057e-01 8.8856434263169e-01 2.3479734322919e-01
1.0593791336307e-01 8.9396208905923e-01 2.1207681603845e-01
7.6846249114420e-01 8.9936216030215e-01 1.8935229882514e-01
4.3023543669145e-01 9.0476329077499e-01 1.6662591978206e-01
9.1449532561619e-02 9.1016464799015e-01 1.4389913001507e-01
7.5226298824720e-01 9.1556567071426e-01 1.2117293502994e-01
4.1280861122939e-01 9.2096596043598e-01 9.8448058899966e-02
7.3200636263216e-02 9.2636521029448e-01 7.5725055370738e-02
7.3353976575107e-01 9.3176315932054e-01 5.3004380314687e-02
3.9391683949249e-01 9.3715956425529e-01 3.0286435954510e-02
5.4415410433102e-02 9.4255418481843e-01 7.5715930608177e-03
7.1474839950383e-01 9.4794897877151e-01 9.8485760575495e-01
3.7568392373555e-01 9.5333974767443e-01 9.6214946289632e-01
3.6958865681509e-02 9.5872805608523e-01 9.3944528931393e-01
6.9863495144577e-01 9.6411378803132e-01 9.1674516363243e-01
3.6076556017679e-01 9.6949696784619e-01 8.9404894935778e-01
2.3392659557595e-02 9.7487783994934e-01 8.7135620592680e-01
6.8654314944101e-01 9.8025695758929e-01 8.4866610449343e-01
3.5022528764905e-01 9.8563526180337e-01 8.2597737629569e-01
1.4426193704472e-02 9.9101412159334e-01 8.0328832885717e-01
6.7911149030854e-01 9.9639530213985e-01 7.8059696076677e-01
3.4481420453265e-01 1.7752360409847e-03 7.5790760065581e-01
1.0313516218069e-02 7.1669516176118e-03 7.3520581926805e-01
6.7609928199187e-01 1.2566966713456e-02 7.1249635491241e-01
3.4209184904159e-01 1.7976620860454e-02 6.8977856270009e-01
8.2107090038891e-03 2.3396540585598e-02 6.6705260019713e-01
6.7437477380933e-01 2.8826554318598e-02 6.4431933615046e-01
3.4050164263184e-01 3.4265743575332e-02 6.2158012560262e-01
6.5077906987083e-03 3.9712614303506e-02 5.9883651873453e-01
6.7231102936123e-01 4.5165345220502e-02 5.7608998187341e-01
3.3783519580396e-01 5.0622053840033e-02 5.5334169540560e-01
3.0156813988371e-03 5.6081021113644e-02 5.3059246131115e-01
-2.7310784446524e-03 8.3576580311529e-01 5.0700532504217e-01
6.7182764353528e-01 8.4124160487793e-01 4.8426436994783e-01
3.4877303221736e-01 8.4671751031087e-01 4.6152351440216e-01
2.7496993694992e-02 8.5217880974270e-01 4.3878910233126e-01
7.0714719438473e-01 8.5760865116157e-01 4.1607402139524e-01
3.8681519363544e-01 8.6299818409551e-01 3.9339037232088e-01
6.5594175169475e-02 8.6835324226784e-01 3.7073884163902e-01
7.4263668172627e-01 8.7369083068954e-01 3.4810600412908e-01
4.1734275664298e-01 8.7902865181799e-01 3.2547302763199e-01
8.9504543446065e-02 8.8437735251482e-01 3.0282619792883e-01
7.5925403726133e-01 8.8973993996302e-01 2.8016034410383e-01
4.2690381035240e-01 8.9511498475787e-01 2.5747638003298e-01
9.2813404816470e-02 9.0049957436489e-01 2.3477782871864e-01
7.5731979592895e-01 9.0589084667238e-01 2.1206861861864e-01
4.2071369013547e-01 9.1128650202821e-01 1.8935215426728e-01
8.3238014393401e-02 9.1668484364082e-01 1.6663109619726e-01
7.4509445278148e-01 9.2208466350798e-01 1.4390743109566e-01
4.0645166355646e-01 9.2748510675456e-01 1.2118262304738e-01
6.7452840132457e-02 9.3288555678699e-01 9.8457762741925e-02
7.2822199141225e-01 9.3828554839374e-01 7.5733689471594e-02
3.8886893917455e-01 9.4368470503358e-01 5.3011083031891e-02
4.9493202069772e-02 9.4908269485316e-01 3.0290529228961e-02
7.1018693637994e-01 9.5447920125268e-01 7.5725631317851e-03
3.7074140069168e-01 9.5987572192576e-01 9.8485555621301e-01
3.1795111243953e-02 9.6526873808127e-01 9.6214376689959e-01
6.9316667774771e-01 9.7065929413489e-01 9.3943601065576e-01
3.5493148779155e-01 9.7604710096365e-01 9.1673264475069e-01
1.7159149613789e-02 9.8143195345230e-01 8.9403386890502e-01
6.7991044843630e-01 9.8681380994193e-01 8.7133962402837e-01
3.4323300432277e-01 9.9219289854875e-01 8.4864947034688e-01
7.1560300931929e-03 9.9756983934287e-01 8.2596246787680e-01
6.7218329564705e-01 2.9409758193932e-03 8.0328257635436e-01
3.3732055231621e-01 8.3172464069934e-03 7.8059711583407e-01
2.9926755383746e-03 1.3696374997480e-02 7.5790874942187e-01
6.6912670766462e-01 1.9080857940202e-02 7.3521496154225e-01
3.3563291480686e-01 2.4473173976602e-02 7.1251359398048e-01
2.4127329576081e-03 2.9875389842471e-02 6.8980324315355e-01
6.6936467181807e-01 3.5288802401719e-02 6.6708350540676e-01
3.3638660586904e-01 4.0713716033220e-02 6.4435498866323e-01
3.3753115633443e-03 4.6149413058627e-02 6.2161909282565e-01
6.7022596706153e-01 5.1594319258250e-02 5.9887763832404e-01
3.3683433260062e-01 5.7046314978420e-02 5.7613246233730e-01
3.1024754474996e-03 6.2503106272890e-02 5.5338509670696e-01
6.6894647757884e-01 6.7962560177886e-02 5.3063659744634e-01
6.6465984064974e-01 8.4765757959104e-01 5.0703467228215e-01
3.3682986512507e-01 8.5312871426114e-01 4.8429225294111e-01
1.0870080022767e-02 8.5860038612292e-01 4.6154993630441e-01
6.8639790603038e-01 8.6406308216567e-01 4.3881130704703e-01
3.6285802092247e-01 8.6950489222444e-01 4.1608434252052e-01
3.9635858488732e-02 8.7491670281177e-01 3.9337860421660e-01
7.1611463848963e-01 8.8029727710275e-01 3.7069935541600e-01
3.9168298907854e-01 8.8565431755146e-01 3.4804309148152e-01
6.5791988752805e-02 8.9100051075894e-01 3.2539868449337e-01
7.3806868774653e-01 8.9634761054824e-01 3.0275321575616e-01
4.0838311386829e-01 9.0170277611707e-01 2.8009738647001e-01
7.6815485549546e-02 9.0706844612713e-01 2.5742717448991e-01
7.4357012356240e-01 9.1244406349049e-01 2.3474257971251e-01
4.0889721808851e-01 9.1782780286225e-01 2.1204567200556e-01
7.3045438258486e-02 9.2321762146215e-01 1.8933915702026e-01
7.3624084908585e-01 9.2861171808138e-01 1.6662562785738e-01
3.9868120931855e-01 9.3400865628718e-01 1.4390728375312e-01
6.0537430088100e-02 9.3940734074149e-01 1.2118588947682e-01
7.2195765606173e-01 9.4480694613851e-01 9.8462834052903e-02
3.8307186096265e-01 9.5020684149811e-01 7.5739217788183e-02
4.3996115974116e-02 9.5560652451909e-01 5.3015937193411e-02
7.0483626398894e-01 9.6100556899392e-01 3.0293757018550e-02
3.6569095659718e-01 9.6640358440242e-01 7.5733666815391e-03
2.6429629326788e-02 9.7180161024296e-01 9.8485374900059e-01
6.8755544711060e-01 9.7719685104214e-01 9.6213837900024e-01
3.4896862671657e-01 9.8258985190303e-01 9.3942674318627e-01
1.0756453975602e-02 9.8798018557190e-01 9.1671944937865e-01
6.7300361354961e-01 9.9336744641084e-01 8.9401702504686e-01
3.3578980381395e-01 9.9875131343586e-01 8.7131982559317e-01
-4.1924294050634e-04 4.1278273283106e-03 8.4863228080411e-01
6.6366855179308e-01 9.5044881496598e-03 8.2594559300962e-01
3.2845302747457e-01 1.4878531964617e-02 8.0326265533956e-01
-6.0637861034157e-03 2.0251318744309e-02 7.8058162624468e-01
6.6008451378239e-01 2.5624947107722e-02 7.5789993260141e-01
3.2682844028148e-01 3.1002166324942e-02 7.3521452437721e-01
-5.9314407805906e-03 3.6386071182757e-02 7.1252235704969e-01
6.6168597709703e-01 4.1779608823747e-02 6.8982099202017e-01
3.2955307473741e-01 4.7185021088198e-02 6.6710911367631e-01
-2.4598451786241e-03 5.2603393502205e-02 6.4438676684053e-01
6.6551680483075e-01 5.8034452161070e-02 6.2165522781949e-01
3.3335086157938e-01 6.3476664794996e-02 5.9891657006918e-01
9.0971203535816e-04 6.8927605805081e-02 5.7617309328598e-01
6.6806681991980e-01 7.4384467690906e-02 5.5342681223617e-01
3.3471228615930e-01 7.9844564120846e-02 5.3067914889037e-01
3.3189387407461e-01 8.5954748432660e-01 5.0706694342633e-01
2.2056093524533e-03 8.6501525398519e-01 4.8432314501469e-01
6.7401285294072e-01 8.7048362176430e-01 4.6157953041620e-01
3.4706286048386e-01 8.7594614774923e-01 4.3883824053801e-01
2.0980374839717e-02 8.8139438297569e-01 4.1610432352398e-01
6.9533473477213e-01 8.8682058088612e-01 3.9338470495733e-01
3.6968847136676e-01 8.9222098668101e-01 3.7068515140104e-01
4.3605674861233e-02 8.9759773536847e-01 3.4800685454264e-01
7.1666037102915e-01 9.0295808293523e-01 3.2534508943136e-01
3.8848514531262e-01 9.0831139223041e-01 3.0269108027098e-01
5.8838294685655e-02 9.1366578691929e-01 2.8003568125841e-01
7.2763603712684e-01 9.1902631450094e-01 2.5737230313725e-01
3.9493156394078e-01 9.2439495554648e-01 2.3469778136049e-01
6.0865609473862e-02 9.2977160889553e-01 2.1201169761006e-01
7.2561880356241e-01 9.3515514184285e-01 1.8931523732153e-01
3.8937945475359e-01 9.4054410771581e-01 1.6661025127137e-01
5.2326606301959e-02 9.4593711861079e-01 1.4389868816584e-01
7.1462341634506e-01 9.5133298904161e-01 1.2118232582949e-01
3.7641622266567e-01 9.5673075876700e-01 9.8462684202469e-02
3.7836267051915e-02 9.6212966311682e-01 7.5741030990108e-02
6.9900242381950e-01 9.6752908627279e-01 5.3018426696014e-02
3.6002412725010e-01 9.7292851333014e-01 3.0295781376680e-02
2.1004144038619e-02 9.7832748707162e-01 7.5739101447914e-03
6.8188967697603e-01 9.8372659237113e-01 9.8485238729296e-01
3.4304246469109e-01 9.8912380257976e-01 9.6213383712024e-01
4.4436978508362e-03 9.9451918995184e-01 9.3941837737948e-01
6.6618993922853e-01 9.9991223802612e-01 9.1670680648893e-01
3.2865619634005e-01 5.2996565265783e-03 8.9400305881858e-01
-8.5854511355830e-03 1.0685984339654e-02 8.7130204299521e-01
6.5480444531084e-01 1.6068416171212e-02 8.4860699280984e-01
3.1890973248944e-01 2.1446687309803e-02 8.2591810984063e-01
-1.6203546499938e-02 2.6820937599671e-02 8.0323499869952e-01
6.4950189727557e-01 3.2191957093854e-02 7.8055641076203e-01
3.1602332878444e-01 3.7561421369952e-02 7.5788006787938e-01
-1.6689727916346e-02 4.2932002587754e-02 7.3520270955023e-01
6.5126466930306e-01 4.8307218700663e-02 7.1252048872342e-01
3.1975062259453e-01 5.3690943549770e-02 6.8982971002507e-01
-1.1389840361583e-02 5.9086652241494e-02 6.6712769790250e-01
6.5767745278767e-01 6.4496635456160e-02 6.4441344585351e-01
3.2678549669706e-01 6.9921465914962e-02 6.2168776136210e-01
-4.2340502345590e-03 7.5359906168615e-02 5.9895285545317e-01
6.6444895019759e-01 8.0809277033886e-02 5.7621157399475e-01
3.3266971903824e-01 8.6266143359091e-02 5.5346659440447e-01
2.8266006688626e-04 9.1727071838340e-02 5.3071987464827e-01
-9.8572208458823e-04 8.7143574694096e-01 5.0710191115024e-01
6.6784721094021e-01 8.7690105756339e-01 4.8435683305583e-01
3.3789377958596e-01 8.8236688424927e-01 4.6161202569916e-01
8.9801906113878e-03 8.8782868974051e-01 4.3886883179662e-01
6.8084494072301e-01 8.9328037946902e-01 4.1613053965877e-01
3.5317852183434e-01 8.9871575943745e-01 3.9340205199234e-01
2.5660234881520e-02 9.0413057424197e-01 3.7068835094635e-01
6.9797128478411e-01 9.0952416299726e-01 3.4799230107835e-01
3.6979399648376e-01 9.1489984998677e-01 3.2531300046473e-01
4.0824936839676e-02 9.2026381535767e-01 3.0264574970794e-01
7.1081178008538e-01 9.2562302327481e-01 2.7998371303007e-01
3.7959308523617e-01 9.3098329067697e-01 2.5732022224394e-01
4.7114275960725e-02 9.3634831505248e-01 2.3465043768564e-01
7.1341394592855e-01 9.4171971811669e-01 2.1197183433052e-01
3.7859404137706e-01 9.4709763691567e-01 1.8928381677931e-01
4.2789823092758e-02 9.5248138474907e-01 1.6658702584873e-01
7.0614781126407e-01 9.5786994463452e-01 1.4388271699104e-01
3.6881275088373e-01 9.6326225935427e-01 1.2117234095071e-01
3.0921440689287e-02 9.6865736781838e-01 9.8457313368746e-02
6.9260085546108e-01 9.7405444826796e-01 7.5738915832911e-02
3.5396862092276e-01 9.7945281301426e-01 5.3018274616732e-02
1.5134632265918e-02 9.8485188148206e-01 3.0296379934502e-02
6.7620314448386e-01 9.9025114554639e-01 7.5741235406603e-03
3.3719855918024e-01 9.9565074166410e-01 9.8485163834881e-01
-1.5541737806576e-03 1.0483834145468e-03 9.6213185062398e-01
6.5981952476094e-01 6.4453912452205e-03 9.3941350947175e-01
3.2149769127664e-01 1.1840606640124e-02 9.1669822403928e-01
-1.6413373868923e-02 1.7233406358165e-02 8.9398696827919e-01
6.4619645664971e-01 2.2623098018672e-02 8.7128076249086e-01
3.0943982397539e-01 2.8008954060554e-02 8.4858060440779e-01
-2.6571928641992e-02 3.3390299919963e-02 8.2588732605319e-01
6.3826379067772e-01 3.8766683499172e-02 8.0320135261553e-01
3.0402883427340e-01 4.4138146527174e-02 7.8052236062246e-01
-2.9231814659317e-02 4.9505584038360e-02 7.5784889044501e-01
6.3847282332227e-01 5.4871101833810e-02 7.3517806615794e-01
3.0706809056463e-01 6.0238180460706e-02 7.1250566729444e-01
-2.3585035032049e-02 6.5611408944360e-02 6.8982676562449e-01
6.4632644873452e-01 7.0995674624331e-02 6.6713688467484e-01
3.1659142415026e-01 7.6394989912606e-02 6.4443325894065e-01
-1.3007277642703e-02 8.1811404110941e-02 6.2171558423624e-01
6.5731239024931e-01 8.7244461300207e-02 5.9898587871005e-01
3.2733040875053e-01 9.2691419073432e-02 5.7624755517488e-01
-3.1700798582917e-03 9.8148113000552e-02 5.5350418828034e-01
6.6561522043045e-01 1.0361009564023e-01 5.3075853295816e-01
6.6604908555453e-01 8.8332256224672e-01 5.0713933817385e-01
3.3368631252094e-01 8.8878606155644e-01 4.8439308876972e-01
2.3180990483450e-03 8.9424994751773e-01 4.6164717989381e-01
6.7182091719129e-01 8.9971090919787e-01 4.3890249238500e-01
3.4200767593999e-01 9.0516445746321e-01 4.1616123401986e-01
1.2651471376143e-02 9.1060573321939e-01 3.9342687909874e-01
6.8351181860746e-01 9.1603076154175e-01 3.7070339867853e-01
3.5434883023087e-01 9.2143768249901e-01 3.4799393092496e-01
2.4924066093442e-02 9.2682742352517e-01 3.2529945118570e-01
6.9500140132569e-01 9.3220349354452e-01 3.0261812505320e-01
3.6436125830158e-01 9.3757096474086e-01 2.7994572769204e-01
3.2826923450234e-02 9.4293509339703e-01 2.5727692165457e-01
7.0028840767778e-01 9.4830017502760e-01 2.3460671471379e-01
3.6671057459462e-01 9.5366900523359e-01 2.1193144446815e-01
3.2123983359105e-02 9.5904292634354e-01 1.8924906603201e-01
6.9660616142152e-01 9.6442219712527e-01 1.6655892456942e-01
3.6026210021257e-01 9.6980642177870e-01 1.4386132078230e-01
2.3209003661424e-02 9.7519489325409e-01 1.2115709304406e-01
6.8556647388970e-01 9.8058681476020e-01 9.8447310538440e-02
3.4745127202639e-01 9.8598141854699e-01 7.5733085916991e-02
8.9752559090074e-03 9.9137801504502e-01 5.3015481133358e-02
6.7024528205221e-01 9.9677600068929e-01 3.0295475244374e-02
3.3140231500649e-01 2.1745374859902e-03 7.5743235965465e-03
-7.5673343842433e-03 7.5740580000288e-03 9.8485181256479e-01
6.5354951622398e-01 1.2973175739416e-02 9.6212983454158e-01
3.1481632407250e-01 1.8371708530894e-02 9.3940889628198e-01
-2.3661285346997e-02 2.3769115565446e-02 9.1668992839038e-01
6.3822811149224e-01 2.9164771321156e-02 8.9397396710056e-01
3.0060333615734e-01 3.4557939403254e-02 8.7126217597703e-01
-3.6408690911478e-02 3.9947763226129e-02 8.4855583583229e-01
6.2732594594257e-01 4.5333293431431e-02 8.2585627470446e-01
2.9194378645993e-01 5.0713585563498e-02 8.0316469649691e-01
-4.2425652096565e-02 5.6087915188946e-02 7.8048186060926e-01
6.2432324938204e-01 6.1456156230065e-02 7.5780758839844e-01
2.9224799969433e-01 6.6819319232761e-02 7.3514016323777e-01
-3.8668811651864e-02 7.2180112294844e-02 7.1247587091071e-01
6.3146244525935e-01 7.7543190024857e-02 6.8980911433256e-01
3.0244238809378e-01 8.2914665434142e-02 6.6713347981532e-01
-2.5989101094575e-02 8.8300725538922e-02 6.4444360880624e-01
6.4588313331612e-01 9.3705785488100e-02 6.2173700719647e-01
3.1777066720137e-01 9.9131050454494e-02 5.9901473704986e-01
-1.0616414552603e-02 1.0457420175104e-01 5.7628059573646e-01
6.6043255561125e-01 1.1003032819213e-01 5.5353932665579e-01
3.3064993050984e-01 1.1549360869059e-01 5.3079488410287e-01
3.3301717663441e-01 8.9520810400446e-01 5.0717898129220e-01
-3.2237185983692e-04 9.0067026438495e-01 4.8443167431058e-01
6.6715722482609e-01 9.0613267545903e-01 4.6168475444195e-01
3.3536571620043e-01 9.1159284430239e-01 4.3893882525317e-01
4.1662125016822e-03 9.1704739134474e-01 4.1619541409962e-01
6.7338959973134e-01 9.2249251118530e-01 3.9345701105309e-01
3.4285269568962e-01 9.2792475008817e-01 3.7072668719366e-01
1.2370988933896e-02 9.3334186999557e-01 3.4800730814988e-01
6.8176195980401e-01 9.3874348829395e-01 3.2530059164196e-01
3.5084383719046e-01 9.4413124494372e-01 3.0260639619366e-01
1.9438966215197e-02 9.4950841727241e-01 2.7992257577655e-01
6.8738663753165e-01 9.5487911749841e-01 2.5724548926218e-01
3.5456137164047e-01 9.6024737971790e-01 2.3457093307686e-01
2.0887492063661e-02 9.6561646199846e-01 2.1189508054451e-01
6.8634333658006e-01 9.7098853509793e-01 1.8921508860227e-01
3.5095501260024e-01 9.7636472317492e-01 1.6652927616863e-01
1.4784244604354e-02 9.8174534344317e-01 1.4383698508841e-01
6.7791539504997e-01 9.8713019243034e-01 1.2113830008701e-01
3.4044481346906e-01 9.9251878811560e-01 9.8433763459217e-02
2.4735390510227e-03 9.9791053795423e-01 7.5724150928734e-02
6.6428382175438e-01 3.3032014156813e-03 5.3012204527650e-02
3.2558080377667e-01 8.6999324934434e-03 3.0294519233549e-02
-1.3323332366105e-02 1.4098144045264e-02 7.5742985057671e-03
6.4759132186173e-01 1.9497607703078e-02 9.8485203209179e-01
3.0861391347287e-01 2.4896887610754e-02 9.6212968178175e-01
-3.0269102579019e-02 3.0296255730051e-02 9.3940720272354e-01
6.3104768976492e-01 3.5695218719723e-02 9.1668546917881e-01
2.9267694784873e-01 4.1093205843957e-02 8.9396547089804e-01
-4.5258819301124e-02 4.6489526364145e-02 8.7124836408384e-01
6.1737520044027e-01 5.1883327937453e-02 8.4853551319372e-01
2.8072781431815e-01 5.7273567678178e-02 8.2582850282586e-01
-5.5038455428985e-02 6.2659021637590e-02 8.0312908031963e-01
6.1024816790507e-01 6.8038382262311e-02 7.8043896293952e-01
2.7675661474037e-01 7.3410524253000e-02 7.5775941907199e-01
-5.5370847262248e-02 7.8775034719341e-02 7.3509054936299e-01
6.1394420279591e-01 8.4133039351370e-02 7.1243033331149e-01
2.8467287860426e-01 8.9488116127798e-02 6.8977384670119e-01
-4.3349998886493e-02 9.4846684709702e-02 6.6711345681668e-01
6.2958431807061e-01 1.0021707030099e-01 6.4444069236831e-01
3.0310849454520e-01 1.0560705296727e-01 6.2174934738492e-01
-2.3167538414800e-02 1.1102095845537e-01 5.9903798372827e-01
6.5036448270483e-01 1.1645798099407e-01 5.7631006753099e-01
3.2331139767381e-01 1.2191271813097e-01 5.5357171518441e-01
-4.7003324178702e-03 1.2737752600795e-01 5.3082869368807e-01
-6.8348429350991e-05 9.0709253199129e-01 5.0722059479962e-01
6.6579014548003e-01 9.1255370286903e-01 4.8447234670918e-01
3.3232338897793e-01 9.1801499893976e-01 4.6172450893872e-01
-5.3608352556188e-04 9.2347448895767e-01 4.3897751366492e-01
6.6710966272929e-01 9.2892957337650e-01 4.1623244901539e-01
3.3513283435156e-01 9.3437725325818e-01 3.9349112857901e-01
3.3923199423249e-03 9.3981461839475e-01 3.7075590738902e-01
6.7174409193285e-01 9.4523944109191e-01 3.4802922064140e-01
3.4004576270060e-01 9.5065069749536e-01 3.2531295058888e-01
8.1562118844565e-03 9.5604884632480e-01 3.0260783031064e-01
6.7593524296186e-01 9.6143576617861e-01 2.7991311305013e-01
3.4324822552383e-01 9.6681436150564e-01 2.5722665105509e-01
9.9766353304653e-03 9.7218796278055e-01 2.3454536256093e-01
6.7603064685214e-01 9.7755971845576e-01 2.1186590001289e-01
3.4135822088249e-01 9.8293215793115e-01 1.8918526789578e-01
5.9473166986669e-03 9.8830700553500e-01 1.6650120956539e-01
6.6982156745598e-01 9.9368521381405e-01 1.4381232224498e-01
3.3303218873575e-01 9.9906712382451e-01 1.2111796945838e-01
-4.0418043105413e-03 4.4498214772975e-03 9.8421356650429e-02
6.5803290961599e-01 9.8390729072467e-03 7.5715834037551e-02
3.1968191251174e-01 1.5231218297220e-02 5.3005648756790e-02
-1.9004943758578e-02 2.0625791659808e-02 3.0291451864487e-02
6.4206335189930e-01 2.6022336082587e-02 7.5739290884227e-03
3.0282625056373e-01 3.1421086866567e-02 9.8485295995697e-01
-3.6282698614404e-02 3.6819817687778e-02 9.6213139252217e-01
6.2464530094985e-01 4.2219261696835e-02 9.3940859287188e-01
2.8571260333707e-01 4.7618994674414e-02 9.1668531855431e-01
-5.2970434771015e-02 5.3018536106996e-02 8.9396242796726e-01
6.0871771027998e-01 5.8417303340051e-02 8.7124094247728e-01
2.7091336238603e-01 6.3814555454743e-02 8.4852211888053e-01
-6.6228463300697e-02 6.9209329259960e-02 8.2580752369733e-01
5.9746946862457e-01 7.4600378090396e-02 8.0309908679693e-01
2.6220827312115e-01 7.9986143676861e-02 7.8039908321282e-01
-7.1790465339159e-02 8.5364830708722e-02 7.5770994225127e-01
5.9570098923109e-01 9.0734717534664e-02 7.3503371871029e-01
2.6488300149914e-01 9.6094897543738e-02 7.1237104140247e-01
-6.4130042891831e-02 1.0144658116284e-01 6.8971955401669e-01
6.0861670307215e-01 1.0679465062527e-01 6.6707252296909e-01
2.8286990857716e-01 1.1214828815949e-01 6.4441919623942e-01
-4.1806862945838e-02 1.1751908105149e-01 6.2174826247825e-01
6.3405712491168e-01 1.2291652365577e-01 5.9905311535782e-01
3.0991511130446e-01 1.2834353608901e-01 5.7633493917865e-01
-1.4781249746358e-02 1.3379522212853e-01 5.5360099125662e-01
6.5943401558328e-01 1.3926167559420e-01 5.3085973564770e-01
6.6680180906872e-01 9.1897599572571e-01 5.0726393330980e-01
3.3200207858651e-01 9.2443643811668e-01 4.8451486140317e-01
-2.2449070444401e-03 9.2989689682690e-01 4.6176620146869e-01
6.6400945126861e-01 9.3535583576279e-01 4.3901827716880e-01
3.3068787230585e-01 9.4081121349919e-01 4.1627188642695e-01
-2.3072938890386e-03 9.4626064276018e-01 3.9352837467148e-01
6.6491437392843e-01 9.5170170097739e-01 3.7078955205066e-01
3.3223942392312e-01 9.5713233589522e-01 3.4805742397848e-01
-4.4476017269699e-04 9.6255126432009e-01 3.2533377581821e-01
6.6675081647329e-01 9.6795825288061e-01 3.0261972121464e-01
3.3371583544878e-01 9.7335419784956e-01 2.7991535685016e-01
3.4151095737351e-04 9.7874097479479e-01 2.5721964625143e-01
6.6652543575744e-01 9.8412109522832e-01 2.3453058191491e-01
3.3217983435591e-01 9.8949726794103e-01 2.1184557269144e-01
-2.7597495745617e-03 9.9487198955041e-01 1.8916192091012e-01
6.6167134264262e-01 1.0002472639514e+00 1.6647723724028e-01
3.2589093003694e-01 5.6205748420115e-03 1.4379418613595e-01
-1.0950779253834e-02 1.1000923576516e-02 1.2110219176904e-01
6.5163881169875e-01 1.6384399951144e-02 9.8405527857730e-02
3.1371941970696e-01 2.1771016765398e-02 7.5704046261127e-02
-2.4638811175021e-02 2.7160579547678e-02 5.2997916621992e-02
6.3664078975643e-01 3.2552783682851e-02 3.0287505262615e-02
2.9763901474282e-01 3.7947279727202e-02 7.5732856927298e-03
-4.1785811823507e-02 4.3344789074685e-02 9.8485448649658e-01
6.1893816320194e-01 4.8742336773083e-02 9.6213477422310e-01
2.7964159074679e-01 5.4141119183177e-02 9.3941290300726e-01
-5.9578798692822e-02 5.9540790473012e-02 9.1668947669490e-01
6.0138197533662e-01 6.4940973567980e-02 8.9396517203904e-01
2.6264042547675e-01 7.0341221990834e-02 8.7124079426791e-01
-7.5671223849802e-02 7.5740966400747e-02 8.4851735438549e-01
5.8660014471541e-01 8.1139442115170e-02 8.2579617019746e-01
2.4963470405793e-01 8.6535595122609e-02 8.0307898826385e-01
-8.6353633701798e-02 9.1927971786366e-02 7.8036810819660e-01
5.7888835502062e-01 9.7314621493577e-02 7.5766645089147e-01
2.4565333939854e-01 1.0269310167163e-01 7.3497742722337e-01
-8.5741407147831e-02 1.0806079746161e-01 7.1230431898823e-01
5.8499957158611e-01 1.1341594118554e-01 6.8964874982055e-01
2.5805492734657e-01 1.1875972099664e-01 6.6700805590710e-01
-6.6644671493143e-02 1.2409906694537e-01 6.4437267670562e-01
6.1049741518061e-01 1.2944769417836e-01 6.2172688041457e-01
2.8880509995000e-01 1.3482201370600e-01 5.9905556651849e-01
-3.2505226961666e-02 1.4023260369695e-01 5.7635326868901e-01
6.4577198986809e-01 1.4567790192406e-01 5.5362661056874e-01
3.2284990770819e-01 1.5114575507717e-01 5.3088779244876e-01
3.3363435540560e-01 9.3085863650037e-01 5.0730875393513e-01
-1.7021653139260e-03 9.3631854669006e-01 4.8455897497958e-01
6.6340837655668e-01 9.4177838198678e-01 4.6180959050956e-01
3.2892620292925e-01 9.4723688998811e-01 4.3906085391516e-01
-5.2079010292735e-03 9.5269243552758e-01 4.1631336707197e-01
6.6093083713853e-01 9.5814309575463e-01 3.9356814775447e-01
3.2725689524866e-01 9.6358685990902e-01 3.7082659874091e-01
-6.3195031363984e-03 9.6902190646111e-01 3.4809035132014e-01
6.6011187261300e-01 9.7444689918523e-01 3.2536099566407e-01
3.2646275627848e-01 9.7986123098397e-01 3.0263975427639e-01
-7.3541221195431e-03 9.8526515353172e-01 2.7992718410877e-01
6.5857449840110e-01 9.9065975688058e-01 2.5722299555844e-01
3.2416402917437e-01 9.9604679948401e-01 2.3452604826872e-01
-1.0119242574102e-02 1.4232757804777e-03 2.1184043689277e-01
6.5454902592655e-01 6.8020064287697e-03 1.8915183468929e-01
3.1868852297406e-01 1.2179566245495e-02 1.6646426698191e-01
-1.7724096503637e-02 1.7557570896333e-02 1.4377579572612e-01
6.4531121430638e-01 2.2937170920437e-02 1.2108491572586e-01
3.0781576671429e-01 2.8319072998085e-02 9.8390607010271e-02
-3.0171637671836e-02 3.3703617770802e-02 7.5692297190891e-02
6.3140109159781e-01 3.9090877917297e-02 5.2989774873370e-02
2.9259560446440e-01 4.4480750995212e-02 3.0283089736802e-02
-4.6519669733096e-02 4.9873034042001e-02 7.5724622502636e-03
6.1383214906157e-01 5.5268954748984e-02 9.8485644481917e-01
2.7435535447450e-01 6.0664830494629e-02 9.6213946740792e-01
-6.5197817310489e-02 6.6062319923457e-02 9.3941966444750e-01
5.9526147029009e-01 7.1461156659031e-02 9.1669747180691e-01
2.5583018982122e-01 7.6861063237431e-02 8.9397336973892e-01
-8.3383496847092e-02 8.2261726956758e-02 8.7124791963404e-01
5.7774394533025e-01 8.7662760503976e-02 8.4852182512997e-01
2.3935702677895e-01 9.3063641913077e-02 8.2579602468076e-01
-9.8371363087153e-02 9.8463625171394e-02 8.0307182548335e-01
5.6476963690674e-01 1.0386161034314e-01 7.8035108814224e-01
2.2904070943587e-01 1.0925596501424e-01 7.5763645916362e-01
-1.0523475975129e-01 1.1464431152280e-01 7.3493160219552e-01
5.6233814305381e-01 1.2002337485399e-01 7.1224124936430e-01
2.3221597280651e-01 1.2538920325142e-01 6.8957060244454e-01
-9.5144194111172e-02 1.3073850094426e-01 6.6692317196223e-01
5.8055703477988e-01 1.3607213305007e-01 6.4429620257543e-01
2.5921207887440e-01 1.4140043831984e-01 6.2167546556105e-01
-5.9855942968090e-02 1.4674499014002e-01 5.9903690785159e-01
6.2226025894337e-01 1.5212908391745e-01 5.7636097825664e-01
3.0435349661898e-01 1.5756129328322e-01 5.5364752061351e-01
-1.4785056765404e-02 1.6302927150629e-01 5.3091264309566e-01
4.3416954563167e-04 9.4274058858595e-01 5.0735481779335e-01
6.6466612154667e-01 9.4820011495639e-01 4.8460444716340e-01
3.2925138008872e-01 9.5365949137584e-01 4.6185443692121e-01
-5.8412274118378e-03 9.5911767256976e-01 4.3910499367410e-01
6.5934256131653e-01 9.6457332399515e-01 4.1635658379192e-01
3.2474453060887e-01 9.7002487773462e-01 3.9360999251345e-01
-9.7021716563192e-03 9.7547066172245e-01 3.7086631947263e-01
6.5593138834203e-01 9.8090908943750e-01 3.4812688898161e-01
3.2157350191144e-01 9.8633887646345e-01 3.2539307575658e-01
-1.2846413874075e-02 9.9175923866699e-01 3.0266607415214e-01
6.5260221211957e-01 9.9717002794909e-01 2.7994666115502e-01
3.1847577580561e-01 2.5658570875579e-03 2.5724179448946e-01
-1.6563076129658e-02 7.9599525061133e-03 2.3453712019829e-01
6.4806308549697e-01 1.3347757804801e-02 2.1183854645793e-01
3.1229495446778e-01 1.8731100465411e-02 1.8914446775626e-01
-2.3915579946071e-02 2.4111809515101e-02 1.6645305109176e-01
6.3939881909219e-01 2.9491522396657e-02 1.4376248297413e-01
3.0222347391183e-01 3.4871561685753e-02 1.2107116767142e-01
-3.5437938746655e-02 4.0252888895670e-02 9.8377845398228e-02
6.2643511737648e-01 4.5636121568404e-02 7.5681628900927e-02
2.8787742607807e-01 5.1021589524201e-02 5.2981977746287e-02
-5.1064902812453e-02 5.6409406455318e-02 3.0278637061625e-02
6.0966295865369e-01 6.1799539493858e-02 7.5715645655037e-03
2.6975672361975e-01 6.7193735522593e-02 9.8485863211566e-01
-6.9952328323175e-02 7.2587642383653e-02 9.6214499303509e-01
5.9020967812995e-01 7.7983378470214e-02 9.3942816818468e-01
2.5032174739058e-01 8.3380754386378e-02 9.1670842919824e-01
-8.9529361450872e-02 8.8779579413717e-02 8.9398607312767e-01
5.7075344377831e-01 9.4179652830270e-02 8.7126143582175e-01
2.3128137347139e-01 9.9580742736056e-02 8.4853492604945e-01
-1.0781489159375e-01 1.0498254832728e-01 8.2580708732576e-01
5.5362228800989e-01 1.1038463733862e-01 8.0307870017451e-01
2.1578797649471e-01 1.1578634396492e-01 7.8035094559348e-01
-1.2107074611938e-01 1.2118660354132e-01 7.5762565963810e-01
5.4336521504907e-01 1.2658369104688e-01 7.3490571184039e-01
2.0951282742171e-01 1.3197483605671e-01 7.1219550770757e-01
-1.2208537884183e-01 1.3735576571101e-01 6.8950145667355e-01
5.4924850577099e-01 1.4272056007328e-01 6.6683173247784e-01
2.2425870403060e-01 1.4806315936551e-01 6.4419347453175e-01
-9.6513170183315e-02 1.5338330057118e-01 6.2158460761570e-01
5.8675947341300e-01 1.5869744710489e-01 5.9898262389354e-01
2.7285672749205e-01 1.6404165995170e-01 5.7634890914872e-01
-4.0088316680229e-02 1.6944749897991e-01 5.5366115196014e-01
6.4594894163612e-01 1.7491148722039e-01 5.3093400008504e-01
6.6720479983109e-01 9.5462198007232e-01 5.0740189083421e-01
3.3109873118899e-01 9.6008123548745e-01 4.8465104206991e-01
-4.7391457334918e-03 9.6554027919980e-01 4.6190050548330e-01
6.5966650388749e-01 9.7099821926733e-01 4.3915045517035e-01
3.2428023090393e-01 9.7645394719351e-01 4.1640126132367e-01
-1.0943000883980e-02 9.8190616903226e-01 3.9365354136596e-01
6.5394471136788e-01 9.8735348858131e-01 3.7090816869724e-01
3.1888736902172e-01 9.9279453714131e-01 3.4816622269479e-01
-1.6172090285742e-02 9.9822813051094e-01 3.2542887633263e-01
6.4940167211459e-01 3.6468780006471e-03 3.0270473832831e-01
3.1415517680777e-01 9.0636960237531e-03 2.7997942764804e-01
-2.1260415708685e-02 1.4471998877578e-02 2.5726110480419e-01
6.4310040063846e-01 1.9872458139121e-02 2.3454969719797e-01
3.0718390595278e-01 2.5266188788528e-02 2.1184459277949e-01
-2.9060851375807e-02 3.0654610406462e-02 1.8914470951266e-01
6.3432114109933e-01 3.6039276634771e-02 1.6644863996147e-01
2.9729446197273e-01 4.1421706953325e-02 1.4375483515547e-01
-4.0163709040746e-02 4.6803251426699e-02 1.2106178466650e-01
6.2193815496630e-01 5.2185006763342e-02 9.8368157334859e-02
2.8360615708635e-01 5.7567786318994e-02 7.5672884457026e-02
-5.5140070310029e-02 6.2952133970545e-02 5.2975185797000e-02
6.0573082092036e-01 6.8338365672023e-02 3.0274551524483e-02
2.6625986527292e-01 7.3726622714731e-02 7.5706978776357e-03
-7.3937326730165e-02 7.9119172532265e-02 9.8486083444534e-01
5.8609264693486e-01 8.4511026194935e-02 9.6215081354955e-01
2.4594582431272e-01 8.9904757818698e-02 9.3943756015432e-01
-9.4310125600962e-02 9.5300252460640e-02 9.1672119679918e-01
5.6539961667539e-01 1.0069739196321e-01 8.9400186694642e-01
2.2515896714606e-01 1.0609605953835e-01 8.7127972752178e-01
-1.1493595835864e-01 1.1149613503450e-01 8.4855495600810e-01
5.4522777790716e-01 1.1689747920331e-01 8.2582777613504e-01
2.0578667811921e-01 1.2229990226679e-01 8.0309851097549e-01
-1.3308935349767e-01 1.2770310671226e-01 7.8036768072901e-01
5.2881747952772e-01 1.3310658441389e-01 7.5763617728376e-01
1.9179445880401e-01 1.3850943021647e-01 7.3490557253258e-01
-1.4376926692975e-01 1.4391000327956e-01 7.1217865393119e-01
5.2266523665008e-01 1.4930532621368e-01 6.8946031003339e-01
1.9185253251811e-01 1.5469011670289e-01 6.6675879341679e-01
-1.3517112546164e-01 1.6005570750270e-01 6.4408668468980e-01
5.4287556475182e-01 1.6539096663607e-01 6.2145820177492e-01
2.2706398494041e-01 1.7069230966810e-01 5.9887440315871e-01
-8.2907231221343e-02 1.7598841023589e-01 5.7629663925057e-01
6.1053937519748e-01 1.8134365246702e-01 5.5366004368299e-01
3.0395047324437e-01 1.8679155584222e-01 5.3095121032565e-01
3.3394879159673e-01 9.6650293359728e-01 5.0744974401147e-01
-2.4102581439638e-03 9.7196200485462e-01 4.8469852875096e-01
6.6141986892317e-01 9.7742081232746e-01 4.6194756579692e-01
3.2541952821425e-01 9.8287857883962e-01 4.3919700500066e-01
-1.0438681578374e-02 9.8833436863725e-01 4.1644714591129e-01
6.5381065265490e-01 9.9378710199367e-01 3.9369848228945e-01
3.1812733613199e-01 9.9923560744895e-01 3.7095171727636e-01
-1.6788816966836e-02 4.6716685678045e-03 3.4821580707990e-01
6.4751942662426e-01 1.0108459356041e-02 3.2547543199157e-01
3.1176126284950e-01 1.5537960733526e-02 3.0273981643659e-01
-2.4107134885813e-02 2.0959669105411e-02 2.8000981897102e-01
6.3987103529509e-01 2.6373485462852e-02 2.5728602640216e-01
3.0365272169753e-01 3.1779738848574e-02 2.3456864741870e-01
-3.2805070312880e-02 3.7179156520403e-02 2.1185745765488e-01
6.3045554354675e-01 4.2572784552429e-02 1.8915180949814e-01
2.9339519336169e-01 4.7961872250384e-02 1.6645070551362e-01
-4.4020142974998e-02 5.3347741181029e-02 1.4375291924352e-01
6.1818358499179e-01 5.8731662064481e-02 1.2105713649181e-01
2.7999069737034e-01 6.4114758915476e-02 9.8362088006273e-02
-5.8602986091162e-02 6.9497951048850e-02 7.5666651038294e-02
6.0241000806922e-01 7.4881933441946e-02 5.2969909023068e-02
2.6304807297549e-01 8.0267188096059e-02 3.0271170419593e-02
-7.6660744864435e-02 8.5654015320980e-02 7.5699557351620e-03
5.8282099056795e-01 9.1045190896356e-02 9.8486285087055e-01
2.4256466181820e-01 9.6435114272165e-02 9.6215639720413e-01
-9.7910394790741e-02 1.0182680881538e-01 9.3944695025001e-01
5.6145122041713e-01 1.0722023293301e-01 9.1673450360092e-01
2.2071125971784e-01 1.1261533065898e-01 8.9401908087571e-01
-1.2006093375106e-01 1.1801204540445e-01 8.7130071446083e-01
5.3921381393438e-01 1.2341032687029e-01 8.4857943297446e-01
1.9862806670573e-01 1.2881013118020e-01 8.2585525858717e-01
-1.4170680686869e-01 1.3421141297781e-01 8.0312821748062e-01
5.1834753837855e-01 1.3961410556824e-01 7.8039837136248e-01
1.7896887750146e-01 1.4501807976125e-01 7.5766588748925e-01
-1.5960624094448e-01 1.5042306054243e-01 7.3493118445087e-01
5.0294828301156e-01 1.5582845529668e-01 7.1219523258096e-01
1.6709832789790e-01 1.6123299055858e-01 6.8946017423729e-01
-1.6646681983097e-01 1.6663393160895e-01 6.6673059453462e-01
5.0330418674910e-01 1.7202543912732e-01 6.4401599287526e-01
1.7803248326858e-01 1.7739556156509e-01 6.2133467221978e-01
-1.3993666758209e-01 1.8272392938472e-01 5.9871462953206e-01
5.5176936705734e-01 1.8799730485244e-01 5.7616794441091e-01
2.5254802261792e-01 1.9327257070475e-01 5.5362016317195e-01
-4.3121147067448e-02 1.9867005268706e-01 5.3096165223754e-01
6.6790370306364e-04 9.7838356708741e-01 5.0749815286030e-01
6.6413498942818e-01 9.8384252241826e-01 4.8474668109086e-01
3.2771628831146e-01 9.8930116736556e-01 4.6199539251615e-01
-8.6038910252161e-03 9.9475881134825e-01 4.3924441693141e-01
6.5515351858961e-01 1.0002146529173e+00 4.1649399952879e-01
3.1974285083973e-01 5.6603820866984e-03 3.9375302109941e-01
-1.6434339867255e-02 1.1109922657265e-02 3.7100497002035e-01
6.4737581826556e-01 1.6554775010366e-02 3.4825916660940e-01
3.1113875421931e-01 2.1993994471427e-02 3.2551646038502e-01
-2.5180210184646e-02 2.7426787889580e-02 3.0277773752201e-01
6.3838476288639e-01 3.2852599380039e-02 2.8004382004276e-01
3.0180001585847e-01 3.8271177206112e-02 2.5731535809948e-01
-3.4967992685256e-02 4.3682610250679e-02 2.3459273400446e-01
6.2804698454669e-01 4.9087327157506e-02 2.1187599604405e-01
2.9081115593292e-01 5.4486057057327e-02 1.8916483552143e-01
-4.6708377492068e-02 5.9879757247044e-02 1.6645861235741e-01
6.1545804163774e-01 6.5269519223156e-02 1.4375642447074e-01
2.7728472051166e-01 7.0656468466239e-02 1.2105720676859e-01
-6.1247315713429e-02 7.6041673814202e-02 9.8359839966518e-02
5.9985128454679e-01 8.1426078771917e-02 7.5663249501312e-02
2.6057907856758e-01 8.6810460923180e-02 5.2966480067310e-02
-7.9056067098169e-02 9.2195419002004e-02 3.0268739309688e-02
5.8096249626798e-01 9.7581382225406e-02 7.5694118540038e-03
2.4009676014509e-01 1.0297160963069e-01 9.8486451309071e-01
-1.0046729990306e-01 1.0835990545212e-01 9.6216127486753e-01
5.5871478071332e-01 1.1374973196384e-01 9.3945551576884e-01
2.1768587332456e-01 1.1914112075670e-01 9.1674712099906e-01
-1.2350574720275e-01 1.2453407314559e-01 8.9403602325280e-01
5.3519413681850e-01 1.2992857867088e-01 8.7132217597375e-01
1.9384698977487e-01 1.3532462844583e-01 8.4860553119533e-01
-1.4747616529142e-01 1.4072222395990e-01 8.2588602175950e-01
5.1130897823143e-01 1.4612138182946e-01 8.0316354771226e-01
1.7030584139583e-01 1.5152213435836e-01 7.8043796760000e-01
-1.7035389636993e-01 1.5692452424602e-01 7.5770909804172e-01
4.8950431071263e-01 1.6232858819343e-01 7.3497673157826e-01
1.5012190021105e-01 1.6773431524936e-01 7.1224069946869e-01
-1.8815180485042e-01 1.7314154221838e-01 6.8950104946408e-01
4.7521343532952e-01 1.7854968263299e-01 6.6675852558268e-01
1.4106530979432e-01 1.8395699217161e-01 6.4401586085941e-01
-1.8916329944774e-01 1.8935849325003e-01 6.2128127262392e-01
4.8708129861741e-01 1.9474014207534e-01 5.9857743465640e-01
1.7437613659010e-01 2.0006618206963e-01 5.7595725227257e-01
-1.2114548615680e-01 2.0529375501278e-01 5.5347104706297e-01
5.9892813922551e-01 2.1056036449747e-01 5.3094995257331e-01
6.6736324850737e-01 9.9026399455682e-01 5.0754689657320e-01
3.3073172022716e-01 9.9572288984525e-01 4.8479527713644e-01
-5.0526533604344e-03 1.1737599603167e-03 4.6205255148706e-01
6.5838347537531e-01 6.6313636191181e-03 4.3930120626509e-01
3.2183252556074e-01 1.2087307866304e-02 4.1655026158632e-01
-1.4724424889238e-02 1.7540821198317e-02 3.9380004412615e-01
6.4869063605703e-01 2.2991070186734e-02 3.7105102610117e-01
3.1205343106479e-01 2.8437196361935e-02 3.4830382211952e-01
-2.4661604296063e-02 3.3878371962061e-02 3.2555915980685e-01
6.3851958710328e-01 3.9313867022320e-02 3.0281782491190e-01
3.0157123804680e-01 4.4743117712034e-02 2.8008058644978e-01
-3.5532145131910e-02 5.0165785273303e-02 2.5734811179661e-01
6.2718395666901e-01 5.5581796222461e-02 2.3462088447635e-01
2.8969371536194e-01 6.0991357157270e-02 2.1189913801933e-01
-4.8029180757498e-02 6.6394941117939e-02 1.8918281742245e-01
6.1398882362764e-01 7.1793246651337e-02 1.6647157543702e-01
2.7572209254989e-01 7.7187135085759e-02 1.4376480468687e-01
-6.2852764552035e-02 8.2577555238681e-02 1.2106169972699e-01
5.9824477753726e-01 8.7965466804166e-02 9.8361337427179e-02
2.5900073155106e-01 9.3351773220041e-02 7.5662761266886e-02
-8.0592220412530e-02 9.8737271936171e-02 5.2965056060083e-02
5.7946589809430e-01 1.0412262566032e-01 3.0267403683028e-02
2.3918241058677e-01 1.0950835378370e-01 7.5691158668990e-03
-1.0205278287966e-01 1.1489816358834e-01 9.8486569792112e-01
5.5705817449779e-01 1.2028527211626e-01 9.6216508072602e-01
2.1588738712655e-01 1.2567356517161e-01 9.3946258215135e-01
-1.2553436254378e-01 1.3106314660127e-01 9.1675799777323e-01
5.3282807312508e-01 1.3645407420355e-01 8.9405118946239e-01
1.9101413728425e-01 1.4184637922940e-01 8.7134206017929e-01
-1.5093201527190e-01 1.4724008204800e-01 8.4863053079374e-01
5.0703953958011e-01 1.5263520423030e-01 8.2591651897807e-01
1.6498643668025e-01 1.5803177772617e-01 8.0319991931471e-01
-1.7702274199632e-01 1.6342985202588e-01 7.8048058303173e-01
4.8109679128675e-01 1.6882950026777e-01 7.5775829512777e-01
1.3945463685210e-01 1.7423082516352e-01 7.3503274605873e-01
-2.0180032835723e-01 1.7963396508206e-01 7.1230349503888e-01
4.5754534692910e-01 1.8503909852691e-01 6.8956992435813e-01
1.1781628456043e-01 1.9044643718917e-01 6.6683119716723e-01
-2.2046019967763e-01 1.9585616614968e-01 6.4408628882100e-01
4.4364048868483e-01 2.0126815650077e-01 6.2133441221298e-01
1.1189638167259e-01 2.0668064909505e-01 5.9857730668707e-01
-2.1185860069947e-01 2.1208378081135e-01 5.7583050359813e-01
4.8174852957308e-01 2.1742657441088e-01 5.5317496227844e-01
2.1279745833972e-01 2.2256724956910e-01 5.3080235281093e-01
3.3484467345975e-01 2.1366192858955e-03 5.0760458731376e-01
-1.8115623760092e-03 7.5955015284174e-03 4.8485293114996e-01
6.6150129821691e-01 1.3053994168531e-02 4.6210128340530e-01
3.2477338643570e-01 1.8511521953966e-02 4.3934973548280e-01
-1.2006227152092e-02 2.3967476307843e-02 4.1659844895880e-01
6.5114979114742e-01 2.9421200532524e-02 3.9384769016674e-01
3.1422683354532e-01 3.4871991699093e-02 3.7109785061947e-01
-2.2791409523667e-02 4.0319121007194e-02 3.4834944919854e-01
6.4007757763323e-01 4.5761870843784e-02 3.2560311370316e-01
3.0281572125259e-01 5.1199583573957e-02 3.0285954195328e-01
-3.4595217003840e-02 5.6631714911144e-02 2.8011944580236e-01
6.2782649430043e-01 6.2057883894289e-02 2.5738348447207e-01
2.9006240363410e-01 6.7477912013599e-02 2.3465219590194e-01
-4.7906429978686e-02 7.2891845578221e-02 2.1192593582201e-01
6.1390031664948e-01 7.8299957678407e-02 1.8920483373529e-01
2.7546226201214e-01 8.3702728828958e-02 1.6648877285215e-01
-6.3241227954060e-02 8.9100808384409e-02 1.4377739739376e-01
5.9776985073969e-01 9.4494961731020e-02 1.2107014608958e-01
2.5847735243316e-01 9.9886010498831e-02 9.8366306086521e-02
-8.1133697319059e-02 1.0527477395177e-01 7.5665078066516e-02
5.7892608673090e-01 1.1066201894185e-01 5.2965642110114e-02
2.3865130834245e-01 1.1604842354247e-01 3.0267215030012e-02
-1.0195778662560e-01 1.2143455641617e-01 7.5690929572941e-03
5.5642034502819e-01 1.2682453232983e-01 9.8486633173268e-01
2.1519323744693e-01 1.3221098249839e-01 9.6216757252031e-01
-1.2633628181694e-01 1.3759819549575e-01 9.3946767038778e-01
5.3185154144382e-01 1.4298634385743e-01 9.1676634802471e-01
1.8977976806552e-01 1.4837554226418e-01 8.9406340833686e-01
-1.5252576207374e-01 1.5376586532300e-01 8.7135871523896e-01
5.0496325487363e-01 1.5915736286500e-01 8.4865217476863e-01
1.6227755835085e-01 1.6455007232254e-01 8.2594371754341e-01
-1.8054932825168e-01 1.6994402831132e-01 8.0323328266973e-01
4.7651976664889e-01 1.7533926992221e-01 7.8052080241585e-01
1.3352728600852e-01 1.8073584650009e-01 7.5780618615002e-01
-2.0947622580351e-01 1.8613382296041e-01 7.3508930095679e-01
4.4757277239060e-01 1.9153328612924e-01 7.1236994447242e-01
1.0475950306209e-01 1.9693435448093e-01 6.8964780175136e-01
-2.3779297823418e-01 2.0233719563042e-01 6.6692236989036e-01
4.2010832737675e-01 2.0774206058646e-01 6.4419281534634e-01
7.8796119705204e-02 2.1314935517418e-01 6.2145768211878e-01
-2.6108654701181e-01 2.1855979718125e-01 5.9871424580275e-01
4.0190619631788e-01 2.2397475793604e-01 5.7595700062343e-01
7.1839796093192e-02 2.2939649569142e-01 5.5317483861972e-01
1.4909646035463e-03 1.4016980976843e-02 5.0765332797651e-01
6.6488446462279e-01 1.9475884935707e-02 4.8490175815996e-01
3.2817441347821e-01 2.4934339042373e-02 4.6215011444844e-01
-8.6467876242521e-03 3.0391805348075e-02 4.3939846601350e-01
6.5441413285518e-01 3.5847747443261e-02 4.1664693717173e-01
3.1735024200559e-01 4.1301607250455e-02 3.9389574194036e-01
-1.9846277802093e-02 4.6752795832719e-02 3.7114520522215e-01
6.4281562113286e-01 5.2200700402677e-02 3.4839576827237e-01
3.0532594536917e-01 5.7644706687129e-02 3.2564797720109e-01
-3.2326068871851e-02 6.3084233189390e-02 3.0290245482180e-01
6.2984830884500e-01 6.8518772060692e-02 2.8015985811012e-01
2.9183742249986e-01 7.3947930449404e-02 2.5742082556449e-01
-4.6370847782461e-02 7.9371466327456e-02 2.3468592044743e-01
6.1521068776459e-01 8.4789313706415e-02 2.1195557687930e-01
2.7656827857147e-01 9.0201593609933e-02 1.8923005582395e-01
-6.2312881540305e-02 9.5608608981612e-02 1.6650941703345e-01
5.9855150111184e-01 1.0101082375662e-01 1.4379351100445e-01
2.5914531331369e-01 1.0640882845615e-01 1.2108199212069e-01
-8.0547160450787e-02 1.1180329658788e-01 9.8374350856229e-02
5.7945976375885e-01 1.1719493744191e-01 7.5669959886482e-02
2.3915419954657e-01 1.2258445118076e-01 5.2968126959287e-02
-1.0147242626139e-01 1.2797249129333e-01 3.0268146939647e-02
5.5757522708934e-01 1.3335963773272e-01 7.5693465996584e-03
2.1555421738073e-01 1.3875037046813e-01 9.8486638773232e-01
-1.2602015188941e-01 1.4413673213950e-01 9.6216863183223e-01
5.3208914792490e-01 1.4952338857795e-01 9.3947050911942e-01
1.8989236586661e-01 1.5491057359067e-01 9.1677168492892e-01
-1.5259796120681e-01 1.6029845741746e-01 8.9407191005989e-01
5.0463218290061e-01 1.6568716108098e-01 8.7137100943637e-01
1.6159746887806e-01 1.7107676973527e-01 8.4866886964726e-01
-1.8168760619077e-01 1.7646734398830e-01 8.2596542720160e-01
4.7479043033348e-01 1.8185892871721e-01 8.0326065872600e-01
1.3104309657583e-01 1.8725155926998e-01 7.8055457389328e-01
-2.1292115939675e-01 1.9264526515234e-01 7.5784721168082e-01
4.4290155258856e-01 1.9804007134276e-01 7.3513864068668e-01
9.8508368326034e-02 2.0343599725732e-01 7.1242896483166e-01
-2.4611387633772e-01 2.0883305303763e-01 6.8971833721673e-01
4.0900515224616e-01 2.1423123206045e-01 6.6700698814698e-01
6.3807531227395e-02 2.1963049679491e-01 6.4429528096738e-01
-2.8181729133056e-01 2.2503075065722e-01 6.2158382902520e-01
3.7191101061356e-01 2.3043177566591e-01 5.9887376420545e-01
6.6811142901573e-01 2.5897476752069e-02 5.0770173384688e-01
3.3162832735343e-01 3.1356452441283e-02 4.8495036023418e-01
-5.0332811351027e-03 3.6814901370687e-02 4.6219883090626e-01
6.5812106274042e-01 4.2272312940522e-02 4.3944718870854e-01
3.2108866896890e-01 4.7728213254402e-02 4.1669551992743e-01
-1.6131844056663e-02 5.3182132388117e-02 3.9394399051002e-01
6.4645826741671e-01 5.8633584481203e-02 3.7119286824884e-01
3.0885714050098e-01 6.4082063168140e-02 3.4844253021258e-01
-2.8938013444638e-02 6.9527051940321e-02 3.2569345545672e-01
6.3306905832662e-01 7.4968046742386e-02 3.0294620384322e-01
2.9487374215820e-01 8.0404586579161e-02 2.8020138283063e-01
-4.3529367839787e-02 8.5836287177168e-02 2.5745960529021e-01
6.1785350054986e-01 9.1262872745010e-02 2.3472144252852e-01
2.7901515344864e-01 9.6684201460947e-02 2.1198737749846e-01
-6.0052769393887e-02 1.0210028130342e-01 1.8925776346246e-01
6.0064005114458e-01 1.0751127408631e-01 1.6653279301070e-01
2.6108261436260e-01 1.1291748697834e-01 1.4381248127922e-01
-7.8737165533749e-02 1.1831935230556e-01 1.2109666549424e-01
5.8116803076810e-01 1.2371739792972e-01 9.8385020768632e-02
2.4078562797120e-01 1.2911221174267e-01 7.5677089752608e-02
-9.9896035320210e-02 1.3450440454295e-01 5.2972321780794e-02
5.5911307828995e-01 1.3989457555417e-01 3.0270116033433e-02
2.1780533750207e-01 1.4528328407796e-01 7.5698632060376e-03
-1.2462287895616e-01 1.5067533488621e-01 9.8486587818700e-01
5.3344968162892e-01 1.5606217788914e-01 9.6216824885375e-01
1.9119792132543e-01 1.6144882732041e-01 9.3947101722863e-01
-1.5137562784506e-01 1.6683556682910e-01 9.1677380649873e-01
5.0573291629779e-01 1.7222261731472e-01 8.9407632091250e-01
1.6252786294122e-01 1.7761014697773e-01 8.7137834224798e-01
-1.8098715536360e-01 1.8299828180839e-01 8.4867972381857e-01
4.7518950916405e-01 1.8838711518699e-01 8.2598038409223e-01
1.3105592873449e-01 1.9377671572249e-01 8.0328030166054e-01
-2.1339542979619e-01 1.9916713281971e-01 7.8057951303900e-01
4.4181954836831e-01 2.0455839968716e-01 7.5787811474609e-01
9.6672678330188e-02 2.0995053355058e-01 7.3517627142989e-01
-2.4888253257736e-01 2.1534353268262e-01 7.1247423267012e-01
4.0507945913144e-01 2.2073736940626e-01 6.8977236278043e-01
5.8439842759071e-02 2.2613197724962e-01 6.6707119095157e-01
-2.8899405011033e-01 2.3152722832301e-01 6.4437149392666e-01
3.3470500707261e-01 3.7778213271761e-02 5.0774958411761e-01
-1.5783597172982e-03 4.3237318797824e-02 4.8499851845909e-01
6.6188077844245e-01 4.8695796422194e-02 4.6224721823652e-01
3.2507888078246e-01 5.4153154609323e-02 4.3949569441847e-01
-1.1982592456248e-02 5.9608976158947e-02 4.1674399225914e-01
6.5070068671914e-01 6.5062873763950e-02 3.9399223103501e-01
3.1313415115631e-01 7.0514458657358e-02 3.7124062761019e-01
-2.4676905852694e-02 7.5963325176896e-02 3.4848950488693e-01
6.3727193765970e-01 8.1409050867989e-02 3.2573928668042e-01
2.9898390443308e-01 8.6851209582407e-02 3.0299048068797e-01
-3.9539061270780e-02 9.2289393770553e-02 2.8024365146260e-01
6.2170372403059e-01 9.7723241638111e-02 2.5749938577467e-01
2.8271168595129e-01 1.0315246488307e-01 2.3475825335147e-01
-5.6517072146222e-02 1.0857687319414e-01 2.1202076653267e-01
6.0401410727503e-01 1.1399639243407e-01 1.8928734269126e-01
2.6430030786703e-01 1.1941107434152e-01 1.6655827321425e-01
-7.5665016109306e-02 1.2482109659873e-01 1.4383370233616e-01
5.8411004878790e-01 1.3022675319799e-01 1.2111361814711e-01
2.4361614939053e-01 1.3562843614832e-01 9.8397856710411e-02
-9.7156877716546e-02 1.4102661059444e-01 7.5686118585671e-02
5.6178058171395e-01 1.4642178620836e-01 5.2977995440760e-02
2.2041855224204e-01 1.5181448808411e-01 3.0273003184285e-02
-1.2125195687083e-01 1.5720523020974e-01 7.5706174451797e-03
5.3590835803145e-01 1.6259910583732e-01 9.8486484412814e-01
1.9362479774373e-01 1.6798696890690e-01 9.6216649802886e-01
-1.4898617800448e-01 1.7337415163298e-01 9.3946926729204e-01
5.0807210764652e-01 1.7876097284644e-01 9.1677274824944e-01
1.6479684769513e-01 1.8414769517004e-01 8.9407660549172e-01
-1.7881515692503e-01 1.8953453056382e-01 8.7138057496857e-01
4.7723131643254e-01 1.9492164745693e-01 8.4868446353427e-01
1.3292840569347e-01 2.0030917811076e-01 8.2598814671007e-01
-2.1173675973143e-01 2.0569722517412e-01 8.0329156626679e-01
4.4321541866005e-01 2.1108586670196e-01 7.8059472897525e-01
9.7753574603027e-02 2.1647515914427e-01 7.5789770760422e-01
-2.4816949761276e-01 2.2186513791418e-01 7.3520064497159e-01
4.0537593675732e-01 2.2725581504644e-01 7.1250376144044e-01
5.8285399826276e-02 2.3264717300590e-01 6.8980736525921e-01
1.2699925087419e-03 4.9659296941152e-02 5.0779665432410e-01
6.6526720975239e-01 5.5118604158834e-02 4.8504601088107e-01
3.2892188025500e-01 6.0577149080379e-02 4.6229505948769e-01
-7.7674217071063e-03 6.6034452071153e-02 4.3954377208501e-01
6.5520533790422e-01 7.1490150342175e-02 4.1679214786753e-01
3.1785073319389e-01 7.6943938269818e-02 3.9404025867380e-01
-1.9818600320148e-02 8.2395522778973e-02 3.7128827434041e-01
6.4221020736053e-01 8.7844596890637e-02 3.4853647131041e-01
3.0394909599276e-01 9.3290830634695e-02 3.2578522794619e-01
-3.4591542104829e-02 9.8733876338823e-02 3.0303500939001e-01
6.2659689850813e-01 1.0417338432727e-01 2.8028634424992e-01
2.8752120336740e-01 1.0960902484667e-01 2.5753979517479e-01
-5.1813626255653e-02 1.1504051227452e-01 2.3479592549876e-01
6.0859564506418e-01 1.2046762817198e-01 2.1205526441712e-01
2.6875045762549e-01 1.2589024039574e-01 1.8931827341596e-01
-7.1349603818942e-02 1.3130831621094e-01 1.6658531676963e-01
5.8829314525255e-01 1.3672192810981e-01 1.4385663874475e-01
2.4767451654625e-01 1.4213125183034e-01 1.2113234965362e-01
-9.3211406058983e-02 1.4753655687766e-01 9.8412422079567e-02
5.6562801487785e-01 1.5293819064335e-01 7.5696697515777e-02
2.2418437244773e-01 1.5833655791204e-01 5.2984902458139e-02
-1.1755131068495e-01 1.6373209802568e-01 3.0276671883603e-02
5.4041189109903e-01 1.6912526213444e-01 7.5715772523470e-03
1.9715953588511e-01 1.7452140072820e-01 9.8486334480571e-01
-1.4548112290428e-01 1.7991077108128e-01 9.6216351088288e-01
5.1155148952741e-01 1.8529899311003e-01 9.3946544107867e-01
1.6824997651148e-01 1.9068640427206e-01 9.1676872010175e-01
-1.7539326446106e-01 1.9607329660703e-01 8.9407298219814e-01
4.8061314821792e-01 2.0145991807972e-01 8.7137791924934e-01
1.3625845102745e-01 2.0684647618865e-01 8.4868328468633e-01
-2.0847176221686e-01 2.1223314264751e-01 8.2598889437740e-01
4.4640250004605e-01 2.1762005809059e-01 8.0329462559954e-01
1.0085304049750e-01 2.2300733594773e-01 7.8060041493160e-01
-2.4516001662516e-01 2.2839506478969e-01 7.5790625542483e-01
4.0830692978944e-01 2.3378330846304e-01 7.3521219263309e-01
6.6780389244907e-01 6.1540834035835e-02 5.0784271542031e-01
3.3216925240867e-01 6.7000435147066e-02 4.8509261138398e-01
-3.9015671701358e-03 7.2459095244998e-02 4.6234213397250e-01
6.5959245628224e-01 7.7916340212127e-02 4.3959120697667e-01
3.2266274596775e-01 8.3371863069202e-02 4.1683977648193e-01
-1.4672880268247e-02 8.8825443677110e-02 3.9408786448699e-01
6.4760638356045e-01 9.4276887793138e-02 3.7133559634858e-01
3.0952180728352e-01 9.9725988703479e-02 3.4858320849719e-01
-2.8906512278529e-02 1.0517250957102e-01 3.2583104246388e-01
6.3233944757479e-01 1.1061618233995e-01 3.0307952963784e-01
2.9327525860705e-01 1.1605671844928e-01 2.8032916964168e-01
-4.6086018963788e-02 1.2149382686246e-01 2.5758050426309e-01
6.1426624922452e-01 1.2692723551181e-01 2.3483408851951e-01
2.7434038702939e-01 1.3235671294283e-01 2.1209046045903e-01
-6.5857508700803e-02 1.3778208762685e-01 1.8935011148829e-01
5.9367646994294e-01 1.4320326305923e-01 1.6661345918598e-01
2.5294405581552e-01 1.4862022737199e-01 1.4388082458690e-01
-8.8055163850240e-02 1.5403305677983e-01 1.2115241573448e-01
5.7067633899979e-01 1.5944191276409e-01 9.8428318868804e-02
2.2913420539937e-01 1.6484703348103e-01 7.5708497955172e-02
-1.1268754671444e-01 1.7024872043160e-01 5.2992802430532e-02
5.4520381284421e-01 1.7564732188403e-01 3.0280982171535e-02
2.0280008922132e-01 1.8104321481998e-01 7.5727079229864e-03
-1.4086386971436e-01 1.8644198088198e-01 9.8486144848544e-01
5.1613879391244e-01 1.9183328340781e-01 9.6215945088835e-01
1.7281811414028e-01 1.9722300121802e-01 9.3945978623360e-01
-1.7083578098925e-01 2.0261147338626e-01 9.1676204229071e-01
4.8516642075103e-01 2.0799900696145e-01 8.9406583449380e-01
1.4081254413956e-01 2.1338587477386e-01 8.7137081736383e-01
-2.0391206423806e-01 2.1877231567465e-01 8.4867669168536e-01
4.5097410348870e-01 2.2415853627494e-01 8.2598320791776e-01
1.0544690416341e-01 2.2954471322654e-01 8.0329016637040e-01
3.3430321087883e-01 7.3422930383524e-02 5.0788753335619e-01
-8.6625759862746e-04 7.8882946006829e-02 4.8513808901927e-01
6.6342262437064e-01 8.4341783523042e-02 4.6238821634398e-01
3.2717396319210e-01 8.9798968621292e-02 4.3963777918987e-01
-9.5940809208095e-03 9.5254255926085e-02 4.1688666126998e-01
6.5314515590483e-01 1.0070751997671e-01 3.9413483118972e-01
3.1542218992813e-01 1.0615867333802e-01 3.7138237197116e-01
-2.2731999215992e-02 1.1160761474588e-01 3.4862948631272e-01
6.3871186787040e-01 1.1705420331017e-01 3.2587648730595e-01
2.9978019684446e-01 1.2249825244098e-01 3.0312378111919e-01
-3.9503954907978e-02 1.2793953723200e-01 2.8037184531926e-01
6.2087907734759e-01 1.3337781003083e-01 2.5762120469679e-01
2.8094573847700e-01 1.3881282007378e-01 2.3487240562008e-01
-5.9290505136712e-02 1.4424433406970e-01 2.1212598974784e-01
6.0018105647418e-01 1.4967215542720e-01 1.8938246806718e-01
2.5936854254376e-01 1.5509614046270e-01 1.6664229645127e-01
-8.1722408133161e-02 1.6051621045032e-01 1.4390585410289e-01
5.7691144262471e-01 1.6593235882672e-01 1.2117342627754e-01
2.3527101642845e-01 1.7134465328195e-01 9.8445192521395e-02
-1.0664496173541e-01 1.7675323287708e-01 7.5721221305450e-02
5.5116021104049e-01 1.8215830073118e-01 5.3001471412542e-02
2.0868144441390e-01 1.8756011318954e-01 3.0285799802814e-02
-1.3408788286431e-01 1.9295896667242e-01 7.5739750419721e-03
5.2183912310787e-01 1.9836065254249e-01 9.8485922541830e-01
1.7848630919341e-01 2.0375424659521e-01 9.6215449323394e-01
-1.6518473916767e-01 2.0914586007516e-01 9.3945257996347e-01
4.9081499174838e-01 2.1453581741169e-01 9.1675309003467e-01
1.4647319745292e-01 2.1992442551646e-01 8.9405563293361e-01
-1.9822427010654e-01 2.2531196894149e-01 8.7135983645189e-01
4.5670577956284e-01 2.3069870719656e-01 8.4866535615992e-01
7.6312103219200e-04 8.5305690463300e-02 5.0793086924969e-01
6.6616893997197e-01 9.0766279918783e-02 4.8518220790181e-01
3.3091143736836e-01 9.6225377312600e-02 4.6243307620218e-01
-5.0011119994539e-03 1.0168250333886e-01 4.3968326248933e-01
6.5845830384103e-01 1.0713748592137e-01 4.1693257623231e-01
3.2132763386725e-01 1.1259031019440e-01 3.9418092849795e-01
-1.6350598278377e-02 1.1804100900050e-01 3.7142836285028e-01
6.4546622283533e-01 1.2348959539885e-01 3.4867505562470e-01
3.0681805231099e-01 1.2893602924965e-01 3.2592130067974e-01
-3.2259230506273e-02 1.3438020785132e-01 3.0316748786581e-01
6.2826567298437e-01 1.3982197189942e-01 2.8041407973027e-01
2.8841952401914e-01 1.4526112022250e-01 2.5766158813238e-01
-5.1775132537568e-02 1.5069742873078e-01 2.3491055113329e-01
6.0770044684552e-01 1.5613067049053e-01 2.1216151018466e-01
2.6686160249368e-01 1.6156063485180e-01 1.8941498778434e-01
-7.4279401854492e-02 1.6698714423971e-01 1.6667146608348e-01
5.8428686255769e-01 1.7241006768857e-01 1.4393136723166e-01
2.4256715975951e-01 1.7782933054592e-01 1.2119503644221e-01
-9.9434283828383e-02 1.8324492005900e-01 9.8462728801200e-02
5.5828433619636e-01 1.8865688682284e-01 7.5734600717607e-02
2.1572253205891e-01 1.9406534233458e-01 5.3010706623972e-02
-1.2712231880612e-01 1.9947045316238e-01 3.0291001120166e-02
5.2974520843617e-01 2.0487243247505e-01 7.5753462640062e-03
1.8526656383934e-01 2.1027726380205e-01 9.8485674317837e-01
-1.5844028517702e-01 2.1567344524976e-01 9.6214881055834e-01
4.9754063532652e-01 2.2106729667589e-01 9.3944410228766e-01
1.5319833381726e-01 2.2645911316087e-01 9.1674225159363e-01
-1.9147990081804e-01 2.3184918671220e-01 8.9404287519221e-01
6.6717696870493e-01 9.7189215737800e-02 5.0797248021337e-01
3.3328618948731e-01 1.0265059065859e-01 4.8522472775623e-01
-1.4118266966426e-03 1.0811005749636e-01 4.6247647832582e-01
6.6309715157438e-01 1.1356712887764e-01 4.3972742347022e-01
3.2685225784405e-01 1.1902172635875e-01 4.1697728334068e-01
-1.0093629186005e-02 1.2447397055988e-01 3.9422590756681e-01
6.5231735806586e-01 1.2992403463701e-01 3.7147330540821e-01
3.1414220125312e-01 1.3537205865306e-01 3.4871963685360e-01
-2.4566394254152e-02 1.4081810965845e-01 3.2596518773679e-01
6.3623841618009e-01 1.4626217142460e-01 3.0321034154439e-01
2.9659719447151e-01 1.5170415138896e-01 2.8045555311570e-01
-4.3455537783882e-02 1.5714389647931e-01 2.5770132534024e-01
6.1610926201830e-01 1.6258121267194e-01 2.3494818826411e-01
2.7531581650939e-01 1.6801588524087e-01 2.1219667966990e-01
-6.5815844791461e-02 1.7344769792602e-01 1.8944732646397e-01
5.9273064365520e-01 1.7887644998312e-01 1.6670062666936e-01
2.5096839319377e-01 1.8430197049976e-01 1.4395703227157e-01
-9.1092441478546e-02 1.8972412961475e-01 1.2121693350652e-01
5.6655554691949e-01 1.9514284644875e-01 9.8480645377453e-02
2.2391717013462e-01 2.0055809369590e-01 7.5748397232671e-02
-1.1900523114072e-01 2.0596989896924e-01 5.3020326130766e-02
5.3778833240856e-01 2.1137834314760e-01 3.0296474481599e-02
1.9429562451038e-01 2.1678355613326e-01 7.5767921118263e-03
-1.5058504674581e-01 2.2219169913881e-01 9.8485406415462e-01
5.0535531429578e-01 2.2759070544414e-01 9.6214256443493e-01
1.6098967119316e-01 2.3298708235558e-01 9.3943461899512e-01
3.3353550714183e-01 1.0907360194267e-01 5.0801212086112e-01
5.0113573161457e-04 1.1453604494511e-01 4.8526540516919e-01
6.6648474393759e-01 1.1999602612629e-01 4.6251818357382e-01
3.3150946239786e-01 1.2545305062181e-01 4.3977002085576e-01
-4.3683774187370e-03 1.3090716734833e-01 4.1702052883705e-01
6.5892447059528e-01 1.3635866984196e-01 3.9426949354772e-01
3.2146601927980e-01 1.4180789967302e-01 3.7151689968099e-01
-1.6668580548602e-02 1.4725514040946e-01 3.4876290558074e-01
6.4458893516885e-01 1.5270057371298e-01 3.2600780359816e-01
3.0529834755767e-01 1.5814427183550e-01 3.0325198247552e-01
-3.4489194095820e-02 1.6358620877234e-01 2.8049589696991e-01
6.2526943871825e-01 1.6902627950461e-01 2.5774004447373e-01
2.8461028877571e-01 1.7446432163061e-01 2.3498494655888e-01
-5.6436685443485e-02 1.7990013654353e-01 2.1223113329096e-01
6.0215331697849e-01 1.8533350883874e-01 1.8947912873645e-01
2.6040072375472e-01 1.9076422338927e-01 1.6672943679501e-01
-8.1677790488243e-02 1.9619207987557e-01 1.4398252711553e-01
5.7593115810807e-01 2.0161690470324e-01 1.2123882133329e-01
2.3323799026804e-01 2.0703856030025e-01 9.8498680187445e-02
-1.0974961208968e-01 2.1245695181326e-01 7.5762392250572e-02
5.4697344819530e-01 2.1787203125351e-01 5.3030165036713e-02
2.0340979657865e-01 2.2328379919268e-01 3.0302119160599e-02
-1.4044031979234e-01 2.2869230418308e-01 7.5782860165589e-03
5.1428039297823e-01 2.3410387300914e-01 9.8485124478362e-01
-1.7431224169113e-04 1.2095893487454e-01 5.0804954548741e-01
6.6783561654871e-01 1.2642282610088e-01 4.8530399557798e-01
3.3464516142023e-01 1.3188351172673e-01 4.6255795042302e-01
2.9127134144367e-04 1.3734049773377e-01 4.3981080441458e-01
6.6485547854795e-01 1.4279401573051e-01 4.1706203745625e-01
3.2843957532412e-01 1.4824458758626e-01 3.9431137441548e-01
-8.8513734227839e-03 1.5369276129497e-01 3.7155879336296e-01
6.5308120301331e-01 1.5913898387754e-01 3.4880447308322e-01
3.1432486639320e-01 1.6458355882611e-01 3.2604873170236e-01
-2.5045122976722e-02 1.7002664713850e-01 3.0329197682815e-01
6.3503463287111e-01 1.7546828704875e-01 2.8053467074464e-01
2.9461690482056e-01 1.8090841948786e-01 2.5777730765333e-01
-4.6254665654743e-02 1.8634691336714e-01 2.3502039859592e-01
6.1245591085809e-01 1.9178358841890e-01 2.1226446035167e-01
2.7077829133230e-01 1.9721823500748e-01 1.8951000575806e-01
-7.1263094126504e-02 2.0265063099839e-01 1.6675753397650e-01
5.8635183768050e-01 2.0808055598160e-01 1.4400752006063e-01
2.4363949067804e-01 2.1350780315588e-01 1.2126040378472e-01
-9.9386928164531e-02 2.1893218911815e-01 9.8516578099621e-02
5.5728292216492e-01 2.2435356172223e-01 7.5776377842884e-02
2.1365673260865e-01 2.2977180609951e-01 5.3040069463936e-02
6.6602868262706e-01 1.3284528384592e-01 5.0808451088747e-01
3.3532069602868e-01 1.3831114014340e-01 4.8534025601305e-01
3.1309761272470e-03 1.4377277721470e-01 4.6259553695714e-01
6.6951874794018e-01 1.4922972666160e-01 4.3984951229565e-01
3.3460365908490e-01 1.5468249399936e-01 4.1710150203779e-01
-1.4714756098087e-03 1.6013191091238e-01 3.9435118258508e-01
6.6143544017977e-01 1.6557878154749e-01 3.7159855731257e-01
3.2345361214113e-01 1.7102373844742e-01 3.4884385835878e-01
-1.5305187847764e-02 1.7646721188534e-01 3.2608745468316e-01
6.4525299031285e-01 1.8190944842011e-01 3.0332978794116e-01
3.0520581642747e-01 1.8735054500116e-01 2.8057133444228e-01
-3.5383080116089e-02 1.9279048385035e-01 2.5781258508899e-01
6.2353821334904e-01 1.9822916293484e-01 2.3505403557591e-01
2.8201202315839e-01 2.0366642104244e-01 2.1229618128247e-01
-5.9927079658519e-02 2.0910205801905e-01 1.8953951337653e-01
5.9774922821694e-01 2.1453585112350e-01 1.6678451419852e-01
2.5506422002711e-01 2.1996756841277e-01 1.4403165104329e-01
-8.7962963377232e-02 2.2539697989531e-01 1.2128136813699e-01
5.6868336126339e-01 2.3082386699448e-01 9.8534077050069e-02
3.3211650996825e-01 1.4473269119266e-01 5.0811677978139e-01
3.0017117840205e-03 1.5020122636634e-01 4.8537394862831e-01
6.7203000771165e-01 1.5566413199717e-01 4.6263070277051e-01
3.3929801070259e-01 1.6112102512652e-01 4.3988586404622e-01
4.9849582668959e-03 1.6657283747156e-01 4.1713856331748e-01
6.6929406648172e-01 1.7202082951116e-01 3.9438846273959e-01
3.3241897384287e-01 1.7746612368518e-01 3.7163564597825e-01
-5.4706066449155e-03 1.8290955892967e-01 3.4888044612296e-01
6.5576767121087e-01 1.8835169153815e-01 3.2612331370164e-01
3.1625040562683e-01 1.9379284467602e-01 3.0336473913227e-01
-2.3928083647013e-02 1.9923316408622e-01 2.8060521566440e-01
6.3530804763194e-01 2.0467266564998e-01 2.5784522621154e-01
2.9401967961065e-01 2.1011127221412e-01 2.3508524190764e-01
-4.7744257484029e-02 2.1554884108600e-01 2.1232572493044e-01
6.1005570668092e-01 2.2098518445157e-01 1.8956713149220e-01
2.6745158853282e-01 2.2642008476869e-01 1.6680991296570e-01
-7.5530468251066e-02 2.3185330674195e-01 1.4405451430000e-01
-1.9540348102353e-03 1.5662115448872e-01 5.0814612484488e-01
6.7094707272922e-01 1.6209337637248e-01 4.8540484512864e-01
3.4147089427437e-01 1.6755795061180e-01 4.6266320930416e-01
9.7797824425765e-03 1.7301471583022e-01 4.3991954292879e-01
6.7614958210911e-01 1.7846528841590e-01 4.1717276895333e-01
3.4087463121493e-01 1.8391152876480e-01 3.9442261330067e-01
4.2213349163685e-03 1.8935494903213e-01 3.7166933136999e-01
6.6641332782600e-01 1.9479660714552e-01 3.4891342210419e-01
3.2763161820800e-01 2.0023717361779e-01 3.2615545055635e-01
-1.1979826436659e-02 2.0567703072779e-01 3.0339596491770e-01
6.4769261692316e-01 2.1111635773043e-01 2.8063546998743e-01
3.0673839817205e-01 2.1655519355595e-01 2.5787442799031e-01
-3.4772018908934e-02 2.2199348021321e-01 2.3511326974899e-01
6.2321703592570e-01 2.2743109220125e-01 2.1235240759540e-01
2.8074976756262e-01 2.3286785646771e-01 1.8959224613889e-01
6.6374697041393e-01 1.6851059402484e-01 5.0817233349996e-01
3.3926449558110e-01 1.7398796963009e-01 4.8543273240103e-01
1.1648448215539e-02 1.7945470124721e-01 4.6269281429788e-01
6.8118343872189e-01 1.8491115751076e-01 4.3995015189219e-01
3.4830846987595e-01 1.9036008532151e-01 4.1720348806100e-01
1.3458739346560e-02 1.9580418222578e-01 3.9445277726110e-01
6.7700453747406e-01 2.0124541782297e-01 3.7169859104738e-01
3.3924138968261e-01 2.0668506000198e-01 3.4894167261444e-01
3.9892290867253e-04 2.1212386340164e-01 3.2618272548031e-01
6.6065380133271e-01 2.1756224095722e-01 3.0342234798889e-01
3.2014164312230e-01 2.2300038484364e-01 2.8066103489923e-01
-2.1033305279962e-02 2.2843834438019e-01 2.5789920231518e-01
6.3720933590203e-01 2.3387607449339e-01 2.3513721615789e-01
3.2909828278853e-01 1.8040078747038e-01 5.0819521408857e-01
8.1331787203856e-03 1.8588554317664e-01 4.8545742007978e-01
6.8287029230450e-01 1.9135498779660e-01 4.6271924671260e-01
3.5383801019329e-01 1.9681073741695e-01 4.3997710249743e-01
2.1764041032768e-02 2.0225744617906e-01 4.1722972813124e-01
6.8730651494496e-01 2.0769894911722e-01 3.9447763534238e-01
3.5098495051500e-01 2.1313770262572e-01 3.7172191779388e-01
1.3190118568142e-02 2.1857513215467e-01 3.4896363080957e-01
6.7421179609987e-01 2.2401202075747e-01 3.2620360409994e-01
3.3426470684211e-01 2.2944877266581e-01 3.0344244234072e-01
-6.1289354782446e-03 1.9229122258197e-01 5.0821460557033e-01
6.7787360775069e-01 1.9778693717158e-01 4.8547875137019e-01
3.5564841040209e-01 2.0325960489376e-01 4.6274211442783e-01
2.8255214627771e-02 2.0871384317865e-01 4.3999932373585e-01
6.9696038958244e-01 2.1415755200443e-01 4.1724973225750e-01
3.6278199308875e-01 2.1959599557901e-01 3.9449501231911e-01
2.6453171677460e-02 2.2503203389965e-01 3.7173700250426e-01
6.8848828127837e-01 2.3046712778779e-01 3.4897705226555e-01
6.5757685934906e-01 2.0418071304367e-01 5.0823039839043e-01
3.4911824502660e-01 2.0969362420194e-01 4.8549661628171e-01
3.0893750883148e-02 2.1516958141909e-01 4.6276055640690e-01
7.0526598578851e-01 2.2062080178201e-01 4.4001445537678e-01
3.7456688220650e-01 2.2606057136051e-01 4.1726007569146e-01
4.0400973385358e-02 2.3149559295040e-01 3.9450114196648e-01
3.1895815367063e-01 2.1606618033347e-01 5.0824259602051e-01
2.3304016793491e-02 2.2160852517748e-01 4.8551092228658e-01
7.1037239776876e-01 2.2708609939401e-01 4.6277177727014e-01
3.8628821110476e-01 2.3253185272007e-01 4.4001648234289e-01
-2.6467419783682e-02 2.2793764585008e-01 5.0825144981025e-01
7.0448909476811e-01 2.3353812114316e-01 4.8552076314223e-01
"""

In [ ]:
%%time

import numpy as np
import torch
from alignn.pretrained import get_figshare_model,get_multiple_predictions
import matplotlib.pyplot as plt
from jarvis.io.vasp.inputs import Poscar
from alignn.ff.ff import AlignnAtomwiseCalculator,default_path,wt10_path,alignnff_fmult,fd_path,ForceField
import time
from jarvis.core.graphs import Graph
import torch.multiprocessing as mp
from jarvis.core.atoms import Atoms
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
atoms=Poscar.from_string(Mo3696).atoms
pr=get_multiple_predictions(atoms_array=[atoms],workers=0,pin_memory=False)
pr

Using chk file jv_formation_energy_peratom_alignn/checkpoint_300.pt from  ['jv_formation_energy_peratom_alignn/checkpoint_300.pt']
Path /usr/local/lib/python3.10/dist-packages/alignn/jv_formation_energy_peratom_alignn.zip
Config /content/jv_formation_energy_peratom_alignn/config.json
data range -9999 -9999


100%|██████████| 1/1 [01:16<00:00, 76.56s/it]


df                                                atoms  prop jid
0  {'lattice_mat': [[2.738736057436, 0.0, 0.0], [... -9999   0
building line graphs


100%|██████████| 1/1 [00:02<00:00,  2.44s/it]


CPU times: user 1min 5s, sys: 9.22 s, total: 1min 14s
Wall time: 1min 20s


In [ ]:
Mo14784="""Mo
1.0000000000000e+00
2.7387360574360e+00 0.0000000000000e+00 0.0000000000000e+00
0.0000000000000e+00 4.3379365973285e+02 0.0000000000000e+00
-1.3693680287180e+00 2.1689682986643e+02 1.9678283917248e+02
Mo
14784
direct
7.5788301174734e-01 2.4240482701947e-01 5.1518925351085e-01
5.8581198563131e-03 2.4291241044465e-01 5.3808778989691e-01
6.6163076525478e-01 2.4561443841297e-01 5.2673758549396e-01
3.1058613364452e-01 2.4831327902532e-01 5.1540202075790e-01
3.3302086656594e-01 2.4344761740554e-01 5.6085187353844e-01
9.8820368744021e-01 2.4614910173430e-01 5.4950185186376e-01
6.4065124100682e-01 2.4884964590529e-01 5.3815580652149e-01
2.8802129010392e-01 2.5154828744178e-01 5.2681505969840e-01
9.2450627869561e-01 2.5424141812260e-01 5.1547798618578e-01
6.7049903814752e-01 2.4399063179576e-01 5.8358776001809e-01
3.2686392172825e-01 2.4669217238892e-01 5.7223520434833e-01
9.8188682460892e-01 2.4939335988200e-01 5.6088416569600e-01
6.3494710441930e-01 2.5209392618318e-01 5.4953484504850e-01
2.8499014739996e-01 2.5479329228932e-01 5.3818705714724e-01
9.3010573872820e-01 2.5748988608988e-01 5.2683937945056e-01
5.6660226372773e-01 2.6017848073371e-01 5.1548747898076e-01
1.1092140766164e-02 2.4453716163360e-01 6.0631423879722e-01
6.6836262413692e-01 2.4723854953417e-01 5.9496027810365e-01
3.2484000089440e-01 2.4993980283403e-01 5.8360703011164e-01
9.8027553751190e-01 2.5264081088375e-01 5.7225454458880e-01
6.3430656866519e-01 2.5534138019785e-01 5.6090276409513e-01
2.8638776874040e-01 2.5804113359699e-01 5.4955138466500e-01
9.3567134035261e-01 2.6073924980883e-01 5.3819959692188e-01
5.8079788183957e-01 2.6343377615101e-01 5.2684572851667e-01
2.1957143802866e-01 2.6611998573125e-01 5.1548733887671e-01
6.9462510530431e-01 2.4238510847867e-01 6.4039060869963e-01
3.5298877836247e-01 2.4508621477035e-01 6.2903560405537e-01
1.0919570774729e-02 2.4778732503272e-01 6.1768092812414e-01
6.6832417773401e-01 2.5048840240579e-01 6.0632661593535e-01
3.2507935753607e-01 2.5318939191942e-01 5.9497268281118e-01
9.8101970624116e-01 2.5589020731758e-01 5.8361910296013e-01
6.3591926697815e-01 2.5859070497486e-01 5.7226577219620e-01
2.8946338028516e-01 2.6129063019695e-01 5.6091244619479e-01
9.4120514449227e-01 2.6398950370306e-01 5.4955864826201e-01
5.9049882643099e-01 2.6668638164666e-01 5.3820356609687e-01
2.3640598757489e-01 2.6937938025046e-01 5.2684606105438e-01
8.7760605033104e-01 2.7206496639102e-01 5.1548519840190e-01
3.6774057477093e-02 2.4293667284741e-01 6.6310852723191e-01
6.9552622517128e-01 2.4563740306927e-01 6.5175335889241e-01
3.5395561669918e-01 2.4833817303170e-01 6.4039836069747e-01
1.2007241031786e-02 2.5103896204969e-01 6.2904354777159e-01
6.6961167981533e-01 2.5373974018517e-01 6.1768892472948e-01
3.2668041927576e-01 2.5644046302646e-01 6.0633447724517e-01
9.8309933947652e-01 2.5914106258740e-01 5.9498015883792e-01
6.3871952548665e-01 2.6184143101360e-01 5.8362587077935e-01
2.9334418110858e-01 2.6454139079716e-01 5.7227143310889e-01
9.4670995730360e-01 2.6724063973266e-01 5.6091654710097e-01
5.9846073402001e-01 2.6993865001194e-01 5.4956075885981e-01
2.4811306367441e-01 2.7263449208686e-01 5.3820346103672e-01
8.9501962080262e-01 2.7532657210241e-01 5.2684403258150e-01
5.3836356419409e-01 2.7801243309552e-01 5.1548230090828e-01
3.7933359640268e-01 2.4349014840418e-01 6.8582337724897e-01
3.8407592547025e-02 2.4619049851177e-01 6.7446820568254e-01
6.9722882112444e-01 2.4889090492623e-01 6.6311311376683e-01
3.5576253843948e-01 2.5159135626229e-01 6.5175810750412e-01
1.3966201450306e-02 2.5429183584874e-01 6.4040318701019e-01
6.7178738014163e-01 2.5699231932867e-01 6.2904834264879e-01
3.2916099415411e-01 2.5969277082452e-01 6.1769354940395e-01
9.8600562319637e-01 2.6239313669172e-01 6.0633875884671e-01
6.4221854517031e-01 2.6509333518052e-01 5.9498388802548e-01
2.9766903635702e-01 2.6779323913288e-01 5.8362880493770e-01
9.5218934980775e-01 2.7049264692946e-01 5.7227331154620e-01
6.0556277731824e-01 2.7319123424445e-01 5.6091712897153e-01
2.5750864537751e-01 2.7588847700012e-01 5.4955989821608e-01
9.0766601333335e-01 2.7858354042257e-01 5.3820122736099e-01
5.5558384510959e-01 2.8127516202524e-01 5.2684084175731e-01
2.0074064997502e-01 2.8396167589807e-01 5.1547889365660e-01
7.2213699255207e-01 2.4404532264030e-01 7.0853566015890e-01
3.8148314285747e-01 2.4674531125438e-01 6.9718057439229e-01
4.0623293522054e-02 2.4944535966472e-01 6.8582551739456e-01
6.9953438907413e-01 2.5214546255801e-01 6.7447048980599e-01
3.5818879213916e-01 2.5484561130888e-01 6.6311548892858e-01
1.6553236316248e-02 2.5754579275504e-01 6.5176050673858e-01
6.7458749278454e-01 2.6024598735762e-01 6.4040552715086e-01
3.3224266088177e-01 2.6294616640082e-01 6.2905052230120e-01
9.8945896372009e-01 2.6564628767709e-01 6.1769544759610e-01
6.4616289366809e-01 2.6834628877419e-01 6.0634023533871e-01
3.0226351253009e-01 2.7104607657231e-01 5.9498478698893e-01
9.5764768121554e-01 2.7374551082565e-01 5.8362896479272e-01
6.1217401531325e-01 2.7644437880102e-01 5.7227258507412e-01
2.6566554884451e-01 2.7914235734429e-01 5.6091541869874e-01
9.1790171977371e-01 2.8183896019940e-01 5.4955721007484e-01
5.6861200846485e-01 2.8453347715118e-01 5.3819773456940e-01
2.1747759402175e-01 2.8722494054608e-01 5.2683691954035e-01
8.6415517276932e-01 2.8991222743060e-01 5.1547503097474e-01
6.5091921107156e-02 2.4460197408295e-01 7.3124577744717e-01
7.2467277684973e-01 2.4730164185525e-01 7.1989080985423e-01
3.8408010529248e-01 2.5000136251402e-01 7.0853584697453e-01
4.3298049271466e-02 2.5270113521890e-01 6.9718088534693e-01
7.0230788943965e-01 2.5540095686917e-01 6.8582591970690e-01
3.6108747037291e-01 2.5810082143050e-01 6.7447094186623e-01
1.9610491686149e-02 2.6080071896703e-01 6.6311593923937e-01
6.7784562746246e-01 2.6350063423866e-01 6.5176089292004e-01
3.3575542692229e-01 2.6620054465118e-01 6.4040577520383e-01
9.9329493655542e-01 2.6890041723801e-01 6.2905054646490e-01
6.5040996944698e-01 2.7160020419116e-01 6.1769515135440e-01
3.0703493362794e-01 2.7429983622949e-01 6.0633951444643e-01
9.6309012485180e-01 2.7699921278779e-01 5.9498353581079e-01
6.1847840760908e-01 2.7969818767430e-01 5.8362708770304e-01
2.7308129225428e-01 2.8239654866449e-01 5.7227001488576e-01
9.2675465648141e-01 2.8509399005740e-01 5.6091214332410e-01
5.7932494375058e-01 2.8779008003702e-01 5.4955330517042e-01
2.3058794369869e-01 2.9048423321809e-01 5.3819339078022e-01
8.8031469295609e-01 2.9317571948165e-01 5.2683243552996e-01
5.2827271832454e-01 2.9586378041411e-01 5.1547072565029e-01
7.4822032756075e-01 2.4246046803851e-01 7.6530899561598e-01
4.0814745425629e-01 2.4515984595517e-01 7.5395412078196e-01
6.7933972471772e-02 2.4785925629669e-01 7.4259925450490e-01
7.2757025824323e-01 2.5055870329009e-01 7.3124438978032e-01
3.8704510000873e-01 2.5325818976520e-01 7.1988951932864e-01
4.6345416571834e-02 2.5595771687639e-01 7.0853463506067e-01
7.0545591910900e-01 2.5865728371317e-01 6.9717972739829e-01
3.6435870399645e-01 2.6135688675632e-01 6.8582478441285e-01
2.3032759867687e-02 2.6405651911646e-01 6.7446979073780e-01
6.8145336815626e-01 2.6675616946389e-01 6.6311472620815e-01
3.3959137136647e-01 2.6945582051714e-01 6.5175956418210e-01
9.9741227754784e-01 2.7215544689939e-01 6.4040426951664e-01
6.5487516343818e-01 2.7485501208986e-01 6.2904879621356e-01
3.1193133392177e-01 2.7755446408772e-01 6.1769308484817e-01
9.6852269527274e-01 2.8025372927185e-01 6.0633706008272e-01
6.2457981179005e-01 2.8295270380422e-01 5.9498062891213e-01
2.8001965612028e-01 2.8565124187054e-01 5.8362368088478e-01
9.3474316572119e-01 2.8834914029399e-01 5.7226609248220e-01
5.8863294462954e-01 2.9104612010576e-01 5.6090773914906e-01
2.4155191121770e-01 2.9374180858706e-01 5.4954851979378e-01
8.9334454815579e-01 2.9643573209224e-01 5.3818839853312e-01
5.4384378290858e-01 2.9912734336924e-01 5.2682746323056e-01
1.9288814271953e-01 3.0181612805982e-01 5.1546598237570e-01
9.1183546562160e-02 2.4301939394865e-01 7.8801597395523e-01
7.5127826478935e-01 2.4571863001763e-01 7.7666111870306e-01
4.1124702153570e-01 2.4841787029455e-01 7.6530629124632e-01
7.1082780951956e-02 2.5111712203509e-01 7.5395148022249e-01
7.3077742763628e-01 2.5381639132622e-01 7.4259667447071e-01
3.9032158669638e-01 2.5651568291770e-01 7.3124186268481e-01
4.9704412412077e-02 2.5921499998857e-01 7.1988703298200e-01
7.0891333891592e-01 2.6191434382655e-01 7.0853217236882e-01
3.6793378479434e-01 2.6461371338945e-01 6.9717726608381e-01
2.6748801679448e-02 2.6731310470566e-01 6.8582229679369e-01
6.8533865472261e-01 2.7001251005355e-01 6.7446724362079e-01
3.4368032033739e-01 2.7271191683574e-01 6.6311208098440e-01
1.7468838974549e-03 2.7541130603195e-01 6.5175677725283e-01
6.5950681753466e-01 2.7811065007065e-01 6.4040129323363e-01
3.1692311663446e-01 2.8080990990549e-01 6.2904558058747e-01
9.7395227483796e-01 2.8350903101929e-01 6.1768958035588e-01
6.3054308490934e-01 2.8620793802105e-01 6.0633322197116e-01
2.8663527369622e-01 2.8890652748581e-01 5.9497642340357e-01
9.4215802674785e-01 2.9160465880260e-01 5.8361909353371e-01
5.9702855561850e-01 2.9430214323179e-01 5.7226113842426e-01
2.5115104803764e-01 2.9699873250900e-01 5.6090247379811e-01
9.0441667565108e-01 2.9969411084493e-01 5.4954304629577e-01
5.5670587520314e-01 3.0238789906170e-01 5.3818286487576e-01
2.0789485204171e-01 3.0507968777616e-01 5.2682203872091e-01
8.5786888412151e-01 3.0776912693555e-01 5.1546080566008e-01
4.3423064500772e-01 2.4357874229079e-01 8.1072221857448e-01
9.4476417226470e-02 2.4627795713416e-01 7.9936726097622e-01
7.5460762581272e-01 2.4897713943212e-01 7.8801236666065e-01
4.1461900619402e-01 2.5167629927802e-01 7.7665752002103e-01
7.4504548746482e-02 2.5437544575058e-01 7.6530270594423e-01
7.3425737887647e-01 2.5707458680659e-01 7.5394790959340e-01
3.9386961833506e-01 2.5977372913455e-01 7.4259311613505e-01
5.3332224318269e-02 2.6247287795683e-01 7.3123831040203e-01
7.1263480217040e-01 2.6517203676404e-01 7.1988347648195e-01
3.7176538663291e-01 2.6787120695951e-01 7.0852859721974e-01
3.0710185554382e-02 2.7057038738418e-01 6.9717365362296e-01
6.8945327880389e-01 2.7326957368179e-01 6.8581862416002e-01
3.4797626384373e-01 2.7596875744998e-01 6.7446348394652e-01
6.2578381238627e-03 2.7866792510464e-01 6.6310820382562e-01
6.6427330739022e-01 2.8136705636124e-01 6.5175274936872e-01
3.2199400863281e-01 2.8406612220849e-01 6.4039707985834e-01
9.7938663766758e-01 2.8676508221918e-01 6.2904114737471e-01
6.3641247600260e-01 2.8946388101687e-01 6.1768489620385e-01
2.9302652288674e-01 2.9216244371453e-01 6.0632826293173e-01
9.4917656196105e-01 2.9486067019979e-01 5.9497117780238e-01
6.0480223742841e-01 2.9755842833996e-01 5.8361356819982e-01
2.5983429772440e-01 3.0025554666543e-01 5.7225536542787e-01
9.1419430791181e-01 3.0295180811641e-01 5.6089651616900e-01
5.6779536148683e-01 3.0564694839054e-01 5.4953699974300e-01
2.2054464941651e-01 3.0834066574512e-01 5.3817685079615e-01
8.7234911395469e-01 3.1103265388002e-01 5.2681618303335e-01
5.2312559849410e-01 3.1372267437404e-01 5.1545520155348e-01
7.7736513979105e-01 2.4413805661887e-01 8.3342832747737e-01
4.3774665792482e-01 2.4683739443273e-01 8.2207310758642e-01
9.8022804048521e-02 2.4953665500603e-01 8.1071800086556e-01
7.5818953733355e-01 2.5223585098913e-01 7.9936298780328e-01
4.1824229840037e-01 2.5493499416191e-01 7.8800804948130e-01
7.8175927179614e-02 2.5763409535554e-01 7.7665316746163e-01
7.3798456877115e-01 2.6033316435141e-01 7.6529832363197e-01
3.9766156543132e-01 2.6303220974960e-01 7.5394350000518e-01
5.7199332439945e-02 2.6573123879734e-01 7.4258867846800e-01
7.1658921515434e-01 2.6843025716510e-01 7.3123384047341e-01
3.7582132404124e-01 2.7112926865435e-01 7.1987896667087e-01
3.4884343890352e-02 2.7382827481635e-01 7.0852403646926e-01
6.9376531276599e-01 2.7652727445432e-01 6.9716902752915e-01
3.5244936557910e-01 2.7922626297357e-01 6.8581391518515e-01
1.0919436535077e-02 2.8192523153282e-01 6.7445867180690e-01
6.6915591428012e-01 2.8462416593707e-01 6.6310326612053e-01
3.2713624360901e-01 2.8732304519722e-01 6.5174766253484e-01
9.8483446861676e-01 2.9002183966569e-01 6.4039182055140e-01
6.4222071507281e-01 2.9272050864585e-01 6.2903569439190e-01
2.9926061612415e-01 2.9541899737318e-01 6.1767923305536e-01
9.5591469781602e-01 2.9811723329845e-01 6.0632238112884e-01
6.1213776363775e-01 3.0081512169984e-01 5.9496508081976e-01
2.6787835701217e-01 3.0351254087594e-01 5.8360727584052e-01
9.2307844686961e-01 3.0620933762722e-01 5.7224891790696e-01
5.7767358575600e-01 3.0890532457821e-01 5.6088997657960e-01
2.3159394113136e-01 3.1160028232345e-01 5.4953045271049e-01
8.8476679040220e-01 3.1429397153904e-01 5.3817039437537e-01
5.3712124201886e-01 3.1698616288779e-01 5.2680991119116e-01
1.8859594210960e-01 3.1967669458349e-01 5.1544917780915e-01
1.2060058650987e-01 2.4469682489614e-01 8.5613495814338e-01
7.8110326958301e-01 2.4739645004543e-01 8.4477928444226e-01
4.4150824786490e-01 2.5009594664848e-01 8.3342378552761e-01
1.0181233768125e-01 2.5279532942505e-01 8.2206843871473e-01
7.6201199089792e-01 2.5549461240737e-01 8.1071322180599e-01
4.2210323835316e-01 2.5819380886804e-01 7.9935811307127e-01
8.2081625043345e-02 2.6089293123627e-01 7.8800309119348e-01
7.4194213601213e-01 2.6359199099725e-01 7.7664813517728e-01
4.0167911169380e-01 2.6629099856853e-01 7.6529322421841e-01
6.1286151228135e-02 2.6898996314568e-01 7.5393833753115e-01
7.2075600199363e-01 2.7168889250818e-01 7.4258345413102e-01
3.8008043329787e-01 2.7438779277349e-01 7.3122855257015e-01
3.9250091823465e-02 2.7708666808479e-01 7.1987361062347e-01
6.9825433606549e-01 2.7978552021315e-01 7.0851860492548e-01
3.5708104663908e-01 2.8248434805034e-01 6.9716351056081e-01
1.5716409033248e-02 2.8518314696166e-01 6.8580830061685e-01
6.7414466523297e-01 2.8788190796088e-01 6.7445294571606e-01
3.3234783078963e-01 2.9058061666082e-01 6.6309741355942e-01
9.9030537467465e-01 2.9327925194507e-01 6.5174166853392e-01
6.4799386107965e-01 2.9597778430047e-01 6.4038567146829e-01
3.0538655602153e-01 2.9867617375180e-01 6.2902937966608e-01
9.6245300845224e-01 3.0137436735782e-01 6.1767274740595e-01
6.1915862757457e-01 3.0407229627861e-01 6.0631572717654e-01
2.7546429826157e-01 3.0676987253502e-01 5.9495827200175e-01
9.3132610914544e-01 3.0946698579835e-01 5.8360033929046e-01
5.8669531841613e-01 3.1216350094026e-01 5.7224189666226e-01
2.4151875562681e-01 3.1485925772898e-01 5.6088293005238e-01
8.9573995418195e-01 3.1755407506544e-01 5.4952345389704e-01
5.4930141457888e-01 3.2024776351536e-01 5.3816352205534e-01
2.0214846659579e-01 3.2294015134346e-01 5.2680323600863e-01
8.5423512844971e-01 3.2563112992544e-01 5.1544274368188e-01
8.0325902211932e-01 2.4255420436514e-01 8.9019941703172e-01
4.6395767383209e-01 2.4525449566680e-01 8.7884280729631e-01
1.2456750704817e-01 2.4795458638928e-01 8.6748646662268e-01
7.8508618888759e-01 2.5065449290930e-01 8.5613037027137e-01
4.4551115675983e-01 2.5335423124838e-01 8.4477449358552e-01
1.0583958575140e-01 2.5605381699306e-01 8.3341881208331e-01
7.6606835053187e-01 2.5875326521411e-01 8.2206330152220e-01
4.2619398122927e-01 2.6145259038150e-01 8.1070793793353e-01
8.6212612643013e-02 2.6415180627164e-01 7.9935269762569e-01
7.4611992606241e-01 2.6685092586251e-01 7.8799755715462e-01
4.0591108283832e-01 2.6954996121180e-01 7.7664249325991e-01
6.5580648713631e-02 2.7224892331220e-01 7.6528748276509e-01
7.2512250775569e-01 2.7494782191643e-01 7.5393250244081e-01
3.8452976455112e-01 2.7764666532362e-01 7.4257752882983e-01
4.3794633126324e-02 2.8034546011594e-01 7.3122253803382e-01
7.0290831085403e-01 2.8304421083241e-01 7.1986750546307e-01
3.6186083541828e-01 2.8574291956321e-01 7.0851240555287e-01
2.0640922769450e-02 2.8844158544421e-01 6.9715721145419e-01
6.7923578396770e-01 2.9114020402695e-01 6.8580189471222e-01
3.3763091898623e-01 2.9383876649450e-01 6.7444642495541e-01
9.9580988608720e-01 2.9653725868928e-01 6.6309076963120e-01
6.5375404655900e-01 2.9923565991627e-01 6.5173489384319e-01
3.1144228683246e-01 3.0193394148645e-01 6.4037876037096e-01
9.6885072393338e-01 3.0463206497600e-01 6.2902232998849e-01
6.2595240687692e-01 3.0732998020448e-01 6.1766556224048e-01
2.8271703743252e-01 3.1002762299306e-01 6.0630841688612e-01
9.3911075071898e-01 3.1272491287408e-01 5.9495085626632e-01
5.9509602189344e-01 3.1542175111599e-01 5.8359284887345e-01
2.5063180242170e-01 3.1811801974661e-01 5.7223437435776e-01
9.0567403939525e-01 3.2081358274786e-01 5.6087543001402e-01
5.6017679086046e-01 3.2350829128421e-01 5.4951603833237e-01
2.1409420514559e-01 3.2620199566293e-01 5.3815625430575e-01
8.6738364855414e-01 3.2889456746249e-01 5.2679616972111e-01
5.2001018045433e-01 3.3158593530540e-01 5.1543590967855e-01
1.4667413846862e-01 2.4310950644973e-01 9.1291015027246e-01
8.0746143668948e-01 2.4581050293972e-01 9.0155257970586e-01
4.6816512397458e-01 2.4851124215179e-01 8.9019535048873e-01
1.2878329960522e-01 2.5121174035969e-01 8.7883843810483e-01
7.8931388399877e-01 2.5391201382932e-01 8.6748181756880e-01
4.4975459860456e-01 2.5661207872145e-01 8.5612546357959e-01
1.1010294165215e-01 2.5931195099624e-01 8.4476935065300e-01
7.7035615934911e-01 2.6201164631742e-01 8.3341345323094e-01
4.3051121209185e-01 2.6471117995419e-01 8.2205774576541e-01
9.0564735208220e-02 2.6741056667774e-01 8.1070220277554e-01
7.5051299369473e-01 2.7010982064936e-01 7.9934679887596e-01
4.1035183034203e-01 2.7280895529601e-01 7.8799150877540e-01
7.0076606561121e-02 2.7550798316897e-01 7.7663630724423e-01
7.2968213512848e-01 2.7820691577987e-01 7.6528116905027e-01
3.8916260396394e-01 2.8090576340773e-01 7.5392606886254e-01
4.8511489944194e-02 2.8360453486902e-01 7.4257098112351e-01
7.0772146164517e-01 2.8630323724115e-01 7.3121587989141e-01
3.6678426981160e-01 2.8900187552802e-01 7.1986073865634e-01
2.5690624295182e-02 2.9170045225357e-01 7.0850553013629e-01
6.8443005622088e-01 2.9439896696699e-01 6.9715022606386e-01
3.4299076429566e-01 2.9709741564016e-01 6.8579479697989e-01
1.3594445607677e-03 2.9979578993588e-01 6.7443921205932e-01
6.5952110365908e-01 3.0249407632384e-01 6.6308343900583e-01
3.1745885707432e-01 3.0519225502275e-01 6.5172744406806e-01
9.7515371615031e-01 3.0789029875355e-01 6.4037119225064e-01
6.3258437153440e-01 3.1058817130415e-01 6.2901464781872e-01
2.8972698675457e-01 3.1328582593840e-01 6.1765777522398e-01
9.4655502494908e-01 3.1598320374009e-01 6.0630054060837e-01
6.0303914560961e-01 3.1868023208338e-01 5.9494291406050e-01
2.5914722797522e-01 3.2137682358358e-01 5.8358487278856e-01
9.1484460446007e-01 3.2407287613195e-01 5.7222640529956e-01
5.7009462072196e-01 3.2676827497425e-01 5.6086751648214e-01
2.2485967441042e-01 3.2946289825317e-01 5.4950823310217e-01
8.7910291050867e-01 3.3215662793173e-01 5.3814860854959e-01
5.3279074360007e-01 3.3484936835414e-01 5.2678872467587e-01
1.8589629771399e-01 3.3754107447991e-01 5.1542868733691e-01
4.9027382839713e-01 2.4366242446455e-01 9.3562369026815e-01
1.5113843374220e-01 2.4636429807575e-01 9.2426494934889e-01
8.1192380386570e-01 2.4906586184749e-01 9.1290661723723e-01
4.7262843007951e-01 2.5176713061972e-01 9.0154867168148e-01
1.3325065036378e-01 2.5446811962266e-01 8.9019108931897e-01
7.9378863839857e-01 2.5716884436914e-01 8.7883384586245e-01
4.5424038956850e-01 2.5986932054793e-01 8.6747691627564e-01
1.1460370359275e-01 2.6256956391746e-01 8.5612027493487e-01
7.7487616342047e-01 2.6526959019888e-01 8.4476389577373e-01
4.3505511001624e-01 2.6796941496693e-01 8.3340775240840e-01
9.5137612637991e-02 2.7066905353678e-01 8.2205181824123e-01
7.5512043418237e-01 2.7336852084431e-01 8.1069606654076e-01
4.1499999113712e-01 2.7606783131703e-01 7.9934047049665e-01
7.4772307637578e-02 2.7876699873217e-01 7.8798500324811e-01
7.3443296307532e-01 2.8146603605768e-01 7.7662963788526e-01
3.9397703265183e-01 2.8416495527145e-01 7.6527434742300e-01
5.3399020213640e-02 2.8686376715275e-01 7.5391910474797e-01
7.1269278265149e-01 2.8956248103926e-01 7.4256388254007e-01
3.7185144510571e-01 2.9226110454120e-01 7.3120865317154e-01
3.0867306207304e-02 2.9495964320331e-01 7.1985338858861e-01
6.8973173262452e-01 2.9765810010319e-01 7.0849806018384e-01
3.4843504231487e-01 3.0035647537345e-01 6.9714263867092e-01
6.9663761619391e-03 3.0305476563341e-01 6.8578709397999e-01
6.6531355819226e-01 3.0575296331575e-01 6.7443139519832e-01
3.2346294545297e-01 3.0845105587446e-01 6.6307551059170e-01
9.8139927008013e-01 3.1114902486449e-01 6.5171940775430e-01
6.3910547838171e-01 3.1384684489294e-01 6.4036305395003e-01
2.9656257529180e-01 3.1654448245970e-01 6.2900641672648e-01
9.5374948785559e-01 3.1924189473808e-01 6.1764946489966e-01
6.1064296914815e-01 3.2193902840055e-01 6.0629217002144e-01
2.6721757497229e-01 3.2463581868160e-01 5.9493450844152e-01
9.2344576048149e-01 3.2733218900207e-01 5.8357646404634e-01
5.7929816266530e-01 3.3002805166952e-01 5.7221803167452e-01
2.3474415615213e-01 3.3272331042233e-01 5.6085922103269e-01
8.8975278989397e-01 3.3541786588564e-01 5.4950006062302e-01
5.4429422187855e-01 3.3811162529185e-01 5.3814060070068e-01
1.9834175185401e-01 3.4080451794092e-01 5.2678091360332e-01
8.5187448478942e-01 3.4349651757167e-01 5.1542108904278e-01
8.3408157120698e-01 2.4421248679615e-01 9.5834060848580e-01
4.9501394215298e-01 2.4691538555717e-01 9.4698051740903e-01
1.5587051676476e-01 2.4961793038078e-01 9.3562089176613e-01
8.1665017903960e-01 2.5232013331119e-01 9.2426171359886e-01
4.7735166852741e-01 2.5502200717961e-01 9.1290296320200e-01
1.3797357542275e-01 2.5772356550102e-01 9.0154461930728e-01
7.9851433364933e-01 2.6042482236856e-01 8.9018665926853e-01
4.5897221162102e-01 2.6312579234592e-01 8.7882905924453e-01
1.1934530035354e-01 2.6582649035773e-01 8.6747179437582e-01
7.7963149859272e-01 2.6852693157751e-01 8.5611483895230e-01
4.3982849461607e-01 2.7122713131239e-01 8.4475816656837e-01
9.9933744354775e-02 2.7392710488350e-01 8.3340175026308e-01
7.5994444547083e-01 2.7662686750040e-01 8.2204556264258e-01
4.1985750700914e-01 2.7932643412763e-01 8.1068957598300e-01
7.9669514226223e-02 2.8202581934077e-01 7.9933376231210e-01
7.3937668817613e-01 2.8472503716918e-01 7.8797809346833e-01
3.9897483961112e-01 2.8742410092172e-01 7.7662254113685e-01
5.8459316732130e-02 2.9012302299139e-01 7.6526707686251e-01
7.1782494630525e-01 2.9282181463384e-01 7.5391167204071e-01
3.7706596765263e-01 2.9552048571404e-01 7.4255629788836e-01
3.6175959038834e-02 2.9821904441441e-01 7.3120092539857e-01
6.9514775602286e-01 3.0091749689675e-01 7.1984552528508e-01
3.5397336145583e-01 3.0361584690935e-01 7.0849006792507e-01
1.2643847010157e-02 3.0631409532998e-01 6.9713452331290e-01
6.7114924648150e-01 3.0901223963523e-01 6.8577886104239e-01
3.2947844168723e-01 3.1171027328748e-01 6.7442305034149e-01
9.8761904270588e-01 3.1440818503324e-01 6.6306706019139e-01
6.4555726561909e-01 3.1710595811267e-01 6.5171085957172e-01
3.0327781304740e-01 3.1980356939061e-01 6.4035441788430e-01
9.6076376590269e-01 3.2250098843824e-01 6.2899770561952e-01
6.1799649927572e-01 3.2519817662560e-01 6.1764069533822e-01
2.7495564165637e-01 3.2789508633329e-01 6.0628336304517e-01
9.3161910514643e-01 3.3059166046487e-01 5.9492569001916e-01
5.8796322518223e-01 3.3328783254609e-01 5.8356766512749e-01
2.4396306121041e-01 3.3598352783869e-01 5.7220928757111e-01
8.9959292328620e-01 3.3867866607349e-01 5.6085056985659e-01
5.5482720014542e-01 3.4137316659868e-01 5.4949154054586e-01
2.0964156518651e-01 3.4406695689320e-01 5.3813224597723e-01
8.6401461720564e-01 3.4675998540441e-01 5.2677274969690e-01
5.1792995866092e-01 3.4945223936424e-01 5.1541312788022e-01
1.7811630496137e-01 2.4475932253964e-01 9.8106134860768e-01
8.3910934630592e-01 2.4746335854394e-01 9.6969977207479e-01
5.0002895738940e-01 2.5016700890473e-01 9.5833870149803e-01
1.6087443501723e-01 2.5287028167577e-01 9.4697812495087e-01
8.2164493169923e-01 2.5557318602971e-01 9.3561802823276e-01
4.8233945407174e-01 2.5827573217754e-01 9.2425839501419e-01
1.4295686019099e-01 2.6097793128129e-01 9.1289920699494e-01
8.0349585543607e-01 2.6367979536065e-01 9.0154044407205e-01
4.6395498673955e-01 2.6638133719415e-01 8.9018208451382e-01
1.2433263484812e-01 2.6908257021515e-01 8.7882410513651e-01
7.8462700430306e-01 2.7178350840262e-01 8.6746648147994e-01
4.4483611082068e-01 2.7448416616673e-01 8.5610918797878e-01
1.0495776574541e-01 2.7718455822848e-01 8.4475219812633e-01
7.6498955724184e-01 2.7988469949263e-01 8.3339548462777e-01
4.2492882788628e-01 2.8258460491269e-01 8.2203901954032e-01
8.4772648313561e-02 2.8528428934628e-01 8.1068277439815e-01
7.4451778656965e-01 2.8798376739899e-01 7.9932672032016e-01
4.0416067281662e-01 2.9068305325411e-01 7.8797082809951e-01
6.3697359034884e-02 2.9338216048548e-01 7.7661506827411e-01
7.2312347337059e-01 2.9608110184984e-01 7.6525941117847e-01
3.8243416878762e-01 2.9877988905471e-01 7.5390382697787e-01
4.1624065711434e-02 3.0147853249733e-01 7.4254828568744e-01
7.0068718840501e-01 3.0417704096914e-01 7.3119275718007e-01
3.5961689491124e-01 3.0687542132033e-01 7.1983721118934e-01
1.8405800553760e-02 3.0957367807807e-01 7.0848161731627e-01
6.7704569524130e-01 3.1227181301224e-01 6.9712594505214e-01
3.3552745521658e-01 3.1496982464317e-01 6.8577016383383e-01
9.9384095049453e-01 3.1766770768736e-01 6.7441424315364e-01
6.5197495013769e-01 3.2036545244095e-01 6.6305815275148e-01
3.0991702884230e-01 3.2306304410746e-01 6.5170186292465e-01
9.6765348021913e-01 3.2576046208711e-01 6.4034534499768e-01
6.2516924485136e-01 3.2845767926362e-01 6.2898857200097e-01
2.8244786493581e-01 3.3115466135191e-01 6.1763151961052e-01
9.3947148230195e-01 3.3385136641198e-01 6.0627416739706e-01
5.9622090302982e-01 3.3654774469358e-01 5.9491650041632e-01
2.5267575970809e-01 3.3924373905768e-01 5.8355851113290e-01
9.0881481107256e-01 3.4193928632443e-01 5.7220020159635e-01
5.6461642694052e-01 3.4463432001834e-01 5.6084158566372e-01
2.2005931116602e-01 3.4732877510031e-01 5.4948269087464e-01
8.7512351192225e-01 3.5002259535054e-01 5.3812355932802e-01
5.2979174989608e-01 3.5271574402212e-01 5.2676424660538e-01
1.8405105264051e-01 3.5540821811090e-01 5.1540481750676e-01
8.6249011545455e-01 2.4259360544168e-01 1.5153919645666e-02
5.2360192468576e-01 2.4529931137799e-01 3.7901795916496e-03
1.8343994327319e-01 2.4800792987074e-01 9.9242305472884e-01
8.4441728503264e-01 2.5071276773596e-01 9.8106043979457e-01
5.0532210087426e-01 2.5341720683083e-01 9.6969834666636e-01
1.6615397540162e-01 2.5612125170856e-01 9.5833676804306e-01
8.2691234641969e-01 2.5882490820778e-01 9.4697569409728e-01
4.8759650408987e-01 2.6152818340049e-01 9.3561511257416e-01
1.4820558941480e-01 2.6423108553016e-01 9.2425500890984e-01
8.0873859185327e-01 2.6693362394030e-01 9.1289536636697e-01
4.6919434583587e-01 2.6963580899422e-01 9.0153616618430e-01
1.2957152592609e-01 2.7233765198625e-01 8.9017738773701e-01
7.8986864035136e-01 2.7503916504466e-01 8.7881900870464e-01
4.5008402261191e-01 2.7774036102646e-01 8.6746100524302e-01
1.1021582086335e-01 2.8044125340395e-01 8.5610335215686e-01
7.7026198476065e-01 2.8314185614260e-01 8.4474602306996e-01
4.3022024944667e-01 2.8584218356977e-01 8.3338899058966e-01
9.0088116366730e-02 2.8854225023314e-01 8.2203222646312e-01
7.4986283059265e-01 2.9124207074793e-01 8.1067570172282e-01
4.0954135434446e-01 2.9394165963109e-01 7.9931938681945e-01
6.9120336407470e-02 2.9664103112063e-01 7.8796325174076e-01
7.2859607715699e-01 2.9934019897785e-01 7.7660726611573e-01
3.8796448892613e-01 3.0203917626975e-01 7.6525139930408e-01
4.7221051487838e-02 3.0473797512844e-01 7.5389562047234e-01
7.0636076247654e-01 3.0743660648427e-01 7.4253989865877e-01
3.6537808265822e-01 3.1013507976866e-01 7.3118420283127e-01
2.4266876083074e-02 3.1283340258281e-01 7.1982850194404e-01
6.8302034534357e-01 3.1553158032811e-01 7.0847276500148e-01
3.4163096243996e-01 3.1822961579490e-01 6.9711696114067e-01
9.0396165790501e-05 3.2092750870719e-01 6.8576105974734e-01
6.5838943752038e-01 3.2362525522327e-01 6.7440503062463e-01
3.1651792550720e-01 3.2632284739652e-01 6.6304884423832e-01
9.7446467687946e-01 3.2902027260743e-01 6.5169247206759e-01
6.3221742506449e-01 3.3171751298878e-01 6.4033588709433e-01
2.8976277577449e-01 3.3441454488269e-01 6.2897906446745e-01
9.4708618983761e-01 3.3711133839254e-01 6.1762198237748e-01
6.0417200768067e-01 3.3980785712774e-01 6.0626462316938e-01
2.6100353469602e-01 3.4250405828694e-01 5.9490697470243e-01
9.1756321225856e-01 3.4519989328697e-01 5.8354903192831e-01
5.7383290486465e-01 3.4789530922014e-01 5.7219079859461e-01
2.2979433852624e-01 3.5059025150321e-01 5.6083228888025e-01
8.8542972697596e-01 3.5328466815299e-01 5.4947352862702e-01
5.4072261698916e-01 3.5597851615096e-01 5.3811455564734e-01
1.9565896760696e-01 3.5867177029332e-01 5.2675541838621e-01
8.5022844535541e-01 3.6136443468973e-01 5.1539617204729e-01
2.0699063934271e-01 2.4313202928513e-01 3.7884748674525e-02
8.6815854420376e-01 2.4583894390783e-01 2.6519440490814e-02
5.2925399522996e-01 2.4854546205485e-01 1.5154650257979e-02
1.9027731513478e-01 2.5125158042770e-01 3.7903836332474e-03
8.5000723425206e-01 2.5396066655120e-01 9.9242266774665e-01
5.1089593984269e-01 2.5666591685372e-01 9.8105952607609e-01
1.7171238551988e-01 2.5937076400957e-01 9.6969691049892e-01
8.3245630516717e-01 2.6207521020177e-01 9.5833481599654e-01
4.9312728428131e-01 2.6477925896947e-01 9.4697323492676e-01
1.5372475885379e-01 2.6748291517877e-01 9.3561215709111e-01
8.1424801377299e-01 2.7018618498164e-01 9.2425156982484e-01
4.7469618058901e-01 2.7288907576321e-01 9.1289145810753e-01
1.3506823444773e-01 2.7559159607753e-01 9.0153180469188e-01
7.9536298997001e-01 2.7829375557211e-01 8.9017259024791e-01
4.5557909582542e-01 2.8099556490122e-01 8.7881379351938e-01
1.1571502772955e-01 2.8369703562807e-01 8.6745539148951e-01
7.7576907957774e-01 2.8639818011583e-01 8.5609735955241e-01
4.3573935241817e-01 2.8909901140717e-01 8.4473967168716e-01
9.5623740962085e-02 2.9179954309197e-01 8.3338230063154e-01
7.5541991732956e-01 2.9449978916250e-01 8.2202521805221e-01
4.1512531173502e-01 2.9719976385529e-01 8.1066839470920e-01
7.4737089828938e-02 2.9989948147849e-01 7.9931180061219e-01
7.3425212643092e-01 3.0259895622337e-01 7.8795540516730e-01
3.9366697541685e-01 3.0529820195830e-01 7.7659917731329e-01
5.2977835561273e-02 3.0799723200326e-01 7.6524308564713e-01
7.1218051218976e-01 3.1069605888280e-01 7.5388709853981e-01
3.7127037457065e-01 3.1339469405492e-01 7.4253118424457e-01
3.0242309080121e-02 3.1609314761358e-01 7.3117531100107e-01
6.8909066832106e-01 3.1879142796229e-01 7.1981944714101e-01
3.4780921658301e-01 3.2148954145679e-01 7.0846356120269e-01
6.3910723203193e-03 3.2418749201570e-01 6.9710762206473e-01
6.6482864873257e-01 3.2688528069931e-01 6.8575159911191e-01
3.2311359409752e-01 3.2958290525967e-01 6.7439546244954e-01
9.8123673428804e-01 3.3228035966944e-01 6.6303918318620e-01
6.3918802097064e-01 3.3497763364343e-01 6.5168273380774e-01
2.9695649041533e-01 3.3767471217749e-01 6.4032608866790e-01
9.5453023973295e-01 3.4037157514357e-01 6.2896922462155e-01
6.1189642977823e-01 3.4306819700137e-01 6.1761212182293e-01
2.6904132695702e-01 3.4576454671496e-01 6.0625476470207e-01
9.2595039970822e-01 3.4846058800080e-01 5.9489714311313e-01
5.8260848927223e-01 3.5115628007959e-01 5.8353925361440e-01
2.3900007797835e-01 3.5385157915789e-01 5.7218110078538e-01
8.9510968068307e-01 3.5654644091900e-01 5.6082269840698e-01
5.5092238451726e-01 3.5924082434200e-01 5.4946407022327e-01
2.0642455636097e-01 3.6193469717029e-01 5.3810524987285e-01
8.6160472348925e-01 3.6462804327857e-01 5.2674627944306e-01
5.1645460777743e-01 3.6732087199604e-01 5.1538720600193e-01
5.5172616193330e-01 2.4366701996818e-01 6.0619636017681e-02
2.1295403550999e-01 2.4637508973979e-01 4.9252827606001e-02
8.7410766613861e-01 2.4908278519034e-01 3.7886506823904e-02
5.3518781347340e-01 2.5179009823945e-01 2.6520686453961e-02
1.9619509191301e-01 2.5449702198909e-01 1.5155376909140e-02
8.5712997110037e-01 2.5720355078676e-01 3.7905861203214e-03
5.1675221791281e-01 2.5991310317432e-01 9.9242228106887e-01
1.7755216887429e-01 2.6261876776869e-01 9.8105861092803e-01
8.3828004358319e-01 2.6532402674745e-01 9.6969546913171e-01
4.9893572821655e-01 2.6802887987354e-01 9.5833285300274e-01
1.5951896058649e-01 2.7073332831346e-01 9.4697075716889e-01
8.2002932863748e-01 2.7343737463246e-01 9.3560917359723e-01
4.8046626862450e-01 2.7614102277653e-01 9.2424809165378e-01
1.4082906285930e-01 2.7884427804089e-01 9.1288749818700e-01
8.0111683687154e-01 2.8154714702474e-01 9.0152737763335e-01
4.6132855579511e-01 2.8424963757219e-01 8.9016771214261e-01
1.2146301975832e-01 2.8695175869933e-01 8.7880848172028e-01
7.8151885803023e-01 2.8965352050726e-01 8.6744966438417e-01
4.4149452165528e-01 2.9235493408104e-01 8.5609123633202e-01
1.0138827429436e-01 2.9505601137439e-01 8.4473317211704e-01
7.6119818098385e-01 2.9775676507976e-01 8.3337544482823e-01
4.2092209452407e-01 3.0045720848344e-01 8.2201802627249e-01
8.0557639216970e-02 3.0315735530518e-01 8.1066088715576e-01
7.4010219168874e-01 3.0585721952153e-01 7.9930399726096e-01
3.9955285855889e-01 3.0855681517206e-01 7.8794732562069e-01
5.8906450753618e-02 3.1125615614742e-01 7.7659084068365e-01
7.1815945431123e-01 3.1395525595805e-01 7.6523451047418e-01
3.7730799759409e-01 3.1665412748222e-01 7.5387830274552e-01
3.6347814910723e-02 3.1935278269206e-01 7.4252218512837e-01
6.9527420667089e-01 3.2205123235644e-01 7.3116612527787e-01
3.5408199635593e-01 3.2474948571962e-01 7.1981009102349e-01
1.2765484802158e-02 3.2744755015539e-01 7.0845405052853e-01
6.7131840258478e-01 3.3014543079741e-01 6.9709797246768e-01
3.2973386168190e-01 3.3284313014833e-01 6.8574182623395e-01
9.8800430812383e-01 3.3554064767285e-01 6.7438558218824e-01
6.4612147803617e-01 3.3823797938455e-01 6.6302921196767e-01
3.0407636041348e-01 3.4093511744242e-01 6.5167268887035e-01
9.6185917116942e-01 3.4363204978229e-01 6.4031598833554e-01
6.1945934455159e-01 3.4632875982130e-01 6.2895908853651e-01
2.7686554992121e-01 3.4902522629103e-01 6.1760197109918e-01
9.3406574417664e-01 3.5172142327806e-01 6.0624462194925e-01
5.9104727265778e-01 3.5441732057973e-01 5.9488703227225e-01
2.4779703397563e-01 3.5711288451741e-01 5.8352919954116e-01
9.0430172640129e-01 3.5980807938664e-01 5.7217112852146e-01
5.6054819442687e-01 3.6250286975806e-01 5.6081283210102e-01
2.1652389265396e-01 3.6519722386243e-01 5.4945433171187e-01
8.7221747862293e-01 3.6789111828155e-01 5.3809565701857e-01
5.2761953466279e-01 3.7058454409784e-01 5.2673684445872e-01
1.8272339969524e-01 3.7327751450030e-01 5.1537793416485e-01
8.9668099687771e-01 2.4419883484837e-01 8.3358273478925e-02
5.5797588446666e-01 2.4690795944548e-01 7.1990089111478e-02
2.1919355971172e-01 2.4961674610951e-01 6.0622343801070e-02
8.8033520171599e-01 2.5232518284459e-01 4.9255056193594e-02
5.4140184479566e-01 2.5503325853070e-01 3.7888243052811e-02
2.0239438103011e-01 2.5774096301854e-01 2.6521919091208e-02
8.6331356158599e-01 2.6044828721926e-01 1.5156096812417e-02
5.2415999706787e-01 2.6315522318804e-01 3.7907863686634e-03
1.8367494837726e-01 2.6586523923418e-01 9.9242189609055e-01
8.4438597520735e-01 2.6857131874988e-01 9.8105769780456e-01
5.0502502427188e-01 2.7127699211482e-01 9.6969402805510e-01
1.6559213502830e-01 2.7398225663345e-01 9.5833088656479e-01
8.2608720023901e-01 2.7668711103701e-01 9.4696827031580e-01
4.8650996397661e-01 2.7939155550472e-01 9.3560617354968e-01
1.4686001943867e-01 2.8209559167080e-01 9.2424458779462e-01
8.0713680650827e-01 2.8479922261656e-01 9.1288350191950e-01
4.6733960895097e-01 2.8750245284695e-01 9.0152290221362e-01
1.2746755109742e-01 2.9020528825108e-01 8.9016277249045e-01
7.8751959382297e-01 2.9290773604631e-01 8.7880309421296e-01
4.4749452960332e-01 2.9560980470567e-01 8.6744384663833e-01
1.0739097639972e-01 2.9831150386824e-01 8.5608500697878e-01
7.6720737010801e-01 3.0101284423255e-01 8.4472655057583e-01
4.2694195529609e-01 3.0371383743247e-01 8.3336845108457e-01
8.6592773951844e-02 3.0641449589556e-01 8.2201068066515e-01
7.4615765197178e-01 3.0911483268357e-01 8.1065321017833e-01
4.0563418313841e-01 3.1181486131463e-01 7.9929600938277e-01
6.5019710363852e-02 3.1451459556680e-01 7.8793904713158e-01
7.2431130401957e-01 3.1721404926249e-01 7.7658229156684e-01
3.8350573722974e-01 3.1991323603325e-01 7.6522571031088e-01
4.2599458081621e-02 3.2261216906442e-01 7.5386927065454e-01
7.0158855880552e-01 3.2531086081929e-01 7.4251293974334e-01
3.6046874210516e-01 3.2800932274271e-01 7.3115668476373e-01
1.9235284986040e-02 3.3070756494435e-01 7.1980047313327e-01
6.7788300064663e-01 3.3340559586294e-01 7.0844427270001e-01
3.3640619927987e-01 3.3610342191355e-01 6.9708805195801e-01
9.9479864900214e-01 3.3880104712240e-01 6.8573178028822e-01
6.5305353860798e-01 3.4149847275602e-01 6.7437542823536e-01
3.1116344447122e-01 3.4419569695600e-01 6.6301896783336e-01
9.6912030471222e-01 3.4689271439627e-01 6.5166237299260e-01
6.2691540475887e-01 3.4958951598796e-01 6.4030561996251e-01
2.8453937967469e-01 3.5228608866793e-01 6.2894868788031e-01
9.4198224011965e-01 3.5498241532146e-01 6.1759155941196e-01
5.9923343051680e-01 3.5767847490810e-01 6.0623422148109e-01
2.5628192981944e-01 3.6037424288173e-01 5.9487666606512e-01
9.1311640699176e-01 3.6306969202135e-01 5.8351889101228e-01
5.6972544454154e-01 3.6576479381434e-01 5.7216090079696e-01
2.2609784355500e-01 3.6845952055521e-01 5.6080270708185e-01
8.8222302179226e-01 3.7115384833020e-01 5.4944432889596e-01
5.3809151137143e-01 3.7384776103950e-01 5.3808579217214e-01
1.9369555319133e-01 3.7654125554699e-01 5.2672712832893e-01
8.4902977061373e-01 3.7923434792316e-01 5.1536837155181e-01
2.4183266281170e-01 2.4472785412161e-01 1.0610020667975e-01
9.0320437188595e-01 2.4743789529641e-01 9.4730816079075e-02
5.6449482441223e-01 2.5014764575223e-01 8.3361802291506e-02
2.2570560565826e-01 2.5285709078303e-01 7.1993188062945e-02
8.8683814936262e-01 2.5556621617236e-01 6.0624994874366e-02
5.4789374475317e-01 2.5827500830163e-01 4.9257242749554e-02
2.0887354143314e-01 2.6098345425803e-01 3.7889950062174e-02
8.6977855244856e-01 2.6369154194152e-01 2.6523133346100e-02
5.3060965583874e-01 2.6639926016994e-01 1.5156807113009e-02
1.9136759495506e-01 2.6910659878081e-01 3.7909836811854e-03
8.5077557991721e-01 2.7181707461505e-01 9.9242151422398e-01
5.1139746551908e-01 2.7452356843790e-01 9.8105679018148e-01
1.7194738454154e-01 2.7722965754313e-01 9.6969259275946e-01
8.3242552868569e-01 2.7993533674973e-01 9.5832892413836e-01
4.9283194634483e-01 2.8264060230171e-01 9.4696578373947e-01
1.5316654001197e-01 2.8534545191649e-01 9.3560316818844e-01
8.1342906359279e-01 2.8804988481887e-01 9.2424107130806e-01
4.7361911961224e-01 2.9075390175908e-01 9.1287948414037e-01
1.3373615625441e-01 2.9345750501382e-01 9.0151839499781e-01
7.9377946412775e-01 2.9616069836952e-01 8.9015778954224e-01
4.5374817260412e-01 2.9886348708675e-01 8.7879765089036e-01
1.1364124554139e-01 3.0156587784553e-01 8.6743795974313e-01
7.7345747616681e-01 3.0426787867089e-01 8.5607869453690e-01
4.3319548087376e-01 3.0696949883841e-01 8.4471983161318e-01
9.2853691668956e-02 3.0967074875959e-01 8.3336134540414e-01
7.5243034700129e-01 3.1237163984687e-01 8.2200320863060e-01
4.1192348070887e-01 3.1507218435811e-01 8.1064539250955e-01
7.1330908838377e-02 3.1777239522070e-01 7.9928786696832e-01
7.3065021412128e-01 3.2047228583501e-01 7.8793060086273e-01
3.8987872793731e-01 3.2317186985747e-01 7.7657356219743e-01
4.9013509657508e-02 3.2587116096325e-01 7.6521671834676e-01
7.0805132334112e-01 3.2857017258888e-01 7.5386003627556e-01
3.6698861186594e-01 3.3126891765531e-01 7.4250348276028e-01
2.5821468705138e-02 3.3396740827227e-01 7.3114702461159e-01
6.8454560773327e-01 3.3666565542545e-01 7.1979062890136e-01
3.4315633165845e-01 3.3936366864887e-01 7.0843426319772e-01
1.6484999493966e-03 3.4206145568602e-01 6.9707789581394e-01
6.6001649746985e-01 3.4475902214537e-01 6.8572149607795e-01
3.1825420546800e-01 3.4745637115859e-01 6.7436503463077e-01
9.7635497711445e-01 3.5015350305339e-01 6.6300848376337e-01
6.3431162046277e-01 3.5285041505829e-01 6.5165181780134e-01
2.9211639253902e-01 3.5554710106368e-01 6.4029501354654e-01
9.4976100926343e-01 3.5824355147252e-01 6.2893805078167e-01
6.0723667705846e-01 3.6093975318609e-01 6.1758091283868e-01
2.6453415326035e-01 3.6363568978441e-01 6.0622358722250e-01
9.2164384372151e-01 3.6633134197780e-01 5.9486606626683e-01
5.7855594707876e-01 3.6902668842450e-01 5.8350834777744e-01
2.3526065574759e-01 3.7172170702581e-01 5.7215043558914e-01
8.9174842331199e-01 3.7441637682276e-01 5.6079233992464e-01
5.4801030596817e-01 3.7711068061814e-01 5.4943407739979e-01
2.0403838127596e-01 3.7980460842675e-01 5.3807567047314e-01
8.5982623991071e-01 3.8249816180280e-01 5.2671714609981e-01
5.1536953491415e-01 3.8519135899432e-01 5.1535853333530e-01
9.2560462989367e-01 2.4254353729489e-01 1.4021554545663e-01
5.8715369666140e-01 2.4525455109437e-01 1.2884486833749e-01
2.4861428901804e-01 2.4796534415114e-01 1.1747447242370e-01
9.0998859924369e-01 2.5067590041745e-01 1.0610438300985e-01
5.7127862855202e-01 2.5338620383144e-01 9.4734624933389e-02
2.3248620500464e-01 2.5609623841837e-01 8.3365222412179e-02
8.9361299790754e-01 2.5880598839429e-01 7.1996198866140e-02
5.5466052904213e-01 2.6151543827294e-01 6.0627576725994e-02
2.1563018107394e-01 2.6422457297605e-01 4.9259377232625e-02
8.7652320347542e-01 2.6693337794683e-01 3.7891620230831e-02
5.3734071629318e-01 2.6964183926599e-01 2.6524323961481e-02
1.9808371208267e-01 2.7234994376935e-01 1.5157504856322e-02
8.5875305632381e-01 2.7505767916550e-01 3.7911773397298e-03
5.1805436113104e-01 2.7776860960604e-01 9.9242113691549e-01
1.7858684624606e-01 2.8047551583898e-01 9.8105589159885e-01
8.3904728721515e-01 2.8318202079802e-01 9.6969116880400e-01
4.9943602487682e-01 2.8588811680186e-01 9.5832697322655e-01
1.5975326196778e-01 2.8859379756614e-01 9.4696330681186e-01
8.1999905983003e-01 2.9129905828030e-01 9.3560016868053e-01
4.8017333508331e-01 2.9400389567012e-01 9.2423755508438e-01
1.4027585630786e-01 2.9670830804392e-01 9.1287545939389e-01
8.0030624073005e-01 2.9941229532092e-01 9.0151387211773e-01
4.6026395085141e-01 3.0211585904016e-01 8.9015278095393e-01
1.2014829091125e-01 3.0481900234878e-01 8.7879217087173e-01
7.7995840302789e-01 3.0752172996872e-01 8.6743202422253e-01
4.3969326282307e-01 3.1022404814112e-01 8.5607232087768e-01
9.9351674302772e-02 3.1292596454787e-01 8.4471303839062e-01
7.5893226374499e-01 3.1562748821011e-01 8.3335415218026e-01
4.1843347233363e-01 3.1832862936332e-01 8.2199563573268e-01
7.7853547278620e-02 3.2102939930936e-01 8.1063746081770e-01
7.3719053117687e-01 3.2372981024534e-01 7.9927959771740e-01
3.9644224939820e-01 3.2642987506980e-01 7.8792201546350e-01
5.5606295326446e-02 3.2912960716668e-01 7.7656468208122e-01
7.1468001335012e-01 3.3182902016773e-01 7.6520756483747e-01
3.7366047958175e-01 3.3452812769437e-01 7.5385063049204e-01
3.2544480393495e-02 3.3722694308022e-01 7.4249384555127e-01
6.9132848899268e-01 3.3992547907613e-01 7.3113717652452e-01
3.5000864042964e-01 3.4262374754020e-01 7.1978059018499e-01
8.5807056399148e-03 3.4532175911618e-01 7.0842405383754e-01
6.6704006538031e-01 3.4801952290504e-01 6.9706753559723e-01
3.2538168523672e-01 3.5071704613617e-01 6.8571100468383e-01
9.8360009327167e-01 3.5341433384764e-01 6.7435443173801e-01
6.4168936235599e-01 3.5611138858773e-01 6.6299778916615e-01
2.9964309980450e-01 3.5880821015514e-01 6.5164105151995e-01
9.5745444761849e-01 3.6150479540083e-01 6.4028419591646e-01
6.1511609742835e-01 3.6420113812229e-01 6.2892720250087e-01
2.7262032510891e-01 3.6689722909039e-01 6.1757005494947e-01
9.2995905095939e-01 3.6959305626018e-01 6.0621274100170e-01
5.8712393220388e-01 3.7228860522956e-01 5.9485525299754e-01
2.4410649521229e-01 3.7498386002248e-01 5.8349758837878e-01
9.0089831501315e-01 3.7767880428466e-01 5.7213975008946e-01
5.5749124900863e-01 3.8037342298544e-01 5.6078174678146e-01
2.1387772984647e-01 3.8306770471560e-01 5.4942359269891e-01
8.7005111862991e-01 3.8576164464958e-01 5.3806530708252e-01
5.2600611357982e-01 3.8845524819486e-01 5.2670691291031e-01
1.8173920067070e-01 3.9114853527225e-01 5.1534843478643e-01
2.7095552949306e-01 2.4306792667709e-01 1.6296301173268e-01
9.3261380738753e-01 2.4577945770905e-01 1.5159161749228e-01
5.9417710625350e-01 2.4849082408857e-01 1.4022043241142e-01
2.5564814572152e-01 2.5120201007177e-01 1.2884948137706e-01
9.1702939755635e-01 2.5391299955313e-01 1.1747878933795e-01
5.7832311531161e-01 2.5662377615088e-01 1.0610838119784e-01
2.3953135928229e-01 2.5933432329419e-01 9.4738281683834e-02
9.0065601716653e-01 2.6204462431403e-01 8.3368515192080e-02
5.6169882081922e-01 2.6475466253910e-01 7.1999105613258e-02
2.2266135946082e-01 2.6746442139803e-01 6.0630076140627e-02
8.8354508969240e-01 2.7017388452812e-01 4.9261449063882e-02
5.4435134266173e-01 2.7288303589083e-01 3.7893245552423e-02
2.0508132872148e-01 2.7559185989348e-01 2.6525485432000e-02
8.6573613991803e-01 2.7830034151627e-01 1.5158186959078e-02
5.2631675064593e-01 2.8100846644304e-01 3.7913665974606e-03
1.8551162520251e-01 2.8371984493133e-01 9.9242076566127e-01
8.4595441588546e-01 2.8642716033494e-01 9.8105500570198e-01
5.0632499532893e-01 2.8913407996846e-01 9.6968976188400e-01
1.6662384823797e-01 2.9184059364852e-01 9.5832504147365e-01
8.2685132837399e-01 2.9454669253306e-01 9.4696084902510e-01
4.8700765446295e-01 2.9725236922742e-01 9.3559718626593e-01
1.4709290608977e-01 2.9995761787701e-01 9.2423405201385e-01
8.0710701968460e-01 3.0266243424385e-01 9.1287144212662e-01
4.6704978465204e-01 3.0536681576477e-01 9.0150934948662e-01
1.2692083963745e-01 3.0807076158915e-01 8.9014776402056e-01
7.8671966886854e-01 3.1077427259447e-01 8.7878667275411e-01
4.4644559845746e-01 3.1347735137812e-01 8.6742605990042e-01
1.0609779250056e-01 3.1618000222452e-01 8.5606590698091e-01
7.6567524877253e-01 3.1888223104651e-01 8.4470619297598e-01
4.2517679378133e-01 3.2158404530084e-01 8.3334689450291e-01
8.4601076930901e-02 3.2428545387719e-01 8.2198798601784e-01
7.4394656353315e-01 3.2698646696106e-01 8.1062944003848e-01
4.0321152642036e-01 3.2968709587077e-01 7.9927122738417e-01
6.2394035934671e-02 3.3238735286914e-01 7.8791331743001e-01
7.2149194811583e-01 3.3508725095070e-01 7.7655567837193e-01
3.8050289097170e-01 3.3778680360575e-01 7.6519827750004e-01
3.9424248801544e-02 3.4048602456258e-01 7.5384108147822e-01
6.9825314465114e-01 3.4318492751016e-01 7.4248405662830e-01
3.5698642111495e-01 3.4588352580348e-01 7.3112716921838e-01
1.5620619867164e-02 3.4858183215531e-01 7.1977038575540e-01
6.7415196050593e-01 3.5127985831836e-01 7.0841367328331e-01
3.3257631953902e-01 3.5397761476370e-01 6.9705699968912e-01
9.9088921063216e-01 3.5667511036277e-01 6.8570033401988e-01
6.4908576759102e-01 3.5937235208278e-01 6.7434364681458e-01
3.0716073196209e-01 3.6206934470831e-01 6.6298691045526e-01
9.6510844762332e-01 3.6476609060579e-01 6.5163009954103e-01
6.2292286529324e-01 3.6746258955275e-01 6.4027319128775e-01
2.8059756050982e-01 3.7015883865966e-01 6.2891616595315e-01
9.3812576929559e-01 3.7285483242001e-01 6.1755900728228e-01
5.9550044638320e-01 3.7555056293245e-01 6.0620170296117e-01
2.5271435144198e-01 3.7824602034831e-01 5.9484424505553e-01
9.0976016908431e-01 3.8094119360656e-01 5.8348663039697e-01
5.6663066834187e-01 3.8363607152497e-01 5.7212886086067e-01
2.2331890651615e-01 3.8633064431849e-01 5.6077094345622e-01
8.7981848049674e-01 3.8902490560930e-01 5.4941289012923e-01
5.3612382545073e-01 3.9171885497318e-01 5.3805471714880e-01
1.9223055592787e-01 3.9441250102805e-01 5.2669644394024e-01
8.4813583780582e-01 3.9710586500810e-01 5.1533809122305e-01
6.1639821302822e-01 2.4359125891197e-01 1.8571168155123e-01
2.7818212990581e-01 2.4630312936374e-01 1.7433978063148e-01
9.3986336344173e-01 2.4901488880560e-01 1.6296802181279e-01
6.0144531967886e-01 2.5172652284239e-01 1.5159642802876e-01
2.6293106344771e-01 2.5443801644308e-01 1.4022502262509e-01
9.2432336364552e-01 2.5714935401088e-01 1.2885382932876e-01
5.8562473230664e-01 2.5986051945091e-01 1.1748287218656e-01
2.4683745803811e-01 2.6257149623858e-01 1.0611217547447e-01
9.0796363431972e-01 2.6528226749127e-01 9.4741763579485e-02
5.6900518311085e-01 2.6799281604577e-01 8.3371660855369e-02
2.2996387416546e-01 2.7070312454313e-01 7.2001891454585e-02
8.9084134042788e-01 2.7341317552249e-01 6.0632479138572e-02
5.5163908986587e-01 2.7612295152468e-01 4.9263447069232e-02
2.1235851408836e-01 2.7883243520589e-01 3.7894817584892e-02
8.7300089409362e-01 2.8154160946145e-01 2.6526611964520e-02
5.3356740349442e-01 2.8425045755873e-01 1.5158850184422e-02
1.9405910956969e-01 2.8695896327793e-01 3.7915506721187e-03
8.5314777619287e-01 2.8967078179562e-01 9.9242040202186e-01
5.1350055339191e-01 2.9237850170943e-01 9.8105413627951e-01
1.7378086266775e-01 2.9508583347246e-01 9.6968837789424e-01
8.3398932501269e-01 2.9779276441215e-01 9.5832313675525e-01
4.9412643974920e-01 3.0049928311244e-01 9.4695842010875e-01
1.5419257954822e-01 3.0320537954965e-01 9.3559423240178e-01
8.1418798541097e-01 3.0591104521657e-01 9.2423057515718e-01
4.7411276178482e-01 3.0861627323127e-01 9.1286744688283e-01
1.3396687192808e-01 3.1132105842772e-01 9.0150484301772e-01
7.9375013357821e-01 3.1402539742547e-01 8.9014275591627e-01
4.5346221491597e-01 3.1672928867591e-01 8.7878117487153e-01
1.1310263075799e-01 3.1943273248306e-01 8.6742008617710e-01
7.7267073885302e-01 3.2213573099736e-01 8.5605947322660e-01
4.3216573610725e-01 3.2483828818113e-01 8.4469931664888e-01
9.1586654524017e-02 3.2754040974500e-01 8.3333959447646e-01
7.5093235661389e-01 3.3024210305492e-01 8.2198028234425e-01
4.1020153001772e-01 3.3294337700963e-01 8.1062135371498e-01
6.9392681086386e-02 3.3564424188916e-01 7.9926278012793e-01
7.2850412718157e-01 3.3834470917501e-01 7.8790453146702e-01
3.8753398750101e-01 3.4104479134321e-01 7.7654657624483e-01
4.6480172292815e-02 3.4374450163186e-01 7.6518888189913e-01
7.0534037040691e-01 3.4644385378520e-01 7.5383141509891e-01
3.6411203523936e-01 3.4914286177677e-01 7.4247414205750e-01
2.2792369260193e-02 3.5184153951506e-01 7.3111702885118e-01
6.8137830747801e-01 3.5453990053559e-01 7.1976004174201e-01
3.3986650039049e-01 3.5723795768467e-01 7.0840314750510e-01
9.9825329719998e-01 3.5993572280112e-01 6.9704631376063e-01
6.5653473034414e-01 3.6263320640422e-01 6.8568950931236e-01
3.1470650270533e-01 3.6533041739789e-01 6.7433270449444e-01
9.7276397922404e-01 3.6802736280413e-01 6.6297587152865e-01
6.3070218505133e-01 3.7072404754190e-01 6.5161898489391e-01
2.8851581283074e-01 3.7342047427186e-01 6.4026202170821e-01
9.4619924218840e-01 3.7611664333234e-01 6.2890496212113e-01
6.0374657500820e-01 3.7881255279785e-01 6.1754778971025e-01
2.6115169052967e-01 3.8150819869768e-01 6.0619049186891e-01
9.1840832465770e-01 3.8420357543892e-01 5.9483306016364e-01
5.7551017800354e-01 3.8689867648400e-01 5.8347549062795e-01
2.3245105693158e-01 3.8959349533671e-01 5.7211778394447e-01
8.8922505106545e-01 3.9228802689010e-01 5.6075994545047e-01
5.4582674907761e-01 3.9498226918234e-01 5.4940198488438e-01
2.0225149191161e-01 3.9767622558844e-01 5.3804391577481e-01
8.5849565869508e-01 4.0036990744338e-01 5.2668575436440e-01
5.1455697547627e-01 4.0306333704233e-01 5.1532751796400e-01
9.6190170747461e-01 2.4411406919047e-01 2.0846090689319e-01
6.2382936010463e-01 2.4682611284999e-01 1.9708869043509e-01
2.8564505723420e-01 2.4953809385194e-01 1.8571655826083e-01
9.4735315560350e-01 2.5224999991919e-01 1.7434453026321e-01
6.0895752847369e-01 2.5496181793582e-01 1.6297262689345e-01
2.7046163193018e-01 2.5767353401197e-01 1.5160086921753e-01
9.3186856268717e-01 2.6038513353870e-01 1.4022927893768e-01
5.9318110833336e-01 2.6309660123733e-01 1.2885787838066e-01
2.5440179080257e-01 2.6580792120706e-01 1.1748669045423e-01
9.1553290372636e-01 2.6851907697415e-01 1.0611573857307e-01
5.7657654422527e-01 2.7123005154572e-01 9.4745046555025e-02
2.3753463961784e-01 2.7394082747084e-01 8.3374638489079e-02
8.9840896946921e-01 2.7665138691110e-01 7.2004538576217e-02
5.5920118336154e-01 2.7936171172254e-01 6.0634770945085e-02
2.1991281474389e-01 2.8207178355024e-01 4.9265359444562e-02
8.8054529120658e-01 2.8478158393643e-01 3.7896327416035e-02
5.4109994152306e-01 2.8749109444221e-01 2.6527697448759e-02
2.0157799980558e-01 2.9020029678264e-01 1.5159491122263e-02
8.6198060712957e-01 2.9290917297388e-01 3.7917287404279e-03
5.2096327130904e-01 2.9562142194430e-01 9.9242004763452e-01
1.8122570261229e-01 2.9832954019031e-01 9.8105328729722e-01
8.4141532043317e-01 3.0103728007988e-01 9.6968702298707e-01
5.0153287197313e-01 3.0374462648082e-01 9.5832126726242e-01
1.6157899434294e-01 3.0645156540165e-01 9.4695603014128e-01
8.2155420861278e-01 3.0915808415761e-01 9.3559131890168e-01
4.8145891375391e-01 3.1186417152700e-01 9.2422713791229e-01
1.4129338071136e-01 3.1456981789406e-01 9.1286348849785e-01
8.0105774679197e-01 3.1727501537450e-01 9.0150036884284e-01
4.6075201048935e-01 3.1997975792005e-01 8.9013777393608e-01
1.2037602680097e-01 3.2268404139907e-01 8.7877569555801e-01
7.7992950302450e-01 3.2538786365032e-01 8.6741412231111e-01
4.3941199495459e-01 3.2809122450754e-01 8.5605303969337e-01
9.8822903340558e-02 3.3079412579343e-01 8.4469243021333e-01
7.5816147041385e-01 3.3349657128142e-01 8.3333227354466e-01
4.1742677625380e-01 3.3619856662479e-01 8.2197254671746e-01
7.6617734735200e-02 3.3890011925290e-01 8.1061322434278e-01
7.3573308879291e-01 3.4160123823488e-01 7.9925427886662e-01
3.9477140475039e-01 3.4430193411156e-01 7.8789568084926e-01
5.3731065491504e-02 3.4700221869713e-01 7.7653739926579e-01
7.1261026231051e-01 3.4970210485236e-01 7.6517940182372e-01
3.7140698535482e-01 3.5240160623185e-01 7.5382165529388e-01
3.0119012679859e-02 3.5510073700854e-01 7.4246412585124e-01
6.8874389806711e-01 3.5779951157923e-01 7.3110677942257e-01
3.4727895791520e-01 3.6049794425600e-01 7.1974958203894e-01
5.7212577023349e-03 3.6319604894924e-01 7.0839250019124e-01
6.6406759873727e-01 3.6589383884939e-01 6.9703550118809e-01
3.2231450616662e-01 3.6859132611602e-01 6.8567855351567e-01
9.8045821949072e-01 3.7128852158488e-01 6.7432162719989e-01
6.3849468715719e-01 3.7398543450557e-01 6.6296469417096e-01
2.9641956715002e-01 3.7668207232587e-01 6.5160772863046e-01
9.5422823586406e-01 3.7937844054157e-01 6.4025070741936e-01
6.1191580793844e-01 3.8207454263501e-01 6.2889361038348e-01
2.6947717009134e-01 3.8477038012993e-01 6.1753642072935e-01
9.2690703231347e-01 3.8746595279472e-01 6.0617912535760e-01
5.8419999997510e-01 3.9016125903107e-01 5.9482171515451e-01
2.4135067039457e-01 3.9285629648858e-01 5.8346418521215e-01
8.9835375708336e-01 3.9555106294736e-01 5.7210653493668e-01
5.5520424407910e-01 3.9824555750912e-01 5.6074876799130e-01
2.1189757133694e-01 4.0093978212884e-01 5.4939089200746e-01
8.6842984991735e-01 4.0363374350364e-01 5.3803291798700e-01
5.2479810258574e-01 4.0632745530803e-01 5.2667485931274e-01
1.8100052145119e-01 4.0902094072527e-01 5.1531673028930e-01
3.0743794594883e-01 2.4463685109860e-01 2.3121008892026e-01
9.6952962110554e-01 2.4734892039017e-01 2.1983771833060e-01
6.3149786246231e-01 2.5006096814424e-01 2.0846538653839e-01
2.9334838412074e-01 2.5277298471315e-01 1.9709310979068e-01
9.5508620690866e-01 2.5548495945257e-01 1.8572090482841e-01
6.1671575448287e-01 2.5819688079882e-01 1.7434878903725e-01
2.7824093700758e-01 2.6090873632583e-01 1.6297678055749e-01
9.3966522407619e-01 2.6362051278779e-01 1.5160489835640e-01
6.0099170824418e-01 2.6633219615205e-01 1.4023316226599e-01
2.6222316029446e-01 2.6904377162693e-01 1.2886159298775e-01
9.2336207715429e-01 2.7175522368799e-01 1.1749021206575e-01
5.8441072321882e-01 2.7446653610649e-01 1.0611904182874e-01
2.4537116570561e-01 2.7717769198324e-01 9.4748105301924e-02
9.0624530456248e-01 2.7988867379078e-01 8.3377426088781e-02
5.6703489737424e-01 2.8259946342667e-01 7.2007028223689e-02
2.2774157965903e-01 2.8531004228010e-01 6.0636935996246e-02
8.8836688091030e-01 2.8802039131365e-01 4.9267173748763e-02
5.4891223672109e-01 2.9073049116167e-01 3.7897765648979e-02
2.0937899732258e-01 2.9344032224589e-01 2.6528735440758e-02
8.6976843287365e-01 2.9614986490830e-01 1.5160106176274e-02
5.3008173585081e-01 2.9885909956070e-01 3.7918999339963e-03
1.8895859909226e-01 3.0157176773753e-01 9.9241970422303e-01
8.4913035269515e-01 3.0428027650817e-01 9.8105246292615e-01
5.0922885997454e-01 3.0698841895292e-01 9.6968570362254e-01
1.6925498153025e-01 3.0969617752857e-01 9.5831944157690e-01
8.2920948196618e-01 3.1240353568424e-01 9.4695368965246e-01
4.8909302291335e-01 3.1511047805702e-01 9.3558845806639e-01
1.4890615581707e-01 3.1781699066131e-01 9.2422375417376e-01
8.0864931479958e-01 3.2052306106707e-01 9.1285958228578e-01
4.6832280985750e-01 3.2322867856245e-01 9.0149594352707e-01
1.2792682059042e-01 3.2593383429638e-01 8.9013283573595e-01
7.8746139058686e-01 3.2863852139698e-01 8.7877025341057e-01
4.4692642251921e-01 3.3134273506235e-01 8.6740818770273e-01
1.0632167392377e-01 3.3404647262064e-01 8.5604662645981e-01
7.6564675357000e-01 3.3674973355695e-01 8.4468555431439e-01
4.2490111825840e-01 3.3945251950531e-01 8.3332495281995e-01
8.4084069832757e-02 3.4215483420456e-01 8.2196480063007e-01
7.4319475215394e-01 3.4485668341776e-01 8.1060507371782e-01
4.0223214776164e-01 3.4755807481507e-01 7.9924574563136e-01
6.1195073950509e-02 3.5025901782125e-01 7.8788678778134e-01
7.2008217801001e-01 3.5295952342886e-01 7.7652816975532e-01
3.7889193142509e-01 3.5565960397967e-01 7.6516985965455e-01
3.7622622908015e-02 3.5835927291678e-01 7.5381182444819e-01
6.9627235023200e-01 3.6105854451128e-01 7.4245403034065e-01
3.5483901096467e-01 3.6375743356773e-01 7.3109644314797e-01
1.3320292353781e-02 3.6645595511380e-01 7.1973902867948e-01
6.7171366080007e-01 3.6915412408049e-01 7.0838175312198e-01
3.3001635156057e-01 3.7185195498071e-01 6.9702458342382e-01
9.8822535956124e-01 3.7454946159508e-01 6.8566748767718e-01
6.4633743245770e-01 3.7724665667605e-01 6.7431043549695e-01
3.0434906736493e-01 3.7994355168307e-01 6.6295339839488e-01
9.6225651297649e-01 3.8264015656405e-01 6.5159635014731e-01
6.2005577910127e-01 3.8533647960116e-01 6.4023926715360e-01
2.7774265594788e-01 3.8803252734204e-01 6.2888212878086e-01
9.3531274575879e-01 3.9072830464078e-01 6.1752491768718e-01
5.9276150960354e-01 3.9342381483650e-01 6.0616762011156e-01
2.5008433223093e-01 3.9611906010034e-01 5.9481022611265e-01
9.0727660778338e-01 3.9881404198379e-01 5.8345272973108e-01
5.6433384880278e-01 4.0150876220131e-01 5.7209512904123e-01
2.2125182019706e-01 4.0420322367752e-01 5.6073742604886e-01
8.7802669857698e-01 4.0689743188139e-01 5.4937962638113e-01
5.3465525551146e-01 4.0959139645570e-01 5.3802173870876e-01
1.9113506073637e-01 4.1228513312778e-01 5.2666377383684e-01
8.4746469822909e-01 4.1497866585549e-01 5.1530574340616e-01
9.9055656295902e-01 2.4244808741207e-01 2.6533109417976e-01
6.5298277901203e-01 2.4516004116409e-01 2.5395869857242e-01
3.1526202311096e-01 2.4787201149282e-01 2.4258629814629e-01
9.7740314250659e-01 2.5058399302955e-01 2.3121390540858e-01
6.3941381729947e-01 2.5329597908981e-01 2.1984153277141e-01
3.0130072519189e-01 2.5600796183376e-01 2.0846919287393e-01
9.6306968294674e-01 2.5871993238103e-01 1.9709689884021e-01
6.2472576831146e-01 2.6143188089099e-01 1.8572466448306e-01
2.8627342546760e-01 2.6414379661707e-01 1.7435250446173e-01
9.4771655649632e-01 2.6685566794140e-01 1.6298043439946e-01
6.0905860088311e-01 2.6956748239556e-01 1.5160847096530e-01
2.7030260470348e-01 2.7227922667177e-01 1.4023663192301e-01
9.3145128081891e-01 2.7499088662896e-01 1.2886493614888e-01
5.9250706115819e-01 2.7770244729719e-01 1.1749340361940e-01
2.5347214195218e-01 2.8041389288446e-01 1.0612205536909e-01
9.1434852262477e-01 2.8312520678892e-01 9.4750913418280e-02
5.7513803891227e-01 2.8583637162016e-01 8.3380000670666e-02
2.3584239068340e-01 2.8854736923241e-01 7.2009340780394e-02
8.9646316485958e-01 2.9125818077250e-01 6.0638957988652e-02
5.5700185378583e-01 2.9396878674520e-01 4.9268876930324e-02
2.1745986937502e-01 2.9667916709755e-01 3.7899122411826e-02
8.7783855333712e-01 2.9938930132388e-01 2.6529719162032e-02
5.3813918381020e-01 3.0209916859197e-01 1.5160691559124e-02
1.9836297872434e-01 3.0480874789029e-01 3.7920633370836e-03
8.5713425348733e-01 3.0752182223713e-01 9.9241937360418e-01
5.1721501441788e-01 3.1023071197048e-01 9.8105166756337e-01
1.7722201150991e-01 3.1293924970429e-01 9.6968442660837e-01
8.3715620294649e-01 3.1564741555495e-01 9.5831766873434e-01
4.9701846774745e-01 3.1835519044845e-01 9.4695140971295e-01
1.5680959773497e-01 3.2106255634476e-01 9.3558566280200e-01
8.1653028907338e-01 3.2376949646000e-01 9.2422043848059e-01
4.7618113376824e-01 3.2647599548114e-01 9.1285574421698e-01
1.3576261146354e-01 3.2918203976769e-01 9.0149158427532e-01
7.9527508181580e-01 3.3188761753503e-01 8.9012795956665e-01
4.5471877765191e-01 3.3459271901430e-01 8.7876486754847e-01
1.1409379903772e-01 3.3729733658447e-01 8.6740230217586e-01
7.7340010829954e-01 3.4000146487244e-01 8.5604025390595e-01
4.3263752595712e-01 3.4270510081800e-01 8.4467870975607e-01
9.1805727447623e-02 3.4540824370109e-01 8.3331765341475e-01
7.5090424045068e-01 3.4811089512959e-01 8.2195706540345e-01
4.0993244257048e-01 3.5081305898676e-01 8.1059692328593e-01
6.8889559094048e-02 3.5351474133805e-01 7.9923720192127e-01
7.2777466053066e-01 3.5621595029806e-01 7.8787787375568e-01
3.8658665964727e-01 3.5891669585907e-01 7.7651890914807e-01
4.5324307751830e-02 3.6161698968327e-01 7.6516027672373e-01
7.0398619004109e-01 3.6431684486194e-01 7.5380194375047e-01
3.6257071992295e-01 3.6701627564536e-01 7.4244387653172e-01
2.1076132345542e-02 3.6971529714847e-01 7.3108604081120e-01
6.7950047631518e-01 3.7241392503796e-01 7.1972840218429e-01
3.3784160696195e-01 3.7511217520783e-01 7.0837092651150e-01
9.9609717771303e-01 3.7781006345173e-01 6.9701358033020e-01
6.5426463335732e-01 3.8050760514137e-01 6.8565633126102e-01
3.1234120502708e-01 3.8320481492246e-01 6.7429914840594e-01
9.7032390837776e-01 3.8590170644080e-01 6.6294200273506e-01
6.2820954650998e-01 3.8859829211362e-01 6.5158486745888e-01
2.8599471943614e-01 3.9129458296303e-01 6.4022771838227e-01
9.4367584213514e-01 3.9399058853120e-01 6.2887053423462e-01
6.0124917343912e-01 3.9668631689896e-01 6.1751329696858e-01
2.5871085812586e-01 3.9938177483201e-01 6.0615599201615e-01
9.1605698460791e-01 4.0207696808077e-01 5.9479860848621e-01
5.7328366045789e-01 4.0477190186055e-01 5.8344113928211e-01
2.3038710762193e-01 4.0746658153817e-01 5.7208358111088e-01
8.8736377847085e-01 4.1016101354750e-01 5.6072593434841e-01
5.4421049274272e-01 4.1285520654911e-01 5.4936820271852e-01
2.0092459387741e-01 4.1554917283698e-01 5.3801039273842e-01
8.5750412121085e-01 4.1824292997632e-01 5.2665251288247e-01
5.1394799203524e-01 4.2093650263115e-01 5.1529457242087e-01
3.3585133337824e-01 2.4297227617216e-01 2.8807861850814e-01
9.9851660738448e-01 2.4568401118956e-01 2.7670628756269e-01
6.6101202532017e-01 2.4839578298989e-01 2.6533393957495e-01
3.2334987458354e-01 2.5110758977432e-01 2.5396158394952e-01
9.8554071799887e-01 2.5381942810397e-01 2.4258922935408e-01
6.4759364950341e-01 2.5653129316234e-01 2.3121688421864e-01
3.0951650977200e-01 2.5924317894240e-01 2.1984455713183e-01
9.7131606899212e-01 2.6195507837776e-01 2.0847225715495e-01
6.3299818258163e-01 2.6466698343152e-01 1.9709999407044e-01
2.9456792450545e-01 2.6737888515317e-01 1.8572777857729e-01
9.5602970193765e-01 2.7009077371085e-01 1.7435562244298e-01
6.1738735428402e-01 2.7280263840513e-01 1.6298353861875e-01
2.7864423899131e-01 2.7551446766901e-01 1.5161154132289e-01
9.3980330609227e-01 2.7822624905849e-01 1.4023964609506e-01
6.0086716304953e-01 2.8093796923749e-01 1.2886786982314e-01
2.6183813114748e-01 2.8364961396074e-01 1.1749623074311e-01
9.2271829442739e-01 2.8636116805845e-01 1.0612474841163e-01
5.8350954195627e-01 2.8907261542604e-01 9.4753443650400e-02
2.4421360405784e-01 2.9178393902282e-01 8.3382338461164e-02
9.0483208300740e-01 2.9449512088276e-01 7.2011455910099e-02
5.6536647859858e-01 2.9720614214093e-01 6.0640819980563e-02
2.2581820892387e-01 2.9991698307854e-01 4.9270455393571e-02
8.8618862667031e-01 3.0262762318922e-01 3.7900387396021e-02
5.4647903121340e-01 3.0533804126870e-01 2.6530641517522e-02
2.0669067679305e-01 3.0804821552924e-01 1.5161243297659e-02
8.6682477707135e-01 3.1075812373953e-01 3.7922179866513e-03
5.2549070378589e-01 3.1347158930458e-01 9.9241905769002e-01
1.8548019239649e-01 3.1618084855069e-01 9.8105090584340e-01
8.4539531877348e-01 3.1888977247334e-01 9.6968319912686e-01
5.0523711956246e-01 3.2159833894446e-01 9.5831595827196e-01
1.6500657203886e-01 3.2430652642708e-01 9.4694920200711e-01
8.2470458502539e-01 3.2701431422655e-01 9.3558294672103e-01
4.8433198912438e-01 3.2972168274491e-01 9.2421720614770e-01
1.4388952672297e-01 3.3242861373240e-01 9.1285199108085e-01
8.0337784219323e-01 3.3513509052962e-01 9.0148730912610e-01
4.6279747265502e-01 3.3784109829419e-01 8.9012316449751e-01
1.2214883959949e-01 3.4054662420564e-01 8.7875955786462e-01
7.8143224158716e-01 3.4325165764296e-01 8.6739648625443e-01
4.4064784814280e-01 3.4595619032967e-01 8.5603394301154e-01
9.9795694873024e-02 3.4866021644225e-01 8.4467191781798e-01
7.5887567973836e-01 3.5136373267824e-01 8.3331039677281e-01
4.1788756032534e-01 3.5406673828157e-01 8.2194936253017e-01
7.6830951890373e-02 3.5676923502358e-01 8.1058879449294e-01
7.3570532589225e-01 3.5947122713893e-01 7.9922866905813e-01
3.9451000869743e-01 3.6217272121683e-01 7.8786895990905e-01
5.3244180136201e-02 3.6487372604885e-01 7.7650963834869e-01
7.1190687161030e-01 3.6757425243550e-01 7.6515067366788e-01
3.7049696350587e-01 3.7027431295490e-01 7.5379203354180e-01
2.9013181750236e-02 3.7297392169765e-01 7.4243368444811e-01
6.8745409346617e-01 3.7567309397309e-01 7.3107559210007e-01
3.4581810182239e-01 3.7837184599351e-01 7.1971772188822e-01
4.1034403511854e-03 3.8107019454339e-01 7.0836003932472e-01
6.6230816719995e-01 3.8376815664275e-01 6.9700251048468e-01
3.2043015999956e-01 3.8646574921434e-01 6.8564510243939e-01
9.7846711226211e-01 3.8916298876642e-01 6.7428778367680e-01
6.3641653245857e-01 3.9185989110379e-01 6.6293052450481e-01
2.9427574716278e-01 3.9455647108214e-01 6.5157329743280e-01
9.5204190987268e-01 3.9725274242170e-01 6.4021607752654e-01
6.0971201731601e-01 3.9994871759860e-01 6.2885884273044e-01
2.6728293519769e-01 4.0264440783368e-01 6.1750157414947e-01
9.2475143542196e-01 4.0533982319988e-01 6.0614425628025e-01
5.8211424679106e-01 4.0803497287051e-01 5.9478687717769e-01
2.3936812099993e-01 4.1072986553042e-01 5.8342942853871e-01
8.9650991536242e-01 4.1342450997049e-01 5.7207190567989e-01
5.5353669306346e-01 4.1611891588255e-01 5.6071430738036e-01
2.1044584077987e-01 4.1881309486440e-01 5.4935663555659e-01
8.6723520221132e-01 4.2150706163414e-01 5.3799889473249e-01
5.2390322440916e-01 4.2420083543756e-01 5.2664109126828e-01
1.8044911183422e-01 4.2689444161104e-01 5.1528323231642e-01
6.8107504460717e-01 2.4349761504414e-01 3.1082473699318e-01
3.4402478850212e-01 2.4620906728725e-01 2.9945249049555e-01
6.7728934406173e-03 2.4892056651258e-01 2.8808023145011e-01
6.6933685098103e-01 2.5163211511773e-01 2.7670796667237e-01
3.3173153101703e-01 2.5434371330805e-01 2.6533570126507e-01
9.9396960035364e-01 2.5705535953925e-01 2.5396343940786e-01
6.5606186790434e-01 2.5976705083026e-01 2.4259118496355e-01
3.1801757109270e-01 2.6247878298321e-01 2.3121894194395e-01
9.7984461492763e-01 2.6519055073605e-01 2.1984671486931e-01
6.4154977279763e-01 2.6790234786507e-01 2.0847450904668e-01
3.0313885614068e-01 2.7061416724955e-01 1.9710233078672e-01
9.6461685866781e-01 2.7332600090735e-01 1.8573018757307e-01
6.2598807965661e-01 2.7603784000773e-01 1.7435808819451e-01
2.8725622991233e-01 2.7874967486653e-01 1.6298604284707e-01
9.4842452326444e-01 2.8146149492776e-01 1.5161406321052e-01
6.0949575588324e-01 2.8417328873536e-01 1.4024216250193e-01
2.7047237523170e-01 2.8688504389850e-01 1.2887035550734e-01
9.3135654009027e-01 2.8959674705391e-01 1.1749865859132e-01
5.9215017278618e-01 2.9230838382870e-01 1.0612708968336e-01
2.5285500451265e-01 2.9501993880735e-01 9.4755668239225e-02
9.1347261442651e-01 2.9773139550660e-01 8.3384415175227e-02
5.7400446305726e-01 3.0044273636207e-01 7.2013352772950e-02
2.3445192044315e-01 3.0315394273041e-01 6.0642504552274e-02
8.9481628932348e-01 3.0586499491070e-01 4.9271895110767e-02
5.5509882366121e-01 3.0857587218847e-01 3.7901549928181e-02
2.1530074273988e-01 3.1128655290530e-01 2.6531495135599e-02
8.7542324107346e-01 3.1399701455627e-01 1.5161757249865e-02
5.3546749438203e-01 3.1670723391682e-01 3.7923628751378e-03
1.9402836049049e-01 3.1942107370826e-01 9.9241875848520e-01
8.5392635318312e-01 3.2213068899105e-01 9.8105018263847e-01
5.1374930955250e-01 3.2483998801978e-01 9.6968202874559e-01
1.7349832196057e-01 3.2754894654670e-01 9.5831432025668e-01
8.3317444636036e-01 3.3025754065983e-01 9.4694707888420e-01
4.9277869222714e-01 3.3296574705821e-01 9.3558032422132e-01
1.5231201146962e-01 3.3567354333680e-01 9.2421407337567e-01
8.1177528684797e-01 3.3838090827430e-01 9.1284834062829e-01
4.7116932040456e-01 3.4108782211673e-01 9.0148313712724e-01
1.3049482237040e-01 3.4379426684943e-01 8.9011847062458e-01
7.8975240094521e-01 3.4650022645018e-01 8.7875434526460e-01
4.4894255326592e-01 3.4920568711652e-01 8.6739076142955e-01
1.0806565778038e-01 3.5191063746096e-01 8.5602771564789e-01
7.6712196813586e-01 3.5461506866839e-01 8.4466520056793e-01
4.2611160858244e-01 3.5731897461126e-01 8.3330320499849e-01
8.5034570785610e-02 3.6002235191873e-01 8.2194171401579e-01
7.4389071184751e-01 3.6272519999741e-01 8.1058070913479e-01
4.0267975325911e-01 3.6542752100236e-01 7.9922016854103e-01
6.1401280450596e-02 3.6812931975799e-01 7.8786006737823e-01
7.2005474258098e-01 3.7083060363007e-01 7.7650037808546e-01
3.7863945221206e-01 3.7353138235070e-01 7.6514107077701e-01
3.7154584550040e-02 3.7623166779963e-01 7.5378211365748e-01
6.9559917601092e-01 3.7893147374614e-01 7.4242347346415e-01
3.5397212197237e-01 3.8163081555723e-01 7.3106511592875e-01
1.2272173713808e-02 3.8432970987861e-01 7.1970700625073e-01
6.7049793471523e-01 3.8702817429655e-01 7.0834910957428e-01
3.2864785667767e-01 3.8972622698981e-01 6.9699139146202e-01
9.8672023584093e-01 3.9242388638204e-01 6.8563381835849e-01
6.4471321039752e-01 3.9512117080668e-01 6.7427635803704e-01
3.0262476002951e-01 3.9781809819744e-01 6.6291898002428e-01
9.6045270881516e-01 4.0051468581912e-01 6.5156165599623e-01
6.1819473295400e-01 4.0321095005459e-01 6.4020436014015e-01
2.7584837492656e-01 4.0590690626524e-01 6.2884706947570e-01
9.3341106582065e-01 4.0860256874325e-01 6.1748976412746e-01
5.9088015759729e-01 4.1129795077453e-01 6.0613242753959e-01
2.4825296700714e-01 4.1399306483170e-01 5.9477504662036e-01
9.0552683267212e-01 4.1668792291553e-01 5.8341761180137e-01
5.6269918647599e-01 4.1938253706119e-01 5.7206011699257e-01
2.1976763982636e-01 4.2207692002198e-01 5.6070255941039e-01
8.7673008452184e-01 4.2477108613655e-01 5.4934493925290e-01
5.3358480686184e-01 4.2746505237651e-01 5.3798725919415e-01
1.9033061227944e-01 4.3015883955776e-01 5.2662952367049e-01
8.4696695620823e-01 4.3285247368249e-01 5.1527173793586e-01
2.6200997991762e-02 2.4402432189828e-01 3.3356921489200e-01
6.8949807901951e-01 2.4673545400828e-01 3.2219701961801e-01
3.5254782095154e-01 2.4944663203528e-01 3.1082483553319e-01
1.5375880503608e-02 2.5215786375557e-01 2.9945266741174e-01
6.7800375875332e-01 2.5486915376587e-01 2.8808051679520e-01
3.4044953207388e-01 2.5758050425639e-01 2.7670838331601e-01
2.7284390319533e-03 2.6029191554373e-01 2.6533626566583e-01
6.6485335494473e-01 2.6300338644097e-01 2.5396416230323e-01
3.2683517832038e-01 2.6571491451528e-01 2.4259207197159e-01
9.8868314778336e-01 2.6842649626602e-01 2.3121999407995e-01
6.5040510384170e-01 2.7113812724518e-01 2.1984792898533e-01
3.1200770663554e-01 2.7384980213474e-01 2.0847587820460e-01
9.7349661835213e-01 2.7656151479091e-01 1.9710384457581e-01
6.3487665711060e-01 2.7927325826230e-01 1.8573183238334e-01
2.9615192766557e-01 2.8198502478704e-01 1.7435984745642e-01
9.5732593314830e-01 2.8469680577286e-01 1.6298789724741e-01
6.1840167117725e-01 2.8740859176340e-01 1.5161599089366e-01
2.7938171697052e-01 2.9012037239391e-01 1.4024413926466e-01
9.4026829553548e-01 2.9283213633914e-01 1.2887235499464e-01
6.0106334456505e-01 2.9554387125671e-01 1.1750065249961e-01
2.6176856931124e-01 2.9825556372924e-01 1.0612904797682e-01
9.2238549041707e-01 3.0096719920878e-01 9.4757559383772e-02
5.8291548545559e-01 3.0367876196753e-01 8.3386206393680e-02
2.4335982473982e-01 3.0639023505897e-01 7.2015010323802e-02
9.0371970182333e-01 3.0910160029351e-01 6.0643994033113e-02
5.6399625900464e-01 3.1181283823322e-01 4.9273181786114e-02
2.2419060807422e-01 3.1452392820959e-01 3.7902599080016e-02
8.8430384649969e-01 3.1723484836826e-01 2.6532272433397e-02
5.4433706922927e-01 3.1994557574403e-01 1.5162229135466e-02
2.0429137630319e-01 3.2265608636873e-01 3.7924969564211e-03
8.6274753722372e-01 3.2537028123706e-01 9.9241847807838e-01
5.2255388931866e-01 3.2808023691772e-01 9.8104950304509e-01
1.8228446212184e-01 3.3078989783457e-01 9.6968092340860e-01
8.4194037701693e-01 3.3349923777748e-01 9.5831276528917e-01
5.0152274528190e-01 3.3620823057962e-01 9.4694505338282e-01
1.6103265716305e-01 3.3891685041262e-01 9.3557781053675e-01
8.2047116951416e-01 3.4162507209915e-01 9.2421105733243e-01
4.7983929258775e-01 3.4433287143561e-01 9.1284481168742e-01
1.3913797656541e-01 3.4704022551700e-01 9.0147908848721e-01
7.9836809838761e-01 3.4974711305577e-01 8.9011389925766e-01
4.5753044939020e-01 3.5245351468594e-01 8.7874925188795e-01
1.1662572417611e-01 3.5515941324429e-01 8.6738515041256e-01
7.7565451105031e-01 3.5786479402075e-01 8.5602159485916e-01
4.3461728423049e-01 3.6056964497105e-01 8.4465858116744e-01
9.3514397920956e-02 3.6327395688542e-01 8.3329610118174e-01
7.5234608221107e-01 3.6597772350851e-01 8.2193414271847e-01
4.1111244064146e-01 3.6868094160695e-01 8.1057268970703e-01
6.9813449177401e-02 3.7138361098210e-01 7.9921172240111e-01
7.2844895624790e-01 3.7408573442716e-01 7.8785121765564e-01
3.8701868345711e-01 3.7678731762902e-01 7.7649114926302e-01
4.5522226555842e-02 3.7948836901655e-01 7.6513148834112e-01
7.0395905627966e-01 3.8218889955852e-01 7.5377220376469e-01
3.6232851871702e-01 3.8488892251551e-01 7.4241326263108e-01
2.0629834966447e-02 3.8758845315152e-01 7.3105463075091e-01
6.7886209997352e-01 3.9028750841241e-01 7.1969627315423e-01
3.3702428060346e-01 3.9298610657950e-01 7.0833815460276e-01
9.9511521319657e-01 3.9568426690807e-01 6.9698024009919e-01
6.5313360106618e-01 3.9838200926174e-01 6.8562249538502e-01
3.1107801262201e-01 4.0107935375519e-01 6.7426488741871e-01
9.6894688102630e-01 4.0377632041868e-01 6.6290738482678e-01
6.2673850650483e-01 4.0647292889928e-01 6.5154995832046e-01
2.8445106262477e-01 4.0916919821460e-01 6.4019258107118e-01
9.4208260800366e-01 4.1186514657563e-01 6.2883522903744e-01
5.9963110500961e-01 4.1456079129615e-01 6.1747788123575e-01
2.5709444703314e-01 4.1725614880602e-01 6.0612051994656e-01
9.1447049583350e-01 4.1995123478541e-01 5.9476313084484e-01
5.7175713026161e-01 4.2264606443580e-01 5.8340570304285e-01
2.2895230731038e-01 4.2534065290110e-01 5.7204822902983e-01
8.8605413591564e-01 4.2803501584829e-01 5.6069070449104e-01
5.4306096320579e-01 4.3072917021111e-01 5.4933312798627e-01
1.9997147196656e-01 4.3342313509180e-01 5.3797550046716e-01
8.5678478696123e-01 4.3611693280508e-01 5.2661782461325e-01
5.1350058647404e-01 4.3881059003430e-01 5.1526010397120e-01
3.7119748711343e-01 2.4455255849620e-01 3.5631189631609e-01
3.4933554915895e-02 2.4726336191010e-01 3.4493965381691e-01
6.9835381021355e-01 2.4997419608639e-01 3.3356747113117e-01
3.6149740452159e-01 2.5268507651251e-01 3.2219535139401e-01
2.4396509605854e-02 2.5539601365289e-01 3.1082329196372e-01
6.8707769728401e-01 2.5810701437393e-01 2.9945128682141e-01
3.4956301164089e-01 2.6081808288507e-01 2.8807932822399e-01
1.1870809392389e-02 2.6352922137150e-01 2.7670740783465e-01
6.7401642414856e-01 2.6624043042645e-01 2.6533551749189e-01
3.3601269589676e-01 2.6895170934979e-01 2.5396364973044e-01
9.9787039655743e-01 2.7166305635524e-01 2.4259179813370e-01
6.5959857482249e-01 2.7437446871270e-01 2.3121995757275e-01
3.2120483786634e-01 2.7708594284304e-01 2.1984812437032e-01
9.8269558335596e-01 2.7979747437668e-01 2.0847629641580e-01
6.4407619207721e-01 2.8250905818323e-01 1.9710447324917e-01
3.0535118915352e-01 2.8522068837769e-01 1.8573265612581e-01
9.6652438005575e-01 2.8793235830670e-01 1.7436084806959e-01
6.2759896624176e-01 2.9064406051782e-01 1.6298905391892e-01
2.8857764421225e-01 2.9335578671446e-01 1.5161728036783e-01
9.4946269095006e-01 2.9606752769874e-01 1.4024553600241e-01
6.1025603806464e-01 2.9877927330485e-01 1.2887383133178e-01
2.7095933645101e-01 3.0149101232576e-01 1.1750217881120e-01
9.3157401286232e-01 3.0420273243612e-01 1.0613059285446e-01
5.9210131946460e-01 3.0691442011521e-01 9.4759089831724e-02
2.5254237717622e-01 3.0962606057363e-01 8.3387688048372e-02
9.1289821338061e-01 3.1233763768811e-01 7.2016407700428e-02
5.7316979443173e-01 3.1504913394919e-01 6.0645270801548e-02
2.3335805324264e-01 3.1776053042649e-01 4.9274301077048e-02
8.9346391215369e-01 3.2047180675666e-01 3.7903523820673e-02
5.5348830121727e-01 3.2318294115858e-01 2.6532965710578e-02
2.1343217200774e-01 3.2589391048045e-01 1.5162654582001e-02
8.7329650706639e-01 3.2860469028228e-01 3.7926191554401e-03
5.3164840890910e-01 3.3131921881715e-01 9.9241821862678e-01
1.9136307949437e-01 3.3402949696588e-01 9.8104887235493e-01
8.5100116765320e-01 3.3673950426681e-01 9.6967989140433e-01
5.1056379284147e-01 3.3944921274107e-01 9.5831130447912e-01
1.7005209417129e-01 3.4215859412391e-01 9.4694313922251e-01
8.2946721883964e-01 3.4486762017439e-01 9.3557542175320e-01
4.8881030862633e-01 3.4757626301128e-01 9.2420817619978e-01
1.4808248509318e-01 3.5028449546782e-01 9.1284142424526e-01
8.0728483414399e-01 3.5299229145676e-01 9.0147518469475e-01
4.6641839060245e-01 3.5569962633638e-01 8.9010947307885e-01
1.2548412342959e-01 3.5840647726782e-01 8.7874430130508e-01
7.8448292213559e-01 3.6111282355380e-01 8.6737967736814e-01
4.4341558484316e-01 3.6381864694938e-01 8.5601560512842e-01
1.0228280833880e-01 3.6652393193592e-01 8.4465208416611e-01
7.6108518031004e-01 3.6922866595050e-01 8.3328910971587e-01
4.1982317382043e-01 3.7193283956428e-01 8.2192667268463e-01
7.8497143929777e-02 3.7463644660471e-01 8.1056475975276e-01
7.3710732623394e-01 3.7733948421790e-01 7.9920335355629e-01
3.9565383698666e-01 3.8004195286934e-01 7.8784243294587e-01
5.4136674382414e-02 3.8274385628242e-01 7.7648197331603e-01
7.1255572053249e-01 3.8544520131611e-01 7.6512194699685e-01
3.7091074365922e-01 3.8814599778436e-01 7.5376232369875e-01
2.9201400070424e-02 3.9084625822184e-01 7.4240307100019e-01
6.8742723555618e-01 3.9354599760159e-01 7.3104415486731e-01
3.4558768597270e-01 3.9624523301200e-01 7.1968554019455e-01
3.6820769384408e-03 3.9894398330203e-01 7.0832719135473e-01
6.6170962275967e-01 4.0164226870468e-01 6.9696907274795e-01
3.1966942491790e-01 4.0434011045056e-01 6.8561114933850e-01
9.7756047068010e-01 4.0703753038427e-01 6.7425338716990e-01
6.3538163262169e-01 4.0973455059799e-01 6.6289575384882e-01
2.9313167006960e-01 4.1243119309714e-01 6.5153821898914e-01
9.5080923366282e-01 4.1512747951446e-01 6.4018075460802e-01
6.0841287437558e-01 4.1782343088876e-01 6.2882333546624e-01
2.6594105843735e-01 4.2051906752530e-01 6.1746593934493e-01
9.2339218959587e-01 4.2321440895424e-01 6.0610854725061e-01
5.8076464008643e-01 4.2590947400263e-01 5.9475114353941e-01
2.3805679147406e-01 4.2860428099396e-01 5.8339371595031e-01
8.9526708620446e-01 4.3129884808640e-01 5.7203625553569e-01
5.5239409022019e-01 4.3399319375705e-01 5.6067875647564e-01
2.0943656635811e-01 4.3668733743356e-01 5.4932121576169e-01
8.6639355744196e-01 4.3938130026765e-01 5.3796363273492e-01
5.2326447703858e-01 4.4207510603479e-01 5.2660600846462e-01
1.8004920479718e-01 4.4476878213333e-01 5.1524834495761e-01
5.1199508604596e-02 2.4237188823337e-01 3.9042530980366e-01
7.1602006010447e-01 2.4508243587055e-01 3.7905270285523e-01
3.8033688110084e-01 2.4779293729340e-01 3.6768022268338e-01
4.4229662999584e-02 2.5050343391492e-01 3.5630788852202e-01
7.0776151813620e-01 2.5321395373083e-01 3.4493570174898e-01
3.7098287892893e-01 2.5592451603665e-01 3.3356365327783e-01
3.3934369773976e-02 2.5863513427208e-01 3.2219172834339e-01
6.9664894660875e-01 2.6134581778498e-01 3.1081990955101e-01
3.5915350893502e-01 2.6405657295727e-01 2.9944817878906e-01
2.1470127722151e-02 2.6676740394459e-01 2.8807651840611e-01
6.8361699063628e-01 2.6947831317532e-01 2.7670491191736e-01
3.4560913710715e-01 2.7218930169497e-01 2.6533334441344e-01
7.4590356095281e-03 2.7490036940759e-01 2.5396180278385e-01
6.6917704138355e-01 2.7761151524589e-01 2.4259027582827e-01
3.3077176276697e-01 2.8032273728980e-01 2.3121875430248e-01
9.9225035710186e-01 2.8303403284586e-01 2.1984723092941e-01
6.5361877195299e-01 2.8574539849554e-01 2.0847570039422e-01
3.1488194355104e-01 2.8845683011781e-01 1.9710415933552e-01
9.7604396154798e-01 2.9116832288943e-01 1.8573260633964e-01
6.3710820706326e-01 2.9387987126574e-01 1.7436104194171e-01
2.9807746940986e-01 2.9659146894384e-01 1.6298946863515e-01
9.5895404567909e-01 2.9930310881015e-01 1.5161789088929e-01
6.1973982643180e-01 3.0201478287412e-01 1.4024631517364e-01
2.8043637001864e-01 3.0472648219009e-01 1.2887474998655e-01
9.4104496748189e-01 3.0743819676988e-01 1.1750320588461e-01
6.0156669953723e-01 3.1014991548874e-01 1.0613169550898e-01
2.6200248676980e-01 3.1286162598810e-01 9.4760233603712e-02
9.2235313388381e-01 3.1557331457899e-01 8.3388837020911e-02
5.8261936860685e-01 3.1828496615058e-01 7.2017524706836e-02
2.4280187565839e-01 3.2099656408863e-01 6.0646317663118e-02
9.0290132604176e-01 3.2370809020949e-01 4.9275238877019e-02
5.6291840180573e-01 3.2641952471487e-01 3.7904313214969e-02
2.2285381634393e-01 3.2913084617340e-01 2.6533567274178e-02
8.8270833025705e-01 3.3184203153421e-01 1.5163029187989e-02
5.4248276279161e-01 3.3455305617777e-01 3.7927283818984e-03
2.0073096658072e-01 3.3726789462789e-01 9.9241798233302e-01
8.6035404508955e-01 3.3997847491186e-01 9.8104829600737e-01
5.1989968876951e-01 3.4268881066506e-01 9.6967894130652e-01
1.7936897967871e-01 3.4539887237320e-01 9.5830994938647e-01
8.3876305278473e-01 3.4810862987202e-01 9.4694135075643e-01
4.9808308405799e-01 3.5081805266436e-01 9.3557317478237e-01
1.5733027606087e-01 3.5352711027359e-01 9.2420544917666e-01
8.1650584168722e-01 3.5623577262581e-01 9.1283819948685e-01
4.7561098676883e-01 3.5894401045237e-01 9.0147144859820e-01
1.3464689229494e-01 3.6165179570230e-01 8.9010521626454e-01
7.9361469698253e-01 3.6435910195386e-01 8.7873951868221e-01
4.5251548088777e-01 3.6706590481389e-01 8.6737436812072e-01
1.1135025066256e-01 3.6977218229368e-01 8.5600977262223e-01
7.7011992693920e-01 3.7247791515054e-01 8.4464573578017e-01
4.2882533417878e-01 3.7518308718547e-01 8.3328225660443e-01
8.7467193153780e-02 3.7788768548830e-01 8.2191932947809e-01
7.4604611606356e-01 3.8059170062332e-01 8.1055694420763e-01
4.0456260411638e-01 3.8329512675035e-01 7.9919508616584e-01
6.3017047262443e-02 3.8599796167767e-01 7.8783373652364e-01
7.2140972564307e-01 3.8870020684542e-01 7.7647287256497e-01
3.7974081223735e-01 3.9140186723984e-01 7.6511246807626e-01
3.8010376149256e-02 3.9410295124053e-01 7.5375249380070e-01
6.9621838598809e-01 3.9680347040484e-01 7.4239291794576e-01
3.5436471285491e-01 3.9950343919514e-01 7.3103370673162e-01
1.2449132557354e-02 4.0220287465648e-01 7.1967482496749e-01
6.7047132682952e-01 4.0490179605388e-01 7.0831623664272e-01
3.2843088352608e-01 4.0760022447989e-01 6.9695790551941e-01
9.8632729598144e-01 4.1029818244485e-01 6.8559979571391e-01
6.4415996196253e-01 4.1299569346335e-01 6.7424187225516e-01
3.0192818288514e-01 4.1569278165174e-01 6.6288410160847e-01
9.5963116419261e-01 4.1838947135267e-01 6.5152645215455e-01
6.1726801799593e-01 4.2108578680300e-01 6.4016889461415e-01
2.7483776922925e-01 4.2378175186194e-01 6.2881140240974e-01
9.3233936666844e-01 4.2647738981619e-01 6.1745395195613e-01
5.8977170017438e-01 4.2917272327812e-01 6.0609652287225e-01
2.4713362544409e-01 4.3186777419167e-01 5.9473909810625e-01
9.0442399736468e-01 4.3456256395884e-01 5.8338166396578e-01
5.6164171275631e-01 4.3725711369643e-01 5.7202421004443e-01
2.1878576285123e-01 4.3995144462875e-01 5.6066672903502e-01
8.7585529528343e-01 4.4264557861687e-01 5.4930921641931e-01
5.3284968466245e-01 4.4533953881771e-01 5.3795167002450e-01
1.8976860999635e-01 4.4803335045872e-01 5.2659408943770e-01
8.4661213634981e-01 4.5072704170334e-01 5.1523647527284e-01
3.9470606213817e-01 2.4290358185622e-01 4.1316496982407e-01
6.0593983109510e-02 2.4561401532789e-01 4.0179164530141e-01
7.2572806333019e-01 2.4832429376959e-01 3.9041858397365e-01
3.9025388091530e-01 2.5103449684785e-01 3.7904582206199e-01
5.4281520762794e-02 2.5374467574508e-01 3.6767335735101e-01
7.1789540986894e-01 2.5645486482859e-01 3.5630116750855e-01
3.8116112025601e-01 2.5916508800038e-01 3.4492922082258e-01
4.4130152839575e-02 2.6187536230120e-01 3.3355748233053e-01
7.0684335573097e-01 2.6458570004862e-01 3.2218591721001e-01
3.6933340416905e-01 2.6729611016360e-01 3.1081449256601e-01
3.1626625620133e-02 2.7000659902975e-01 2.9944317829310e-01
6.9374436116896e-01 2.7271717107252e-01 2.8807194741450e-01
3.5570399398900e-01 2.7542782916293e-01 2.7670077613499e-01
1.7519735696192e-02 2.7813857490646e-01 2.6532964374663e-01
6.7920323450229e-01 2.8084940885316e-01 2.5395853246761e-01
3.4076405074509e-01 2.8356033065084e-01 2.4258742726073e-01
2.2100326960337e-03 2.8627133915522e-01 2.3121631565703e-01
6.6354761666939e-01 2.8898243250583e-01 2.1984518759834e-01
3.2478206916538e-01 2.9169360817333e-01 2.0847403530541e-01
9.8591768426756e-01 2.9440486298211e-01 1.9710285317413e-01
6.4695794624154e-01 2.9711619311062e-01 1.8573163769978e-01
3.0790566488132e-01 2.9982759407160e-01 1.7436038742797e-01
9.6876308936842e-01 3.0253906067330e-01 1.6298910292995e-01
6.2953200507269e-01 3.0525058696340e-01 1.5161778679934e-01
2.9021381670877e-01 3.0796216615684e-01 1.4024644366674e-01
9.5080962048137e-01 3.1067379054930e-01 1.2887508022838e-01
6.1132026724438e-01 3.1338545141830e-01 1.1750370528426e-01
2.7174641821718e-01 3.1609713891443e-01 1.0613232978077e-01
9.3208859441693e-01 3.1880884194584e-01 9.4760966852437e-02
5.9234722065287e-01 3.2152054805962e-01 8.3389631856414e-02
2.5252266467436e-01 3.2423224332448e-01 7.2018342393474e-02
9.1261527185660e-01 3.2694391221986e-01 6.0647118308298e-02
5.7262539564055e-01 3.2965553753700e-01 4.9275981662016e-02
2.3255342381303e-01 3.3236710029831e-01 3.7904956669766e-02
8.9239980061758e-01 3.3507857970127e-01 2.6534069596450e-02
5.5216504462554e-01 3.3778995309365e-01 1.5163348604481e-02
2.1184976227124e-01 3.4050119598629e-01 3.7928235484203e-03
8.6999496933467e-01 3.4321631821182e-01 9.9241777141333e-01
5.2952670349325e-01 3.4592717780877e-01 9.8104777952180e-01
1.8898011453677e-01 3.4863782153068e-01 9.6967808188450e-01
8.4835620694016e-01 3.5134821860378e-01 9.5830871192342e-01
5.0765607475085e-01 3.5405833720861e-01 9.4693970287822e-01
1.6688088976886e-01 3.5676814479560e-01 9.3557108728551e-01
8.2603188658162e-01 3.5947760844808e-01 9.2420289643014e-01
4.8511034509974e-01 3.6218669528569e-01 9.1283515978205e-01
1.4411757134245e-01 3.6489537289959e-01 9.0146790443458e-01
8.0305487729202e-01 3.6760360980877e-01 8.9010115456073e-01
4.6192356066501e-01 3.7031137592562e-01 8.7873493090472e-01
1.2072488543151e-01 3.7301864301777e-01 8.6736925032529e-01
7.7946006384737e-01 3.7572538515322e-01 8.5600412540641e-01
4.3813024065320e-01 3.7843157911571e-01 8.4463956414867e-01
9.6736479943018e-02 3.8113720477829e-01 8.3327556975223e-01
7.5527975502459e-01 3.8384224542422e-01 8.2191214049922e-01
4.1376094139630e-01 3.8654668800600e-01 8.1054926973992e-01
7.2180812763925e-02 3.8925052333513e-01 7.9918694598390e-01
7.3054003983189e-01 3.9195374619748e-01 7.8782515309369e-01
3.8883919143884e-01 3.9465635539125e-01 7.7646387057569e-01
4.7078737481112e-02 3.9735835368658e-01 7.6510307396011e-01
7.0525905298781e-01 4.0005974770843e-01 7.5374273525919e-01
3.6338042268671e-01 4.0276054774598e-01 7.4238282349148e-01
2.1443045432608e-02 4.0546076749439e-01 7.3102330525799e-01
6.7944703796071e-01 4.0816042373629e-01 7.1966414535546e-01
3.3739243757288e-01 4.1085953597268e-01 7.0830530741131e-01
9.9527920355685e-01 4.1355812601434e-01 6.9694675452469e-01
6.5310721736756e-01 4.1625621754692e-01 6.8558844989868e-01
3.1087628185245e-01 4.1895383568398e-01 6.7423035744899e-01
9.6858612006192e-01 4.2165100652396e-01 6.6287244237572e-01
6.2623637443598e-01 4.2434775672756e-01 6.5151467168573e-01
2.8382660737893e-01 4.2704411313316e-01 6.4015701465472e-01
9.4135630440718e-01 4.2974010242755e-01 6.2879944321895e-01
5.9882488116443e-01 4.3243575088921e-01 6.1744193228827e-01
2.5623169562547e-01 4.3513108422043e-01 6.0608445997264e-01
9.1357606674225e-01 4.3782612748264e-01 5.9472700771530e-01
5.7085730061211e-01 4.4052090514735e-01 5.8336956032619e-01
2.2807472495953e-01 4.4321544127156e-01 5.7201210590920e-01
8.8522773231765e-01 4.4590975980257e-01 5.6065463567701e-01
5.4231583177535e-01 4.4860388501182e-01 5.4929714364739e-01
1.9933870853111e-01 4.5129784205121e-01 5.3793962621531e-01
8.5629628978858e-01 4.5399165761798e-01 5.2658208159655e-01
5.1318881477554e-01 4.5668536070571e-01 5.1522450914146e-01
7.3711094334181e-01 2.4343666996565e-01 4.3590367960935e-01
4.0476690639317e-01 2.4614729540202e-01 4.2452887593658e-01
7.1157139198615e-02 2.4885752766004e-01 4.1315458497613e-01
7.3658926873029e-01 2.5156754166767e-01 4.0178089109784e-01
4.0127952097903e-01 2.5427743902659e-01 3.9040777990615e-01
6.5383487793955e-02 2.5698728369057e-01 3.7903519425023e-01
7.2901547868042e-01 2.5969711858872e-01 3.6766306236031e-01
3.9226101753107e-01 2.6240697392714e-01 3.5629131131966e-01
5.5185115957879e-02 2.6511687164597e-01 3.4491987305496e-01
7.1783787798738e-01 2.6782682798901e-01 3.3354868659935e-01
3.8025837783656e-01 2.7053685508657e-01 3.2217769855729e-01
4.2477395194559e-02 2.7324696198886e-01 3.1080686275206e-01
7.0451937911757e-01 2.7595715537428e-01 2.9943613954828e-01
3.6640388128162e-01 2.7866744005373e-01 2.8806549509062e-01
2.8146617898038e-02 2.8137781934015e-01 2.7669490057174e-01
6.8976026766046e-01 2.8408829532448e-01 2.6532433157704e-01
3.5125507933073e-01 2.8679886908345e-01 2.5395376752196e-01
1.2639340229076e-02 2.8950954083588e-01 2.4258319118180e-01
6.7391974186092e-01 2.9222031005796e-01 2.3121258830735e-01
3.3510166864009e-01 2.9493117556503e-01 2.1984194731756e-01
9.9618942854581e-01 2.9764213556440e-01 2.0847125905962e-01
6.5718643954433e-01 3.0035318768278e-01 1.9710051662809e-01
3.1809538204088e-01 3.0306432897045e-01 1.8572971523499e-01
9.7891832505666e-01 3.0577555588376e-01 1.7435885212426e-01
6.3965683194744e-01 3.0848686424724e-01 1.6298792652421e-01
3.0031205009202e-01 3.1119824919635e-01 1.5161693963272e-01
9.6088478793700e-01 3.1390970510196e-01 1.4024589462970e-01
6.2137558199307e-01 3.1662122547778e-01 1.2887479671188e-01
2.8178475576587e-01 3.1933280287247e-01 1.1750365314452e-01
9.4211247212399e-01 3.2204442874838e-01 1.0613247332477e-01
6.0235878022252e-01 3.2475609334980e-01 9.4761268850373e-02
2.6252365778782e-01 3.2746778556396e-01 8.3390053587651e-02
9.2260704931230e-01 3.3017949277904e-01 7.2018843731299e-02
5.8260890049363e-01 3.3289120074419e-01 6.0647657848674e-02
2.4252918907678e-01 3.3560289343726e-01 4.9276516900373e-02
9.0236795211309e-01 3.3831455294691e-01 3.7905444228808e-02
5.6212530954008e-01 3.4102615937598e-01 2.6534465506443e-02
2.2180148390728e-01 3.4373769077391e-01 1.5163608635707e-02
8.8139681602988e-01 3.4644912310554e-01 3.7929035934132e-03
5.3943989411649e-01 3.4916450057330e-01 9.9241758805668e-01
1.9888071878486e-01 3.5187561412083e-01 9.8104732840760e-01
8.5824231233506e-01 3.5458654267974e-01 9.6967732197939e-01
5.1752555666754e-01 3.5729725453746e-01 9.5830760421226e-01
1.7673146310071e-01 3.6000771651332e-01 9.4693821087648e-01
8.3586116118130e-01 3.6271789426216e-01 9.3556917753878e-01
4.9491588366034e-01 3.6542775263683e-01 9.2420053898458e-01
1.5389694823101e-01 3.6813725610415e-01 9.1283232860955e-01
8.1280573677396e-01 3.7084636920585e-01 9.0146457779717e-01
4.7164367297539e-01 3.7355505705417e-01 8.9009731530050e-01
1.3041219926074e-01 3.7626328584906e-01 8.7873056664792e-01
7.8911275401236e-01 3.7897102340306e-01 8.6736435359238e-01
4.4774675000761e-01 3.8167823965870e-01 8.5599869362348e-01
1.0631555492322e-01 3.8438490718319e-01 8.4463359955948e-01
7.6482047460774e-01 3.8709100162566e-01 8.3326907923404e-01
4.2326273963397e-01 3.8979650212332e-01 8.2190513528919e-01
8.1643495423085e-02 3.9250139164453e-01 8.1054176508223e-01
7.3996379599468e-01 3.9520565725888e-01 7.9917896071025e-01
3.9822460116301e-01 3.9790929032673e-01 7.8781670915210e-01
5.6426776783961e-02 4.0061228660301e-01 7.7645499252345e-01
7.1457109747817e-01 4.0331464625313e-01 7.6509378843734e-01
3.7265825112520e-01 4.0601637378083e-01 7.5373307045924e-01
3.0688844353745e-02 4.0871747787094e-01 7.4237280864329e-01
6.8866340824892e-01 4.1141797115190e-01 7.3101297013422e-01
3.4658240356190e-01 4.1411786988583e-01 7.1965351981792e-01
4.4462248380848e-03 4.1681719359566e-01 7.0829442100307e-01
6.6225520306922e-01 4.1951596464111e-01 6.9693563611553e-01
3.2000960671245e-01 4.2221420775741e-01 6.8557712738777e-01
9.7770964118949e-01 4.2491194957209e-01 6.7421885752591e-01
6.3535544726401e-01 4.2760921811673e-01 6.6286079033839e-01
2.9294709897435e-01 4.3030604235156e-01 6.5150289131136e-01
9.5048460205096e-01 4.3300245172161e-01 6.4014512811790e-01
6.0796789395128e-01 4.3569847576284e-01 6.2878747104961e-01
2.6539684678074e-01 4.3839414377651e-01 6.1742989336122e-01
9.2277127441857e-01 4.4108948458864e-01 6.0607237152042e-01
5.8009094511921e-01 4.4378452640945e-01 5.9471488535662e-01
2.3735560070360e-01 4.4647929680530e-01 5.8335741810361e-01
8.9456498318710e-01 4.4917382279161e-01 5.7199995633212e-01
5.5171886931179e-01 4.5186813105160e-01 5.6064248976879e-01
2.0881711296943e-01 4.5456224828005e-01 5.4928501099902e-01
8.6585969493172e-01 4.5725620164556e-01 5.3792751505186e-01
5.2284677867095e-01 4.5995001935813e-01 5.2656999886667e-01
1.7977877039063e-01 4.6264373132115e-01 5.1521246064343e-01
7.6952814848654e-02 2.4397049393795e-01 4.5864282251541e-01
7.4814590155698e-01 2.4668214580276e-01 4.4726488971548e-01
4.1675466092666e-01 2.4939274782705e-01 4.3588796621702e-01
8.3613527475539e-02 2.5210279963058e-01 4.2451232684644e-01
7.4923773318822e-01 2.5481254025073e-01 4.1313790150566e-01
4.1395984519581e-01 2.5752210328252e-01 4.0176451553902e-01
7.8002807877448e-02 2.6023157240310e-01 3.9039198144909e-01
7.4152095893683e-01 2.6294100411566e-01 3.7902012996539e-01
4.0462409803988e-01 2.6565043841895e-01 3.6764881769171e-01
6.7392171975993e-02 2.6835990446214e-01 3.5627792629461e-01
7.2988454798015e-01 2.7106942385664e-01 3.4490735909722e-01
3.9214605022157e-01 2.7377901276927e-01 3.3353703729119e-01
5.4210996652266e-02 2.7648868332205e-01 3.2216689652666e-01
7.1610596435781e-01 2.7919844456758e-01 3.1079688407437e-01
3.7785172527455e-01 2.8190830319006e-01 2.9942695654601e-01
3.9464627905748e-02 2.8461826402067e-01 2.8805707809374e-01
7.0095760130913e-01 2.8732833042277e-01 2.7668721899968e-01
3.6234089666032e-01 2.9003850458302e-01 2.6531735457547e-01
2.3622643382329e-02 2.9274878773211e-01 2.5394746430590e-01
6.8480927222410e-01 2.9545918031147e-01 2.4257753118473e-01
3.4590584154970e-01 2.9816968209707e-01 2.3120754120231e-01
6.9162923420849e-03 3.0088029228775e-01 2.1983748295412e-01
6.6784365012566e-01 3.0359100956343e-01 2.0846734734665e-01
3.2869018697893e-01 3.0630183211676e-01 1.9709712738258e-01
9.8945755328795e-01 3.0901275766043e-01 1.8572681801136e-01
6.5014688639264e-01 3.1172378341178e-01 1.7435641603442e-01
3.1075890148014e-01 3.1443490605585e-01 1.6298592005653e-01
9.7129396876774e-01 3.1714612168757e-01 1.5161533047603e-01
6.3175218004581e-01 3.1985742573386e-01 1.4024464950827e-01
2.9213340691961e-01 3.2256881285653e-01 1.2887388123639e-01
9.5243735252792e-01 3.2528027683708e-01 1.1750303168439e-01
6.1266359807706e-01 3.2799181044503e-01 1.0613210890697e-01
2.7281164517736e-01 3.3070340529196e-01 9.4761123090258e-02
9.3288095468119e-01 3.3341505167410e-01 8.3390086657222e-02
5.9287098248070e-01 3.3612673840734e-01 7.2019014370503e-02
2.5278121251842e-01 3.3883845265928e-01 6.0647923425038e-02
9.1261118708904e-01 3.4155017978402e-01 4.9276833521848e-02
5.7236053436228e-01 3.4426190316620e-01 3.7905766913896e-02
2.3202899293455e-01 3.4697360408204e-01 2.6534748414503e-02
8.9161643312403e-01 3.4968526158528e-01 1.5163805358756e-02
5.5112287466154e-01 3.5239685242702e-01 3.7929675087076e-03
2.0906488423811e-01 3.5511245425928e-01 9.9241743437425e-01
8.6841545053924e-01 3.5782379385089e-01 9.8104694805065e-01
5.2768587722707e-01 3.6053498140924e-01 9.6967667034357e-01
1.8687687388298e-01 3.6324598464899e-01 9.5830663839451e-01
8.4598932212874e-01 3.6595676937497e-01 9.4693689023048e-01
5.0502426573645e-01 3.6866729976119e-01 9.3556746423205e-01
1.6398289619620e-01 3.7137753870075e-01 9.2419839853897e-01
8.2286653423672e-01 3.7408744822303e-01 9.1282973040652e-01
4.8167660799346e-01 3.7679698997092e-01 9.0146149552908e-01
1.4041462869573e-01 3.7950612572810e-01 8.9009372735036e-01
7.9908216488128e-01 3.8221481798335e-01 8.7872645638217e-01
4.5768081622993e-01 3.8492303051662e-01 8.6735970955457e-01
1.1621218812491e-01 3.8763072899023e-01 8.5599350962027e-01
7.7467786799687e-01 3.9033788152726e-01 8.4462787463490e-01
4.3307940438178e-01 3.9304445925982e-01 8.3326281753262e-01
9.1418289436130e-02 3.9575043683011e-01 8.2189834581286e-01
7.4969594541596e-01 3.9845579282918e-01 8.1053446135034e-01
4.0791371535532e-01 4.0116051016008e-01 7.9917116033553e-01
6.6072857897569e-02 4.0386457631498e-01 7.8780843334822e-01
7.2417454596001e-01 4.0656798355844e-01 7.7644626556194e-01
3.8221986866999e-01 4.0927072901228e-01 7.6508463707260e-01
4.0209835817993e-02 4.1197281464035e-01 7.5372352334060e-01
6.9814538394486e-01 4.1467424713479e-01 7.4236289573233e-01
3.5602738312766e-01 4.1737503770793e-01 7.3100272214414e-01
1.3856643555808e-02 4.2007520179723e-01 7.1964296768971e-01
6.7163392109687e-01 4.2277475869282e-01 7.0828359543062e-01
3.2935992123329e-01 4.2547373110007e-01 6.9692456712890e-01
9.8703530099525e-01 4.2817214465166e-01 6.8556584400070e-01
6.4466066880584e-01 4.3087002738570e-01 6.7420738745047e-01
3.0223658247018e-01 4.3356740920801e-01 6.6284915976647e-01
9.5976354585820e-01 4.3626432135824e-01 6.5149112476009e-01
6.1724200512509e-01 4.3896079589959e-01 6.4013324833320e-01
2.7467234555994e-01 4.4165686525279e-01 6.2877549896056e-01
9.3205489032775e-01 4.4435256179354e-01 6.1741784807636e-01
5.8938990245489e-01 4.4704791753192e-01 6.0606027035673e-01
2.4667759138753e-01 4.4974296388953e-01 5.9470274389211e-01
9.0391812531942e-01 4.5243773158750e-01 5.8334525024574e-01
5.6111165023707e-01 4.5513225065455e-01 5.7198777439594e-01
2.1825831627280e-01 4.5782655055970e-01 5.6063030456171e-01
8.7535831150070e-01 4.6052066046921e-01 5.4927283191195e-01
5.3241190277684e-01 4.6321460962137e-01 5.3791535016040e-01
1.8941948263913e-01 4.6590842780641e-01 5.2655785504927e-01
8.4638162067864e-01 4.6860214593224e-01 5.1520034372693e-01
4.0749503558623e-01 2.4450141403899e-01 4.8138670724062e-01
8.9254888285790e-02 2.4721793060742e-01 4.7000151712163e-01
7.6295309047693e-01 2.4993018088399e-01 4.5861796965577e-01
4.3241535091523e-01 2.5264082258614e-01 4.4723791211280e-01
9.9409737863094e-02 2.5535065325903e-01 4.3586089038210e-01
7.6484914526435e-01 2.5806002552141e-01 4.2448616376648e-01
4.2924747102746e-01 2.6076913433507e-01 4.1311311452294e-01
9.2914226841513e-02 2.6347810045930e-01 4.0174128433295e-01
7.5604609556847e-01 2.6618700346856e-01 3.9037034160652e-01
4.1877357561502e-01 2.6889589783794e-01 3.7900004585965e-01
8.1186348361627e-02 2.7160482182981e-01 3.6763022057751e-01
7.4334784409316e-01 2.7431380279881e-01 3.5626073422136e-01
4.0530399715387e-01 2.7702286052833e-01 3.4489148721643e-01
6.7088717693116e-02 2.7973200941195e-01 3.3352240303163e-01
7.2872742832528e-01 2.8244125992482e-01 3.2215342200699e-01
3.9023941804228e-01 2.8515061964241e-01 3.1078449702823e-01
5.1639449880659e-02 2.8786009396205e-01 2.9941559045440e-01
7.1293888442191e-01 2.9056968662398e-01 2.8804667190703e-01
3.7414648131658e-01 2.9327940009361e-01 2.7667771666010e-01
3.5268981949857e-02 2.9598923584527e-01 2.6530870445602e-01
6.9631154041357e-01 2.9869919457411e-01 2.5393961862833e-01
3.5727804749915e-01 3.0140927635381e-01 2.4257044544921e-01
1.8171378076542e-02 3.0411948075230e-01 2.3120117364469e-01
6.7899358285861e-01 3.0682980691322e-01 2.1983179403733e-01
3.3974603931875e-01 3.0954025360861e-01 2.0846229928789e-01
4.2957230947626e-04 3.1225081926621e-01 1.9709268371541e-01
6.6104455202413e-01 3.1496150197354e-01 1.8572294318080e-01
3.2159097491261e-01 3.1767229945993e-01 1.7435307502323e-01
9.8206853171920e-01 3.2038320905728e-01 1.6298307804087e-01
6.4247666577014e-01 3.2309422763981e-01 1.5161295250972e-01
3.0281462387561e-01 3.2580535154307e-01 1.4024270023536e-01
9.6308150163977e-01 3.2851657646243e-01 1.2887232463305e-01
6.2327628453685e-01 3.3122789733161e-01 1.1750183083167e-01
2.8339788576814e-01 3.3393930818207e-01 1.0613122579725e-01
9.4344518163806e-01 3.3665080198490e-01 9.4760518470962e-02
6.0341704495575e-01 3.3936237047723e-01 8.3389719916037e-02
2.6331237676896e-01 3.4207400397641e-01 7.2018843467209e-02
9.2313013656052e-01 3.4478569118607e-01 6.0647904875153e-02
5.8286937087915e-01 3.4749741899919e-01 4.9276922437515e-02
2.4252924023427e-01 3.5020917230480e-01 3.7905917107090e-02
9.0210904395940e-01 3.5292093380569e-01 2.6534912566871e-02
5.6160824264423e-01 3.5563268385588e-01 1.5163935261170e-02
2.2102647765565e-01 3.5834440032745e-01 3.7930143718327e-03
8.7886869772776e-01 3.6106019341498e-01 9.9241731233952e-01
5.3812990172892e-01 3.6377172865454e-01 9.8104664357576e-01
1.9731007964288e-01 3.6648314666175e-01 9.6967613544178e-01
8.5640971446889e-01 3.6919441498914e-01 9.5830582638810e-01
5.1542950603467e-01 3.7190549882778e-01 9.4693575634181e-01
1.7437036319480e-01 3.7461636124750e-01 9.3556596619349e-01
8.3323339080267e-01 3.7732696351991e-01 9.2419649720254e-01
4.9201987178923e-01 3.8003726552282e-01 9.1282739033213e-01
1.5073124493172e-01 3.8274722622121e-01 9.0145868552945e-01
8.0936907913439e-01 3.8545680421563e-01 8.9009042097113e-01
4.6793504525036e-01 3.8816595834568e-01 8.7872263229729e-01
1.2643088662902e-01 3.9087464833290e-01 8.6735535185983e-01
7.8485838965764e-01 3.9358283544460e-01 8.5598860801183e-01
4.4321935556856e-01 3.9629048315892e-01 8.4462242446429e-01
1.0151557469862e-01 3.9899755781022e-01 8.3325681973511e-01
7.5974880421820e-01 4.0170402919468e-01 8.2189180671141e-01
4.1792075010488e-01 4.0440987111670e-01 8.1052739234270e-01
7.6033053839215e-02 4.0711506185933e-01 7.9916357747674e-01
7.3408728396967e-01 4.0981958456411e-01 7.8780035684478e-01
3.9208493235854e-01 4.1252342750947e-01 7.7643771919694e-01
5.0027414605146e-02 4.1522658428000e-01 7.6507564758289e-01
7.0791607387400e-01 4.1792905382251e-01 7.5371411976808e-01
3.6575218715399e-01 4.2063084038871e-01 7.4235310877094e-01
2.3536972856882e-02 4.2333195336740e-01 7.3099258350312e-01
6.8127159863935e-01 4.2603240701287e-01 7.1963250949146e-01
3.3895718840567e-01 4.2873222007913e-01 7.0827284965887e-01
9.9659482761358e-01 4.3143141537244e-01 6.9691356513955e-01
6.5418556625322e-01 4.3413001923772e-01 6.8555461610418e-01
3.1173041918008e-01 4.3682806099637e-01 6.7419596257073e-01
9.6923036383516e-01 4.3952557235538e-01 6.6283756517803e-01
6.2668633575060e-01 4.4222258680884e-01 6.5147938590097e-01
2.8409922258836e-01 4.4491913905422e-01 6.4012138868881e-01
9.4146985776188e-01 4.4761526444556e-01 6.2876354001065e-01
5.9879901491946e-01 4.5031099850526e-01 6.1740580929574e-01
2.5608740469973e-01 4.5300637651479e-01 6.0604816925918e-01
9.1333567518730e-01 4.5570143320165e-01 5.9469059610691e-01
5.7054441739232e-01 4.5839620253732e-01 5.8333306961709e-01
2.2771417685010e-01 4.6109071765601e-01 5.7197557309711e-01
8.8484547209528e-01 4.6378501089985e-01 5.6061809321823e-01
5.4193882032467e-01 4.6647911399024e-01 5.4926061973127e-01
1.9899477004794e-01 4.6917305831949e-01 5.3790314506858e-01
8.5601393996465e-01 4.7186687535056e-01 5.2654566383905e-01
5.1299706273629e-01 4.7456059710648e-01 5.1518817222455e-01
5.8554801417376e-01 2.4488960948211e-01 5.0411544071670e-01
4.1320118018683e-01 2.4774718140912e-01 4.9275165125056e-01
1.1240102473669e-01 2.5047104675210e-01 4.8133982568403e-01
7.8786329542973e-01 2.5318408561776e-01 4.6994678616931e-01
4.5631620500886e-01 2.5589429299045e-01 4.5856560774892e-01
1.2181851444593e-01 2.5860329258213e-01 4.4719033771982e-01
7.8580765559798e-01 2.6131172039965e-01 4.3581824415376e-01
4.4891651688241e-01 2.6401987837613e-01 4.2444798826739e-01
1.1146593392571e-01 2.6672792700625e-01 4.1307885143645e-01
7.7363538222300e-01 2.6943595865006e-01 4.0171041690489e-01
4.3553278579977e-01 2.7214402963206e-01 3.9034242737736e-01
9.7226694632699e-02 2.7485217578534e-01 3.7897471572854e-01
7.5876252577543e-01 2.7756042059303e-01 3.6760716876309e-01
4.2017136755375e-01 2.8026877973223e-01 3.5623970707739e-01
8.1475035880291e-02 2.8297726374817e-01 3.4487227328486e-01
7.4268912005193e-01 2.8568587970087e-01 3.3350482482908e-01
4.0382489266866e-01 2.8839463221964e-01 3.2213732943371e-01
6.4890549164113e-02 2.9110352420243e-01 3.1076976213038e-01
7.2589203663563e-01 2.9381255729393e-01 2.9940210326493e-01
3.8683362283293e-01 2.9652173222144e-01 2.8803433712955e-01
4.7718296083104e-02 2.9923104903601e-01 2.7666645100727e-01
7.0854805249288e-01 3.0194050728847e-01 2.6529843449511e-01
3.6932410637066e-01 3.0465010615893e-01 2.5393027902089e-01
3.0047047370480e-02 3.0735984455146e-01 2.4256197749804e-01
6.9071696007338e-01 3.1006972116155e-01 2.3119352408146e-01
3.5133351673325e-01 3.1277973452089e-01 2.1982491399984e-01
1.1896050650139e-02 3.1548988302223e-01 2.0845614344741e-01
6.7240361545758e-01 3.1820016492583e-01 1.9708720952385e-01
3.3285503413199e-01 3.2091057834795e-01 1.8571811021433e-01
9.9324894051208e-01 3.2362112123147e-01 1.7434884440428e-01
6.5358381539867e-01 3.2633179129811e-01 1.6297941192514e-01
3.1385801879237e-01 3.2904258598161e-01 1.5160981362812e-01
9.7406981945061e-01 3.3175350234129e-01 1.4024005148392e-01
6.3421742266373e-01 3.3446453695518e-01 1.2887012870611e-01
2.9429899692986e-01 3.3717568579248e-01 1.1750004989633e-01
9.5431270002979e-01 3.3988694406536e-01 1.0612982120836e-01
6.1425670485065e-01 3.4259830606053e-01 9.4759450528205e-02
2.7412922517027e-01 3.4530976495202e-01 8.3388947665732e-02
9.3392854148496e-01 3.4802131259736e-01 7.2018324552939e-02
5.9365302683840e-01 3.5073293932034e-01 6.0647595442223e-02
2.5330117248576e-01 3.5344463368498e-01 4.9276777096935e-02
9.1287161310507e-01 3.5615638226657e-01 3.7905888964998e-02
5.7236315115056e-01 3.5886816942696e-01 2.6534953325132e-02
2.3177477983353e-01 3.6157997710291e-01 1.5163995394078e-02
8.9110570412297e-01 3.6429178461753e-01 3.7930433824819e-03
5.4884965682378e-01 3.6700773380727e-01 9.9241722371935e-01
2.0802266691074e-01 3.6971943193321e-01 9.8104641968583e-01
8.6711381241579e-01 3.7243104918160e-01 9.6967572521371e-01
5.2612329143168e-01 3.7514255339514e-01 9.5830517959101e-01
1.8505156499374e-01 3.7785390960530e-01 9.4693482419810e-01
8.4389935225812e-01 3.8056508021845e-01 9.3556470203372e-01
5.0266761984740e-01 3.8327602529552e-01 9.2419485713986e-01
1.6135756546834e-01 3.8598670292718e-01 9.1282533393337e-01
8.1997059620067e-01 3.8869706970253e-01 9.0145617645827e-01
4.7850830215764e-01 3.9140708126448e-01 8.9008742757768e-01
1.3697242650402e-01 3.9411669294086e-01 8.7871912812963e-01
7.9536483305788e-01 3.9682586043581e-01 8.6735131607456e-01
4.5368747287408e-01 3.9953454056224e-01 8.5598402566492e-01
1.1194235128906e-01 4.0224269199351e-01 8.4461728666825e-01
7.7013149688501e-01 4.0495027601065e-01 8.3325112367425e-01
4.2825693370339e-01 4.0765725722094e-01 8.2188555551347e-01
8.6320657808518e-02 4.1036360422430e-01 8.1052059481144e-01
7.4432461898966e-01 4.1306929020580e-01 7.9915624769640e-01
4.0227070801736e-01 4.1577429343542e-01 7.8779251367238e-01
6.0160749468995e-02 4.1847859765959e-01 7.7642938566298e-01
7.1799649974158e-01 4.2118219237299e-01 7.6506685022383e-01
3.7577964951011e-01 4.2388507296343e-01 7.5370488791587e-01
3.3511829596968e-02 4.2658724072694e-01 7.4234347382524e-01
6.9119461901552e-01 4.2928870275426e-01 7.3098257821081e-01
3.4882955385639e-01 4.3198947169419e-01 7.1962216725646e-01
6.4181357063361e-03 4.3468956540297e-01 7.0826220390199e-01
6.6396183842608e-01 4.3738900649246e-01 6.9690264872491e-01
3.2146211235580e-01 4.4008782179333e-01 6.8554346084437e-01
9.7892038534702e-01 4.4278604175212e-01 6.7418459881876e-01
6.3633806041072e-01 4.4548369978380e-01 6.6282602150956e-01
2.9371651019389e-01 4.4818083160309e-01 6.5146768888628e-01
9.5105706891231e-01 4.5087747455940e-01 6.4010956274986e-01
6.0836102274238e-01 4.5357366700011e-01 6.2875160735562e-01
2.6562959997349e-01 4.5626944768681e-01 6.1739378992075e-01
9.2286396241655e-01 4.5896485528709e-01 6.0603608100537e-01
5.8006519963327e-01 4.6165992796206e-01 5.9467845476052e-01
2.3723432748278e-01 4.6435470306591e-01 5.8332088904041e-01
8.9437229227879e-01 4.6704921696948e-01 5.7196336537981e-01
5.5147998152017e-01 4.6974350501409e-01 5.6060586884067e-01
2.0855824172321e-01 4.7243760159662e-01 5.4924838773429e-01
8.6560790337120e-01 4.7513154038025e-01 5.3789091322779e-01
5.2262981244125e-01 4.7782535461923e-01 5.2653343884487e-01
1.7962486740781e-01 4.8051907757992e-01 5.1517595987265e-01
9.1887162943767e-01 2.5099384936968e-01 5.0411541104195e-01
2.4653675510875e-01 2.5376839330347e-01 4.9246207541865e-01
8.5479853315914e-01 2.5646532291596e-01 4.8116268308243e-01
5.0370625515257e-01 2.5916430495689e-01 4.6982689035152e-01
1.5940690384252e-01 2.6186722405883e-01 4.5847581459381e-01
8.1741728751610e-01 2.6457172734952e-01 4.4711894871495e-01
4.7647895358294e-01 2.6727707187032e-01 4.3575926277724e-01
1.3610239052716e-01 2.6998295929121e-01 4.2439795289061e-01
7.9605758778815e-01 2.7268924731943e-01 4.1303558917641e-01
4.5622201136729e-01 2.7539585921484e-01 4.0167247683133e-01
1.1652417327072e-01 2.7810274953095e-01 3.9030879358611e-01
7.7691933930031e-01 2.8080988923613e-01 3.7894464973232e-01
4.3737787787352e-01 2.8351725855296e-01 3.6758011717176e-01
9.7879189854253e-02 2.8622484322850e-01 3.5621524464254e-01
7.5840833196281e-01 2.8893263246713e-01 3.4485006622561e-01
4.1895403589154e-01 2.9164061772700e-01 3.3348460635244e-01
7.9507493884705e-02 2.9434879199140e-01 3.2211888288491e-01
7.4006158615336e-01 2.9705714931392e-01 3.1075290908187e-01
4.0061037370673e-01 2.9976568452769e-01 2.9938669489677e-01
6.1148756394966e-02 3.0247439305609e-01 2.8802024785985e-01
7.2167223704238e-01 3.0518327078759e-01 2.7665357369498e-01
3.8217675562102e-01 3.0789231399148e-01 2.6528667676351e-01
4.2658570840767e-02 3.1060151925980e-01 2.5391956039341e-01
7.0311417459587e-01 3.1331088346503e-01 2.4255222713196e-01
3.6354022968997e-01 3.1602040372683e-01 2.3118467894753e-01
2.3933524418288e-02 3.1873007738210e-01 2.1981691739847e-01
6.8429093963229e-01 3.2143990195456e-01 2.0844894378163e-01
3.4460942526728e-01 3.2414987512016e-01 1.9708075927002e-01
4.8859842270608e-03 3.2685999466545e-01 1.8571236504679e-01
6.6511766214696e-01 3.2957025843614e-01 1.7434376244101e-01
3.2530154199156e-01 3.3228066427334e-01 1.6297495306970e-01
9.8543474274891e-01 3.3499120993536e-01 1.5160593898963e-01
6.4551442169672e-01 3.3770189300273e-01 1.4023672286151e-01
3.0553777986372e-01 3.4041271076477e-01 1.2886730812858e-01
9.6550207040773e-01 3.4312366008608e-01 1.1749769921068e-01
6.2540460969065e-01 3.4583473725186e-01 1.0612790171396e-01
2.8524279085748e-01 3.4854593779155e-01 9.4757922655611e-02
9.4501409972487e-01 3.5125725628088e-01 8.3387770701339e-02
6.0471613275861e-01 3.5396868612349e-01 7.2017456412515e-02
2.6434661687056e-01 3.5668021931426e-01 6.0646992497898e-02
9.2390343069693e-01 3.5939184618786e-01 4.9276394063477e-02
5.8338462693462e-01 3.6210355515743e-01 3.7905678852069e-02
2.4278845521475e-01 3.6481533245012e-01 2.6534867462558e-02
9.0211338488914e-01 3.6752716184755e-01 1.5163983537006e-02
5.6135812700377e-01 3.7023902444148e-01 3.7930539023845e-03
2.1900560011277e-01 3.7295509280766e-01 9.9241716999745e-01
8.7809188209118e-01 3.7566691889782e-01 9.8104628047974e-01
5.3709553842642e-01 3.7837870165415e-01 9.6967544680008e-01
1.9601642063243e-01 3.8109040969798e-01 9.5830470853200e-01
8.5485468952873e-01 3.8380200840567e-01 9.4693410796760e-01
5.1361081435850e-01 3.8651346002460e-01 9.3556368970542e-01
1.7228556561436e-01 3.8922472389155e-01 9.2419350011797e-01
8.3088000137749e-01 3.9193575675952e-01 9.1282358670262e-01
4.8939544730345e-01 3.9464651323489e-01 9.0145399732637e-01
1.4783347074676e-01 3.9735694632169e-01 8.9008477938121e-01
8.0619584988065e-01 4.0006700806433e-01 8.7871597887358e-01
4.6448453900590e-01 4.0277665027486e-01 8.6734763947707e-01
1.2270163151962e-01 4.0548582532563e-01 8.5597980158167e-01
7.8084932220126e-01 4.0819448698419e-01 8.4461250137575e-01
4.3892987054407e-01 4.1090259126391e-01 8.3324576999736e-01
9.6945566802595e-02 4.1361009726249e-01 8.2187963278983e-01
7.5489870223831e-01 4.1631696795974e-01 8.1051410869314e-01
4.1279154473835e-01 4.1902317094789e-01 7.9914920979670e-01
7.0626320580244e-02 4.2172867906969e-01 7.8778494107254e-01
7.2840520265211e-01 4.2443347094358e-01 7.7642130030004e-01
3.8613030495300e-01 4.2713753135935e-01 7.6505827818491e-01
4.3803682732977e-02 4.2984085153289e-01 7.5369585866742e-01
7.0142733722797e-01 4.3254342921338e-01 7.4233401940716e-01
3.5900322363374e-01 4.3524526864182e-01 7.3097273242518e-01
1.6533260773508e-02 4.3794638036417e-01 7.1961196487879e-01
6.7401934086106e-01 4.4064678090771e-01 7.0825167994021e-01
3.3146333784951e-01 4.4334649233310e-01 6.9689183774736e-01
9.8886711308124e-01 4.4604554167885e-01 6.8553239639347e-01
6.4623251729586e-01 4.4874396031867e-01 6.7417331292203e-01
3.0356138848603e-01 4.5144178325503e-01 6.6281454429437e-01
9.6085554557604e-01 4.5413904837482e-01 6.5145604829966e-01
6.1811677839580e-01 4.5683579569481e-01 6.4009778437982e-01
2.7534683488640e-01 4.5953206662506e-01 6.2873971434647e-01
9.3254740686261e-01 4.6222790327815e-01 6.1738180297125e-01
5.8972011593305e-01 4.6492334785049e-01 6.0602401843645e-01
2.4686650131486e-01 4.6761844209881e-01 5.9466633263806e-01
9.0398801126044e-01 4.7031322693133e-01 5.8330872133846e-01
5.6108599963848e-01 4.7300774212758e-01 5.7195116417076e-01
2.1816172889178e-01 4.7570202619522e-01 5.6059364450091e-01
8.7521638015573e-01 4.7839611636575e-01 5.4923614915699e-01
5.3225107079599e-01 4.8109004872456e-01 5.3787866803750e-01
1.8926687905343e-01 4.8378385846383e-01 5.2652119361258e-01
8.4626487491005e-01 4.8647758024112e-01 5.1516372033323e-01
5.3280364677031e-01 2.5690338159725e-01 5.0412730366380e-01
7.9872196933457e-02 2.5950005891296e-01 4.9275159249845e-01
6.2692896144902e-01 2.6237084814756e-01 4.8116265400633e-01
2.3519086713471e-01 2.6513206553443e-01 4.6973468624191e-01
8.7161640033736e-01 2.6784001252651e-01 4.5838594183375e-01
5.1890247421697e-01 2.7054210773769e-01 4.4704230112532e-01
1.7124079574699e-01 2.7324454967400e-01 4.3569450971021e-01
8.2628818119153e-01 2.7594796614200e-01 4.2434265066637e-01
4.8294054825313e-01 2.7865224449210e-01 4.1298772242697e-01
1.4061490427315e-01 2.8135721364570e-01 4.0163054134082e-01
7.9897587249341e-01 2.8406273837155e-01 3.9027167892058e-01
4.5781724217593e-01 2.8676872134212e-01 3.7891152694876e-01
1.1700528089962e-01 2.8947509318874e-01 3.6755035683288e-01
7.7644937091503e-01 2.9218180378680e-01 3.5618836019727e-01
4.3608579242027e-01 2.9488881614095e-01 3.4482567513250e-01
9.5868281353920e-02 2.9759610230551e-01 3.3346240310867e-01
7.5576228267955e-01 3.0030364068052e-01 3.2209862001549e-01
4.1574131877476e-01 3.0301141420523e-01 3.1073438350332e-01
7.5784619129999e-02 3.0571940913497e-01 2.9936973796322e-01
7.3587552938595e-01 3.0842761420102e-01 2.8800471797307e-01
3.9600041707392e-01 3.1113602002501e-01 2.7663935072853e-01
5.6147902705716e-02 3.1384461870470e-01 2.6527365779023e-01
7.1630830923273e-01 3.1655340351656e-01 2.5390765636276e-01
3.7647326132453e-01 3.1926236869822e-01 2.4254136024854e-01
3.6635389513814e-02 3.2197150928571e-01 2.3117478057347e-01
6.9678810911369e-01 3.2468082098753e-01 2.1980792635125e-01
3.5692545338637e-01 3.2739030008279e-01 2.0844080493325e-01
1.7041946739307e-02 3.3009994333366e-01 1.9707342237787e-01
6.7713250794577e-01 3.3280974790466e-01 1.8570578376392e-01
3.3719237626594e-01 3.3551971128276e-01 1.7433789346654e-01
9.9721705533869e-01 3.3822983119335e-01 1.6296975540969e-01
6.5720227111522e-01 3.4094010550772e-01 1.5160137330589e-01
3.1714394110083e-01 3.4365053213834e-01 1.4023275089183e-01
9.7703815288398e-01 3.4636110891878e-01 1.2886389216601e-01
6.3688115044434e-01 3.4907183346523e-01 1.1749480163365e-01
2.9666932709783e-01 3.5178270301758e-01 1.0612548456199e-01
9.5639922419447e-01 3.5449371425799e-01 9.4755947248075e-02
6.1606753485844e-01 3.5720486310610e-01 8.3386197299530e-02
2.7567111216687e-01 3.5991614449062e-01 7.2016243927155e-02
9.3520698121612e-01 3.6262755209808e-01 6.0646098246413e-02
5.9467235453105e-01 3.6533907810115e-01 4.9275773583014e-02
2.5406465024101e-01 3.6805071287005e-01 3.7905285775612e-02
9.1338151238343e-01 3.7076244467267e-01 2.6534653466498e-02
5.7262083260908e-01 3.7347425937081e-01 1.5163898368893e-02
2.3178077246207e-01 3.7618614012203e-01 3.7930454974026e-03
8.8933383896272e-01 3.7890228932774e-01 9.9241715229200e-01
5.4833517784779e-01 3.8161420659423e-01 9.8104622925285e-01
2.0725324032468e-01 3.8432611881846e-01 9.6967530623663e-01
8.6608746730440e-01 3.8703799591688e-01 9.5830442247231e-01
5.2483765430601e-01 3.8974980415935e-01 9.4693362052660e-01
1.8350395549380e-01 3.9246150619919e-01 9.3556294597765e-01
8.4208688118601e-01 3.9517306121137e-01 9.2419244695141e-01
5.0058728827619e-01 3.9788442514988e-01 9.1282217351870e-01
1.5900636334881e-01 4.0059555113112e-01 9.0145217695816e-01
8.1734559867274e-01 4.0330638994469e-01 8.9008250889813e-01
4.7560676169379e-01 4.0601689068697e-01 8.7871322035838e-01
1.3379185908549e-01 4.0872700150634e-01 8.6734436072038e-01
7.9190309681559e-01 4.1143667044238e-01 8.5597597666135e-01
4.4994283800134e-01 4.1414584633553e-01 8.4460811109221e-01
1.0791356052411e-01 4.1685447977881e-01 8.3324080214228e-01
7.6581781642341e-01 4.1956252407995e-01 8.2187408223337e-01
4.2365819497890e-01 4.2226993620063e-01 8.1050797728363e-01
8.1437291119472e-02 4.2497667763964e-01 7.9914250607620e-01
7.3915768038878e-01 4.2768271522910e-01 7.8777767982045e-01
3.9682190118284e-01 4.3038802181601e-01 7.7641350192495e-01
5.4432444401148e-02 4.3309257680638e-01 7.6504996799175e-01
7.1199175007062e-01 4.3579636655478e-01 7.5368706603136e-01
3.6950220996351e-01 4.3849938458788e-01 7.4232477688878e-01
2.6966174783593e-02 4.4120163165702e-01 7.3096307486209e-01
6.8438596417894e-01 4.4390311562057e-01 7.1960192848786e-01
3.4176387768256e-01 4.4660385116289e-01 7.0824130146184e-01
9.9910220469678e-01 4.4930385936186e-01 6.9688115365917e-01
6.5640323182378e-01 4.5200316712214e-01 6.8552144221561e-01
3.1366924618788e-01 4.5470180649577e-01 6.7416212263049e-01
9.7090253386432e-01 4.5739981391552e-01 6.6280314985260e-01
6.2810537308561e-01 4.6009722937001e-01 6.5144447931237e-01
2.8528002248803e-01 4.6279409555130e-01 6.4008606786709e-01
9.4242870523479e-01 4.6549045700743e-01 6.2872787463084e-01
5.9955359035373e-01 4.6818635933195e-01 6.1736986166615e-01
2.5665677300890e-01 4.7088184842085e-01 6.0601199452105e-01
9.1374025564919e-01 4.7357696982453e-01 5.9465424260145e-01
5.7080593202278e-01 4.7627176821768e-01 5.8329657937574e-01
2.2785557590727e-01 4.7896628700460e-01 5.7193898241414e-01
8.8489083609695e-01 4.8166056807050e-01 5.6058143327085e-01
5.4191323873798e-01 4.8435465168258e-01 5.4922391722145e-01
1.9892419754960e-01 4.8704857653714e-01 5.3786642287084e-01
8.5592503186375e-01 4.8974237994185e-01 5.2650894164959e-01
5.1291699180440e-01 4.9243609811593e-01 5.1515146721760e-01
1.8739350650549e-01 2.6284197601421e-01 5.0412578230287e-01
7.8476598777841e-01 2.6546837954957e-01 4.9276322918407e-01
3.6932619914316e-01 2.6818801112692e-01 4.8133973846685e-01
9.6667534148799e-01 2.7100765094724e-01 4.6982683281286e-01
5.8741969562234e-01 2.7377285518383e-01 4.5838591337244e-01
2.2384536084575e-01 2.7649584278362e-01 4.4700708787684e-01
8.6886548953447e-01 2.7920474416450e-01 4.3564960202079e-01
5.1878762544894e-01 2.8190969463856e-01 4.2429717249829e-01
1.7170050302251e-01 2.8461381280418e-01 4.1294472598887e-01
8.2654824940363e-01 2.8731804983303e-01 4.0159088419387e-01
4.8270875580643e-01 2.9002267911729e-01 3.9023542360717e-01
1.3979469966963e-01 2.9272775557124e-01 3.7887846056145e-01
7.9755346863192e-01 2.9543326467864e-01 3.6752018898924e-01
4.5581369249203e-01 2.9813917282858e-01 3.5616079827992e-01
1.1445512194281e-01 3.0084544438322e-01 3.4480045104014e-01
7.7339086346971e-01 3.0355204706619e-01 3.3343928056764e-01
4.3255647724119e-01 3.0625895320075e-01 3.2207739424630e-01
9.1903051420237e-02 3.0896613952832e-01 3.1071487813759e-01
7.5139266742798e-01 3.1167358661496e-01 2.9935180124401e-01
4.1099535061271e-01 3.1438127820985e-01 2.8798821905392e-01
7.0686970881962e-02 3.1708920067626e-01 2.7662417636242e-01
7.3044776677836e-01 3.1979734252245e-01 2.6525970947676e-01
3.9026128831781e-01 3.2250569402740e-01 2.5389484793163e-01
5.0113627070298e-02 3.2521424694605e-01 2.4252961582561e-01
7.0999284707103e-01 3.2792299427719e-01 2.3116403286908e-01
3.6988855863429e-01 3.3063193007909e-01 2.1979811521423e-01
2.9791595526763e-02 3.3334104931998e-01 2.0843187612135e-01
6.8969376806157e-01 3.3605034775296e-01 1.9706532650323e-01
3.4958767279148e-01 3.3875982180640e-01 1.8569847537947e-01
9.4665450157623e-03 3.4146946848270e-01 1.7433133026581e-01
6.6932414414980e-01 3.4417928525902e-01 1.6296389751770e-01
3.2915466470091e-01 3.4688926998466e-01 1.5159618264360e-01
9.8895266751082e-01 3.4959942077027e-01 1.4022819060037e-01
6.4871302730407e-01 3.5230973586464e-01 1.2885992608046e-01
3.0843089358229e-01 3.5502021351518e-01 1.1749139379878e-01
9.6810166263204e-01 3.5773085180879e-01 1.0612259878529e-01
6.2772095894433e-01 3.6044164849021e-01 9.4753546687920e-02
2.8728462472381e-01 3.6315260075563e-01 8.3384244088283e-02
9.4678871643193e-01 3.6586370502003e-01 7.2014698831639e-02
6.0622950748234e-01 3.6857495665789e-01 6.0644920370060e-02
2.6560349630721e-01 3.7128634971788e-01 4.9274920116025e-02
9.2490741905077e-01 3.7399787661387e-01 3.7904711801260e-02
5.8413826613238e-01 3.7670952779628e-01 2.6534311833129e-02
2.4329330186431e-01 3.7942129140997e-01 1.5163739638245e-02
9.0237008621931e-01 3.8213315294707e-01 3.7930179802838e-03
5.5983111999302e-01 3.8484934370006e-01 9.9241717127035e-01
2.1874963742091e-01 3.8756131388102e-01 9.8104626828593e-01
8.7758437398367e-01 3.9027331754250e-01 9.6967530812341e-01
5.3633429490334e-01 3.9298532642940e-01 9.5830432896602e-01
1.9499876273746e-01 3.9569730828822e-01 9.4693337292598e-01
8.5357754726570e-01 3.9840922679638e-01 9.3556248582849e-01
5.1207082891874e-01 4.0112104160158e-01 9.2419171684503e-01
1.7047919477214e-01 4.0383270848743e-01 9.1282111796688e-01
8.2880362645365e-01 4.0654417967815e-01 9.0145074331761e-01
4.8704547973280e-01 4.0925540428975e-01 8.9008064831107e-01
1.4520645605533e-01 4.1196632892849e-01 8.7871088867154e-01
8.0328856681774e-01 4.1467689843028e-01 8.6734151934195e-01
4.6129409170356e-01 4.1738705672632e-01 8.5597259331541e-01
1.1922553287048e-01 4.2009674781292e-01 8.4460416043293e-01
7.7708556713457e-01 4.2280591679616e-01 8.3323626619542e-01
4.3487699849861e-01 4.2551451097705e-01 8.2186895064088e-01
9.2602713383965e-02 4.2822248093881e-01 8.1050224733664e-01
7.5026564073797e-01 4.3092978159686e-01 7.9913618253270e-01
4.0786871880850e-01 4.3363637317296e-01 7.8777077451515e-01
6.5414869837097e-02 4.3634222205786e-01 7.7640603318926e-01
7.2290698326745e-01 4.3904730153184e-01 7.6504195991088e-01
3.8034790736011e-01 4.4175159231833e-01 7.5367854757275e-01
3.7740448411792e-02 4.4445508295298e-01 7.4231578094085e-01
6.9508737616348e-01 4.4715776995783e-01 7.3095363722455e-01
3.5239143350201e-01 4.4985965781745e-01 7.1959208685487e-01
9.6553482592761e-03 4.5256075876119e-01 7.0823109443691e-01
6.6688184481716e-01 4.5526109236230e-01 6.9687061983668e-01
3.2407365334610e-01 4.5796068497097e-01 6.8551061935811e-01
9.8123351482746e-01 4.6065956900403e-01 6.7415104696442e-01
6.3836418050878e-01 4.6335778211891e-01 6.6279185549754e-01
2.9546840506915e-01 4.6605536630395e-01 6.5143299785158e-01
9.5254893347192e-01 4.6875236691988e-01 6.4007442805971e-01
6.0960848218753e-01 4.7144883172962e-01 6.2871610225928e-01
2.6664971611119e-01 4.7414480995355e-01 6.1735797950659e-01
9.2367522301675e-01 4.7684035138620e-01 6.0600002242032e-01
5.8068748772799e-01 4.7953550560722e-01 5.9464219764074e-01
2.3768886831999e-01 4.8223032131440e-01 5.8328447609997e-01
8.9468157657537e-01 4.8492484580045e-01 5.7192683310639e-01
5.5166766462599e-01 4.8761912458757e-01 5.6056924825278e-01
2.0864901924213e-01 4.9031320122569e-01 5.4921170516364e-01
8.6562736463659e-01 4.9300711725209e-01 5.3785419110094e-01
5.2260427398824e-01 4.9570091230203e-01 5.2649669645038e-01
1.7958118921762e-01 4.9839462435289e-01 5.1513921411143e-01
8.4878844560688e-01 2.6878851460167e-01 5.0412404685993e-01
4.6593394774517e-01 2.7143647511970e-01 4.9276195075784e-01
7.4231646832312e-02 2.7411079203040e-01 4.8138656191575e-01
6.8251788832196e-01 2.7686800375345e-01 4.6994667111424e-01
2.9962891233222e-01 2.7965579970286e-01 4.5847572922100e-01
9.2878810532064e-01 2.8241439290765e-01 4.4704224483234e-01
5.6747998415868e-01 2.8514441961644e-01 4.3564957419035e-01
2.1250024924863e-01 2.8785972302011e-01 4.2427928442415e-01
8.6165199969226e-01 2.9056860525371e-01 4.1291879906565e-01
5.1357467937504e-01 2.9327488447297e-01 4.0156195466754e-01
1.6741912862768e-01 2.9598022901558e-01 3.9020600237783e-01
8.2263845280818e-01 2.9868536610550e-01 3.7884977571575e-01
4.7886821211848e-01 3.0139061191657e-01 3.6749281690580e-01
1.3585769463899e-01 3.0409609969815e-01 3.5613498171084e-01
7.9342940626783e-01 3.0680187984924e-01 3.4477626152108e-01
4.5145441794036e-01 3.0950796504445e-01 3.3341670280047e-01
1.0983688978588e-01 3.1221435110362e-01 3.2205637060672e-01
7.6850407333090e-01 3.1492102681799e-01 3.1069533235765e-01
4.2739968198588e-01 3.1762797860651e-01 2.9933365099682e-01
8.6479390103110e-02 3.2033519269807e-01 2.8797138252499e-01
7.4570771133589e-01 3.2304265611287e-01 2.7660857552446e-01
4.0505579147431e-01 3.2575035705873e-01 2.6524527154735e-01
6.4499820390155e-02 3.2845828504543e-01 2.5388150583233e-01
7.2401987122816e-01 3.3116643086772e-01 2.4251730810034e-01
3.8359903967673e-01 3.3387478653158e-01 2.3115270331962e-01
4.3222797461880e-02 3.3658334515993e-01 2.1978771239770e-01
7.0287850111116e-01 3.3929210089429e-01 2.0842235279041e-01
3.6255501487812e-01 3.4200104879875e-01 1.9705663903273e-01
2.2242418754872e-02 3.4471018476746e-01 1.8569058320221e-01
6.8193178074981e-01 3.4741950543403e-01 1.7432419532779e-01
3.4161497834513e-01 3.5012900808011e-01 1.6295748375664e-01
1.2845580937491e-03 3.5283869053952e-01 1.5159045549096e-01
6.6093362520368e-01 3.5554855109434e-01 1.4022311650528e-01
3.2055575704361e-01 3.5825858835890e-01 1.2885547205365e-01
9.8014493602463e-01 3.6096880114792e-01 1.1748752697492e-01
6.3969549842992e-01 3.6367918832514e-01 1.0611928600285e-01
2.9920209658813e-01 3.6638974862893e-01 9.4750754086841e-02
9.5865967239162e-01 3.6910048047182e-01 8.3381936727357e-02
6.1806344060108e-01 3.7181138171147e-01 7.2012840328859e-02
2.7740888068891e-01 3.7452244939136e-01 6.0643472571281e-02
9.3669173618026e-01 3.7723367945037e-01 4.9273842799026e-02
5.9590802057171e-01 3.7994506640222e-01 3.7903962424237e-02
2.5505402895441e-01 3.8265660298682e-01 2.6533845338449e-02
9.1412635445430e-01 3.8536827979829e-01 1.5163508323832e-02
5.7312190853488e-01 3.8808008489636e-01 3.7929714520925e-03
2.3049359532956e-01 3.9079627749917e-01 9.9241722706422e-01
8.8933176155740e-01 3.9350826135107e-01 9.8104639863025e-01
5.4808582747207e-01 3.9622031684940e-01 9.6967545528009e-01
2.0675421803384e-01 3.9893241810384e-01 9.5830443339074e-01
8.6533579402263e-01 4.0164453494173e-01 9.4693337380831e-01
5.2382986857151e-01 4.0435663272459e-01 9.3556232176546e-01
1.8223621770137e-01 4.0706867227128e-01 9.2419132664005e-01
8.4055508342951e-01 4.0978060990964e-01 9.1282044153829e-01
4.9878716830075e-01 4.1249239767604e-01 9.0144972269229e-01
1.5693362059268e-01 4.1520398367744e-01 8.9007922867070e-01
8.1499600996608e-01 4.1791531262399e-01 8.7870901941151e-01
4.7297629393859e-01 4.2062632653231e-01 8.6733915509796e-01
1.3087677620888e-01 4.2333696559027e-01 8.5596969490980e-01
7.8870005848993e-01 4.2604716916456e-01 8.4460069569335e-01
4.4644898805837e-01 4.2875687692293e-01 8.3323221060253e-01
1.0412660362413e-01 4.3146603003501e-01 8.2186428776889e-01
7.6173608231850e-01 4.3417457240941e-01 8.1049696906086e-01
4.1928069055176e-01 4.3688245192118e-01 7.9913028899048e-01
7.6763741195812e-02 4.3958962158278e-01 7.8776427382003e-01
7.3418855902112e-01 4.4229604061365e-01 7.7639894091158e-01
3.9155845560726e-01 4.4500167536788e-01 7.6503429835002e-01
4.8876714119846e-02 4.4770650008590e-01 7.5367034485676e-01
7.0614658348426e-01 4.5041049744391e-01 7.4230706999650e-01
3.6337128067251e-01 4.5311365888344e-01 7.3094445466547e-01
2.0553999133071e-02 4.5581598471233e-01 7.1958247183703e-01
6.7769792089964e-01 4.5851748397728e-01 7.0822108752925e-01
3.3480622966051e-01 4.6121817411630e-01 6.9686026195086e-01
9.9188212207941e-01 4.6391808040742e-01 6.8549995077536e-01
6.4892881493010e-01 4.6661723523713e-01 6.7414010648983e-01
3.0594954611569e-01 4.6931567721839e-01 6.6278067976382e-01
9.6294756834190e-01 4.7201345019348e-01 6.5142162078489e-01
6.1992613503256e-01 4.7471060216137e-01 6.4006288051144e-01
2.7688847893658e-01 4.7740718417207e-01 6.2870441179885e-01
9.3383778469160e-01 4.8010324923143e-01 6.1734617036313e-01
5.9077715729763e-01 4.8279885125909e-01 6.0598811555477e-01
2.4770958894585e-01 4.8549404413882e-01 5.9463021092585e-01
9.0463792689563e-01 4.8818888089559e-01 5.8327242458318e-01
5.6156484507594e-01 4.9088341302635e-01 5.7191472933027e-01
2.1849282181578e-01 4.9357769000294e-01 5.6055710260919e-01
8.7542412561806e-01 4.9627175895625e-01 5.4919952626087e-01
5.3236081023953e-01 4.9896566454101e-01 5.3784198612726e-01
1.8930471959645e-01 5.0165944897143e-01 5.2648447152236e-01
8.4625750225516e-01 5.0435315220975e-01 5.1512697460036e-01
5.1250563189438e-01 2.7473815684433e-01 5.0412192514956e-01
1.4010446718561e-01 2.7739873978697e-01 4.9275987650640e-01
7.6172031689776e-01 2.8006713464489e-01 4.8139491918230e-01
3.8112560731007e-01 2.8277945742520e-01 4.7000134459465e-01
2.7190515914113e-03 2.8553896676650e-01 4.5856546549797e-01
6.3027286555759e-01 2.8830815438184e-01 4.4711883615123e-01
2.6510438911769e-01 2.9105973460710e-01 4.3569442623002e-01
9.0621272637588e-01 2.9379189101972e-01 4.2429711748310e-01
5.5200365411640e-01 2.9651131863843e-01 4.1291877188211e-01
2.0115554533684e-01 2.9922370317368e-01 4.0155128206322e-01
8.5272870386512e-01 3.0193245067685e-01 3.9018939219293e-01
5.0607593159259e-01 3.0463940953687e-01 3.7883011492301e-01
1.6074865575022e-01 3.0734555651582e-01 3.6747183736411e-01
8.1643034096475e-01 3.1005140615093e-01 3.5611371279881e-01
4.7289295121949e-01 3.1275722977879e-01 3.4475530983557e-01
1.2996880202788e-01 3.1546316973341e-01 3.3339642010890e-01
7.8753217185210e-01 3.1816929907778e-01 3.2203695428771e-01
4.4548707919225e-01 3.2087565328233e-01 3.1067688581968e-01
1.0375900194535e-01 3.2358224731435e-01 2.9931622036881e-01
7.6228916104612e-01 3.2628908501329e-01 2.8795497916177e-01
4.2103050315246e-01 3.2899616431021e-01 2.7659318993230e-01
7.9944829829936e-02 3.3170348016732e-01 2.6523088205340e-01
7.3900071401667e-01 3.3441102624540e-01 2.5386808399327e-01
3.9817196608448e-01 3.3711879585122e-01 2.4250482206406e-01
5.7436489554068e-02 3.3982678247217e-01 2.3114111988796e-01
7.1677541710948e-01 3.4253498007166e-01 2.1977699825692e-01
3.7617245102285e-01 3.4524338324449e-01 2.0841247520377e-01
3.5613354590954e-02 3.4795198728867e-01 1.9704756618261e-01
6.9508555652556e-01 3.5066078822651e-01 1.8568228430201e-01
3.5457784085298e-01 3.5336978279277e-01 1.7431664058142e-01
1.4080102318964e-02 3.5607896839983e-01 1.6295064421585e-01
6.7358315257827e-01 3.5878834308412e-01 1.5158430284346e-01
3.3307856624402e-01 3.6149790543549e-01 1.4021762281508e-01
9.9255855862958e-01 3.6420765450868e-01 1.2885060946812e-01
6.5201588903882e-01 3.6691758971517e-01 1.1748326740832e-01
3.1144378495804e-01 3.6962771069282e-01 1.0611560080368e-01
9.7083588361682e-01 3.7233801715034e-01 9.4747613695039e-02
6.3018618821687e-01 3.7504850868340e-01 8.3379310327045e-02
2.8948903674543e-01 3.7775918455942e-01 7.2010695502848e-02
9.4873908174199e-01 3.8047004346847e-01 6.0641774964404e-02
6.0793127970717e-01 3.8318108323832e-01 4.9272555798893e-02
2.6706088905443e-01 3.8589230051290e-01 3.7903046869639e-02
9.2612347562405e-01 3.8860369039476e-01 2.6533259267563e-02
5.8511492481785e-01 3.9131524605414e-01 1.5163206776197e-02
2.4403145938208e-01 3.9402695830964e-01 3.7929063400238e-03
9.0131680229569e-01 3.9674311329924e-01 9.9241731918974e-01
5.6007744186344e-01 3.9945507118875e-01 9.8104661989875e-01
2.1875388738794e-01 4.0216713788308e-01 9.6967574840120e-01
8.7734396133598e-01 4.0487929037927e-01 9.5830473846533e-01
5.3584595448962e-01 4.0759150119381e-01 9.4693362879281e-01
1.9425864969561e-01 4.1030373805802e-01 9.3556246308945e-01
8.5258134053263e-01 4.1301596371052e-01 9.2419128997640e-01
5.1081384302049e-01 4.1572813581428e-01 9.1282016271664e-01
1.6895649868079e-01 4.1844020702450e-01 9.0144913873671e-01
8.2701016759888e-01 4.2115212522984e-01 8.9007827893215e-01
4.8497621063676e-01 4.2386383398409e-01 8.7870764675583e-01
1.4285646059016e-01 4.2657527313686e-01 8.6733730710126e-01
8.0065318283082e-01 4.2928637966228e-01 8.5596732500728e-01
4.5836902676722e-01 4.3199708867295e-01 8.4459776422477e-01
1.1600697021571e-01 4.3470733459490e-01 8.3322868569873e-01
7.7357025941213e-01 4.3741705246814e-01 8.2186014602958e-01
4.3106234783101e-01 4.4012617932779e-01 8.1049219598360e-01
8.8486837147591e-02 4.4283465561423e-01 7.9912487912404e-01
7.4584742354046e-01 4.4554242655685e-01 7.8775823063064e-01
4.0314785208664e-01 4.4824944347589e-01 7.7639227636697e-01
6.0391881280916e-02 4.5095566495037e-01 7.6502703224232e-01
7.1758325878636e-01 4.5366105780609e-01 7.5366250389905e-01
3.7472570848546e-01 4.5636559788641e-01 7.4229868673923e-01
3.1822927858577e-02 4.5906927057834e-01 7.3093556628689e-01
6.8887859377374e-01 4.6177207107723e-01 7.1957311886531e-01
3.4589637402578e-01 4.6447400438432e-01 7.0821131255470e-01
2.8799414857186e-03 4.6717508504154e-01 6.9685010838295e-01
6.5983298756191e-01 4.6987533661817e-01 6.8548946169360e-01
3.1675923184661e-01 4.7257479097286e-01 6.7412932362900e-01
9.7366242530931e-01 4.7527348732280e-01 6.6276964266412e-01
6.3054634515295e-01 4.7797147115895e-01 6.5141036612673e-01
2.8741478039488e-01 4.8066879305207e-01 6.4005144164341e-01
9.4427150827538e-01 4.8336550739837e-01 6.2869281845690e-01
6.0112026262338e-01 4.8606167115590e-01 6.1733444856896e-01
2.5796469621567e-01 4.8875734262215e-01 6.0597628767383e-01
9.1480833985553e-01 4.9145258030079e-01 5.9461829585896e-01
5.7165456130232e-01 4.9414744189938e-01 5.8326043806233e-01
2.2850652726753e-01 4.9684198349235e-01 5.7190268428789e-01
8.8536717145899e-01 4.9953625887336e-01 5.6054500959140e-01
5.4223917114231e-01 5.0223031911020e-01 5.4918739385822e-01
1.9912493396444e-01 5.0492421230415e-01 5.3782982140121e-01
8.5602659593616e-01 5.0761798354474e-01 5.2647228041139e-01
5.1294603058798e-01 5.1031167504134e-01 5.1511476229573e-01
1.7728707743220e-01 2.8068939534165e-01 5.0411935504027e-01
8.1133658949465e-01 2.8335798810965e-01 4.9275740584949e-01
4.4115885217299e-01 2.8602862336350e-01 4.8139509858284e-01
6.8625515539245e-02 2.8872180716667e-01 4.7002035273053e-01
6.9608132675025e-01 2.9145074644434e-01 4.5861777058230e-01
3.2587092784707e-01 2.9420522915798e-01 4.4719016892982e-01
9.5986565362034e-01 2.9696248788139e-01 4.3575912368063e-01
5.9871173061382e-01 2.9970816939854e-01 4.2434254065820e-01
2.4195485292701e-01 3.0244021171644e-01 4.1294464444936e-01
8.8873626011406e-01 3.0516187631191e-01 4.0156190096211e-01
5.3823795841520e-01 3.0787683818152e-01 3.9018936567224e-01
1.8981126209119e-01 3.1058777922811e-01 3.7882308886555e-01
8.4297115652837e-01 3.1329640734215e-01 3.6746040835985e-01
4.9735974905299e-01 3.1600375665608e-01 3.5609965152998e-01
1.5271171378711e-01 3.1871044275082e-01 3.4473980156196e-01
8.0882807508414e-01 3.2141683071224e-01 3.3338025188474e-01
4.6555753781882e-01 3.2412313713814e-01 3.2202064651899e-01
1.2278340326465e-01 3.2682949040868e-01 3.1066078108451e-01
7.8041438272841e-01 3.2953596613524e-01 2.9930054250129e-01
4.3837808962779e-01 3.3224260812612e-01 2.8793987243154e-01
9.6616374611095e-02 3.3494944091202e-01 2.7657874509059e-01
7.5508193943626e-01 3.3765647731821e-01 2.6521715373660e-01
4.1373584803134e-01 3.4036372309824e-01 2.5385510242065e-01
7.2545674715941e-02 3.4307117980248e-01 2.4249260094431e-01
7.3148411052910e-01 3.4577884657267e-01 2.3112966178622e-01
3.9052790293444e-01 3.4848672127426e-01 2.1976629824556e-01
4.9657040993068e-02 3.5119480121521e-01 2.0840252334227e-01
7.0885412333229e-01 3.5390308360267e-01 1.9703834919068e-01
3.6810386373893e-01 3.5661156583085e-01 1.8567378667119e-01
2.7392701482936e-02 3.5932024565750e-01 1.7430884529130e-01
6.8670849218829e-01 3.6202912130482e-01 1.6294353316879e-01
3.4604026131725e-01 3.6473819150616e-01 1.5157785709576e-01
5.3780068658074e-03 3.6744745551164e-01 1.4021182265907e-01
6.6471254118717e-01 3.7015691305963e-01 1.2884543440301e-01
3.2403536431553e-01 3.7286656431744e-01 1.1747869602691e-01
9.8333856299340e-01 3.7557640979215e-01 1.0611161061455e-01
6.4261473098165e-01 3.7828645021045e-01 9.4744180895121e-02
3.0185690727043e-01 3.8099668636587e-01 8.3376409536642e-02
9.6105852959249e-01 3.8370711893060e-01 7.2008299473693e-02
6.2021340122908e-01 3.8641774822916e-01 6.0639854270983e-02
2.7931566953019e-01 3.8912857397142e-01 4.9271078523727e-02
9.3835981487643e-01 3.9183959494285e-01 3.7901978294412e-02
5.9734064900583e-01 3.9455080865112e-01 2.6532561583556e-02
2.5625332173034e-01 3.9726221092958e-01 1.5162838829508e-02
9.1509333508265e-01 3.9997379550058e-01 3.7928234290604e-03
5.7229565397112e-01 4.0268987436734e-01 9.9241744647718e-01
2.3098194428461e-01 4.0540176695689e-01 9.8104693007469e-01
8.8958421435201e-01 4.0811380380210e-01 9.6967618572843e-01
5.4809962627537e-01 4.1082596527779e-01 9.5830524377585e-01
2.0652583525883e-01 4.1353822718170e-01 9.4693413985157e-01
8.6486102081479e-01 4.1625056030550e-01 9.3556291512595e-01
5.2310391430904e-01 4.1896293008686e-01 9.2419161639231e-01
1.8125382056361e-01 4.2167529637476e-01 9.1282029596884e-01
8.3931063129001e-01 4.2438761334091e-01 9.0144901138489e-01
4.9727482835082e-01 4.2709982956821e-01 8.9007782482788e-01
1.5514747526938e-01 4.2981188834344e-01 8.7870680233728e-01
8.1293019602945e-01 4.3252372817349e-01 8.6733601274592e-01
4.7062514100664e-01 4.3523528353460e-01 8.5596552638453e-01
1.2823494079830e-01 4.3794648585150e-01 8.4459541358323e-01
7.8576264968784e-01 4.4065726468915e-01 8.3322574302073e-01
4.4321168138998e-01 4.4336754912569e-01 8.2185657998776e-01
1.0058574046409e-01 4.4607726926159e-01 8.1048798464252e-01
7.5788875325104e-01 4.4878635780924e-01 7.9912001034274e-01
4.1512480230554e-01 4.5149475169955e-01 7.8775270213918e-01
7.2298068023076e-02 4.5420239363902e-01 7.7638609550923e-01
7.2941278050798e-01 4.5690923355155e-01 7.6502021539740e-01
3.8647318375442e-01 4.5961522984495e-01 7.5365507561246e-01
4.3483513015571e-02 4.6232035045025e-01 7.4229067861119e-01
7.0044798494537e-01 4.6502457359360e-01 7.3092701567714e-01
3.5737079885046e-01 4.6772788827285e-01 7.1956406748145e-01
1.4256146321363e-02 4.7043029442429e-01 7.0820180499381e-01
6.7110822573234e-01 4.7313180277805e-01 6.9684019069465e-01
3.2793125769141e-01 4.7583243441325e-01 6.8547918002681e-01
9.8472949752569e-01 4.7853222003537e-01 6.7411872301586e-01
6.4150724130294e-01 4.8123119900913e-01 6.6275876598304e-01
2.9826882267445e-01 4.8392941818938e-01 6.5139925327367e-01
9.5501859889492e-01 4.8662693060023e-01 6.4004012892695e-01
6.1176092561844e-01 4.8932379401873e-01 6.2868133821908e-01
2.6850012135116e-01 4.9202006952299e-01 6.1732282902169e-01
9.2524042362079e-01 4.9471582006492e-01 6.0596455292990e-01
5.8198593987227e-01 4.9741110912586e-01 5.9460646612835e-01
2.3874059671671e-01 5.0010599950687e-01 5.8324852997942e-01
8.9550809138867e-01 5.0280055229688e-01 5.7189071133238e-01
5.5229184909559e-01 5.0549482605053e-01 5.6053298256660e-01
2.0909498941225e-01 5.0818887619409e-01 5.4917532139347e-01
8.6592030406237e-01 5.1088275466494e-01 5.3781771045042e-01
5.2277024744622e-01 5.1357650977645e-01 5.2646013672631e-01
1.7964694023178e-01 5.1627018628902e-01 5.1510259085951e-01
8.4264384019325e-01 2.8664161863973e-01 5.0411631836602e-01
4.8106813337768e-01 2.8931566791369e-01 4.9275451808135e-01
1.1637316604447e-01 2.9198978125545e-01 4.8139301968627e-01
7.4958669608724e-01 2.9467592444121e-01 4.7002583578783e-01
3.8208048319313e-01 2.9738561032571e-01 4.5864256669728e-01
1.5273096269039e-02 3.0012015410144e-01 4.4723768716323e-01
6.5053608510208e-01 3.0286888685010e-01 4.3581804949630e-01
2.8889678790510e-01 3.0561790242241e-01 4.2439778793390e-01
9.3071421197380e-01 3.0835881149922e-01 4.1298758656479e-01
5.7576223652956e-01 3.1108980883446e-01 4.0159077680519e-01
2.2354722677548e-01 3.1381247655572e-01 3.9020592282686e-01
8.7354643689929e-01 3.1652914923739e-01 3.7883006255917e-01
5.2530736852899e-01 3.1924182441938e-01 3.6746038251789e-01
1.7846741248025e-01 3.2195194634022e-01 3.5609471455661e-01
8.3274402164235e-01 3.2466048838343e-01 3.4473153094483e-01
4.8791841052242e-01 3.2736808688167e-01 3.3336980123249e-01
1.4382055280530e-01 3.3007515277179e-01 3.2200884623759e-01
8.0031724519662e-01 3.3278195043452e-01 3.1064822442793e-01
4.5730308796838e-01 3.3548864988887e-01 2.9928765178116e-01
1.1469378761732e-01 3.3819536046000e-01 2.8792694765122e-01
7.7242119781695e-01 3.4090215231757e-01 2.7656599890754e-01
4.3042963305187e-01 3.4360907026389e-01 2.6520473636395e-01
8.8673108912977e-02 3.4631614260825e-01 2.5384311931350e-01
7.4711325863512e-01 3.4902338692347e-01 2.4248112536974e-01
4.0571774610965e-01 3.5173081381381e-01 2.3111874377796e-01
6.4459046254948e-02 3.5443842940426e-01 2.1975597100135e-01
7.2331349956776e-01 3.5714623700009e-01 2.0839280780517e-01
3.8226057315710e-01 3.5985423820215e-01 1.9702925733350e-01
4.1282278696448e-02 3.6256243366095e-01 1.8566532384802e-01
7.0036271071910e-01 3.6527082358737e-01 1.7430101191252e-01
3.5948767805773e-01 3.6797940809668e-01 1.6293632588067e-01
1.8644408054016e-02 3.7068818743524e-01 1.5157126959354e-01
6.7782130817477e-01 3.7339716212210e-01 1.4020584622528e-01
3.3700777338310e-01 3.7610633302575e-01 1.2884005823731e-01
9.9619403038526e-01 3.7881570138855e-01 1.1747390741520e-01
6.5537101196260e-01 3.8152526880603e-01 1.0610739497266e-01
3.1453025618298e-01 3.8423503716457e-01 9.4740521713213e-02
9.7366382649563e-01 3.8694500853815e-01 8.3373288244822e-02
6.3276424963961e-01 3.8965518504366e-01 7.2005695245253e-02
2.9182446900403e-01 3.9236556865241e-01 6.0637743777725e-02
9.5083781161312e-01 3.9507616095561e-01 4.9269435656924e-02
6.0979796730345e-01 3.9778696288090e-01 3.7900773865211e-02
2.6869897893529e-01 4.0049797435795e-01 2.6531763017747e-02
9.2753524264891e-01 4.0320919393183e-01 1.5162409873318e-02
5.8630151724857e-01 4.0592061832473e-01 3.7927238848572e-03
2.4342444170221e-01 4.0863658429481e-01 9.9241760701566e-01
9.0203990406435e-01 4.1134837330954e-01 9.8104732535065e-01
5.6057182928976e-01 4.1406033959223e-01 9.6967676275963e-01
2.1901666772672e-01 4.1677246733405e-01 9.5830594533498e-01
8.7737137832942e-01 4.1948473616692e-01 9.4693490470646e-01
5.3563346714039e-01 4.2219712061336e-01 9.3556367845483e-01
1.9380102404905e-01 4.2490958959471e-01 9.2419231039208e-01
8.5187275523164e-01 4.2762210603320e-01 9.1282085066671e-01
5.0984800856394e-01 4.3033462658617e-01 9.0144935565321e-01
1.6772678935580e-01 4.3304710155201e-01 8.9007788758881e-01
8.2550976403160e-01 4.3575947498510e-01 8.7870651392933e-01
4.8319824990648e-01 4.3847168505163e-01 8.6733530640853e-01
1.4079418998572e-01 4.4118366464906e-01 8.5596433980243e-01
7.9830011271455e-01 4.4389534229882e-01 8.4459369041972e-01
4.5571907775392e-01 4.4660664330655e-01 8.3322343436154e-01
1.1305461004691e-01 4.4931749116636e-01 8.2185364561450e-01
7.7031062553331e-01 4.5202780916831e-01 8.1048439405947e-01
4.2749135272338e-01 4.5473752215203e-01 7.9911574349201e-01
8.4601254826731e-02 4.5744655833687e-01 7.8774774975627e-01
7.4164495715694e-01 4.6015485115113e-01 7.7638045909332e-01
3.9862718406238e-01 4.6286234097978e-01 7.6501390679444e-01
5.5552708698565e-02 4.6556897675370e-01 7.5364811623403e-01
7.1242631765032e-01 4.6827471731083e-01 7.4228309833382e-01
3.6925279087651e-01 4.7097953247178e-01 7.3091885148508e-01
2.6036895858640e-02 4.7368340378711e-01 7.1955536192867e-01
6.8278339336772e-01 4.7638632492915e-01 7.0819260456757e-01
3.3949705108429e-01 4.7908830171772e-01 6.9683054416415e-01
9.9618266054469e-01 4.8178935178504e-01 6.8546913685566e-01
6.5284505262441e-01 4.8448950389954e-01 6.7410833190789e-01
3.0948910703500e-01 4.8718879698240e-01 6.6274807361852e-01
9.6611975207516e-01 4.8988727886250e-01 6.5138830327758e-01
6.2274195205541e-01 4.9258500482614e-01 6.4002896109444e-01
2.7936068127863e-01 4.9528203602599e-01 6.2866998800675e-01
9.3598088504122e-01 4.9797843781956e-01 6.1731132729722e-01
5.9260742963951e-01 5.0067427810919e-01 6.0595292595867e-01
2.4924504465101e-01 5.0336962575428e-01 5.9459473576463e-01
9.0589826166564e-01 5.0606454912015e-01 5.8323671402160e-01
5.6257135407398e-01 5.0875911481830e-01 5.7187882399804e-01
2.1926828244954e-01 5.1145338667941e-01 5.6052103504262e-01
8.7599264952530e-01 5.1414742498518e-01 5.4916332241966e-01
5.3274766785376e-01 5.1684128596852e-01 5.3780566690113e-01
1.8953614208804e-01 5.1953502157605e-01 5.2644805416183e-01
8.4636046657524e-01 5.2222867947238e-01 5.1509047402807e-01
5.0834658356236e-01 2.9259453215257e-01 5.0411281173091e-01
1.4993247804204e-01 2.9527249985847e-01 4.9275118942923e-01
7.8913163495489e-01 2.9794983870423e-01 4.8139000624460e-01
4.2655322851517e-01 3.0063389702573e-01 4.7002608450477e-01
6.3052231927527e-02 3.0333310121298e-01 4.5865297209293e-01
6.9954126617477e-01 3.0605188293835e-01 4.4726460869529e-01
3.3693284747862e-01 3.0878733696279e-01 4.3586064024157e-01
9.7608163481216e-01 3.1153097669397e-01 4.2444776842876e-01
6.1759627884405e-01 3.1427397017722e-01 4.1303539904706e-01
2.6169482572641e-01 3.1701101563982e-01 4.0163038031328e-01
9.0825698583817e-01 3.1974063248859e-01 3.9023529105916e-01
5.5698344861012e-01 3.2246355862762e-01 3.7884967101023e-01
2.0752955342433e-01 3.2518127246950e-01 3.6747175984931e-01
8.5957491571722e-01 3.2789522474431e-01 3.5609960053944e-01
5.1284724642276e-01 3.3060658087903e-01 3.4473150579742e-01
1.6712400946094e-01 3.3331619897877e-01 3.3336617023819e-01
8.2222606154998e-01 3.3602468480758e-01 3.2200263772319e-01
4.7800963930271e-01 3.3873245732543e-01 3.1064022844552e-01
1.3435902482140e-01 3.4143980360718e-01 2.9927846720877e-01
7.9118050339752e-01 3.4414691966467e-01 2.8791702527971e-01
4.4839759562202e-01 3.4685393923186e-01 2.7655567689851e-01
1.0594736035599e-01 3.4956095357621e-01 2.6519426800297e-01
7.6377754339441e-01 3.5226802499860e-01 2.5383269417783e-01
4.2184437669991e-01 3.5497519601284e-01 2.4247088525618e-01
8.0110873088230e-02 3.5768249560179e-01 2.3110879460774e-01
7.3854549749227e-01 3.6038994350219e-01 2.1974639169256e-01
3.9712112511874e-01 3.6309755315820e-01 2.0838365687983e-01
5.5814219194076e-02 3.6580533377143e-01 1.9702057783744e-01
7.1460417779381e-01 3.6851329173391e-01 1.8565714701387e-01
3.7347281174933e-01 3.7122143163557e-01 1.7429335988316e-01
3.2403924912061e-02 3.7392975697525e-01 1.6292921372676e-01
6.9138297496892e-01 3.7663827066183e-01 1.5156470679688e-01
3.5039679817290e-01 3.7934697536404e-01 1.4019983775452e-01
9.4333852421245e-03 3.8205587374806e-01 1.2883460530911e-01
6.6848169862485e-01 3.8476496862883e-01 1.1746900801001e-01
3.2753152357296e-01 3.8747426305190e-01 1.0610304415639e-01
9.8657334718983e-01 3.9018376031604e-01 9.4736711803570e-02
6.4559826095002e-01 3.9289346394227e-01 8.3370008851888e-02
3.0459788322146e-01 3.9560337759178e-01 7.2002933209574e-02
9.6356429912397e-01 3.9831350493272e-01 6.0635483025186e-02
6.2249001567929e-01 4.0102384945422e-01 4.9267656987275e-02
2.8136793068160e-01 4.0373441422525e-01 3.7899454690208e-02
9.4019131406508e-01 4.0644520159541e-01 2.6530877065187e-02
5.9895380078311e-01 4.0915621283505e-01 1.5161926874528e-02
2.5764939434751e-01 4.1186744771326e-01 3.7926092653317e-03
9.1469685017299e-01 4.1458326657273e-01 9.9241779811787e-01
5.7324533313072e-01 4.1729491563062e-01 9.8104780001098e-01
2.3171111182480e-01 4.2000677179087e-01 9.6967747201564e-01
8.9008988205242e-01 4.2271882342823e-01 9.5830683520363e-01
5.4837785314077e-01 4.2543105449826e-01 9.4693591628163e-01
2.0657179303016e-01 4.2814344387661e-01 9.3556474817763e-01
8.6466907405260e-01 4.3085596472769e-01 9.2419337052248e-01
5.2266771663853e-01 4.3356858393957e-01 9.1282182997885e-01
1.8056642787988e-01 4.3628126166733e-01 9.0145018036726e-01
8.3836463172069e-01 4.3899395103142e-01 8.9007848253955e-01
4.9606248760639e-01 4.4170659801851e-01 8.7870680395521e-01
1.5366089476234e-01 4.4441914162981e-01 8.6733521792667e-01
8.1116147993003e-01 4.4713151431492e-01 8.5596380251576e-01
4.6856656736888e-01 4.4984364271732e-01 8.4459263908446e-01
1.2587913118721e-01 4.5255544874148e-01 8.3322181052852e-01
7.8310273149888e-01 4.5526685093179e-01 8.2185139924905e-01
4.4024143736440e-01 4.5797776613151e-01 8.1048148494332e-01
9.7299740786683e-02 4.6068811136863e-01 7.9911214231749e-01
7.5428246700527e-01 4.6339780589614e-01 7.8774343883927e-01
4.1119468680836e-01 4.6610677329978e-01 7.7637543265387e-01
6.8041636454716e-02 4.6881494357813e-01 7.6500817078242e-01
7.2482865004342e-01 4.7152225509842e-01 7.5364168770733e-01
3.8156110785188e-01 4.7422865633715e-01 7.4227600442611e-01
3.8244402424868e-02 4.7693410732636e-01 7.3091112802592e-01
6.9488392224325e-01 4.7963858074211e-01 7.1954705179872e-01
3.5148505086321e-01 4.8234206259101e-01 7.0818375588454e-01
8.0531777241737e-03 4.8504455247052e-01 6.9682120839993e-01
6.6459371557498e-01 4.8774606339845e-01 6.8545936698333e-01
3.2111211880538e-01 4.9044662122677e-01 6.7409818066860e-01
9.7761389697016e-01 4.9314626367179e-01 6.6273759198409e-01
6.3410461845445e-01 4.9584503900929e-01 6.5137753916777e-01
2.9058990047512e-01 4.9854300449685e-01 6.4001795838739e-01
9.4707538330024e-01 5.0124022459712e-01 6.2865878586054e-01
6.0356668849761e-01 5.0393676908419e-01 6.1729995978037e-01
2.6006936296319e-01 5.0663271111945e-01 6.0594142196903e-01
9.1658881219814e-01 5.0932812538264e-01 5.9458311920103e-01
5.7313022759531e-01 5.1202308633851e-01 5.8322500416163e-01
2.2969851321806e-01 5.1471766670836e-01 5.7186703602886e-01
8.8629821764319e-01 5.1741193620064e-01 5.6050918069026e-01
5.4293347592144e-01 5.2010596053635e-01 5.4915141062509e-01
1.9960796568838e-01 5.2279980078491e-01 5.3779370449787e-01
8.5632488009949e-01 5.2549351300704e-01 5.2643604651917e-01
5.1308691875501e-01 5.2818714818325e-01 5.1507842563402e-01
1.7427355146931e-01 2.9854797563860e-01 5.0410883782710e-01
8.1825105326116e-01 3.0122886987062e-01 4.9274740940297e-01
4.6034756389551e-01 3.0390892176596e-01 4.8138640379705e-01
1.0094672550799e-01 3.0659287751210e-01 4.7002402503164e-01
7.4060648945329e-01 3.0928668974407e-01 4.5865627727552e-01
3.7994558946427e-01 3.1199491966375e-01 4.4727837013393e-01
1.9586480183779e-02 3.1471819607500e-01 4.3588766069342e-01
6.6014768631257e-01 3.1745268310768e-01 4.2448588913479e-01
3.0218674122664e-01 3.2019205682002e-01 4.1307860711764e-01
9.4608666031023e-01 3.2293046269010e-01 4.0167226223148e-01
5.9198894850283e-01 3.2566434553982e-01 3.9027149343098e-01
2.3982642328576e-01 3.2839251143278e-01 3.7887830355858e-01
8.8940936528633e-01 3.3111526413012e-01 3.6749268775138e-01
5.4050384323261e-01 3.3383354005896e-01 3.5611361083985e-01
1.9287922940168e-01 3.3654838143690e-01 3.4473972613079e-01
8.4632944371541e-01 3.3926070507689e-01 3.3336975164681e-01
5.0067885565610e-01 3.4197123881759e-01 3.2200261328608e-01
1.5578106597902e-01 3.4468053107648e-01 3.1063746808424e-01
8.1151510666714e-01 3.4738898373997e-01 2.9927367826422e-01
4.6778116418606e-01 3.5009688701580e-01 2.8791077151399e-01
1.2449665866924e-01 3.5280444894212e-01 2.7654840252171e-01
7.8159292729932e-01 3.5551181819805e-01 2.6518632080846e-01
4.3901251979328e-01 3.5821910097512e-01 2.5382434558964e-01
9.6707029840129e-02 3.6092637318146e-01 2.4246234681483e-01
7.5463536793524e-01 3.6363368917715e-01 2.3110023114246e-01
4.1276238749044e-01 3.6634108800311e-01 2.1973793167295e-01
7.1057790008415e-02 3.6904859782427e-01 2.0837540047239e-01
7.2949524961153e-01 3.7175623910790e-01 1.9701260313995e-01
3.8805170993191e-01 3.7446402690715e-01 1.8564951486220e-01
4.6706816760586e-02 3.7717197250966e-01 1.7428611754701e-01
7.0544245806613e-01 3.7988008463350e-01 1.6292239774237e-01
3.6424238938117e-01 3.8258837029744e-01 1.5155834512842e-01
2.3091927489428e-02 3.8529683545441e-01 1.4019395143111e-01
6.8197769915221e-01 3.8800548544975e-01 1.2882920964961e-01
3.4088743455389e-01 3.9071432534692e-01 1.1746411352099e-01
9.9980979742632e-01 3.9342336014979e-01 1.0609865716825e-01
6.5873424558953e-01 3.9613259494091e-01 9.4732834894505e-02
3.1765091703894e-01 3.9884203494800e-01 8.3366641097803e-02
9.7655053780616e-01 4.0155168554601e-01 7.2000070289904e-02
6.3542434868806e-01 4.0426155219787e-01 6.0633117208572e-02
2.9426404862312e-01 4.0697164033457e-01 4.9265777017259e-02
9.5306175303240e-01 4.0968195517305e-01 3.7898045588990e-02
6.1180996585390e-01 4.1239250146914e-01 2.6529919872819e-02
2.7050156430191e-01 4.1510328320215e-01 1.5161398341571e-02
9.2912979558145e-01 4.1781430318801e-01 3.7924815187580e-03
5.8610598506236e-01 4.2052994412139e-01 9.9241801630974e-01
2.4459164033343e-01 4.2324141960203e-01 9.8104834637064e-01
9.0299581165639e-01 4.2595312812017e-01 9.6967830288650e-01
5.6131340803545e-01 4.2866506251197e-01 9.5830790120499e-01
2.1953984545898e-01 4.3137721145852e-01 9.4693716224990e-01
8.7767109737166e-01 4.3408955873276e-01 9.3556611326779e-01
5.3570374865744e-01 4.3680208244180e-01 9.2419478850776e-01
1.9363505037762e-01 4.3951475430019e-01 9.1282322978373e-01
8.5146297200990e-01 4.4222753897805e-01 9.0145148686142e-01
5.0918624753277e-01 4.4494039357521e-01 8.9007961760897e-01
1.6680441148486e-01 4.4765326727764e-01 8.7870768785078e-01
8.2431782119128e-01 4.5036610125361e-01 8.6733577087032e-01
4.8172766176990e-01 4.5307882884424e-01 8.5596394652122e-01
1.3903593136171e-01 4.5579137609374e-01 8.4459229992601e-01
7.9624540527022e-01 4.5850366264954e-01 8.3322091976880e-01
4.5335957928231e-01 4.6121560304223e-01 8.2184989621973e-01
1.1038259423908e-01 4.6392710832937e-01 8.1047931856257e-01
7.6731914572910e-01 4.6663808806064e-01 7.9910927262841e-01
4.2417438434689e-01 4.6934845249486e-01 7.8773983816440e-01
8.0953813055571e-02 4.7205811497645e-01 7.7637108628275e-01
7.3766318861312e-01 4.7476699436294e-01 7.6500307713954e-01
3.9430843365506e-01 4.7747501738634e-01 7.5363585798391e-01
5.0895564868315e-02 4.8018212083272e-01 7.4226946169684e-01
7.0743064085849e-01 4.8288825343347e-01 7.3090390590713e-01
3.6391973102293e-01 4.8559337737843e-01 7.1953919273424e-01
2.0368904280034e-02 4.8829746938258e-01 7.0817530916669e-01
6.7678423418021e-01 4.9100052126235e-01 6.9681222804526e-01
3.3317181502512e-01 4.9370254000259e-01 6.8544990958456e-01
9.8953778238234e-01 4.9640354731976e-01 6.7408830333781e-01
6.4588833095059e-01 4.9910357874980e-01 6.6272735048816e-01
3.0222972315910e-01 5.0180268231019e-01 6.5136698633680e-01
9.5856828311972e-01 5.0450091680414e-01 6.4000714285357e-01
6.1491037244236e-01 5.0719834985072e-01 6.2864775115911e-01
2.7126234674025e-01 5.0989505573672e-01 6.1728874381874e-01
9.2763049409984e-01 5.1259111319332e-01 6.0593005684632e-01
5.8402095906277e-01 5.1528660320224e-01 5.9457163134008e-01
2.4043965747738e-01 5.1798160693129e-01 5.8321341470000e-01
8.9689218870753e-01 5.2067620388730e-01 5.7185536140576e-01
5.5338375202201e-01 5.2337047035729e-01 5.6049743336286e-01
2.0991907353037e-01 5.2606447818648e-01 5.4913959984987e-01
8.6650234889741e-01 5.2875829391742e-01 5.3778183712004e-01
5.2313720545898e-01 5.3145197829035e-01 5.2642412772379e-01
1.7982668552148e-01 5.3414558608287e-01 5.1506645962550e-01
8.4035413725595e-01 3.0450185346546e-01 5.0410440244503e-01
4.8620403134022e-01 3.0718499951260e-01 4.9274317600058e-01
1.3053536389418e-01 3.0986727800475e-01 4.8138231419516e-01
7.7360195171326e-01 3.1255189402859e-01 4.7002074587628e-01
4.1578170740672e-01 3.1524306615520e-01 4.5865594183550e-01
5.7509490277745e-02 3.1794466958468e-01 4.4728443084583e-01
6.9922846464221e-01 3.2065859036284e-01 4.3590331882494e-01
3.4138169058405e-01 3.2338377529035e-01 4.2451199753276e-01
9.8440371474262e-01 3.2611661531818e-01 4.1311281611456e-01
6.2867192872911e-01 3.2885245162510e-01 4.0171014882150e-01
2.7443942017469e-01 3.3158724767768e-01 3.9030855523255e-01
9.2180279091683e-01 3.3431850674512e-01 3.7891131771502e-01
5.7072316110804e-01 3.3704526625164e-01 3.6752000825057e-01
2.2107568859684e-01 3.3976760405197e-01 3.5613482882773e-01
8.7269734221267e-01 3.4248611206163e-01 3.4475518415382e-01
5.2541928509957e-01 3.4520153598327e-01 3.3338015273548e-01
1.7908403054287e-01 3.4791458717993e-01 3.2200877293731e-01
8.3355232348341e-01 3.5062586874781e-01 3.1064018029614e-01
4.8870439169391e-01 3.5333586157995e-01 2.9927365455310e-01
1.4443859496965e-01 3.5604493618813e-01 2.8790862102472e-01
8.0066912857470e-01 3.5875337249711e-01 2.7654463293820e-01
4.5732358165382e-01 3.6146137968254e-01 2.6518134845173e-01
1.1434068980081e-01 3.6416911316312e-01 2.5381850855203e-01
7.7166839426608e-01 3.6687668814612e-01 2.4245591811972e-01
4.2926221813262e-01 3.6958419002787e-01 2.3109343062063e-01
8.7083929103719e-02 3.7229168221944e-01 2.1973093612048e-01
7.4510044641614e-01 3.7499921197312e-01 2.0836835207474e-01
4.0328294957586e-01 3.7770681470040e-01 1.9700561632745e-01
6.1606151442139e-02 3.8041451716884e-01 1.8564268182172e-01
7.2004770420935e-01 3.8312233987217e-01 1.7427951261669e-01
3.7858771261788e-01 3.8583029879201e-01 1.6291608089362e-01
3.7208333756542e-02 3.8853840671130e-01 1.5155236470887e-01
6.9589344699774e-01 3.9124667419569e-01 1.4018834631131e-01
3.5462838099686e-01 3.9395511032687e-01 1.2882401088834e-01
1.3399687405452e-02 3.9666372324783e-01 1.1745934564052e-01
6.7219495311647e-01 3.9937252056305e-01 1.0609433911147e-01
3.3100264459033e-01 4.0208150962332e-01 9.4728980720232e-02
9.8981197919279e-01 4.0479069771550e-01 8.3363260458422e-02
6.4861281958555e-01 4.0750009217063e-01 7.1997168726025e-02
3.0739558810381e-01 4.1020970039795e-01 6.0630696288428e-02
9.6615119877618e-01 4.1291952984847e-01 4.9263834343584e-02
6.2487100522360e-01 4.1562958790874e-01 3.7896574692926e-02
2.8354676313597e-01 4.1833988172274e-01 2.6528910012843e-02
9.4217060638442e-01 4.2105041793868e-01 1.5160834225293e-02
6.0073503608712e-01 4.2376120237618e-01 3.7923429666790e-03
2.5764441265186e-01 4.2647663878808e-01 9.9241825734897e-01
9.1607166432110e-01 4.2918791070981e-01 9.8104895478154e-01
5.7441907321146e-01 4.3189943703042e-01 9.6967924157711e-01
2.3268074152552e-01 4.3461121522085e-01 9.5830912676082e-01
8.9085126006102e-01 4.3732323897979e-01 9.4693862471330e-01
5.4892576232855e-01 4.4003549741403e-01 9.3556775605338e-01
2.0689998525172e-01 4.4274797417663e-01 9.2419654850058e-01
8.6477033391976e-01 4.4546064659505e-01 9.1282503766658e-01
5.2253394719053e-01 4.4817348483121e-01 9.0145326771442e-01
1.8018876029393e-01 4.5088645112651e-01 8.9008129181573e-01
8.3773356005872e-01 4.5359949919354e-01 8.7870917233141e-01
4.9516802809330e-01 4.5631257382287e-01 8.6733698064179e-01
1.5249276731553e-01 4.5902561077538e-01 8.5596479656017e-01
8.0970930775435e-01 4.6173853702596e-01 8.4459270728050e-01
4.6682008860337e-01 4.6445127141331e-01 8.3322080583568e-01
1.2382841509135e-01 4.6716372573085e-01 8.2184918907700e-01
7.8073839075353e-01 4.6987580626700e-01 8.1047795522549e-01
4.3755482795053e-01 4.7258741577151e-01 7.9910720108757e-01
9.4283141712158e-02 4.7529845578962e-01 7.8773701906662e-01
7.5092923385956e-01 4.7800882927374e-01 7.7636749413229e-01
4.0749937556624e-01 4.8071844335449e-01 7.6499870092725e-01
6.4000096812277e-02 4.8342721213524e-01 7.5363070119210e-01
7.2043809045053e-01 4.8613505936665e-01 7.4226354167398e-01
3.7682013687676e-01 4.8884192086243e-01 7.3089725265304e-01
3.3153052775672e-02 4.9154774653253e-01 7.1953184717875e-01
6.8944366442456e-01 4.9425250193306e-01 7.0816732105800e-01
3.4569880297042e-01 4.9695616926124e-01 6.9680365358638e-01
1.9253163628983e-03 4.9965874775424e-01 6.8544080896571e-01
6.5813009057985e-01 5.0236025348267e-01 6.7407873831013e-01
3.1432007168867e-01 5.0506071855874e-01 6.6271738211080e-01
9.7050228028380e-01 5.0776018980713e-01 6.5135667300803e-01
6.2668381092762e-01 5.1045872697118e-01 6.3999653870842e-01
2.8287181125912e-01 5.1315640054848e-01 6.2863690488503e-01
9.3907343815947e-01 5.1585328936715e-01 6.1727769790825e-01
5.9529579141557e-01 5.1854947802551e-01 6.0591884727483e-01
2.5154582829740e-01 5.2124505432286e-01 5.9456028762989e-01
9.0783026496528e-01 5.2394010680542e-01 5.8320196031014e-01
5.6415547232594e-01 5.2663472253985e-01 5.7184381437302e-01
2.2052737466518e-01 5.2932898520627e-01 5.6048580711296e-01
8.7695135906091e-01 5.3202297357705e-01 5.4912790409900e-01
5.3343220234152e-01 5.3471676041733e-01 5.3777007879469e-01
1.8997402044737e-01 5.3741041181275e-01 5.2641231183979e-01
8.4658024279301e-01 5.4010398690250e-01 5.1505459008250e-01
5.0654459078112e-01 3.1045610406624e-01 5.0409951327798e-01
1.5390013496576e-01 3.1314102324941e-01 4.9273849218882e-01
8.0000516091227e-01 3.1582512341134e-01 4.8137777144028e-01
4.4503456986210e-01 3.1851065305339e-01 4.7001668269186e-01
8.9252586337528e-02 3.2120064948331e-01 4.5865357950111e-01
7.3297299604724e-01 3.2389820855090e-01 4.4728600419266e-01
3.7652226909509e-01 3.2660546435882e-01 4.3591145634166e-01
2.0226404725275e-02 3.2932276010320e-01 4.2452849207420e-01
6.6441166527550e-01 3.3204845164596e-01 4.1313754912984e-01
3.0939142096260e-01 3.3477947115436e-01 4.0174096287697e-01
9.5542927317549e-01 3.3751236208095e-01 3.9034213625961e-01
6.0269929251556e-01 3.4024424388094e-01 3.7894438835637e-01
2.5127008526807e-01 3.4297329721778e-01 3.6755012458748e-01
9.0111856938352e-01 3.4569874116760e-01 3.5616059453908e-01
5.5215991251943e-01 3.4842053660485e-01 3.4477608564405e-01
2.0427773472576e-01 3.5113905450996e-01 3.3339627144026e-01
8.5734637780964e-01 3.5385482776591e-01 3.2202052438867e-01
5.1124421007182e-01 3.5656840434061e-01 3.1064812815123e-01
1.6586013071180e-01 3.5928027689195e-01 2.9927839608643e-01
8.2109585209921e-01 3.6199085842334e-01 2.8791072483221e-01
4.7686590881533e-01 3.6470048151895e-01 2.7654460996868e-01
1.3309660935785e-01 3.6740940764773e-01 2.6517964243132e-01
7.8972460523586e-01 3.7011783943426e-01 2.5381549676454e-01
4.4669541766056e-01 3.7282593260312e-01 2.4245191712448e-01
1.0396207488402e-01 3.7553380632589e-01 2.3108870384972e-01
7.6148391577808e-01 3.7824155167708e-01 2.1972570173364e-01
4.1922556791148e-01 3.8094923832539e-01 2.0836279028296e-01
7.7156087519672e-02 3.8365691973232e-01 1.9699987585863e-01
7.3524824121939e-01 3.8636463715032e-01 1.8563688544658e-01
3.9347790811756e-01 3.8907242268199e-01 1.7427376177419e-01
5.1823582524747e-02 3.9178030161668e-01 1.6291045950736e-01
7.1026596004023e-01 3.9448829421537e-01 1.5154694230208e-01
3.6878759249762e-01 3.9719641707564e-01 1.4018318052583e-01
2.7372599788720e-02 3.9990468417646e-01 1.2881914950267e-01
6.8600642879158e-01 4.0261310767726e-01 1.1745482816855e-01
3.4467565149678e-01 4.0532169852653e-01 1.0609019805009e-01
3.3677959815289e-03 4.0803046691987e-01 9.4725242501732e-02
6.6207120516610e-01 4.1073942263636e-01 8.3359946152779e-02
3.2077491934736e-01 4.1344857527274e-01 7.1994294529440e-02
9.7946857937880e-01 4.1615793438866e-01 6.0628273827292e-02
6.3814234809089e-01 4.1886750957016e-01 4.9261870817704e-02
2.9678684814495e-01 4.2157731041488e-01 3.7895072878019e-02
9.5539311500797e-01 4.2428734643873e-01 2.6527868139997e-02
6.1395256413789e-01 4.2699762690130e-01 1.5160245754294e-02
2.7245697178349e-01 4.2970816054543e-01 3.7921962706897e-03
9.2930421005624e-01 4.3242337083058e-01 9.9241851627506e-01
5.8767771815049e-01 4.3513441370162e-01 9.8104961371526e-01
2.4597347317426e-01 4.3784572716118e-01 9.6968027116916e-01
9.0418476401113e-01 4.4055731337309e-01 9.5831049087724e-01
5.6230533812843e-01 4.4326917121779e-01 9.4694028005627e-01
2.2032945712185e-01 4.4598129543791e-01 9.3556965188322e-01
8.7825196198479e-01 4.4869367570355e-01 9.2419862651140e-01
5.3606834601962e-01 4.5140629562262e-01 9.1282723207170e-01
1.9377483251078e-01 4.5411913173343e-01 9.0145550559851e-01
8.5136845341613e-01 4.5683215252990e-01 8.9008349380677e-01
5.0884712449230e-01 4.5954531758283e-01 8.7871125363505e-01
1.6620971157701e-01 4.6225857683275e-01 8.6733885246373e-01
8.2345608235583e-01 4.6497187013778e-01 8.5596636791695e-01
4.8058713800666e-01 4.6768512716282e-01 8.4459388716762e-01
1.3760481978296e-01 4.7039826769069e-01 8.3322150568547e-01
7.9451208695418e-01 4.7311120242093e-01 8.2184932540164e-01
4.5131286456088e-01 4.7582383429674e-01 8.1047745230651e-01
1.0801196202709e-01 4.7853606036586e-01 7.9910599354902e-01
7.6461496653764e-01 4.8124777414055e-01 7.8773505415788e-01
4.2112811785253e-01 4.8395886837829e-01 7.7636473355344e-01
7.7558173459960e-02 4.8666923816442e-01 7.6499512205436e-01
7.3391227425704e-01 4.8937878414608e-01 7.5362629760132e-01
3.9019782099826e-01 4.9208741574655e-01 7.4225832291653e-01
4.6422370460575e-02 4.9479505418458e-01 7.3089124329221e-01
7.0259355775260e-01 4.9750163513276e-01 7.1952508515738e-01
3.5871904776557e-01 5.0020711087219e-01 7.0815985551470e-01
1.4806514897110e-02 5.0291145183249e-01 6.9679554227525e-01
6.7086364641931e-01 5.0561464744439e-01 6.8543211545574e-01
3.2689816173492e-01 5.0831670627135e-01 6.7406952914537e-01
9.8291783770003e-01 5.1101765542582e-01 6.6270772410262e-01
6.3893052946823e-01 5.1371753931195e-01 6.5134663080828e-01
2.9494417702606e-01 5.1641641776972e-01 6.3998617278076e-01
9.5096678964623e-01 5.1911436372469e-01 6.2862626995355e-01
6.0700640360182e-01 5.2181146047142e-01 6.1726684192204e-01
2.6307101223209e-01 5.2450779873650e-01 6.0590781088723e-01
9.1916847130169e-01 5.2720347367640e-01 5.9454910415495e-01
5.7530638600756e-01 5.2989858196462e-01 5.8319065608942e-01
2.3149198848808e-01 5.3259321911117e-01 5.7183240946491e-01
8.8773201595754e-01 5.3528747713456e-01 5.6047431620622e-01
5.4403259951207e-01 5.3798144267566e-01 5.4911633755129e-01
2.0039917234019e-01 5.4067519560553e-01 5.3775844370523e-01
8.5683640381552e-01 5.4336880814128e-01 5.2640061308024e-01
5.1334816316143e-01 5.4606234444817e-01 5.1504283122958e-01
1.7281636643468e-01 3.1641068512843e-01 5.0409417937308e-01
8.2140881368467e-01 3.1909702617875e-01 4.9273336400613e-01
4.6895435218162e-01 3.2178262287308e-01 4.8137278978521e-01
1.1557767730174e-01 3.2446910464971e-01 4.7001202364716e-01
7.6146870877806e-01 3.2715870149926e-01 4.5864996297807e-01
4.0685876258077e-01 3.2985383828291e-01 4.4728486724181e-01
5.1994233573557e-02 3.3255650617788e-01 4.3591474395723e-01
6.9712233434469e-01 3.3526764589933e-01 4.2453801432704e-01
3.4249017400868e-01 3.3798681029396e-01 4.1315417877720e-01
9.8834383701422e-01 3.4071226620332e-01 4.0176414084359e-01
6.3491412199624e-01 3.4344150263267e-01 3.9036999784656e-01
2.8239022447327e-01 3.4617191948069e-01 3.7897440232119e-01
9.3089590879806e-01 3.4890139917637e-01 3.6757983351931e-01
5.8048144918730e-01 3.5162857545900e-01 3.5618810568722e-01
2.3113289729930e-01 3.5435280921281e-01 3.4480022504524e-01
8.8279096599742e-01 3.5707400265118e-01 3.3341650467877e-01
5.3537074218711e-01 3.5979238367301e-01 3.2203678338257e-01
1.8877720613461e-01 3.6250833278584e-01 3.1066063672464e-01
8.4291538190535e-01 3.6522227056324e-01 2.9928753328068e-01
4.9769599192381e-01 3.6793459597241e-01 2.8791693193819e-01
1.5303802693442e-01 3.7064565888133e-01 2.7654833362417e-01
8.0886945893139e-01 3.7335575239964e-01 2.6518130326873e-01
4.6512695489749e-01 3.7606511529971e-01 2.5381547455218e-01
1.2175512205909e-01 3.7877393871798e-01 2.4245054581848e-01
7.7870558589886e-01 3.8148237402423e-01 2.3108627218941e-01
4.3593605892790e-01 3.8419054036048e-01 2.1972245665314e-01
9.3409475152720e-02 3.8689853124095e-01 2.0835894151715e-01
7.5109321967920e-01 3.8960642005819e-01 1.9699560082619e-01
4.0895845937810e-01 3.9231426454999e-01 1.8563233392406e-01
6.6979569294967e-02 3.9502211036413e-01 1.7426906012960e-01
7.2513364484061e-01 3.9772999387556e-01 1.6290571440727e-01
3.8340008854256e-01 4.0043794440145e-01 1.5154224388446e-01
4.1760260521927e-02 4.0314598593834e-01 1.4017860507070e-01
7.0019718295157e-01 4.0585413852333e-01 1.2881476164911e-01
3.5869529007395e-01 4.0856241930006e-01 1.1745068273129e-01
1.7240216676317e-02 4.1127084335209e-01 1.0608634148666e-01
6.7581861913683e-01 4.1397942435148e-01 9.4721714075878e-02
3.3441802423023e-01 4.1668817505811e-01 8.3356778833618e-02
9.9302670181446e-01 4.1939710769572e-01 7.1991515659534e-02
6.5163355832063e-01 4.2210623422260e-01 6.0625905586985e-02
3.1022804865502e-01 4.2481556650858e-01 4.9259930508313e-02
9.6880010470932e-01 4.2752511642490e-01 3.7893573042686e-02
6.2734007917795e-01 4.3023489584899e-01 2.6526816538411e-02
2.8583870380815e-01 4.3294491658294e-01 1.5159645206180e-02
9.4428706156529e-01 4.3565519018147e-01 3.7920443830128e-03
6.0107702408248e-01 4.3837015840747e-01 9.9241878749194e-01
2.5940164400782e-01 4.4108095201358e-01 9.8105030992704e-01
9.1765106974763e-01 4.4379202673333e-01 9.6968137181138e-01
5.7581778456913e-01 4.4650338936129e-01 9.5831196830150e-01
2.3389468953221e-01 4.4921504398315e-01 9.4694209900553e-01
8.9187515787913e-01 4.5192699112326e-01 9.3557176902428e-01
5.4975310214425e-01 4.5463922677537e-01 9.2420099007909e-01
2.0752305255104e-01 4.5735174133323e-01 9.1282978168730e-01
8.6518024441427e-01 4.6006451844957e-01 9.0145817233054e-01
5.2272071125244e-01 4.6277753386726e-01 8.9008620054390e-01
1.8014137920667e-01 4.6549075428251e-01 8.7871391583687e-01
8.3744015727049e-01 4.6820413631703e-01 8.6734137934267e-01
4.9461601692236e-01 4.7091762569132e-01 8.5596866408347e-01
1.5166905421979e-01 4.7363115670187e-01 8.4459585473919e-01
8.0860052747915e-01 4.7634465210864e-01 8.3322304681990e-01
4.6541286451877e-01 4.7905802353179e-01 8.2185034516903e-01
1.2210963520185e-01 4.8177117243714e-01 8.1047786176727e-01
7.7869548765872e-01 4.8448399175693e-01 7.9910571286349e-01
4.3517604990185e-01 4.8719636814709e-01 7.8773401552414e-01
9.1557802192503e-02 4.8990818483019e-01 7.7636288375939e-01
7.4784792894835e-01 4.9261932491773e-01 7.6499242444304e-01
4.0405416161060e-01 4.9532967505601e-01 7.5362273328481e-01
6.0184625190046e-02 4.9803912920218e-01 7.4225389113015e-01
7.1624770083865e-01 5.0074759231623e-01 7.3088596085157e-01
3.7225191467736e-01 5.0345498375412e-01 7.1951898505543e-01
2.8205859488824e-02 5.0616124016510e-01 7.0815298476631e-01
6.8411815126149e-01 5.0886631773012e-01 6.9678795917416e-01
3.3999741763195e-01 5.1157019362206e-01 6.8542388644761e-01
9.9585231075137e-01 5.1427286661824e-01 6.7406072553860e-01
6.5169153366564e-01 5.1697435684563e-01 6.6269841883613e-01
3.0752386744530e-01 5.1967470468757e-01 6.5133689547468e-01
9.6335818629709e-01 5.2237396892485e-01 6.3997607506828e-01
6.1920344973068e-01 5.2507222422385e-01 6.2861587162489e-01
2.7506866428035e-01 5.2776955811787e-01 6.1725619739804e-01
9.3096281178926e-01 5.3046606765375e-01 6.0589696645149e-01
5.8689474618935e-01 5.3316185589249e-01 5.9453809774790e-01
2.4287306531591e-01 5.3585702845581e-01 5.8317951761935e-01
8.9890596789949e-01 5.3855169030086e-01 5.7182116153415e-01
5.5500110797357e-01 5.4124594288023e-01 5.6046297513304e-01
2.1116545928769e-01 5.4393988180740e-01 5.4910491456430e-01
8.6740520099286e-01 5.4663359510251e-01 5.3774694619540e-01
5.2372563321607e-01 5.4932716204457e-01 5.2638904581320e-01
1.8013112769095e-01 5.5202065261031e-01 5.1503119744447e-01
8.3915005203120e-01 3.2236556582499e-01 5.0408841084221e-01
4.8877648394561e-01 3.2505306377521e-01 4.9272779954878e-01
1.3751424937047e-01 3.2773989782361e-01 4.8136737817309e-01
7.8545484679426e-01 3.3042728194677e-01 4.7000685704508e-01
4.3273814627469e-01 3.3311688631900e-01 4.5864547786036e-01
7.9537869602397e-02 3.3581060505598e-01 4.4728199581686e-01
7.2604332076550e-01 3.3851016571716e-01 4.3591487166882e-01
3.7244653655511e-01 3.4121669571788e-01 4.2454266579524e-01
1.8938867832366e-02 3.4393040106815e-01 4.1316450996857e-01
6.6571214541182e-01 3.4665048163820e-01 4.0178046331830e-01
3.1295526131103e-01 3.4937532278358e-01 3.9039158519111e-01
9.6084131932522e-01 3.5210289576287e-01 3.7899968055390e-01
6.0950936545570e-01 3.5483121353742e-01 3.6760683382541e-01
2.5904986804376e-01 3.5755867907974e-01 3.5621493947397e-01
9.0950060236049e-01 3.6028424037447e-01 3.4482539911928e-01
5.6085303825266e-01 3.6300736936683e-01 3.3343903308130e-01
2.1306468878292e-01 3.6572794131434e-01 3.2205615100406e-01
8.6607234597681e-01 3.6844609063364e-01 3.1067669344286e-01
5.1980305864671e-01 3.7116208851268e-01 2.9930037667781e-01
1.7418183942459e-01 3.7387625716973e-01 2.8792680769402e-01
8.2913638640525e-01 3.7658891801608e-01 2.7655556210597e-01
4.8459957889265e-01 3.7930036475477e-01 2.6518623046445e-01
1.4051050959884e-01 3.8201085242581e-01 2.5381844192596e-01
7.9681464385622e-01 3.8472059557340e-01 2.4245187347130e-01
4.5346350651383e-01 3.8742977104323e-01 2.3108625074970e-01
1.1041414597975e-01 3.9013852272614e-01 2.1972134457173e-01
7.6762852056197e-01 3.9284696677460e-01 2.0835696550868e-01
4.2507288603652e-01 3.9555519655704e-01 1.9699295799648e-01
8.2717223662239e-02 3.9826328703811e-01 1.8562919467550e-01
7.4053472514516e-01 4.0097129849752e-01 1.7426557130323e-01
3.9850133867916e-01 4.0367927961187e-01 1.6290200235431e-01
5.6595373870747e-02 4.0638726997086e-01 1.5153841730582e-01
7.1479716047269e-01 4.0909530211327e-01 1.4017475755047e-01
3.7308875485010e-01 4.1180340316575e-01 1.2881097385933e-01
3.1453688086604e-02 4.1451159615788e-01 1.1744702431301e-01
6.8987675010227e-01 4.1721990107523e-01 1.0608287262630e-01
3.4834380482140e-01 4.1992833570024e-01 9.4718486801785e-02
6.8416321512336e-03 4.2263691628036e-01 8.3353838058487e-02
6.6535779323868e-01 4.2534565805332e-01 7.1988899992746e-02
3.2388051612996e-01 4.2805457565179e-01 6.0623647938327e-02
9.8239859955559e-01 4.3076368340268e-01 4.9258058501325e-02
6.4090133310106e-01 4.3347299553082e-01 3.7892109253073e-02
2.9937843250498e-01 4.3618252627179e-01 2.6525778570670e-02
9.5781998925556e-01 4.3889228989442e-01 1.5159045620160e-02
6.1621643403568e-01 4.4160230062994e-01 3.7918904817484e-03
2.7295413614372e-01 4.4431701709768e-01 9.9241906488254e-01
9.3123487575866e-01 4.4702754718845e-01 9.8105102870175e-01
5.8944346175069e-01 4.4973836289183e-01 9.6968252104348e-01
2.4757159382108e-01 4.5244947545165e-01 9.5831352986360e-01
9.0561133932865e-01 4.5516089403654e-01 9.4694404692221e-01
5.6355518374034e-01 4.5787262492814e-01 9.3557406883113e-01
2.2139609642226e-01 4.6058467056085e-01 9.2420359824088e-01
8.7912761114575e-01 4.6329702841900e-01 9.1283264513960e-01
5.3674391987052e-01 4.6600968980889e-01 9.0146122821956e-01
1.9423997730357e-01 4.6872263853815e-01 8.9008937624744e-01
8.5161161244738e-01 4.7143584955339e-01 8.7871712935245e-01
5.0885564192578e-01 4.7414928760786e-01 8.6734454012447e-01
1.6596997845405e-01 4.7686290605264e-01 8.5597167439524e-01
8.2295372661472e-01 4.7957664586449e-01 8.4459861156357e-01
4.7980725738130e-01 4.8229043503782e-01 8.3322544432483e-01
1.3653225289383e-01 4.8500418847240e-01 8.2185227768079e-01
7.9313171409339e-01 4.8771780847944e-01 8.1047922714064e-01
4.4960992612141e-01 4.9043118600235e-01 7.9910641607982e-01
1.0597237984496e-01 4.9314420260514e-01 7.8773397229632e-01
7.6222565219852e-01 4.9585673322271e-01 7.7636202388853e-01
4.1837725269357e-01 4.9856864959864e-01 7.6499069466298e-01
7.4435447706586e-02 5.0127982426664e-01 7.5362009935299e-01
7.3040907721894e-01 5.0399013487053e-01 7.4225033897621e-01
3.8630737986986e-01 5.0669946857394e-01 7.3088149667259e-01
4.2139841108131e-02 5.0940772629115e-01 7.1951363433947e-01
6.9791607590332e-01 5.1211482647678e-01 7.0814679032084e-01
3.5364575234259e-01 5.1482070824220e-01 6.9678097832059e-01
9.3385562766922e-03 5.1752533361495e-01 6.8541618760804e-01
6.6500419096143e-01 5.2022868881760e-01 6.7405238447961e-01
3.2065240035054e-01 5.2293078450639e-01 6.6268951484555e-01
9.7629300115631e-01 5.2563165497380e-01 6.5132750773246e-01
6.3193590754767e-01 5.2833135637851e-01 6.3996627943621e-01
2.8759113359050e-01 5.3102996412016e-01 6.2860573802776e-01
9.4326876206306e-01 5.3372756952291e-01 6.1724578790633e-01
5.9897887355701e-01 5.3642427602975e-01 6.0588633411009e-01
2.5473143597766e-01 5.3912019513540e-01 5.9452728613155e-01
9.1053616070485e-01 5.4181544229676e-01 5.8316856104031e-01
5.6640233679354e-01 5.4451013305255e-01 5.7181008578468e-01
2.2233865790050e-01 5.4720437955778e-01 5.6045179861867e-01
8.7835305768054e-01 5.4989828769490e-01 5.4909364967496e-01
5.3445256818903e-01 5.5259195486773e-01 5.3773560076840e-01
1.9064321274733e-01 5.5528546852226e-01 5.2637762456283e-01
8.4692994045341e-01 5.5797890537886e-01 5.1501970326237e-01
5.0553199113801e-01 3.2832072247801e-01 5.0408221868651e-01
1.5603533205223e-01 3.3100917286058e-01 4.9272180840179e-01
8.0577544463945e-01 3.3369703698443e-01 4.8136154478319e-01
4.5482114111710e-01 3.3638524107711e-01 4.7000122878586e-01
1.0327766547590e-01 3.3907505996013e-01 4.5864032613098e-01
7.5127768342167e-01 3.4176798039861e-01 4.4727793436673e-01
3.9896872384408e-01 3.4446545497059e-01 4.3591287620824e-01
4.6502632533930e-02 3.4716860332543e-01 4.2454393106564e-01
6.9403088653068e-01 3.4987794993889e-01 4.1317015211029e-01
3.4170486045879e-01 3.5259328344276e-01 4.0179116461534e-01
9.8967609283803e-01 3.5531368693065e-01 3.9040733131647e-01
6.3809160035480e-01 3.5803773415493e-01 3.7901971291639e-01
2.8708333146014e-01 3.6076378879833e-01 3.6762983449859e-01
9.3675560768657e-01 3.6349030797858e-01 3.5623935138311e-01
5.8717611342049e-01 3.6621606030662e-01 3.4484974031574e-01
2.3837380873580e-01 3.6894021844072e-01 3.3346210636822e-01
8.9034343196053e-01 3.7166234192346e-01 3.2207712604554e-01
5.4305382923357e-01 3.7438229638625e-01 3.1069509205217e-01
1.9645706808075e-01 3.7710015476154e-01 2.9931600729953e-01
8.5049632180998e-01 3.7981610947691e-01 2.8793968592479e-01
5.0511172883663e-01 3.8253040712200e-01 2.7656583827506e-01
1.6024425517459e-01 3.8524330581874e-01 2.6519413254196e-01
8.1583795813544e-01 3.8795505063778e-01 2.5382423458281e-01
4.7184111944474e-01 3.9066586151066e-01 2.4245583083532e-01
1.2820664785859e-01 3.9337592895974e-01 2.3108863954157e-01
7.8489204596469e-01 3.9608541428090e-01 2.1972241456070e-01
4.4185913947016e-01 3.9879445198621e-01 2.0835694485705e-01
9.9073694017676e-02 4.0150315318124e-01 1.9699205171321e-01
7.5650499409552e-01 4.0421160913219e-01 1.8562758494400e-01
4.1412542593797e-01 4.0691989463961e-01 1.7426341885907e-01
7.1910086117903e-02 4.0962807104865e-01 1.6289944838080e-01
7.2983642553695e-01 4.1233618884479e-01 1.5153558551554e-01
3.8788393168533e-01 4.1504428984515e-01 1.4017175625336e-01
4.6033848362548e-02 4.1775240902402e-01 1.2880789790393e-01
7.0426893024079e-01 4.2046057602116e-01 1.1744395684151e-01
3.6257322892518e-01 4.2316881638116e-01 1.0607988661727e-01
2.0931907042484e-02 4.2587715256820e-01 9.4715646393574e-02
6.7933107708920e-01 4.2858560479354e-01 8.3351199657098e-02
3.3775766213017e-01 4.3129419168648e-01 7.1986513173003e-02
9.9619927586614e-01 4.3400293083286e-01 6.0621556148936e-02
6.5464412003637e-01 4.3671183919876e-01 4.9256299584385e-02
3.1308089756357e-01 4.3942093345173e-01 3.7890715787571e-02
9.7149874027302e-01 4.4213023018667e-01 2.6524778048215e-02
6.2988715042039e-01 4.4483974605894e-01 1.5158460460140e-02
2.8823595564061e-01 4.4754949782320e-01 3.7917378926395e-03
9.4492653022518e-01 4.5026395947315e-01 9.9241934195229e-01
6.0316850649414e-01 4.5297421831060e-01 9.8105175417764e-01
2.6134185497627e-01 4.5568476102400e-01 9.6968369425158e-01
9.1943752771066e-01 4.5839560301329e-01 9.5831514300647e-01
5.7744678685503e-01 4.6110675826468e-01 9.4694608434026e-01
2.3536124840573e-01 4.6381823861770e-01 9.3557650621632e-01
8.9317294329266e-01 4.6653005285932e-01 9.2420640184964e-01
5.5087439611721e-01 4.6924220563960e-01 9.1283577106597e-01
2.0845872112096e-01 4.7195469621318e-01 9.0146462180417e-01
8.6591973397648e-01 4.7466751702436e-01 8.9009297171239e-01
5.2325207666403e-01 4.7738065217224e-01 8.7872084976320e-01
1.8045135106754e-01 4.8009407581540e-01 8.6734829778544e-01
8.3751425508671e-01 4.8280775060227e-01 8.5597537178135e-01
4.9443871320421e-01 4.8552162624131e-01 8.4460214287268e-01
1.5122399184889e-01 4.8823563835091e-01 8.3322869770261e-01
8.0787078892140e-01 4.9094970774765e-01 8.2185513811866e-01
4.6438128690310e-01 4.9366374033757e-01 8.1048157998186e-01
1.2075916040929e-01 4.9637762776256e-01 7.9910815099256e-01
7.7700953208036e-01 4.9909124891960e-01 7.8773498749645e-01
4.3313887526149e-01 5.0180447241244e-01 7.7636223033012e-01
8.9154867582603e-02 5.0451715991805e-01 7.6499001987477e-01
7.4506620553837e-01 5.0722917036069e-01 7.5361849059460e-01
4.0088239545115e-01 5.0994036469789e-01 7.4224776542472e-01
5.6613539699477e-02 5.1265061104752e-01 7.3087795042643e-01
7.1227013794700e-01 5.1535978983549e-01 7.1950913013278e-01
3.6786292090204e-01 5.1806779862733e-01 7.0814136396311e-01
2.3402729058688e-02 5.2077455632488e-01 6.9677468399558e-01
6.7890044167870e-01 5.2348000645669e-01 6.8540909426682e-01
3.3436695315661e-01 5.2618411936034e-01 6.7404457163317e-01
9.8981318623650e-01 5.2888689313483e-01 6.6268106809722e-01
6.4525012560189e-01 5.3158835332454e-01 6.5131851438878e-01
3.0068885212088e-01 5.3428855137648e-01 6.3995682450166e-01
9.5614055802228e-01 5.3698756198554e-01 6.2859590083010e-01
6.1161652650590e-01 5.3968547950767e-01 6.1723563952431e-01
2.6712806522408e-01 5.4238241367506e-01 6.0587593569121e-01
9.2268639073192e-01 5.4507848488753e-01 5.9451668810415e-01
5.7830246910623e-01 5.4777381937625e-01 5.8315780314826e-01
2.3398682510949e-01 5.5046854453492e-01 5.7179919781230e-01
8.8974933733844e-01 5.5316278468705e-01 5.6044080163325e-01
5.4559903898543e-01 5.5585665750785e-01 5.4908255759641e-01
2.0154394297201e-01 5.5855027124998e-01 5.3772442208071e-01
8.5759090670015e-01 5.6124372284383e-01 5.2636636400533e-01
5.1374554637060e-01 5.6393709686496e-01 5.1500836337526e-01
1.7195230094813e-01 3.3427613601435e-01 5.0407561467430e-01
8.2320836906110e-01 3.3696537802020e-01 4.9271540129636e-01
4.7380237501344e-01 3.3965410523394e-01 4.8135529843463e-01
1.2378718003128e-01 3.4234303906034e-01 4.6999516612658e-01
7.7324349322229e-01 3.4503316976557e-01 4.5863461965376e-01
4.2227417926629e-01 3.4772567411705e-01 4.4727299500016e-01
7.0995626983310e-02 3.5042175764227e-01 4.3590939173951e-01
7.1952950770585e-01 3.5312244964435e-01 4.2454280871664e-01
3.6799789884632e-01 3.5582840198909e-01 4.1317235220092e-01
1.6522367669812e-02 3.5853974724566e-01 4.0179746185569e-01
6.6522478808655e-01 3.6125605763627e-01 3.9041808324084e-01
3.1422642433502e-01 3.6397641936927e-01 3.7903472563533e-01
9.6364305437487e-01 3.6669960371571e-01 3.6764838063770e-01
6.1357672800984e-01 3.6942428592407e-01 3.5626032815634e-01
2.6410713514547e-01 3.7214924964643e-01 3.4487189762213e-01
9.1528591312668e-01 3.7487352696834e-01 3.3348426049052e-01
5.6713562323907e-01 3.7759645536896e-01 3.2209830333812e-01
2.1965299014805e-01 3.8031766408081e-01 3.1071459001381e-01
8.7281467382572e-01 3.8303701850539e-01 2.9933339078099e-01
5.2658368744979e-01 3.8575455099159e-01 2.8795474619363e-01
1.8091514486053e-01 3.8847039693352e-01 2.7657853869525e-01
8.3576073554017e-01 3.9118474480495e-01 2.6520455585196e-01
4.9107185231906e-01 3.9389780149579e-01 2.5383253884522e-01
1.4680157085497e-01 3.9660977067369e-01 2.4246221594315e-01
8.0290576526639e-01 3.9932084079310e-01 2.3109332347696e-01
4.5934362791208e-01 4.0203117959190e-01 2.1972561757067e-01
1.1607780628364e-01 4.0474093261040e-01 2.0835887957321e-01
7.7307431047170e-01 4.0745022400120e-01 1.9699291749556e-01
4.3030229468362e-01 4.1015915850051e-01 1.8562756509580e-01
8.7733779062661e-02 4.1286782386880e-01 1.7426267971226e-01
7.4534335225210e-01 4.1557629340274e-01 1.6289813954305e-01
4.0310787811731e-01 4.1828462830789e-01 1.5153384079241e-01
6.1006219219125e-02 4.2099287983480e-01 1.4016969492104e-01
7.1901898299213e-01 4.2370109114707e-01 1.2880562612163e-01
3.7712829265567e-01 4.2640929892526e-01 1.1744156907039e-01
3.5317582677283e-02 4.2911753472771e-01 1.0607746696506e-01
6.9357141746254e-01 4.3182582613626e-01 9.4713269837619e-02
3.5187533144345e-01 4.3453419771551e-01 8.3348933120554e-02
1.0215688616955e-02 4.3724267181187e-01 7.1984416441708e-02
6.6857955968544e-01 4.3995126921503e-01 6.0619682624278e-02
3.2695461517422e-01 4.4266000969967e-01 4.9254696871406e-02
9.8532903319193e-01 4.4536891246096e-01 3.7889426119280e-02
6.4369142082412e-01 4.4807799645254e-01 2.6523838547541e-02
3.0203074849223e-01 4.5078728063137e-01 1.5157903240811e-02
9.6033629695817e-01 4.5349678410958e-01 3.7915900000898e-03
6.1698496066926e-01 4.5621099474722e-01 9.9241961199726e-01
2.7519335786882e-01 4.5892098148447e-01 9.8105246973869e-01
9.3333713269149e-01 4.6163124408287e-01 9.6968486524477e-01
5.9140653819098e-01 4.6434180170738e-01 9.5831677249727e-01
2.4939207489915e-01 4.6705267276455e-01 9.4694816775134e-01
9.0728452522777e-01 4.6976387428312e-01 9.3557903043455e-01
5.6507500652296e-01 4.7247542110623e-01 9.2420934426965e-01
2.2275504393648e-01 4.7518732487906e-01 9.1283909862236e-01
8.8031666381231e-01 4.7789959282193e-01 9.0146829006288e-01
5.3775250748015e-01 4.8061222629016e-01 8.9009692411045e-01
1.9505596413314e-01 4.8332521913807e-01 8.7872501710565e-01
8.5222131981367e-01 4.8603855592779e-01 8.6735259812354e-01
5.0924391745901e-01 4.8875221005221e-01 8.5597971080501e-01
1.6612032058580e-01 4.9146614187578e-01 8.4460641494608e-01
8.2284847075197e-01 4.9418029703323e-01 8.3323278765126e-01
4.7942782680371e-01 4.9689460506015e-01 8.2185892383395e-01
1.3585947258964e-01 4.9960897855460e-01 8.1048493584406e-01
7.9214617986260e-01 5.0232331307661e-01 7.9911095202756e-01
4.4829241499044e-01 5.0503748797618e-01 7.8773711408313e-01
1.0430428213751e-01 5.0775136829378e-01 7.7636357317709e-01
7.6018940164093e-01 5.1046480779959e-01 7.6499048490429e-01
4.1595672976431e-01 5.1317765313470e-01 7.5361800332614e-01
7.1616333737154e-02 5.1588974890065e-01 7.4224627445395e-01
7.2717914252298e-01 5.1860094342996e-01 7.3087542965443e-01
3.8265669763640e-01 5.2131109487915e-01 7.1950557951000e-01
3.8060928466705e-02 5.2402007723100e-01 7.0813680865752e-01
6.9340397258801e-01 5.2672778578452e-01 6.9676917205366e-01
3.4869805393538e-01 5.2943414174700e-01 6.8540269298403e-01
3.9554217728233e-03 5.3213909561599e-01 6.7403736296831e-01
6.5918834284625e-01 5.3484262913555e-01 6.6267314353766e-01
3.1440912989506e-01 5.3754475571873e-01 6.5130996969809e-01
9.6963018342758e-01 5.4024551933523e-01 6.3994775475825e-01
6.2486402144391e-01 5.4294499196371e-01 6.2858639610513e-01
2.8012327399844e-01 5.4564326979846e-01 6.1722578145768e-01
9.3542062575019e-01 5.4834046847739e-01 6.0586579511974e-01
5.9076869885466e-01 5.5103671765905e-01 5.9450632378586e-01
2.4617987907028e-01 5.5373215531429e-01 5.8314726152382e-01
9.0166609789143e-01 5.5642692210825e-01 5.7178851365870e-01
5.5723859110645e-01 5.5912115622407e-01 5.6042999940385e-01
2.1290765814134e-01 5.6181498892296e-01 5.4907165321130e-01
8.6868244639915e-01 5.6450854105052e-01 5.3771342493066e-01
5.2457078091486e-01 5.6720192059868e-01 5.2635527895935e-01
1.8057905310012e-01 5.6989522133018e-01 5.1499719262620e-01
8.3840365643751e-01 3.4023179041087e-01 5.0406861124529e-01
4.9031247496699e-01 3.4292169552535e-01 4.9270858988642e-01
1.4164207577066e-01 3.4561115013897e-01 4.8134864894577e-01
7.9243382355758e-01 3.4830072634500e-01 4.6998868801662e-01
4.4275066637401e-01 3.5099120580163e-01 4.5862842461110e-01
9.2673478297253e-02 3.5368352905095e-01 4.4726736286195e-01
7.4229509674144e-01 3.5637869268724e-01 4.3590481066827e-01
3.9171404577175e-01 3.5907760819622e-01 4.2453995898439e-01
4.1030061283791e-02 3.6178095176246e-01 4.1317201711895e-01
6.9034241391689e-01 3.6448904065538e-01 4.0180037605796e-01
3.3975027334505e-01 3.6720176713602e-01 3.9042475713847e-01
9.8935307250165e-01 3.6991860776179e-01 3.7904530208073e-01
6.3924883029689e-01 3.7263870779559e-01 3.6766257451953e-01
2.8952967647971e-01 3.7536102065110e-01 3.5627747003598e-01
9.4027557452641e-01 3.7808446693238e-01 3.4489106196678e-01
5.9154835843039e-01 3.8080807403832e-01 3.3350443000044e-01
2.4338807481154e-01 3.8353106822069e-01 3.2211851787453e-01
8.9581252338480e-01 3.8625291057054e-01 3.1073404769370e-01
5.4881963128485e-01 3.8897328627128e-01 2.9935149400296e-01
2.0239154809659e-01 3.9169206544310e-01 2.8797110320565e-01
8.5649925731435e-01 3.9440925365999e-01 2.7659293787319e-01
5.1110682592755e-01 3.9712494480011e-01 2.6521692826164e-01
1.6617484383034e-01 3.9983928261728e-01 2.5384291973206e-01
8.2166294576099e-01 4.0255243270141e-01 2.4247071086310e-01
4.7753150643981e-01 4.0526456384508e-01 2.3110008121814e-01
1.3374267924345e-01 4.0797583679745e-01 2.1973080993084e-01
7.9026095633003e-01 4.1068639829560e-01 2.0836268707957e-01
4.4705340224599e-01 4.1339637859558e-01 1.9699551984623e-01
1.0408967760592e-01 4.1610589116724e-01 1.8562913514184e-01
7.6134193620518e-01 4.1881503362340e-01 1.7426337998032e-01
4.1878465217356e-01 4.2152388927324e-01 1.6289812051359e-01
7.6394413996992e-02 4.2423252892043e-01 1.5153324034627e-01
7.3414970839155e-01 4.2694101268296e-01 1.4016863852986e-01
3.9203070770658e-01 4.2964939171365e-01 1.2880422748337e-01
5.0019068476537e-02 4.3235770976319e-01 1.1743993098447e-01
7.0809774491938e-01 4.3506600456559e-01 1.0607568230834e-01
3.6625081885051e-01 4.3777430904680e-01 9.4711422549752e-02
2.4463346128704e-02 4.4048265236789e-01 8.3347099139015e-02
6.8272121902523e-01 4.4319106081900e-01 7.1982664548690e-02
3.4101104357974e-01 4.4589955858038e-01 6.0618075181461e-02
9.9932003092928e-01 4.4860816836595e-01 4.9253290427396e-02
6.5763590167559e-01 4.5131691196153e-01 3.7888271879690e-02
3.1594680253352e-01 4.5402581066713e-01 2.6522982700458e-02
9.7424123474536e-01 4.5673488564841e-01 1.5157387131715e-02
6.3250799403020e-01 4.5944415819896e-01 3.7914501508642e-03
2.8912001655419e-01 4.6215812851947e-01 9.9241986829053e-01
9.4730004787836e-01 4.6486784938343e-01 9.8105315846191e-01
6.0541992683209e-01 4.6757783194767e-01 9.6968600693465e-01
2.6346926735728e-01 4.7028809867174e-01 9.5831838130056e-01
9.2143786561608e-01 4.7299867187233e-01 9.4695025062048e-01
5.7931572289074e-01 4.7570957324567e-01 9.3558158617460e-01
2.3709308755134e-01 4.7842082319650e-01 9.2421236246091e-01
8.9476051806408e-01 4.8113243994810e-01 9.1284255845819e-01
5.5230896882350e-01 4.8384443841018e-01 9.0147215916069e-01
2.0972990007724e-01 4.8655682878853e-01 8.9010115737696e-01
8.6701541226810e-01 4.8926961493287e-01 8.7872955576245e-01
5.2415840364814e-01 4.9198279243902e-01 8.6735736902448e-01
1.8115274796135e-01 4.9469634654980e-01 8.5598462619884e-01
8.3799348635491e-01 4.9741024993580e-01 8.4461137285089e-01
4.9467702459901e-01 5.0012446048091e-01 8.3323767300464e-01
1.5120132347268e-01 5.0283891924543e-01 8.2186361055787e-01
8.0756606731307e-01 5.0555354882476e-01 8.1048928991003e-01
4.6377279390953e-01 5.0826825235505e-01 7.9911483552310e-01
1.1982496893424e-01 5.1098291342833e-01 7.8774039016872e-01
7.7572799063708e-01 5.1369739715720e-01 7.7636611169349e-01
4.3148911597054e-01 5.1641155256613e-01 7.6499216826353e-01
8.7117307451467e-02 5.1912521638110e-01 7.5361873221773e-01
7.4262300996627e-01 5.2183821815109e-01 7.4224597284759e-01
3.9801787673208e-01 5.2455038648084e-01 7.3087404859791e-01
5.3314471662537e-02 5.2726155600905e-01 7.1950309931342e-01
7.0852597946412e-01 5.2997157465562e-01 7.0813323921210e-01
3.6366595357903e-01 5.3268031060493e-01 6.9676455123099e-01
1.8748125378022e-02 5.3538765849855e-01 6.8539708326826e-01
6.7378628681903e-01 5.3809354437530e-01 6.7403084665550e-01
3.2879424518751e-01 5.4079792900591e-01 6.6266581696940e-01
9.8378583512947e-01 5.4350080940386e-01 6.5130193706467e-01
6.3877496254496e-01 5.4620221843625e-01 6.3993912201009e-01
2.9377564903918e-01 5.4890222259594e-01 6.2857726545540e-01
9.4880204562174e-01 5.5160091812229e-01 6.1721624685872e-01
6.0386839020247e-01 5.5429842576752e-01 6.0585593896581e-01
2.5898889406174e-01 5.5699488459584e-01 5.9449621495173e-01
9.1417755586524e-01 5.5969044526509e-01 5.8313695470914e-01
5.6944791537932e-01 5.6238526326768e-01 5.7177804988659e-01
2.2481277021682e-01 5.6507949259103e-01 5.6041940743203e-01
8.8028388564318e-01 5.6777328019498e-01 5.4906095156271e-01
5.3587172866922e-01 5.7046676160100e-01 5.3770262424163e-01
1.9158525356939e-01 5.7316005775929e-01 5.2634438437066e-01
8.4743175791857e-01 5.7585327322471e-01 5.1498620599817e-01
5.0488051615506e-01 3.4618767171288e-01 5.0406122142868e-01
1.5736030679136e-01 3.4887813582505e-01 4.9270138659006e-01
8.0932965019118e-01 3.5156820660892e-01 4.8134160714426e-01
4.6082144317266e-01 3.5425834501934e-01 4.6998180977593e-01
1.1188552632400e-01 3.5694917712341e-01 4.5862178338407e-01
7.6258629803901e-01 3.5964146236471e-01 4.4726115234900e-01
4.1299856288698e-01 3.6233602540904e-01 4.3589938072095e-01
6.3202811718380e-02 3.6503365820485e-01 4.2453581823267e-01
7.1328138214904e-01 3.6773500915763e-01 4.1316978791804e-01
3.6331646229231e-01 3.7044048206467e-01 4.0180070145066e-01
1.3389879923051e-02 3.7315016680023e-01 3.9042817722432e-01
6.6358362301161e-01 3.7586381758383e-01 3.7905213172932e-01
3.1397962870977e-01 3.7858088485324e-01 3.6767281893395e-01
9.6465778673304e-01 3.8130059492118e-01 3.5629080506669e-01
6.1569209933482e-01 3.8402206002749e-01 3.4490688444872e-01
2.6714596075946e-01 3.8674439396170e-01 3.3352195941316e-01
9.1906800483884e-01 3.8946680880222e-01 3.2213691625601e-01
5.7148971816010e-01 3.9218867675218e-01 3.1075252574094e-01
2.2442527957723e-01 3.9490955340003e-01 2.9936938384033e-01
8.7787332991603e-01 3.9762916925831e-01 2.8798789351564e-01
5.3181993828501e-01 4.0034740159787e-01 2.7660827792271e-01
1.8624197599976e-01 4.0306423843257e-01 2.6523061172548e-01
8.4111030097275e-01 4.0577974327308e-01 2.5385485868930e-01
4.9639242036092e-01 4.0849402537368e-01 2.4248090754314e-01
1.5205452257223e-01 4.1120721708410e-01 2.3110860197957e-01
8.0806291136474e-01 4.1391945801170e-01 2.1973776352243e-01
4.6438494202480e-01 4.1663088481702e-01 2.0836820766666e-01
1.2098957676172e-01 4.1934162524951e-01 1.9699975444340e-01
7.7784766686488e-01 4.2205179515597e-01 1.8563223473774e-01
4.3493204904989e-01 4.2476149744829e-01 1.7426549356758e-01
9.2217521891787e-02 4.2747082228352e-01 1.6289939130332e-01
7.4968074944451e-01 4.3017984793692e-01 1.5153380356637e-01
4.0730012433754e-01 4.3288864202439e-01 1.4016862033436e-01
6.5055611695947e-02 4.3559726285691e-01 1.2880374461447e-01
7.2292858748695e-01 4.3830576079729e-01 1.1743909092345e-01
3.8090167960094e-01 4.4101417954635e-01 1.0607458372489e-01
3.8958616456360e-02 4.4372255732294e-01 9.4710155914893e-02
6.9708408564802e-01 4.4643092792385e-01 8.3345747406951e-02
3.5526360374510e-01 4.4913932166321e-01 7.1981303842057e-02
1.3483397509554e-02 4.5184776619691e-01 6.0616775434414e-02
6.7173029635695e-01 4.5455628724089e-01 4.9252115954667e-02
3.2999163569082e-01 4.5726490919205e-01 3.7887281848629e-02
9.8825517072333e-01 4.5997365565928e-01 2.6522231488633e-02
6.4650900049656e-01 4.6268254990984e-01 1.5156924555825e-02
3.0474150200441e-01 4.6539161523322e-01 3.7913215544269e-03
9.6132217969678e-01 4.6810536263401e-01 9.9242010427934e-01
6.1947905383429e-01 4.7081483089420e-01 9.8105380360236e-01
2.7758068636459e-01 4.7352454085399e-01 9.6968709209260e-01
9.3561612125369e-01 4.7623451774005e-01 9.5831993158336e-01
5.9357451927794e-01 4.7894478718113e-01 9.4695228460195e-01
2.5144516781215e-01 4.8165537489197e-01 9.3558411493263e-01
9.0921750920598e-01 4.8436630616933e-01 9.2421538843735e-01
5.6688119036610e-01 4.8707760516757e-01 9.1284607416380e-01
2.2442613627307e-01 4.8978929391952e-01 9.0147614576633e-01
8.8184265000837e-01 4.9250139107080e-01 8.9010558325527e-01
5.3912154132003e-01 4.9521391030247e-01 8.7873437507340e-01
1.9625428466993e-01 4.9792685843158e-01 8.6736252047065e-01
8.5323320593630e-01 5.0064023320257e-01 8.5599003211058e-01
5.1005169438644e-01 5.0335402081729e-01 8.4461693879157e-01
1.6670443317117e-01 5.0606819329798e-01 8.3324328810628e-01
8.2318763757604e-01 5.0878270583454e-01 8.2186914879704e-01
4.7949928598240e-01 5.1149749433015e-01 8.1049461250395e-01
1.3563932462203e-01 5.1421247342031e-01 7.9911979455788e-01
7.9160982470371e-01 5.1692753528537e-01 7.8774483346099e-01
4.4741507045076e-01 5.1964254959239e-01 7.7636988873862e-01
1.0306156000962e-01 5.2235736487092e-01 7.6499513695345e-01
7.5855790858973e-01 5.2507181154128e-01 7.5362076584552e-01
4.1391465423254e-01 5.2778570666977e-01 7.4224696678920e-01
6.9143979848815e-02 5.3049886033821e-01 7.3087392600749e-01
7.2425937816104e-01 5.3321108331088e-01 7.1950181521061e-01
3.7927529597323e-01 5.3592219549628e-01 7.0813078248260e-01
3.4206798068529e-02 5.3863203456920e-01 6.9676094428593e-01
6.8906928745210e-01 5.4134046406534e-01 6.8539237937653e-01
3.4387830777624e-01 5.4404738029369e-01 6.7402512522906e-01
9.9864943755952e-01 5.4675271751951e-01 6.6265917730079e-01
6.5339826750938e-01 5.4945645103090e-01 6.5129449115831e-01
3.0814043564810e-01 5.5215859788671e-01 6.3993098720139e-01
9.6289168341819e-01 5.5485921533074e-01 6.2856855747748e-01
6.1766789182525e-01 5.5755839703344e-01 6.1720707391122e-01
2.7248506078164e-01 5.6025626747883e-01 6.0584639718352e-01
9.2735920632449e-01 5.6295297494653e-01 5.9448638548707e-01
5.8230616697872e-01 5.6564868363785e-01 5.8312690245385e-01
2.3734132882479e-01 5.6834356555010e-01 5.7176782368744e-01
8.9247929504327e-01 5.7103779270078e-01 5.6040904152165e-01
5.4773353657679e-01 5.7373153023911e-01 5.4905046784426e-01
2.0311606412135e-01 5.7642493085865e-01 5.3769203504006e-01
8.5863715777050e-01 5.7911813076028e-01 5.2633369529082e-01
5.1430518084337e-01 5.8181124723604e-01 5.1497541859747e-01
1.7137861320037e-01 3.5214376739782e-01 5.0405345876894e-01
8.2436153603707e-01 3.5483470518387e-01 4.9269380446781e-01
4.7689180396118e-01 3.5752530019430e-01 4.8133418476768e-01
1.2899595590310e-01 3.6021592912947e-01 4.6997454526988e-01
7.8071398983656e-01 3.6290710014521e-01 4.5861472568795e-01
4.3209778514169e-01 3.6559943269849e-01 4.4725443762159e-01
8.3208185272792e-02 3.6829361166687e-01 4.3589326245074e-01
7.3411146632386e-01 3.7099031875662e-01 4.2453067762831e-01
3.8487616643544e-01 3.7369015098464e-01 4.1316611335066e-01
3.5571070858091e-02 3.7639354061326e-01 4.0179903308181e-01
6.8626457867164e-01 3.7910069180838e-01 3.9042902354386e-01
3.3702498994123e-01 3.8181154659933e-01 3.7905587514361e-01
9.8792079275870e-01 3.8452578745266e-01 3.6767963392271e-01
6.3902002962087e-01 3.8724287687639e-01 3.5630061148268e-01
2.9038828193608e-01 3.8996212688011e-01 3.4491934921782e-01
9.4208549867401e-01 3.9268278448733e-01 3.3353654508188e-01
5.9416246399612e-01 3.9540411619780e-01 3.2215296084979e-01
2.4665787883725e-01 3.9812547600951e-01 3.1076933143477e-01
8.9959678206177e-01 4.0084634777721e-01 2.9938629405749e-01
5.5299054607301e-01 4.0356636054929e-01 2.8800434637014e-01
2.0683821603005e-01 4.0628528188455e-01 2.7662383336119e-01
8.6112869849565e-01 4.0900299721081e-01 2.6524495649853e-01
5.1584327046145e-01 4.1171948318138e-01 2.5386779623294e-01
1.7095799556893e-01 4.1443478100156e-01 2.4249233979402e-01
8.2644580113187e-01 4.1714897322565e-01 2.3111850854472e-01
4.8227811771014e-01 4.1986216544813e-01 2.1974618166886e-01
1.3842608401632e-01 4.2257447294365e-01 2.0837521493630e-01
7.9486137399279e-01 4.2528601159742e-01 1.9700545454260e-01
4.5155672247782e-01 4.2799689221637e-01 1.8563674666226e-01
1.0848622498133e-01 4.3070721732612e-01 1.7426894358074e-01
7.6562547649548e-01 4.3341707969428e-01 1.6290190726155e-01
4.2295160072685e-01 4.3612656199026e-01 1.5153551108526e-01
8.0443208198989e-02 4.3883573714982e-01 1.4016964034543e-01
7.3808031080810e-01 4.4154466914404e-01 1.2880419194044e-01
3.9584421194637e-01 4.4425341395145e-01 1.1743907357708e-01
5.3717385028300e-02 4.4696202060459e-01 1.0607420270521e-01
7.1168334876535e-01 4.4967053223231e-01 9.4709505330492e-02
3.6972654441770e-01 4.5237898705296e-01 8.3344914800802e-02
2.7832218147263e-02 4.5508741929539e-01 7.1980370625572e-02
6.8598631021783e-01 4.5779586003804e-01 6.0615817363546e-02
3.4417535192252e-01 4.6050433796456e-01 4.9251203598649e-02
2.3863706113738e-03 4.6321288003822e-01 3.7886481014460e-02
6.6060680293584e-01 4.6592151209869e-01 2.6521603572741e-02
3.1882441634961e-01 4.6863025938475e-01 1.5156526799708e-02
9.7702723895808e-01 4.7133914698448e-01 3.7912071841083e-03
6.3358187380635e-01 4.7405269516325e-01 9.9242031378490e-01
2.9172076766216e-01 4.7676193087826e-01 9.8105438909567e-01
9.4980973949802e-01 4.7947138292472e-01 9.6968809415430e-01
6.0783733957471e-01 4.8218107873590e-01 9.5832138581275e-01
2.6579217165525e-01 4.8489104659599e-01 9.4695422090999e-01
9.2366288876125e-01 4.8760131549529e-01 9.3558655662095e-01
5.8143820619015e-01 4.9031191481275e-01 9.2421835105782e-01
2.3910693483474e-01 4.9302287378922e-01 9.1284956416343e-01
8.9665803824661e-01 4.9573422074948e-01 9.0148015893740e-01
5.5408071711270e-01 4.9844598202844e-01 8.9011010303259e-01
2.1136452473960e-01 5.0115818055748e-01 8.7873937074932e-01
8.6849951659257e-01 5.0387083407419e-01 8.6736794543520e-01
5.2547643571199e-01 5.0658395293465e-01 8.5599582226635e-01
1.8228693371664e-01 5.0929753753537e-01 8.4462301133877e-01
8.3892382392593e-01 5.1201157539501e-01 8.3324954093542e-01
4.9538135883127e-01 5.1472603800508e-01 8.2187546075629e-01
1.5165551888267e-01 5.1744087763301e-01 8.1050084481203e-01
8.0774429382519e-01 5.2015602434500e-01 7.9912579359779e-01
4.6364793254277e-01 5.2287138359754e-01 7.8775043510205e-01
1.1936913387853e-01 5.2558683480935e-01 7.7637492418871e-01
7.7491315077698e-01 5.2830223134643e-01 7.6499943994519e-01
4.3028778476475e-01 5.3101740231056e-01 7.5362418073339e-01
8.5503257970614e-02 5.3373215640145e-01 7.4224935691680e-01
7.4057196500466e-01 5.3644628792414e-01 7.3087518154530e-01
3.9550812486210e-01 5.3915958475774e-01 7.1950185961219e-01
5.0327369934932e-02 5.4187183782923e-01 7.0812957678972e-01
7.0504632092885e-01 5.4458285139897e-01 6.9675848873185e-01
3.5968219937873e-01 5.4729245331095e-01 6.8538871205656e-01
1.4252521844951e-02 5.5000050432140e-01 6.7402031797034e-01
6.6877490272533e-01 5.5270690569701e-01 6.6265332920171e-01
3.2326696942452e-01 5.5541160444610e-01 6.5128772052470e-01
9.7774636946541e-01 5.5811459577745e-01 6.3992342274725e-01
6.3223082934556e-01 5.6081592263309e-01 6.2856032967447e-01
2.8673821385590e-01 5.6351567238742e-01 6.1719830727607e-01
9.4128653457162e-01 5.6621397102898e-01 6.0583720411279e-01
5.9589386729963e-01 5.6891097533471e-01 5.9447686201568e-01
2.5057815803543e-01 5.7160686369992e-01 5.8311712606163e-01
9.0535692112663e-01 5.7430182638539e-01 5.7175785303848e-01
5.6024685652175e-01 5.7699605596845e-01 5.6039891782479e-01
2.1526343011879e-01 5.7968973872471e-01 5.4904021739186e-01
8.7042046882797e-01 5.8238304752169e-01 5.3768167242878e-01
5.2572981905891e-01 5.8507613659645e-01 5.2632322684980e-01
1.8120110562029e-01 5.8776913835034e-01 5.1496484563130e-01
8.3789461115264e-01 3.5810006595439e-01 5.0404533725630e-01
4.9132367516603e-01 3.6079140679163e-01 4.9268585712271e-01
1.4434919653983e-01 3.6348244943808e-01 4.8132639433829e-01
7.9699285847925e-01 3.6617350567933e-01 4.6996690792971e-01
4.4928715094169e-01 3.6886499300183e-01 4.5860727440719e-01
1.0127431544380e-01 3.7155742165711e-01 4.4724726950933e-01
7.5300427516682e-01 3.7425136332729e-01 4.3588656327997e-01
4.0453199383304e-01 3.7694740259614e-01 4.2452473463577e-01
5.5914915270774e-02 3.7964607690145e-01 4.1316130798001e-01
7.0721109535382e-01 3.8234781411078e-01 4.0179580912266e-01
3.5847833359739e-01 3.8505287820629e-01 3.9042783235482e-01
9.7741663275939e-03 3.8776133259449e-01 3.7905710661769e-01
6.6115619740110e-01 3.9047302777346e-01 3.6768354741546e-01
3.1268209109081e-01 3.9318761609653e-01 3.5630728296684e-01
9.6440870312143e-01 3.9590459164216e-01 3.4492864802917e-01
6.1639021178665e-01 3.9862334839637e-01 3.3354814602035e-01
2.6867555645086e-01 4.0134324625941e-01 3.2216638759987e-01
9.2130579514858e-01 4.0406367318047e-01 3.1078401917665e-01
5.7431203323090e-01 4.0678409367065e-01 2.9940165589678e-01
2.2771436656763e-01 4.0950407834313e-01 2.8801983036862e-01
8.8152195524069e-01 4.1222331420204e-01 2.7663896249301e-01
5.3573405098288e-01 4.1494159933553e-01 2.6525934986107e-01
1.9034163875404e-01 4.1765882753449e-01 2.5388117418596e-01
8.4532933022375e-01 4.2037496828855e-01 2.4250451772190e-01
5.0067721857204e-01 4.2309004636622e-01 2.3112938406861e-01
1.5636251111874e-01 4.2580412358612e-01 2.1975571921411e-01
8.1236085586087e-01 4.2851728398128e-01 2.0838343031431e-01
4.6864735016406e-01 4.3122962257559e-01 1.9701240107305e-01
1.2519726217908e-01 4.3394123742911e-01 1.8564250351593e-01
7.8198651426146e-01 4.3665222436738e-01 1.7427360647764e-01
4.3899198107373e-01 4.3936267376779e-01 1.6290558135377e-01
9.6191650224630e-02 4.4207266883964e-01 1.5153830571490e-01
7.5356468510785e-01 4.4478228493765e-01 1.4017166533030e-01
4.1109142088942e-01 4.4749158955685e-01 1.2880555505752e-01
6.8753316760556e-02 4.5020064275227e-01 1.1743987895623e-01
7.2653288113820e-01 4.5290949780341e-01 1.0607454989534e-01
3.8441358150441e-01 4.5561820200233e-01 9.4709488848366e-02
4.2379746856841e-02 4.5832679748614e-01 8.3344624014398e-02
7.0041646805841e-01 4.6103532206511e-01 7.1979889858447e-02
3.5850949948984e-01 4.6374381001764e-01 6.0615226133611e-02
1.6645164124563e-02 4.6645229283652e-01 4.9250576925569e-02
6.7481026330202e-01 4.6916079991959e-01 3.7885889745090e-02
3.3299199195424e-01 4.7186935920207e-01 2.6521114684884e-02
9.9117785975541e-01 4.7457799773051e-01 1.5156203651832e-02
6.4935561855562e-01 4.7728674217817e-01 3.7911096832913e-03
3.0588950301695e-01 4.8000012052362e-01 9.9242049119633e-01
9.6401554125875e-01 4.8270915006658e-01 9.8105490005716e-01
6.2209734691337e-01 4.8541836582870e-01 9.6968898803742e-01
2.8012305779721e-01 4.8812779687991e-01 9.5832270790017e-01
9.3808080589832e-01 4.9083747347584e-01 9.4695601178850e-01
5.9595869892510e-01 4.9354742708338e-01 9.3558885134850e-01
2.5374481705420e-01 4.9625769025703e-01 9.2422117808469e-01
9.1142722809750e-01 4.9896829632830e-01 9.1285294399396e-01
5.6899402494669e-01 5.0167927886258e-01 9.0148410253005e-01
2.2643338973289e-01 5.0439067083056e-01 8.9011460995358e-01
8.8373368954482e-01 5.0710250343588e-01 8.7874442712096e-01
5.4088360861744e-01 5.0981480453964e-01 8.6737352175545e-01
1.9787232140613e-01 5.1252759662893e-01 8.5600187122252e-01
8.5468970961902e-01 5.1524089429347e-01 8.4462946579340e-01
5.1132662377944e-01 5.1795470120792e-01 8.3325631232346e-01
1.6777518595228e-01 5.2066900666979e-01 8.2188243819147e-01
8.2402912475278e-01 5.2338378181869e-01 8.1050789524395e-01
4.8008412681733e-01 5.2609897575980e-01 7.9913276337220e-01
1.3593818118653e-01 5.2881451192682e-01 7.8775715323477e-01
7.9159188578686e-01 5.3153028513146e-01 7.7638120754989e-01
4.4704868029151e-01 5.3424615983309e-01 7.6500510035620e-01
1.0231496937648e-01 5.3696197019207e-01 7.5362903371180e-01
7.5740010684708e-01 5.3967752241085e-01 7.4225323149883e-01
4.1231622529547e-01 5.4239259969790e-01 7.3087793032728e-01
6.7077916882808e-02 5.4510696991246e-01 7.1950336795451e-01
7.2170179507522e-01 5.4782039559585e-01 7.0812977008486e-01
3.7620598918497e-01 5.5053264573033e-01 6.9675733678812e-01
3.0609637200476e-02 5.5324350826519e-01 6.8538622998158e-01
6.8493244284069e-01 5.5595280227748e-01 6.7401656339341e-01
3.3919434850190e-01 5.5866038862809e-01 6.6264839616105e-01
9.9341534931428e-01 5.6136617812411e-01 6.5128173076922e-01
6.4761544089980e-01 5.6407013646659e-01 6.3991651549034e-01
3.0181466237541e-01 5.6677228559162e-01 6.2855265095409e-01
9.5603317404381e-01 5.6947270135442e-01 6.1719000002409e-01
6.1029130122723e-01 5.7217150782657e-01 6.0582839984583e-01
2.6460948367419e-01 5.7486886876083e-01 5.9446767477318e-01
9.1900809205858e-01 5.7756497701139e-01 5.8310764888244e-01
5.7350710412601e-01 5.8026004286343e-01 5.7174815693379e-01
2.2812566565648e-01 5.8295428229851e-01 5.6038905291659e-01
8.8288158785254e-01 5.8564790617881e-01 5.4903021568104e-01
5.3779084701911e-01 5.8834111116837e-01 5.3767155155630e-01
1.9286715194115e-01 5.9103407294396e-01 5.2631299422236e-01
8.4812163072617e-01 5.9372694192922e-01 5.1495450237966e-01
5.0442586579310e-01 3.6405655659974e-01 5.0403687126063e-01
1.5825264392318e-01 3.6674824153619e-01 4.9267755861392e-01
8.1171804595603e-01 3.6943966756992e-01 4.8131824903992e-01
4.6483998965669e-01 3.7213109575118e-01 4.6995891122235e-01
1.1764515556833e-01 3.7482287291136e-01 4.5859944872748e-01
7.7016827462810e-01 3.7751542330379e-01 4.4723968507813e-01
4.2245064535807e-01 3.8020922692068e-01 4.3587935781301e-01
7.4538153918907e-02 3.8290478480232e-01 4.2451812607891e-01
7.2647923269662e-01 3.8560257462501e-01 4.1315559422348e-01
3.7832321170327e-01 3.8830300239489e-01 4.0179135054489e-01
3.0119362054776e-02 3.9100635741225e-01 3.9042501668178e-01
6.8191665345987e-01 3.9371277752714e-01 3.7905630010004e-01
3.3376395919487e-01 3.9642223029167e-01 3.6768504053332e-01
9.8571026090600e-01 3.9913451330441e-01 3.5631124149739e-01
6.3780440558539e-01 4.0184927414275e-01 3.4493508025382e-01
2.9009414820714e-01 4.0456604707331e-01 3.3355689362511e-01
9.4262450197740e-01 4.0728430073615e-01 3.2217714209296e-01
5.9543569548791e-01 4.1000348902134e-01 3.1079635928764e-01
2.4856119108688e-01 4.1272309715427e-01 2.9941509676701e-01
9.0202620017526e-01 4.1544267676711e-01 2.8803387394845e-01
5.5584696875717e-01 4.1816186683864e-01 2.7665314041241e-01
2.1003088647092e-01 4.2088040070087e-01 2.6527325378374e-01
8.6457728465976e-01 4.2359810179858e-01 2.5389447256412e-01
5.1947868581537e-01 4.2631487205428e-01 2.4251696072010e-01
1.7472225166469e-01 4.2903067664557e-01 2.3114079982867e-01
8.3029122241605e-01 4.3174552820353e-01 2.1976600482636e-01
4.8616620972125e-01 4.3445947238565e-01 2.0839254033072e-01
1.4232627369344e-01 4.3717257581517e-01 1.9702033559790e-01
7.9874976607031e-01 4.3988491666894e-01 1.8564929713333e-01
4.5541495428286e-01 4.4259657776040e-01 1.7427931865982e-01
1.1230045769475e-01 4.4530764174846e-01 1.6291028856950e-01
7.6938553250638e-01 4.4801818803760e-01 1.5154209519830e-01
4.2665024037562e-01 4.5072829095408e-01 1.4017463033441e-01
8.4075531125583e-02 4.5343801884293e-01 1.2880779136215e-01
7.4164326418610e-01 4.5614743380250e-01 1.1744148239289e-01
3.9933618786599e-01 4.5885659184113e-01 1.0607561467095e-01
5.7137890741446e-02 4.6156554329917e-01 9.4710106479355e-02
7.1503273554300e-01 4.6427433342558e-01 8.3344882714786e-02
3.7300578289995e-01 4.6698300303389e-01 7.1979874255603e-02
3.1042710038894e-02 4.6969158918761e-01 6.0615017211418e-02
6.8912972788876e-01 4.7240012588402e-01 4.9250252115867e-02
3.4725349888567e-01 4.7510864471718e-01 3.7885523104958e-02
5.4010569830406e-03 4.7781717550966e-01 2.6520777109872e-02
6.6355973086671e-01 4.8052574690698e-01 1.5155963084223e-02
3.2171707108641e-01 4.8323438693172e-01 3.7910312805643e-03
9.7823547887547e-01 4.8594762972323e-01 9.9242063165047e-01
6.3635372044640e-01 4.8865648509785e-01 9.8105532325585e-01
2.9443374391481e-01 4.9136549258833e-01 9.6968975093746e-01
9.5246335890407e-01 4.9407468234352e-01 9.5832386434270e-01
6.1043031523589e-01 4.9678408590936e-01 9.4695761201563e-01
2.6832227082655e-01 4.9949373641389e-01 9.3559094129685e-01
9.2612677215083e-01 5.0220366863207e-01 9.2422379842372e-01
5.8383125043160e-01 5.0491391889500e-01 9.1285612887938e-01
2.4142303751471e-01 5.0762452479816e-01 9.0148787804839e-01
8.9888940624268e-01 5.1033552465253e-01 8.9011899224587e-01
5.5621764097066e-01 5.1304695661195e-01 8.7874942019310e-01
2.1339514453823e-01 5.1575885740168e-01 8.6737911501345e-01
8.7040958831431e-01 5.1847126056855e-01 8.5600803681377e-01
5.2724911157153e-01 5.2118419417799e-01 8.4463615589359e-01
1.8390257503296e-01 5.2389767790098e-01 8.3326345658070e-01
8.4035987051457e-01 5.2661171947253e-01 8.2188994162100e-01
4.9661228372134e-01 5.2932631056780e-01 8.1051563694889e-01
1.5265290016615e-01 5.3204142223902e-01 7.9914059653030e-01
8.0847703498361e-01 5.3475700018429e-01 7.8776490681364e-01
4.6408265690037e-01 5.3747296027124e-01 7.7638869016441e-01
1.1947076652428e-01 5.4018918489309e-01 7.6501210653344e-01
7.7464568214122e-01 5.4290552085571e-01 7.5363535256637e-01
4.2961518570936e-01 5.4562177953694e-01 7.4225865745761e-01
8.4390490792443e-02 5.4833773997515e-01 7.3088227512558e-01
7.3898601419025e-01 5.5105315530344e-01 7.1950647274215e-01
3.9341896233308e-01 5.5376776255369e-01 7.0813151623553e-01
4.7708776667193e-02 5.5648129536559e-01 6.9675765399211e-01
7.0187651085665e-01 5.5919349864488e-01 6.8538510046211e-01
3.5594422771393e-01 5.6190414383496e-01 6.7401402159011e-01
9.9344991601381e-03 5.6461304328666e-01 6.6264452386900e-01
6.6387006746095e-01 5.6732006226820e-01 6.5127664836904e-01
3.1777368593010e-01 5.7002512742493e-01 6.3991037042183e-01
9.7166811229417e-01 5.7272823090518e-01 6.2854560488597e-01
6.2557618954651e-01 5.7542942983360e-01 6.1718221622595e-01
2.7952092717313e-01 5.7812884127236e-01 6.0582003210014e-01
9.3352549562704e-01 5.8082663322822e-01 5.9445885882896e-01
5.8761306886153e-01 5.8352301261791e-01 5.8309849701695e-01
2.4180648808671e-01 5.8621821137444e-01 5.7173875572616e-01
8.9612776490087e-01 5.8891247202866e-01 5.6037946391652e-01
5.5059748199213e-01 5.9160603409629e-01 5.4902047833602e-01
2.0523417465078e-01 5.9429912242342e-01 5.3766168758354e-01
8.6005378093668e-01 5.9699193831005e-01 5.2630301258832e-01
5.1506923325213e-01 5.9968465380533e-01 5.1494440416141e-01
1.7097025652775e-01 3.7001322908760e-01 5.0402807546812e-01
8.2515316624278e-01 3.7270520855544e-01 4.9266892337954e-01
4.7901124907349e-01 3.7539696374301e-01 4.8130976260531e-01
1.3255946253093e-01 3.7808871554015e-01 4.6995056885064e-01
7.8581993330390e-01 3.8078075504914e-01 4.5859126584740e-01
4.3882147292888e-01 3.8347343817020e-01 4.4723171315293e-01
9.1598505687550e-02 3.8616717041875e-01 4.3587170010164e-01
7.4418957834895e-01 3.8886238187483e-01 4.2451094926423e-01
3.9663574370537e-01 3.9155949415300e-01 4.1314913156084e-01
4.8979149012919e-02 3.9425888311817e-01 4.0178589309146e-01
7.0126208745475e-01 3.9696084223499e-01 3.9042089126890e-01
3.5352660325192e-01 3.9966555166108e-01 3.7905383558817e-01
5.8145385025161e-03 4.0237305753265e-01 3.6768452688003e-01
6.5816774765975e-01 4.0508326456501e-01 3.5631288851457e-01
3.1062814396274e-01 4.0779594331527e-01 3.4493898389669e-01
9.6323730373956e-01 4.1051075138321e-01 3.3356301671141e-01
6.1603551668040e-01 4.1322726567561e-01 3.2218531346229e-01
2.6906037273134e-01 4.1594502102690e-01 3.1080629127311e-01
9.2234514375940e-01 4.1866354947820e-01 2.9942641677105e-01
5.7591729113867e-01 4.2138241475843e-01 2.8804616325652e-01
2.2979739125090e-01 4.2410123796583e-01 2.7666597288693e-01
8.8399864856462e-01 4.2681971263408e-01 2.6528622856432e-01
5.3852701671726e-01 4.2953760956577e-01 2.5390723746105e-01
1.9338182539438e-01 4.3225477342125e-01 2.4252922558304e-01
8.4855674389368e-01 4.3497111379702e-01 2.3115234108327e-01
5.0404090200147e-01 4.3768659349817e-01 2.1977666335928e-01
1.5982001794000e-01 4.4040121618668e-01 2.0840221510130e-01
8.1587742972946e-01 4.4311501488044e-01 1.9702897505267e-01
4.7219497303038e-01 4.4582804211040e-01 1.8565688998218e-01
1.2875368584112e-01 4.4854036202935e-01 1.7428588503905e-01
7.8553434545986e-01 4.5125204442957e-01 1.6291587216960e-01
4.4251785707366e-01 4.5396316044382e-01 1.5154675660786e-01
9.9685519052638e-02 4.5667377963072e-01 1.4017844163785e-01
7.5701919047699e-01 4.5938396814110e-01 1.2881083190514e-01
4.1450138400082e-01 4.6209378769336e-01 1.1744383556905e-01
7.2115303513756e-02 4.6480329513197e-01 1.0607736556323e-01
7.2984484220063e-01 4.6751254239121e-01 9.4711340193321e-02
3.8767455305235e-01 4.7022157672970e-01 8.3345683256573e-02
4.5589600872700e-02 4.7293044113739e-01 7.1980323827853e-02
7.0357570240400e-01 4.7563917484543e-01 6.0615195821146e-02
3.6161905931986e-01 4.7834781389092e-01 4.9250237407283e-02
1.9706287426784e-02 4.8105639170483e-01 3.7885390346375e-02
6.7782434439467e-01 4.8376493970243e-01 2.6520599276792e-02
3.3596045762136e-01 4.8647348786347e-01 1.5155810990606e-02
9.9410205336502e-01 4.8918206529436e-01 3.7909737173657e-03
6.5061023579283e-01 4.9189521073529e-01 9.9242073119014e-01
3.0872566699297e-01 4.9460392870331e-01 9.8105564754361e-01
9.6680917032972e-01 4.9731276155005e-01 9.6969036306723e-01
6.2484831275083e-01 5.0002173997569e-01 9.5832482531148e-01
2.8283055360997e-01 5.0273089616557e-01 9.4695898037659e-01
9.4074320012516e-01 5.0544026411454e-01 9.3559277260689e-01
5.9857337239334e-01 5.0814987986356e-01 9.2422614444414e-01
2.5630798085201e-01 5.1085978161786e-01 9.1285903648659e-01
9.1393371995200e-01 5.1357000970483e-01 9.0149138781252e-01
5.7143708283041e-01 5.1628060631775e-01 8.9012313664000e-01
2.2880440291249e-01 5.1899161497686e-01 8.7875422140750e-01
8.8602192954462e-01 5.2170307962490e-01 8.6738458237075e-01
5.4307594581157e-01 5.2441504326066e-01 8.5601416381625e-01
1.9995293736448e-01 5.2712754600509e-01 8.4464291698617e-01
8.5663982099407e-01 5.2984062249581e-01 8.3327080378622e-01
5.1312424029343e-01 5.3255429852298e-01 8.2189780129836e-01
1.6939493241179e-01 5.3526858686329e-01 8.1052390703460e-01
8.2544216391986e-01 5.3798348234825e-01 7.9914914473882e-01
4.8125822468102e-01 5.4069895632639e-01 7.8777357036721e-01
1.3683795636142e-01 5.4341495084954e-01 7.7639727766494e-01
7.9217927779429e-01 5.4613137312006e-01 7.6502040253327e-01
4.4728365522180e-01 5.4884809095141e-01 7.5364312519988e-01
1.0215645540541e-01 5.5156493016665e-01 7.4226566913784e-01
7.5680711870731e-01 5.5428167492100e-01 7.3088829580678e-01
4.1124910211673e-01 5.5699807181268e-01 7.1951129467495e-01
6.5499570830718e-02 5.5971383829933e-01 7.0813496862653e-01
7.1957885852176e-01 5.6242867537562e-01 6.9675961568131e-01
3.7350976211765e-01 5.6514228378123e-01 6.8538550877939e-01
2.7316773815853e-02 5.6785438234627e-01 6.7401287599170e-01
6.8102536819264e-01 5.7056472660951e-01 6.6264188371424e-01
3.3466144800051e-01 5.7327312568243e-01 6.5127262510683e-01
9.8825101000182e-01 5.7597945550890e-01 6.3990511530363e-01
6.4182003263194e-01 5.7868366712043e-01 6.2853929392969e-01
2.9539452684935e-01 5.8138578909523e-01 6.1717503442480e-01
9.4900064703800e-01 5.8408592407964e-01 6.0581215879550e-01
6.0266474251519e-01 5.8678423984556e-01 5.9445045580588e-01
2.5641324649903e-01 5.8948095589558e-01 5.8308970033086e-01
9.1027234387069e-01 5.9217632705881e-01 5.7172967163505e-01
5.6426741949202e-01 5.9487062580133e-01 5.6037016868168e-01
2.1842234838363e-01 5.9756412505004e-01 5.4901102115995e-01
8.7275873130498e-01 6.0025708315657e-01 5.3765209565009e-01
5.2729519353728e-01 6.0294973221814e-01 5.2629329708663e-01
1.8204684952553e-01 6.0564227040158e-01 5.1493456629440e-01
8.3752606483624e-01 3.7597007357741e-01 5.0401896482077e-01
4.9202905403110e-01 3.7866230564060e-01 4.9265996616604e-01
1.4623917947022e-01 3.8135434394910e-01 4.8130094921499e-01
8.0016904243497e-01 3.8404637724149e-01 4.6994189481963e-01
4.5383715734637e-01 3.8673865217756e-01 4.5858274192290e-01
1.0726762104496e-01 3.8943146984251e-01 4.4722337756246e-01
7.6048933089630e-01 3.9212517500425e-01 4.3586363115027e-01
4.1353485756137e-01 3.9482013784805e-01 4.2450327551037e-01
6.6439168957609e-02 3.9751672925626e-01 4.1314203645195e-01
7.1923844446694e-01 4.0021529202050e-01 4.0177961133610e-01
3.7196918915597e-01 4.0291611132182e-01 3.9041569542452e-01
2.4667759487270e-02 4.0561938817186e-01 3.7905001315972e-01
6.7737027520899e-01 4.0832521925473e-01 3.6768235028407e-01
3.3011276300065e-01 4.1103358586521e-01 3.5631258211571e-01
9.8293129987954e-01 4.1374435355012e-01 3.4494069350061e-01
6.3586192437676e-01 4.1645728273402e-01 3.3356678699132e-01
2.8894016107495e-01 4.1917204914428e-01 3.2219107758859e-01
9.4220013321854e-01 4.2188827142404e-01 3.1081387465986e-01
5.9567337490689e-01 4.2460554222658e-01 2.9943555393954e-01
2.4938755267516e-01 4.2732345864766e-01 2.8805652421638e-01
9.0336533556496e-01 4.3004164825456e-01 2.7667719393331e-01
5.5762361026127e-01 4.3275978811997e-01 2.6529794232334e-01
2.1217314718888e-01 4.3547761581051e-01 2.5391909816642e-01
8.6701872121759e-01 4.3819493275775e-01 2.4254092734141e-01
5.2215960905771e-01 4.4091160149541e-01 2.3116362864228e-01
1.7759034116792e-01 4.4362753873335e-01 2.1978733619707e-01
8.3330157886575e-01 4.4634270621981e-01 2.0841212636062e-01
4.8928100641922e-01 4.4905710099531e-01 1.9703802702178e-01
1.4551415937147e-01 4.5177074615943e-01 1.8566502765566e-01
8.0198514294086e-01 4.5448368280417e-01 1.7429308895519e-01
4.5867722140750e-01 4.5719596339192e-01 1.6292215135222e-01
1.1557327851997e-01 4.5990764659849e-01 1.5155214211561e-01
7.7265616046239e-01 4.6261879348960e-01 1.4018298097418e-01
4.2990891843661e-01 4.6532946482813e-01 1.2881458436952e-01
8.7314969341354e-02 4.6803971929177e-01 1.1744686852165e-01
7.4485819202393e-01 4.7074961239441e-01 1.0607975151613e-01
4.0252297436167e-01 4.7345919593270e-01 9.4713154614496e-02
6.0294223799929e-02 4.7616851781242e-01 8.3347002967585e-02
7.1815735139540e-01 4.7887762214124e-01 7.1981225881907e-02
3.7609823712287e-01 4.8158654950193e-01 6.0615756760219e-02
3.4103182288258e-02 4.8429533734362e-01 4.9250532810606e-02
6.9215885335746e-01 4.8700402044632e-01 3.7885494595940e-02
3.5025222032592e-01 4.8971263142786e-01 2.6520585477386e-02
8.3704918683862e-03 4.9242120127262e-01 1.5155750991616e-02
6.6650104887356e-01 4.9512975986844e-01 3.7909381910545e-03
3.2300423589730e-01 4.9784284898487e-01 9.9242078689431e-01
9.8112176914547e-01 5.0055147003403e-01 9.8105586422126e-01
6.3921388801790e-01 5.0326016652336e-01 9.6969080830857e-01
2.9726800220351e-01 5.0596896922025e-01 9.5832556563947e-01
9.5527137245564e-01 5.0867791035092e-01 9.4696008103839e-01
6.1321105536853e-01 5.1138702403686e-01 9.3559429719009e-01
2.7107385451068e-01 5.1409634667694e-01 9.2422815427065e-01
9.2884628038733e-01 5.1680591725055e-01 9.1286158973207e-01
5.8651452256035e-01 5.1951577750663e-01 9.0149453829547e-01
2.4406443833055e-01 5.2222597199087e-01 8.9012693222275e-01
9.0148156369430e-01 5.2493654784733e-01 8.7875870195436e-01
5.5875115379727e-01 5.2764755431212e-01 8.6738977721923e-01
2.1585826174471e-01 5.3035904179656e-01 8.5602008873971e-01
8.7278786621822e-01 5.3307106043697e-01 8.4464957066649e-01
5.2952505958409e-01 5.3578365797233e-01 8.3327816387772e-01
1.8605530855965e-01 5.3849687680595e-01 8.2190582025575e-01
8.4236479830914e-01 5.4121075012171e-01 8.1053250799966e-01
4.9844086712339e-01 5.4392529697407e-01 7.9915821792934e-01
1.5427253154580e-01 5.4664051636677e-01 7.8778297057251e-01
8.0985109004971e-01 5.4935638049408e-01 7.7640682359365e-01
4.6517077692991e-01 5.5207282754543e-01 7.6502987883220e-01
1.2022941811320e-01 5.5478975475918e-01 7.5365228790203e-01
7.7502902041772e-01 5.5750701271505e-01 7.4227425503955e-01
4.2957621116830e-01 5.6022440210312e-01 7.3089603577961e-01
8.3882443540739e-02 5.6294167429647e-01 7.1951793021344e-01
7.3796390159844e-01 5.6565853687686e-01 7.0814027013191e-01
3.9184108056095e-01 5.6837466475354e-01 6.9676340028557e-01
4.5538077402791e-02 5.7108971670060e-01 6.8538765516016e-01
6.9908169024241e-01 5.7380335616568e-01 6.7401333379818e-01
3.5250047196491e-01 5.7651527430905e-01 6.6264067595422e-01
5.8238954282629e-03 5.7922521266151e-01 6.5126984298759e-01
6.5908175578102e-01 5.8193298269676e-01 6.3990090629115e-01
3.1230386622788e-01 5.8463847999911e-01 6.2853384487643e-01
9.6552001685430e-01 5.8734169144101e-01 6.1716855228563e-01
6.1876008805577e-01 5.9004269468137e-01 6.0580485161003e-01
2.7205416425650e-01 5.9274165019889e-01 5.9444251631372e-01
9.2543249359707e-01 5.9543878689344e-01 5.8308129392502e-01
5.7892518512102e-01 5.9813438297073e-01 5.7172092950482e-01
2.3256161304550e-01 6.0082874431681e-01 5.6036118611283e-01
8.8636958441749e-01 6.0352218278846e-01 5.4900186020128e-01
5.4037439563900e-01 6.0621499671740e-01 5.3764279084357e-01
1.9459793515989e-01 6.0890745543727e-01 5.2628386276338e-01
8.4905797772400e-01 6.1159978888039e-01 5.1492500404970e-01
5.0409188512891e-01 3.8192708054503e-01 5.0400955445873e-01
1.5888341371369e-01 3.8461952953727e-01 4.9265070196316e-01
8.1341026599941e-01 3.8731181170502e-01 4.8129182340709e-01
4.6768314664521e-01 3.9000408978666e-01 4.6993290343585e-01
1.2171767334891e-01 3.9269657464434e-01 4.5857389258775e-01
7.7553415075438e-01 3.9538952303156e-01 4.4721469907386e-01
4.2915699567466e-01 3.9808322992294e-01 4.3585518357021e-01
8.2613879867805e-02 4.0077801507475e-01 4.2449515886890e-01
7.3593472337014e-01 4.0347420440063e-01 4.1313439582250e-01
3.8915071028562e-01 4.0617210769517e-01 4.0177263617588e-01
4.2293492000684e-02 4.0887199498456e-01 3.9040961175204e-01
6.9539468702113e-01 4.1157407416336e-01 3.7904506827977e-01
3.4848569675749e-01 4.1427847253801e-01 3.6767879116443e-01
1.5977600574386e-03 4.1698522449793e-01 3.5631062990031e-01
6.5476209617826e-01 4.1969426687038e-01 3.4494051771643e-01
3.0800995876749e-01 4.2240544266539e-01 3.3356848348845e-01
9.6137245055480e-01 4.2511851293898e-01 3.2219465393057e-01
6.1488002157441e-01 4.2783317547375e-01 3.1081924550482e-01
2.6856167027388e-01 4.3054908804140e-01 2.9944254712648e-01
9.2244395674506e-01 4.3326589338059e-01 2.8806489625104e-01
5.7654999309299e-01 4.3598324288519e-01 2.7668665191984e-01
2.3089858289466e-01 4.3870081642692e-01 2.6530816855384e-01
8.8550364281058e-01 4.4141833662264e-01 2.5392977369957e-01
5.4037397264696e-01 4.4413557694917e-01 2.4255175178002e-01
1.9551336934915e-01 4.4685236411593e-01 2.3117433456477e-01
8.5092102524415e-01 4.4956857581173e-01 2.1979769790801e-01
5.0659212101797e-01 4.5228413526701e-01 2.0842196353132e-01
1.6251851886704e-01 4.5499900405951e-01 1.9704720430076e-01
8.1868947397209e-01 4.5771317435595e-01 1.8567345148218e-01
4.7509230401813e-01 4.6042666144908e-01 1.7430070271746e-01
1.3171297947654e-01 4.6313949711735e-01 1.6292892981234e-01
7.8853661713896e-01 4.6585172405893e-01 1.5155808576692e-01
4.4554787399199e-01 4.6856339145385e-01 1.4018811076065e-01
1.0273124786950e-01 4.7127455158400e-01 1.2881893700645e-01
7.6007129631540e-01 4.7398525737582e-01 1.1745049251882e-01
4.1755278694744e-01 4.7669556070696e-01 1.0608270391266e-01
7.5160792466259e-02 4.7940551132082e-01 9.4715498379640e-02
7.3288074219526e-01 4.8211515620891e-01 8.3348804993038e-02
3.9069844028944e-01 4.8482453934340e-01 7.1982555480185e-02
4.8600058904914e-02 4.8753370166502e-01 6.0616684583433e-02
7.0657211289615e-01 4.9024268125296e-01 4.9251130109673e-02
3.6460142110856e-01 4.9295151362127e-01 3.7885832748604e-02
2.2675058093162e-02 4.9566023210150e-01 2.6520735722414e-02
6.8078029908285e-01 4.9836886828269e-01 1.5155784314806e-02
3.3890456031233e-01 5.0107745248895e-01 3.7909253156793e-03
9.9540796508577e-01 5.0379052793131e-01 9.9242079697486e-01
6.5353244207077e-01 5.0649909511972e-01 9.8105596732671e-01
3.1163818690098e-01 5.0920769708568e-01 9.6969107474892e-01
9.6971253634211e-01 5.1191636423124e-01 9.5832606566626e-01
6.2774264310933e-01 5.1462512829340e-01 9.4696088476569e-01
2.8571541211263e-01 5.1733402286045e-01 9.3559547438077e-01
9.4361743984443e-01 5.2004308386615e-01 9.2422977393838e-01
6.0143495887743e-01 5.2275235004382e-01 9.1286371950265e-01
2.5915379019948e-01 5.2546186331357e-01 9.0149724346319e-01
9.1675930713217e-01 5.2817166906433e-01 8.9013027443036e-01
5.7423641590155e-01 5.3088181627629e-01 8.7876273741971e-01
2.3156955958362e-01 5.3359235741010e-01 8.6739455443110e-01
8.8874275413285e-01 5.3630334796440e-01 8.5602564555344e-01
5.4573966745544e-01 5.3901484557609e-01 8.4465593075240e-01
2.0254375483085e-01 5.4172690850787e-01 8.3328533251938e-01
8.5913846605470e-01 5.4443959334166e-01 8.2191377956962e-01
5.1550754084343e-01 5.4715295168066e-01 8.1054121176113e-01
1.7163540833167e-01 5.4986702567166e-01 7.9916758634864e-01
8.2750770255739e-01 5.5258184220942e-01 7.8779288556525e-01
4.8311189705263e-01 5.5529740580007e-01 7.7641712531727e-01
1.3843804652125e-01 5.5801369026394e-01 7.6504036446970e-01
7.9347960141600e-01 5.6073062976247e-01 7.5366271379819e-01
4.4823423333323e-01 5.6344811003022e-01 7.4228434323404e-01
1.0270458010552e-01 5.6616596112669e-01 7.3090548563667e-01
7.5689879785656e-01 5.6888395338554e-01 7.1952643515604e-01
4.1083080467086e-01 5.7160179837095e-01 7.0814753848699e-01
6.4520127839467e-02 5.7431915638639e-01 6.9676917811534e-01
7.1799132797601e-01 5.7703565132136e-01 6.8539174800876e-01
3.7127305983238e-01 5.7975089242341e-01 6.7401562388404e-01
2.4396917951875e-02 5.8246450119152e-01 6.6264113173832e-01
6.7739627367182e-01 5.8517614038277e-01 6.5126851924777e-01
3.3030531175525e-01 5.8788554146709e-01 6.3989793454498e-01
9.8315841217215e-01 5.9059252693988e-01 6.2852941575317e-01
6.3598991193232e-01 5.9329702464435e-01 6.1716289279328e-01
2.8883415892768e-01 5.9599907242044e-01 6.0579820089446e-01
9.4172567507196e-01 5.9869881270425e-01 5.9443510340965e-01
5.9469921064486e-01 6.0139647794808e-01 5.8307332028013e-01
2.4778950711898e-01 6.0409236881015e-01 5.7171255794245e-01
9.0103068040780e-01 6.0678682789455e-01 5.6035253663613e-01
5.5445525940855e-01 6.0948021229792e-01 5.4899301188026e-01
2.0809302532354e-01 6.1217286821081e-01 5.3763378817799e-01
8.6196985999577e-01 6.1486511026780e-01 5.2627472451432e-01
5.1610680972038e-01 6.1755720733149e-01 5.1491573259982e-01
1.7066655836761e-01 3.8788424072198e-01 5.0399985966584e-01
8.2571879898849e-01 3.9057687617434e-01 4.9264114594348e-01
4.8053141914079e-01 3.9326936857435e-01 4.8128239999666e-01
1.3511358114781e-01 3.9596185944457e-01 4.6992360927405e-01
7.8947855641346e-01 3.9865453054991e-01 4.5856473323503e-01
4.4364357130821e-01 4.0134760250405e-01 4.4720569655860e-01
9.7629381661664e-02 4.0404132925160e-01 4.3584638450211e-01
7.5145961730066e-01 4.0673598809574e-01 4.2448664179178e-01
4.0515998715605e-01 4.0943186557311e-01 4.1312627618794e-01
5.8757468456509e-02 4.1212924015882e-01 4.0176506732824e-01
7.1227960731391e-01 4.1482836336256e-01 3.9040278068926e-01
3.6575402781577e-01 4.1752944112724e-01 3.7903918566328e-01
1.9208189010518e-02 4.2023261750939e-01 3.6767407598584e-01
6.7266935975559e-01 4.2293796242382e-01 3.5630729021020e-01
3.2616472015201e-01 4.2564546483453e-01 3.4493872969628e-01
9.7972146266310e-01 4.2835503222782e-01 3.3356837171408e-01
6.3336676602015e-01 4.3106649655415e-01 3.2219627576651e-01
2.8712755065535e-01 4.3377962610842e-01 3.1082258208678e-01
9.4102998565882e-01 4.3649414210226e-01 2.9944750236254e-01
5.9509878668269e-01 4.3920973808002e-01 2.8807130389941e-01
2.4935640195398e-01 4.4192609998434e-01 2.7669428941428e-01
9.0382221375518e-01 4.4464292469355e-01 2.6531677520707e-01
5.5851187895237e-01 4.4735993524788e-01 2.5393907046565e-01
2.1343689915986e-01 4.5007689165737e-01 2.4256145994303e-01
8.6860446204891e-01 4.5279359696313e-01 2.3118419138749e-01
5.2401754571123e-01 4.5550989891920e-01 2.1980746815880e-01
1.7967524009811e-01 4.5822568814273e-01 2.0843144665453e-01
8.3557321910020e-01 4.6094089380003e-01 1.9705623763498e-01
4.9170429320231e-01 4.6365547788749e-01 1.8568191030226e-01
1.4805898123628e-01 4.6636942900447e-01 1.7430849800395e-01
8.0462605474329e-01 4.6908275628162e-01 1.6293600460567e-01
4.6139302485609e-01 4.7179548389003e-01 1.5156441081975e-01
1.1834655615641e-01 4.7450764635270e-01 1.4019368002299e-01
7.7547280312426e-01 4.7721928472780e-01 1.2882376330602e-01
4.3275767231983e-01 4.7993044363289e-01 1.1745460365448e-01
9.0187017803827e-02 4.8264116902208e-01 1.0608613926906e-01
7.4774677927225e-01 4.8535150660295e-01 9.4718306094607e-02
4.0542307273190e-01 4.8806150077598e-01 8.3351039660119e-02
6.3202242917530e-02 4.9077119398768e-01 7.1984276340552e-02
7.2107088552888e-01 4.9348062640251e-01 6.0617954147639e-02
3.7901584606412e-01 4.9618983581581e-01 4.9252013145621e-02
3.7024200740806e-02 4.9889885774503e-01 3.7886395573609e-02
6.9508322383075e-01 5.0160772565182e-01 2.6521045741660e-02
3.5318034473858e-01 5.0431647125873e-01 1.5155909754118e-02
1.1303097336491e-02 5.0702512493482e-01 3.7909351019740e-03
6.6781192277992e-01 5.0973822972400e-01 9.9242076083618e-01
3.2594812042800e-01 5.1244678744173e-01 9.8105595383367e-01
9.8407238139967e-01 5.1515533904203e-01 9.6969115508075e-01
6.4217205217130e-01 5.1786391418372e-01 9.5832631190397e-01
3.0023426566656e-01 5.2057254366211e-01 9.4696136992439e-01
9.5824587135874e-01 5.2328125997056e-01 9.3559627235458e-01
6.1619336542670e-01 5.2599009786908e-01 9.2423095930924e-01
2.7406282240352e-01 5.2869909494854e-01 9.1286536715656e-01
9.3183983036298e-01 5.3140829217288e-01 9.0149942794721e-01
5.8950943256186e-01 5.3411773437141e-01 8.9013306897227e-01
2.4705607966013e-01 5.3682747064017e-01 8.7876621252036e-01
9.0446359824744e-01 5.3953755459224e-01 8.6739877592872e-01
5.6171518347894e-01 5.4224804437261e-01 8.5603067205341e-01
2.1879342620667e-01 5.4495900232206e-01 8.4466181031868e-01
8.7568038805468e-01 5.4767049413727e-01 8.3329209854429e-01
5.3235774126135e-01 5.5038258733225e-01 8.2192144578480e-01
1.8880699339796e-01 5.5309534876503e-01 8.1054976643908e-01
8.4500981949328e-01 5.5580884096213e-01 7.9917698588396e-01
5.0094852435312e-01 5.5852311696893e-01 7.8780304782989e-01
1.5660665404845e-01 5.6123821350284e-01 7.7642792345659e-01
8.1196976519471e-01 5.6395414232084e-01 7.6505162212654e-01
4.6702634121961e-01 5.6667087997234e-01 7.5367420306616e-01
1.2176881477352e-01 5.6938835651925e-01 7.4229578684685e-01
7.7619461606312e-01 5.7210644436060e-01 7.3091656485107e-01
4.3030712453435e-01 5.7482494892781e-01 7.1953680432137e-01
8.4116368883348e-02 5.7754360355439e-01 7.0815684631247e-01
7.3763931594773e-01 5.8026207102346e-01 6.9677709420036e-01
3.9089962989438e-01 5.8297995389439e-01 6.8539799158062e-01
4.3926875161361e-02 5.8569681455768e-01 6.7401999039584e-01
6.9675526402845e-01 5.8841220417674e-01 6.6264351257544e-01
3.4942217312741e-01 5.9112569766866e-01 6.5126891061536e-01
1.9667224393668e-03 5.9383693026477e-01 6.3989643356860e-01
6.5442868819389e-01 5.9654563050315e-01 6.2852620437649e-01
3.0684790509761e-01 5.9925164493207e-01 6.1715821244322e-01
9.5926414064680e-01 6.0195495113947e-01 6.0579232246008e-01
6.1171725662448e-01 6.0465565754023e-01 5.9442829754067e-01
2.6424737081347e-01 6.0735399023998e-01 5.8306583240668e-01
9.1689473195259e-01 6.1005026900119e-01 5.7170459103638e-01
5.6969912302280e-01 6.1274487572600e-01 5.6034424296143e-01
2.2269877559587e-01 6.1543821979024e-01 5.4898449321454e-01
8.7592895064323e-01 6.1813070481524e-01 5.3762510259100e-01
5.2942045718250e-01 6.2082270089888e-01 5.2626589702223e-01
1.8319840226607e-01 6.2351452501039e-01 5.1490676696052e-01
8.3724912195334e-01 3.9384154505473e-01 5.0398989581880e-01
4.9253732543179e-01 3.9653434084108e-01 4.9263131340666e-01
1.4760834979440e-01 3.9922701456887e-01 4.8127269400344e-01
8.0247009139917e-01 4.0191969030947e-01 4.6991402712907e-01
4.5713390354994e-01 4.0461252598103e-01 4.5855527916195e-01
1.1161449234357e-01 4.0730571251550e-01 4.4719638769948e-01
7.6592959976948e-01 4.0999946986560e-01 4.3583725747131e-01
4.2009949986143e-01 4.1269403954846e-01 4.2447775884711e-01
7.4146369847999e-02 4.1538967386298e-01 4.1311772984340e-01
7.2809362517334e-01 4.1808662240943e-01 4.0175698218571e-01
3.8196531555578e-01 4.2078511698235e-01 3.9039531146389e-01
3.5785664148820e-02 4.2348535621072e-01 3.7903251081595e-01
6.8957879557743e-01 4.2618749143246e-01 3.6766838699844e-01
3.4336865001890e-01 4.2889161521175e-01 3.5630277724752e-01
9.9717903233408e-01 4.3159775367254e-01 3.4493556507687e-01
6.5103371023258e-01 4.3430586347136e-01 3.3356669313643e-01
3.0495646269035e-01 4.3701583380048e-01 3.2219617149030e-01
9.5897099261496e-01 4.3972749331999e-01 3.1082408015692e-01
6.1310065278740e-01 4.4244062139576e-01 2.9945056545510e-01
2.6736798228382e-01 4.4515496252661e-01 2.8807583052432e-01
9.2179409919492e-01 4.4787024246507e-01 2.7670012119745e-01
5.7639803176185e-01 4.5058618436251e-01 2.6532370902866e-01
2.3119608515906e-01 4.5330252335827e-01 2.5394687357685e-01
8.8620133286554e-01 4.5601901837259e-01 2.4256988595489e-01
5.4142329464551e-01 4.5873546037483e-01 2.3119299522260e-01
1.9686782701561e-01 4.6145167695819e-01 2.1981641856111e-01
8.5253721695638e-01 4.6416753353815e-01 2.0844033548880e-01
5.0843044322003e-01 4.6688293182367e-01 1.9706488580852e-01
1.6454355533484e-01 4.6959780636184e-01 1.8569017059949e-01
8.2087011775646e-01 4.7231211995070e-01 1.7431625539842e-01
4.7740167249790e-01 4.7502585860208e-01 1.6294317471874e-01
1.3412818411437e-01 4.7773902657081e-01 1.5157093717522e-01
7.9103844273457e-01 4.8045164179369e-01 1.4019953065230e-01
4.4812041160564e-01 4.8316373192993e-01 1.2882892713348e-01
1.0536151417144e-01 4.8587533107801e-01 1.1745908696612e-01
7.6274886172813e-01 4.8858647716240e-01 1.0608996245876e-01
4.2026942642829e-01 4.9129720993549e-01 9.4721500794707e-02
7.7910166312373e-02 4.9400756951456e-01 8.3353646300330e-02
7.3565810965915e-01 4.9671759536672e-01 7.1986342136535e-02
3.9350040572860e-01 4.9942732565744e-01 6.0619531494033e-02
5.1424348283074e-02 5.0213679688717e-01 4.9253158373805e-02
7.0941737737084e-01 5.0484604375228e-01 3.7887168028067e-02
3.6746706390622e-01 5.0755509917810e-01 2.6521507127562e-02
2.5561080681034e-02 5.1026399448327e-01 1.5156123710416e-02
6.8368716264399e-01 5.1297275964432e-01 3.7909669573591e-03
3.4020660840259e-01 5.1568593590599e-01 9.9242067909535e-01
9.9835924594528e-01 5.1839452859789e-01 9.8105582375288e-01
6.5650679998332e-01 5.2110307502156e-01 9.6969104684767e-01
3.1463670730296e-01 5.2381160376789e-01 9.5832629749679e-01
9.7273616637813e-01 5.2652014431830e-01 9.4696152323007e-01
6.3079206380496e-01 5.2922872761666e-01 9.3559666924998e-01
2.8879089792796e-01 5.3193738667324e-01 9.2423167766037e-01
9.4671870059747e-01 5.3464615719540e-01 9.1286648668302e-01
6.0456095841803e-01 5.3735507823518e-01 9.0150102988809e-01
2.6230253551458e-01 5.4006419283745e-01 8.9013523547586e-01
9.1992760083099e-01 5.4277354866127e-01 8.7876902566399e-01
5.7741956437678e-01 5.4548319853194e-01 8.6740231625698e-01
2.3476102876490e-01 5.4819320085877e-01 8.5603501650502e-01
8.9193376495569e-01 5.5090361982426e-01 8.4466702939810e-01
5.4891872444242e-01 5.5361452521057e-01 8.3329825258897e-01
2.0569610422079e-01 5.5632599167977e-01 8.2192858019281e-01
8.6224548566072e-01 5.5903809726568e-01 8.1055790576789e-01
5.1854607342968e-01 5.6175092077225e-01 7.9918612682801e-01
1.7457706498177e-01 5.6446453771780e-01 7.8781315125466e-01
8.3031818313598e-01 5.6717901443774e-01 7.7643890593102e-01
4.8575040123073e-01 5.6989439999599e-01 7.6506334776475e-01
1.4085687854378e-01 5.7261071570841e-01 7.5368647694460e-01
7.9562409889778e-01 5.7532794241001e-01 7.4230835173408e-01
4.5004316448936e-01 5.7804600615390e-01 7.3092910332741e-01
1.0411114083086e-01 5.8076476381793e-01 7.1954894834989e-01
7.5783228602939e-01 5.8348399101813e-01 7.0816819569301e-01
4.1121894809976e-01 5.8620337553931e-01 6.9678724392167e-01
6.4291906495548e-02 5.8892251979965e-01 6.8540656596236e-01
7.1707999643467e-01 5.9164095524193e-01 6.7402667955549e-01
3.6961899529872e-01 5.9435816976281e-01 6.6264810498770e-01
2.1949940575151e-02 5.9707364657356e-01 6.5127131522390e-01
6.7411722122785e-01 5.9978690996905e-01 6.3989668650628e-01
3.2616685903717e-01 6.0249757135855e-01 6.2852445853441e-01
9.7814532390848e-01 6.0520536834014e-01 6.1715471189867e-01
6.3009902160183e-01 6.0791019071056e-01 6.0578736692086e-01
2.8207433029294e-01 6.1061208961206e-01 5.9442220362470e-01
9.3411784764841e-01 6.1331126880729e-01 5.8305889850373e-01
5.8627642775150e-01 6.1600805977568e-01 5.7169707097346e-01
2.3859666955283e-01 6.1870288463079e-01 5.6033633127964e-01
8.9112373324014e-01 6.2139621254543e-01 5.4897632220806e-01
5.4389962295558e-01 6.2408851614126e-01 5.3761674897686e-01
1.9696127982892e-01 6.2678023385825e-01 5.2625739469077e-01
8.5033890183115e-01 6.2947174264442e-01 5.1489812192595e-01
5.0383877142193e-01 3.9979898467761e-01 5.0397967834015e-01
1.5934075760813e-01 4.0249191832614e-01 4.9262121972819e-01
8.1464581081358e-01 4.0518474846100e-01 4.8126272058722e-01
4.6976077940169e-01 4.0787758469617e-01 4.6990417196234e-01
1.2469543515261e-01 4.1057056525926e-01 4.5854554563605e-01
7.7946240406903e-01 4.1326385653483e-01 4.4718678941926e-01
4.3407692923209e-01 4.1595765016305e-01 4.3582782357654e-01
8.8556481589199e-02 4.1865215742672e-01 4.2446853917677e-01
7.4292026127419e-01 4.2134760096928e-01 4.1310879908341e-01
3.9718865356483e-01 4.2404420424296e-01 4.0174844207276e-01
5.1382713194216e-02 4.2674217942323e-01 3.9038729021451e-01
7.0552374431391e-01 4.2944171477800e-01 3.7902515921882e-01
3.5963302114080e-01 4.3214296260539e-01 3.6766187102031e-01
1.3731660973781e-02 4.3484602884253e-01 3.5629726750085e-01
6.6784062484414e-01 4.3755096531584e-01 3.4493122400693e-01
3.2198079046592e-01 4.4025776538066e-01 3.3356366122591e-01
9.7617302478514e-01 4.4296636340959e-01 3.2219455406898e-01
6.3043818343293e-01 4.4567663825167e-01 3.1082393647113e-01
2.8479698176690e-01 4.4838842041341e-01 2.9945190144787e-01
9.3926971279517e-01 4.5110150233566e-01 2.8807859630641e-01
5.9387582371064e-01 4.5381565080419e-01 2.7670421351934e-01
2.4863339612038e-01 4.5653062030003e-01 2.6532897832659e-01
9.0355859733104e-01 4.5924616602391e-01 2.5395313452356e-01
5.5866517647839e-01 4.6196205544519e-01 2.4257693003686e-01
2.1396406968546e-01 4.6467807750817e-01 2.3120060376151e-01
8.6946315693351e-01 4.6739404901573e-01 2.1982437478084e-01
5.2516718661409e-01 4.7010981811490e-01 2.0844843461163e-01
1.8107785859437e-01 4.7282526515333e-01 1.9707294262707e-01
8.3719403797307e-01 4.7554030140766e-01 1.8569802440347e-01
4.9351206162825e-01 4.7825486629105e-01 1.7432377246766e-01
1.5002609769733e-01 4.8096892364291e-01 1.6295024879813e-01
8.0672852220722e-01 4.8368245762450e-01 1.5157748843250e-01
4.6361028457602e-01 4.8639546862464e-01 1.4020550361411e-01
1.2066124230433e-01 4.8910796945392e-01 1.2883428803324e-01
7.7787045321182e-01 4.9181998199135e-01 1.1746382084925e-01
4.3522642014924e-01 4.9453153435762e-01 1.0609407029838e-01
9.2717287925046e-02 4.9724265862376e-01 9.4724996787511e-02
7.5033099532110e-01 4.9995338902276e-01 8.3356555444229e-02
4.0805538682720e-01 5.0266376060902e-01 7.1988698139942e-02
6.5878289452871e-02 5.0537380830089e-01 6.0621375030296e-02
7.2378756000811e-01 5.0808356624172e-01 4.9254535670376e-02
3.8177110785469e-01 5.1079306741970e-01 3.7888129765623e-02
3.9816897516734e-02 5.1350234349461e-01 2.6522107616834e-02
6.9791293484209e-01 5.1621142478858e-01 1.5156420311471e-02
3.5604723971224e-01 5.1892034040644e-01 3.7910197059448e-03
1.2582507854213e-02 5.2163362813777e-01 9.9242055356198e-01
6.7075625328417e-01 5.2434229903249e-01 9.8105558013216e-01
3.2893177112250e-01 5.2705088518624e-01 9.6969075252760e-01
9.8709666687846e-01 5.2975941384540e-01 9.5832602245550e-01
6.4523828674964e-01 5.3246791288231e-01 9.4696134020976e-01
3.0334364281835e-01 5.3517641134450e-01 9.3559665394356e-01
9.6139933286765e-01 5.3788494005653e-01 9.2423190887207e-01
6.1939145532366e-01 5.4059353227346e-01 9.1286704641660e-01
2.7730552005562e-01 5.4330222438320e-01 9.0150200330592e-01
9.3512635620039e-01 5.4601105665111e-01 8.9013671065809e-01
5.9283801889126e-01 5.4872007399337e-01 8.7877109307884e-01
2.5042369786383e-01 5.5142932675405e-01 8.6740506783115e-01
9.0786563251134e-01 5.5413887144396e-01 8.5603854415724e-01
5.6514504023637e-01 5.5684877137367e-01 8.4467142287128e-01
2.2224206808845e-01 5.5955909707744e-01 8.3330359638990e-01
8.7913578188803e-01 5.6226992637572e-01 8.2193494943052e-01
5.3580421246965e-01 5.6498134385864e-01 8.1056536068324e-01
1.9222448532354e-01 5.6769343949292e-01 7.9919470584716e-01
8.4837306741316e-01 5.7040630596101e-01 7.8782286250268e-01
5.0422617226353e-01 5.7312003425003e-01 7.7644971734430e-01
1.5976036941161e-01 5.7583470694241e-01 7.6507517627843e-01
8.1495344325896e-01 5.7855038867268e-01 7.5369917773009e-01
4.6978553369439e-01 5.8126711337683e-01 7.4232170911113e-01
1.2424055965468e-01 5.8398486837289e-01 7.3094282568604e-01
7.7830787100561e-01 5.8670357606570e-01 7.1956266997821e-01
4.3198399321534e-01 5.8942307519767e-01 7.0818148846955e-01
8.5274235154864e-02 5.9214310494453e-01 6.9679964097159e-01
7.3819385336550e-01 5.9486329640146e-01 6.8541759729704e-01
3.9076845330023e-01 5.9758317648464e-01 6.7403591653799e-01
4.3033405441587e-02 6.0030218828192e-01 6.6265520698559e-01
6.9503227362586e-01 6.0301972907405e-01 6.5127606939756e-01
3.4681454070753e-01 6.0573520305439e-01 6.3989903170550e-01
9.9843315586802e-01 6.0844808150861e-01 6.2852448726717e-01
6.4994249082766e-01 6.1115796050320e-01 6.1715264948594e-01
3.0139711945072e-01 6.1386460601510e-01 6.0578353244118e-01
9.5285148281034e-01 6.1656797881657e-01 5.9441696120776e-01
6.0436011935188e-01 6.1926823530936e-01 5.8305260889503e-01
2.5597789520846e-01 6.2196570473382e-01 5.7169005205974e-01
9.0775966971833e-01 6.2466084701074e-01 5.6032883316669e-01
5.5975907577159e-01 6.2735419848739e-01 5.4896851851173e-01
2.1202647703442e-01 6.3004631461727e-01 5.3760874228435e-01
8.6460651651583e-01 6.3273771858111e-01 5.2624923157738e-01
5.1753584370509e-01 6.3542886283007e-01 5.1488981199625e-01
1.7043483083441e-01 4.0575655089522e-01 5.0396922265562e-01
8.2613057606391e-01 4.0844960302784e-01 4.9261088031281e-01
4.8164778220231e-01 4.1114256802936e-01 4.8125249499021e-01
1.3699242328998e-01 4.1383554346043e-01 4.6989405884820e-01
7.9217295669973e-01 4.1652865117905e-01 4.5853554791629e-01
4.4720027858567e-01 4.1922203713714e-01 4.4717691813858e-01
1.0208754584200e-01 4.2191586926865e-01 4.3581810225659e-01
7.5684987414003e-01 4.2461033323968e-01 4.2445900813511e-01
4.1150394145971e-01 4.2730562605074e-01 4.1309951909482e-01
6.6067543504158e-02 4.3000194774164e-01 4.0173949666902e-01
7.2055915675767e-01 4.3269949170525e-01 3.9037878595841e-01
3.7499756336535e-01 4.3539843432214e-01 3.7901722345074e-01
2.9401578779423e-02 4.3809892474944e-01 3.6765464684727e-01
6.8378990063337e-01 4.4080107572113e-01 3.5629090614618e-01
3.3818107048476e-01 4.4350495614994e-01 3.4492587501426e-01
9.9259351498423e-01 4.4621058618150e-01 3.3355946136327e-01
6.4704561534184e-01 4.4891793515895e-01 3.2219161604990e-01
3.0155574760461e-01 4.5162692272458e-01 3.1082233874837e-01
9.5614224262904e-01 4.5433742302361e-01 2.9945168029849e-01
6.1082323233139e-01 4.5704927169713e-01 2.8807974119486e-01
2.6561637389408e-01 4.5976227508070e-01 2.7670666631783e-01
9.2053847043924e-01 4.6247622079989e-01 2.6533263645211e-01
5.7560502917540e-01 4.6519088881768e-01 2.5395785751893e-01
2.3082981126782e-01 4.6790606197553e-01 2.4258254868816e-01
8.8622442927412e-01 4.7062153518906e-01 2.3120693059134e-01
5.4179803862287e-01 4.7333712268948e-01 2.1983121472195e-01
1.9755715264598e-01 4.7605266299415e-01 2.0845559482587e-01
8.5350559082990e-01 4.7876802158538e-01 1.9708024072591e-01
5.0964455177148e-01 4.8148309152254e-01 1.8570529466622e-01
1.6597278897631e-01 4.8419779237774e-01 1.7433086996895e-01
8.2248686041537e-01 4.8691206795999e-01 1.6295705160046e-01
4.7918142128620e-01 4.8962588329072e-01 1.5158389815333e-01
1.3604953228317e-01 4.9233922123594e-01 1.4021144474589e-01
7.9308296107900e-01 4.9505207911443e-01 1.2883970639753e-01
4.5027246103003e-01 4.9776446550767e-01 1.1746868152568e-01
1.0760801715507e-01 5.0047639741131e-01 1.0609835530708e-01
7.6507905450833e-01 5.0318789779810e-01 9.4728702735594e-02
4.2267460790054e-01 5.0589899360989e-01 8.3359691283771e-02
8.0383454533772e-02 5.0860971416182e-01 7.1991283131008e-02
7.3819421267925e-01 5.1132008992124e-01 6.0623436960932e-02
3.9609541028187e-01 5.1403015161427e-01 4.9256109354604e-02
5.4075527554874e-02 5.1673992961055e-01 3.7889255819473e-02
7.1212301743572e-01 5.1944945353939e-01 2.6522831498903e-02
3.7022630736609e-01 5.2215875209579e-01 1.5156791606758e-02
2.8373785348831e-02 5.2486785300110e-01 3.7910916277635e-03
6.8493008326895e-01 5.2758128891263e-01 9.9242038717858e-01
3.4312955761637e-01 5.3029007880243e-01 9.8105522895508e-01
1.3376092416239e-03 5.3299874802281e-01 9.6969027944962e-01
6.5954208851215e-01 5.3570732223987e-01 9.5832549365747e-01
3.1773056828794e-01 5.3841582749072e-01 9.4696082535801e-01
9.7589027015643e-01 5.4112429068079e-01 9.3559622644613e-01
6.3400798303783e-01 5.4383274014956e-01 9.2423164616220e-01
2.9206997431935e-01 5.4654120630671e-01 9.1286703022590e-01
9.5006189371903e-01 5.4924972234080e-01 9.0150231988043e-01
6.0796867028321e-01 5.5195832500207e-01 8.9013745085773e-01
2.6577440334602e-01 5.5466705545725e-01 8.7877235228312e-01
9.2346224901331e-01 5.5737596020785e-01 8.6740694555083e-01
5.8101430490357e-01 5.6008509205168e-01 8.5604114322253e-01
2.3841149766043e-01 5.6279451104928e-01 8.4467484803146e-01
8.9563348038030e-01 5.6550428542801e-01 8.3330795211825e-01
5.5265855086879e-01 5.6821449231506e-01 8.2194033668544e-01
2.0946360689597e-01 5.7092521812973e-01 8.1057187233871e-01
8.6602416169710e-01 5.7363655838271e-01 7.9920242048667e-01
5.2231445206592e-01 5.7634861652062e-01 7.8783183629433e-01
1.7830768231769e-01 5.7906150132028e-01 7.7645997381751e-01
8.3397645911914e-01 5.8177532219148e-01 7.6508669405545e-01
4.8929348209405e-01 5.8449018162013e-01 7.5371187669092e-01
1.4423255816354e-01 5.8720616394272e-01 7.4233543614299e-01
7.9876999549277e-01 5.8992331979624e-01 7.3095734209688e-01
4.5288639445044e-01 5.9264164609210e-01 7.1957764373341e-01
1.0656877682773e-01 5.9536106238683e-01 7.0819649533796e-01
7.5981287675268e-01 5.9808138615643e-01 6.9681417889431e-01
4.1262527298685e-01 6.0080231156872e-01 6.8543111711457e-01
6.5024920183461e-02 6.0352339830315e-01 6.7404786909922e-01
7.1704361452725e-01 6.0624407773284e-01 6.6266510180928e-01
3.6872508497010e-01 6.0896368217037e-01 6.5128353482950e-01
2.0122750932364e-02 6.1168149833556e-01 6.3990386325053e-01
6.7129662868455e-01 6.1439683962201e-01 6.2852667164835e-01
3.2231019443458e-01 6.1710912548223e-01 6.1715235748340e-01
9.7322802011663e-01 6.1981795298764e-01 6.0578108184096e-01
6.2411463339476e-01 6.2252314668086e-01 5.9441275900098e-01
2.7503446854796e-01 6.2522477751566e-01 5.8304708641522e-01
9.2605225053521e-01 6.2792314806147e-01 5.7168360695920e-01
5.7723294612707e-01 6.3061874750079e-01 5.6032178864015e-01
2.2864062439548e-01 6.3331218527818e-01 5.4896110454211e-01
8.8033609321472e-01 6.3600411586710e-01 5.3760109773167e-01
5.3237376326722e-01 6.3869516813313e-01 5.2624142133081e-01
1.8479855552458e-01 6.4138589055631e-01 5.1488185129627e-01
8.3703672965492e-01 4.1171423517117e-01 5.0395854415591e-01
4.9290802940121e-01 4.1440738904257e-01 4.9260031055278e-01
1.4861761466538e-01 4.1710047025351e-01 4.8124203248611e-01
8.0417072486165e-01 4.1979356625907e-01 4.6988370292286e-01
4.5957471684687e-01 4.2248678522433e-01 4.5852530124885e-01
1.1483903786596e-01 4.2518025599125e-01 4.4716678992814e-01
7.6997509170345e-01 4.2787412654322e-01 4.3580811178706e-01
4.2499600358442e-01 4.3056856077217e-01 4.2444918838994e-01
7.9916304196825e-02 4.3326373351107e-01 4.1308991995263e-01
7.3475156651965e-01 4.3595982398544e-01 4.0173018714420e-01
3.8951803759522e-01 4.3865700803227e-01 3.9036985496003e-01
4.4232308359076e-02 4.4135544959954e-01 3.7900877862849e-01
6.9891105713797e-01 4.4405529214995e-01 3.6764681127216e-01
3.5357088752131e-01 4.4675665063267e-01 3.5628381281481e-01
8.2282619445415e-03 4.4945960465972e-01 3.4491965944068e-01
6.6289951255620e-01 4.5216419343896e-01 3.3355425279492e-01
3.1760089493634e-01 4.5487041288755e-01 3.2218752807745e-01
9.7234864132146e-01 4.5757821519040e-01 3.1081946023479e-01
6.2715896999786e-01 4.6028751088393e-01 2.9945006759787e-01
2.8204801645040e-01 4.6299817334403e-01 2.8807941254214e-01
9.3703166787053e-01 4.6571004534948e-01 2.7670759904418e-01
5.9212530240769e-01 4.6842294720019e-01 2.6533476734172e-01
2.4734345392678e-01 4.7113668572116e-01 2.5396108622602e-01
9.0269943795253e-01 4.7385106340845e-01 2.4258674375113e-01
5.5820498199563e-01 4.7656588699138e-01 2.3121193728709e-01
2.1386990249064e-01 4.7928097479664e-01 2.1983686384958e-01
8.6970186213026e-01 4.8199616248481e-01 2.0846171151077e-01
5.2570622807392e-01 4.8471130695129e-01 1.9708665247115e-01
1.8188603668241e-01 4.8742628840105e-01 1.8571183810089e-01
8.3824205714944e-01 4.9014101078481e-01 1.7433739599521e-01
4.9477293673503e-01 4.9285540090246e-01 1.6296342887415e-01
1.5147540506278e-01 4.9556940653381e-01 1.5159001501387e-01
8.0834451382827e-01 4.9828299395462e-01 1.4021720982863e-01
4.6537389028984e-01 5.0099614515495e-01 1.2884504821693e-01
1.2255598683334e-01 5.0370885501347e-01 1.1747354732478e-01
7.7988231357507e-01 5.0642112861146e-01 1.0610270944253e-01
4.3734364552974e-01 5.0913297880544e-01 9.4732524823782e-02
9.4930199796437e-02 5.1184442412225e-01 8.3362974283077e-02
7.5263178127041e-01 5.1455548699914e-01 7.1994031487448e-02
4.1043789754710e-01 5.1726619236235e-01 6.0625664902088e-02
6.8337845049197e-02 5.1997656651955e-01 4.9257839383069e-02
7.2632076913395e-01 5.2268663633247e-01 3.7890517431347e-02
3.8437570119323e-01 5.2539642863204e-01 2.6523660134625e-02
4.2491575693192e-02 5.2810596983997e-01 1.5157227829308e-02
7.0065722983375e-01 5.3081528576356e-01 3.7911805156985e-03
3.5723976906597e-01 5.3352890223589e-01 9.9242018392380e-01
1.5469547196099e-02 5.3623784834884e-01 9.8105477894235e-01
6.7371460426847e-01 5.3894664118536e-01 9.6968963954780e-01
3.3196310921379e-01 5.4165530462771e-01 9.5832472461214e-01
9.9020293724771e-01 5.4436386270956e-01 9.4695999198084e-01
6.4842159918887e-01 5.4707234003707e-01 9.3559539790270e-01
3.0660616123186e-01 5.4978076228999e-01 9.2423089633455e-01
9.6474315507398e-01 5.5248915680570e-01 9.1286643812175e-01
6.2281847817892e-01 5.5519755325187e-01 9.0150197005421e-01
2.8081728395639e-01 5.5790598439437e-01 8.9013743379777e-01
9.3872386178786e-01 5.6061448696731e-01 8.7877276470431e-01
5.9652150720639e-01 5.6332310264974e-01 8.6740789051083e-01
2.5419238324801e-01 5.6603187914785e-01 8.5604272995105e-01
9.1171737519306e-01 5.6874087137058e-01 8.4467719132447e-01
5.6907594283381e-01 5.7145014266789e-01 8.3331117110151e-01
2.2624597736085e-01 5.7415976606996e-01 8.2194455269354e-01
8.8320367438345e-01 5.7686982541790e-01 8.1057720559014e-01
5.3992344103131e-01 5.7958041620403e-01 7.9920898516845e-01
1.9637786414560e-01 5.8229164583566e-01 7.8783973361773e-01
8.5253777881468e-01 5.8500363289092e-01 7.7646928259760e-01
5.0837249212914e-01 5.8771650474690e-01 7.6509745845269e-01
1.6385023519778e-01 5.9043039273604e-01 7.5372409100706e-01
8.1893893425048e-01 5.9314542376297e-01 7.4234902708806e-01
4.7360740241593e-01 5.9586170717430e-01 7.3097214984409e-01
1.2782704539854e-01 5.9857931577744e-01 7.1959340440860e-01
7.8157412863841e-01 6.0129826049850e-01 7.0821282924024e-01
4.3483254989421e-01 6.0401845955524e-01 6.9683059028456e-01
8.7596887907513e-02 6.0673970540969e-01 6.8544701196907e-01
7.3987527442028e-01 6.0946163598634e-01 6.7406259560706e-01
3.9169143887404e-01 6.1218371977543e-01 6.6267801363721e-01
4.3085236302602e-02 6.1490526569254e-01 6.5129406959472e-01
6.9411122495325e-01 6.1762546586363e-01 6.3991162065439e-01
3.4483543046052e-01 6.2034347198277e-01 6.2853147141401e-01
9.9533111841845e-01 6.2305849537885e-01 6.1715426006034e-01
6.4567482356805e-01 6.2576991177675e-01 6.0578036484375e-01
2.9594373692991e-01 6.2847734831237e-01 5.9440985552162e-01
9.4621483265992e-01 6.3118073406743e-01 5.8304250202165e-01
5.9656518464218e-01 6.3388030412678e-01 5.7167783647253e-01
2.4707228317946e-01 6.3657655748702e-01 5.6031525112670e-01
8.9781316213857e-01 6.3927017855067e-01 5.4895410738487e-01
5.4886178655521e-01 6.4196193899875e-01 5.3759383123261e-01
2.0028508246132e-01 6.4465260013122e-01 5.2623397714450e-01
8.5213871024422e-01 6.4734283390426e-01 5.1487425348355e-01
5.0364398452901e-01 4.1767202912102e-01 5.0394765816349e-01
1.5967417514522e-01 4.2036527023600e-01 4.9258952579120e-01
8.1555814197550e-01 4.2305845146929e-01 4.8123134833547e-01
4.7130050318327e-01 4.2575165176167e-01 4.6987311933751e-01
1.2690768780965e-01 4.2844496775996e-01 4.5851482084951e-01
7.8238792123532e-01 4.3113851389844e-01 4.4715642059543e-01
4.3775112467738e-01 4.3383242128806e-01 4.3579786960259e-01
9.3008731701094e-02 4.3652683524911e-01 4.2443910066688e-01
7.4817343587426e-01 4.3922191143718e-01 4.1308002800539e-01
4.0325889410646e-01 4.4191781065337e-01 4.0172054838832e-01
5.8279417528456e-02 4.4461469257782e-01 3.9036054392571e-01
7.1324968397117e-01 4.4731270879742e-01 3.7899988652617e-01
3.6818450217970e-01 4.5001199559389e-01 3.6763844386454e-01
2.3098648198477e-02 4.5271266700553e-01 3.5627608651712e-01
6.7800678059904e-01 4.5541480867191e-01 3.4491269577041e-01
3.3292342605498e-01 4.5811847292210e-01 3.3354817150014e-01
9.8786301318492e-01 4.6082367548279e-01 3.2218243946375e-01
6.4283992323234e-01 4.6353039407295e-01 3.1081545735838e-01
2.9786852282733e-01 4.6623856902142e-01 2.9944721907144e-01
9.5296314741222e-01 4.6894810589892e-01 2.8807775668982e-01
6.0813801327796e-01 4.7165888000080e-01 2.7670714003143e-01
2.6340704975522e-01 4.7437074236320e-01 2.6533547370135e-01
9.1878365828453e-01 4.7708352686052e-01 2.5396289192664e-01
5.7428041886737e-01 4.7979705783666e-01 2.4258955165455e-01
2.2990877411299e-01 4.8251115768664e-01 2.3121562456801e-01
8.8567872503174e-01 4.8522565383829e-01 2.1984128875308e-01
5.4159857053318e-01 4.8794038468258e-01 2.0846672073187e-01
1.9767471528333e-01 4.9065520414850e-01 1.9709208846076e-01
8.5391156016773e-01 4.9336998478683e-01 1.8571754571560e-01
5.1031147846182e-01 4.9608461938830e-01 1.7434322807937e-01
1.6687487101362e-01 4.9879902129194e-01 1.6296925055171e-01
8.2360028666491e-01 5.0151312362523e-01 1.5159570664128e-01
4.8048459028430e-01 5.0422687775711e-01 1.4022266869613e-01
1.3752315993091e-01 5.0694025124368e-01 1.2885018917514e-01
7.9471009611265e-01 5.0965322551613e-01 1.1747830255865e-01
4.5203842897632e-01 5.1236579351376e-01 1.0610702762326e-01
1.0950031275375e-01 5.1507795741247e-01 9.4736369852794e-02
7.6708720025714e-01 5.1778972654924e-01 8.3366323811077e-02
4.2478999326774e-01 5.2050111560034e-01 7.1996875354470e-02
8.2599167096259e-02 5.2321214303763e-01 6.0628003607766e-02
7.4050486936911e-01 5.2592282986351e-01 4.9259682663448e-02
3.9849699426511e-01 5.2863319860918e-01 3.7891882991315e-02
5.6565234096720e-02 5.3134327257226e-01 2.6524572563863e-02
7.1469911041403e-01 5.3405307526600e-01 1.5157717714060e-02
3.7288798681973e-01 5.3676263005188e-01 3.7912837478422e-03
2.9501976802775e-02 5.3947645424177e-01 9.9241994868228e-01
6.8776658380086e-01 5.4218558924408e-01 9.8105424126361e-01
3.4605301807147e-01 5.4489454235449e-01 9.6968884896185e-01
4.3498380708844e-03 5.4760333549177e-01 9.5832373500149e-01
6.6264529373089e-01 5.5031199056305e-01 9.4695886173392e-01
3.2092726061903e-01 5.5302052979046e-01 9.3559419019645e-01
9.7918316297322e-01 5.5572897612519e-01 9.2422967953044e-01
6.3739988532758e-01 5.5843735375220e-01 9.1286528625722e-01
2.9556367220736e-01 5.6114568869208e-01 9.0150096340635e-01
9.5366001519505e-01 5.6385400950990e-01 8.9013665948906e-01
6.1167352658754e-01 5.6656234814342e-01 8.7877231730797e-01
2.6958779892305e-01 5.6927074086415e-01 8.6740787259670e-01
9.2738524993506e-01 5.7197922938459e-01 8.5604325248798e-01
5.8504695311133e-01 5.7468786212091e-01 8.4467837382091e-01
2.4255245518752e-01 5.7739669561109e-01 8.3331314131914e-01
8.9987958387324e-01 5.8010579607111e-01 8.2194744570701e-01
5.5700425231053e-01 5.8281524103973e-01 8.1058116188657e-01
2.1390027180002e-01 5.8552512101034e-01 7.9921414738424e-01
8.7053919201037e-01 5.8823554086423e-01 7.8784624140223e-01
5.2689019924256e-01 5.9094662079183e-01 7.7647726498961e-01
1.8292011949054e-01 5.9365849620002e-01 7.6510702309037e-01
8.3859359492322e-01 5.9637131584331e-01 7.5373530953089e-01
4.9387352984046e-01 5.9908523708237e-01 7.4236191634786e-01
1.4872193226465e-01 6.0180041679867e-01 7.3098664914529e-01
8.0310130183099e-01 6.0451699618189e-01 7.1960935022286e-01
4.5697671622743e-01 6.0723507758082e-01 7.0822993070867e-01
1.1031871809843e-01 6.0995469225793e-01 6.9684841137988e-01
7.6310696458657e-01 6.1267575974506e-01 6.8546496876021e-01
4.1533434443364e-01 6.1539804302216e-01 6.7407997829149e-01
6.7010913853415e-02 6.1812110880465e-01 6.6269404078781e-01
7.1816667367211e-01 6.2084430737311e-01 6.5130797463181e-01
3.6885214100697e-01 6.2356678847675e-01 6.3992275840262e-01
1.9136102286611e-02 6.2628756517509e-01 6.2853942017828e-01
6.6910088812512e-01 6.2900562468828e-01 6.1715888926517e-01
3.1883658388679e-01 6.3172006824730e-01 6.0578184536233e-01
9.6843609934209e-01 6.3443024872821e-01 5.9440860788024e-01
6.1799250002782e-01 6.3713587253712e-01 5.8303909826430e-01
2.6759864897431e-01 6.3983704147541e-01 5.7167288501362e-01
9.1734781309340e-01 6.4253422610436e-01 5.6030929564876e-01
5.6733330505236e-01 6.4522817859931e-01 5.4894756206971e-01
2.1764581492165e-01 6.4791980663120e-01 5.3758696021016e-01
8.6836848024859e-01 6.5061003791438e-01 5.2622691174884e-01
5.1957109715325e-01 6.5329970499824e-01 5.1486703164214e-01
1.7025618480587e-01 4.2362992450752e-01 5.0393657990425e-01
8.2642991209421e-01 4.2632324030048e-01 4.9257854129043e-01
4.8247176971381e-01 4.2901650749347e-01 4.8122045774735e-01
1.3838584736857e-01 4.3170979782300e-01 4.6986232321661e-01
7.9417778680858e-01 4.3440319819768e-01 4.5850412187994e-01
4.4985477632460e-01 4.3709681085641e-01 4.4714582573101e-01
1.0542547564355e-01 4.3979075257348e-01 4.3578739250588e-01
7.6089987257886e-01 4.4248515277247e-01 4.2442876425559e-01
4.1628908122545e-01 4.4518015049439e-01 4.1306986684299e-01
7.1605099634745e-02 4.4787589027042e-01 4.0171061060533e-01
7.2686055107766e-01 4.5057251706272e-01 3.9035089234292e-01
3.8206843568362e-01 4.5327017054129e-01 3.7899059868057e-01
3.7241917509121e-02 4.5596897904608e-01 3.6762961071359e-01
6.9239416592363e-01 4.5866905363033e-01 3.5626780971238e-01
3.4753826055255e-01 4.6137048259100e-01 3.4490508353566e-01
2.6871576051635e-03 4.6407332686630e-01 3.3354133330574e-01
6.5785370433476e-01 4.6677761662640e-01 3.2217647984132e-01
3.1305067966092e-01 4.6948334930777e-01 3.1081046932884e-01
9.6829083420150e-01 4.7219048924945e-01 2.9944327774684e-01
6.2358690296905e-01 4.7489896898517e-01 2.8807491363921e-01
2.7895156878038e-01 4.7760869213062e-01 2.7670541896066e-01
9.3439736315955e-01 4.8031953768685e-01 2.6533486788290e-01
5.8993650260806e-01 4.8303136546767e-01 2.5396336363871e-01
2.4558066954222e-01 4.8574402226564e-01 2.4259103366975e-01
9.0134075685973e-01 4.8845734831173e-01 2.3121802350723e-01
5.5722660127740e-01 4.9117118357196e-01 2.1984448992514e-01
2.1324673233028e-01 4.9388537346281e-01 2.0847059394708e-01
8.6940816125512e-01 4.9659977365266e-01 1.9709649425460e-01
5.2571622781921e-01 4.9931425373322e-01 1.8572234144658e-01
1.8217451501782e-01 5.0202869967358e-01 1.7434827345839e-01
8.3878483313129e-01 5.0474301509013e-01 1.6297441229209e-01
4.9554726733930e-01 5.0745712146130e-01 1.5160086206193e-01
1.5246027784954e-01 5.1017095747970e-01 1.4022770824088e-01
8.0952083863537e-01 5.1288447776270e-01 1.2885501791494e-01
4.6672460021327e-01 5.1559765114193e-01 1.1748284081223e-01
1.2406606293107e-01 5.1831045872988e-01 1.0611121086847e-01
7.8153874936100e-01 5.2102289192669e-01 9.4740148110124e-02
4.3913536701147e-01 5.2373495049064e-01 8.3369660670417e-02
9.6847955224733e-02 5.2644664075733e-01 7.1999746794340e-02
7.5466801250528e-01 5.2915797405910e-01 6.0630396728768e-02
4.1258660246500e-01 5.3186896536929e-01 4.9261594429689e-02
7.0594438024745e-02 5.3457963217629e-01 3.7893319047843e-02
7.2868194450433e-01 5.3728999357883e-01 2.6525546176346e-02
3.8683930282983e-01 5.4000006958587e-01 1.5158248859547e-02
4.5056474370033e-02 5.4270988060076e-01 3.7913983723413e-03
7.0170711062636e-01 5.4542393372510e-01 9.9241968708641e-01
3.6001101326505e-01 5.4813328488514e-01 9.8105362917133e-01
1.8343090489090e-02 5.5084243007869e-01 9.6968792750049e-01
6.7669235795954e-01 5.5355138909806e-01 9.5832255001410e-01
3.3504750908290e-01 5.5626018163306e-01 9.4695746387421e-01
9.9339685545609e-01 5.5896882749107e-01 9.3559263517331e-01
6.5172825493906e-01 5.6167734690900e-01 9.2422802849438e-01
3.1002902694995e-01 5.6438576095900e-01 9.1286360631939e-01
9.6828585425178e-01 5.6709409205417e-01 9.0149932827882e-01
6.2648467019539e-01 5.6980236456486e-01 8.9013515023034e-01
2.8461053011723e-01 5.7251060556020e-01 8.7877102315026e-01
9.4264746538992e-01 5.7521884569354e-01 8.6740689181930e-01
6.0057831853651e-01 5.7792712025397e-01 8.5604269328267e-01
2.5838455792867e-01 5.8063547040782e-01 8.4467835506774e-01
9.1604607102734e-01 5.8334394465337e-01 8.3331379315112e-01
5.7354093610885e-01 5.8605260050609e-01 8.2194890968571e-01
2.3084517429159e-01 5.8876150641708e-01 8.1058359053270e-01
8.8793248696636e-01 5.9147074389744e-01 7.9921770270227e-01
5.4477398923479e-01 5.9418040976733e-01 7.8785109190317e-01
2.0133795883735e-01 5.9689061835526e-01 7.7648358055139e-01
8.5758963390838e-01 5.9960150332099e-01 7.6511496681532e-01
5.1349111375913e-01 6.0231321853873e-01 7.5374502560085e-01
1.6900144689894e-01 6.0502593712512e-01 7.4237351282095e-01
8.2407703093676e-01 6.0773984721135e-01 7.3100017483653e-01
4.7867249798553e-01 6.1045514245653e-01 7.1962476551001e-01
1.3274230729169e-01 6.1317200469458e-01 7.0824707364383e-01
7.8624329014659e-01 6.1589057580331e-01 6.9686696320012e-01
4.3913834502264e-01 6.1861091648533e-01 6.8548442681721e-01
9.1401296214148e-02 6.2133295208858e-01 6.7409964877203e-01
7.4302254439911e-01 6.2405641083428e-01 6.6271306612509e-01
3.9401456617699e-01 6.2678076794142e-01 6.5132540750241e-01
4.4415758981407e-02 6.2950521782357e-01 6.3993768226970e-01
6.9429100164097e-01 6.3222870015149e-01 6.2855109631955e-01
3.4372805990864e-01 6.3494999722760e-01 6.1716689054637e-01
9.9283060441795e-01 6.3766789771038e-01 6.0578613112192e-01
6.4171029725778e-01 6.4038139352199e-01 5.9440951049785e-01
2.9048087693299e-01 6.4308985833269e-01 5.8303722433886e-01
9.3925581403227e-01 6.4579315819587e-01 5.7166896529837e-01
5.8814868064264e-01 6.4849166537662e-01 5.6030403252419e-01
2.3727351981601e-01 6.5118617429550e-01 5.4894151729541e-01
8.8674250139022e-01 6.5387774429783e-01 5.3758050513397e-01
5.3666000719668e-01 6.5656751201802e-01 5.2622023749110e-01
1.8711471796343e-01 6.5925652132163e-01 5.1486019815787e-01
8.3687298095236e-01 4.2958791323726e-01 5.0392532448454e-01
4.9317600612430e-01 4.3228129280121e-01 4.9256737220591e-01
1.4936054590917e-01 4.3497463372383e-01 4.8120937584716e-01
8.0543023810551e-01 4.3766800162364e-01 4.6985132962176e-01
4.6139005234324e-01 4.4036147513811e-01 4.5849321942234e-01
1.1724629117153e-01 4.4305514613323e-01 4.4713502073053e-01
7.7300653449612e-01 4.4574911914583e-01 4.3577669680284e-01
4.2867952769771e-01 4.4844350994274e-01 4.2441819735594e-01
8.4275020188311e-02 4.5113844314236e-01 4.1305945797697e-01
7.3980356754395e-01 4.5383404892310e-01 4.0170040045795e-01
3.9527631548493e-01 4.5653045892946e-01 3.9034093419730e-01
5.0704786718499e-02 4.5922780156972e-01 3.7898095872838e-01
7.0610069122807e-01 4.6192619696669e-01 3.6762036733484e-01
3.6147577679744e-01 4.6462575186723e-01 3.5625905160608e-01
1.6841729889556e-02 4.6732655483259e-01 3.4489690668628e-01
6.7221012854150e-01 4.7002867202128e-01 3.3353383688807e-01
3.2759244014044e-01 4.7273214384201e-01 3.2216976128527e-01
9.8300004932432e-01 4.7543698270317e-01 3.1080461887386e-01
6.3844429610282e-01 4.7814317201953e-01 2.9943837289915e-01
2.9393650248168e-01 4.8085066656184e-01 2.8807101399166e-01
9.4948796769838e-01 4.8355939415037e-01 2.7670256183414e-01
6.0510991733820e-01 4.8626925860425e-01 2.6533306519383e-01
2.6081339923385e-01 4.8898014376781e-01 2.5396260028995e-01
9.1660912786282e-01 4.9169191835339e-01 2.4259126761336e-01
5.7250728742915e-01 4.9440444127560e-01 2.3121918745586e-01
2.2851731050045e-01 4.9711756711799e-01 2.1984649452071e-01
8.8464765288265e-01 4.9983115137408e-01 2.0847333205699e-01
5.4090558586062e-01 5.0254505514479e-01 1.9709984597732e-01
1.9729702420453e-01 5.0525914904589e-01 1.8572617938089e-01
8.5382640322070e-01 5.0797331617161e-01 1.7435246782105e-01
5.1049661175468e-01 5.1068745405957e-01 1.6297883554458e-01
1.6730898163284e-01 5.1340147569264e-01 1.5160539280945e-01
8.2426332854998e-01 5.1611530964492e-01 1.4023223427619e-01
4.8135803549193e-01 5.1882889952587e-01 1.2885943837901e-01
1.3859016763133e-01 5.2154220289826e-01 1.1748706752277e-01
7.9595560711435e-01 5.2425518984473e-01 1.0611516892152e-01
4.5344919690849e-01 5.2696784134157e-01 9.4743775889171e-02
1.1106488446143e-01 5.2968014757158e-01 8.3372909406924e-02
7.6879585790262e-01 5.3239210627784e-01 7.2002579817209e-02
4.2663466955708e-01 5.3510372123033e-01 6.0632788524899e-02
8.4573343400966e-02 5.3781500085089e-01 4.9263529617478e-02
7.4260346463506e-01 5.4052595702049e-01 3.7894791344109e-02
4.0071625073860e-01 5.4323660407633e-01 2.6526557417119e-02
5.8902604196978e-02 5.4594695799502e-01 1.5158808117739e-02
7.1715314761599e-01 5.4865703575076e-01 3.7915212012130e-03
3.7384559426137e-01 5.5137133256878e-01 9.9241940533645e-01
3.2193187499884e-02 5.5408092110756e-01 9.8105295757173e-01
6.9057505683902e-01 5.5679028456510e-01 9.6968689798977e-01
3.4898073874787e-01 5.5949944045594e-01 9.5832119950043e-01
7.3994347503616e-03 5.6220840618227e-01 9.4695583425625e-01
6.6581995926645e-01 5.6491719914203e-01 9.3559077352059e-01
3.2423067417086e-01 5.6762583693043e-01 9.2422598738610e-01
9.8261940958117e-01 5.7033433763517e-01 9.1286144434072e-01
6.4097337001016e-01 5.7304272023011e-01 9.0149711067322e-01
2.9927902462700e-01 5.7575100507682e-01 8.9013294970454e-01
9.5752197997386e-01 5.7845921454862e-01 8.7876892082546e-01
6.1568683308995e-01 5.8116737379700e-01 8.6740497831487e-01
2.7375700262563e-01 5.8387551168647e-01 8.5604106990757e-01
9.3171453522901e-01 5.8658366192954e-01 8.4467713507588e-01
5.8953988433324e-01 5.8929186445898e-01 8.3331310298791e-01
2.4721165860206e-01 5.9200016707805e-01 8.2194889010873e-01
9.0470633793777e-01 5.9470862742907e-01 8.1058439742373e-01
5.6199795650771e-01 5.9741731531174e-01 7.9921950729322e-01
2.1905775534163e-01 6.0012631535896e-01 7.8785408000395e-01
8.7585381270919e-01 6.0283573002653e-01 7.7648795018503e-01
5.3235067028987e-01 6.0554568275670e-01 7.6512092342631e-01
1.8850898946113e-01 6.0825632100307e-01 7.5375277371316e-01
8.4428529817423e-01 6.1096781851594e-01 7.4238324271573e-01
4.9963192901658e-01 6.1368037582714e-01 7.3101204258909e-01
1.5449730721892e-01 6.1639421718649e-01 7.1963886477340e-01
8.0882682455781e-01 6.1910958126623e-01 7.0826339826203e-01
4.6256462255919e-01 6.2182670186458e-01 6.9688536165274e-01
1.1565667512377e-01 6.2454577398927e-01 6.8550455332524e-01
7.6805553637254e-01 6.2726690100326e-01 6.7412092199476e-01
4.1972688522133e-01 6.2999002158658e-01 6.6273465380619e-01
7.0657419738794e-02 6.3271482309440e-01 6.5134626054354e-01
7.2086271163450e-01 6.3544066119509e-01 6.3995663776076e-01
3.7039264619484e-01 6.3816652077612e-01 6.2856704933780e-01
1.9331837372601e-02 6.4089105953933e-01 6.1717900050024e-01
6.6779378727491e-01 6.4361276018657e-01 6.0579399672119e-01
3.1591045366559e-01 6.4633017657813e-01 5.9441324345695e-01
9.6382152907618e-01 6.4904221203823e-01 5.8303738746144e-01
6.1166776619380e-01 6.5174834417697e-01 5.7166639788758e-01
2.5958955090213e-01 6.5444872554871e-01 5.6029963065071e-01
9.0772784030288e-01 6.5714413195658e-01 5.4893604563696e-01
5.5622273582064e-01 6.5983577847676e-01 5.3757449244597e-01
2.0520680871399e-01 6.6252506198326e-01 5.2621396660661e-01
8.5479437935980e-01 6.6521330757210e-01 5.1485376456846e-01
5.0349407522361e-01 4.3554598735760e-01 5.0391390687324e-01
1.5991311091071e-01 4.3823942121308e-01 4.9255603356525e-01
8.1622621766226e-01 4.4093282521959e-01 4.8119811765044e-01
4.7243664501375e-01 4.4362625978452e-01 4.6984015352144e-01
1.2854878550152e-01 4.4631979649106e-01 4.5848212845538e-01
7.8456818064060e-01 4.4901351834226e-01 4.4712402080254e-01
4.4050148056758e-01 4.5170751938342e-01 4.3576579838979e-01
9.6356357044739e-02 4.5440190360537e-01 4.2440741731624e-01
7.5214137444998e-01 4.5709678307703e-01 4.1304882132200e-01
4.0786582838907e-01 4.5979227531497e-01 4.0168994189464e-01
6.3539565798322e-02 4.6248849996440e-01 3.9033069923771e-01
7.1917280300082e-01 4.6518557493168e-01 3.7897100416607e-01
3.7477595849315e-01 4.6788361216438e-01 3.6761076091746e-01
3.0359515016215e-02 4.7058271331485e-01 3.5624987078779e-01
6.8593392092327e-01 4.7328296554255e-01 3.4488823641778e-01
3.4150953467002e-01 4.7598443770920e-01 3.3352576648966e-01
9.9709660940323e-01 4.7868717720078e-01 3.2216238054063e-01
6.5270530822374e-01 4.8139120757591e-01 3.1079801357019e-01
3.0834573577669e-01 4.8409652719371e-01 2.9943262011363e-01
9.6402796915438e-01 4.8680310891972e-01 2.8806617747520e-01
6.1976207110302e-01 4.8951090094632e-01 2.7669868786293e-01
2.7555807120391e-01 4.9221982869723e-01 2.6533017924054e-01
9.3142590563507e-01 4.9492979771503e-01 2.5396070480764e-01
5.8737531253323e-01 4.9764069736303e-01 2.4259034112905e-01
2.4341568683921e-01 5.0035240511300e-01 2.3121918504803e-01
8.9955590471340e-01 5.0306479114787e-01 2.1984734962136e-01
5.5580413216563e-01 5.0577772298987e-01 2.0847495939380e-01
2.1216763478448e-01 5.0849106987383e-01 1.9710214537310e-01
8.6865260515528e-01 5.1120470662304e-01 1.8572904006946e-01
5.2526402197782e-01 5.1391851684461e-01 1.7435577292581e-01
1.8200555067844e-01 5.1663239533477e-01 1.6298246639594e-01
8.3887949031129e-01 5.1934624966097e-01 1.5160923283901e-01
4.9588676662101e-01 5.2206000095578e-01 1.4023617230575e-01
1.5302696698820e-01 5.2477358401194e-01 1.2886337120659e-01
8.1029841002027e-01 5.2748694680266e-01 1.1749090178414e-01
4.6769824091764e-01 5.3020004956752e-01 1.0611882226696e-01
1.2522254333216e-01 5.3291286360382e-01 9.4747177557398e-02
7.8286645898128e-01 5.3562536989094e-01 8.3376000302327e-02
4.4062430746407e-01 5.3833755765510e-01 7.2005312204959e-02
9.8489700231955e-02 5.4104942295870e-01 6.0635125455256e-02
7.5645564423934e-01 5.4376096737514e-01 4.9265444179723e-02
4.1451463225672e-01 5.4647219678910e-01 3.7896265835542e-02
7.2658718061516e-02 5.4918312034495e-01 2.6527582496284e-02
7.3087957568475e-01 5.5189374955224e-01 1.5159381995458e-02
3.8916854258077e-01 5.5460409754776e-01 3.7916489091762e-03
4.5907909542262e-02 5.5731864605343e-01 9.9241911000664e-01
7.0430349844777e-01 5.6002848669774e-01 9.8105224255067e-01
3.6273917781824e-01 5.6273808838949e-01 9.6968578553355e-01
2.1205063433834e-02 5.6544746622314e-01 9.5831971698494e-01
6.7969092468943e-01 5.6815663525410e-01 9.4695401411646e-01
3.3818614061084e-01 5.7086561049551e-01 9.3558865334845e-01
9.9667964202225e-01 5.7357440700772e-01 9.2422361019210e-01
6.5515983931337e-01 5.7628304007902e-01 9.1285885898464e-01
3.1361453485941e-01 5.7899152550025e-01 9.0149437247067e-01
9.7203081870846e-01 5.8169987994058e-01 8.9013012121640e-01
6.3039494566976e-01 5.8440812143683e-01 8.7876607283709e-01
2.8869219141018e-01 5.8711627001486e-01 8.6740219101035e-01
9.4690668459732e-01 5.8982434846854e-01 8.5603843423925e-01
6.0502121152885e-01 5.9253238332970e-01 8.4467475432015e-01
2.6301698908668e-01 5.9524040607175e-01 8.3331109446088e-01
9.2087340131811e-01 5.9794845459900e-01 8.2194738698699e-01
5.7856769459666e-01 6.0065657508389e-01 8.1058355056910e-01
2.3607462638723e-01 6.0336482422117e-01 7.9921948690770e-01
8.9336606354622e-01 6.0607327196957e-01 7.8785507683073e-01
5.5041052855304e-01 6.0878200483913e-01 7.7649017580615e-01
2.0717269732689e-01 6.1149112974230e-01 7.6512460896889e-01
8.6361286238671e-01 6.1420077833642e-01 7.5375816594280e-01
5.1968639336821e-01 6.1691111160399e-01 7.4239059605606e-01
1.7534325840822e-01 6.1962232408325e-01 7.3102160507742e-01
8.3052772188321e-01 6.2233464658277e-01 7.1965085543651e-01
4.8517841534614e-01 6.2504834527348e-01 7.0827797312107e-01
1.3922909662902e-01 6.2776371364294e-01 6.9690256607378e-01
7.9261056338057e-01 6.3048105194525e-01 6.8552426066702e-01
4.4525433601340e-01 6.3320062691287e-01 6.7414276380199e-01
9.7098767112177e-02 6.3592260394593e-01 6.6275795628399e-01
7.4809796181419e-01 6.3864694754267e-01 6.5137001487590e-01
3.9823302513870e-01 6.4137329741007e-01 6.3997954215949e-01
4.7523561849678e-02 6.4410084997910e-01 6.2858765588978e-01
6.9603556557658e-01 6.4682830220172e-01 6.1719596621637e-01
3.4388138094198e-01 6.4955392635167e-01 6.0580637828013e-01
9.9121003694102e-01 6.5227581439996e-01 5.9442072337254e-01
6.3819151519750e-01 6.5499225470469e-01 5.8304032499062e-01
2.8500252665428e-01 6.5770212413547e-01 5.7166567428227e-01
9.3181979540756e-01 6.6040515396846e-01 5.6029635765276e-01
5.7882014835998e-01 6.6310197492795e-01 5.4893126216095e-01
2.2618060488835e-01 6.6579393168230e-01 5.3756896022719e-01
8.7407166071921e-01 6.6848273844374e-01 5.2620811190964e-01
5.2264306468123e-01 6.7117009832915e-01 5.1484774138080e-01
1.7011921411260e-01 4.4150413905332e-01 5.0390234188931e-01
8.2664178466090e-01 4.4419761894962e-01 4.9254454025270e-01
4.8307027680794e-01 4.4689107676570e-01 4.8118669804264e-01
1.3940760512347e-01 4.4958456845985e-01 4.6982880976669e-01
7.9565766394433e-01 4.5227815957638e-01 4.5847086383297e-01
4.5182533706329e-01 4.5497192551311e-01 4.4711284096901e-01
1.0791646935701e-01 4.5766595128216e-01 4.3575471281006e-01
7.6393779966167e-01 4.6036033068184e-01 4.2439644079833e-01
4.1989685801151e-01 4.6305516483193e-01 4.1303797553938e-01
7.5801834306604e-02 4.6575056006966e-01 4.0167925675109e-01
7.3166142894778e-01 4.6844662525551e-01 3.9032021391271e-01
3.8748469832690e-01 4.7114346858972e-01 3.7896076767782e-01
4.3280908771850e-02 4.7384119408664e-01 3.6760083205861e-01
6.9905941135092e-01 4.7653989788917e-01 3.5624031732554e-01
3.5482954690673e-01 4.7923966462518e-01 3.4487913350334e-01
1.0600586216025e-02 4.8194056401251e-01 3.3351719428677e-01
6.6638170491947e-01 4.8464264790831e-01 3.2215442115393e-01
3.2218198763546e-01 4.8734594797569e-01 3.1079074742488e-01
9.7801045125790e-01 4.9005047410817e-01 2.9942612200081e-01
6.3387607450110e-01 4.9275621371263e-01 2.8806051252674e-01
2.8978781972964e-01 4.9546313190597e-01 2.7669390774789e-01
9.4575463414686e-01 4.9817117263005e-01 2.6532631887732e-01
6.0178542036386e-01 5.0088026063612e-01 2.5395777985811e-01
2.5788897079646e-01 5.0359030423520e-01 2.4258834648341e-01
9.1407386558212e-01 5.0630119865956e-01 2.3121809442568e-01
5.7034833897547e-01 5.0901282983855e-01 2.1984711631336e-01
2.2672012367262e-01 5.1172507836375e-01 2.0847551806924e-01
8.8319628556760e-01 5.1443782341089e-01 1.9710341477871e-01
5.3978306265872e-01 5.1715094639858e-01 1.8573092638231e-01
1.9648572113305e-01 5.1986433419844e-01 1.7435817348100e-01
8.5330843932635e-01 5.2257788176017e-01 1.6298527350768e-01
5.1025422680330e-01 5.2529149407365e-01 1.5161233746082e-01
1.6732488186599e-01 5.2800508744901e-01 1.4023946732590e-01
8.2452098700153e-01 5.3071859014820e-01 1.2886675423778e-01
4.8184193860752e-01 5.3343194244216e-01 1.1749427737187e-01
1.3928600508004e-01 5.3614509619441e-01 1.0612210349979e-01
7.9685040609658e-01 5.3885801408397e-01 9.4750287110404e-02
4.5453140559478e-01 5.4157066858014e-01 8.3378870978020e-02
1.1232441134543e-01 5.4428304077251e-01 7.2007887056008e-02
7.7022407490758e-01 5.4699511914402e-01 6.0637357580930e-02
4.2822438690457e-01 5.4970689835688e-01 4.9267296286581e-02
8.6318763776461e-02 5.5241837810316e-01 3.7897709645305e-02
7.4450012330777e-01 5.5512956205474e-01 2.6528598072806e-02
4.0276094721078e-01 5.5784045693385e-01 1.5159957050317e-02
6.1093329821439e-02 5.6055107171374e-01 3.7917781333175e-03
7.1788464556725e-01 5.6326587304100e-01 9.9241880784543e-01
3.7633242343948e-01 5.6597597378669e-01 9.8105150087465e-01
3.4825781654238e-02 5.6868582709968e-01 9.6968461671771e-01
6.9335546751189e-01 5.7139544552097e-01 9.5831813857777e-01
3.5191187214552e-01 5.7410484170465e-01 9.4695204869825e-01
1.0484995164564e-02 5.7681402830938e-01 9.3558632853760e-01
6.6906439523686e-01 5.7952301798019e-01 9.2422095880934e-01
3.2763912600007e-01 5.8223182341728e-01 9.1285591938599e-01
9.8619765630564e-01 5.8494045753251e-01 9.0149118906135e-01
6.4472777334608e-01 5.8764893369799e-01 8.9012674516028e-01
3.0321646669218e-01 5.9035726609597e-01 8.7876256297865e-01
9.6164979066418e-01 5.9306547018514e-01 8.6739861502790e-01
6.2001270176552e-01 5.9577356330535e-01 8.5603487004936e-01
2.7828886711137e-01 5.9848156545134e-01 8.4467129175177e-01
9.3646043887829e-01 6.0118950025647e-01 8.3330783721188e-01
5.9450778879454e-01 6.0389739624023e-01 8.2194445486325e-01
2.5240919561748e-01 6.0660528838824e-01 8.1058108198125e-01
9.1014047749725e-01 6.0931322015076e-01 7.9921764155665e-01
5.6767456030506e-01 6.1202124596393e-01 7.8785403843860e-01
2.2498097280604e-01 6.1472943441372e-01 7.7649015462743e-01
8.8202526073996e-01 6.1743787216933e-01 7.6512584362368e-01
5.3876831583167e-01 6.2014666879557e-01 7.5376092381623e-01
1.9516562499038e-01 6.2285596248607e-01 7.4239517106310e-01
8.5116646383252e-01 6.2556592659015e-01 7.3102831095754e-01
5.0671309450801e-01 6.2827677644974e-01 7.1966001189416e-01
1.6174009227037e-01 6.3098877537359e-01 7.0828988112640e-01
8.1617403511908e-01 6.3370223734235e-01 6.9691746774996e-01
4.6993396597593e-01 6.3641752199341e-01 6.8554227912903e-01
1.2293328856012e-01 6.3913501440349e-01 6.7416382056822e-01
7.7508405583526e-01 6.4185507850322e-01 6.6278167094375e-01
4.2630481022284e-01 6.4457797041166e-01 6.5139560638733e-01
7.6532881796537e-02 6.4730370125024e-01 6.4000577356563e-01
7.2574076198263e-01 6.5003185621018e-01 6.2861288518914e-01
3.7395342499124e-01 6.5276141484876e-01 6.1721836136134e-01
2.1260097606327e-02 6.5549066826869e-01 6.0582429460394e-01
6.6781307816391e-01 6.5821735149542e-01 5.9443313208617e-01
3.1381139922433e-01 6.6093904781620e-01 5.8304709628389e-01
9.5947698707388e-01 6.6365377461752e-01 5.7166755567551e-01
6.0503657604634e-01 6.6636052660435e-01 5.6029464983178e-01
2.5071639679788e-01 6.6905954585527e-01 5.4892735921821e-01
8.9674339030271e-01 6.7175221129970e-01 5.3756396946565e-01
5.4333989612465e-01 6.7444060516112e-01 5.2620268842347e-01
1.9070556672814e-01 6.7712694196839e-01 5.1484213784966e-01
8.3674818221583e-01 4.4746236064259e-01 5.0389064419433e-01
4.9336250368752e-01 4.5015587938513e-01 4.9253290699899e-01
1.4989399691948e-01 4.5284938292384e-01 4.8117513176465e-01
8.0634528646582e-01 4.5554292341226e-01 4.6981731307279e-01
4.6271983433886e-01 4.5823656120783e-01 4.5845944026712e-01
1.1902195260818e-01 4.6093036515590e-01 4.4710149606275e-01
7.7525678462054e-01 4.6362441245843e-01 4.3574345529104e-01
4.3143025391820e-01 4.6631878805781e-01 4.2438528389364e-01
8.7548982895168e-02 4.6901358349229e-01 4.1302693828445e-01
7.4362018635927e-01 4.7170889521405e-01 4.0166836519106e-01
3.9965154801069e-01 4.7440482240119e-01 3.9030950206791e-01
5.5651089922651e-02 4.7710146433626e-01 3.7895027814111e-01
7.1162704600551e-01 4.7979891746170e-01 3.6759061610248e-01
3.6758774990319e-01 4.8249727225284e-01 3.5623043442176e-01
2.3541545793471e-02 4.8519661006844e-01 3.4486965019551e-01
6.7949672736696e-01 4.8789700014610e-01 3.3350818240430e-01
3.3546150628224e-01 4.9059849690548e-01 3.2214595541029e-01
9.9144400720454e-01 4.9330113770780e-01 3.1078290249870e-01
6.4745228273433e-01 4.9600494119739e-01 2.9941896925122e-01
3.0349433866089e-01 4.9870990632218e-01 2.8805411653830e-01
9.5957815843600e-01 5.0141601209643e-01 2.7668832293144e-01
6.1571171576715e-01 5.0412321813111e-01 2.6532158636583e-01
2.7190296577915e-01 5.0683146591659e-01 2.5395392492822e-01
9.2815980807470e-01 5.0954068079935e-01 2.4258537670164e-01
5.8449001883698e-01 5.1225077455231e-01 2.3121599866475e-01
2.4090115332170e-01 5.1496164840031e-01 2.1984586471364e-01
8.9740042407985e-01 5.1767319633229e-01 2.0847506293978e-01
5.5399456346767e-01 5.2038530851472e-01 1.9710369235307e-01
2.1068968099953e-01 5.2309787461910e-01 1.8573185927533e-01
8.6749112665499e-01 5.2581078689114e-01 1.7435967364379e-01
5.2440337038023e-01 5.2852394281926e-01 1.6298724545380e-01
1.8142990597350e-01 5.3123724730045e-01 1.5161468153838e-01
8.3857318472438e-01 5.3395061424828e-01 1.4024208283479e-01
4.9583458106635e-01 5.3666396763326e-01 1.2886954223235e-01
1.5321438955324e-01 5.3937724198670e-01 1.1749714303886e-01
8.1071185002744e-01 5.4209038242971e-01 1.0612495805274e-01
4.6832519611446e-01 5.4480334430975e-01 9.4753049187317e-02
1.2605172120885e-01 5.4751609253592e-01 8.3381467567031e-02
7.8388785584697e-01 5.5022860070444e-01 7.2010253999984e-02
4.4182925065070e-01 5.5294085009839e-01 6.0639439728362e-02
9.9870859697748e-02 5.5565282863380e-01 4.9269047362958e-02
7.5800702005540e-01 5.5836452981040e-01 3.7899091918978e-02
4.1623152415862e-01 5.6107595171090e-01 2.6529581884110e-02
7.4537682610366e-02 5.6378709607906e-01 1.5160520264633e-02
7.3291837576066e-01 5.6649796749598e-01 3.7919055693864e-03
3.8976655670108e-01 5.6921301603019e-01 9.9241850557823e-01
4.8270580492312e-02 5.7192337811409e-01 9.8105074948815e-01
7.0682531733444e-01 5.7463348969189e-01 9.6968341879241e-01
3.6542219069664e-01 5.7734336062934e-01 9.5831650183342e-01
2.4052260288204e-02 5.8005300112593e-01 9.4694998577936e-01
6.8270619546879e-01 5.8276242151202e-01 9.3558385692812e-01
3.4137423474542e-01 5.8547163213200e-01 9.2421810089023e-01
4.6130686219039e-05 5.8818064330929e-01 9.1285270264996e-01
6.5871107922389e-01 5.9088946539202e-01 9.0148764650099e-01
3.1735763188063e-01 5.9359810888091e-01 8.9012291584847e-01
9.7597358891508e-01 5.9630658464518e-01 8.7875849286420e-01
6.3454587075594e-01 5.9901490423708e-01 8.6739435797401e-01
2.9306036432605e-01 6.0172308032200e-01 8.5603048914721e-01
9.5150173995091e-01 6.0443112724886e-01 8.4466686094671e-01
6.0985323345434e-01 6.0713906179572e-01 8.3330344328327e-01
2.6809638674059e-01 6.0984690413827e-01 8.2194019979952e-01
9.2621073861619e-01 6.1255467910551e-01 8.1057708578683e-01
5.8417345580418e-01 6.1526241780809e-01 7.9921404551135e-01
2.4195889207627e-01 6.1797015975091e-01 7.8785100879438e-01
8.9953806129639e-01 6.2067795557356e-01 7.7648788665982e-01
5.5687800824352e-01 6.2338587059793e-01 7.6512456583252e-01
2.1394106005385e-01 6.2609398939697e-01 7.5376090185971e-01
8.7068394241406e-01 6.2880242162020e-01 7.4239671065912e-01
5.2705675105480e-01 6.3151130929080e-01 7.3103175842323e-01
1.8300178598384e-01 6.3422083567011e-01 7.1966575012274e-01
8.3845229312395e-01 6.3693123545368e-01 7.0829831752034e-01
4.9333123299601e-01 6.3964280531323e-01 6.9692900890400e-01
1.4755033754781e-01 6.4235591227478e-01 6.8555728509169e-01
8.0100996601986e-01 6.4507099458957e-01 6.7418253015571e-01
4.5360067403303e-01 6.4778854499641e-01 6.6280409105258e-01
1.0520796904370e-01 6.5050905946785e-01 6.5142136699720e-01
7.5572226993817e-01 6.5323292771726e-01 6.4003397255841e-01
4.0505603324417e-01 6.5596024247145e-01 6.2864198668789e-01
5.3168160888072e-02 6.5869052905479e-01 6.1724625441572e-01
7.0009095623020e-01 6.6142246384202e-01 6.0584861794862e-01
3.4594826827161e-01 6.6415374882560e-01 5.9445186026843e-01
9.9095160972489e-01 6.6688135478552e-01 5.8305917270960e-01
6.3537191150804e-01 6.6960221261981e-01 5.7167321963327e-01
2.7950351451111e-01 6.7231413538157e-01 5.6029523422923e-01
9.2364299799721e-01 6.7501653636545e-01 5.4892467309966e-01
5.6808704498543e-01 6.7771058498514e-01 5.3755962720898e-01
2.1313030761825e-01 6.8039873991737e-01 5.2619771718043e-01
8.5904423551375e-01 6.8308390652027e-01 5.1483696172829e-01
5.0338079723795e-01 4.5342064457193e-01 5.0387882829017e-01
1.6007567345234e-01 4.5611419587102e-01 4.9252114837620e-01
8.1669846340699e-01 4.5880773807230e-01 4.8116343340394e-01
4.7325153981002e-01 4.6150132007286e-01 4.6980567800674e-01
1.2973798763812e-01 4.6419499776199e-01 4.5844787231542e-01
7.8616161912157e-01 4.6688883431767e-01 4.4709000072467e-01
4.4252696278297e-01 4.6958290016103e-01 4.3573204076989e-01
9.8839225473420e-02 4.7227727250948e-01 4.2437396220634e-01
7.5510422867173e-01 4.7497203448899e-01 4.1301572638717e-01
4.1132832280263e-01 4.7766727379305e-01 4.0165728603267e-01
6.7518285658965e-02 4.8036308090724e-01 3.9029858546907e-01
7.2368121281581e-01 4.8305954695106e-01 3.7893956139190e-01
3.7982440887923e-01 4.8575676122010e-01 3.6758014417648e-01
3.5955288297510e-02 4.8845480853702e-01 3.5622025972087e-01
6.9208129321975e-01 4.9115376653802e-01 3.4485983176670e-01
3.4820983363422e-01 4.9385370303036e-01 3.3349878460473e-01
4.3482520394158e-03 4.9655467355594e-01 3.2213704603867e-01
6.6050381159486e-01 4.9925671928737e-01 3.1077455045483e-01
3.1668370351490e-01 5.0195986536749e-01 2.9941124182842e-01
9.7289506683706e-01 5.0466411978221e-01 2.8804707649186e-01
6.2914501196750e-01 5.0736947283196e-01 2.7668202549586e-01
2.8544063880524e-01 5.1007589723815e-01 2.6531607641153e-01
9.4178904085479e-01 5.1278334889057e-01 2.5394923445092e-01
5.9819728847352e-01 5.1549176820783e-01 2.4258152282317e-01
2.5467238704135e-01 5.1820108204948e-01 2.3121298229187e-01
9.1122120903572e-01 5.2091120608582e-01 2.1984366995380e-01
5.6785040231564e-01 5.2362204750301e-01 2.0847365731352e-01
2.2456627991557e-01 5.2633350790105e-01 1.9710302777888e-01
8.8137469892396e-01 5.2904548623153e-01 1.8573187374171e-01
5.3828093728920e-01 5.3175788162515e-01 1.7436029343226e-01
1.9528957753746e-01 5.3447059597314e-01 1.6298838774227e-01
8.5240440546225e-01 5.3718353615333e-01 1.5161625719922e-01
5.0962833007168e-01 5.3989661582440e-01 1.4024399924671e-01
1.6696332877889e-01 5.4260975674926e-01 1.2887170594655e-01
8.2441041935432e-01 5.4532288964411e-01 1.1749946217341e-01
4.8196965785303e-01 5.4803595458113e-01 1.0612734432859e-01
1.3964015983550e-01 5.5074890099666e-01 9.4755419559673e-02
7.9742014085524e-01 5.5346168737216e-01 8.3383745441778e-02
4.5530697142903e-01 5.5617428066252e-01 7.2012370054712e-02
1.1329724148346e-01 5.5888665554601e-01 6.0641332378647e-02
7.7138682948028e-01 5.6159879356483e-01 4.9270662927997e-02
4.2957097193021e-01 5.6431068221579e-01 3.7900384546786e-02
8.7844329681507e-02 5.6702231403984e-01 2.6530513297020e-02
7.4620104810194e-01 5.6973368574760e-01 1.5161059382330e-02
4.0463480897592e-01 5.7244479740744e-01 3.7920280607831e-03
6.1544626953617e-02 5.7516008108765e-01 9.9241820972110e-01
7.2010879128485e-01 5.7787069915761e-01 9.8105000502874e-01
3.7872841681790e-01 5.8058106894620e-01 9.6968221886762e-01
3.7395639054805e-02 5.8329119753818e-01 9.5831484460645e-01
6.9610222490977e-01 5.8600109262647e-01 9.4694787416713e-01
3.5483955473154e-01 5.8871076222936e-01 9.3558129843268e-01
1.3598590920629e-02 5.9142021448796e-01 9.2421510755067e-01
6.7236983240589e-01 5.9412945753968e-01 9.1284929113134e-01
3.3114325393402e-01 5.9683849946503e-01 9.0148383833825e-01
9.8990822871820e-01 5.9954734830720e-01 8.9011873786642e-01
6.4865343248026e-01 6.0225601216681e-01 8.7875397779868e-01
3.0736672628821e-01 6.0496449937809e-01 8.6738954532557e-01
9.6603501481300e-01 6.0767281877761e-01 8.5602542630563e-01
6.2464407567144e-01 6.1038098008361e-01 8.4466160463354e-01
2.8317835434160e-01 6.1308899441171e-01 8.3329806136939e-01
9.4162071767479e-01 6.1579687496507e-01 8.2193477356544e-01
5.9995215721523e-01 6.1850463795130e-01 8.1057171270304e-01
2.5815143129198e-01 6.2121230379929e-01 7.9920884262304e-01
9.1619463208020e-01 6.2391989877619e-01 7.8784611679382e-01
5.7405466046460e-01 6.2662745714101e-01 7.7648347471259e-01
2.3170058755378e-01 6.2933502401909e-01 7.6512083717553e-01
8.8909687721251e-01 6.3204265924210e-01 7.5375810008421e-01
5.4620243945913e-01 6.3475044247153e-01 7.4239512638659e-01
2.0296948121787e-01 6.3745848000158e-01 7.3103173570438e-01
8.5934212147202e-01 6.4016691369806e-01 7.1966769122539e-01
5.1525474824784e-01 6.4287593251593e-01 7.0830268364721e-01
1.7063012722957e-01 6.4558578682036e-01 6.9693631256275e-01
8.2537735017322e-01 6.4829680504211e-01 6.8556806708450e-01
4.7938988064008e-01 6.5100941049330e-01 6.7419731042336e-01
1.3254429236574e-01 6.5372413249450e-01 6.6282327867320e-01
7.8470091430833e-01 6.5644159886451e-01 6.5144511347827e-01
4.3570859487349e-01 6.5916248490590e-01 6.4006196170173e-01
8.5417050484291e-02 6.6188737858748e-01 6.2867318745926e-01
7.3370087945686e-01 6.6461651389401e-01 6.1727873364095e-01
3.8049689815537e-01 6.6734935640954e-01 6.0587960025144e-01
2.5847713059911e-02 6.7008414530843e-01 5.9447823696209e-01
6.6993078169863e-01 6.7281769762159e-01 5.8307844648350e-01
3.1304823634906e-01 6.7554587533612e-01 5.7168445037484e-01
9.5557690858186e-01 6.7826480978782e-01 5.6029933773734e-01
5.9791497170989e-01 6.8097235528882e-01 5.4892381453961e-01
2.4046084610794e-01 6.8366892672196e-01 5.3755613599683e-01
8.8361291332825e-01 6.8635723105834e-01 5.2619323425925e-01
5.2774842136351e-01 6.8904108859392e-01 5.1483221906674e-01
1.7001690592191e-01 4.5937898340988e-01 5.0386690852141e-01
8.2678163756536e-01 4.6207256174691e-01 4.9250927879773e-01
4.8348459805290e-01 4.6476613643642e-01 4.8115161739104e-01
1.4012794087310e-01 4.6745975358865e-01 4.6979391898036e-01
7.9671442058247e-01 4.7015346523402e-01 4.5843617437343e-01
4.5324740985820e-01 4.7284732963034e-01 4.4707836940264e-01
1.0973089505329e-01 4.7554141128704e-01 4.3572048391268e-01
7.6616944912107e-01 4.7823578065531e-01 4.2436249091478e-01
4.2256818261132e-01 4.8093051344886e-01 4.1300435598642e-01
7.8932675435617e-02 4.8362568958045e-01 4.0164603699304e-01
7.3526889401113e-01 4.8632139172472e-01 3.9028748419814e-01
3.9158309994613e-01 4.8901770354376e-01 3.7892864081091e-01
4.7881757371443e-02 4.9171470763766e-01 3.6756944399620e-01
7.0417144616323e-01 4.9441248330386e-01 3.5620982634266e-01
3.6045878756620e-01 4.9711110420581e-01 3.4484971775309e-01
1.6750387164862e-02 4.9981063606032e-01 3.3348904769029e-01
6.7305279794310e-01 5.0251113445557e-01 3.2212774768306e-01
3.2937250359587e-01 5.0521264290670e-01 3.1076575397434e-01
9.8571591964324e-01 5.0791519124587e-01 2.9940301017711e-01
6.4208940758092e-01 5.1061879442831e-01 2.8803946979566e-01
2.9849929557744e-01 5.1332345181786e-01 2.7667509847003e-01
9.5495189831008e-01 5.1602914699370e-01 2.6530987581121e-01
6.1145352853371e-01 5.1873584809649e-01 2.5394379671293e-01
2.6801049389996e-01 5.2144350870656e-01 2.4257687205347e-01
9.2462907427394e-01 5.2415206921977e-01 2.3120912873563e-01
5.8131547712103e-01 5.2686145865994e-01 2.1984060905443e-01
2.3807577116113e-01 5.2957159684191e-01 2.0847136942537e-01
8.9491580108019e-01 5.3228239677882e-01 1.9710147854945e-01
5.5184108831605e-01 5.3499376721294e-01 1.8573101512848e-01
2.0885672452152e-01 5.3770561514481e-01 1.7436006527248e-01
8.6596726507297e-01 5.4041784823967e-01 1.6298871975639e-01
5.2317662988406e-01 5.4313037700448e-01 1.5161707128480e-01
1.8048801787882e-01 5.4584311665108e-01 1.4024521191058e-01
8.3790383996051e-01 5.4855598858827e-01 1.2887323072923e-01
4.9542567343614e-01 5.5126892151522e-01 1.1750121193839e-01
1.5305423889131e-01 5.5398185211684e-01 1.0612923331629e-01
8.1078939869681e-01 5.5669472538616e-01 9.4757365137529e-02
4.6863017484720e-01 5.5940749461699e-01 8.3385669513686e-02
1.2657478277949e-01 5.6212012112268e-01 7.2014200121793e-02
7.8462067722216e-01 5.6483257374171e-01 6.0643002266737e-02
4.4276460593996e-01 5.6754482819124e-01 4.9272113214535e-02
1.0100266738887e-01 5.7025686632523e-01 3.7901562730737e-02
7.5933036867906e-01 5.7296867534655e-01 2.6531373760902e-02
4.1774268076301e-01 5.7568024701394e-01 1.5161563196345e-02
7.6234088334017e-02 5.7839157687527e-01 3.7921426767893e-03
7.3321013776503e-01 5.8110707766453e-01 9.9241792641265e-01
3.9183814893465e-01 5.8381794012913e-01 9.8104928337970e-01
5.0525963397523e-02 5.8652856161399e-01 9.6968104315580e-01
7.0926645470672e-01 5.8923894633910e-01 9.5831320395389e-01
3.6805212630671e-01 5.9194909944285e-01 9.4694576222892e-01
2.6875101745615e-02 5.9465902663616e-01 9.3557871316062e-01
6.8572710233557e-01 5.9736873392992e-01 9.2421205104994e-01
3.4459941145812e-01 6.0007822743179e-01 9.1284576962780e-01
3.4828244582236e-03 6.0278751320938e-01 9.0147986227477e-01
6.6236758271312e-01 6.0549659721741e-01 8.9011432214909e-01
3.2124328999195e-01 6.0820548528898e-01 8.7874914221781e-01
9.8009880862247e-01 6.1091418319304e-01 8.6738431518199e-01
6.3892213222482e-01 6.1362269676431e-01 8.5601983328101e-01
2.9770023082209e-01 6.1633103211629e-01 8.4465568795346e-01
9.5641886294158e-01 6.1903919595479e-01 8.3329186932092e-01
6.1506234780757e-01 6.2174719601787e-01 8.2192836544690e-01
2.7361328880432e-01 6.2445504168033e-01 8.1056516130269e-01
9.3205223693360e-01 6.2716274477748e-01 7.9920223734535e-01
5.9035727984795e-01 6.2987032072620e-01 7.8783956757545e-01
2.4850353801242e-01 6.3257779005421e-01 7.7647711689206e-01
9.0646254439639e-01 6.3528518049402e-01 7.6511483749403e-01
5.6420147756898e-01 6.3799252986267e-01 7.5375266398547e-01
2.2168220994728e-01 6.4069989003668e-01 7.4239050672503e-01
8.7886012321019e-01 6.4340733245148e-01 7.3102824281197e-01
5.3568263210700e-01 6.4611495570876e-01 7.1966570393709e-01
1.9208734808839e-01 6.4882289605634e-01 7.0830266018155e-01
8.4799981082621e-01 6.5153134167284e-01 6.9693879882252e-01
5.0333073190848e-01 6.5424055172316e-01 6.8557369754312e-01
1.5797276053394e-01 6.5695088073889e-01 6.7420679267464e-01
8.1179695979015e-01 6.5966280730307e-01 6.6283734630426e-01
4.6464960280945e-01 6.6237696174869e-01 6.5146442329182e-01
1.1635079294855e-01 6.6509413772905e-01 6.4008690368365e-01
7.6669810790706e-01 6.6781525281052e-01 6.2870358096314e-01
4.1548110860250e-01 6.7054119131227e-01 6.1731342876007e-01
6.2515012784154e-02 6.7327243206629e-01 6.0591612847380e-01
7.0769941684896e-01 6.7600839607521e-01 5.9451285035490e-01
3.5109193641720e-01 6.7874666793980e-01 5.8310695078685e-01
9.9295689691617e-01 6.8148268068511e-01 5.7170379383852e-01
6.3374028696015e-01 6.8421066221768e-01 5.6030902251394e-01
2.7397948302474e-01 6.8692588334147e-01 5.4892592728206e-01
9.1422790533658e-01 6.8962689743837e-01 5.3755390394744e-01
5.5503907855310e-01 6.9231616053424e-01 5.2618931316434e-01
1.9695069240293e-01 6.9499862710433e-01 5.1482791437408e-01
8.3665638073954e-01 4.6533736983954e-01 5.0385489908229e-01
4.9348068511287e-01 4.6803097034735e-01 4.9249731252290e-01
1.5025317901981e-01 4.7072457211116e-01 4.8113969800113e-01
8.0697582479577e-01 4.7341821885919e-01 4.6978205024882e-01
4.6365108603016e-01 4.7611195928209e-01 4.5842436067206e-01
1.2028194652615e-01 4.7880584735062e-01 4.4706661635317e-01
7.7687191143220e-01 4.8149994239818e-01 4.3570879913074e-01
4.3342498936640e-01 4.8419430892401e-01 4.2435088481915e-01
8.9945655818790e-02 4.8688901608493e-01 4.1299284263227e-01
7.4643879972735e-01 4.8958413686022e-01 4.0163463487466e-01
4.0290965669987e-01 4.9227974689373e-01 3.9027621695665e-01
5.9363733686922e-02 4.9497592303886e-01 3.7891753777651e-01
7.1580673080192e-01 4.9767274165273e-01 3.6755854049452e-01
3.7224446627455e-01 5.0037027670462e-01 3.5619916370140e-01
2.8682810134246e-02 5.0306859777800e-01 3.4483934295836e-01
6.8512763116187e-01 5.0576776805497e-01 3.3347901266094e-01
3.4158476003093e-01 5.0846784237546e-01 3.2211810815535e-01
9.9805996957369e-01 5.1116886546146e-01 3.1075656801571e-01
6.5455897099454e-01 5.1387087039017e-01 2.9939433637597e-01
3.1108742287100e-01 5.1657387738914e-01 2.8803136520062e-01
9.6765094817137e-01 5.1927789301281e-01 2.7666761637617e-01
6.2425515346883e-01 5.2198290974382e-01 2.6530306350926e-01
2.8090564416858e-01 5.2468890604431e-01 2.5393769332874e-01
9.3760802992941e-01 5.2739584686255e-01 2.4257150660756e-01
5.9436791551252e-01 5.3010368457939e-01 2.3120451854499e-01
2.5119087390707e-01 5.3281236035714e-01 2.1983675858016e-01
9.0808240057121e-01 5.3552180583248e-01 2.0846826964191e-01
5.6504784975335e-01 5.3823194507615e-01 1.9709910687531e-01
2.2209235586055e-01 5.4094269672737e-01 1.8572933591847e-01
8.7922074448121e-01 5.4365397620177e-01 1.7435903083122e-01
5.3643743875544e-01 5.4636569786973e-01 1.6298827179721e-01
1.9374636721088e-01 5.4907777710808e-01 1.5161714273042e-01
8.5115087893002e-01 5.5179013214077e-01 1.4024572891536e-01
5.0865367106762e-01 5.5450268560327e-01 1.2887411479798e-01
1.6625673244815e-01 5.5721536578777e-01 1.1750238202330e-01
8.2396130544041e-01 5.5992810754988e-01 1.0613060778969e-01
4.8176786673443e-01 5.6264085288003e-01 9.4758863560319e-02
1.3967612621093e-01 5.6535355116161e-01 8.3387214144723e-02
7.9768504192723e-01 5.6806615915251e-01 7.2015717138890e-02
4.5579284841419e-01 5.7077864073608e-01 6.0644422692744e-02
1.1399709500260e-01 5.7349096649151e-01 4.9273373560813e-02
7.7229469074150e-01 5.7620311313424e-01 3.7902605384268e-02
4.3068195257594e-01 5.7891506287279e-01 2.6532147150387e-02
8.9154653740553e-02 5.8162680272363e-01 1.5162021777372e-02
7.4770806972366e-01 5.8433832381792e-01 3.7922467773214e-03
4.0475473597158e-01 5.8705401831261e-01 9.9241766127046e-01
6.3450087336067e-02 5.8976510784560e-01 9.8104859927770e-01
7.2220916289435e-01 5.9247596845795e-01 9.6967991629240e-01
3.8102558963306e-01 5.9518660144969e-01 9.5831161513017e-01
3.9892626660965e-02 5.9789700936553e-01 9.4694369652129e-01
6.9880316295468e-01 6.0060719560444e-01 9.3557615963740e-01
3.5774970371889e-01 6.0331716409395e-01 9.2420900255086e-01
1.6724344494913e-02 6.0602691902695e-01 9.1284222262416e-01
6.7571873196151e-01 6.0873646465771e-01 9.0147581682811e-01
3.3472401007640e-01 6.1144580515467e-01 8.9010978198766e-01
9.9373074979581e-01 6.1415494450775e-01 8.7874411495312e-01
6.5272886006674e-01 6.1686388649006e-01 8.6737881269281e-01
3.1170747708393e-01 6.1957263467561e-01 8.5601387229917e-01
9.7065482790497e-01 6.2228119251813e-01 8.4464929089551e-01
6.2955806336577e-01 6.2498956350021e-01 8.3328506542321e-01
2.8840305376049e-01 6.2769775136819e-01 8.2192119227825e-01
9.4717413884172e-01 6.3040576047646e-01 8.1055766675182e-01
6.0585382121600e-01 6.3311359627690e-01 7.9919448220846e-01
2.6442238896576e-01 6.3582126600633e-01 7.8783162890583e-01
9.2285744906014e-01 6.3852877964962e-01 7.7646909231807e-01
5.8113334747186e-01 6.4123615129241e-01 7.6510685076443e-01
2.3922044442608e-01 6.4394340103095e-01 7.5374487205898e-01
8.9708420325714e-01 6.4665055768495e-01 7.4238310877418e-01
5.5468403817818e-01 6.4935766267574e-01 7.3102149153817e-01
2.1197184902507e-01 6.5206477560136e-01 7.1965991954489e-01
8.6889014902756e-01 6.5477198228453e-01 7.0829824713440e-01
5.2536966508944e-01 6.5747940640859e-01 6.9693626489909e-01
1.8132626200170e-01 6.6018722629386e-01 6.8557367334627e-01
8.3665702339841e-01 6.6289569883875e-01 6.7421004690187e-01
4.9123534402847e-01 6.6560519288879e-01 6.6284478654546e-01
1.4490502941136e-01 6.6831623349672e-01 6.5147706204312e-01
7.9747385009686e-01 6.7102955457529e-01 6.4010575282387e-01
4.4870817680509e-01 6.7374614535000e-01 6.2872941657982e-01
9.8333020827478e-02 6.7646724589702e-01 6.1734635727615e-01
7.4604701718174e-01 6.7919418559767e-01 6.0595493667635e-01
3.9156895665120e-01 6.8192787190792e-01 5.9455434054290e-01
3.4732854603278e-02 6.8466773471123e-01 5.8314589464473e-01
6.7561918516905e-01 6.8741033210164e-01 5.7173437051817e-01
3.1464095811451e-01 6.9014882770373e-01 5.6032762366499e-01
9.5248243018950e-01 6.9287503428116e-01 5.4893319308464e-01
5.8993389591430e-01 6.9558367712919e-01 5.3755380056052e-01
2.2779498417411e-01 6.9827554692091e-01 5.2618612349504e-01
8.6685619884963e-01 7.0095672413974e-01 5.1482405234279e-01
5.0329911721493e-01 4.7129579664971e-01 5.0384281402776e-01
1.6017305660377e-01 4.7398941500452e-01 4.9248526366590e-01
8.1700485713235e-01 4.7668303907674e-01 4.8112768936031e-01
4.7379631427464e-01 4.7937671056409e-01 4.6977008591423e-01
1.3054963414178e-01 4.8207047526183e-01 4.5841244527988e-01
7.8726745431280e-01 4.8476438339186e-01 4.4705475564641e-01
4.4395284988701e-01 4.8745848973570e-01 4.3569700059630e-01
1.0060932354625e-01 4.9015285353276e-01 4.2433915838059e-01
7.5724077959517e-01 4.9284753811457e-01 4.1298120136657e-01
4.1385148326051e-01 4.9554261025931e-01 4.0162309571031e-01
7.0446007837850e-02 4.9823813926658e-01 3.9026480130153e-01
7.2702917341863e-01 5.0093419576976e-01 3.7890627201853e-01
3.8360598175314e-01 5.0363085032055e-01 3.6754745631695e-01
4.0181552177499e-02 5.0632817179592e-01 3.5618829815837e-01
6.9676106337690e-01 5.0902622569057e-01 3.4482873826533e-01
3.5334970506341e-01 5.1172507236668e-01 3.3346871566874e-01
9.9526424780943e-03 5.1442476533722e-01 3.2210816949422e-01
6.6657499511188e-01 5.1712534965904e-01 3.1074704091673e-01
3.2322182935300e-01 5.1982686050779e-01 2.9938527519915e-01
9.7989816310609e-01 5.2252932199959e-01 2.8802282372132e-01
6.3660897898258e-01 5.2523274631393e-01 2.7665964590159e-01
2.9335924159697e-01 5.2793713316051e-01 2.6529571091401e-01
9.5015391393968e-01 5.3064246961852e-01 2.5393099911069e-01
6.0699796779564e-01 5.3334873036170e-01 2.4256550307090e-01
2.6389638374079e-01 5.3605587826608e-01 2.3119922821595e-01
9.2085413731323e-01 5.3876386537989e-01 2.1983219295086e-01
5.7787616940506e-01 5.4147263421770e-01 2.0846442831998e-01
2.3496734059071e-01 5.4418211932452e-01 1.9709597719175e-01
8.9213237080740e-01 5.4689224904115e-01 1.8572689301599e-01
5.4937576733737e-01 5.4960294739173e-01 1.7435723822276e-01
2.0670174523637e-01 5.5231413600859e-01 1.6298708235282e-01
8.6411414507996e-01 5.5502573601004e-01 1.5161650002325e-01
5.2161635309546e-01 5.5773766975261e-01 1.4024556883721e-01
1.7921122842244e-01 5.6044986239131e-01 1.2887436734352e-01
8.3690104147340e-01 5.6316224319720e-01 1.1750297314000e-01
4.9468742627727e-01 5.6587474659992e-01 1.0613146119691e-01
1.5257134843492e-01 5.6858731294227e-01 9.4759902454916e-02
8.1055308905220e-01 5.7129988895103e-01 8.3388362728649e-02
4.6863224387469e-01 5.7401242794378e-01 7.2016901924819e-02
1.2680773592309e-01 5.7672488980255e-01 6.0645573564018e-02
7.8507783926990e-01 5.7943724075255e-01 4.9274424580373e-02
4.4344021121539e-01 5.8214945298744e-01 3.7903495362334e-02
1.0189192998837e-01 5.8486150418290e-01 2.6532819992008e-02
7.6042953516910e-01 5.8757337693720e-01 1.5162426638531e-02
4.1904906824906e-01 5.9028505817331e-01 3.7923380623549e-03
7.6170498719681e-02 5.9300091831849e-01 9.9241741927654e-01
7.3493644870647e-01 5.9571221248760e-01 9.8104796598905e-01
3.9376961415814e-01 5.9842329415200e-01 9.6967886075929e-01
5.2664379807582e-02 6.0113416167122e-01 9.5831011071395e-01
7.1161476878980e-01 6.0384481497235e-01 9.4694172057422e-01
3.7061444707176e-01 6.0655525513344e-01 9.3557369319589e-01
2.9656717926777e-02 6.0926548402238e-01 9.2420603006117e-01
6.8873450553572e-01 6.1197550399032e-01 9.1283873171785e-01
3.4784032667287e-01 6.1468531761786e-01 9.0147179816386e-01
6.9662492072550e-03 6.1739492751110e-01 8.9010522917625e-01
6.6610383581176e-01 6.2010433614518e-01 8.7873902458463e-01
3.2524407080523e-01 6.2281354575296e-01 8.6737318448796e-01
9.8437726743195e-01 6.2552255825804e-01 8.5600770941261e-01
6.4349295207200e-01 6.2823137525265e-01 8.4464260040673e-01
3.0257972082121e-01 6.3093999802363e-01 8.3327785906157e-01
9.6162506250752e-01 6.3364842763348e-01 8.2191348744448e-01
6.2061514041635e-01 6.3635666506811e-01 8.1054948791875e-01
2.7953452263651e-01 6.3906471147092e-01 7.9918586281234e-01
9.3836584780675e-01 6.4177256849306e-01 7.8782261387753e-01
5.9708940885735e-01 6.4448023880636e-01 7.7645974145469e-01
2.5568263169946e-01 6.4718772684897e-01 7.6509724320992e-01
9.1411941815178e-01 6.4989503991101e-01 7.5373511225175e-01
5.7236931190712e-01 6.5260218972331e-01 7.4237333433483e-01
2.3039643189440e-01 6.5530919480011e-01 7.3101188371116e-01
8.8815809739305e-01 6.5801608392216e-01 7.1965071696767e-01
5.4560304151541e-01 6.6072290136003e-01 7.0828976385325e-01
2.0266907132468e-01 6.6342971477024e-01 6.9692891359875e-01
8.5927998058614e-01 6.6613662721259e-01 6.8556799450498e-01
5.1534145244354e-01 6.6884379551711e-01 6.7420674356423e-01
1.7073560681943e-01 6.7155145834302e-01 6.6284476163307e-01
8.2531377212034e-01 6.7425997865977e-01 6.5148144857505e-01
4.7888707012793e-01 6.7696990644656e-01 6.4011591327272e-01
1.3121473239698e-01 6.7968206583561e-01 6.2874685971011e-01
7.8199136564030e-01 6.8239765952927e-01 6.1737247352255e-01
4.3083825095918e-01 6.8511834307371e-01 6.0599039826568e-01
7.7313330689974e-02 6.8784611296141e-01 5.9459803651019e-01
7.2097240480365e-01 6.9058263884789e-01 5.8319363171783e-01
3.6152739069271e-01 6.9332750983099e-01 5.7177855675242e-01
9.9909564680189e-01 6.9607557693506e-01 5.6035991864939e-01
6.3436231716481e-01 6.9881607156896e-01 5.4894973496762e-01
2.6842686289458e-01 7.0153736087362e-01 5.3755777145560e-01
9.0250170296753e-01 7.0423516872460e-01 5.2618409620985e-01
5.3779936125642e-01 7.0691567418343e-01 5.1482064576043e-01
1.6994503178151e-01 4.7725425672496e-01 5.0383066728828e-01
8.2685894875862e-01 4.7994788904760e-01 4.9247314620873e-01
4.8374016905523e-01 4.8264153120849e-01 4.8111560545620e-01
1.4059034243123e-01 4.8533522318258e-01 4.6975803993382e-01
7.9741144599595e-01 4.8802900825207e-01 4.5840044210999e-01
4.5420580697682e-01 4.9072293334874e-01 4.4704280117496e-01
1.1097611326597e-01 4.9341704923254e-01 4.3568510225916e-01
7.6772541050023e-01 4.9611141047159e-01 4.2432732575538e-01
4.2445708539352e-01 4.9880607519726e-01 4.1296944678885e-01
8.1174836122585e-02 5.0150110461821e-01 4.0161143487887e-01
7.3788263166194e-01 5.0419656229082e-01 3.9025325383183e-01
3.9458466294897e-01 5.0689251315699e-01 3.7889486189828e-01
5.1285289511844e-02 5.0958902237512e-01 3.6753621221572e-01
7.0798898560426e-01 5.1228615398315e-01 3.5617725354525e-01
3.6470028989203e-01 5.1498396944363e-01 3.4481793129481e-01
2.1423762304954e-02 5.1768252612924e-01 3.3345818880487e-01
6.7816395083745e-01 5.2038187581162e-01 3.2209796885665e-01
3.3492536993741e-01 5.2308206321783e-01 3.1073721534830e-01
9.9171249079084e-01 5.2578312471616e-01 2.9937587507216e-01
6.4852974243633e-01 5.2848508718859e-01 2.8801389951774e-01
3.0538152138233e-01 5.3118796713916e-01 2.7665124661958e-01
9.6227220638687e-01 5.3389177007895e-01 2.6528788236824e-01
6.1920617440429e-01 5.3659649021721e-01 2.5392378220594e-01
2.7618781347465e-01 5.3930211047662e-01 2.4255893213588e-01
9.3322152855402e-01 5.4200860283766e-01 2.3119332948621e-01
5.9031173693445e-01 5.4471592900319e-01 2.1982698328554e-01
2.4746285091284e-01 5.4742404136041e-01 2.0845991422650e-01
9.0467924663184e-01 5.5013288420316e-01 1.9709215421547e-01
5.6196521939582e-01 5.5284239516497e-01 1.8572374553011e-01
2.1932492711116e-01 5.5555250680244e-01 1.7435473962821e-01
8.7676232466562e-01 5.5826314826119e-01 1.6298519567164e-01
5.3428109292162e-01 5.6097424695356e-01 1.5161517884393e-01
1.9188456646917e-01 5.6368573017868e-01 1.4024475855000e-01
8.4957566433141e-01 5.6639752662177e-01 1.2887400658837e-01
5.0735682746134e-01 5.6910956767984e-01 1.1750299538251e-01
1.6522996618000e-01 5.7182178857461e-01 1.0613179634703e-01
8.2319641978859e-01 5.7453412922828e-01 9.4760478449470e-02
4.8125692956020e-01 5.7724653489330e-01 8.3389107010539e-02
1.3941162529667e-01 5.7995895654163e-01 7.2017742766514e-02
7.9766002472336e-01 5.8267135103034e-01 6.0646441199799e-02
4.5600104425586e-01 5.8538368107006e-01 4.9275252127995e-02
1.1443301914717e-01 5.8809591502786e-01 3.7904219529613e-02
7.7295373071309e-01 5.9080802659943e-01 2.6533381576033e-02
4.3156043822141e-01 5.9351999438487e-01 1.5162770834478e-02
9.0249913077342e-02 5.9623180140073e-01 3.7924146051414e-03
7.4744999232480e-01 5.9894779527641e-01 9.9241720469474e-01
4.0628954613617e-01 6.0165926726365e-01 9.8104739506374e-01
6.5199375424809e-02 6.0437054705503e-01 9.6967789642931e-01
7.2417461588439e-01 6.0708163008915e-01 9.5830871989684e-01
3.8321005206245e-01 6.0979251367322e-01 9.4693987387538e-01
4.2300127480700e-02 6.1250319655825e-01 9.3557136460305e-01
7.0143894642808e-01 6.1521367855932e-01 9.2420319664252e-01
3.6062026647657e-01 6.1792396022226e-01 9.1283537334026e-01
1.9837480707007e-02 6.2063404253611e-01 9.0146789724612e-01
6.7908358847668e-01 6.2334392668964e-01 8.9010077049194e-01
3.3835115330221e-01 6.2605361386946e-01 8.7873399512932e-01
9.9763224605410e-01 6.2876310509700e-01 8.6736757342765e-01
6.5691837114116e-01 6.3147240110178e-01 8.5600150813341e-01
3.1620037267029e-01 6.3418150222923e-01 8.4463580269110e-01
9.7546831665155e-01 6.3689040838206e-01 8.3327046142477e-01
6.3471134411203e-01 6.3959911899642e-01 8.2190548967592e-01
2.9391748838710e-01 6.4230763305610e-01 8.1054089388966e-01
9.5307344773701e-01 6.4501594915243e-01 7.9917668163365e-01
6.1216430159481e-01 6.4772406560261e-01 7.8781286152436e-01
2.7117315490903e-01 6.5043198064839e-01 7.7644944301885e-01
9.3008068979970e-01 6.5313969276969e-01 7.6508643600562e-01
5.8886459651570e-01 6.5584720116887e-01 7.5372385008952e-01
2.4749884561152e-01 6.5855450651375e-01 7.4236169340515e-01
9.0595274909049e-01 6.6126161208006e-01 7.3099997069697e-01
5.6418973811064e-01 6.6396852551925e-01 7.1963868025107e-01
2.2216575590038e-01 6.6667526161832e-01 7.0827780901580e-01
8.7982712254404e-01 6.6938184665219e-01 6.9691732484723e-01
5.3710766691374e-01 6.7208832532293e-01 6.8555716416262e-01
1.9392483088131e-01 6.7479477194630e-01 6.7419721222465e-01
8.5017431892924e-01 6.7750130867333e-01 6.6283727157815e-01
5.0572267651820e-01 6.8020813541811e-01 6.5147701151736e-01
1.6039692289516e-01 6.8291557919105e-01 6.4011588766053e-01
8.1397006993418e-01 6.8562417492497e-01 6.2875301630338e-01
4.6614123040319e-01 6.8833479439982e-01 6.1738698695130e-01
1.1650985707922e-01 6.9104883703201e-01 6.0601561275873e-01
7.6454813452827e-01 6.9376845678364e-01 5.9463569810581e-01
4.0959122782063e-01 6.9649663302019e-01 5.8324315978688e-01
5.0906397109480e-02 6.9923641557221e-01 5.7183442752411e-01
6.8796195232312e-01 7.0198795524603e-01 5.6041053514806e-01
3.2093968617178e-01 7.0474303792071e-01 5.4898273047971e-01
9.5106635454623e-01 7.0748364801501e-01 5.3757031299716e-01
5.8021417765276e-01 7.1019401695952e-01 5.2618442996805e-01
2.1036232083222e-01 7.1287589026752e-01 5.1481774895487e-01
8.3659406009659e-01 4.8321274303454e-01 5.0381847268787e-01
4.9353851831456e-01 4.8590638579902e-01 4.9246097401760e-01
1.5045954784968e-01 4.8860004228164e-01 4.8110346015237e-01
8.0735867125826e-01 4.9129375100643e-01 4.6974592613239e-01
4.6423766086763e-01 4.9398755307301e-01 4.5838836493138e-01
1.2109856363992e-01 4.9668149251518e-01 4.4703076666632e-01
7.7794371597608e-01 4.9937561652208e-01 4.3567311786508e-01
4.3477574717345e-01 5.0206997549081e-01 4.2431540082698e-01
9.1597571050405e-02 5.0476462288577e-01 4.1295759311281e-01
7.4841236615616e-01 5.0745961488912e-01 4.0159966720050e-01
4.0522354591152e-01 5.1015500983735e-01 3.9024159033983e-01
6.2034720880969e-02 5.1285086745093e-01 3.7888332463392e-01
7.1884965606023e-01 5.1554724787564e-01 3.6752482736712e-01
3.7567222648852e-01 5.1824421056594e-01 3.5616605158795e-01
3.2506374599182e-02 5.2094181305006e-01 3.4480694694386e-01
6.8935607247953e-01 5.2364010962427e-01 3.3344746075085e-01
3.4622529163495e-01 5.2633915002851e-01 3.2208753926816e-01
3.1179819992984e-03 5.2903897815731e-01 3.1072712913556e-01
6.6003806089154e-01 5.3173963085934e-01 2.9936617892061e-01
3.1698941149868e-01 5.3444113687524e-01 2.8800464071476e-01
9.7397588938590e-01 5.3714351595861e-01 2.7664247171194e-01
6.3100133460943e-01 5.3984677821757e-01 2.6527963570107e-01
2.8806958633143e-01 5.4255092370645e-01 2.5391610440499e-01
9.4518449647687e-01 5.4525594228778e-01 2.4255185860277e-01
6.0234993896734e-01 5.4796181377472e-01 2.3118688898051e-01
2.5956981141482e-01 5.5066850835286e-01 2.1982119666683e-01
9.1684802682708e-01 5.5337598726941e-01 2.0845479342396e-01
5.7418849379686e-01 5.5608420376567e-01 1.9708770148176e-01
2.3159508472699e-01 5.5879310421801e-01 1.8571995302086e-01
8.8907159277057e-01 5.6150262944232e-01 1.7435158932948e-01
5.4662167922266e-01 5.6421271610937e-01 1.6298265967648e-01
2.0424881397793e-01 5.6692329821314e-01 1.5161321995813e-01
8.6195621227932e-01 5.6963430853315e-01 1.4024333118728e-01
5.1974677127250e-01 5.7234568003356e-01 1.2887305789979e-01
1.7762300983381e-01 5.7505734714837e-01 1.1750246655372e-01
8.3558701478316e-01 5.7776924691057e-01 1.0613162399198e-01
4.9364039598672e-01 5.8048131989472e-01 9.4760596030051e-02
1.5178425208270e-01 5.8319351095492e-01 8.3389446217117e-02
8.1001914772014e-01 5.8590576975217e-01 7.2018234804485e-02
4.6834510237454e-01 5.8861805107695e-01 6.0647017939490e-02
1.2676159006984e-01 5.9133031498199e-01 4.9275847088823e-02
7.8526754874414e-01 5.9404252674752e-01 3.7904768682638e-02
4.4386139757345e-01 5.9675465670560e-01 2.6533823960978e-02
1.0254106031550e-01 5.9946667995258e-01 1.5163048997226e-02
7.6130399263908e-01 6.0217857597847e-01 3.7924748692970e-03
4.1858632250209e-01 6.0489466862191e-01 9.9241702102066e-01
7.7502218309662e-02 6.0760628800100e-01 9.8104689617141e-01
7.3649102341407e-01 6.1031773887752e-01 9.6967704024236e-01
3.9554860697536e-01 6.1302901383255e-01 9.5830746795319e-01
5.4670501569725e-02 6.1574010757916e-01 9.4693819108527e-01
7.1385191791568e-01 6.1845101654578e-01 9.3556921896799e-01
3.7308775302362e-01 6.2116173849369e-01 9.2420055895234e-01
3.2372590810613e-02 6.2387227217210e-01 9.1283221686364e-01
6.9170069421441e-01 6.2658261701195e-01 9.0146419742790e-01
3.5106598775498e-01 6.2929277285812e-01 8.9009650466925e-01
1.0462029315454e-02 6.3200273973847e-01 8.7872914227820e-01
6.6988196963646e-01 6.3471251766711e-01 8.6736211395162e-01
3.2931849761506e-01 6.3742210647925e-01 8.5599542370720e-01
9.8876376894762e-01 6.4013150569454e-01 8.4462907617509e-01
6.4820931491104e-01 6.4284071440611e-01 8.3326307687046e-01
3.0764592708207e-01 6.4554973119300e-01 8.2189743244964e-01
9.6706351246607e-01 6.4825855405471e-01 8.1053215095207e-01
6.2645091172310e-01 6.5096718036773e-01 7.9916724202793e-01
2.8579567077024e-01 6.5367560686644e-01 7.8780271714787e-01
9.4508375275172e-01 6.5638382965412e-01 7.7643858978552e-01
6.0429917284045e-01 6.5909184425517e-01 7.6507487555327e-01
2.6342353202410e-01 6.6179964572876e-01 7.5371159225584e-01
9.2243541713467e-01 6.6450722887844e-01 7.4234875979871e-01
5.8130962168504e-01 6.6721458861628e-01 7.3098639984313e-01
2.4001612372027e-01 6.6992172058079e-01 7.1962453502231e-01
8.9851872998709e-01 6.7262862217781e-01 7.0826318740181e-01
5.5677325572672e-01 6.7533529433502e-01 6.9690237563977e-01
2.1472504904851e-01 6.7804174447726e-01 6.8554210990560e-01
8.7230557641455e-01 6.8074799162004e-01 6.7418238291287e-01
5.2942764210059e-01 6.8345407519631e-01 6.6282315416653e-01
1.8597858818871e-01 6.8616007057071e-01 6.5146432226242e-01
8.4181046180822e-01 6.8886611672323e-01 6.4010567599836e-01
4.9672556466249e-01 6.9157246636731e-01 6.2874680780052e-01
1.5045491592773e-01 6.9427957764754e-01 6.1738696065509e-01
8.0262592979307e-01 6.9698828190708e-01 6.0602476158068e-01
4.5271455262224e-01 6.9970008259552e-01 5.9465778013158e-01
9.9979467202683e-02 7.0241763802025e-01 5.8328189813970e-01
7.4339606740697e-01 7.0514530002271e-01 5.7189082880594e-01
3.8169312684108e-01 7.0788865267836e-01 5.6047746302911e-01
1.3807782804824e-02 7.1064950507993e-01 5.4904126863257e-01
6.4004837061533e-01 7.1341353511097e-01 5.3760156543665e-01
2.6284284967177e-01 7.1614856238559e-01 5.2619078028744e-01
8.8564841352825e-01 7.1883783506983e-01 5.1481560918686e-01
5.0324615575534e-01 4.8917124862051e-01 5.0380624396485e-01
1.6021188497914e-01 4.9186489856832e-01 4.9244876086242e-01
8.1716333128045e-01 4.9455856597188e-01 4.8109126720622e-01
4.7410190629488e-01 4.9725228814689e-01 4.6973375821853e-01
1.3102919812842e-01 4.9994610429782e-01 4.5837622738404e-01
7.8794699855790e-01 5.0264005589611e-01 4.4701866569895e-01
4.4485732201880e-01 5.0533418694330e-01 4.3566106097636e-01
1.0176241503592e-01 5.0802854408984e-01 4.2430339723714e-01
7.5866475543011e-01 5.1072317658672e-01 4.1294565421674e-01
4.1556704140463e-01 5.1341813605441e-01 4.0158780701781e-01
7.2472171413358e-02 5.1611347606261e-01 3.9022982593643e-01
7.2938321646316e-01 5.1880925152454e-01 3.7887167648530e-01
3.8630338712611e-01 5.2150551791926e-01 3.6751331963097e-01
4.3235997961685e-02 5.2420233036532e-01 3.5615471225374e-01
7.0018443222199e-01 5.2689974257787e-01 3.4479580782930e-01
3.5715210960474e-01 5.2959780574757e-01 3.3343655732034e-01
1.4142459434271e-02 5.3229656738473e-01 3.2207691025553e-01
6.7115890102315e-01 5.3499607017439e-01 3.1071681596324e-01
3.2820483216046e-01 5.3769635088769e-01 2.9935622491792e-01
9.8528362576812e-01 5.4039743939348e-01 2.8799509015000e-01
6.4239863385794e-01 5.4309935780973e-01 2.7663336866470e-01
2.9955319709864e-01 5.4580211982958e-01 2.6527102281248e-01
9.5675065764827e-01 5.4850573025018e-01 2.5390802155344e-01
6.1399437248344e-01 5.5121018472549e-01 2.4254434156057e-01
2.7128772430629e-01 5.5391546975610e-01 2.3117996811185e-01
9.2863412724485e-01 5.5662156292048e-01 2.1981489572906e-01
5.8603702497117e-01 5.5932843334282e-01 2.0844912853192e-01
2.4349987950475e-01 5.6203604238342e-01 1.9708268028852e-01
9.0102614978478e-01 5.6474434452767e-01 1.8571557415781e-01
5.5861926000304e-01 5.6745328844140e-01 1.7434784213506e-01
2.1628255860130e-01 5.7016281815233e-01 1.6297952422634e-01
8.7401926966089e-01 5.7287287431179e-01 1.5161066739228e-01
5.3183243906807e-01 5.7558339548776e-01 1.4024132431249e-01
1.8972487826451e-01 5.7829431943957e-01 1.2887155203027e-01
8.4769910855572e-01 5.8100558432769e-01 1.1750141054108e-01
5.0575730884623e-01 5.8371712981726e-01 1.0613096138759e-01
1.6390126932851e-01 5.8642889804210e-01 9.4760266319589e-02
8.2213235312351e-01 5.8914083440527e-01 8.3389386068515e-02
4.8045146722241e-01 5.9185288820262e-01 7.2018379275644e-02
1.3885904338554e-01 5.9456501306591e-01 6.0647301600560e-02
7.9735502898298e-01 5.9727716723094e-01 4.9276205024051e-02
4.5593888716836e-01 5.9998931364410e-01 3.7905137347824e-02
1.1460960530042e-01 6.0270141992613e-01 2.6534141883343e-02
7.7336571018530e-01 6.0541345821569e-01 1.5163257314181e-02
4.3220528850685e-01 6.0812540491687e-01 3.7925177107080e-03
8.9573173974111e-02 6.1084155914803e-01 9.9241687096316e-01
7.4856787411490e-01 6.1355329268576e-01 9.8104647701867e-01
4.0763770259075e-01 6.1626488426392e-01 9.6967630601514e-01
6.6779234606440e-02 6.1897632371500e-01 9.5830637589851e-01
7.2598872721963e-01 6.2168760321674e-01 9.4693670149882e-01
3.8526213755314e-01 6.2439871689788e-01 9.3556729495778e-01
4.4595136223209e-02 6.2710966046690e-01 9.2419816615943e-01
7.0398311504424e-01 6.2982043086901e-01 9.1282932315377e-01
3.6342118810866e-01 6.3253102597460e-01 9.0146077256590e-01
2.2904185316683e-02 6.3524144430036e-01 8.9009251998659e-01
6.8242663733532e-01 6.3795168476279e-01 8.7872457034221e-01
3.4198275078598e-01 6.4066174646253e-01 8.6735692824435e-01
1.5663721806616e-03 6.4337162849736e-01 8.5598959832112e-01
6.6117093872484e-01 6.4608132980068e-01 8.4462258553370e-01
3.2078941355047e-01 6.4879084900248e-01 8.3325589548280e-01
9.8041420218757e-01 6.5150018430902e-01 8.2188953471103e-01
6.4003704606393e-01 6.5420933339796e-01 8.1052351100780e-01
2.9964888744242e-01 6.5691829332538e-01 7.9915783372436e-01
9.5923969832780e-01 6.5962706044170e-01 7.8779251410685e-01
6.1879826329571e-01 6.6233563031422e-01 7.7642756565558e-01
2.7831190261990e-01 6.6504399765500e-01 7.6506300451796e-01
9.3776611705102e-01 6.6775215625498e-01 7.5369884992032e-01
5.9714412846155e-01 6.7046009892912e-01 7.4233512463896e-01
2.5642628028039e-01 6.7316781748379e-01 7.3097185550040e-01
9.1558924659050e-01 6.7587530273026e-01 7.1960907387993e-01
5.7460497640409e-01 6.7858254459018e-01 7.0824681612786e-01
2.3343926585262e-01 6.8128953238085e-01 6.9688512377568e-01
8.9204979907895e-01 6.8399625544681e-01 6.8552404322650e-01
5.5038341716612e-01 6.8670270445495e-01 6.7416362434753e-01
2.0837224406357e-01 6.8940887396727e-01 6.6280391682062e-01
8.6592808514417e-01 6.9211476750054e-01 6.5144496198948e-01
5.2293415672456e-01 6.9482040751056e-01 6.4008677567802e-01
1.7923259182976e-01 6.9752585534171e-01 6.2872931278279e-01
8.3460509057103e-01 7.0023125185183e-01 6.1737239464501e-01
4.8874215519417e-01 7.0293690204437e-01 6.0601555949697e-01
1.4119288406676e-01 7.0564345535439e-01 5.9465775316723e-01
7.9128136466163e-01 7.0835229451735e-01 5.8329669462366e-01
4.3796804044319e-01 7.1106635597160e-01 5.7192764110060e-01
7.9641996987333e-02 7.1379163249505e-01 5.6054144429423e-01
7.1396663217518e-01 7.1653832502061e-01 5.4912410589083e-01
3.3857641610160e-01 7.1931306679357e-01 5.3766958392175e-01
9.5407945720235e-01 7.2208742821180e-01 5.2621510132562e-01
5.6600018209796e-01 7.2480125484780e-01 5.1481546980834e-01
1.6990128936131e-01 4.9512976658515e-01 5.0379399479477e-01
8.2687913363311e-01 4.9782342064375e-01 4.9243652043863e-01
4.8385176815644e-01 5.0051709585214e-01 4.8107904028952e-01
1.4082050803129e-01 5.0321082853685e-01 4.6972154980389e-01
7.9778677451197e-01 5.0590465625843e-01 4.5836404299756e-01
4.5475212486914e-01 5.0859861821386e-01 4.4700651172168e-01
1.1171827563534e-01 5.1129275554220e-01 4.3564894499502e-01
7.6868711780935e-01 5.1398711150611e-01 4.2429132841755e-01
4.2566072318310e-01 5.1668173152717e-01 4.1293364369046e-01
8.2641341627052e-02 5.1937666305983e-01 4.0157586826716e-01
7.3963138987149e-01 5.2207195529647e-01 3.9021797515812e-01
3.9663343299007e-01 5.2476765870462e-01 3.7885993290900e-01
5.3650160364904e-02 5.2746382440612e-01 3.6750170576641e-01
7.1068435833041e-01 5.3016050341637e-01 3.5614325403944e-01
3.6773888191015e-01 5.3285774576920e-01 3.4478453465709e-01
2.4816628049904e-02 5.3555559955915e-01 3.3342550191337e-01
6.8192051250821e-01 5.3825410993716e-01 3.2206610838302e-01
3.3905345211289e-01 5.4095331809838e-01 3.1070630598232e-01
9.9621835347389e-01 5.4365326030127e-01 2.9934604714143e-01
6.5341810851900e-01 5.4635396695618e-01 2.8798528604873e-01
3.1065559646138e-01 5.4905546181883e-01 2.7662397992211e-01
9.6793369109084e-01 5.5175776132036e-01 2.6526209025947e-01
6.2525527167101e-01 5.5446087406062e-01 2.5389958401958e-01
2.8262323528130e-01 5.5716480048567e-01 2.4253643468529e-01
9.4004050820374e-01 5.5986953276436e-01 2.3117262316455e-01
5.9751005394448e-01 5.6257505487161e-01 2.1980813848989e-01
2.5503487569624e-01 5.6528134287889e-01 2.0844297828489e-01
9.1261801147358e-01 5.6798836544460e-01 1.9707714897528e-01
5.7026252074690e-01 5.7069608448914e-01 1.8571066573390e-01
2.2797146210941e-01 5.7340445603188e-01 1.7434355216064e-01
8.8574786226690e-01 5.7611343116027e-01 1.6297583971150e-01
5.4359467737187e-01 5.7882295709554e-01 1.5160756690289e-01
2.0151474836074e-01 5.8153297831524e-01 1.4023877832842e-01
8.5951075243900e-01 5.8424343769104e-01 1.2886952353391e-01
5.1758515314855e-01 5.8695427760040e-01 1.1749985580207e-01
1.7574015152841e-01 5.8966544097384e-01 1.0612983090078e-01
8.3397764074471e-01 5.9237687224419e-01 9.4759505847155e-02
4.9229916624787e-01 5.9508851817148e-01 8.3388937735600e-02
1.5070589305627e-01 5.9780032852466e-01 7.2018182670216e-02
8.0919858122211e-01 6.0051225661015e-01 6.0647294833073e-02
4.6777756996325e-01 6.0322425964521e-01 4.9276325709487e-02
1.2644277039825e-01 6.0593629898177e-01 3.7905323481190e-02
7.8519366634088e-01 6.0864834019203e-01 2.6534332588748e-02
4.4402932222586e-01 6.1136035303227e-01 1.5163393456385e-02
1.0294839696076e-01 6.1407231130375e-01 3.7925423659431e-03
7.6040483812807e-01 6.1678848852451e-01 9.9241675645468e-01
4.1948052552053e-01 6.1950030097547e-01 9.8104614334275e-01
7.8633162694396e-02 6.2221200031567e-01 9.6967570437921e-01
7.3785999466733e-01 6.2492357378311e-01 9.5830546033266e-01
3.9715797023325e-01 6.2763501111535e-01 9.4693542875323e-01
5.6523762559761e-02 6.3034630418956e-01 9.3556562432713e-01
7.1595378675757e-01 6.3305744667412e-01 9.2419605924824e-01
3.7544421352897e-01 6.3576843369920e-01 9.1282674359498e-01
3.4990978073905e-02 6.3847926155120e-01 9.0145768570338e-01
6.9458978346122e-01 6.4118992739385e-01 8.9008889253039e-01
3.5423609764897e-01 6.4390042901716e-01 8.7872037000335e-01
1.3925143251497e-02 6.4661076461424e-01 8.6735212335449e-01
6.7365187899013e-01 6.4932093258449e-01 8.5598415744063e-01
3.3341097151341e-01 6.5203093136125e-01 8.4461647705065e-01
9.9319675591005e-01 6.5474075926093e-01 8.3324908720457e-01
6.5300318273093e-01 6.5745041435052e-01 8.2188199344978e-01
3.1282374864126e-01 6.6015989432960e-01 8.1051520216169e-01
9.7265140687301e-01 6.6286919642278e-01 7.9914872085762e-01
6.3247845234457e-01 6.6557831727816e-01 7.8778255853500e-01
2.9229637451640e-01 6.6828725286674e-01 7.7641672604723e-01
9.5209566854566e-01 6.7099599837745e-01 7.6505123653379e-01
6.1186559176969e-01 6.7370454810177e-01 7.5368610592497e-01
2.7159384750053e-01 6.7641289530109e-01 7.4232135354640e-01
9.3126617079607e-01 6.7912103204965e-01 7.3095700285474e-01
5.9086578009825e-01 6.8182894904512e-01 7.1959308234261e-01
2.5037264249038e-01 6.8453663537991e-01 7.0822962665821e-01
9.0976247570172e-01 6.8724407826932e-01 6.9686667799033e-01
5.6900537165838e-01 6.8995126274197e-01 6.8550428776700e-01
2.2806386539715e-01 6.9265817132104e-01 6.7414251869184e-01
8.8689017378682e-01 6.9536478377938e-01 6.6278144706389e-01
5.4542216225892e-01 6.9807107717815e-01 6.5142116511540e-01
2.0357731146183e-01 7.0077702669422e-01 6.4006178257133e-01
8.6124344669362e-01 7.0348260844408e-01 6.2870342532298e-01
5.1826405514885e-01 7.0618780723316e-01 6.1734622585053e-01
1.7441421937807e-01 7.0889263654371e-01 6.0599029176433e-01
8.2935960302726e-01 7.1159718977413e-01 5.9463561722384e-01
4.8258341029777e-01 7.1430177460338e-01 5.8328184355755e-01
1.3324984959526e-01 7.1700727912572e-01 5.7192761348402e-01
7.7993638751654e-01 7.1971620844958e-01 5.6056882408381e-01
4.2009813004709e-01 7.2243560015200e-01 5.4919414130577e-01
4.9165285886410e-02 7.2518334665641e-01 5.3777711296247e-01
6.6101023585164e-01 7.2798088887003e-01 5.2629257684883e-01
2.5722538412574e-01 7.3075948378925e-01 5.1482496334757e-01
8.3655944792285e-01 5.0108829007804e-01 5.0378173881486e-01
4.9354031585491e-01 5.0378194528222e-01 4.9242426639101e-01
1.5052502291534e-01 5.0647562538636e-01 4.8106679301134e-01
8.0751480042082e-01 5.0916936592847e-01 4.6970931442512e-01
4.6451091731224e-01 5.1186320304638e-01 4.5835182521265e-01
1.2151471309136e-01 5.1455717390957e-01 4.4699431807597e-01
7.7852762553804e-01 5.1725131706969e-01 4.3563678318817e-01
4.3555121167407e-01 5.1994567270330e-01 4.2427920762222e-01
9.2587160968336e-02 5.2264028272541e-01 4.1292157488035e-01
7.4963730038061e-01 5.2533519075868e-01 4.0156386454369e-01
4.0670359147424e-01 5.2803044194987e-01 3.9020605206092e-01
6.3788120436526e-02 5.3072608263298e-01 3.7884810869145e-01
7.2089308238005e-01 5.3342215984570e-01 3.6749000161502e-01
3.7802076170747e-01 5.3611872071348e-01 3.5613169421473e-01
3.5173510569318e-02 5.3881581172185e-01 3.4477314653391e-01
6.9235372750329e-01 5.4151347790324e-01 3.3341431590111e-01
3.4956383820796e-01 5.4421176196857e-01 3.2205515770998e-01
6.8062800865054e-03 5.4691070341654e-01 3.1069562633381e-01
6.6408349172744e-01 5.4961033765440e-01 2.9933567614822e-01
3.2139790791434e-01 5.5231069516379e-01 2.8797526262955e-01
9.7875196013154e-01 5.5501180074298e-01 2.7661434349217e-01
6.3614808191507e-01 5.5771367285435e-01 2.6525287982498e-01
2.9358871784635e-01 5.6041632310207e-01 2.5389083718546e-01
9.5107633454794e-01 5.6311975586027e-01 2.4252818661021e-01
6.0861343175690e-01 5.6582396806716e-01 2.3116490550248e-01
2.6620255144386e-01 5.6852894919504e-01 2.1980097836086e-01
9.2384628302546e-01 5.7123468139997e-01 2.0843639731888e-01
5.8154726297597e-01 5.7394113984877e-01 1.9707116247200e-01
2.3930816755869e-01 5.7664829321453e-01 1.8570528197723e-01
8.9713169793118e-01 5.7935610432527e-01 1.7433877192014e-01
5.5502055748441e-01 5.8206453094414e-01 1.6297165595458e-01
2.1297742189819e-01 5.8477352665433e-01 1.5160396473058e-01
8.7100490297801e-01 5.8748304181715e-01 1.4023573513599e-01
5.2910550782905e-01 5.9019302456914e-01 1.2886700938589e-01
1.8728159527804e-01 5.9290342182290e-01 1.1749783400005e-01
8.4553533164560e-01 5.9561418023704e-01 1.0612825871299e-01
5.0386864798911e-01 5.9832524712384e-01 9.4758335362096e-02
1.6228320078809e-01 6.0103657126731e-01 8.3388116797564e-02
8.2078033775343e-01 6.0374810363051e-01 7.2017655853835e-02
4.7936107004746e-01 6.0645979793773e-01 6.0647004415551e-02
1.3802605174587e-01 6.0917161112373e-01 4.9276212603456e-02
7.9677556690135e-01 6.1188350364962e-01 3.7905328097085e-02
4.5560952412235e-01 6.1459543969024e-01 2.6534395602642e-02
1.1452745819060e-01 6.1730738720348e-01 1.5163456466660e-02
7.7352853793020e-01 6.2001931789519e-01 3.7925484293671e-03
4.3107622808675e-01 6.2273547883801e-01 9.9241667868650e-01
9.0234807812915e-02 6.2544733370603e-01 9.8104589897268e-01
7.4947178339131e-01 6.2815910608029e-01 9.6967524283534e-01
4.0878502598338e-01 6.3087078080090e-01 9.5830473345410e-01
6.8172135429631e-02 6.3358234529833e-01 9.4693439076814e-01
7.2763046219822e-01 6.3629378927637e-01 9.3556423174892e-01
3.8715712795553e-01 6.3900510439621e-01 9.2419427070164e-01
4.6749043937201e-02 6.4171628397016e-01 9.1282451958278e-01
7.0640292639170e-01 6.4442732267132e-01 9.0145498832347e-01
3.6611530843711e-01 6.4713821626368e-01 8.9008568515069e-01
2.5882547708647e-02 6.4984896135537e-01 8.7871661690075e-01
6.8570082917010e-01 6.5255955517626e-01 8.6734778932145e-01
3.4556616240896e-01 6.5526999538000e-01 8.5597920736143e-01
5.4743726234142e-03 6.5798027986969e-01 8.4461087544687e-01
6.6542108432670e-01 6.6069040664514e-01 8.3324279774734e-01
3.2540169652218e-01 6.6340037366975e-01 8.2187497843429e-01
9.8541134771683e-01 6.6611017875379e-01 8.1050742193704e-01
6.4544486860252e-01 6.6881981945090e-01 7.9914013320288e-01
3.0549671948950e-01 6.7152929296389e-01 7.8777311796983e-01
9.6556090854302e-01 6.7423859605544e-01 7.7640638306303e-01
6.2563088542925e-01 6.7694772495867e-01 7.6503993672862e-01
2.8569940293697e-01 6.7965667528143e-01 7.5367378902347e-01
9.4575833622351e-01 6.8236544189704e-01 7.4230795228460e-01
6.0579844509781e-01 6.8507401881209e-01 7.3094244171056e-01
2.6580905850510e-01 6.8778239899923e-01 7.1957727609859e-01
9.2577765099489e-01 6.9049057417874e-01 7.0821247879860e-01
5.8568926658762e-01 6.9319853452663e-01 6.9684807896975e-01
2.4552572298887e-01 6.9590626827832e-01 6.8548411326267e-01
9.0526449313231e-01 6.9861376118400e-01 6.7412062810562e-01
5.6487710194639e-01 7.0132099575329e-01 6.6275768285571e-01
2.2432677633968e-01 7.0402795020079e-01 6.5139535420102e-01
8.8356491196574e-01 7.0673459697144e-01 6.4003374238073e-01
5.4252560448296e-01 7.0944090069383e-01 6.2867298004243e-01
2.0111689703178e-01 7.1214681542327e-01 6.1731324484179e-01
8.5920621660235e-01 7.1485228125368e-01 6.0595477697977e-01
5.1659500860292e-01 7.1755722138070e-01 5.9459790174389e-01
1.7297209421355e-01 7.2026154467788e-01 5.8324305064479e-01
8.2782211447160e-01 7.2296517461257e-01 5.7189074596732e-01
4.8023092170653e-01 7.2566819194739e-01 5.6054138842363e-01
1.2842940229791e-01 7.2837149192915e-01 5.4919411305297e-01
7.6859101134700e-01 7.3108002030646e-01 5.3784115679470e-01
3.9134858025817e-01 7.3381927766202e-01 5.2644204083747e-01
9.7662667664522e-01 7.3666016327960e-01 5.1490092953840e-01
5.0322063455607e-01 5.0704681228298e-01 5.0376948964950e-01
1.6019545081630e-01 5.0974046569780e-01 4.9241201233867e-01
8.1718317860126e-01 5.1243414792056e-01 4.8105453894240e-01
4.7418497677459e-01 5.1512789388733e-01 4.6969706556777e-01
1.3120197393764e-01 5.1782173850932e-01 4.5833958740480e-01
7.8823530495131e-01 5.2051571714038e-01 4.4698209802054e-01
4.4528614347830e-01 5.2320986597608e-01 4.3562458871559e-01
1.0235572886836e-01 5.2590422235837e-01 4.2426704796117e-01
7.5944538620106e-01 5.2859882496432e-01 4.1290946093355e-01
4.1655653886140e-01 5.3129371386383e-01 4.0155180916208e-01
7.3690713611457e-02 5.3398893043728e-01 3.9019407030510e-01
7.3084953869013e-01 5.3668451715108e-01 3.7883621806612e-01
3.8803473598018e-01 5.3938051719578e-01 3.6747822225951e-01
4.5248108685188e-02 5.4207697399782e-01 3.5612004903106e-01
7.0249152622695e-01 5.4477393062199e-01 3.4476166123396e-01
3.5976690818386e-01 5.4747142908643e-01 3.3340301895561e-01
1.7076209035507e-02 5.5016950961593e-01 3.2204408018492e-01
6.7442140526503e-01 5.5286820986170e-01 3.1068480160382e-01
3.3180448602022e-01 5.5556756411711e-01 2.9932513948180e-01
9.8922744808126e-01 5.5826760255861e-01 2.8796505064688e-01
6.4669229536964e-01 5.6096835054009e-01 2.7660449350281e-01
3.0420104270643e-01 5.6366982796676e-01 2.6524342905940e-01
9.6175572303646e-01 5.6637204877138e-01 2.5388182194080e-01
6.1935839691890e-01 5.6907502051265e-01 2.4251964133647e-01
2.7701116278193e-01 5.7177874411098e-01 2.3115686186042e-01
9.3471616627162e-01 5.7448321373289e-01 2.1979346428639e-01
5.9247560700886e-01 5.7718841683002e-01 2.0842943613086e-01
2.5029174120097e-01 5.7989433433399e-01 1.9706477206046e-01
9.0816687882754e-01 5.8260094100270e-01 1.8569947410704e-01
5.6610337450746e-01 5.8530820590853e-01 1.7433355168116e-01
2.2410361162457e-01 5.8801609305321e-01 1.6296702138306e-01
8.8216997980101e-01 5.9072456208951e-01 1.5159990661868e-01
5.4030484631620e-01 5.9343356912540e-01 1.4023223703735e-01
1.9851052253120e-01 5.9614306758317e-01 1.2886404781497e-01
8.5678922675267e-01 5.9885300908398e-01 1.1749537881368e-01
5.1514304522759e-01 6.0156334432798e-01 1.0612627365340e-01
1.7357389308062e-01 6.0427402394118e-01 9.4756778733547e-02
8.3208347698585e-01 6.0698499926299e-01 8.3386942243438e-02
4.9067326121368e-01 6.0969622305250e-01 7.2016813198080e-02
1.4934443843343e-01 6.1240765009640e-01 6.0646440531968e-02
8.0809790631415e-01 6.1511923770733e-01 4.9275872278101e-02
4.6693425058466e-01 6.1783094610710e-01 3.7905154852732e-02
1.2585373482566e-01 6.2054273869472e-01 2.6534332459314e-02
7.8485629690209e-01 6.2325458220423e-01 1.5163446618106e-02
4.4394155162842e-01 6.2596644676116e-01 3.7925358214851e-03
1.0158288121890e-01 6.2868255216809e-01 9.9241663816407e-01
7.6082598698288e-01 6.3139441241280e-01 9.8104574594676e-01
4.2014858565202e-01 6.3410622203063e-01 9.6967492590730e-01
7.9549124961821e-02 6.3681796369873e-01 9.5830420322355e-01
7.3902580861941e-01 6.3952962271052e-01 9.4693359989239e-01
3.9857661867639e-01 6.4224118670778e-01 9.3556313491675e-01
5.8199338010237e-02 6.4495264540421e-01 9.2419282453086e-01
7.1789157218618e-01 6.4766399030963e-01 9.1282268245070e-01
3.7765077007448e-01 6.5037521446243e-01 9.0145272013887e-01
3.7474242693638e-02 6.5308631217582e-01 8.9008294707428e-01
6.9735917983261e-01 6.5579727880210e-01 8.7871337101966e-01
3.5730266406686e-01 6.5850811051737e-01 8.6734399828134e-01
1.7301681820088e-02 6.6121880412827e-01 8.5597483395762e-01
6.7735313113070e-01 6.6392935690088e-01 8.4460588217357e-01
3.3745382575535e-01 6.6663976641152e-01 8.3323714630158e-01
9.9760049517469e-01 6.6935003041806e-01 8.2186862916805e-01
6.5778977995104e-01 6.7206014675029e-01 8.1050033324805e-01
3.1801822171180e-01 6.7477011321700e-01 7.9913226085044e-01
9.7828224677693e-01 6.7747992752764e-01 7.8776441429755e-01
6.3857814208768e-01 6.8018958722548e-01 7.7639679610414e-01
2.9890202158378e-01 6.8289908962953e-01 7.6502940916242e-01
9.5924978045476e-01 6.8560843178152e-01 7.5366225694126e-01
6.1961703366085e-01 6.8831761039397e-01 7.4229534371062e-01
2.7999903362677e-01 6.9102662179457e-01 7.3092867480572e-01
9.4039055981961e-01 6.9373546186084e-01 7.1956225695081e-01
6.0078576964058e-01 6.9644412593700e-01 7.0819609867051e-01
2.6117799505536e-01 6.9915260872240e-01 6.9683021082854e-01
9.2155946158404e-01 7.0186090411589e-01 6.8546460735291e-01
5.8192089381638e-01 7.0456900499333e-01 6.7409930623648e-01
2.4225095120445e-01 7.0727690288300e-01 6.6273433095112e-01
9.0253540343604e-01 7.0998458748343e-01 6.5136971249570e-01
5.6275589459858e-01 7.1269204593206e-01 6.4000549244030e-01
2.2288803650482e-01 7.1539926166959e-01 6.2864172758300e-01
8.8289836574086e-01 7.1810621262546e-01 6.1727849730759e-01
5.4273928984032e-01 7.2081286821771e-01 6.0591591564854e-01
2.0234028525847e-01 7.2351918418573e-01 5.9455415194775e-01
8.6159165812731e-01 7.2622509324644e-01 5.8319346806021e-01
5.2031236845372e-01 7.2893048720381e-01 5.7183428949677e-01
1.7818022283737e-01 7.3163518040858e-01 5.6047735131012e-01
8.3456118297072e-01 7.3433883036405e-01 5.4912402114354e-01
4.8801687594552e-01 7.3704076617518e-01 5.3777705583545e-01
1.3448780934181e-01 7.3973987118792e-01 5.2644201196450e-01
7.5724524915529e-01 7.4244372770205e-01 5.1511369756710e-01
1.6988486848497e-01 5.1300532640511e-01 5.0375726093594e-01
8.2684452557779e-01 5.1569897504928e-01 4.9239977190071e-01
4.8382623833027e-01 5.1839265667179e-01 4.8104229164058e-01
1.4083110322597e-01 5.2108640578330e-01 4.6968481669152e-01
7.9786011812124e-01 5.2378025624374e-01 4.5832734290978e-01
4.5491422297431e-01 5.2647424177271e-01 4.4696986475791e-01
1.1199433771278e-01 5.2916839640258e-01 4.3561237465909e-01
7.6910139537608e-01 5.3186275484743e-01 4.2425486243514e-01
4.2623636914812e-01 5.3455735276639e-01 4.1289731484194e-01
8.3400292437503e-02 5.3725222690583e-01 4.0153971521466e-01
7.4059427172900e-01 5.3994741511083e-01 3.9018204323352e-01
3.9781949247774e-01 5.4264295620300e-01 3.7882427481915e-01
5.5077218810007e-02 5.4533888972761e-01 3.6746638216420e-01
7.1236878819646e-01 5.4803525557905e-01 3.5610833390468e-01
3.6969560254804e-01 5.5073209351876e-01 3.4475009543126e-01
2.7059117333201e-02 5.5342944260425e-01 3.3339162933644e-01
6.8446083032029e-01 5.5612734055136e-01 3.2203289598890e-01
3.4190227141420e-01 5.5882582305411e-01 3.1067385422223e-01
9.9938499479903e-01 5.6152492308811e-01 2.9931446212027e-01
6.5691057424251e-01 5.6422467022326e-01 2.8795467787770e-01
3.1448060199868e-01 5.6692508997107e-01 2.7659446071064e-01
9.7209669129852e-01 5.6962620319021e-01 2.6523377178982e-01
6.2976048198502e-01 5.7232802557150e-01 2.5387257516663e-01
2.8747364846352e-01 5.7503056722082e-01 2.4251083866192e-01
9.4523790883320e-01 5.7773383235517e-01 2.3114853468759e-01
6.0305503386409e-01 5.8043781912342e-01 2.1978564097243e-01
2.6092685440190e-01 5.8314251955926e-01 2.0842214116629e-01
9.1885526582446e-01 5.8584791966999e-01 1.9705802530035e-01
5.7684222833677e-01 5.8855399966004e-01 1.8569329008602e-01
2.3488976215969e-01 5.9126073428373e-01 1.7432793904080e-01
8.9299993702093e-01 5.9396809331728e-01 1.6296198243615e-01
5.5117485576446e-01 5.9667604213568e-01 1.5159543706927e-01
2.0941663232338e-01 5.9938454237589e-01 1.4022832586817e-01
8.6772736471457e-01 6.0209355266487e-01 1.2886067734631e-01
5.2610910407746e-01 6.0480302938823e-01 1.1749252492933e-01
1.8456382106344e-01 6.0751292747415e-01 1.0612390618145e-01
8.4309337106580e-01 6.1022320116730e-01 9.4754861962727e-02
5.0169945985033e-01 6.1293380476867e-01 8.3385435549921e-02
1.6038361110346e-01 6.1564469331983e-01 7.2015671753777e-02
8.1914713726874e-01 6.1835582321367e-01 6.0645616063280e-02
4.7799111481221e-01 6.2106715271808e-01 4.9275313844062e-02
1.3691636476888e-01 6.2377864240378e-01 3.7904809613205e-02
7.9592343910459e-01 6.2649025547212e-01 2.6534146407141e-02
4.5501261310774e-01 6.2920195798358e-01 1.5163365253396e-02
1.1418388372845e-01 6.3191371899142e-01 3.7925047511309e-03
7.7192094969803e-01 6.3462973021030e-01 9.9241663477726e-01
4.3124995468416e-01 6.3734155888237e-01 9.8104568467368e-01
9.0659230144713e-02 6.4005336955620e-01 9.6967475537503e-01
7.5014774652463e-01 6.4276514301315e-01 9.5830387364998e-01
4.0971425928901e-01 6.4547686261407e-01 9.4693306322351e-01
6.9357330928604e-02 6.4818851408396e-01 9.3556234487889e-01
7.2907535435632e-01 6.5090008527956e-01 9.2419173660494e-01
3.8886657707662e-01 6.5361156594957e-01 9.1282125377616e-01
4.8729125611451e-02 6.5632294749539e-01 9.0145090935450e-01
7.0866102977664e-01 6.5903422273926e-01 8.9008071410263e-01
3.6866024651504e-01 6.6174538570463e-01 8.7871067680049e-01
2.8724683103627e-02 6.6445643141284e-01 8.6734080445801e-01
6.8885221965721e-01 6.6716735569834e-01 8.5597110251842e-01
3.4904073095275e-01 6.6987815504428e-01 8.4460157504814e-01
9.2881076953116e-03 6.7258882643893e-01 8.3323222490992e-01
6.6959227744342e-01 6.7529936725310e-01 8.2186305391709e-01
3.2995122551263e-01 6.7800977513796e-01 8.1049406296742e-01
9.9036301628833e-01 6.8072004794241e-01 7.9912525215554e-01
6.5082581551214e-01 6.8343018364911e-01 7.8775662086309e-01
3.1133791427497e-01 6.8614018032781e-01 7.7638816782580e-01
9.7189775567175e-01 6.8885003610509e-01 7.6501989117640e-01
6.3250396537777e-01 6.9155974914933e-01 7.5365178846168e-01
2.9315538783467e-01 6.9426931767058e-01 7.4228385663105e-01
9.5385113034913e-01 6.9697873993468e-01 7.3091609199232e-01
6.1459061830611e-01 6.9968801429231e-01 7.1954849012822e-01
2.7537366603727e-01 7.0239713922407e-01 7.0818104576371e-01
9.3620056991454e-01 7.0510611340424e-01 6.9681375256894e-01
5.9707223337879e-01 7.0781493578759e-01 6.8544660287462e-01
2.5799033857925e-01 7.1052360572695e-01 6.7407958726422e-01
9.1895758734832e-01 7.1323212313362e-01 6.6271269398696e-01
5.7997804764900e-01 7.1594048870083e-01 6.5134590810221e-01
2.4105766469056e-01 7.1864870422369e-01 6.3997921020828e-01
9.0220503698183e-01 7.2135677307242e-01 6.2861257450696e-01
5.6343263379527e-01 7.2406470091914e-01 6.1724596576703e-01
2.2475877871788e-01 7.2677249690134e-01 6.0587933438621e-01
8.8621102856978e-01 7.2948017557511e-01 5.9451260800858e-01
5.4783224559841e-01 7.3218776037785e-01 5.8314567653818e-01
2.0969225040846e-01 7.3489529018015e-01 5.7177836359190e-01
8.7191211567040e-01 7.3760283271591e-01 5.6041036762515e-01
5.3472059794742e-01 7.4031051507620e-01 5.4904112742415e-01
1.9860616314640e-01 7.4301860309990e-01 5.3766946968998e-01
8.6482642749699e-01 7.4572775248635e-01 5.2629249024106e-01
5.3786395627386e-01 7.4844008915629e-01 5.1490087118718e-01
8.3655218533853e-01 5.1896382565855e-01 5.0374506634971e-01
4.9348749479333e-01 5.2165746642694e-01 4.9238755872189e-01
1.5045412529051e-01 5.2435114471524e-01 4.8103006467663e-01
8.0745311987359e-01 5.2704489477881e-01 4.6967258125604e-01
4.6448535297890e-01 5.2973874958441e-01 4.5831510504995e-01
1.2155157612534e-01 5.3243274137229e-01 4.4695763146220e-01
7.7865246176781e-01 5.3512690217000e-01 4.3560015405313e-01
4.3578864321378e-01 5.3782126423015e-01 4.2424266397146e-01
9.2960748989786e-02 5.4051586036979e-01 4.1288514948627e-01
7.5016943059215e-01 5.4321072419540e-01 4.0152759562798e-01
4.0741538312255e-01 5.4590589020347e-01 3.9016998394574e-01
6.4699358854633e-02 5.4860139375303e-01 3.7881229238685e-01
7.2202217426854e-01 5.5129727091196e-01 3.6745449530234e-01
3.7938471149670e-01 5.5399355818467e-01 3.5609656358019e-01
3.6787915424269e-02 5.5669029213321e-01 3.4473846490553e-01
6.9423278786564e-01 5.5938750890810e-01 3.3338016414362e-01
3.5172038028926e-01 5.6208524370801e-01 3.2202162383842e-01
9.2517864891946e-03 5.6478353019015e-01 3.1066280481561e-01
6.6682813641558e-01 5.6748239985390e-01 2.9930366687583e-01
3.2445059209820e-01 5.7018188142106e-01 2.8794416956002e-01
9.8212034625417e-01 5.7288200023522e-01 2.7658427296645e-01
6.3983862379256e-01 5.7558277770190e-01 2.6522393859610e-01
2.9760668604863e-01 5.7828423078892e-01 2.5386313020158e-01
9.5542583722763e-01 5.8098637160447e-01 2.4250181461397e-01
6.1329743224213e-01 5.8368920706751e-01 2.3113996252160e-01
2.7122288490782e-01 5.8639273868209e-01 2.1977754917569e-01
9.2920367533778e-01 5.8909696242424e-01 2.0841455499955e-01
5.8724135535187e-01 5.9180186874624e-01 1.9705096608053e-01
2.4533755079646e-01 5.9450744269977e-01 1.8568677452753e-01
9.0349395984424e-01 5.9721366417547e-01 1.7432197868125e-01
5.6171234659808e-01 5.9992050825272e-01 1.6295658316967e-01
2.1999452963827e-01 6.0262794564955e-01 1.5159059880657e-01
8.7834236550232e-01 6.0533594325919e-01 1.4022404233744e-01
5.3675772744293e-01 6.0804446475640e-01 1.2885693604274e-01
1.9524248014434e-01 6.1075347125434e-01 1.1748930721445e-01
8.5379845136246e-01 6.1346292199109e-01 1.0612118752649e-01
5.1242740166913e-01 6.1617277502376e-01 9.4752612323813e-02
1.7113099360903e-01 6.1888298790907e-01 8.3383619857234e-02
8.2991076161329e-01 6.2159351835010e-01 7.2014250492272e-02
4.8876808395796e-01 6.2430432479148e-01 6.0644545919664e-02
1.4770415791844e-01 6.2701536694855e-01 4.9274548393311e-02
8.0671997906999e-01 6.2972660625940e-01 3.7904300018135e-02
4.6581632543979e-01 6.3243800625313e-01 2.6533842106196e-02
1.2499374695182e-01 6.3514953283133e-01 1.5163214614631e-02
7.8425256034373e-01 6.3786115446368e-01 3.7924556740318e-03
4.4208718019340e-01 6.4057703395384e-01 9.9241666788032e-01
1.0150321171350e-01 6.4328879474815e-01 9.8104571412379e-01
7.6100000351727e-01 6.4600057048469e-01 9.6967473056565e-01
4.2057699218712e-01 6.4871234034156e-01 9.5830374516901e-01
8.0233429487136e-02 6.5142408598161e-01 9.4693278306130e-01
7.3996840336192e-01 6.5413579139066e-01 9.3556186655549e-01
3.9978086138054e-01 6.5684744269169e-01 9.2419101522139e-01
5.9669636128090e-02 6.5955902794435e-01 9.1282024599570e-01
7.1963347213997e-01 6.6227053693822e-01 9.0144957331988e-01
3.7967105408709e-01 6.6498196098685e-01 8.9007900929570e-01
3.9781036048901e-02 6.6769329272852e-01 8.7870856384947e-01
6.9996207185196e-01 6.7040452593821e-01 8.6733824489700e-01
3.6021284660530e-01 6.7311565535456e-01 8.5596805850245e-01
2.0532109730625e-02 6.7582667652400e-01 8.4459800902534e-01
6.8091870997268e-01 6.7853758566401e-01 8.3322809925074e-01
3.4137163308998e-01 6.8124837954637e-01 8.2185833049848e-01
1.8900431838497e-03 6.8395905540087e-01 8.1048870270741e-01
6.6247332894223e-01 6.8666961083967e-01 7.9911921449092e-01
3.2312115648368e-01 6.8938004380233e-01 7.8774986315985e-01
9.8383353102511e-01 6.9209035252136e-01 7.7638064470782e-01
6.4461087032278e-01 6.9480053550856e-01 7.6501155375307e-01
3.0545409383380e-01 6.9751059156273e-01 7.5364258342892e-01
9.6636473294142e-01 7.0022051979986e-01 7.4227372521183e-01
6.2734506957136e-01 7.0293031970804e-01 7.3090496867208e-01
2.8839831339125e-01 7.0563999123070e-01 7.1953630112579e-01
9.4952883199030e-01 7.0834953488381e-01 7.0816770715829e-01
6.1074245471623e-01 7.1105895191561e-01 6.9679916797549e-01
2.7204688040239e-01 7.1376824452205e-01 6.8543066052070e-01
9.3345223406083e-01 7.1647741613756e-01 6.7406215626485e-01
5.9497184119773e-01 7.1918647183130e-01 6.6269361953247e-01
2.5662332682425e-01 7.2189541885518e-01 6.5132500515486e-01
9.1843021059622e-01 7.2460426741611e-01 6.3995625512761e-01
5.8042428078660e-01 7.2731303178903e-01 6.2858729376325e-01
2.4264922912080e-01 7.3002173196209e-01 6.1721802051925e-01
9.0516640031925e-01 7.3273039613951e-01 6.0584829915434e-01
5.6806423677735e-01 7.3543906467669e-01 5.9447794096447e-01
2.3147449744437e-01 7.3814779651214e-01 5.8310667832021e-01
8.9560162333665e-01 7.4085668020055e-01 5.7173412230222e-01
5.6077942684836e-01 7.4356585412294e-01 5.6035969538923e-01
2.2758954300127e-01 7.4627554751500e-01 5.4898253286576e-01
8.9713490433377e-01 7.4898618091239e-01 5.3760139414460e-01
5.7175775364825e-01 7.5169871681931e-01 5.2621495701643e-01
2.5726565262118e-01 7.5441676348440e-01 5.1482484666738e-01
5.0322263775431e-01 5.2492230325490e-01 5.0373291962913e-01
1.6012427981478e-01 5.2761593283902e-01 4.9237538649768e-01
8.1706668127665e-01 5.3030960496989e-01 4.8101787165958e-01
4.7405083963845e-01 5.3300335381468e-01 4.6966037274687e-01
1.3107751100936e-01 5.3569721159108e-01 4.5830288716106e-01
7.8814726167084e-01 5.3839120919125e-01 4.4694541130753e-01
4.4526051875888e-01 5.4108537676504e-01 4.3558793991630e-01
1.0241761766718e-01 5.4377974423294e-01 4.2423046546053e-01
7.5961884429080e-01 5.4647434170506e-01 4.1287297768045e-01
4.1686447080702e-01 5.4916919978946e-01 4.0151546321845e-01
7.4154784278821e-02 5.5186434977937e-01 3.9015790536923e-01
7.3149010792896e-01 5.5455982371516e-01 3.7880028394741e-01
3.8887081543516e-01 5.5725565432235e-01 3.6744257527377e-01
4.6297339015688e-02 5.5995187483224e-01 3.5608475227975e-01
7.0377017239083e-01 5.6264851869585e-01 3.4472678472804e-01
3.6128986990851e-01 5.6534561920589e-01 3.3336863954477e-01
1.8857043207845e-02 5.6804320904383e-01 3.2201028125664e-01
6.7647235676593e-01 5.7074131977168e-01 3.1065167252353e-01
3.3413652354202e-01 5.7343998128878e-01 2.9929277475419e-01
9.9185030171502e-01 5.7613922127464e-01 2.8793354879041e-01
6.4961449322843e-01 5.7883906463837e-01 2.7657395564225e-01
3.0742994453513e-01 5.8153953299435e-01 2.6521395725503e-01
9.6529754960082e-01 5.8424064418221e-01 2.5385351728720e-01
6.2321825490462e-01 5.8694241184754e-01 2.4249260187726e-01
2.8119306589330e-01 5.8964484509707e-01 2.3113118037736e-01
9.3922305412027e-01 5.9234794824027e-01 2.1976922603187e-01
5.9730936414928e-01 5.9505172062579e-01 2.0840671658002e-01
2.5545321923307e-01 5.9775615657911e-01 1.9704363476352e-01
9.1365592479374e-01 6.0046124544422e-01 1.8567996872298e-01
5.7191886883145e-01 6.0316697172911e-01 1.7431571226981e-01
2.3024351856218e-01 6.0587331535189e-01 1.6295086502462e-01
8.8863141282056e-01 6.0858025198067e-01 1.5158543241747e-01
5.4708415004051e-01 6.1128775345769e-01 1.4021942555037e-01
2.0560337192456e-01 6.1399578829479e-01 1.2885286092743e-01
8.6419074320764e-01 6.1670432222529e-01 1.1748576006307e-01
5.2284792819036e-01 6.1941331879504e-01 1.0611814898342e-01
1.8157656494054e-01 6.2212273997452e-01 9.4750057639662e-02
8.4037823822259e-01 6.2483254677307e-01 8.3381519255370e-02
4.9925445230265e-01 6.2754269983741e-01 7.2012569631364e-02
1.5820660478925e-01 6.3025316001738e-01 6.0643246432419e-02
8.1723596260807e-01 6.3296388888434e-01 4.9273588479252e-02
4.7634364108431e-01 6.3567484919035e-01 3.7903635067432e-02
1.3553058693209e-01 6.3838600525933e-01 2.6533425331912e-02
7.9479756574483e-01 6.4109732330491e-01 1.5162997672320e-02
4.5414515436167e-01 6.4380877167301e-01 3.7923892500626e-03
1.1207888412514e-01 6.4652448341775e-01 9.9241673637666e-01
7.7158284136337e-01 6.4923614113875e-01 9.8104583203795e-01
4.3116779389052e-01 6.5194784664816e-01 9.6967484868114e-01
9.0833586641480e-02 6.5465957783112e-01 9.5830381508343e-01
7.5057991038618e-01 6.5737131491166e-01 9.4693275745538e-01
4.1040632123953e-01 6.6008304034326e-01 9.3556169938842e-01
7.0312263434465e-02 6.6279473867016e-01 9.2419066185517e-01
7.3029709498528e-01 6.6550639636792e-01 9.1281966325261e-01
3.9036011594760e-01 6.6821800167170e-01 9.0144871947806e-01
5.0500599087602e-02 6.7092954439925e-01 8.9007784402837e-01
7.1071782290549e-01 6.7364101577504e-01 8.7870704811324e-01
3.7101110711362e-01 6.7635240826064e-01 8.6733634077428e-01
3.1379850849368e-02 6.7906371539561e-01 8.5596572900252e-01
6.9182357411047e-01 6.8177493165213e-01 8.4459521784748e-01
3.5234196314386e-01 6.8448605230584e-01 8.3322481051206e-01
1.2934920809387e-02 6.8719707332460e-01 8.2185450842753e-01
6.7360262329625e-01 6.8990799127651e-01 8.1048431130374e-01
3.3434558501754e-01 6.9261880325787e-01 7.9911421714915e-01
9.9516473408498e-01 6.9532950684190e-01 7.8774422225531e-01
6.5606150152439e-01 6.9804010004862e-01 7.7637432113951e-01
3.1703792840816e-01 7.0075058133689e-01 7.6500450643791e-01
9.7809679646534e-01 7.0346094961952e-01 7.5363476873964e-01
6.3924178964348e-01 7.0617120430339e-01 7.4226509634945e-01
3.0047769678138e-01 7.0888134535727e-01 7.3089547496242e-01
9.6181066937299e-01 7.1159137341136e-01 7.1952588722898e-01
6.2324855392338e-01 7.1430128989486e-01 7.0815631218044e-01
2.8480132649729e-01 7.1701109722042e-01 6.9678672447554e-01
9.4648166914842e-01 7.1972079902859e-01 6.8541709341356e-01
6.0830574628480e-01 7.2243040051155e-01 6.7404738164100e-01
2.7029426749447e-01 7.2513990884440e-01 6.6267754345265e-01
9.3247396844849e-01 7.2784933376675e-01 6.5130752255508e-01
5.9487971464650e-01 7.3055868837973e-01 6.3993724912068e-01
2.5755755459641e-01 7.3326799026242e-01 6.2856663592202e-01
9.2056925793229e-01 7.3597726307886e-01 6.1719557332500e-01
5.8399924407004e-01 7.3868653897756e-01 6.0582392301370e-01
2.4796548691509e-01 7.4139586235739e-01 5.9445151074158e-01
9.1263728223154e-01 7.4410529621092e-01 5.8307811976779e-01
5.7826537087823e-01 7.4681493393579e-01 5.7170349066712e-01
2.4523540854974e-01 7.4952492452838e-01 5.6032734475650e-01
9.1416804325706e-01 7.5223553610795e-01 5.4894948102597e-01
5.8611789440040e-01 7.5494734770508e-01 5.3757008471160e-01
2.6299518244337e-01 7.5766193145780e-01 5.2619057833248e-01
9.4849152770318e-01 7.6038451424469e-01 5.1481529484374e-01
1.6989629630825e-01 5.3088075239283e-01 5.0372083459823e-01
8.2675476716756e-01 5.3357436719807e-01 4.9236326899797e-01
4.8366366371249e-01 5.3626803018298e-01 4.8100572626124e-01
1.4062394479958e-01 5.3896177559403e-01 4.6964820470069e-01
7.9763625103614e-01 5.4165563503256e-01 4.5829070261864e-01
4.5470096331699e-01 5.4434963815258e-01 4.4693321749642e-01
1.1181826139070e-01 5.4704381332437e-01 4.3557574528306e-01
7.6898817968105e-01 5.4973818823095e-01 4.2421827979275e-01
4.2621065816613e-01 5.5243279037237e-01 4.1286081221610e-01
8.3485586750441e-02 5.5512764746005e-01 4.0150333074750e-01
7.4081284220154e-01 5.5782278769020e-01 3.9014582032886e-01
3.9819231732162e-01 5.6051823989212e-01 3.7878826250887e-01
5.5623942554110e-02 5.6321403355228e-01 3.6743063541591e-01
7.1310770066493e-01 5.6591019872042e-01 3.5607291384181e-01
3.7064363547332e-01 5.6860676580782e-01 3.4471506943279e-01
2.8231855830899e-02 5.7130376529109e-01 3.3335707098244e-01
6.8587253616007e-01 5.7400122733761e-01 3.2199888482005e-01
3.4356591486755e-01 5.7669918137036e-01 3.1064047528566e-01
1.3122918583894e-03 5.7939765559085e-01 2.9928180528158e-01
6.5911202620271e-01 5.8209667647951e-01 2.8792283688749e-01
3.1696553477297e-01 5.8479626829216e-01 2.7656353202498e-01
9.7487329239302e-01 5.8749645257086e-01 2.6520385315511e-01
6.3283583374577e-01 5.9019724768586e-01 2.5384376399159e-01
2.9085375699533e-01 5.9289866842384e-01 2.4248323021100e-01
9.4892772881269e-01 5.9560072563596e-01 2.3112222014100e-01
6.0705849027056e-01 5.9830342595671e-01 2.1976070540762e-01
2.6524686290478e-01 6.0100677160275e-01 2.0839866152329e-01
9.2349375413935e-01 6.0371076025832e-01 1.9703606839837e-01
5.8180016124246e-01 6.0641538505109e-01 1.8567291076107e-01
2.4016717302321e-01 6.0912063462019e-01 1.7430917847276e-01
8.9859596858853e-01 6.1182649327483e-01 1.6294486672903e-01
5.5708781264727e-01 6.1453294123976e-01 1.5157997614019e-01
2.1564404706126e-01 6.1723995498060e-01 1.4021451268897e-01
8.7426607858433e-01 6.1994750759983e-01 1.2884848756735e-01
5.3295536298219e-01 6.2265556929177e-01 1.1748191689887e-01
1.9171338596946e-01 6.2536410784288e-01 1.0611482135665e-01
8.5054164161677e-01 6.2807308916223e-01 9.4747225691227e-02
5.0944160905535e-01 6.3078247782649e-01 8.3379158185359e-02
1.6841472842678e-01 6.3349223762321e-01 7.2010650055106e-02
8.2746237708506e-01 6.3620233207709e-01 6.0641734817794e-02
4.8658584705469e-01 6.3891272494527e-01 4.9272447647652e-02
1.4578632468695e-01 6.4162338066937e-01 3.7902824739067e-02
8.0506487334473e-01 6.4433426477461e-01 2.6532902695625e-02
4.6442241979778e-01 6.4704534420908e-01 1.5162717960568e-02
1.2385974483803e-01 6.4975658761889e-01 3.7923063011432e-03
7.8189390013417e-01 6.5247209744655e-01 9.9241683880420e-01
4.4148647426584e-01 6.5518361838465e-01 9.8104603514243e-01
1.0116005719209e-01 6.5789521950387e-01 9.6967510514310e-01
7.6091484454683e-01 6.6060687771698e-01 9.5830407803733e-01
4.2075090836872e-01 6.6331857208659e-01 9.4693298080824e-01
8.0668214792600e-02 6.6603028376531e-01 9.3556183807390e-01
7.4066664559079e-01 6.6874199590287e-01 9.2419067202372e-01
4.0074602331616e-01 6.7145369352804e-01 9.1281950240005e-01
6.0906139860185e-02 6.7416536341302e-01 9.0144834651644e-01
7.2114678833352e-01 6.7687699392739e-01 8.9007721930378e-01
3.8146779830221e-01 6.7958857488788e-01 8.7870613337569e-01
4.1869074569360e-02 6.8230009740953e-01 8.6733509910517e-01
7.0235063988050e-01 6.8501155376277e-01 8.5596412470494e-01
3.6291268215669e-01 6.8772293724032e-01 8.4459321630452e-01
2.3555607129975e-02 6.9043424203681e-01 8.3322237801776e-01
6.8428009758395e-01 6.9314546314358e-01 8.2185161199495e-01
3.4508718080853e-01 6.9585659626023e-01 8.1048091845417e-01
5.9783063882898e-03 6.9856763772457e-01 7.9911029568630e-01
6.6695543709932e-01 7.0127858446169e-01 7.8773974002858e-01
3.2802115654470e-01 7.0398943395344e-01 7.7636924580061e-01
9.8917879829325e-01 7.0670018422904e-01 7.6499880519660e-01
6.5043260281272e-01 7.0941083387839e-01 7.5362840812616e-01
3.1178791057215e-01 7.1212138208957e-01 7.4225804199462e-01
9.7325140256661e-01 7.1483182871335e-01 7.3088769141230e-01
6.3483140355017e-01 7.1754217435838e-01 7.1951733781938e-01
2.9653826898375e-01 7.2025242052260e-01 7.0814695901132e-01
9.5838488493702e-01 7.2296256976913e-01 6.9677652854700e-01
6.2038732220312e-01 7.2567262595854e-01 6.8540601502119e-01
2.8256570371106e-01 7.2838259455580e-01 6.7403538118478e-01
9.4494537120779e-01 7.3109248303961e-01 6.6266458290562e-01
6.0755847847730e-01 7.3380230145836e-01 6.5129356798799e-01
2.7044620304637e-01 7.3651206320591e-01 6.3992227492596e-01
9.3366187181930e-01 7.3922178614488e-01 6.2855063179180e-01
5.9727546553047e-01 7.4193149431329e-01 6.1717855572586e-01
2.6138025133954e-01 7.4464122067954e-01 6.0580595404922e-01
9.2610278332889e-01 7.4735101192512e-01 5.9443272917437e-01
5.9161838070654e-01 7.5006093745922e-01 5.8305879187807e-01
2.5817578264025e-01 7.5277110795866e-01 5.7168409237021e-01
9.2613766009546e-01 7.5548171693636e-01 5.6030868806827e-01
5.9604934432732e-01 7.5819314159909e-01 5.4893288291537e-01
2.6875863836209e-01 7.6090620324663e-01 5.3755748626551e-01
9.4562494950341e-01 7.6362285458879e-01 5.2618417044523e-01
6.2884423856525e-01 7.6634782254531e-01 5.1481537600464e-01
8.3657325080060e-01 5.3683916624929e-01 5.0370882518746e-01
4.9337880634794e-01 5.3953276230737e-01 4.9235122008899e-01
1.5024474106637e-01 5.4222641291340e-01 4.8099364223898e-01
8.0717198109944e-01 5.4492015256433e-01 4.6963609072913e-01
4.6416105199756e-01 5.4761401236856e-01 4.5827856486336e-01
1.2121214556883e-01 5.5030802083240e-01 4.4692106328760e-01
7.7832518763781e-01 5.5300220461696e-01 4.3556358323522e-01
4.3549990346945e-01 5.5569658922917e-01 4.2420611989544e-01
9.2735878280973e-02 5.5839119961923e-01 4.1284866590716e-01
7.5003261098335e-01 5.6108606066567e-01 4.0149121097656e-01
4.0738956000494e-01 5.6378119753659e-01 3.9013374161535e-01
6.4806180707224e-02 5.6647663592283e-01 3.7877624099476e-01
7.2228195447572e-01 5.6917240214432e-01 3.6741868891042e-01
3.7981641003946e-01 5.7186852313581e-01 3.5606106185272e-01
3.7409137930113e-02 5.7456502632208e-01 3.4470333317697e-01
6.9505979923512e-01 5.7726193939570e-01 3.3334547336714e-01
3.5276812993147e-01 5.7995929001256e-01 3.2198745038644e-01
1.0533942115293e-02 5.8265710542207e-01 3.1062923010650e-01
6.6835712337969e-01 5.8535541204959e-01 2.9927077680594e-01
3.2623763545270e-01 5.8805423504891e-01 2.8791205372760e-01
9.8417551300724e-01 5.9075359784238e-01 2.7655302368231e-01
6.4217086331310e-01 5.9345352166551e-01 2.6519364968547e-01
3.0022386713760e-01 5.9615402513168e-01 2.5383389561190e-01
9.5833478103501e-01 5.9885512383128e-01 2.4247372685330e-01
6.1650394091433e-01 6.0155682997798e-01 2.3111311096204e-01
2.7473176655432e-01 6.0425915211286e-01 2.1975201826515e-01
9.3301876655962e-01 6.0696209487545e-01 2.0839042243225e-01
5.9136554312867e-01 6.0966565884847e-01 1.9702830098242e-01
2.4977279594209e-01 6.1236984048107e-01 1.8566563571577e-01
9.0824132447829e-01 6.1507463209306e-01 1.7430241305783e-01
5.6677202812176e-01 6.1778002196046e-01 1.6293862430653e-01
2.2536590354150e-01 6.2048599448021e-01 1.5157426577498e-01
8.8402403897394e-01 6.2319253040963e-01 1.4020933882578e-01
5.4274760523493e-01 6.2589960717411e-01 1.2884384979645e-01
2.0153784349384e-01 6.2860719923396e-01 1.1747780981904e-01
8.6039605005512e-01 6.3131527849999e-01 1.0611123454054e-01
5.1932355859294e-01 6.3402381478545e-01 9.4744143754104e-02
1.7832172045906e-01 6.3673277628115e-01 8.3376560954123e-02
8.3739188381976e-01 6.3944213003996e-01 7.2008512830477e-02
4.9653537246798e-01 6.4215184245707e-01 6.0640028719125e-02
1.5575346519523e-01 6.4486187973280e-01 4.9271140026830e-02
8.1504737659588e-01 6.4757220830644e-01 3.7901879647914e-02
4.7441824011740e-01 6.5028279525091e-01 2.6532281389986e-02
1.3386709407275e-01 6.5299360862026e-01 1.5162379423950e-02
7.9339487120500e-01 6.5570461774467e-01 3.7922077713714e-03
4.5153055051541e-01 6.5841989356323e-01 9.9241697341771e-01
1.1121224648408e-01 6.6113124578493e-01 9.8104631936001e-01
7.7097477612874e-01 6.6384270981586e-01 9.6967549393749e-01
4.3081862941584e-01 6.6655426191999e-01 9.5830452649834e-01
9.0744202691242e-02 6.6926588028765e-01 9.4693344449902e-01
7.5075181436238e-01 6.7197754502108e-01 9.3556227333278e-01
4.1084172480280e-01 6.7468923808482e-01 9.2419103621057e-01
7.1014160350146e-02 6.7740094322781e-01 9.1281975408798e-01
7.3126934131144e-01 6.8011264588398e-01 9.0144844563124e-01
3.9160751396829e-01 6.8282433305806e-01 8.9007712721488e-01
5.2028986686639e-02 6.8553599320260e-01 8.7870581294329e-01
7.1253417038327e-01 6.8824761609179e-01 8.6733451468888e-01
3.7312362378536e-01 6.9095919269690e-01 8.5596324213972e-01
3.3798104144873e-02 6.9367071506737e-01 8.4459200285003e-01
6.9455862434459e-01 6.9638217622105e-01 8.3322080228741e-01
3.5540651766940e-01 6.9909357004630e-01 8.2184964387135e-01
1.6343511933631e-02 7.0180489121807e-01 8.1047852899786e-01
6.7737181517860e-01 7.0451613512989e-01 7.9910745704565e-01
3.3849421582200e-01 7.0722729784305e-01 7.8773642535953e-01
9.9971420100311e-01 7.0993837605428e-01 7.7636542920699e-01
6.6103609799884e-01 7.1264936708321e-01 7.6499446170417e-01
3.2246524508763e-01 7.1536026888106e-01 7.5362351370772e-01
9.8400820025773e-01 7.1807108006244e-01 7.4225257366939e-01
6.4567299890395e-01 7.2078179996304e-01 7.3088162745116e-01
3.0746947543870e-01 7.2349242872725e-01 7.1951065810045e-01
9.6940966900950e-01 7.2620296743162e-01 7.0813964558833e-01
6.3150834093811e-01 7.2891341825358e-01 6.9676856651965e-01
2.9378364209125e-01 7.3162378469962e-01 6.8539739383581e-01
9.5625798371634e-01 7.3433407191599e-01 6.7402609655045e-01
6.1895918774860e-01 7.3704428711926e-01 6.6265463959508e-01
2.8192202604708e-01 7.3975444020992e-01 6.5128298391412e-01
9.4519030859029e-01 7.4246454467965e-01 6.3991108706051e-01
6.0881975839171e-01 7.4517461901221e-01 6.2853890473799e-01
2.7288203241939e-01 7.4788468895477e-01 6.1716639407747e-01
9.3747044112174e-01 7.5059479139469e-01 6.0579352002710e-01
6.0270823184456e-01 7.5330498132947e-01 5.9442026724226e-01
2.6876081416926e-01 7.5601534504255e-01 5.8304666149200e-01
9.3585415060881e-01 7.5872602618674e-01 5.7167280693983e-01
6.0430290626700e-01 7.6143727948376e-01 5.6029894788787e-01
2.7455398845574e-01 7.6414958424184e-01 5.4892556100749e-01
9.4725313598550e-01 7.6686388433491e-01 5.3755345857716e-01
6.2333879302469e-01 7.6958206356773e-01 5.2618377921933e-01
3.0413154280817e-01 7.7230765509117e-01 5.1481745764411e-01
5.0325361194161e-01 5.4279753797246e-01 5.0369690545178e-01
1.5999620686939e-01 5.4549111084772e-01 4.9233925375273e-01
8.1680948651064e-01 5.4818474551428e-01 4.8098163345639e-01
4.7369434923208e-01 5.5087847689767e-01 4.6962404454079e-01
1.3065120339261e-01 5.5357233572951e-01 4.5826648742484e-01
7.8768004414069e-01 5.5626634944007e-01 4.4690896202253e-01
4.4478053198031e-01 5.5896054302465e-01 4.3555146693256e-01
1.0195206927937e-01 5.6165493984269e-01 4.2419399876936e-01
7.5919387168519e-01 5.6434956231870e-01 4.1283655163435e-01
4.1650503225341e-01 5.6704443252494e-01 4.0147911672206e-01
7.3884576913883e-02 5.6973957263395e-01 3.9012168205358e-01
7.3133151063886e-01 5.7243500523684e-01 3.7876423232859e-01
3.8884485431337e-01 5.7513075352949e-01 3.6740674888726e-01
4.6423672819287e-02 5.7782684137339e-01 3.5604920977391e-01
7.0406709522965e-01 5.8052329324167e-01 3.4469158989443e-01
3.6177432827114e-01 5.8322013406370e-01 3.3333386125996e-01
1.9544664359193e-02 5.8591738898340e-01 3.2197599330945e-01
6.7737748555351e-01 5.8861508304752e-01 3.1061795330307e-01
3.3527226473566e-01 5.9131324084109e-01 2.9925970677816e-01
9.9322856519068e-01 5.9401188608662e-01 2.8790121805830e-01
6.5124603959730e-01 5.9671104122401e-01 2.7654245080527e-01
3.0932442921396e-01 5.9941072698668e-01 2.6518336860248e-01
9.6746356380710e-01 6.0211096198887e-01 2.5382393555769e-01
6.2566336262140e-01 6.0481176233738e-01 2.4246411691180e-01
2.8392383645394e-01 6.0751314127992e-01 2.3110387964013e-01
9.4224509067947e-01 6.1021510890033e-01 2.1974319303234e-01
6.0062732889402e-01 6.1291767186950e-01 2.0838202923694e-01
2.5907085670761e-01 6.1562083325879e-01 1.9702036375707e-01
9.1757608512994e-01 6.1832459242129e-01 1.8565817588505e-01
5.7614353295865e-01 6.2102894494402e-01 1.7429544906428e-01
2.3477382759781e-01 6.2373388267248e-01 1.6293217116881e-01
8.9346770380127e-01 6.2643939380694e-01 1.5156833469332e-01
5.5222599994579e-01 6.2914546306781e-01 1.4020393684820e-01
2.1104965158326e-01 6.3185207192544e-01 1.2883897955755e-01
8.6993968218938e-01 6.3455919888805e-01 1.1747346936084e-01
5.2889719120590e-01 6.3726681983935e-01 1.0610741722191e-01
1.8792333965215e-01 6.3997490841634e-01 9.4740838252118e-02
8.4701933374631e-01 6.4268343641625e-01 8.3373751357116e-02
5.0618640711741e-01 6.4539237422116e-01 7.2006178819184e-02
1.6542580229560e-01 6.4810169122820e-01 6.0638145828898e-02
8.2473875223525e-01 6.5081135627376e-01 4.9269679981127e-02
4.8412646265014e-01 6.5352133804061e-01 3.7900810750900e-02
1.4359009592356e-01 6.5623160543805e-01 2.6531568964451e-02
8.0313075730192e-01 6.5894212794692e-01 1.5161986279910e-02
4.6274948399722e-01 6.6165287592275e-01 3.7920946905552e-03
1.2098759335584e-01 6.6436788787578e-01 9.9241713826535e-01
7.8075875263393e-01 6.6707904143316e-01 9.8104668000946e-01
4.4061041364025e-01 6.6979033739946e-01 9.6967600794526e-01
1.0054330654103e-01 6.7250175170985e-01 9.5830515122691e-01
7.6055809682498e-01 6.7521326197678e-01 9.4693413749858e-01
4.2065539890557e-01 6.7792484751569e-01 9.3556299267931e-01
8.0835794128604e-02 6.8063648933512e-01 9.2419174079759e-01
7.4109985316504e-01 6.8334817009670e-01 9.1282040387129e-01
4.0144816279603e-01 6.8605987405098e-01 9.0144900182791e-01
6.1881357157726e-02 6.8877158695514e-01 8.9007755245891e-01
7.2240015360857e-01 6.9148329597813e-01 8.7870607140202e-01
3.8300539351374e-01 6.9419498959868e-01 8.6733457214497e-01
4.3698088410862e-02 6.9690665750088e-01 8.5596306604438e-01
7.0447947223377e-01 6.9961829047173e-01 8.4459156235406e-01
3.6535106053417e-01 7.0232988030411e-01 8.3322006825708e-01
2.6314717964399e-02 7.0504141970842e-01 8.2184858889694e-01
6.8737273568622e-01 7.0775290223529e-01 8.1047712740362e-01
3.4852792087205e-01 7.1046432221171e-01 7.9910568491089e-01
9.7837010997044e-03 7.1317567469212e-01 7.8773426056237e-01
6.7114424725355e-01 7.1588695542639e-01 7.7636285150442e-01
3.3261461959700e-01 7.1859816084624e-01 7.6499145286519e-01
9.9420094305864e-01 7.2130928807204e-01 7.5362005772074e-01
6.5591061959960e-01 7.2402033494263e-01 7.4224865705109e-01
3.1775258797205e-01 7.2673130007169e-01 7.3087723969235e-01
9.7973764448223e-01 7.2944218293607e-01 7.1950579229587e-01
6.4187884288428e-01 7.3215298400433e-01 7.0813429931287e-01
3.0419199776172e-01 7.3486370491828e-01 6.9676274303457e-01
9.6669632444474e-01 7.3757434874813e-01 6.8539110373647e-01
6.2941526077077e-01 7.4028492035426e-01 6.7401936000426e-01
2.9237753347862e-01 7.4299542691053e-01 6.6264748936475e-01
9.5561855724859e-01 7.4570587868149e-01 6.5127546941701e-01
6.1918229120675e-01 7.4841629021300e-01 6.3990327977198e-01
2.8312373202398e-01 7.5112668221739e-01 6.2853090528281e-01
9.4751230369619e-01 7.5383708466009e-01 6.1715834131240e-01
6.1243652563978e-01 7.5654754198011e-01 6.0578560216433e-01
2.7801052343377e-01 7.5925812219099e-01 5.9441273429690e-01
9.4438321765175e-01 7.6196893317447e-01 5.8303983641606e-01
6.1175141013398e-01 7.6468015246586e-01 5.7166708845993e-01
2.8037845556193e-01 7.6739208229797e-01 5.6029478913159e-01
9.5062047963008e-01 7.7010525050638e-01 5.4892339230433e-01
6.2296093556460e-01 7.7282058666663e-01 5.3755350530433e-01
2.9804715447236e-01 7.7553968466118e-01 5.2618574915926e-01
9.7669598320654e-01 7.7826500148052e-01 5.1482029643247e-01
1.6993751349453e-01 5.4875586067699e-01 5.0368508958543e-01
8.2660673445765e-01 5.5144940536470e-01 4.9232738410338e-01
4.8335736961483e-01 5.5414302011464e-01 4.8096971390108e-01
1.4019029343845e-01 5.5683674046945e-01 4.6961207996046e-01
7.9710579208285e-01 5.5953059689422e-01 4.5825448394308e-01
4.5410365212680e-01 5.6222461579639e-01 4.4689692715003e-01
1.1118325193776e-01 5.6491882052141e-01 4.3553940964220e-01
7.6834365110369e-01 5.6761323227649e-01 4.2418192952452e-01
4.2558367434235e-01 5.7030787094836e-01 4.1282448238957e-01
8.2901987307022e-02 5.7300275579258e-01 4.0146706091054e-01
7.4029716267036e-01 5.7569790598264e-01 3.9010965457102e-01
3.9776773572342e-01 5.7839334101531e-01 3.7875224951841e-01
5.5312249443562e-02 5.8108908097553e-01 3.6739482852807e-01
7.1292928955019e-01 5.8378514666874e-01 3.5603737106679e-01
3.7061751048429e-01 5.8648155963264e-01 3.4467985344501e-01
2.8375653525682e-02 5.8917834204222e-01 3.3332224904892e-01
6.8620255841787e-01 5.9187551652395e-01 3.2196452864373e-01
3.4409716991880e-01 5.9457310589544e-01 3.1060666074052e-01
2.0585406568257e-03 5.9727113284747e-01 2.9924861201854e-01
6.6008583156012e-01 5.9996961958458e-01 2.8789034779484e-01
3.1817831096238e-01 6.0266858744044e-01 2.7653183253252e-01
9.7633535328445e-01 6.0536805648289e-01 2.6517303037803e-01
6.3455643796571e-01 6.0806804512262e-01 2.5381390571753e-01
2.9284114908529e-01 6.1076856973826e-01 2.4245442374122e-01
9.5118917588607e-01 6.1346964432922e-01 2.3109455100443e-01
6.0960031420374e-01 6.1617128020624e-01 2.1973425597360e-01
2.6807446862204e-01 6.1887348572799e-01 2.0837350954530e-01
9.2661165502891e-01 6.2157626609074e-01 1.9701228552670e-01
5.8521200314413e-01 6.2427962317619e-01 1.8565056106619e-01
2.4387575853028e-01 6.2698355546149e-01 1.7428831702395e-01
9.0260328358510e-01 6.2968805799318e-01 1.6292553827336e-01
5.6139505704473e-01 6.3239312242574e-01 1.5156221392576e-01
2.2025167159827e-01 6.3509873712332e-01 1.4019833747284e-01
8.7917382931946e-01 6.3780488732168e-01 1.2883390684333e-01
5.3816233475270e-01 6.4051155534579e-01 1.1746892437299e-01
1.9721808563845e-01 6.4321872087682e-01 1.0610339668942e-01
8.5634206141751e-01 6.4592636126100e-01 9.4737334515735e-02
5.1553530980401e-01 6.4863445185154e-01 8.3370752400052e-02
1.7479893185273e-01 6.5134296637399e-01 7.2003668379327e-02
8.3413406606019e-01 6.5405187730481e-01 6.0636103588652e-02
4.9354187212265e-01 6.5676115625291e-01 4.9268081828141e-02
1.5302351502473e-01 6.5947077433412e-01 3.7899629100640e-02
8.1258015014782e-01 6.6218070252925e-01 2.6530773133709e-02
4.7221291006952e-01 6.6489091201748e-01 1.5161542899087e-02
1.3192289367771e-01 6.6760137447831e-01 3.7919681419412e-03
7.9026417202834e-01 6.7031609503159e-01 9.9241733125759e-01
4.5012499574916e-01 6.7302702209900e-01 9.8104711198764e-01
1.1006586262141e-01 6.7573812092793e-01 9.6967663924842e-01
7.7008769184520e-01 6.7844936743554e-01 9.5830594171601e-01
4.3019136549812e-01 6.8116073894996e-01 9.4693504695237e-01
9.0377740085740e-02 6.8387221427059e-01 9.3556398115716e-01
7.5064766251891e-01 6.8658377369331e-01 9.2419276896531e-01
4.1100199059490e-01 6.8929539900536e-01 9.1282143328808e-01
7.1441618071992e-02 6.9200707345457e-01 9.0144999519422e-01
7.3196750447078e-01 6.9471878169818e-01 8.9007847382677e-01
3.9258070981245e-01 6.9743050973626e-01 8.7870688635054e-01
5.3282434616245e-02 7.0014224483488e-01 8.6733524792534e-01
7.1407406562814e-01 7.0285397544352e-01 8.5596357169948e-01
3.7495722794382e-01 7.0556569111102e-01 8.4459186881885e-01
3.5933844429060e-02 7.0827738240376e-01 8.3322014844697e-01
6.9700620363734e-01 7.1098904082959e-01 8.2184841779207e-01
3.5817703779309e-01 7.1370065877008e-01 8.1047668213822e-01
1.9449612867485e-02 7.1641222942395e-01 7.9910494487815e-01
6.8082783334942e-01 7.1912374676369e-01 7.8773320754677e-01
3.4231636504389e-01 7.2183520550777e-01 7.7636146985563e-01
3.9207801630766e-03 7.2454660111073e-01 7.6498972973057e-01
6.6564773018145e-01 7.2725792977386e-01 7.5361798335683e-01
3.2750515350003e-01 7.2996918848037e-01 7.4224622523948e-01
9.8950252703662e-01 7.3268037505993e-01 7.3087444829137e-01
6.5165117361226e-01 7.3539148829062e-01 7.1950264396717e-01
3.1396464069470e-01 7.3810252804967e-01 7.0813080247174e-01
9.7645917104919e-01 7.4081349553124e-01 6.9675891308412e-01
6.3915429265098e-01 7.4352439355961e-01 6.8538696465868e-01
3.0207356457084e-01 7.4623522704301e-01 6.7401494639336e-01
9.6524552851536e-01 7.4894600364128e-01 6.6264284899660e-01
6.2870493382255e-01 7.5165673476696e-01 6.5127066644386e-01
2.9249432917714e-01 7.5436743711854e-01 6.3989839859704e-01
9.5666615023749e-01 7.5707813508024e-01 6.2852605507007e-01
6.2128548300622e-01 7.5978886455692e-01 6.1715366085657e-01
2.8643375341815e-01 7.6249967922049e-01 6.0578126436314e-01
9.5221368921495e-01 7.6521066084803e-01 5.9440894851894e-01
6.1875601975917e-01 7.6792193662496e-01 5.8303684530413e-01
2.8622849764328e-01 7.7063370819796e-01 5.7166515273343e-01
9.5484783671524e-01 7.7334629992025e-01 5.6029414966384e-01
6.2489471612974e-01 7.7606023589696e-01 5.4892419507048e-01
2.9673008218263e-01 7.7877635092937e-01 5.3755568084973e-01
9.7080497659363e-01 7.8149590753148e-01 5.2618888162801e-01
6.4764089573737e-01 7.8422057164075e-01 5.1482364513126e-01
8.3662511495238e-01 5.5471412744169e-01 5.0367339193297e-01
4.9321010626195e-01 5.5740763825641e-01 4.9231562540025e-01
1.4988774578930e-01 5.6010122860013e-01 4.8095789769917e-01
8.0665888683149e-01 5.6279493483514e-01 4.6960021094540e-01
4.6352368502713e-01 5.6548878726546e-01 4.5824256818703e-01
1.2048170150447e-01 5.6818281130993e-01 4.4688497224858e-01
7.7753200950135e-01 5.7087702865118e-01 4.3552742476615e-01
4.3467329899376e-01 5.7357145830485e-01 4.2416992541495e-01
9.1903975068031e-02 5.7626611757017e-01 4.1281247131996e-01
7.4922224700603e-01 5.7896102283767e-01 4.0145505663433e-01
4.0662620620772e-01 5.8165619024211e-01 3.9009767226722e-01
6.4113892080819e-02 5.8435163615807e-01 3.7874030574242e-01
7.2168334584908e-01 5.8704737754348e-01 3.6738294117030e-01
3.7933265286308e-01 5.8974343214088e-01 3.5602555931771e-01
3.7059974450387e-02 5.9243981855012e-01 3.4466813776256e-01
6.9486357063562e-01 5.9513655618812e-01 3.3331065112201e-01
3.5274181521870e-01 5.9783366515241e-01 3.2195307134487e-01
1.0693204745136e-02 6.0053116600559e-01 3.1059536805998e-01
6.6871636285841e-01 6.0322907949767e-01 2.9923750897298e-01
3.2681004141130e-01 6.0592742624259e-01 2.8787946030430e-01
9.8497311954843e-01 6.0862622636464e-01 2.7652118726077e-01
6.4320460177351e-01 6.1132549912909e-01 2.6516265453321e-01
3.0150361579497e-01 6.1402526257062e-01 2.5380382681160e-01
9.5986941071775e-01 6.1672553313149e-01 2.4244466930907e-01
6.1830135592699e-01 6.1942632532025e-01 2.3108514828511e-01
2.7679894080273e-01 6.2212765140052e-01 2.1972523155848e-01
9.3536177522165e-01 6.2482952111784e-01 2.0836488899962e-01
5.9398959064944e-01 6.2753194147120e-01 1.9700409299261e-01
2.5268224151043e-01 6.3023491653480e-01 1.8564281885697e-01
9.1143970644380e-01 6.3293844733373e-01 1.7428104521995e-01
5.7026208901784e-01 6.3564253177631e-01 1.6291875433105e-01
2.2914961747638e-01 6.3834716464408e-01 1.5155593231144e-01
8.8810264313075e-01 6.4105233763925e-01 1.4019256933238e-01
5.4712163708263e-01 6.4375803948775e-01 1.2882865971870e-01
2.0620718506247e-01 6.4646425609486e-01 1.1746420197495e-01
8.6535998028631e-01 6.4917097074874e-01 1.0609919873430e-01
5.2458081436396e-01 6.5187816436610e-01 9.4733656632226e-02
1.8387056642411e-01 6.5458581577307e-01 8.3367586109277e-02
8.4323019074847e-01 6.5729390201338e-01 7.2001001149397e-02
5.0266070331991e-01 6.6000239867532e-01 6.0633918961973e-02
1.6216316778210e-01 6.6271128022854e-01 4.9266359617419e-02
8.2173868137449e-01 6.6542052036199e-01 3.7898345647197e-02
4.8138836144567e-01 6.6813009231420e-01 2.6529901619839e-02
1.4111333315746e-01 6.7083996918818e-01 1.5161053704663e-02
8.0091471897516e-01 6.7355012424412e-01 3.7918292345701e-03
4.5935976354157e-01 6.7626452821326e-01 9.9241755022715e-01
1.1931033546810e-01 6.7897520316044e-01 9.8104760993037e-01
7.7934039356607e-01 6.8168607779783e-01 9.6967737940395e-01
4.3945099839874e-01 6.8439712832145e-01 9.5830688658940e-01
9.9643199297657e-02 6.8710833206278e-01 9.4693615871389e-01
7.5991804396513e-01 6.8981966757726e-01 9.3556522201906e-01
4.2027659239458e-01 6.9253111469997e-01 9.2419410153070e-01
8.0719935268407e-02 6.9524265457154e-01 9.1282282086902e-01
7.4124921699449e-01 6.9795426963814e-01 9.0145140209753e-01
4.0186566357128e-01 7.0066594363000e-01 8.9007986560788e-01
6.2570615530208e-02 7.0337766152264e-01 8.7870823003722e-01
7.2336556629597e-01 7.0608940948556e-01 8.6733651221430e-01
3.8425220643138e-01 7.0880117482265e-01 8.5596472713022e-01
4.5232474399886e-02 7.1151294590830e-01 8.4459288793020e-01
7.0630861468877e-01 7.1422471212327e-01 8.3322100592272e-01
3.6748324439900e-01 7.1693646379356e-01 8.2184909060329e-01
2.8759429733537e-02 7.1964819213559e-01 8.1047714969073e-01
6.9014077422087e-01 7.2235988921062e-01 7.9910518917487e-01
3.5163152101480e-01 7.2507154789110e-01 7.8773321337610e-01
1.3236672245231e-02 7.2778316184194e-01 7.7636122501859e-01
6.7496212919521e-01 7.3049472551987e-01 7.6498922532138e-01
3.3681485810092e-01 7.3320623419472e-01 7.5361721411414e-01
9.9880308768404e-01 7.3591768399782e-01 7.4224518998817e-01
6.6093654622639e-01 7.3862907200456e-01 7.3087315049813e-01
3.2322674821774e-01 7.4134039636169e-01 7.1950109243663e-01
9.8568734352673e-01 7.4405165647472e-01 7.0812901221299e-01
6.4833454590985e-01 7.4676285327884e-01 6.9675690637999e-01
3.1118766282066e-01 7.4947398962925e-01 6.8538477236914e-01
9.7426975537430e-01 7.5218507086648e-01 6.7401260951799e-01
6.3760846658896e-01 7.5489610564309e-01 6.6264042050233e-01
3.0123706850954e-01 7.5760710714864e-01 6.5126821332366e-01
9.6519579560108e-01 7.6031809494925e-01 6.3989600404484e-01
6.2953355417023e-01 7.6302909778798e-01 6.2852382050780e-01
2.9431012674940e-01 7.6574015789970e-01 6.1715170728370e-01
9.5959902661467e-01 7.6845133772393e-01 6.0577973204706e-01
6.2549119760649e-01 7.7116273040782e-01 5.9440799331559e-01
2.9209978446019e-01 7.7387447622451e-01 5.8303662882094e-01
9.5956617723182e-01 7.7658678793452e-01 5.7166582221664e-01
6.2806734754486e-01 7.7929998871939e-01 5.6029580261464e-01
2.9782386307966e-01 7.8201456486251e-01 5.4892682558423e-01
9.6910624911688e-01 7.8473122660647e-01 5.3755911573594e-01
6.4223333219720e-01 7.8745094163222e-01 5.2619274568935e-01
3.1754842164009e-01 7.9017483292068e-01 5.1482744943490e-01
5.0331660484744e-01 5.6067233131016e-01 5.0366182699629e-01
1.5980598490870e-01 5.6336580176177e-01 4.9230399205675e-01
8.1639984310728e-01 5.6605936259257e-01 4.8094619912581e-01
4.7309901294981e-01 5.6875305120503e-01 4.6958845159770e-01
1.2990350738139e-01 5.7144689784324e-01 4.5823075406985e-01
7.8681263938582e-01 5.7414092695122e-01 4.4687311104569e-01
4.4382514690269e-01 5.7683515850462e-01 4.3551552586675e-01
1.0093931546005e-01 5.7952960925064e-01 4.2415799987233e-01
7.5815309343461e-01 5.8222429381141e-01 4.1280053177191e-01
4.1546419638475e-01 5.8491922562463e-01 4.0144311720813e-01
7.2870198309519e-02 5.8761441770931e-01 3.9008574848508e-01
7.3036860888413e-01 5.9030988325647e-01 3.7872841443692e-01
3.8795693667783e-01 5.9300563605198e-01 3.6737110041379e-01
4.5632739072614e-02 5.9570169074469e-01 3.5601378836515e-01
7.0339366009452e-01 5.9839806297591e-01 3.4465645700429e-01
3.6123745767471e-01 6.0109476938828e-01 3.3329908204036e-01
1.9162022009450e-02 6.0379182753229e-01 3.2194163646754e-01
6.7716538671960e-01 6.0648925568907e-01 3.1058409090256e-01
3.3524573445025e-01 6.0918707262679e-01 2.9922641396325e-01
9.9340139842304e-01 6.1188529730749e-01 2.8786857268166e-01
6.5163086127954e-01 6.1458394855981e-01 2.7651053294542e-01
3.0993275234850e-01 6.1728304473177e-01 2.6515225996118e-01
9.6830584424870e-01 6.1998260333659e-01 2.5379371873338e-01
6.2674904951307e-01 6.2268264070301e-01 2.4243487455139e-01
2.8526141769968e-01 6.2538317164046e-01 2.3107569347737e-01
9.4384213324912e-01 6.2808420912797e-01 2.1971614282706e-01
6.0249051416288e-01 6.3078576403448e-01 2.0835619163505e-01
2.6120601142041e-01 6.3348784487696e-01 1.9699581109619e-01
9.1998820892583e-01 6.3619045762165e-01 1.8563497497467e-01
5.7883682368405e-01 6.3889360553227e-01 1.7427365997269e-01
2.3775170584996e-01 6.4159728906806e-01 1.6291184605111e-01
8.9673283827395e-01 6.4430150583334e-01 1.5154951669519e-01
5.5578033518085e-01 6.4700625057882e-01 1.4018665911972e-01
2.1489443966371e-01 6.4971151525394e-01 1.2882326440897e-01
8.7407551974512e-01 6.5241728910822e-01 1.1745932758860e-01
5.3332406285070e-01 6.5512355883828e-01 1.0609484762809e-01
1.9264066864589e-01 6.5783030877624e-01 9.4729827374488e-02
8.5202604030129e-01 6.6053752111405e-01 8.3364273419693e-02
5.1148097436585e-01 6.6324517615746e-01 7.1998195905756e-02
1.7100634953438e-01 6.6595325260260e-01 6.0631608274129e-02
8.3060311469022e-01 6.6866172782759e-01 4.9264526966462e-02
4.9027227667972e-01 6.7137057819162e-01 3.7896971085749e-02
1.5001488832906e-01 6.7407977933362e-01 2.6528962027459e-02
8.0983203724449e-01 6.7678930646352e-01 1.5160523090818e-02
4.6972483594369e-01 6.7949913463923e-01 3.7916790804194e-03
1.2827412705692e-01 6.8221319916886e-01 9.9241779297945e-01
7.8831443593587e-01 6.8492359858034e-01 9.8104816834996e-01
4.4843360160301e-01 6.8763422404774e-01 9.6967821968119e-01
1.0863278215048e-01 6.9034505232554e-01 9.5830797395088e-01
7.6891314846547e-01 6.9305606102562e-01 9.4693745781729e-01
4.2927589204265e-01 6.9576722872813e-01 9.3556669733411e-01
8.9722236905039e-02 6.9847853506221e-01 9.2419571770102e-01
7.5025345586032e-01 7.0118996075882e-01 9.1282454305069e-01
4.1087089131342e-01 7.0390148767843e-01 9.0145319627210e-01
7.1575980872828e-02 7.0661309881695e-01 8.9008169886895e-01
7.3237028803088e-01 7.0932477829380e-01 8.7871007085120e-01
3.9325553826984e-01 7.1203651132586e-01 8.6733833065719e-01
5.4233661049280e-02 7.1474828419141e-01 8.5596649510375e-01
7.1530683826890e-01 7.1746008418796e-01 8.4459457936150e-01
3.7647755997994e-01 7.2017189958756e-01 8.3322259695299e-01
3.7748688343416e-02 7.2288371959342e-01 8.2185055977182e-01
6.9912353111372e-01 7.2559553430103e-01 8.1047847812153e-01
3.6060592627319e-01 7.2830733466720e-01 7.9910636077424e-01
2.2200339873746e-02 7.3101911249026e-01 7.8773421505039e-01
6.8391197967763e-01 7.3373086040520e-01 7.7636204692241e-01
3.4574692785868e-01 7.3644257189753e-01 7.6498986114769e-01
7.7122968695611e-03 7.3915424134114e-01 7.5361766143860e-01
6.6981641364817e-01 7.4186586406670e-01 7.4224545068124e-01
3.3206903869835e-01 7.4457743646970e-01 7.3087323121882e-01
9.9448162832601e-01 7.4728895617118e-01 7.1950100522176e-01
6.5706765056194e-01 7.5000042224977e-01 7.0812877517395e-01
3.1984296821439e-01 7.5271183557299e-01 6.9675654451445e-01
9.8282630627506e-01 7.5542319926869e-01 6.8538431848568e-01
6.4603982581305e-01 7.5813451939837e-01 6.7401210525373e-01
3.0950983285868e-01 7.6084580592510e-01 6.6263991738195e-01
9.7326765897433e-01 7.6355707411618e-01 6.5126777375442e-01
6.3735076058523e-01 7.6626834659305e-01 6.3989570205331e-01
3.0180409684739e-01 7.6897965634954e-01 6.2852374188217e-01
9.6668186032165e-01 7.7169105122091e-01 6.1715194856759e-01
6.3204964862304e-01 7.7440260051560e-01 6.0578039751312e-01
2.9798717164415e-01 7.7711440482610e-01 5.9440918862662e-01
9.6459157054054e-01 7.7982661037148e-01 5.8303844963015e-01
6.3198134060375e-01 7.8253942940364e-01 5.7166833573872e-01
3.0030060978578e-01 7.8525316765086e-01 5.6029902096479e-01
9.6972294027244e-01 7.8796825695652e-01 5.4893067313352e-01
6.4045254807257e-01 7.9068528273325e-01 5.3756340186698e-01
3.1271840220144e-01 7.9340497512742e-01 5.2619717176622e-01
9.8675298150268e-01 7.9612809258000e-01 5.1483169657809e-01
1.7001220482120e-01 5.6663046529457e-01 5.0365040943697e-01
8.2639397117358e-01 5.6932388794925e-01 4.9229249864498e-01
4.8289274602317e-01 5.7201741342802e-01 4.8093463261156e-01
1.3950934288882e-01 5.7471108041631e-01 4.6957681617253e-01
7.9624361522045e-01 5.7740491919534e-01 4.5821905566008e-01
4.5309459824681e-01 5.8009895322495e-01 4.4686135743394e-01
1.1006065596855e-01 5.8279320069456e-01 4.3550372668974e-01
7.6713962526549e-01 5.8548767596459e-01 4.2414616653844e-01
4.2432895102360e-01 5.8818239084716e-01 4.1278867733525e-01
8.1625808116972e-02 5.9087735569699e-01 4.0143125622707e-01
7.3902720781585e-01 5.9357258030149e-01 3.9007389688485e-01
3.9653008763535e-01 5.9626807457227e-01 3.7871658938808e-01
5.4131384758485e-02 5.9896384904897e-01 3.6735932023178e-01
7.1182809397549e-01 6.0165991523242e-01 3.5600207243126e-01
3.6961731161633e-01 6.0435628576687e-01 3.4464482570419e-01
2.7496267264559e-02 6.0705297449220e-01 3.3328755671445e-01
6.8546234517555e-01 6.0974999638723e-01 3.2193023936536e-01
3.4351309732679e-01 6.1244736742400e-01 3.1057284513343e-01
1.6462499368317e-03 6.1514510435201e-01 2.9921534343527e-01
6.5985970513473e-01 6.1784322442948e-01 2.8785770202188e-01
3.1815153926401e-01 6.2054174511750e-01 2.7649988739535e-01
9.7651999908408e-01 6.2324068375103e-01 2.6514186524294e-01
6.3496349689917e-01 6.2594005719944e-01 2.5378360088349e-01
2.9348060541352e-01 6.2863988152752e-01 2.4242505972096e-01
9.5207005288681e-01 6.3134017166693e-01 2.3106620769915e-01
6.1073071895663e-01 6.3404094110639e-01 2.1970701175134e-01
2.6946163130646e-01 6.3674220160780e-01 2.0834744023830e-01
9.2826196319655e-01 6.3944396295453e-01 1.9698746336756e-01
5.8713103174124e-01 6.4214623273675e-01 1.8562705358696e-01
2.4606829671443e-01 6.4484901617765e-01 1.7426618594536e-01
9.0507335959440e-01 6.4755231600361e-01 1.6290483841382e-01
5.6414596252171e-01 6.5025613236013e-01 1.5154299216092e-01
2.2328598683631e-01 6.5296046277424e-01 1.4018063177665e-01
8.8249345088260e-01 6.5566530216342e-01 1.2881774544028e-01
5.4176850681808e-01 6.5837064288970e-01 1.1745432502881e-01
2.0111143623035e-01 6.6107647485681e-01 1.0609036616379e-01
8.6052264445230e-01 6.6378278564710e-01 9.4725868196034e-02
5.2000265356053e-01 6.6648956069414e-01 8.3360834129094e-02
1.7955209414287e-01 6.6919678348601e-01 7.1995270484228e-02
8.3917169601882e-01 6.7190443579363e-01 6.0629187110917e-02
4.9886227818867e-01 6.7461249791782e-01 4.9262596948626e-02
1.5862473836618e-01 6.7732094894858e-01 3.7895515746654e-02
8.1846004251304e-01 6.8002976702983e-01 2.6527961749968e-02
4.7836921483778e-01 6.8273892962322e-01 1.5159955359560e-02
1.3835332874689e-01 6.8544841376462e-01 3.7915187762455e-03
7.9700725367590e-01 6.8816211826960e-01 9.9241805733360e-01
4.5713721460940e-01 6.9087222092044e-01 9.8104878174875e-01
1.1734535418062e-01 6.9358257432412e-01 9.6967915126077e-01
7.7763288819124e-01 6.9629315605329e-01 9.5830919168071e-01
4.3800106727032e-01 6.9900394426349e-01 9.4693892888275e-01
9.8451183089363e-02 7.0171491782035e-01 9.3556838851349e-01
7.5898457840377e-01 7.0442605640018e-01 9.2419759573106e-01
4.1960266119981e-01 7.0713734056524e-01 9.1282657497610e-01
8.0306923215431e-02 7.0984875181547e-01 9.0145534977582e-01
7.4109896311097e-01 7.1256027261937e-01 8.9008394258122e-01
4.0198051459500e-01 7.1527188642701e-01 8.7871237463681e-01
6.2953479864602e-02 7.1798357766850e-01 8.6734066588183e-01
7.2401996880084e-01 7.2069533174157e-01 8.5596883488113e-01
3.8518234447320e-01 7.2340713499166e-01 8.4459689878447e-01
4.6443275654271e-02 7.2611897468840e-01 8.3322487331261e-01
7.0780579723370e-01 7.2883083900184e-01 8.2185277276918e-01
3.6927337965541e-01 7.3154271698220e-01 8.1048061007818e-01
3.0850008792617e-02 7.3425459854666e-01 7.9910839684754e-01
6.9254027803419e-01 7.3696647447701e-01 7.8773614346074e-01
3.5434949479923e-01 7.3967833643240e-01 7.7636385919989e-01
1.6283804245655e-02 7.4239017698216e-01 7.6499155240568e-01
6.7835033362053e-01 7.4510198966461e-01 7.5361923068216e-01
3.4055736155187e-01 7.4781376908006e-01 7.4224690115722e-01
2.9145176486158e-03 7.5052551102853e-01 7.3087457081341e-01
6.6543301911804e-01 7.5323721270721e-01 7.1950224690835e-01
3.2812595280236e-01 7.5594887298882e-01 7.0812993750924e-01
9.9100861317119e-01 7.5866049281095e-01 6.9675765217216e-01
6.5409890949467e-01 7.6137207571994e-01 6.8538540280369e-01
3.1741785878601e-01 7.6408362863229e-01 6.7401320474828e-01
9.8099018525852e-01 7.6679516290488e-01 6.6264107814877e-01
6.4484505205500e-01 7.6950669584637e-01 6.5126904962480e-01
3.0901695677317e-01 7.7221825286075e-01 6.3989715429715e-01
9.7354682832257e-01 7.7492987049606e-01 6.2852543814181e-01
6.3848336746636e-01 7.7764160078035e-01 6.1715396056118e-01
3.0388467357795e-01 7.8035351736201e-01 6.0578279687075e-01
9.6982018800583e-01 7.8306572410634e-01 5.9441204005323e-01
6.3637294375092e-01 7.8577836685493e-01 5.8304180053792e-01
3.0364200900288e-01 7.8849164881193e-01 5.7167220182090e-01
9.7174478537226e-01 7.9120584897345e-01 5.6030336843511e-01
6.4081836013387e-01 7.9392134008564e-01 5.4893540144971e-01
3.1101825626460e-01 7.9663859582152e-01 5.3756833672548e-01
9.8251155194650e-01 7.9935816346050e-01 5.2620208637649e-01
6.5545972558266e-01 8.0208055738824e-01 5.1483638189066e-01
8.3671217461995e-01 5.7258852238315e-01 5.0363915407395e-01
4.9297359498251e-01 5.7528188870579e-01 4.9228115989565e-01
1.4936537536478e-01 5.7797537213269e-01 4.8092321274389e-01
8.0588830716868e-01 5.8066901290211e-01 4.6956531908174e-01
4.6254206192243e-01 5.8336284142462e-01 4.5820748718864e-01
1.1932535913560e-01 5.8605688013956e-01 4.4684972548352e-01
7.7623614008536e-01 5.8875114533023e-01 4.3549204118491e-01
4.3327173769866e-01 5.9144564880495e-01 4.2413443929671e-01
9.0429035203434e-02 5.9414039938450e-01 4.1277692188828e-01
7.4770460867532e-01 5.9683540416457e-01 4.0141948762754e-01
4.0509485076896e-01 5.9953066954368e-01 3.9006213152257e-01
6.2596074676302e-02 6.0222620202251e-01 3.7870484482935e-01
7.2020459867455e-01 6.0492200879083e-01 3.6734761508863e-01
3.7791681254324e-01 6.0761809812394e-01 3.5599042626064e-01
3.5729227706307e-02 6.1031447961318e-01 3.4463325893356e-01
6.9363851326040e-01 6.1301116425584e-01 3.3327609058686e-01
3.5164152015435e-01 6.1570816442820e-01 3.2191889589615e-01
9.7352957437611e-03 6.1840549376465e-01 3.1056164706962e-01
6.6791709080753e-01 6.2110316696289e-01 2.9920431420931e-01
3.2618436091442e-01 6.2380119953367e-01 2.8784686569204e-01
9.8453476379450e-01 6.2649960751113e-01 2.7648926856597e-01
6.4296615412062e-01 6.2919840713780e-01 2.6513148895949e-01
3.0147657685304e-01 6.3189761453676e-01 2.5377349249877e-01
9.6006426005440e-01 6.3459724538156e-01 2.4241524473025e-01
6.1872760785007e-01 6.3729731457314e-01 2.3105671154403e-01
2.7746519399508e-01 6.3999783593185e-01 2.1969785959344e-01
9.3627575631734e-01 6.4269882191107e-01 2.0833865670552e-01
5.9515819214023e-01 6.4540028333832e-01 1.9697907227712e-01
2.5411155464896e-01 6.4810222918843e-01 1.8561907765054e-01
9.1313505005451e-01 6.5080466639243e-01 1.7425864646291e-01
5.7222803532803e-01 6.5350759968520e-01 1.6289775496333e-01
2.3139001622721e-01 6.5621103149357e-01 1.5153638229208e-01
8.9062064531263e-01 6.5891496186632e-01 1.4017451071679e-01
5.4991971965600e-01 6.6161938844605e-01 1.2881212582089e-01
2.0928717796983e-01 6.6432430648258e-01 1.1744921664073e-01
8.6872309693698e-01 6.6702970888628e-01 1.0608577574866e-01
5.2822768658427e-01 6.6973558631918e-01 9.4721799280658e-02
1.8780128462297e-01 6.7244192732063e-01 8.3357286908314e-02
8.4744434976433e-01 6.7514871846389e-01 7.1992241756457e-02
5.0715745410717e-01 6.7785594453883e-01 6.0626670268982e-02
1.6694127477994e-01 6.8056358875592e-01 4.9260582026879e-02
8.2679658509854e-01 6.8327163296588e-01 3.7893989523634e-02
4.8672424556874e-01 6.8598005788928e-01 2.6526907904211e-02
1.4672519511580e-01 6.8868884335041e-01 1.5159354674652e-02
8.0680044296151e-01 6.9139796850977e-01 3.7913493898805e-03
4.6555931839163e-01 6.9411129458817e-01 9.9241834115407e-01
1.2577879293814e-01 6.9682108138571e-01 9.8104944470909e-01
7.8607574043552e-01 6.9953114188686e-01 9.6968016539511e-01
4.4645139863376e-01 7.0224145472083e-01 9.5831052767838e-01
1.0690706007805e-01 7.0495199883366e-01 9.4694055645281e-01
7.6744407693750e-01 7.0766275362608e-01 9.3557027675086e-01
4.2806386933046e-01 7.1037369906883e-01 9.2419971347809e-01
8.8767937462567e-02 7.1308481579572e-01 9.1282889117474e-01
7.4955787788168e-01 7.1579608517565e-01 9.0145783380618e-01
4.1043540415544e-01 7.1850748936530e-01 8.9008656458394e-01
7.1402372295928e-02 7.2121901134490e-01 8.7871510581755e-01
7.3246081129493e-01 7.2393063493983e-01 8.6734347879732e-01
3.9361295919583e-01 7.2664234483112e-01 8.5597170370762e-01
5.4861305216579e-02 7.2935412655809e-01 8.4459979957009e-01
7.1620863855730e-01 7.3206596651658e-01 8.3322778421622e-01
3.7765810467828e-01 7.3477785195641e-01 8.2185567428905e-01
3.9213270026516e-02 7.3748977098147e-01 8.1048348527443e-01
7.0087819642666e-01 7.4020171255670e-01 7.9911123156273e-01
3.6265752664565e-01 7.4291366652573e-01 7.8773892654325e-01
2.4556582999648e-02 7.4562562364419e-01 7.7636658273468e-01
6.8658148131322e-01 7.4833757563424e-01 7.6499421195683e-01
3.4873926308167e-01 7.5104951526714e-01 7.5362182555043e-01
1.1038049353335e-02 7.5376143648286e-01 7.4224943465452e-01
6.7348722066746e-01 7.5647333455864e-01 7.3087705055338e-01
3.3609762839152e-01 7.5918520634244e-01 7.1950468510791e-01
9.9888184404066e-01 7.6189705057359e-01 7.0813235128959e-01
6.6185445467639e-01 7.6460886832133e-01 6.9676006383791e-01
3.2503241431485e-01 7.6732066358435e-01 6.8538784006412e-01
9.8843546345314e-01 7.7003244411140e-01 6.7401570082381e-01
6.5208663132637e-01 7.7274422252698e-01 6.6264367167559e-01
3.1601283820969e-01 7.7545601787853e-01 6.5127178422900e-01
9.8024561762019e-01 7.7816785776475e-01 6.3990007765432e-01
6.4482197982142e-01 7.8087978125918e-01 6.2852860026868e-01
3.0978543682136e-01 7.8359184290623e-01 6.1715741100831e-01
9.7518720150492e-01 7.8630411812413e-01 6.0578658041863e-01
6.4108755267355e-01 7.8901671036509e-01 5.9441619050711e-01
3.0755731042694e-01 7.9172976026735e-01 5.8304633237078e-01
9.7467926885976e-01 7.9444345659784e-01 5.7167709992465e-01
6.4254924589124e-01 7.9715804764824e-01 5.6030857742343e-01
3.1127607552421e-01 7.9987384923519e-01 5.4894081820423e-01
9.8097932666271e-01 8.0259124056390e-01 5.3757381328609e-01
6.5178279960694e-01 8.0531063104488e-01 5.2620745346373e-01
3.2380122205007e-01 8.0803237075827e-01 5.1484150088584e-01
5.0341681822906e-01 5.7854649555161e-01 5.0362807587614e-01
1.5954430436791e-01 5.8123979572555e-01 4.9226999069301e-01
8.1581646379794e-01 5.8393322939596e-01 4.8091195426369e-01
4.7223406125848e-01 5.8662683865628e-01 4.6955397489232e-01
1.2879655698447e-01 5.8932065413232e-01 4.5819606305110e-01
7.8550230658071e-01 5.9201469717382e-01 4.4683822945092e-01
4.4234876752144e-01 5.9470898199008e-01 4.3548048352455e-01
9.9332700154607e-02 5.9740351761733e-01 4.2412283230347e-01
7.5645035437111e-01 6.0009830964888e-01 4.1276527964470e-01
4.1369763624887e-01 6.0279336169439e-01 4.0140782575215e-01
7.1070251696113e-02 6.0548867656143e-01 3.9005046693474e-01
7.2856382626225e-01 6.0818425717090e-01 3.7869319554721e-01
3.8617400184625e-01 6.1088010722907e-01 3.6733600006704e-01
4.3896512070547e-02 6.1357623168531e-01 3.5597886526958e-01
7.0172723869890e-01 6.1627263700606e-01 3.4462177247211e-01
3.5966225175961e-01 6.1896933129564e-01 3.3326469982522e-01
1.7697836085102e-02 6.2166632429192e-01 3.2190762263640e-01
6.7583050687103e-01 6.2436362726261e-01 3.1055051371562e-01
3.3405701665280e-01 6.2706125282445e-01 2.9919334373630e-01
9.9237435583720e-01 6.2975921470487e-01 2.8783608160741e-01
6.5077974864192e-01 6.3245752746304e-01 2.7647869485397e-01
3.0927064600590e-01 6.3515620618441e-01 2.6512115000401e-01
9.6784471675140e-01 6.3785526616104e-01 2.5376341297984e-01
6.2649983801196e-01 6.4055472256815e-01 2.4240544949170e-01
2.8523408569426e-01 6.4325459014553e-01 2.3104722543117e-01
9.4404572551935e-01 6.4595488289126e-01 2.1968870726119e-01
6.0293320499069e-01 6.4865561377400e-01 2.0832986239909e-01
2.6189514646570e-01 6.5135679446902e-01 1.9697065958834e-01
9.2093034136311e-01 6.5405843512219e-01 1.8561106925443e-01
5.8003774542221e-01 6.5676054414543e-01 1.7425106384008e-01
2.3921647484074e-01 6.5946312804630e-01 1.6289061811449e-01
8.9846580305652e-01 6.6216619129361e-01 1.5152970945184e-01
5.5778515790210e-01 6.6486973622034e-01 1.4016831807413e-01
2.1717411885050e-01 6.6757376296437e-01 1.2880642725403e-01
8.7663241408236e-01 6.7027826944699e-01 1.1744402347393e-01
5.3615991713623e-01 6.7298325138814e-01 1.0608109653818e-01
1.9575664295330e-01 6.7568870235695e-01 9.4717639636493e-02
8.5542274319015e-01 6.7839461385541e-01 8.3353649357439e-02
5.1515850074522e-01 6.8110097543205e-01 7.1989125652203e-02
1.7496432352193e-01 6.8380777482227e-01 6.0624071749928e-02
8.3484073752993e-01 6.8651499811125e-01 4.9258494027210e-02
4.9478837950081e-01 6.8922262991484e-01 3.7892401835401e-02
1.5480798926329e-01 6.9193065357385e-01 2.6525807290474e-02
8.1490040218141e-01 6.9463905135653e-01 1.5158725031011e-02
4.7506654200554e-01 6.9734780466452e-01 3.7911719506123e-03
1.3393063489835e-01 7.0006073599135e-01 9.9241864237395e-01
7.9423944968561e-01 7.0277018989221e-01 9.8105015196108e-01
4.5462502306347e-01 7.0547993864750e-01 9.6968125353224e-01
1.1508858296292e-01 7.0818996215950e-01 9.5831197005347e-01
7.7563142999350e-01 7.1090024039356e-01 9.4694232526069e-01
4.3625494110238e-01 7.1361075352151e-01 9.3557234337817e-01
9.6960576373044e-02 7.1632148204663e-01 9.2420204885473e-01
7.5774988930316e-01 7.1903240691005e-01 9.1283146612148e-01
4.1862454089032e-01 7.2174350957892e-01 9.0146061937423e-01
7.9586317852424e-02 7.2445477211766e-01 8.9008953238262e-01
7.4063715532088e-01 7.2716617724380e-01 8.7871822832781e-01
4.0177916437214e-01 7.2987770837074e-01 8.6734672966942e-01
6.3014664808901e-02 7.3258934963997e-01 8.5597505804242e-01
7.2434622367031e-01 7.3530108594571e-01 8.4460323418432e-01
3.8577670004337e-01 7.3801290295505e-01 8.3323127789279e-01
4.7309296868188e-02 7.4072478712706e-01 8.2185920801419e-01
7.0894762058360e-01 7.4343672573458e-01 8.1048704246380e-01
3.7069574964979e-01 7.4614870689257e-01 7.9911479827908e-01
3.2558313218948e-02 7.4886071959749e-01 7.8774249170800e-01
6.9454058151077e-01 7.5157275378280e-01 7.7637013833569e-01
3.5664856979693e-01 7.5428480039665e-01 7.6499775325356e-01
1.8889158319291e-02 7.5699685150914e-01 7.5362535127676e-01
6.8127023097542e-01 7.5970890045869e-01 7.4225294721698e-01
3.4380083621836e-01 7.6242094204987e-01 7.3088055621954e-01
6.4913743531734e-03 7.6513297281904e-01 7.1950819417499e-01
6.6935381628813e-01 7.6784499138989e-01 7.0813587821648e-01
3.3240195982458e-01 7.7055699894848e-01 6.9676362731449e-01
9.9565173074367e-01 7.7326899987813e-01 6.8539146297862e-01
6.5912153723349e-01 7.7598100260828e-01 6.7401941007124e-01
3.2283268749466e-01 7.7869302075011e-01 6.6264749772714e-01
9.8680988143189e-01 7.8140507461531e-01 6.5127576035279e-01
6.5108178770595e-01 7.8411719324228e-01 6.3990423864355e-01
3.1568171615170e-01 7.8682941708541e-01 6.2853298049700e-01
9.8064839091186e-01 7.8954180154834e-01 6.1716204160492e-01
6.4602681838840e-01 7.9225442154529e-01 6.0579148535911e-01
3.1186922059095e-01 7.9496737721585e-01 5.9442138149326e-01
9.7823595896726e-01 7.9768080071999e-01 5.8305180260105e-01
6.4519629073223e-01 8.0039486355458e-01 5.7168281735635e-01
3.1282865647340e-01 8.0310978280668e-01 5.6031447905431e-01
9.8121996834467e-01 8.0582582280598e-01 5.4894680835279e-01
6.5046304102885e-01 8.0854328532248e-01 5.3757977058310e-01
3.2065093792166e-01 8.1126247668366e-01 5.2621325201789e-01
9.9186682127963e-01 8.1398363533809e-01 5.1484704753587e-01
1.7012649142370e-01 5.8450437777898e-01 5.0361718994985e-01
8.2610545189579e-01 5.8719760049798e-01 4.9225900606469e-01
4.8224452572822e-01 5.8989097553938e-01 4.8090087205641e-01
1.3854444336668e-01 5.9258454719288e-01 4.6954279831955e-01
7.9500441569223e-01 5.9527834637623e-01 4.5818479780506e-01
4.5162237356511e-01 5.9797239323984e-01 4.4682688378403e-01
1.0839521451281e-01 6.0066669969302e-01 4.3546906811996e-01
7.6531904153215e-01 6.0336127171537e-01 4.2411136001986e-01
4.2238938330814e-01 6.0605611137326e-01 4.1275376520407e-01
7.9601389228413e-02 6.0875121850602e-01 4.0139628542108e-01
7.3694999600275e-01 6.1144659207941e-01 3.9003891823227e-01
3.9443006532961e-01 6.1414223122604e-01 3.7868165699823e-01
5.2036493899730e-02 6.1683813600460e-01 3.6732449100860e-01
7.0976429819325e-01 6.1953430791588e-01 3.5596740570953e-01
3.6760867715592e-01 6.2223075021433e-01 3.4461038299388e-01
2.5565056073217e-02 6.2492746805214e-01 3.3325340152957e-01
6.8362911492618e-01 6.2762446848926e-01 3.2189643710928e-01
3.4179680430767e-01 6.3032176039855e-01 3.1053946301114e-01
6.4351678205295e-05 6.3301935429126e-01 2.9918245036419e-01
6.5842826039626e-01 6.3571726208373e-01 2.8782536851555e-01
3.1688530360420e-01 6.3841549682315e-01 2.7646818539806e-01
9.7543251470638e-01 6.4111407238683e-01 2.6511086789760e-01
6.3406717585397e-01 6.4381300316708e-01 2.5375338222019e-01
2.9278680555555e-01 6.4651230375184e-01 2.4239569425811e-01
9.5158914626741e-01 6.4921198860907e-01 2.3103776995413e-01
6.1047215258225e-01 6.5191207178192e-01 2.1967957566242e-01
2.6943398042934e-01 6.5461256660035e-01 2.0832107850356e-01
9.2847297752310e-01 6.5731348541356e-01 1.9696224671116e-01
5.8758767514814e-01 6.6001483934744e-01 1.8560304996615e-01
2.4677678124807e-01 6.6271663808972e-01 1.7424345971532e-01
9.0603917469005e-01 6.6541888970550e-01 1.6288344946954e-01
5.6537390050806e-01 6.6812160048474e-01 1.5152299507746e-01
2.2478016588229e-01 6.7082477482310e-01 1.4016207497039e-01
8.8425733658992e-01 6.7352841513674e-01 1.2880067037424e-01
5.4380493366162e-01 6.7623252181111e-01 1.1743876548462e-01
2.0342262999576e-01 6.7893709318345e-01 1.0607634760219e-01
8.6311024671763e-01 6.8164212555784e-01 9.4713407225408e-02
5.2286774911901e-01 6.8434761325129e-01 8.3349938099303e-02
1.8269524207294e-01 6.8705354866877e-01 7.1985937219385e-02
8.4259296488471e-01 6.8975992240435e-01 6.0621404791017e-02
5.0256128561004e-01 6.9246672336546e-01 4.9256344145608e-02
1.6260069494134e-01 6.9517393891662e-01 3.7890761616016e-02
8.2271179982952e-01 6.9788155503868e-01 2.6524666374537e-02
4.8289531706910e-01 7.0058955649949e-01 1.5158070237748e-02
1.4315206712594e-01 7.0329792703170e-01 3.7909874432218e-03
8.0212161377243e-01 7.0601044924145e-01 9.9241895901031e-01
4.6251957746717e-01 7.0871955515188e-01 9.8105089842995e-01
1.2299359331125e-01 7.1142897523280e-01 9.6968240740551e-01
7.8354485149883e-01 7.1413869085416e-01 9.5831350726785e-01
4.4417463072142e-01 7.1684868321071e-01 9.4694422043456e-01
1.0488430086964e-01 7.1955893346605e-01 9.3557457014112e-01
7.6567532850771e-01 7.2226942288263e-01 9.2420458018385e-01
4.2654928547130e-01 7.2498013293701e-01 9.1283427467905e-01
8.7507860930900e-02 7.2769104542021e-01 9.0146367784187e-01
7.4855287726103e-01 7.3040214252381e-01 8.9009281378798e-01
4.0968631006143e-01 7.3311340691294e-01 8.7872170636030e-01
7.0910312695474e-02 7.3582482178781e-01 8.6735037898263e-01
7.3222724574363e-01 7.3853637093591e-01 8.5597885454158e-01
3.9363971182018e-01 7.4124803877741e-01 8.4460715529783e-01
5.5150596271960e-02 7.4395981040657e-01 8.3323530282765e-01
7.1676311437075e-01 7.4667167163256e-01 8.2186331799557e-01
3.7848086573033e-01 7.4938360902294e-01 8.1049122095948e-01
4.0307896827908e-02 7.5209560995407e-01 7.9911903120942e-01
7.0224877269211e-01 7.5480766267282e-01 7.8774676764214e-01
3.6430865904195e-01 7.5751975637482e-01 7.7637444867404e-01
2.6493416427633e-02 7.6023188130549e-01 7.6500209239578e-01
6.8880970824335e-01 7.6294402889159e-01 7.5362971677284e-01
3.5126512486018e-01 7.6565619191277e-01 7.4225733989673e-01
1.3868326572083e-02 7.6836836472547e-01 7.3088498029270e-01
6.7662920856389e-01 7.7108054355515e-01 7.1951265728947e-01
3.3955909169774e-01 7.7379272687751e-01 7.0814039145630e-01
2.6709435613984e-03 7.7650491591631e-01 6.9676820511071e-01
6.6597963489621e-01 7.7921711529326e-01 6.8539612289613e-01
3.2950223714921e-01 7.8192933387667e-01 6.7402417242094e-01
9.9325836732677e-01 7.8464158588861e-01 6.6265238493699e-01
6.5727058628676e-01 7.8735389234527e-01 6.5128079601339e-01
3.2156485558685e-01 7.9006628292134e-01 6.3990944612649e-01
9.8617105511032e-01 7.9277879834058e-01 6.2853838103243e-01
6.5112355736284e-01 7.9549149339468e-01 6.1716765171005e-01
3.1646184195179e-01 7.9820444066006e-01 6.0579731355006e-01
9.8223111104037e-01 8.0091773488571e-01 5.9442742432281e-01
6.4848282685807e-01 8.0363149780028e-01 5.8305804029363e-01
3.1527502612682e-01 8.0634588263057e-01 5.7168920972437e-01
9.8267216195264e-01 8.0906107677927e-01 5.6032096303219e-01
6.5074407252726e-01 8.1177729967916e-01 5.4895329934269e-01
3.1956348609562e-01 8.1449479068143e-01 5.3758617055775e-01
9.8920129985851e-01 8.1721377915058e-01 5.2621946703803e-01
6.5971887333848e-01 8.1993442690019e-01 5.1485301410982e-01
8.3684161109818e-01 5.9046216206854e-01 5.0360651152101e-01
4.9265627793278e-01 5.9315529429438e-01 4.9224822116610e-01
1.4864782029685e-01 5.9584860048014e-01 4.8088998113760e-01
8.0481692270407e-01 5.9854212749858e-01 4.6953180421448e-01
4.6116249761855e-01 6.0123590662251e-01 4.5817370616253e-01
1.1768197448153e-01 6.0392995664167e-01 4.4681570312378e-01
7.7437159608842e-01 6.0662428686787e-01 4.3545780963724e-01
4.3122670356285e-01 6.0931889986214e-01 4.2410003724606e-01
8.8241996949138e-02 6.1201379379039e-01 4.1274239360823e-01
7.4541176376092e-01 6.1470896437206e-01 4.0138488201321e-01
4.0273007218262e-01 6.1740440642637e-01 3.9002750120746e-01
6.0190928861658e-02 6.2010011504707e-01 3.7867024544229e-01
7.1778840341003e-01 6.2279608645008e-01 3.6731310467249e-01
3.7551672302763e-01 6.2549231854358e-01 3.5595606485017e-01
3.3370341277472e-02 6.2818881126958e-01 3.4459910827309e-01
6.9134398518564e-01 6.3088556676226e-01 3.3324221395947e-01
3.4943268467505e-01 6.3358258936255e-01 3.2188535803132e-01
7.6317880005614e-03 6.3627988552304e-01 3.1052851409574e-01
6.6593696642483e-01 6.3897746363099e-01 2.9917165361921e-01
3.2434421091956e-01 6.4167533377268e-01 2.8781474629283e-01
9.8284982323285e-01 6.4437350745747e-01 2.7645776038925e-01
6.4145040324891e-01 6.4707199731667e-01 2.6510066311214e-01
3.0014283419303e-01 6.4977081678903e-01 2.5374342094002e-01
9.5892426690275e-01 6.5246997980265e-01 2.4238599996554e-01
6.1779210409674e-01 6.5516950046075e-01 2.3102836623089e-01
2.7674398532182e-01 6.5786939273768e-01 2.1967048605924e-01
9.3577777304384e-01 6.6056967019006e-01 2.0831232638131e-01
5.9489154016585e-01 6.6327034568711e-01 1.9695385505553e-01
2.5408355910474e-01 6.6597143116343e-01 1.8559504117922e-01
9.1335229243289e-01 6.6867293739673e-01 1.7423585538815e-01
5.7269638499296e-01 6.7137487381265e-01 1.6287627014083e-01
2.3211465731902e-01 6.7407724831819e-01 1.5151625998402e-01
8.9160610014548e-01 6.7678006716478e-01 1.4015580179561e-01
5.5116986975457e-01 6.7948333484177e-01 1.2879487500093e-01
2.1080528390219e-01 6.8218705400047e-01 1.1743346175907e-01
8.7051181806848e-01 6.8489122540862e-01 1.0607154711574e-01
5.3028910180263e-01 6.8759584793473e-01 9.4709119119990e-02
1.9013691496758e-01 6.9030091856104e-01 8.3346168902570e-02
8.5005518373796e-01 6.9300643242378e-01 7.1982690714581e-02
5.1004397626036e-01 6.9571238287859e-01 6.0618681925774e-02
1.7010349794622e-01 6.9841876158883e-01 4.9254142982909e-02
8.3023408643134e-01 7.0112555863396e-01 3.7889077329410e-02
4.9043620629879e-01 7.0383276263475e-01 2.6523491288516e-02
1.5071044372185e-01 7.0654036089221e-01 1.5157393912988e-02
8.1105750123735e-01 7.0924833953647e-01 3.7907968052072e-03
4.7013272438136e-01 7.1196044010126e-01 9.9241928917286e-01
1.3061964291157e-01 7.1466918476843e-01 9.8105167926527e-01
7.9118192953101e-01 7.1737826106697e-01 9.6968361909270e-01
4.5182071242937e-01 7.2008765200798e-01 9.5831512823367e-01
1.1253722209652e-01 7.2279734020673e-01 9.4694622764312e-01
7.7333279326903e-01 7.2550730802315e-01 9.3557693940033e-01
4.3420886928567e-01 7.2821753769240e-01 9.2420728646299e-01
9.5167009202524e-02 7.3092801144478e-01 9.1283729243217e-01
7.5620889801442e-01 7.3363871161418e-01 9.0146698133191e-01
4.1733636032806e-01 7.3634962073544e-01 8.9009637740710e-01
7.8551377836697e-02 7.3906072163119e-01 8.7872550494260e-01
7.3985611095829e-01 7.4177199748941e-01 8.6735438810590e-01
4.0125292502265e-01 7.4448343193332e-01 8.5598305081562e-01
6.2744421428581e-02 7.4719500908584e-01 8.4461151663721e-01
7.2433347424444e-01 7.4990671363113e-01 8.3323980870768e-01
3.8602327279446e-01 7.5261853087600e-01 8.2186794969041e-01
4.7817370863346e-02 7.5533044681489e-01 8.1049596176167e-01
7.0971974326398e-01 7.5804244820187e-01 7.9912386663017e-01
3.7173485065159e-01 7.6075452263443e-01 7.8775168559150e-01
3.3867713634618e-02 7.6346665865413e-01 7.7637943961949e-01
6.9612399743133e-01 7.6617884587019e-01 7.6500714949682e-01
3.5851010855485e-01 7.6889107511383e-01 7.5363483598756e-01
2.1033305279988e-02 7.7160333863242e-01 7.4226252005453e-01
6.8370182395274e-01 7.7431563033534e-01 7.3089022312386e-01
3.4652502354439e-01 7.7702794610632e-01 7.1951796739887e-01
9.5135512139636e-03 7.7974028420120e-01 7.0814577622321e-01
6.7267953200066e-01 7.8245264575516e-01 6.9677367449040e-01
3.3603678606990e-01 7.8516503542987e-01 6.8540168909115e-01
9.9960107709136e-01 7.8787746223857e-01 6.7402984938106e-01
6.6339039516831e-01 7.9058994059547e-01 6.6265818763713e-01
3.2742527698409e-01 7.9330249164403e-01 6.5128673945036e-01
9.9172916400181e-01 7.9601514492448e-01 6.3991554396995e-01
6.5632879585930e-01 7.9872794043916e-01 6.2854464387025e-01
3.2125462925926e-01 8.0144093115645e-01 6.1717408485046e-01
9.8654126065347e-01 8.0415418594339e-01 6.0580391439976e-01
6.5222781082328e-01 8.0686779280632e-01 5.9443417947204e-01
3.1835819679074e-01 8.0958186210115e-01 5.8306492263617e-01
9.8498116601993e-01 8.1229652898342e-01 5.7169617625032e-01
6.5214989461146e-01 8.1501195371215e-01 5.6032795435693e-01
3.1992085481437e-01 8.1772831741368e-01 5.4896024249747e-01
9.8835155178448e-01 8.2044580954955e-01 5.3759298679046e-01
6.5749666139643e-01 8.2316460188238e-01 5.2622608573217e-01
3.2740217108560e-01 8.2588480306375e-01 5.1485939137524e-01
5.0356266685216e-01 5.9641984147425e-01 5.0359605591205e-01
1.5919588992491e-01 5.9911286815201e-01 4.9223765125964e-01
8.1502430571838e-01 6.0180609368701e-01 4.8087929663266e-01
4.7104853589345e-01 6.0449956797594e-01 4.6952100754561e-01
1.2726713134395e-01 6.0719332268935e-01 4.5816280297745e-01
7.8367692340174e-01 6.0988737502830e-01 4.4680470230297e-01
4.4027338206272e-01 6.1258173132062e-01 4.3544672301355e-01
9.7050957691058e-02 6.1527639025276e-01 4.2408887916047e-01
7.5400339042612e-01 6.1797134562912e-01 4.1273118040729e-01
4.1112397849928e-01 6.2066658862474e-01 4.0137363156136e-01
6.8405802233133e-02 6.2336210954768e-01 3.9001623245937e-01
7.2584190448824e-01 6.2605789915741e-01 3.7865897809755e-01
3.8342543088078e-01 6.2875394960052e-01 3.6730185891853e-01
4.1149734496994e-02 6.3145025502842e-01 3.5594486118837e-01
6.9900845038630e-01 6.3414681195930e-01 3.4458796741645e-01
3.5699554510473e-01 6.3684361943930e-01 3.3323115678702e-01
1.5105346227464e-02 6.3954067905058e-01 3.2187440558358e-01
6.7333255621801e-01 6.4223799480506e-01 3.1051768759603e-01
3.3167225444224e-01 6.4493557295560e-01 2.9916097450709e-01
9.9011989051728e-01 6.4763342174958e-01 2.8780423625788e-01
6.4867127162651e-01 6.5033155114455e-01 2.7644744139524e-01
3.0732254593112e-01 6.5302997250116e-01 2.6509055740403e-01
9.6607018377138e-01 6.5572869826515e-01 2.5373355102789e-01
6.2491095797474e-01 6.5842774164753e-01 2.4237638858149e-01
2.8384192426943e-01 6.6112711630995e-01 2.3101903625665e-01
9.4286040253236e-01 6.6382683606072e-01 2.1966146042368e-01
6.0196395937513e-01 6.6652691456573e-01 2.0830362792850e-01
2.6115039238337e-01 6.6922736507754e-01 1.9694550638494e-01
9.2041771616985e-01 6.7192820018531e-01 1.8558706446117e-01
5.7976415027495e-01 6.7462943158758e-01 1.7422827215808e-01
2.3918810884782e-01 6.7733106988944e-01 1.6286910107704e-01
8.9868819196375e-01 6.8003312442539e-01 1.5150952467418e-01
5.5826317837740e-01 6.8273560310860e-01 1.4014951849754e-01
2.1791201947505e-01 6.8543851230739e-01 1.2878906040398e-01
8.7763383417058e-01 6.8814185674890e-01 1.1742813075211e-01
5.3742790448799e-01 6.9084563945009e-01 1.0606671256811e-01
1.9729367158658e-01 6.9354986167568e-01 9.4704791681410e-02
8.5723073201056e-01 6.9625452292232e-01 8.3342356827722e-02
5.1723883398273e-01 6.9895962092786e-01 7.1979399717513e-02
1.7731787360734e-01 7.0166515170454e-01 6.0615915068580e-02
8.3746789090048e-01 7.0437110959415e-01 4.9251900602303e-02
4.9768906562272e-01 7.0707748734314e-01 3.7887357003836e-02
1.5798171294698e-01 7.0978427619531e-01 2.6522287847401e-02
8.1834627905210e-01 7.1249146599949e-01 1.5156699488637e-02
4.7878333678665e-01 7.1519904532919e-01 3.7906009267364e-03
1.3796446074791e-01 7.1791071343861e-01 9.9241963106662e-01
7.9854015071355e-01 7.2061908533907e-01 9.8105248985431e-01
4.5919055958947e-01 7.2332780446653e-01 9.6968488104802e-01
1.1991673271080e-01 7.2603685562663e-01 9.5831682237207e-01
7.8071982956739e-01 7.2874622302784e-01 9.4694833318867e-01
4.4160112518301e-01 7.3145589041452e-01 9.3557943426596e-01
1.0256201355267e-01 7.3416584119417e-01 9.2421014754697e-01
7.6360401347948e-01 7.3687605855784e-01 9.1284049592409e-01
4.2472877715376e-01 7.3958652559282e-01 9.0147050302316e-01
8.5938101818030e-02 7.4229722538754e-01 8.9010019300110e-01
7.4723394486621e-01 7.4500814112882e-01 8.7872959036032e-01
4.0861844273370e-01 7.4771925619243e-01 8.6735871978292e-01
7.0093933951235e-02 7.5043055422808e-01 8.5598760598729e-01
7.3166298676075e-01 7.5314201924076e-01 8.4461627360916e-01
3.9332843172786e-01 7.5585363567043e-01 8.3324474710917e-01
5.5093399835654e-02 7.5856538847289e-01 8.2187305070858e-01
7.1696136660935e-01 7.6127726320474e-01 8.1050120835478e-01
3.7893620290333e-01 7.6398924611621e-01 7.9912924371799e-01
4.1022233082501e-02 7.6670132425598e-01 7.8775718022085e-01
7.0322430145070e-01 7.6941348559303e-01 7.7638504110232e-01
3.6554784792014e-01 7.7212571916142e-01 7.6501284951732e-01
2.7998994078916e-02 7.7483801523517e-01 7.5364062867365e-01
6.9058464099616e-01 7.7755036554195e-01 7.4226840200689e-01
3.5331258030155e-01 7.8026276352640e-01 7.3089619339371e-01
1.6191620278626e-02 7.8297520467635e-01 7.1952402740210e-01
6.7923172889986e-01 7.8568768692850e-01 7.0815192957500e-01
3.4244419587154e-01 7.8840021117389e-01 6.9677992673960e-01
5.8418157902805e-03 7.9111278188790e-01 6.8540804732887e-01
6.6943909433978e-01 7.9382540791444e-01 6.7403632169244e-01
3.3325247891482e-01 7.9653810343809e-01 6.6266478236093e-01
9.9730061388393e-01 7.9925088918087e-01 6.5129346420876e-01
6.6160461847321e-01 8.0196379385886e-01 6.3992240443429e-01
3.2618838131371e-01 8.0467685592295e-01 6.2855164224151e-01
9.9107885904704e-01 8.0739012558067e-01 6.1718121806359e-01
6.5630635556035e-01 8.1010366703797e-01 6.0581117211921e-01
3.2190474139131e-01 8.1281756078968e-01 5.9444154204528e-01
9.8791154703901e-01 8.1553190559449e-01 5.8307235932722e-01
6.5436782669361e-01 8.1824681945180e-01 5.7170364429253e-01
3.2131763891932e-01 8.2096243840689e-01 5.6033539962417e-01
9.8880693132515e-01 8.2367891132254e-01 5.4896760280971e-01
6.5688156123786e-01 8.2639638791872e-01 5.3760019883386e-01
3.2558416800854e-01 8.2911499664426e-01 5.2623309588731e-01
9.9494969792316e-01 8.3183480890394e-01 5.1486616884758e-01
1.7029023546847e-01 6.0237740913369e-01 5.0358583851370e-01
8.2572323659310e-01 6.0507031285404e-01 4.9222731168838e-01
4.8137158265675e-01 6.0776344412609e-01 4.8086883375067e-01
1.3723580849092e-01 6.1045685637473e-01 4.6951042337452e-01
7.9331402202355e-01 6.1315058168026e-01 4.5815210322847e-01
4.4960233423615e-01 6.1584463534011e-01 4.4679389634359e-01
1.0609529492027e-01 6.1853902019941e-01 4.3543582347630e-01
7.6278630449713e-01 6.2123373049883e-01 4.2407790136725e-01
4.1966798270713e-01 6.2392875511567e-01 4.1272014174012e-01
7.6732484211199e-02 6.2662408016955e-01 4.0136255086802e-01
7.3397175827705e-01 6.2931969102604e-01 3.9000512954497e-01
3.9137775463565e-01 6.3201557376721e-01 3.7864787332535e-01
4.8942580562283e-02 6.3471171621238e-01 3.6729077292304e-01
7.0665861570090e-01 6.3740810857411e-01 3.5593381469140e-01
3.6451859156209e-01 6.4010474382736e-01 3.4457698112992e-01
2.2515642369961e-02 6.4280161785986e-01 3.3322025138339e-01
6.8064333330746e-01 6.4549872945986e-01 3.2186360171461e-01
3.3889567141855e-01 6.4819608018652e-01 3.1050700594190e-01
9.9726710360123e-01 6.5089367415838e-01 2.9915043584007e-01
6.5575250534336e-01 6.5359151778720e-01 2.8779386150729e-01
3.1434716315238e-01 6.5628961947777e-01 2.7643725170300e-01
9.7304675302286e-01 6.5898798930912e-01 2.6508057416242e-01
6.3184731677296e-01 6.6168663870867e-01 2.5372379589340e-01
2.9074523765339e-01 6.6438558012766e-01 2.4236688346070e-01
9.4973721627975e-01 6.6708482672415e-01 2.3100980326159e-01
6.0882024764960e-01 6.6978439205826e-01 2.1965252179583e-01
2.6799159976864e-01 6.7248428980282e-01 2.0829500593185e-01
9.2724879421721e-01 6.7518453347211e-01 1.9693722316978e-01
5.8658958883091e-01 6.7788513617058e-01 1.8557914190104e-01
2.4601196254292e-01 6.8058611036279e-01 1.7422073166356e-01
9.0551410233441e-01 6.8328746766573e-01 1.6286196339054e-01
5.6509439216217e-01 6.8598921866435e-01 1.5150280965055e-01
2.2475140367393e-01 6.8869137275070e-01 1.4014324487595e-01
8.8448388848271e-01 6.9139393798718e-01 1.2878324557664e-01
5.4429077174795e-01 6.9409692099408e-01 1.1742279053585e-01
2.0417114680296e-01 6.9680032686120e-01 1.0606186098469e-01
8.6412427057366e-01 6.9950415908365e-01 9.4700440752513e-02
5.2414955955145e-01 7.0220841952116e-01 8.3338516390177e-02
1.8424658611113e-01 7.0491310838026e-01 7.1976077263912e-02
8.4441507500239e-01 7.0761822421847e-01 6.0613115617971e-02
5.0465489988786e-01 7.1032376396929e-01 4.9249626604820e-02
1.6496607985041e-01 7.1302972298640e-01 3.7885608282364e-02
8.2534877584525e-01 7.1573609510545e-01 2.6521061578416e-02
4.8580328712614e-01 7.1844287272141e-01 1.5155990223383e-02
1.4633004772794e-01 7.2115004687928e-01 3.7904006528787e-03
8.0561731149328e-01 7.2386127332671e-01 9.9241998298958e-01
4.6628161173134e-01 7.2656926255754e-01 9.8105332582182e-01
1.2702002730250e-01 7.2927761274233e-01 9.6968618611103e-01
7.8783350300793e-01 7.3198631061581e-01 9.5831857963802e-01
4.4872310778710e-01 7.3469534213469e-01 9.4695052405452e-01
1.0969003570186e-01 7.3740469259982e-01 9.3558203867405e-01
7.7073560864777e-01 7.4011434677642e-01 9.2421314425878e-01
4.3186128111692e-01 7.4282428901130e-01 9.1284386280686e-01
9.3068647343385e-02 7.4553450334596e-01 9.0147421734430e-01
7.5435945116738e-01 7.4824497362532e-01 8.9010423172545e-01
4.1573559895984e-01 7.5095568360192e-01 8.7873393044564e-01
7.7199175956418e-02 7.5366661703625e-01 8.6736333846930e-01
7.3875246636147e-01 7.5637775779389e-01 8.5599248107649e-01
4.0039797759978e-01 7.5908908994115e-01 8.4462138373032e-01
6.2138469118721e-02 7.6180059784073e-01 8.3325007196778e-01
7.2397698617743e-01 7.6451226624996e-01 8.2187857131565e-01
3.8591689910407e-01 7.6722408042416e-01 8.1050690723359e-01
4.7961948558707e-02 7.6993602622863e-01 7.9913510508563e-01
7.1011629740765e-01 7.7264809026301e-01 7.8776319014169e-01
3.7238458989731e-01 7.7536026000268e-01 7.7639118761000e-01
3.4772018908956e-02 7.7807252396266e-01 7.6501912270126e-01
6.9728440218084e-01 7.8078487189064e-01 7.5364702072493e-01
3.5992826849464e-01 7.8349729499692e-01 7.4227490721696e-01
2.2710954933064e-02 7.8620978623083e-01 7.3090280809750e-01
6.8564071641020e-01 7.8892234061531e-01 7.1953074985477e-01
3.4872684873336e-01 7.9163495565321e-01 7.0815875974905e-01
1.1979826436690e-02 7.9434763182225e-01 6.9678686602600e-01
6.7541145647245e-01 7.9706037317744e-01 6.8541509812292e-01
3.3903504845869e-01 7.9977318808288e-01 6.7404348684304e-01
2.8656014118179e-03 8.0248609009586e-01 6.6267206446075e-01
6.6692000550736e-01 8.0519909902464e-01 6.5130086470548e-01
3.3121725506073e-01 8.0791224217530e-01 6.3992992255061e-01
9.9577866508366e-01 8.1062555578697e-01 6.2855927370895e-01
6.6062807770715e-01 8.1333908662286e-01 6.1718895370675e-01
3.2579203545175e-01 8.1605289362626e-01 6.0581899637997e-01
9.9129988448435e-01 8.1876704945015e-01 5.9444943161784e-01
6.5718375122423e-01 8.2148164150634e-01 5.8308028219000e-01
3.2347830891529e-01 8.2419677193075e-01 5.7171155956724e-01
9.9022021755744e-01 8.2691255550480e-01 5.6034325883864e-01
6.5744708515855e-01 8.2962911411303e-01 5.4897535321608e-01
3.2519577239232e-01 8.3234656581964e-01 5.3760778928731e-01
9.9349987033195e-01 8.3506500630768e-01 5.2624048517852e-01
6.6238626060414e-01 8.3778448064493e-01 5.1487333501144e-01
8.3702499913498e-01 6.0833485830853e-01 5.0357587475168e-01
4.9223707558314e-01 6.1102761890376e-01 4.9221721784460e-01
1.4768682358615e-01 6.1372064019294e-01 4.8085860775219e-01
8.0337465762854e-01 6.1641397970765e-01 4.6950006682570e-01
4.5929813740242e-01 6.1910766990431e-01 4.5814162199781e-01
1.1545249836373e-01 6.2180172374712e-01 4.4678330045421e-01
7.7183118540425e-01 6.2449613995700e-01 4.3542512656883e-01
4.2842635185233e-01 6.2719090761520e-01 4.2406711995784e-01
8.5229300461022e-02 6.2988600998118e-01 4.1270929443673e-01
7.4223085533599e-01 6.3258142750753e-01 4.0135165765032e-01
3.9942166291847e-01 6.3527714011181e-01 3.8999421116579e-01
5.6792426158728e-02 6.3797312880516e-01 3.7863695085553e-01
7.1433407940156e-01 6.4066937679140e-01 3.6727986743826e-01
3.7203791293780e-01 6.4336587014706e-01 3.5592294708520e-01
2.9895656311709e-02 6.4606259818089e-01 3.4456617203035e-01
6.8789952884079e-01 6.4875955355583e-01 3.3320952114866e-01
3.4604226479087e-01 6.5145673224022e-01 3.2185297048378e-01
4.3171195063874e-03 6.5415413334043e-01 3.1049649371949e-01
6.6271786167834e-01 6.5685175885449e-01 2.9914006259626e-01
3.2123875591913e-01 6.5954961337623e-01 2.8778364727909e-01
9.7987453893762e-01 6.6224770377130e-01 2.7642721668455e-01
6.3862039187480e-01 6.6494603884074e-01 2.6507073877583e-01
2.9747191075905e-01 6.6764462898263e-01 2.5371418083373e-01
9.5642507655424e-01 6.7034348585985e-01 2.4235750971086e-01
6.1547622588428e-01 6.7304262207878e-01 2.3100069207489e-01
2.7462202320680e-01 6.7574205088278e-01 2.1964369464526e-01
9.3385943496094e-01 6.7844178586274e-01 2.0828648442654e-01
5.9318570601688e-01 6.8114184068620e-01 1.9692902894020e-01
2.5259833859754e-01 6.8384222884628e-01 1.8557129645552e-01
9.1209507371832e-01 6.8654296343087e-01 1.7421325621935e-01
5.7167387509218e-01 6.8924405691278e-01 1.6285487868356e-01
2.3133291537154e-01 6.9194552096080e-01 1.5149613572827e-01
8.9107056454117e-01 6.9464736627219e-01 1.4013700087861e-01
5.5088538023539e-01 6.9734960242638e-01 1.2877744951219e-01
2.1077609972757e-01 7.0005223776010e-01 1.1741745905416e-01
8.7074163332702e-01 7.0275527926375e-01 1.0605700915693e-01
5.3078105891860e-01 7.0545873249890e-01 9.4696081861193e-02
1.9089361739126e-01 7.0816260153658e-01 8.3334661735553e-02
8.5107870872254e-01 7.1086688891611e-01 7.1972735991856e-02
5.1133588851604e-01 7.1357159562362e-01 6.0610294574043e-02
1.7166486482545e-01 7.1627672108973e-01 4.9247330218693e-02
8.3206549514210e-01 7.1898226320539e-01 3.7883838486001e-02
4.9253778346957e-01 7.2168821835453e-01 2.6519817760608e-02
1.5308187745882e-01 7.2439458146242e-01 1.5155269222353e-02
8.1369806562714e-01 7.2710134605789e-01 3.7901967877045e-03
4.7309173390646e-01 7.2981212313746e-01 9.9242034332628e-01
1.3384451519647e-01 7.3251972131472e-01 9.8105418301858e-01
7.9467086297198e-01 7.3522769230400e-01 9.6968752749586e-01
4.5557160674603e-01 7.3793602488645e-01 9.5832039051617e-01
1.1654770836100e-01 7.4064470690545e-01 9.4695278791329e-01
7.7760026273453e-01 7.4335372537495e-01 9.3558473741261e-01
4.3873050002611e-01 7.4606306658940e-01 9.2421625843854e-01
9.9939789525197e-02 7.4877271623417e-01 9.1284737191742e-01
7.6122964557374e-01 7.5148265949547e-01 9.0147810008029e-01
4.2260173584624e-01 7.5419288116929e-01 8.9010846626906e-01
8.4057892318008e-02 7.5690336576907e-01 8.7873849475020e-01
7.4560012526047e-01 7.5961409763233e-01 8.6736821053909e-01
4.0723064061119e-01 7.6232506102696e-01 8.5599763923849e-01
6.8951861078369e-02 7.6503624025819e-01 8.4462680689378e-01
7.3076645135537e-01 7.6774761977768e-01 8.3325573986774e-01
3.9267734784274e-01 7.7045918429673e-01 8.2188446473689e-01
5.4687793302776e-02 7.7317091890593e-01 8.1051300821151e-01
7.1680137690718e-01 7.7588280920427e-01 7.9914139708069e-01
3.7902208018043e-01 7.7859484144110e-01 7.8776965818393e-01
4.1354329391182e-02 7.8130700267506e-01 7.7639781840973e-01
7.0380305499848e-01 7.8401928095489e-01 7.6502590472180e-01
3.6637375881664e-01 7.8673166552796e-01 7.5365394421202e-01
2.9072589616006e-02 7.8944414708330e-01 7.4228196417898e-01
6.9190642791757e-01 7.9215671803741e-01 7.3090999222886e-01
3.5488298075179e-01 7.9486937287233e-01 7.1953805639380e-01
1.8010887112043e-02 7.9758210853723e-01 7.0816618525958e-01
6.8129983470253e-01 8.0029492492633e-01 6.9679440809078e-01
3.4476068830143e-01 8.0300782544717e-01 6.8542275493548e-01
8.4056295440191e-03 8.0572081769395e-01 6.7405125668461e-01
6.7224830704492e-01 8.0843391423962e-01 6.6267994505049e-01
3.3630399432975e-01 8.1114713355614e-01 6.5130885241709e-01
5.8975075219460e-04 8.1386050106341e-01 6.3993801149851e-01
6.6512457703962e-01 8.1657405028854e-01 6.2856745472518e-01
3.2992955180816e-01 8.1928782408525e-01 6.1719721325908e-01
9.9502792764381e-01 8.2200187580812e-01 6.0582731552623e-01
6.6044515440465e-01 8.2471627024977e-01 5.9445778515486e-01
3.2620878259772e-01 8.2743108401645e-01 5.8308863823636e-01
9.9234818089373e-01 8.3014640482773e-01 5.7171987990993e-01
6.5889398032416e-01 8.3286232897241e-01 5.6035150044675e-01
3.2587713759647e-01 8.3557895585254e-01 5.4898347132237e-01
9.9332750038198e-01 8.3829637826292e-01 5.3761574225935e-01
6.6127177459113e-01 8.4101466693083e-01 5.2624824089773e-01
3.2973086091292e-01 8.4373384814470e-01 5.1488087750334e-01
5.0376776856674e-01 6.1429218243381e-01 5.0356618004852e-01
1.5873593259848e-01 6.1698477649004e-01 4.9220738513304e-01
8.1396668405054e-01 6.1967766962625e-01 4.8084863391090e-01
4.6946027048738e-01 6.2237092414588e-01 4.6948995305034e-01
1.2521356650648e-01 6.2506457278006e-01 4.5813137444717e-01
7.8122073399195e-01 6.2775862557759e-01 4.4677293003048e-01
4.3747388042607e-01 6.3045307631067e-01 4.3541464818748e-01
9.3963666957520e-02 6.3314790801049e-01 4.2405655159412e-01
7.5067983790666e-01 6.3584309747700e-01 4.1269865615245e-01
4.0761165985780e-01 6.3853861876761e-01 4.0134097072643e-01
6.4748270277099e-02 6.4123444576389e-01 3.8998349740381e-01
7.2207894292084e-01 6.4393055396036e-01 3.7862623206651e-01
3.7959328105117e-01 6.4662692162879e-01 3.6726916510974e-01
3.7281350708906e-02 6.4932353050215e-01 3.5591228220148e-01
6.9513376594728e-01 6.5202036610193e-01 3.4455556501484e-01
3.5314173672954e-01 6.5471741780976e-01 3.3319899189648e-01
1.1297088633017e-02 6.5741467876252e-01 3.2184253845536e-01
6.6959226188974e-01 6.6011214563050e-01 3.1048617806982e-01
3.2802029579877e-01 6.6280981832241e-01 2.9912988231920e-01
9.8657480324418e-01 6.6550769964902e-01 2.8777362135029e-01
6.4524993896705e-01 6.6820579496705e-01 2.7641736419071e-01
3.0404036435863e-01 6.7090411181869e-01 2.6506107902108e-01
9.6294121084159e-01 6.7360265957647e-01 2.5370473341698e-01
6.2194804335597e-01 6.7630144910014e-01 2.4234829457008e-01
2.8105682503898e-01 6.7900049240938e-01 2.3099172949623e-01
9.4026388385567e-01 6.8169980237481e-01 2.1963500523640e-01
5.9956588168115e-01 6.8439939242835e-01 2.0827808905487e-01
2.5895978613587e-01 6.8709927629368e-01 1.9692094863762e-01
9.1844284532107e-01 6.8979946773676e-01 1.8556355229231e-01
5.7801256548136e-01 6.9249998033636e-01 1.7420586915042e-01
2.3766669152844e-01 6.9520082727428e-01 1.6284786937099e-01
8.9740319028807e-01 6.9790202114510e-01 1.5148952434456e-01
5.5722023627882e-01 7.0060357378495e-01 1.4013080689556e-01
2.1711619979177e-01 7.0330549611930e-01 1.2877169148048e-01
8.7708963701480e-01 7.0600779802930e-01 1.1741215437918e-01
5.3713928193028e-01 7.0871048823659e-01 1.0605217387631e-01
1.9726403971184e-01 7.1141357420634e-01 9.4691730429712e-02
8.5746298135143e-01 7.1411706206832e-01 8.3330806822682e-02
5.1773533926314e-01 7.1682095655578e-01 7.1969388297282e-02
1.7808050363327e-01 7.1952526096168e-01 6.0607462665920e-02
8.3849801931624e-01 7.2222997711211e-01 4.9245020398936e-02
4.9898758311227e-01 7.2493510535612e-01 3.7882054686337e-02
1.5954904130404e-01 7.2764064457141e-01 2.6518561472341e-02
8.2018238737412e-01 7.3034659218503e-01 1.5154539462013e-02
4.8088775987169e-01 7.3305294420805e-01 3.7899900998783e-03
1.4038813213271e-01 7.3576326562532e-01 9.9242071053804e-01
8.0122930504084e-01 7.3847046579349e-01 9.8105505750068e-01
4.6214355803322e-01 7.4117804876301e-01 9.6968889876432e-01
1.2313159327438e-01 7.4388600546292e-01 9.5832224599274e-01
7.8419425172889e-01 7.4659432574642e-01 9.4695511310252e-01
4.4533251381199e-01 7.4930299848272e-01 9.3558751610577e-01
1.0654750118190e-01 7.5201201165367e-01 9.2421947294027e-01
7.6784047994499e-01 7.5472135245410e-01 9.1285100329071e-01
4.2921286556381e-01 7.5743100739514e-01 9.0148212840476e-01
9.0666229772541e-02 7.6014096240982e-01 8.9011287090794e-01
7.5220230981590e-01 7.6285120296062e-01 8.7874325462072e-01
4.1382302033739e-01 7.6556171414915e-01 8.6737330438176e-01
7.5530468251094e-02 7.6827248082816e-01 8.5600304588030e-01
7.3732697093997e-01 7.7098348771684e-01 8.4463250550100e-01
3.9921507813388e-01 7.7369471952042e-01 8.3326171018382e-01
6.1197597832340e-02 7.7640616105571e-01 8.2189068730155e-01
7.2327762665244e-01 7.7911779738461e-01 8.1051946455693e-01
3.8545858500010e-01 7.8182961395799e-01 7.9914806990103e-01
4.7744257484056e-02 7.8454159677302e-01 7.8777653147417e-01
7.1013883901520e-01 7.8725373254735e-01 7.7640487756988e-01
3.7264698705604e-01 7.8996600891458e-01 7.6503313662194e-01
3.5273880504190e-02 7.9267841464567e-01 7.5366133721355e-01
6.9802528570143e-01 7.9539093990228e-01 7.4228950810635e-01
3.6090763004730e-01 7.9810357652868e-01 7.3091767828570e-01
2.3928083647044e-02 8.0081631838979e-01 7.1954587701628e-01
6.8709464931389e-01 8.0352916176407e-01 7.0817413389930e-01
3.5041626103986e-01 8.0624210580052e-01 6.9680247891872e-01
1.3902890693708e-02 8.0895515304936e-01 6.8543094245899e-01
6.7756566212248e-01 8.1166831007508e-01 6.7405955527039e-01
3.4141697091201e-01 8.1438158815803e-01 6.6268834834992e-01
5.4706066449448e-03 8.1709500408495e-01 6.5131735269610e-01
6.6974187236485e-01 8.1980858101796e-01 6.3994659888517e-01
3.3424769282383e-01 8.2252234941282e-01 6.2857611640481e-01
9.9900669859487e-01 8.2523634792652e-01 6.1720593267219e-01
6.6403926685163e-01 8.2795062420546e-01 6.0583607166051e-01
3.2936749115551e-01 8.3066523537313e-01 5.9446655206889e-01
9.9501504173312e-01 8.3338024793089e-01 5.8309738500793e-01
6.6100686480036e-01 8.3609573664408e-01 5.7172857125452e-01
3.2736865596411e-01 8.3881178180903e-01 5.6036009827992e-01
9.9412603215030e-01 8.4152846410470e-01 5.4899193749998e-01
6.6130332562171e-01 8.4424585607934e-01 5.3762404255414e-01
3.2892194324292e-01 8.4696400931280e-01 5.2625634987389e-01
9.9699828821601e-01 8.4968293660038e-01 5.1488878326160e-01
1.7051951261572e-01 6.2024937517833e-01 5.0355676978092e-01
8.2521804932651e-01 6.2294177544072e-01 4.9219782892918e-01
4.8020719029670e-01 6.2563451939713e-01 4.8083892746903e-01
1.3548695151941e-01 6.2832767488833e-01 4.6948009718465e-01
7.9105334336847e-01 6.3102127471881e-01 4.5812137579232e-01
4.4689919973702e-01 6.3371532523516e-01 4.4676280066299e-01
1.0301499633724e-01 6.3640981420012e-01 4.3540440463818e-01
7.5938959606987e-01 6.3910471748291e-01 4.2404621362440e-01
4.1601088728586e-01 6.4180000440027e-01 4.1268824554861e-01
7.2866302955224e-02 6.4449564175108e-01 4.0133051026011e-01
7.2994322392078e-01 6.4719159670270e-01 3.8997301002481e-01
3.8722927822037e-01 6.4988783873439e-01 3.7861574033901e-01
4.4712552427835e-02 6.5258434084526e-01 3.6725869087037e-01
7.0238173170297e-01 6.5528108021346e-01 3.5590184640153e-01
3.6022618410394e-01 6.5797803846152e-01 3.4454518770421e-01
1.8236002674360e-02 6.6067520165045e-01 3.3318869230843e-01
6.7640201649953e-01 6.6337256009509e-01 3.2183233515619e-01
3.3471577969211e-01 6.6607010806839e-01 3.1047608914393e-01
9.9316954529234e-01 6.6876784344268e-01 2.9911992556600e-01
6.5175622941479e-01 6.7146576730085e-01 2.8776381447535e-01
3.1046936963898e-01 6.7416388353948e-01 2.7640772497812e-01
9.6930308059230e-01 6.7686219847792e-01 2.6505162547806e-01
6.2825200886403e-01 6.7956072048205e-01 2.5369548388475e-01
2.8731128877251e-01 6.8225945960748e-01 2.4233926779760e-01
9.4647650004229e-01 6.8495842726467e-01 2.3098294467517e-01
6.0574362809840e-01 6.8765763590687e-01 2.1962648199715e-01
2.6510902742295e-01 6.9035709874063e-01 2.0826984742481e-01
9.2456938822261e-01 6.9305682945841e-01 1.9691300896333e-01
5.8412170650729e-01 6.9575684199225e-01 1.8555593512871e-01
2.4376325756889e-01 6.9845715028757e-01 1.7419859511998e-01
9.0349157276398e-01 7.0115776809594e-01 1.6284095899697e-01
5.6330441944067e-01 7.0385870878615e-01 1.5148299786265e-01
2.2319978380161e-01 7.0655998517253e-01 1.4012468404854e-01
8.8317585646106e-01 7.0926160935997e-01 1.2876599130096e-01
5.4323102042984e-01 7.1196359260509e-01 1.1740689496573e-01
2.0336384124792e-01 7.1466594519308e-01 1.0604737216823e-01
8.6357305897960e-01 7.1736867632993e-01 9.4687401985971e-02
5.2385758178849e-01 7.2007179404993e-01 8.3326965610400e-02
1.8421648082039e-01 7.2277530513799e-01 7.1966046494839e-02
8.4464898613938e-01 7.2547921506706e-01 6.0604630485663e-02
5.0515448348693e-01 7.2818352795004e-01 4.9242705933923e-02
1.6573251166365e-01 7.3088824650634e-01 3.7880263785046e-02
8.2638276036882e-01 7.3359337204272e-01 2.6517297644657e-02
4.8710506837246e-01 7.3629890444794e-01 1.5153803819050e-02
1.4789942193713e-01 7.3900484220094e-01 3.7897813294072e-03
8.0750683944183e-01 7.4171470299992e-01 9.9242108315050e-01
4.6843636030618e-01 7.4442149955530e-01 9.8105594550134e-01
1.2943854377589e-01 7.4712868703076e-01 9.6969029378597e-01
7.9051395509300e-01 7.4983625859019e-01 9.5832413750792e-01
4.5166330310923e-01 7.5254420620569e-01 9.4695748857349e-01
1.1288744097279e-01 7.5525252073052e-01 9.3559036116293e-01
7.7418736770625e-01 7.5796119197966e-01 9.2422277158537e-01
4.3556423091312e-01 7.6067020881709e-01 9.1285473812058e-01
9.7019330874776e-02 7.6337955924915e-01 9.0148628085100e-01
7.5855412631989e-01 7.6608923052337e-01 8.9011742148772e-01
4.2017024216367e-01 7.6879920923257e-01 8.7874818319385e-01
8.1869479527395e-02 7.7150948142392e-01 8.6737859040882e-01
7.4365382835792e-01 7.7422003271347e-01 8.5600866867716e-01
4.0552548297450e-01 7.7693084840648e-01 8.4463844448472e-01
6.7486860876819e-02 7.7964191362446e-01 8.3326794510538e-01
7.2954062515325e-01 7.8235321344025e-01 8.2189719846093e-01
3.9168971083409e-01 7.8506473302260e-01 8.1052623299616e-01
5.3937355538926e-02 7.8777645779234e-01 7.9915507757102e-01
7.1628713477157e-01 7.9048837359261e-01 7.8778376137120e-01
3.7874300221778e-01 7.9320046687607e-01 7.7641231383755e-01
4.1309335397877e-02 7.9591272491251e-01 7.6504076461425e-01
7.0399098701467e-01 7.9862513602095e-01 7.5366914351468e-01
3.6679334230958e-01 8.0133768983077e-01 7.4229748050230e-01
2.9722382686647e-02 8.0405037757714e-01 7.3092580568274e-01
6.9278475577010e-01 8.0676319243656e-01 7.1955414930092e-01
3.5598785190351e-01 8.0947612990856e-01 7.0818254173422e-01
1.9339886845382e-02 8.1218918824998e-01 6.9681101346978e-01
6.8284999002020e-01 8.1490236896728e-01 6.8543959504901e-01
3.4652829708021e-01 8.1761567737075e-01 6.7406831695790e-01
1.0386044629937e-02 8.2032912319109e-01 6.6269720943478e-01
6.7443566364007e-01 8.2304272125166e-01 6.5132630216050e-01
3.3869086616665e-01 8.2575649217913e-01 6.3995562378870e-01
3.1667172930544e-03 8.2847046311670e-01 6.2858520126705e-01
6.6787968048879e-01 8.3118466837642e-01 6.1721505889761e-01
3.3284761959044e-01 8.3389914992529e-01 6.0584521708764e-01
9.9808973418368e-01 8.3661395754057e-01 5.9447569075940e-01
6.6362639734522e-01 8.3932914838834e-01 5.8310648742618e-01
3.2947885600422e-01 8.4204478567553e-01 5.7173760501634e-01
9.9566874600791e-01 8.4476093590473e-01 5.6036902965067e-01
6.6221736916632e-01 8.4747766414217e-01 5.4900073376735e-01
3.2914468296982e-01 8.5019502663483e-01 5.3763267524526e-01
9.9646797239456e-01 8.5291306015676e-01 5.2626479848060e-01
6.6420021554994e-01 8.5563176773936e-01 5.1489703865106e-01
8.3728139657535e-01 6.2620643051864e-01 5.0354765923273e-01
4.9168131644502e-01 6.2889860515879e-01 4.9218856453269e-01
1.4640359572420e-01 6.3159117556530e-01 4.8082950358639e-01
8.0144792884166e-01 6.3428421599684e-01 4.6947051430299e-01
4.5680922558316e-01 6.3697775898238e-01 4.5811164127945e-01
1.1247866252250e-01 6.3967180610303e-01 4.4675292815922e-01
7.6844470850560e-01 6.4236633774428e-01 4.3539441272363e-01
4.2469404395550e-01 6.4506132122401e-01 4.2403612424858e-01
8.1212333156061e-02 6.4775671713232e-01 4.1267808253990e-01
7.3798483873483e-01 6.5045248399059e-01 4.0132029808707e-01
3.9499688701355e-01 6.5314858147617e-01 3.8996277287442e-01
5.2234203269357e-02 6.5584497250349e-01 3.7860550150903e-01
7.0968313979117e-01 6.5854162444137e-01 3.6724847243578e-01
3.6733081938204e-01 6.6123850970740e-01 3.5589166909966e-01
2.5165214563137e-02 6.6393560593281e-01 3.4453507098132e-01
6.8317517947236e-01 6.6663289584489e-01 3.3317865447571e-01
3.4135044809509e-01 6.6933036697453e-01 3.2182239361181e-01
9.9968160941653e-01 6.7202801126448e-01 3.1046626062656e-01
6.5816006750939e-01 6.7472582462989e-01 2.9911022641404e-01
3.1677799249248e-01 6.7742380650466e-01 2.8775426087290e-01
9.7552826667457e-01 6.8012195939474e-01 2.7639833317475e-01
6.3440442896240e-01 6.8282028845070e-01 2.6504241197389e-01
2.9340061968776e-01 6.8551880106624e-01 2.5368646557569e-01
9.5251152732614e-01 6.8821750650530e-01 2.4233046207811e-01
6.1173233808158e-01 6.9091641555843e-01 2.3097436949784e-01
2.7105868895028e-01 6.9361554022729e-01 2.1961815588978e-01
9.3048662461460e-01 6.9631489343593e-01 2.0826178946643e-01
5.9001255833034e-01 6.9901448876656e-01 1.9690523872177e-01
2.4963323683388e-01 7.0171434021798e-01 1.8554847256254e-01
9.0934570919685e-01 7.0441446198461e-01 1.7419146044847e-01
5.6914729948404e-01 7.0711486825432e-01 1.6283417254183e-01
2.2903558301725e-01 7.0981557302346e-01 1.5147657986600e-01
8.8900836600991e-01 7.1251658992771e-01 1.4011865447156e-01
5.4906366830934e-01 7.1521793208757e-01 1.2876036960819e-01
2.0919970896511e-01 7.1791961196776e-01 1.1740169990010e-01
8.6941489433043e-01 7.2062164124959e-01 1.0604262152197e-01
5.2970780839880e-01 7.2332403071600e-01 9.4683112372824e-02
1.9007720507954e-01 7.2602679014889e-01 8.3323152244333e-02
8.5052200212316e-01 7.2872992823860e-01 7.1962722980470e-02
5.1104127642096e-01 7.3143345250544e-01 6.0601808625338e-02
1.7163426042189e-01 7.3413736923340e-01 4.9240395556036e-02
8.3230033943443e-01 7.3684168341598e-01 3.7878472597766e-02
4.9303904961076e-01 7.3954639871442e-01 2.6516031118646e-02
1.5385007644468e-01 7.4225151742814e-01 1.5153065102107e-02
8.1473325365296e-01 7.4495704047765e-01 3.7895711952457e-03
4.7444810452024e-01 7.4766643698592e-01 9.9242145973920e-01
1.3546597957879e-01 7.5037282561654e-01 9.8105684339714e-01
7.9655617407504e-01 7.5307961140924e-01 9.6969170668791e-01
4.5771910906525e-01 7.5578678983656e-01 9.5832605689258e-01
1.1895535240518e-01 7.5849435508569e-01 9.4695990381839e-01
7.8026561959503e-01 7.6120230011045e-01 9.3559325969998e-01
4.4165077529467e-01 7.6391061669328e-01 9.2422613908091e-01
1.0311183570852e-01 7.6661929551658e-01 9.1285855867816e-01
7.6464997206998e-01 7.6932832624302e-01 9.0149053723254e-01
4.2626651548026e-01 7.7203769760431e-01 8.9012209534834e-01
8.7962963377263e-02 7.7474739749823e-01 8.7875325532544e-01
7.4974098792673e-01 7.7745741309389e-01 8.6738404098737e-01
4.1160244663958e-01 7.8016773094513e-01 8.5601447750866e-01
7.3549395527915e-02 7.8287833711267e-01 8.4464459124474e-01
7.3558410511642e-01 7.8558921729539e-01 8.3327440956741e-01
3.9770907962777e-01 7.8830035697189e-01 8.2190396070857e-01
5.9927079658546e-02 7.9101174155325e-01 8.1053327361477e-01
7.2224114865747e-01 7.9372335654888e-01 7.9916237781425e-01
3.8465464350830e-01 7.9643518774718e-01 7.8779130329816e-01
4.7171269505098e-02 7.9914722141326e-01 7.7642008041633e-01
7.0979511998433e-01 8.0185944450661e-01 7.6504873978780e-01
3.7253072084283e-01 8.0457184492161e-01 7.5367731222478e-01
3.5383080116126e-02 8.0728441175448e-01 7.4230582866762e-01
6.9835774270828e-01 8.0999713560037e-01 7.3093432012703e-01
3.6146085023923e-01 8.1271000888470e-01 7.1956281762735e-01
2.4699205786680e-02 8.1542302623275e-01 7.0819135214264e-01
6.8808034302052e-01 8.1813618488085e-01 6.9681995451427e-01
3.5161259881731e-01 8.2084948513173e-01 6.8544865533487e-01
1.5305187847791e-02 8.2356293085387e-01 6.7407748477963e-01
6.7916827677836e-01 8.2627653002066e-01 6.6270647236105e-01
3.4321305452557e-01 8.2899029527846e-01 6.5133564657819e-01
7.4517933316131e-03 8.3170424452133e-01 6.3996503442731e-01
6.7189789315359e-01 8.3441840143405e-01 6.2859466073765e-01
3.3656589883236e-01 8.3713279594017e-01 6.1722454729678e-01
1.4714756098274e-03 8.3984746445723e-01 6.0585471173776e-01
6.6663132376196e-01 8.4256244981358e-01 5.9448516617934e-01
3.3206300758180e-01 8.4527780061937e-01 5.8311591564777e-01
9.9778466824052e-01 8.4799356980937e-01 5.7174695637341e-01
6.6381458539318e-01 8.5070981199388e-01 5.6037827415520e-01
3.3017055097374e-01 8.5342657918319e-01 5.4900984312922e-01
9.9686902387279e-01 8.5614391442295e-01 5.3764162546083e-01
6.6392405071463e-01 8.5886184294412e-01 5.2627357268280e-01
3.3134597063801e-01 8.6158036066423e-01 5.1490562956923e-01
5.0405483229285e-01 6.3216334283068e-01 5.0353886354409e-01
1.5812318651246e-01 6.3485525453470e-01 4.9217960711617e-01
8.1255019568814e-01 6.3754762309118e-01 4.8082037728270e-01
4.6733510670557e-01 6.4024053018710e-01 4.6946121936681e-01
1.2247141392475e-01 6.4293400750932e-01 4.5810218616754e-01
7.7794820662694e-01 6.4562805043361e-01 4.4674332858991e-01
4.3375146531978e-01 6.4832263019912e-01 4.3538468987851e-01
8.9865202773682e-02 6.5101770383410e-01 4.2402630275635e-01
7.4627240182588e-01 6.5371322169386e-01 4.1266818863790e-01
4.0295574288648e-01 6.5640913282602e-01 4.0131035815581e-01
5.9898138868685e-02 6.5910538853971e-01 3.8995281240124e-01
7.1708310708335e-01 6.6180194458972e-01 3.7859554445408e-01
3.7449501069577e-01 6.6449876235110e-01 3.6723854093341e-01
3.2119200354671e-02 6.6719580929398e-01 3.5588178341571e-01
6.8994208225942e-01 6.6989305899788e-01 3.4452524964989e-01
3.4795113391943e-01 6.7259049088064e-01 3.3316891455041e-01
6.1348840989164e-03 6.7528808976505e-01 3.2181275097923e-01
6.6448286934400e-01 6.7798584536608e-01 3.1045673034290e-01
3.2298557617248e-01 6.8068375175224e-01 2.9910082303677e-01
9.8163437542869e-01 6.8338180681426e-01 2.8774499876824e-01
6.4042145336212e-01 6.8608001175973e-01 2.7638922678864e-01
2.9933974255522e-01 6.8877837064343e-01 2.6503347605900e-01
9.5838285478813e-01 6.9147688993706e-01 2.5367771537088e-01
6.1754501718771e-01 6.9417557813851e-01 2.4232191343904e-01
2.7682101248656e-01 6.9687444541864e-01 2.3096603897922e-01
9.3620612385408e-01 6.9957350330272e-01 2.1961006078094e-01
5.9569608451008e-01 7.0227276438309e-01 2.0825394778265e-01
2.5528703215926e-01 7.0497224205955e-01 1.9689766915437e-01
9.1497546816686e-01 7.0767195030427e-01 1.8554119439112e-01
5.7475822131616e-01 7.1037190344815e-01 1.7418449341902e-01
2.3463241593343e-01 7.1307211598607e-01 1.6282753671483e-01
8.9459544412954e-01 7.1577260239857e-01 1.5147029543855e-01
5.5464494188143e-01 7.1847337698815e-01 1.4011274157825e-01
2.1477876866030e-01 7.2117445372844e-01 1.2875484810541e-01
8.7499499030206e-01 7.2387584612501e-01 1.1739658913853e-01
5.3529186480933e-01 7.2657756708680e-01 1.0603794011260e-01
1.9566783077257e-01 7.2927962880742e-01 9.4678877951465e-02
8.5612149809938e-01 7.3198204265587e-01 8.3319381239915e-02
5.1665164074724e-01 7.3468481907651e-01 7.1959430392266e-02
1.7725719116502e-01 7.3738796749812e-01 6.0599007814079e-02
8.3793723616316e-01 7.4009149625235e-01 4.9238098053658e-02
4.9869101395189e-01 7.4279541250163e-01 3.7876687940141e-02
1.5951791211092e-01 7.4549972217717e-01 2.6514766705156e-02
8.2041746628264e-01 7.4820442992714e-01 1.5152326085354e-02
4.8138935941432e-01 7.5090953907571e-01 3.7893604034898e-03
1.4121208172967e-01 7.5361846886909e-01 9.9242183891375e-01
8.0231836938642e-01 7.5632444651269e-01 9.8105774766994e-01
4.6349671205921e-01 7.5903082567185e-01 9.6969313179683e-01
1.2474738152805e-01 7.6173760418925e-01 9.5832799629305e-01
7.8607079796065e-01 7.6444477855145e-01 9.4696234878071e-01
4.4746753103084e-01 7.6715234391812e-01 9.3559619943815e-01
1.0893830151635e-01 7.6986029416312e-01 9.2422956090989e-01
7.7048398353397e-01 7.7256862192719e-01 9.1286244819607e-01
4.3210560760538e-01 7.7527731868216e-01 9.0149487852362e-01
9.3804364778020e-02 7.7798637480650e-01 8.9012687120208e-01
7.5558161205120e-01 7.8069577967195e-01 8.7875844746716e-01
4.1743887937882e-01 7.8340552174132e-01 8.6738963031508e-01
7.9377878535566e-02 7.8611558867753e-01 8.5602044433108e-01
7.4140051414376e-01 7.8882596746406e-01 8.4465091551533e-01
4.0350889716316e-01 7.9153664453719e-01 8.3328107110930e-01
6.5705361145077e-02 7.9424760593086e-01 8.2191093942543e-01
7.2799248154685e-01 7.9695883743480e-01 8.1054054968280e-01
3.9037309839012e-01 7.9967032476716e-01 7.9916993185072e-01
5.2850342527637e-02 8.0238205376301e-01 7.8779911650204e-01
7.1542766575616e-01 8.0509401058036e-01 7.7642813467687e-01
3.7810887497398e-01 8.0780618192565e-01 7.6505701775708e-01
4.0898170528516e-02 8.1051855530077e-01 7.5368579735044e-01
7.0380018882454e-01 8.1323111927423e-01 7.4231450518241e-01
3.6682004915912e-01 8.1594386377860e-01 7.3094317299199e-01
2.9963404602569e-02 8.1865678043696e-01 7.1957183242607e-01
6.9323649653860e-01 8.2136986292014e-01 7.0820051492487e-01
3.5664621218608e-01 8.2408310733619e-01 6.9682925158826e-01
2.0200144018536e-02 8.2679651265151e-01 6.8545807300991e-01
6.8390664943122e-01 8.2951008114074e-01 6.7408700906285e-01
3.4777490822414e-01 8.3222381885799e-01 6.6271608861683e-01
1.1814974403389e-02 8.3493773611523e-01 6.5134533916430e-01
6.7603781737139e-01 8.3765184794343e-01 6.3997478632972e-01
3.4045534568945e-01 8.4036617449748e-01 6.2860445323640e-01
5.0804041746395e-03 8.4308074134460e-01 6.1723435970825e-01
6.6992673209261e-01 8.4579557954760e-01 6.0586452129331e-01
3.3500886667550e-01 8.4851072541698e-01 5.9449494811547e-01
3.4197235357826e-04 8.5122621975904e-01 5.8312564359442e-01
6.6594157246769e-01 8.5394210639421e-01 5.7175660312864e-01
3.3182315783105e-01 8.5665842966626e-01 5.6038781291513e-01
9.9800164698191e-01 8.5937523062314e-01 5.4901924918769e-01
6.6449067870874e-01 8.6209254154857e-01 5.3765087828557e-01
3.3130173140443e-01 8.6481037859499e-01 5.2628265809922e-01
9.9844308886092e-01 8.6752873246034e-01 5.1491454153838e-01
1.7084154454866e-01 6.3812010700497e-01 5.0353039765700e-01
8.2454055936210e-01 6.4081171182507e-01 4.9217097166933e-01
4.7864008596628e-01 6.4350384558808e-01 4.8081156337303e-01
1.3313874591086e-01 6.4619659856196e-01 4.6945222717093e-01
7.8802819425368e-01 6.4889000070244e-01 4.5809302572414e-01
4.4329486620255e-01 6.5158403922309e-01 4.4673401837570e-01
9.8921630927275e-02 6.5427867392020e-01 4.3537525437842e-01
7.5488921699584e-01 6.5697384935460e-01 4.2401676987333e-01
4.1117735481038e-01 6.5966950382165e-01 4.1265858743250e-01
6.7765637217588e-02 6.6236557550013e-01 4.0130071712907e-01
7.2463413068323e-01 6.6506200635003e-01 3.8994315835416e-01
3.8176378210938e-01 6.6775874433828e-01 3.7858590185836e-01
3.9136667441804e-02 6.7045574449399e-01 3.6722893170749e-01
6.9673612323597e-01 6.7315296918956e-01 3.5587222699411e-01
3.5454679504984e-01 6.7585038794062e-01 3.4451576324613e-01
1.2554629048261e-02 6.7854797693066e-01 3.3315951353255e-01
6.7074682656946e-01 6.8124571839834e-01 3.2180344929716e-01
3.2911177597912e-01 6.8394359997586e-01 3.1044754096389e-01
9.8763897192753e-01 6.8664161403200e-01 2.9909175835984e-01
6.4631892897563e-01 6.8933975705009e-01 2.8773607099905e-01
3.0514309426364e-01 6.9203802905587e-01 2.7638044826407e-01
9.6410376226432e-01 6.9473643310070e-01 2.6502485951638e-01
6.2319399352327e-01 6.9743497479993e-01 2.5366927416007e-01
2.8240753850578e-01 7.0013366192324e-01 2.4231366167829e-01
9.4173876714121e-01 7.0283250403195e-01 2.3095799165694e-01
6.0118260430519e-01 7.0553151215807e-01 2.1960223380640e-01
2.6073447125135e-01 7.0823069851967e-01 2.0824635798911e-01
9.2039023285910e-01 7.1093007626744e-01 1.9689033425850e-01
5.8014615047400e-01 7.1362965925803e-01 1.8553413291243e-01
2.3999884006315e-01 7.1632946184997e-01 1.7417772456352e-01
8.9994523537753e-01 7.1902949871862e-01 1.6282108022685e-01
5.5998255579655e-01 7.2172978468733e-01 1.5146417142516e-01
2.2010827852228e-01 7.2443033457189e-01 1.4010697031047e-01
8.8032011478820e-01 7.2713116303649e-01 1.2874944980093e-01
5.4061598974666e-01 7.2983228445923e-01 1.1739158373061e-01
2.0099402570187e-01 7.3253371280585e-01 1.0603334701000e-01
8.6145252835737e-01 7.3523546151079e-01 9.4674715794502e-02
5.2198997575200e-01 7.3793754336483e-01 8.3315667657649e-02
1.8260500956425e-01 7.4063997040886e-01 7.1956181765975e-02
8.4329642847332e-01 7.4334275383391e-01 6.0596239051996e-02
5.0406318327672e-01 7.4604590388739e-01 4.9235822381815e-02
1.6490437347894e-01 7.4874942978603e-01 3.7874916713853e-02
8.2581924508466e-01 7.5145333963609e-01 2.6513509245285e-02
4.8680718935325e-01 7.5415764036143e-01 1.5151589542966e-02
1.4786774229887e-01 7.5686233764034e-01 3.7891496559143e-03
8.0779882528388e-01 7.5957079952781e-01 9.9242221930100e-01
4.6899363649943e-01 7.6227636434937e-01 9.8105865486586e-01
1.3026032064574e-01 7.6498233313268e-01 9.6969456357544e-01
7.9159899721323e-01 7.6768870614038e-01 9.5832994808703e-01
4.5300993405804e-01 7.7039548223163e-01 9.4696481375288e-01
1.1449354890480e-01 7.7310265886689e-01 9.3559916858584e-01
7.7605041116844e-01 7.7581023212590e-01 9.2423302319977e-01
4.3768124420781e-01 7.7851819673924e-01 9.1286639072599e-01
9.9386928164561e-02 7.8122654613357e-01 9.0149928670777e-01
7.6116850357676e-01 7.8393527249065e-01 8.9013172897494e-01
4.2302717598866e-01 7.8664436682033e-01 8.7876373750158e-01
8.4964321804295e-02 7.8935381904750e-01 8.6739533424937e-01
7.4698149565286e-01 7.9206361811309e-01 8.5602654299999e-01
4.0908043954755e-01 7.9477375208948e-01 8.4465738917958e-01
7.1263094126532e-02 7.9748420831033e-01 8.3328789967842e-01
7.3353161226438e-01 8.0019497351549e-01 8.2191810266883e-01
3.9588837533439e-01 8.0290603401141e-01 8.1054802742400e-01
5.8336012385925e-02 8.0561737584779e-01 7.9917770414011e-01
7.2087742247528e-01 8.0832898501133e-01 7.8780716376356e-01
3.8351580035226e-01 8.1104084763770e-01 7.7643643782461e-01
4.6254665654775e-02 8.1375295024284e-01 7.6506555827780e-01
7.0909789569677e-01 8.1646527997487e-01 7.5369455734873e-01
3.7204976184614e-01 8.1917782488810e-01 7.4232346738545e-01
3.5114969369596e-02 8.2189057424015e-01 7.3095232071154e-01
6.9829870046225e-01 8.2460351881329e-01 7.1958114947616e-01
3.6160665996426e-01 8.2731665126028e-01 7.0820998549463e-01
2.5045122976753e-02 8.3002996647404e-01 6.9683886007078e-01
6.8862098319790e-01 8.3274346197858e-01 6.8546780379007e-01
3.5234180027349e-01 8.3545713833586e-01 6.7409684627003e-01
1.6215843753828e-02 8.3817099955901e-01 6.6272601585193e-01
6.8025213032004e-01 8.4088505351578e-01 6.5135533921597e-01
3.4446044973387e-01 8.4359931229711e-01 6.3998484090116e-01
8.8513734228053e-03 8.4631379251269e-01 6.2861454271246e-01
6.7343623770037e-01 8.4902851545778e-01 6.1724446300212e-01
3.3822709134258e-01 8.5174350707267e-01 6.0587461582257e-01
3.2365946834358e-03 8.5445879758694e-01 5.9450500996635e-01
6.6847785478541e-01 8.5717442070617e-01 5.8313564793854e-01
3.3396417905036e-01 8.5989041216179e-01 5.7176652495164e-01
9.9970872865859e-01 8.6260680741017e-01 5.6039762809574e-01
6.6572405468033e-01 8.6532363824759e-01 5.4902893591672e-01
3.3202150524646e-01 8.6804092811932e-01 5.3766041872788e-01
9.9861050369851e-01 8.7075868597004e-01 5.2629204006948e-01
6.6549771678464e-01 8.7347689863787e-01 5.1492375978677e-01
8.3764366019940e-01 6.4407671859349e-01 5.0352227625765e-01
4.9092962934898e-01 6.4676796448412e-01 4.9216267293867e-01
1.4466484396538e-01 6.4945982499298e-01 4.8080307639551e-01
7.9884704672988e-01 6.5215240026970e-01 4.6944355229013e-01
4.5346547581709e-01 6.5484571716970e-01 4.5808417524648e-01
1.0850315729624e-01 6.5753975207171e-01 4.4672501443938e-01
7.6393903578952e-01 6.6023445033594e-01 4.3536612566218e-01
4.1974976680380e-01 6.6292974132479e-01 4.2400754826735e-01
7.5911111971485e-02 6.6562554907217e-01 4.1264930527228e-01
7.3239895460690e-01 6.6832179927588e-01 4.0129140521085e-01
3.8918999146222e-01 6.7101842347096e-01 3.8993384471883e-01
4.6262168667717e-02 6.7371536119565e-01 3.7857661121839e-01
7.0359492680515e-01 6.7641256081733e-01 3.6721968535521e-01
3.6116931016573e-01 6.7910997952103e-01 3.5586304303735e-01
1.8967983537572e-02 6.8180758281488e-01 3.4450665704249e-01
6.7697518894919e-01 6.8450534378968e-01 3.3315049822437e-01
3.3517666570437e-01 6.8720324228336e-01 3.2179453637755e-01
9.9355954997146e-01 6.8990126404063e-01 3.1043874082735e-01
6.5211226493327e-01 6.9259939991823e-01 2.9908308080880e-01
3.1082440872503e-01 6.9529764516013e-01 2.8772752569022e-01
9.6968664474085e-01 6.9799599875167e-01 2.7637204508713e-01
6.2869059707739e-01 7.0069446285212e-01 2.6501660890306e-01
2.8782875266806e-01 7.0339304230050e-01 2.5366118732536e-01
9.4709437086393e-01 7.0609174418698e-01 2.4230575079704e-01
6.0648140070777e-01 7.0879057748178e-01 2.3095026998029e-01
2.6598440583113e-01 7.1148955271321e-01 2.1959471572289e-01
9.2559849671705e-01 7.1418868168748e-01 2.0823905903524e-01
5.8531926996733e-01 7.1688797724350e-01 1.9688327108325e-01
2.4514275416179e-01 7.1958745303665e-01 1.8552732320050e-01
9.0506536187907e-01 7.2228712334655e-01 1.7417118692126e-01
5.6508384744830e-01 7.2498700290423e-01 1.6281483403545e-01
2.2519527001129e-01 7.2768710673517e-01 1.5145823666500e-01
8.8539696148943e-01 7.3038745001489e-01 1.4010136736113e-01
5.4568649906471e-01 7.3308804793451e-01 1.2874419922092e-01
2.0606168179896e-01 7.3578891557406e-01 1.1738670602153e-01
8.6652051102784e-01 7.3849006778192e-01 1.0602886236967e-01
5.2706117417725e-01 7.4119151905896e-01 9.4670643863835e-02
1.8768203165849e-01 7.4389328344644e-01 8.3312027266229e-02
8.4838160650761e-01 7.4659537441726e-01 7.1952990681348e-02
5.0915857644166e-01 7.4929780477013e-01 6.0593513737649e-02
1.7001176801472e-01 7.5200058652694e-01 4.9233577768759e-02
8.3094015256689e-01 7.5470373083370e-01 3.7873165990512e-02
4.9194284367421e-01 7.5740724786566e-01 2.6512263670087e-02
1.5301909582452e-01 7.6011114673756e-01 1.5150858283595e-02
8.1416830406674e-01 7.6281543542007e-01 3.7889396586225e-03
4.7420828731128e-01 7.6552342944939e-01 9.9242259952781e-01
1.3549178409288e-01 7.6822858083895e-01 9.8105956155268e-01
7.9684705688664e-01 7.7093413670269e-01 9.6969599655535e-01
4.5827407190650e-01 7.7364009976168e-01 9.5833190479341e-01
1.1977294206224e-01 7.7634647131025e-01 9.4696728926479e-01
7.8134392895430e-01 7.7905325119482e-01 9.3560215570795e-01
4.4298744505887e-01 7.8176043780684e-01 9.2423651257466e-01
1.0470405616734e-01 7.8446802809073e-01 9.1287037097591e-01
7.6649448418928e-01 7.8717601756731e-01 9.0150374460217e-01
4.2835961046978e-01 7.8988440037330e-01 8.9013664961973e-01
9.0300479809850e-02 7.9259316931718e-01 8.7876910454521e-01
7.5231830541059e-01 7.9530231595187e-01 8.6740113010104e-01
4.1441447498815e-01 7.9801183066421e-01 8.5603274905448e-01
7.6590558324424e-02 8.0072170278170e-01 8.4466398604343e-01
7.3884831652971e-01 8.0343192069652e-01 8.3329486739215e-01
4.0118971329465e-01 8.0614247200719e-01 8.2192542091980e-01
6.3616928401577e-02 8.0885334367809e-01 8.1055567574480e-01
7.2613237374710e-01 8.1156452221730e-01 7.9918566208766e-01
3.8873871209844e-01 8.1427599387296e-01 7.8781541107434e-01
5.1438878763640e-02 8.1698774484892e-01 7.7644495454179e-01
7.1423610629821e-01 8.1969976153995e-01 7.6507432484662e-01
3.7713395229464e-01 8.2241203078706e-01 7.5370355467688e-01
4.0136330202443e-02 8.2512454015334e-01 7.4233267686588e-01
7.0324754299777e-01 8.2783727822039e-01 7.3096172420595e-01
3.6647231935490e-01 8.3055023490498e-01 7.1959072925820e-01
2.9815851755135e-02 8.3326340179491e-01 7.0821972415327e-01
6.9328383568519e-01 8.3597677250165e-01 6.9684874037566e-01
3.5688250870816e-01 8.3869034302557e-01 6.8547780852298e-01
2.0618687708849e-02 8.4140411212677e-01 6.7410695802895e-01
6.8449980199250e-01 8.4411808169031e-01 6.6273621683811e-01
3.4853391911757e-01 8.4683225706896e-01 6.5136561101842e-01
1.2729759334692e-02 8.4954664737833e-01 6.3999516429878e-01
6.7709669325854e-01 8.5226126570833e-01 6.2862489752024e-01
3.4164471590688e-01 8.5497612920048e-01 6.1725482799506e-01
6.3843885340506e-03 8.5769125892209e-01 6.0588496877777e-01
6.7132673786366e-01 8.6040667944612e-01 5.9451532786909e-01
3.3648310060788e-01 8.6312241802051e-01 5.8314590739886e-01
1.8648999348285e-03 8.6583850318481e-01 5.7177670287073e-01
6.6748334043353e-01 8.6855496267148e-01 5.6040770259999e-01
3.3334901001856e-01 8.7127182042129e-01 5.4903888753559e-01
9.9947138224578e-01 8.7398909256110e-01 5.3767023172257e-01
6.6585822188971e-01 8.7670678225406e-01 5.2630170372028e-01
3.3251491126115e-01 8.7942487345683e-01 5.1493326932104e-01
5.0446382972917e-01 6.5003317400115e-01 5.0351451371591e-01
1.5728567880277e-01 6.5272399892781e-01 4.9215472536227e-01
8.1061410236995e-01 6.5541554112550e-01 4.8079493053069e-01
4.6444559814729e-01 6.5810791206424e-01 4.6943520903095e-01
1.1876618977255e-01 6.6080113340955e-01 4.5807565012765e-01
7.7355447652887e-01 6.6349516703311e-01 4.4671633446375e-01
4.2878440692884e-01 6.6618993994143e-01 4.3535732482853e-01
8.4427546796109e-02 6.6888536287309e-01 4.2399866329282e-01
7.4045478576980e-01 6.7158134295862e-01 4.1264037223228e-01
3.9683752981033e-01 6.7427779157620e-01 4.0128245729116e-01
5.3548472934583e-02 6.7697462868615e-01 3.8992491098278e-01
7.1056204978360e-01 6.7967178477854e-01 3.7856771617088e-01
3.6785465987461e-01 6.8236920131524e-01 3.6721084906576e-01
2.5404736616394e-02 6.8506683029633e-01 3.5585428161347e-01
6.8319271601042e-01 6.8776463337209e-01 3.4449798329156e-01
3.4120094435004e-01 6.9046258076676e-01 3.3314192238841e-01
9.9941355198862e-01 6.9316065017220e-01 3.2178606686443e-01
6.5781631118975e-01 6.9585882569819e-01 3.1043038489181e-01
3.1639648963808e-01 6.9855709692133e-01 2.9907484515787e-01
9.7514270675701e-01 7.0125545804745e-01 2.8771941700180e-01
6.3404479700637e-01 7.0395390718719e-01 2.7636407043964e-01
2.9309368238594e-01 7.0665244573648e-01 2.6500877611908e-01
9.5228125512933e-01 7.0935107785023e-01 2.5365350523510e-01
6.1160027080515e-01 7.1204980999649e-01 2.4229822942863e-01
2.7104425158802e-01 7.1474865057887e-01 2.3094292068407e-01
9.3060739921084e-01 7.1744760961588e-01 2.1958755123642e-01
5.9028451698660e-01 7.2014669846760e-01 2.0823209349544e-01
2.5007094024397e-01 7.2284592960083e-01 1.9687651999122e-01
9.0996247452353e-01 7.2554531638576e-01 1.8552080334406e-01
5.6995534090944e-01 7.2824487291783e-01 1.7416491626002e-01
2.3004612791119e-01 7.3094461385952e-01 1.6280883155235e-01
8.9023174935404e-01 7.3364455429781e-01 1.5145252218869e-01
5.5050940777894e-01 7.3634470961343e-01 1.4009596136295e-01
2.1087656289130e-01 7.3904509535895e-01 1.2873912259071e-01
8.7133090463085e-01 7.4174572714287e-01 1.1738197982618e-01
5.3187033046172e-01 7.4444662051788e-01 1.0602450759892e-01
1.9249292650368e-01 7.4714779087138e-01 9.4666681167631e-02
8.5319695214162e-01 7.4984925331718e-01 8.3308476688520e-02
5.1398082776325e-01 7.5255102258753e-01 7.1949871394992e-02
1.7484312528464e-01 7.5525311292498e-01 6.0590843785448e-02
8.3578256113174e-01 7.5795553797424e-01 4.9231373815548e-02
4.9679799135424e-01 7.6065831067417e-01 3.7871443091144e-02
1.5788840855812e-01 7.6336144315082e-01 2.6511035057950e-02
8.1905294035515e-01 7.6606494661232e-01 1.5150135183946e-02
4.8029084904392e-01 7.6876883124706e-01 3.7887311305838e-03
1.4044031979238e-01 7.7147635873097e-01 9.9242297820384e-01
8.0181271174666e-01 7.7418109732220e-01 9.8106046427657e-01
4.6325687008804e-01 7.7688623893180e-01 9.6969742526819e-01
1.2477260879086e-01 7.7959178876624e-01 9.5833385897852e-01
7.8635988513806e-01 7.8229775060701e-01 9.4696976596649e-01
4.4801880231365e-01 7.8500412676225e-01 9.3560514958654e-01
1.0974961208971e-01 7.8771091803277e-01 9.2424001599593e-01
7.7155271761972e-01 7.9041812369399e-01 9.1287437413255e-01
4.3342867639866e-01 7.9312574149490e-01 9.0150823566385e-01
9.5378203497738e-02 7.9583376767484e-01 8.9014161490776e-01
7.5740217522530e-01 7.9854219699903e-01 8.7877452872730e-01
4.1950163340416e-01 8.0125102281329e-01 8.6740699640115e-01
8.1677790488271e-02 8.0396023711849e-01 8.5603903947254e-01
7.4393203576580e-01 8.0666983066500e-01 8.4467068157946e-01
4.0626594291198e-01 8.0937979306727e-01 8.3330194826916e-01
6.8681279160192e-02 8.1209011293893e-01 8.2193286680018e-01
7.3118001635488e-01 8.1480077804816e-01 8.1056346593481e-01
3.9376434413215e-01 8.1751177549373e-01 7.9919377572495e-01
5.6436685443521e-02 8.2022309190140e-01 7.8782382729392e-01
7.1919971459239e-01 8.2293471364096e-01 7.7645365261614e-01
3.8205637789099e-01 8.2564662706348e-01 7.6508328429632e-01
4.5009916963376e-02 8.2835881875874e-01 7.5371275534853e-01
7.0806389461511e-01 8.3107127583211e-01 7.4234209897526e-01
3.7122222305637e-01 8.3378398620025e-01 7.3097134834461e-01
3.4489194095850e-02 8.3649693890387e-01 7.1960053636353e-01
6.9786951070862e-01 8.3921012443542e-01 7.0822969544256e-01
3.6136831910902e-01 8.4192353507792e-01 6.9685885724709e-01
2.4991240116156e-02 8.4463716524941e-01 6.8548805242778e-01
6.8874439816450e-01 8.4735101184481e-01 6.7411731032213e-01
3.5263444576582e-01 8.5006507456303e-01 6.6274665861767e-01
1.6668580548627e-02 8.5277935620220e-01 6.5137612296721e-01
6.8085455116862e-01 8.5549386289862e-01 6.4000572654726e-01
3.4520064738691e-01 8.5820860427605e-01 6.2863548955344e-01
9.7156684619728e-03 8.6092359345997e-01 6.1726542863208e-01
6.7440886273808e-01 8.6363884689720e-01 6.0589555625645e-01
3.3928983004281e-01 8.6635438390421e-01 5.9452588007102e-01
4.3683774187677e-03 8.6907022584964e-01 5.8315640224814e-01
6.6965431812935e-01 8.7178639485912e-01 5.7178711893108e-01
3.3515720426884e-01 8.7450291191853e-01 5.6041801987515e-01
8.8598531861229e-04 8.7721979425206e-01 5.4904908844248e-01
6.6684858939577e-01 8.7993705187192e-01 5.3768030215249e-01
3.3305143150113e-01 8.8265468325094e-01 5.2631163402793e-01
9.9949886426841e-01 8.8537267016814e-01 5.1494305499116e-01
1.7130539582691e-01 6.5598947074097e-01 5.0350712402201e-01
8.2360279424527e-01 6.5867980020102e-01 4.9214714299932e-01
4.7647497058149e-01 6.6137097109167e-01 4.8078713951130e-01
1.2991664109378e-01 6.6406310773723e-01 4.6942721139844e-01
7.8390949672901e-01 6.6675622343239e-01 4.5806746600144e-01
4.3842632090980e-01 6.6945026046102e-01 4.4670799732032e-01
9.3434639691838e-02 6.7214512232787e-01 4.3534887540286e-01
7.4889960919070e-01 6.7484069685575e-01 4.2399014409161e-01
4.0478610243546e-01 6.7753687112736e-01 4.1263182349939e-01
6.1060097552934e-02 6.8023354014332e-01 4.0127391453977e-01
7.1768951953022e-01 6.8293061110764e-01 3.8991640385237e-01
3.7464468622457e-01 6.8562800491659e-01 3.7855926825266e-01
3.1898484795855e-02 6.8832565600395e-01 3.6720247835474e-01
6.8942637463711e-01 6.9102351131918e-01 3.5584600131211e-01
3.4720628545869e-01 6.9372152892673e-01 3.4448980276656e-01
5.2184573045719e-03 6.9641967651407e-01 3.3313384814962e-01
6.6344525311330e-01 6.9911792996419e-01 3.2177810348682e-01
3.2187096308125e-01 7.0181627206712e-01 3.1042253583839e-01
9.8048161240116e-01 7.0451469139693e-01 2.9906711348575e-01
6.3926477890483e-01 7.0721318135454e-01 2.8771180594843e-01
2.9820942391855e-01 7.0991173936316e-01 2.7635658389497e-01
9.5730573764615e-01 7.1261036619820e-01 2.6500141899360e-01
6.1654499920450e-01 7.1530906543155e-01 2.5364628373493e-01
2.7591945076454e-01 7.1800784297183e-01 2.4229115124911e-01
9.3542218490408e-01 7.2070670668366e-01 2.3093599513222e-01
5.9504704413239e-01 7.2340566607159e-01 2.1958078929147e-01
2.5478853151848e-01 7.2610473201634e-01 2.0822550781526e-01
9.1464173139473e-01 7.2880391655314e-01 1.9687012487128e-01
5.7460223918038e-01 7.3150323268361e-01 1.8551461463461e-01
2.3466609945778e-01 7.3420269421402e-01 1.7415895124614e-01
8.9482975152404e-01 7.3690231561403e-01 1.6280310880122e-01
5.5508998172629e-01 7.3960211189088e-01 1.5144706136774e-01
2.1544388196578e-01 7.4230209847477e-01 1.4009078303261e-01
8.7588881382220e-01 7.4500229111204e-01 1.2873424797528e-01
5.3642237780605e-01 7.4770270576289e-01 1.1737743056653e-01
1.9704238729172e-01 7.5040335850144e-01 1.0602030549079e-01
8.5774684671931e-01 7.5310426541586e-01 9.4662847889728e-02
5.1853393368026e-01 7.5580544250717e-01 8.3305033524649e-02
1.7940198452082e-01 7.5850690558557e-01 7.1946838954864e-02
8.4034948311103e-01 7.6120867016352e-01 6.0588241728955e-02
5.0137505243592e-01 7.6391075134544e-01 4.9229220585404e-02
1.6247744867271e-01 7.6661316371426e-01 3.7869755658855e-02
8.2365555742351e-01 7.6931592121533e-01 2.6509828687945e-02
4.8490839178057e-01 7.7201903703888e-01 1.5149423220510e-02
1.4623509191066e-01 7.7472252350231e-01 3.7885248118261e-03
8.0649468038311e-01 7.7742958706486e-01 9.9242335390472e-01
4.6795621928334e-01 7.8013391477424e-01 9.8106135951944e-01
1.2948960369082e-01 7.8283864203569e-01 9.6969884417671e-01
7.9109449844667e-01 7.8554377655588e-01 9.5833580316128e-01
4.5277070649456e-01 7.8824932464460e-01 9.4697223450807e-01
1.1451817217364e-01 7.9095529113789e-01 9.3560813907667e-01
7.7633698454603e-01 7.9366167933551e-01 9.2424352059539e-01
4.3822738071399e-01 7.9636849095472e-01 9.1287838567377e-01
1.0018974913424e-01 7.9907572610193e-01 9.0151274378311e-01
7.6222463298896e-01 8.0178338326377e-01 8.9014660720524e-01
4.2433273372301e-01 8.0449145931884e-01 8.7877999095073e-01
8.6514914902732e-02 8.0719994957097e-01 8.6741291264791e-01
7.4877220659151e-01 8.0990884780486e-01 8.5604539240507e-01
4.1110581046881e-01 8.1261814636448e-01 8.4467745264835e-01
7.3517105942500e-02 8.1532783625462e-01 8.3330911793834e-01
7.3600765751477e-01 8.1803790726575e-01 8.2194041476839e-01
3.9857922366206e-01 8.2074834812203e-01 8.1057137134802e-01
6.1233767474040e-02 8.2345914665241e-01 7.9920201737447e-01
7.2397346926765e-01 8.2617028998447e-01 7.8783238379522e-01
3.8680074134591e-01 8.2888176476039e-01 7.7646250256419e-01
4.9718245007271e-02 8.3159355737462e-01 7.6509240639357e-01
7.1272890982620e-01 8.3430565423199e-01 7.5372212850257e-01
3.7583595511749e-01 8.3701804202514e-01 7.4235170236383e-01
3.9042913360803e-02 8.3973070802956e-01 7.3098116144686e-01
7.0235365519261e-01 8.4244364041350e-01 7.1961053895729e-01
3.6577241536413e-01 8.4515682855961e-01 7.0823986756902e-01
2.9303818805101e-02 8.4787026339339e-01 6.9686917914570e-01
6.9295290561595e-01 8.5058393771192e-01 6.8549850444621e-01
3.5672515342027e-01 8.5329784650366e-01 6.7412787280844e-01
2.0626495033001e-02 8.5601198724690e-01 6.6275731180458e-01
6.8466332882480e-01 8.5872636016955e-01 6.5138684686137e-01
3.4884251847945e-01 8.6144096844730e-01 6.4001650084020e-01
1.3171378051239e-02 8.6415581830932e-01 6.2864629357424e-01
6.7765763734955e-01 8.6687091901149e-01 6.1727624136551e-01
3.4230938175891e-01 8.6958628262549e-01 6.0590635645283e-01
7.1349597383944e-03 8.7230192358025e-01 5.9453664647444e-01
6.7214285060073e-01 8.7501785787862e-01 5.8316711396669e-01
3.3734148499939e-01 8.7773410190176e-01 5.7179775596369e-01
2.7390112135838e-03 8.8045067070723e-01 5.6042856379161e-01
6.6834300227624e-01 8.8316757573119e-01 5.4905952318433e-01
3.3416010276403e-01 8.8588482182617e-01 5.3769061487914e-01
1.9562013528591e-04 8.8860240361169e-01 5.2632181587595e-01
6.6645307409389e-01 8.9132030119397e-01 5.1495310154849e-01
8.3817263166408e-01 6.6194560778296e-01 5.0350012072041e-01
4.8987346948412e-01 6.6463535150286e-01 4.9213993945284e-01
1.4223123683914e-01 6.6732608846989e-01 4.8077971652119e-01
7.9523806874028e-01 6.7001795738260e-01 4.6941957309548e-01
4.4886972975327e-01 6.7271095831183e-01 4.5805963902381e-01
1.0309126492410e-01 6.7540500686941e-01 4.4670002377624e-01
7.5786185870081e-01 6.7809997626979e-01 4.3534080452582e-01
4.1313853024561e-01 6.8079572605915e-01 4.2398202523187e-01
6.8878678397947e-02 6.8349211957676e-01 4.1262370136813e-01
7.2504167421732e-01 6.8618903320731e-01 4.0126582663634e-01
3.8158974615387e-01 6.8888636025907e-01 3.8990837959208e-01
3.8488379566374e-02 6.9158401164039e-01 3.7855132923787e-01
6.9570640421412e-01 6.9428191482139e-01 3.6719463931135e-01
3.5321589437484e-01 6.9698001201590e-01 3.5583827133878e-01
1.0991965814651e-02 6.9967825812935e-01 3.4448218666305e-01
6.6901252383072e-01 7.0237661876484e-01 3.3312634768430e-01
3.2725799572631e-01 7.0507506842529e-01 3.2177071852986e-01
9.8571106722073e-01 7.0777358896104e-01 3.1041526533034e-01
6.4435643341515e-01 7.1047216826490e-01 2.9905995623721e-01
3.0318056942376e-01 7.1317079919376e-01 2.8770476128107e-01
9.6217152247544e-01 7.1586947868713e-01 2.7634965214048e-01
6.2131872539452e-01 7.1856820705128e-01 2.6499460186948e-01
2.8061283035078e-01 7.2126698737990e-01 2.5363958461541e-01
9.4004556128454e-01 7.2396582508566e-01 2.4228457534755e-01
5.9960958324407e-01 7.2666472752129e-01 2.3092954960911e-01
2.5929838688380e-01 7.2936370367198e-01 2.1957448329964e-01
9.1910618647806e-01 7.3206276390480e-01 2.0821935249171e-01
5.7902782995628e-01 7.3476191976301e-01 1.9686413328320e-01
2.3905871963191e-01 7.3746118379566e-01 1.8550880168553e-01
8.9919474246044e-01 7.4016056941450e-01 1.7415333354698e-01
5.5943220881523e-01 7.4286009077168e-01 1.6279770451018e-01
2.1976779890554e-01 7.4555976265279e-01 1.5144189000245e-01
8.8019851608025e-01 7.4825960038054e-01 1.4008586525755e-01
5.4072164636162e-01 7.5095961972531e-01 1.2872960536737e-01
2.0133472363708e-01 7.5365983681906e-01 1.1737308536206e-01
8.6203550000451e-01 7.5636026806999e-01 1.0601628031686e-01
5.2282192081968e-01 7.5906093007530e-01 9.4659165483689e-02
1.8369210403445e-01 7.6176183953037e-01 8.3301716445498e-02
8.4464432344311e-01 7.6446301313280e-01 7.1943909290769e-02
5.0567699547354e-01 7.6716446748022e-01 6.0585720805472e-02
1.6678866917213e-01 7.6986621896157e-01 4.9227128679204e-02
8.2797801903759e-01 7.7256828364156e-01 3.7868111721931e-02
4.8924384036279e-01 7.7527067713902e-01 2.6508650087333e-02
1.5058504674586e-01 7.7797341449986e-01 1.5148725498447e-02
8.1200066943565e-01 7.8067651006629e-01 3.7883214710429e-03
4.7237104215632e-01 7.8338311370826e-01 9.9242372515621e-01
1.3392201446710e-01 7.8608703379463e-01 9.8106224365798e-01
7.9554500092481e-01 7.8879134790672e-01 9.6970024760769e-01
4.5723952250828e-01 7.9149606625321e-01 9.5833772971945e-01
1.1900523114076e-01 7.9420119770153e-01 9.4697468541947e-01
7.8084191378163e-01 7.9690674967171e-01 9.3561111296044e-01
4.4274949653208e-01 7.9961272804383e-01 9.2424701350488e-01
1.0472804864861e-01 8.0231913708135e-01 9.1288239117513e-01
7.6677778641388e-01 8.0502597937303e-01 9.0151725306885e-01
4.2889907687034e-01 8.0773325579506e-01 8.9015160923929e-01
9.1092441478580e-02 8.1044096549566e-01 8.7878547264068e-01
7.5335855981517e-01 8.1314910590311e-01 8.6741885903965e-01
4.1569827347292e-01 8.1585767275878e-01 8.5605178689452e-01
7.8112590366638e-02 8.1856666017561e-01 8.4468427720453e-01
7.4060268967817e-01 8.2127606072273e-01 8.3331635333199e-01
4.0316992769366e-01 8.2398586553634e-01 8.2194804080025e-01
6.5815844791501e-02 8.2669606445690e-01 8.1057936707104e-01
7.2854217382989e-01 8.2940664619208e-01 7.9921036130438e-01
3.9135085015731e-01 8.3211759850511e-01 7.8784105410445e-01
5.4244023424737e-02 8.3482890842753e-01 7.7647147725455e-01
7.1722407131506e-01 8.3754056249520e-01 7.6510166344370e-01
3.8029361521981e-01 8.4025254700598e-01 7.5373164598713e-01
4.3455537783916e-02 8.4296484829714e-01 7.4236145854200e-01
7.0671300210359e-01 8.4567745303995e-01 7.3099113481919e-01
3.7006947219519e-01 8.4839034854795e-01 7.1962070829064e-01
3.3528734150943e-02 8.5110352309477e-01 7.0825021189120e-01
6.9709491715326e-01 8.5381696623575e-01 6.9687967771265e-01
3.6077251322634e-01 8.5653066912590e-01 6.8550913668664e-01
2.4566394254176e-02 8.5924462482445e-01 6.7413861825282e-01
6.8848182437961e-01 8.6195882857337e-01 6.6276815000772e-01
3.5252446541359e-01 8.6467327803278e-01 6.5139775733078e-01
1.6700372326183e-02 8.6738797345174e-01 6.4002746298458e-01
6.8101597526751e-01 8.7010291774643e-01 6.2865728668984e-01
3.4547804390265e-01 8.7281811645004e-01 6.1728724467985e-01
1.0093629186040e-02 8.7553357749062e-01 6.0591734924690e-01
6.7486997713487e-01 8.7824931074360e-01 5.9454760830352e-01
3.3981440882265e-01 8.8096532729681e-01 5.8317802499632e-01
4.9341609030015e-03 8.8368163835931e-01 5.7180859742828e-01
6.7023618175898e-01 8.8639825374300e-01 5.6043931856792e-01
3.3572688018488e-01 8.8911517985259e-01 5.4907017644768e-01
1.4118266966734e-03 8.9183241713927e-01 5.3770115477649e-01
6.6729541242691e-01 8.9454995701205e-01 5.2633223410700e-01
3.3338047717938e-01 8.9726777826378e-01 5.1496339369764e-01
5.0507108496585e-01 6.6790158604453e-01 5.0349351684032e-01
1.5608803928899e-01 6.7059063350012e-01 4.9213312778337e-01
8.0786224795467e-01 6.7328086218688e-01 4.8077267408263e-01
4.6038204860049e-01 6.7597242644310e-01 4.6941230758899e-01
1.1361496552551e-01 6.7866530566586e-01 4.5805218639335e-01
7.6751560413877e-01 6.8135937883526e-01 4.4669243766026e-01
4.2203219552946e-01 6.8405447990203e-01 4.3533314480875e-01
7.7111344190636e-02 6.8675043348245e-01 4.2397434916557e-01
7.3270111038488e-01 6.8944707491430e-01 4.1261605813370e-01
3.8875279293307e-01 6.9214425963969e-01 4.0125825490537e-01
4.5221802533380e-02 6.9484186609503e-01 3.8990090721923e-01
7.0206794900729e-01 6.9753979506869e-01 3.7854397424334e-01
3.5925537841825e-01 7.0023796740296e-01 3.6718741150520e-01
1.6752318820735e-02 7.0293632112849e-01 3.5583117415283e-01
6.7453073524151e-01 7.0563480860644e-01 3.4447521893093e-01
3.3256595436787e-01 7.0833339394633e-01 3.3311950523502e-01
9.9083629327441e-01 7.1103205079531e-01 3.2176399554015e-01
6.4932271020385e-01 7.1373076050588e-01 3.1040865542845e-01
3.0800848560943e-01 7.1642951064752e-01 2.9905345337586e-01
9.6687893599832e-01 7.1912829381324e-01 2.8769836040802e-01
6.2592116000477e-01 7.2182710667014e-01 2.7634334969844e-01
2.8512381454622e-01 7.2452594920739e-01 2.6498839615509e-01
9.4447691823893e-01 7.2722482414197e-01 2.5363347602375e-01
6.0397167908005e-01 7.2992373644925e-01 2.4227856652427e-01
2.6360034350497e-01 7.3262269299189e-01 2.3092364552780e-01
9.2335606416494e-01 7.3532170222583e-01 2.1956869127864e-01
5.8323278405907e-01 7.3802077396659e-01 2.0821368216085e-01
2.4322513495361e-01 7.4071991920260e-01 1.9685859650883e-01
9.0332834830587e-01 7.4341914994485e-01 1.8550341245959e-01
5.6353817716911e-01 7.4611847910448e-01 1.7414810784478e-01
2.2385082778361e-01 7.4881792039115e-01 1.6279266012027e-01
8.8426289975775e-01 7.5151748822663e-01 1.5143704633113e-01
5.4477133391168e-01 7.5421719766853e-01 1.4008124311038e-01
2.0537336699819e-01 7.5691706434018e-01 1.2872522670983e-01
8.6606649263226e-01 7.5961710436298e-01 1.1736897306154e-01
5.2684842785548e-01 7.6231733428800e-01 1.0601245786842e-01
1.8771708483595e-01 7.6501777102428e-01 9.4655656722462e-02
8.4867054726134e-01 7.6771843176144e-01 8.3298545248913e-02
5.0970705102394e-01 7.7041933388479e-01 7.1941099274403e-02
1.7082496882476e-01 7.7312049488151e-01 6.0583295016653e-02
8.3202279834047e-01 7.7582193223709e-01 4.9225109292982e-02
4.9329915360557e-01 7.7852366332155e-01 3.7866519744350e-02
1.5465275926428e-01 7.8122570526579e-01 2.6507505071145e-02
8.1608244734521e-01 7.8392807482861e-01 1.5148045276483e-02
4.7758715620972e-01 7.8663078825594e-01 3.7881219123414e-03
1.3806900730957e-01 7.8933693743789e-01 9.9242409041976e-01
7.9970972460698e-01 7.9204045458109e-01 9.8106311292556e-01
4.6142271427350e-01 7.9474435810824e-01 9.6970162968834e-01
1.2320736100512e-01 7.9744866071714e-01 9.5833963079948e-01
7.8506317116837e-01 8.0015337384911e-01 9.4697710899319e-01
4.4698977778200e-01 8.0285850755317e-01 9.3561405980281e-01
1.0898694553520e-01 8.0556407036190e-01 9.2425048168582e-01
7.7105457567155e-01 8.0827006918268e-01 9.1288637611451e-01
4.3319271062424e-01 8.1097650920701e-01 9.0152174763002e-01
9.5401538347884e-02 8.1368339384087e-01 8.9015660385799e-01
7.5768139635421e-01 8.1639072465848e-01 8.7879095548532e-01
4.2003277551950e-01 8.1909850138145e-01 8.6742481619829e-01
8.2456323788334e-02 8.2180672188503e-01 8.5605820258310e-01
7.4495284994811e-01 8.2451538223263e-01 8.4469113399063e-01
4.0752332768943e-01 8.2722447673941e-01 8.3332363236801e-01
7.0168900195866e-02 8.2993399806525e-01 8.2195572205990e-01
7.3289088552092e-01 8.3264393733731e-01 8.1058742958307e-01
3.9569078300821e-01 8.3535428430149e-01 7.9921878337769e-01
5.8570280990658e-02 8.3806502750225e-01 7.8784981353919e-01
7.2153126596521e-01 8.4077615448961e-01 7.7648055153428e-01
3.8457583337844e-01 8.4348765205152e-01 7.6511102990441e-01
4.7706300075087e-02 8.4619950646988e-01 7.5374128195975e-01
7.1092521835368e-01 8.4891170379729e-01 7.4237134146603e-01
3.7423539143872e-01 8.5162423015149e-01 7.3100124232600e-01
3.7639890015923e-02 8.5433707202320e-01 7.1963101825655e-01
7.0114206928178e-01 8.5705021659242e-01 7.0826070246146e-01
3.6474558572786e-01 8.5976365204666e-01 6.9689032729937e-01
2.8454412610138e-02 8.6247736789303e-01 6.8551992394527e-01
6.9227285273959e-01 8.6519135525404e-01 6.7414952204373e-01
3.5620554686869e-01 8.6790560713415e-01 6.6277914935158e-01
2.0257475538927e-02 8.7062011864091e-01 6.5140883136853e-01
6.8443395179890e-01 8.7333488714023e-01 6.4003859095506e-01
3.4874060171412e-01 8.7604991232035e-01 6.2866844794009e-01
1.3183329091827e-02 8.7876519613340e-01 6.1729841872443e-01
6.7776826038304e-01 8.8148074257677e-01 6.0592851589312e-01
3.4250166537285e-01 8.8419655727007e-01 5.9455874785792e-01
7.3898491392997e-03 8.8691264677761e-01 5.8318911856357e-01
6.7243901100666e-01 8.8962901762241e-01 5.7181962730565e-01
3.3765506703918e-01 8.9234567493825e-01 5.6045026872504e-01
3.0434342516021e-03 8.9506262071379e-01 5.4908103306140e-01
6.6860877738670e-01 8.9777985160094e-01 5.3771190676419e-01
3.3435472300238e-01 9.0049735629166e-01 5.2634287356878e-01
2.8355073503960e-04 9.0321511251772e-01 5.1497391614226e-01
1.7200808703351e-01 6.7385740910102e-01 5.0348732482144e-01
8.2223385291409e-01 6.7654562331761e-01 4.9212672040900e-01
4.7334130451591e-01 6.7923525494647e-01 4.8076602393320e-01
1.2531309021636e-01 6.8192647447546e-01 4.6940542831021e-01
7.7810506438733e-01 6.8461922908260e-01 4.5804512729579e-01
4.3165753458825e-01 6.8731334699392e-01 4.4668526779734e-01
8.5904181703604e-02 6.9000861102934e-01 4.3532593725610e-01
7.4077816897907e-01 6.9270480271319e-01 4.2396716994433e-01
3.9621582444612e-01 6.9540172462947e-01 4.1260896031157e-01
5.2158553767397e-02 6.9809920904184e-01 4.0125127673943e-01
7.0855359845970e-01 7.0079711888957e-01 3.8989407287246e-01
3.6535410574975e-01 7.0349534512152e-01 3.7853729584796e-01
2.2518825018174e-02 7.0619380266243e-01 3.6718089172918e-01
6.8001162766629e-01 7.0889242621018e-01 3.5582480876465e-01
3.3780096588505e-01 7.1159116641324e-01 3.4446899910247e-01
9.9585933382904e-01 7.1428998662438e-01 3.3311341947821e-01
6.5416276320166e-01 7.1698886025000e-01 3.2175803126278e-01
3.1269036699725e-01 7.1968776863682e-01 3.1040280013986e-01
9.7142393503652e-01 7.2238669941170e-01 2.9904769559229e-01
6.3034757966307e-01 7.2508564518914e-01 2.8769269031132e-01
2.8944742748852e-01 7.2778460256999e-01 2.7633775959781e-01
9.4871135214673e-01 7.3048357136801e-01 2.6498288079489e-01
6.0812874291861e-01 7.3318255401292e-01 2.5362803277238e-01
2.6769030440229e-01 7.3588155508955e-01 2.4227319547173e-01
9.2738788289606e-01 7.3858058098212e-01 2.3091834951311e-01
5.8721431570678e-01 7.4127963959920e-01 2.1956347586287e-01
2.4716330012902e-01 7.4397874016097e-01 2.0820855555728e-01
9.0722927932973e-01 7.4667789303440e-01 1.9685356947818e-01
5.6740734280751e-01 7.4937710960506e-01 1.8549849817609e-01
2.2769313947193e-01 7.5207640217682e-01 1.7414332173671e-01
8.8808280171105e-01 7.5477578389229e-01 1.6278801968737e-01
5.4857287908809e-01 7.5747526866800e-01 1.5143257094071e-01
2.0916028053686e-01 7.6017487113954e-01 1.4007695377316e-01
8.6984222411599e-01 7.6287460661216e-01 1.2872114583669e-01
5.3061619353829e-01 7.6557449101323e-01 1.1736512420231e-01
1.9147990081807e-01 7.6827454084303e-01 1.0600886543420e-01
8.5243125447988e-01 7.7097477312093e-01 9.4652345693380e-02
5.1346833284944e-01 7.7367520532433e-01 8.3295540869853e-02
1.7458936200499e-01 7.7637585531805e-01 7.1938426741646e-02
8.3579269800685e-01 7.7907674127245e-01 6.0580979159199e-02
4.9707681304747e-01 7.8177788156878e-01 4.9223174252782e-02
1.5844028517705e-01 7.8447929469114e-01 3.7864988660222e-02
8.1988179126163e-01 7.8718099910468e-01 2.6506399771546e-02
4.8140010282736e-01 7.8988301312062e-01 1.5147385986571e-02
1.4299408443607e-01 7.9258535474895e-01 3.7879269808341e-03
8.0358812489720e-01 7.9529105648975e-01 9.9242444808012e-01
4.6531891212217e-01 7.9799417688712e-01 9.8106396337841e-01
1.2712231880614e-01 8.0069767385171e-01 9.6970298428840e-01
7.8899760344045e-01 8.0340156254086e-01 9.5834149823238e-01
4.5094413460779e-01 8.0610585697124e-01 9.4697949517286e-01
1.1296139693212e-01 8.0881056985286e-01 9.3561696781231e-01
7.7504899713700e-01 8.1151571243274e-01 9.2425391176241e-01
4.3720666997275e-01 8.1422129435197e-01 9.1289032567872e-01
9.9434283828410e-02 8.1692732352029e-01 9.0152621135698e-01
7.6173184590677e-01 8.1963380601140e-01 8.9016157378847e-01
4.2409950690718e-01 8.2234074598245e-01 8.7879642117283e-01
8.6537565229170e-02 8.2504814562029e-01 8.6743076488753e-01
7.4904647077568e-01 8.2775600511675e-01 8.5606461941447e-01
4.1162682849523e-01 8.3046432267483e-01 8.4469800222492e-01
7.4279401854524e-02 8.3317309454681e-01 8.3333093362493e-01
7.3700511647254e-01 8.3588231510518e-01 8.2196343656407e-01
3.9980506417301e-01 8.3859197694639e-01 8.1059553641086e-01
6.2680507719694e-02 8.4130207102719e-01 7.9922726069881e-01
7.2563288648784e-01 8.4401258683266e-01 7.8785863884706e-01
3.8866382329231e-01 8.4672351257456e-01 7.7648970185990e-01
5.1775132537595e-02 8.4943483541816e-01 7.6512048200911e-01
7.1496882977496e-01 8.5214654173489e-01 7.5375101250305e-01
3.7824714264755e-01 8.5485861737793e-01 7.4238132714593e-01
4.1612523072346e-02 8.5757104797663e-01 7.3101145999005e-01
7.0506766034898e-01 8.6028381924510e-01 7.1964144498335e-01
3.6861549469729e-01 8.6299691729919e-01 7.0827131561389e-01
3.2259230506289e-02 8.6571032897477e-01 6.9690110455194e-01
6.9600234832684e-01 8.6842404213857e-01 6.8553084329014e-01
3.5984861435571e-01 8.7113804598124e-01 6.7416056178134e-01
2.3802085842008e-02 8.7385233127957e-01 6.6279028807402e-01
6.8786711049803e-01 8.7656689061226e-01 6.5142004794523e-01
3.5204831762478e-01 8.7928171851003e-01 6.4004986453260e-01
1.6350598278409e-02 8.8199681151707e-01 6.2867975796880e-01
6.8077907143547e-01 8.8471216813622e-01 6.1730974502579e-01
3.4533903279945e-01 8.8742778862574e-01 6.0593983878133e-01
1.0035882693561e-02 8.9014367461059e-01 5.9457004832749e-01
6.7487502949232e-01 8.9285982846789e-01 5.8320037855051e-01
3.3986176537639e-01 8.9557625244425e-01 5.7183083002248e-01
5.0011119994744e-03 8.9829294746477e-01 5.6046139905837e-01
6.7029763509704e-01 9.0100991160128e-01 5.4909207800490e-01
3.3575522929834e-01 9.0372713818353e-01 5.3772285583831e-01
1.3768775268256e-03 9.0644461356390e-01 5.2635371915383e-01
6.6716439336140e-01 9.0916231458606e-01 5.1498465362485e-01
8.3899352695455e-01 6.7981308424919e-01 5.0348155643209e-01
4.8829403664345e-01 6.8250029302204e-01 4.9212072898368e-01
1.3863330951988e-01 6.8518922100350e-01 4.8075977690056e-01
7.8998528283666e-01 6.8788005356011e-01 4.6939894912456e-01
4.4228893858226e-01 6.9057268753474e-01 4.5803848461558e-01
9.5464676861462e-02 6.9326688020815e-01 4.4667855125342e-01
7.4942661365198e-01 6.9596234762015e-01 4.3531923591964e-01
4.0409042044050e-01 6.9865881839032e-01 4.2396055888680e-01
5.9377611930655e-02 7.0135605732267e-01 4.1260249482747e-01
7.1521744099645e-01 7.0405387167163e-01 4.0124499185599e-01
3.7154735973425e-01 7.0675210888035e-01 3.8988798577951e-01
2.8312707759525e-02 7.0945065094555e-01 3.7853140953281e-01
6.8546603719222e-01 7.1214940807463e-01 3.6717519877915e-01
3.4296630864273e-01 7.1484831283644e-01 3.5581929480561e-01
7.7808262222279e-04 7.1754731523072e-01 3.4446364566192e-01
6.5887076633974e-01 7.2024637872876e-01 3.3310820624072e-01
3.1721794000118e-01 7.2294547718257e-01 3.2175293777290e-01
9.7579676784490e-01 7.2564459245227e-01 3.1039780704136e-01
6.3458748990428e-01 7.2834371260269e-01 2.9904278549710e-01
2.9357298651900e-01 7.3104283053976e-01 2.8768784838306e-01
9.5273840641867e-01 7.3374194298208e-01 2.7633297392091e-01
6.1207084938179e-01 7.3644104968580e-01 2.6497814258573e-01
2.7155909526907e-01 7.3914015286024e-01 2.5362333647585e-01
9.3119337224657e-01 7.4183925672716e-01 2.4226853877546e-01
5.9096515804831e-01 7.4453836718842e-01 2.3091373330280e-01
2.5086700908861e-01 7.4723749157566e-01 2.1955890413503e-01
9.1089241308489e-01 7.4993663846224e-01 2.0820403530062e-01
5.7103566158393e-01 7.5263581752268e-01 1.9684911053086e-01
2.3129173940578e-01 7.5533503942809e-01 1.8549411306321e-01
8.9165622854096e-01 7.5803431576897e-01 1.7413902549044e-01
5.5212522447199e-01 7.6073365899809e-01 1.6278382965078e-01
2.1269526325159e-01 7.6343308238801e-01 1.5142850655549e-01
8.7336325796976e-01 7.6613259999804e-01 1.4007303635117e-01
5.3412644348833e-01 7.6883222664653e-01 1.2871739831501e-01
1.9498232852406e-01 7.7153197788453e-01 1.1736157088164e-01
8.5592865432504e-01 7.7423186996736e-01 1.0600553170108e-01
5.1696335931616e-01 7.7693191982078e-01 9.4649257726977e-02
1.7808454919110e-01 7.7963214499874e-01 8.3292725334732e-02
8.3929047200574e-01 7.8233256363017e-01 7.1935910468923e-02
5.0057949788162e-01 7.8503319435236e-01 6.0578788818967e-02
1.6195010296403e-01 7.8773405622910e-01 4.9221336021631e-02
8.2340085729761e-01 7.9043516865242e-01 3.7863527888318e-02
4.8493041628783e-01 7.9313655122694e-01 2.6505340654371e-02
1.4653751541131e-01 7.9583822363697e-01 1.5146751246888e-02
8.0822096782591e-01 7.9854020549696e-01 3.7877375667393e-03
4.6892791264164e-01 8.0124546848507e-01 9.9242479643549e-01
1.3074909413264e-01 8.0394819996350e-01 9.8106479086753e-01
7.9264332944663e-01 8.0665129595626e-01 9.6970430496986e-01
4.5460976368119e-01 8.0935477403164e-01 9.5834332345849e-01
1.1664763793045e-01 8.1205865076564e-01 9.4698183345084e-01
7.7875629626208e-01 8.1476294154632e-01 9.3561982471029e-01
4.4093519293012e-01 8.1746766038454e-01 9.2425728986224e-01
1.0318389951218e-01 8.2017281973559e-01 9.1289422457586e-01
7.6550211171200e-01 8.2287843033667e-01 9.0153062770682e-01
4.2788965563085e-01 8.2558450106424e-01 8.9016650139703e-01
9.0346493381431e-02 8.2829103881563e-01 8.7880185112845e-01
7.5287272799299e-01 8.3099804841822e-01 8.6743668572934e-01
4.1546860763116e-01 8.3370553256941e-01 8.5607101733230e-01
7.8134529228687e-02 8.3641349180971e-01 8.4470486128462e-01
7.4087104168753e-01 8.3912192453073e-01 8.3333823601264e-01
4.0367884886655e-01 8.4183082701931e-01 8.2197116284250e-01
6.6558812607623e-02 8.4454019353819e-01 8.1060366578080e-01
7.2951195607393e-01 8.4725001644296e-01 7.9923577125896e-01
3.9253946767374e-01 8.4996028633476e-01 7.8786750784584e-01
5.5642705820518e-02 8.5267099224688e-01 7.7649890593329e-01
7.1882320473219e-01 8.5538212186354e-01 7.6512999739944e-01
3.8208268139930e-01 8.5809366176762e-01 7.5376081525442e-01
4.5423043751299e-02 8.6080559771407e-01 7.4239139327301e-01
7.0884639992310e-01 8.6351791492442e-01 7.3102176561836e-01
3.7235506836922e-01 8.6623059839716e-01 7.1965196646018e-01
3.5951588390980e-02 8.6894363322745e-01 7.0828202959071e-01
6.9963873043310e-01 8.7165700492863e-01 6.9691198804054e-01
3.6341950527100e-01 8.7437069974634e-01 6.8554187369636e-01
2.7297170950107e-02 8.7708470495446e-01 6.7417171692212e-01
6.9127523605603e-01 8.7979900911990e-01 6.6280154618480e-01
3.5535745759535e-01 8.8251360232117e-01 6.5143138768677e-01
1.9547831458808e-02 8.8522847630255e-01 6.4006126500729e-01
6.8385057313688e-01 8.8794362454288e-01 6.2869119875784e-01
3.4827008608560e-01 8.9065904221464e-01 6.1732120625883e-01
1.2810914933277e-02 8.9337472600526e-01 6.0595130124971e-01
6.7747768062832e-01 8.9609067377024e-01 5.9458149365025e-01
3.4227499472479e-01 8.9880688398475e-01 5.8321178939816e-01
7.2073503765384e-03 9.0152335496103e-01 5.7184219039734e-01
6.7227898837056e-01 9.0424008380120e-01 5.6047269462029e-01
3.3749373779024e-01 9.0695706506307e-01 5.4910329641803e-01
2.8548327598869e-03 9.0967428913070e-01 5.3773398709798e-01
6.6836470923596e-01 9.1239174030361e-01 5.2636475583311e-01
3.3402478893250e-01 9.1510939465083e-01 5.1499559096072e-01
5.0604106744549e-01 6.8576862415770e-01 5.0347622267340e-01
1.5424558570268e-01 6.8845460728879e-01 4.9211516423860e-01
8.0369123396876e-01 6.9114270296471e-01 4.8075394280308e-01
4.5433826126330e-01 6.9383310628076e-01 4.6939288535197e-01
1.0610067762270e-01 6.9652563488462e-01 4.5803228806802e-01
7.5887066139467e-01 6.9921994603546e-01 4.4667233885813e-01
4.1253568485838e-01 7.0191566853162e-01 4.3531311540381e-01
6.6988473078183e-02 7.0461246669955e-01 4.2395461329953e-01
7.2213175844658e-01 7.0731006275510e-01 4.1259677814127e-01
3.7787984866650e-01 7.1000823803058e-01 4.0123953115218e-01
3.4158481296400e-02 7.1270682548216e-01 3.8988278637822e-01
6.9090387902110e-01 7.1540569987978e-01 3.7852646080364e-01
3.4806151651766e-01 7.1810476851521e-01 3.6717047947326e-01
5.5848599943886e-03 7.2080396341822e-01 3.5581477745177e-01
6.6343419901955e-01 7.2350323522410e-01 3.4445929995119e-01
3.2157561527948e-01 7.2620254850306e-01 3.3310400150190e-01
9.7998009567597e-01 7.2890187827068e-01 3.2174884470312e-01
6.3862278027731e-01 7.3160120740820e-01 3.1039379886071e-01
2.9748232921232e-01 7.3430052476513e-01 2.9903883867561e-01
9.5654039129451e-01 7.3699982376633e-01 2.8768394306022e-01
6.1578115817711e-01 7.3969910138884e-01 2.7632909411026e-01
2.7519098974780e-01 7.4239835740952e-01 2.6497427623315e-01
9.3475809858145e-01 7.4509759385077e-01 2.5361947542095e-01
5.9447228326872e-01 7.4779681457205e-01 2.4226467865056e-01
2.5432470220632e-01 7.5049602496904e-01 2.3090987339295e-01
9.1430768094079e-01 7.5319523175313e-01 2.1955504721422e-01
5.7441454741644e-01 7.5589444279109e-01 2.0820018745344e-01
2.3463949051647e-01 7.5859366699017e-01 1.9684528096485e-01
8.9497743813905e-01 7.6129291421783e-01 1.8549031391397e-01
5.5542395174760e-01 7.6399219524750e-01 1.7413527161997e-01
2.1597513490535e-01 7.6669152172382e-01 1.6278013843804e-01
8.7662755377051e-01 7.6939090614198e-01 1.5142489767762e-01
5.3737816791066e-01 7.7209036183649e-01 1.4006953155350e-01
1.9822427010657e-01 7.7478990297514e-01 1.2871402116799e-01
8.5916343407099e-01 7.7748954455452e-01 1.1735834652326e-01
5.2019346921377e-01 7.8018930239329e-01 1.0600248656350e-01
1.8131238175088e-01 7.8288919311988e-01 9.4646419247148e-02
8.4251834158498e-01 7.8558923415130e-01 8.3290121649333e-02
5.0380965448537e-01 7.8828944365985e-01 7.1933570094796e-02
1.6518473916770e-01 7.9098984052519e-01 6.0576740321294e-02
8.2664210892295e-01 7.9369044426904e-01 4.9219607672952e-02
4.8818035747328e-01 7.9639127497079e-01 3.7862147322425e-02
1.4979814874257e-01 7.9909235316251e-01 2.6504334519969e-02
8.1149421022550e-01 8.0179369970278e-01 1.5146144866606e-02
4.7326732962354e-01 8.0449533562945e-01 3.7875546076140e-03
1.3408788286434e-01 8.0720017034345e-01 9.9242513369114e-01
7.9599976613210e-01 8.0990252248411e-01 9.8106559101792e-01
4.5798522225280e-01 8.1260522479051e-01 9.6970558494708e-01
1.2004329874542e-01 8.1530829717124e-01 9.5834509746428e-01
7.8217312229287e-01 8.1801175872532e-01 9.4698411277858e-01
4.4437390677634e-01 8.2071562751790e-01 9.3562261761283e-01
1.0664496173544e-01 8.2341992035800e-01 9.2426060146849e-01
7.6898570086909e-01 8.2612465258385e-01 9.1289805685776e-01
4.3139565023825e-01 8.2882983786119e-01 9.0153497949688e-01
9.3874455891814e-02 8.3153548799998e-01 8.9017136845511e-01
7.5642189070866e-01 8.3424161279443e-01 8.7880722625517e-01
4.1903786032908e-01 8.3694821989100e-01 8.6744255892211e-01
8.1722408133192e-02 8.3965531468826e-01 8.5607737597886e-01
7.4447571930789e-01 8.4236290027194e-01 8.4471169038804e-01
4.0729812412293e-01 8.4507097738770e-01 8.3334551844103e-01
7.0190100605707e-02 8.4777954445326e-01 8.2197887959618e-01
7.3315227685918e-01 8.5048859761108e-01 8.1061179626944e-01
3.9618543330353e-01 8.5319813082141e-01 7.9924429358142e-01
5.9290505136738e-02 8.5590813599546e-01 7.8787639906547e-01
7.2246858529906e-01 8.5861860316677e-01 7.7650814234219e-01
3.8572092818969e-01 8.6132952069904e-01 7.6513955476572e-01
4.9048954318935e-02 8.6404087552694e-01 7.5377066904761e-01
7.1245425598576e-01 8.6675265342614e-01 7.4240151886373e-01
3.7593860394874e-01 8.6946483930776e-01 7.3103213844955e-01
3.9503954907993e-02 8.7217741753118e-01 7.1966256218886e-01
7.0315245943962e-01 8.7489037222852e-01 7.0829282420021e-01
3.6688646982223e-01 8.7760368763232e-01 6.9692295792525e-01
3.0708546972417e-02 8.8031734839708e-01 6.8555299572230e-01
6.9462146546296e-01 8.8303133990336e-01 6.7418296846824e-01
3.5862821534300e-01 8.8574564853166e-01 6.6281290517180e-01
2.2731999216023e-02 8.8846026189113e-01 6.5144283260122e-01
6.8693622277319e-01 8.9117516898604e-01 6.4007277493033e-01
3.5124447673856e-01 8.9389036030069e-01 6.2870275340836e-01
1.5660507982372e-02 8.9660582778072e-01 6.1733278606122e-01
6.8018817742853e-01 8.9932156468697e-01 6.0596288743544e-01
3.4483140236547e-01 9.0203756529560e-01 5.9459306840058e-01
9.5940809208254e-03 9.0475382441791e-01 5.8322333603168e-01
6.7447999684793e-01 9.0747033671374e-01 5.7185369360006e-01
3.3949270347459e-01 9.1018709577593e-01 5.6048414070810e-01
4.6353867605240e-03 9.1290409297064e-01 5.4911467361023e-01
6.6991070896273e-01 9.1562131603069e-01 5.3774528576741e-01
3.3532063644119e-01 9.1833874741767e-01 5.2637596868341e-01
8.6625759865860e-04 9.2105636249442e-01 5.1500671306612e-01
1.7317012697910e-01 6.9172404953364e-01 5.0347133365726e-01
8.2005632007368e-01 6.9440851970294e-01 4.9211003575879e-01
4.6845064712144e-01 6.9709562712449e-01 4.8074853045990e-01
1.1829117175738e-01 6.9978556323141e-01 4.6938725594186e-01
7.6945394639567e-01 7.0247801968795e-01 4.5802658003421e-01
4.2179034781572e-01 7.0517251166727e-01 4.4666670477178e-01
7.5151090330835e-02 7.0786855446677e-01 4.3530768313095e-01
7.2939857349302e-01 7.1056573573003e-01 4.2394947000641e-01
3.8441164527322e-01 7.1326373144122e-01 4.1259196973282e-01
4.0086245862897e-02 7.1596229779703e-01 4.0123506920175e-01
6.9633414457119e-01 7.1866125577168e-01 3.8987865727204e-01
3.5308098383721e-01 7.2136047570952e-01 3.7852263437547e-01
1.0264225647798e-02 7.2405986446240e-01 3.6716691606168e-01
6.6783125785514e-01 7.2675935547809e-01 3.5581143319174e-01
3.2573774853224e-01 7.2945890147202e-01 3.4445613049812e-01
9.8394625813488e-01 7.3215846912770e-01 3.3310096450820e-01
6.4242508686247e-01 7.3485803530676e-01 3.2174590136561e-01
3.0114732422093e-01 7.3755758435127e-01 3.1039091480769e-01
9.6009006722567e-01 7.4025710616437e-01 2.9903598440298e-01
6.1923377703010e-01 7.4295659484195e-01 2.8768109407090e-01
2.7856174827049e-01 7.4565604769385e-01 2.7632623086676e-01
9.3805966988385e-01 7.4835546454104e-01 2.6497138399889e-01
5.9771526011818e-01 7.5105484720938e-01 2.5361654404069e-01
2.5751796179506e-01 7.5375419916478e-01 2.4226170230256e-01
9.1745868662271e-01 7.5645352525075e-01 2.3090685033261e-01
5.7752959957129e-01 7.5915283150129e-01 2.1955197952037e-01
2.3772393609780e-01 7.6185212500976e-01 2.0819708078274e-01
8.9803584642836e-01 7.6455141383995e-01 1.9684214431552e-01
5.5846026224143e-01 7.6725070696931e-01 1.8548715939801e-01
2.1899278200575e-01 7.6995001425654e-01 1.7413211424107e-01
8.7962957195764e-01 7.7264934642792e-01 1.6277699587186e-01
5.4036728029181e-01 7.7534871507727e-01 1.5142179005051e-01
2.0120296261622e-01 7.7804813267535e-01 1.4006648121587e-01
8.6213401710821e-01 7.8074761258503e-01 1.2871105245859e-01
5.2315812812250e-01 7.8344716907827e-01 1.1735548552128e-01
1.8427321725615e-01 7.8614681735158e-01 1.0599976082594e-01
8.4547740107955e-01 7.8884657353618e-01 9.4643857529572e-02
5.0676895490548e-01 7.9154645469939e-01 8.3287753609192e-02
1.6814628209414e-01 7.9424647883385e-01 7.1931425977442e-02
8.2960788848642e-01 7.9694666483104e-01 6.0574850629843e-02
4.9115236162329e-01 7.9964703243628e-01 4.9218002824320e-02
1.5277835445012e-01 8.0234760218243e-01 3.7860857292973e-02
8.1448457322262e-01 8.0504839530024e-01 2.6503388485256e-02
4.7626976933098e-01 8.0774943360385e-01 1.5145570840710e-02
1.3813273474194e-01 8.1045073935097e-01 3.7873790883044e-03
7.9906757254256e-01 8.1315515818476e-01 9.9242545795705e-01
4.6107042977833e-01 8.1585714245687e-01 9.8106635921711e-01
1.2314745976895e-01 8.1855946019644e-01 9.6970681705986e-01
7.8529763157570e-01 8.2126213355642e-01 9.5834681073463e-01
4.4751997363068e-01 8.2396518409875e-01 9.4698632149386e-01
1.0981358281929e-01 8.2666863254309e-01 9.3562533292956e-01
7.7217763424791e-01 8.2937249851247e-01 9.2426383128840e-01
4.3461139123857e-01 8.3207680028178e-01 9.1290180575721e-01
9.7114215128876e-02 8.3478155453559e-01 9.0153924871085e-01
7.5968557450901e-01 8.3748677614152e-01 8.9017615591565e-01
4.2232505359827e-01 8.4019247794534e-01 8.7881252668235e-01
8.5032359546882e-02 8.4289867059341e-01 8.6744836396419e-01
7.4780732854109e-01 8.4560536238760e-01 8.5608367439670e-01
4.1064993068553e-01 8.4831255917688e-01 8.4471846827790e-01
7.3560273731115e-02 8.5102026428924e-01 8.3335275948871e-01
7.3653860580284e-01 8.5372847850629e-01 8.2198656535487e-01
3.9958531735595e-01 8.5643720008232e-01 8.1061990645332e-01
6.2700942643328e-02 8.5914642480832e-01 7.9925280636678e-01
7.2588616101020e-01 8.6185614612081e-01 7.8788529139160e-01
3.8914179833789e-01 8.6456635525400e-01 7.7651739020437e-01
5.2468828936209e-02 8.6727704143317e-01 7.6514913349376e-01
7.1586837813038e-01 8.6998819210609e-01 7.5378055356402e-01
3.7934172570431e-01 8.7269979320809e-01 7.4241168391685e-01
4.2890310246193e-02 8.7541182945578e-01 7.3104255881829e-01
7.0651573429802e-01 8.7812428466290e-01 7.1967321285700e-01
3.7021977003790e-01 8.8083714207104e-01 7.0830368049995e-01
3.4004365028456e-02 8.8355038468630e-01 6.9693399565113e-01
6.9787164734846e-01 8.8626399561207e-01 6.8556419121824e-01
3.6182392919500e-01 8.8897795836633e-01 6.7419429869228e-01
2.5863707805813e-02 8.9169225717052e-01 6.6282434774422e-01
6.8999366231521e-01 8.9440687719526e-01 6.5145436584350e-01
3.5421664493289e-01 8.9712180474667e-01 6.4008437790308e-01
1.8535664638074e-02 8.9983702737502e-01 6.2871440595723e-01
6.8295386143218e-01 9.0255253388591e-01 6.1734446887963e-01
3.4747446911663e-01 9.0526831423287e-01 6.0597458215219e-01
1.2100764594061e-02 9.0798435926913e-01 5.9460475769853e-01
6.7683600187366e-01 9.1070066033642e-01 5.8323500380016e-01
3.4168332934491e-01 9.1341720867046e-01 5.7186532511889e-01
6.6456895253002e-03 9.1613399460607e-01 5.6049572285393e-01
6.7172570143958e-01 9.1885100657218e-01 5.4912619506708e-01
3.3692552708868e-01 9.2156822987740e-01 5.3775673721266e-01
2.2467251208639e-03 9.2428564530274e-01 5.2638734290862e-01
6.6769009677394e-01 9.2700322753863e-01 5.1501800498063e-01
8.4040925745650e-01 6.9767939363429e-01 5.0346689843080e-01
4.8567982959694e-01 7.0036196668557e-01 4.9210535164051e-01
1.3282096310452e-01 7.0304789657397e-01 4.8074354805028e-01
7.8173340084319e-01 7.0573734012728e-01 4.6938208815131e-01
4.3223371432434e-01 7.0842978568796e-01 4.5802142669656e-01
8.4113024881448e-02 7.1112454553422e-01 4.4666176339129e-01
7.3717067649566e-01 7.1382098900511e-01 4.3530309965429e-01
3.9122874371722e-01 7.1651861526458e-01 4.2394532647193e-01
4.6135674643872e-02 7.1921705323859e-01 4.1258829201953e-01
7.0176490975179e-01 7.2191603756730e-01 4.0123184174774e-01
3.5801167647028e-01 7.2461538178517e-01 3.8987583776937e-01
1.4789463620624e-02 7.2731495585770e-01 3.7852016571341e-01
6.7202678498570e-01 7.3001466931646e-01 3.6716473500921e-01
3.2966442056427e-01 7.3271445923692e-01 3.5580947622773e-01
9.8765314530126e-01 7.3541428188621e-01 3.4445433745150e-01
6.4595188954287e-01 7.3811410698808e-01 3.3309928064046e-01
3.0452625924022e-01 7.4081391379935e-01 3.2174427840483e-01
9.6334734727966e-01 7.4351368842163e-01 3.1038931131035e-01
6.2239077711030e-01 7.4621342195131e-01 2.9903436570690e-01
2.8163593061222e-01 7.4891310919817e-01 2.8767943201651e-01
9.4106532187220e-01 7.5161274779173e-01 2.7632450340157e-01
6.0066408664133e-01 7.5431233755377e-01 2.6496957473731e-01
2.6041956375900e-01 7.5701188005565e-01 2.5361464182203e-01
9.2032094995571e-01 7.5971137830552e-01 2.4225970077100e-01
5.8035901345141e-01 7.6241083652827e-01 2.3090474755044e-01
2.4052585488024e-01 7.6511026001299e-01 2.1954977761822e-01
9.0081470649621e-01 7.6780965501038e-01 2.0819478564544e-01
5.6121976250327e-01 7.7050902866812e-01 1.9683976529965e-01
2.2173603483172e-01 7.7320838899529e-01 1.8548470907545e-01
8.8235922984466e-01 7.7590774484954e-01 1.7412960816233e-01
5.4308564237686e-01 7.7860710594194e-01 1.6277445234261e-01
2.0391206423809e-01 7.8130648285532e-01 1.5141922991489e-01
8.6483570489660e-01 7.8400588707257e-01 1.4006392764055e-01
5.2585412252714e-01 7.8670533101134e-01 1.2870853071202e-01
1.8696516398612e-01 7.8940482806166e-01 1.1735302274327e-01
8.4816691258233e-01 7.9210439262317e-01 1.0599738578340e-01
5.0945764275761e-01 7.9480404013810e-01 9.4641600355284e-02
1.7083578098928e-01 7.9750378711639e-01 8.3285645521311e-02
8.3229987238093e-01 8.0020365114908e-01 7.1929498978650e-02
4.9384855252642e-01 8.0290365090614e-01 6.0573137186233e-02
1.5548052431743e-01 8.0560380611506e-01 4.9216535525362e-02
8.1719453942093e-01 8.0830413751688e-01 3.7859668495218e-02
4.7898938418206e-01 8.1100466679652e-01 2.6502509943706e-02
1.4086386971440e-01 8.1370541648542e-01 1.5145033333038e-02
8.0281682592412e-01 8.1640640983455e-01 3.7872120381711e-03
4.6386657744539e-01 8.1911042722205e-01 9.9242576725063e-01
1.2596062479570e-01 8.2181205712089e-01 9.8106709061497e-01
7.8812952048915e-01 8.2451400139565e-01 9.6970799376272e-01
4.5037217787561e-01 8.2721628431890e-01 9.5834845322477e-01
1.1268754671447e-01 8.2991892982749e-01 9.4698844726958e-01
7.7507462324343e-01 8.3262196124732e-01 9.3562795628483e-01
4.3753246126732e-01 8.3532540100840e-01 9.2426696314363e-01
1.0006018374002e-01 8.3802927035711e-01 9.1290545354557e-01
7.6265699433358e-01 8.4073358907250e-01 9.0154341632317e-01
4.2532218853184e-01 8.4343837519441e-01 8.9018084370485e-01
8.8055163850278e-02 8.4614364477026e-01 8.7881773152670e-01
7.5085542886742e-01 8.4884941162778e-01 8.6745407938677e-01
4.1372261085117e-01 8.5155568717981e-01 8.5608989073685e-01
7.6656461870174e-02 8.5426248026682e-01 8.4472517290873e-01
7.3965686339042e-01 8.5696979704181e-01 8.3335993707367e-01
4.0272382946261e-01 8.5967764090111e-01 8.2199419813541e-01
6.5857508700836e-02 8.6238601246363e-01 8.1062797455889e-01
7.2905818533038e-01 8.6509490959995e-01 7.9926128813837e-01
3.9232627963730e-01 8.6780432751127e-01 7.8789416370988e-01
5.5662348178704e-02 8.7051425885754e-01 7.7652662881400e-01
7.1906708410883e-01 8.7322469393240e-01 7.6515871331600e-01
3.8254131793733e-01 8.7593562088194e-01 7.5379044899081e-01
4.6086018963811e-02 8.7864702596271e-01 7.4242186908057e-01
7.0970229752605e-01 8.8135889383362e-01 7.3105300783290e-01
3.7339140805966e-01 8.8407120787493e-01 7.1968390002764e-01
3.7154752799401e-02 8.8678395052636e-01 7.0831458049982e-01
7.0099388575858e-01 8.8949710363522e-01 6.9694508366593e-01
3.6491051641132e-01 8.9221064880396e-01 6.8557544306053e-01
2.8906512278561e-02 8.9492456772531e-01 6.7420569088927e-01
6.9298389939115e-01 8.9763884249174e-01 6.6283585760447e-01
3.5714485938318e-01 9.0035345586471e-01 6.5146597150898e-01
2.1391721766903e-02 9.0306839148766e-01 6.4009605839434e-01
6.8572694977291e-01 9.0578363402551e-01 6.2872614121965e-01
3.5015311802379e-01 9.0849916921250e-01 6.1735623983898e-01
1.4672880268268e-02 9.1121498378915e-01 6.0598637078681e-01
6.7928892546458e-01 9.1393106530931e-01 5.9461654713342e-01
3.4400392069894e-01 9.1664740179873e-01 5.8324677842570e-01
8.8204397948727e-03 9.1936398124882e-01 5.7187707073214e-01
6.7374087705113e-01 9.2208079093290e-01 5.6050742681470e-01
3.3876734637454e-01 9.2479781653895e-01 5.4913784645363e-01
3.9015671701731e-03 9.2751504112209e-01 5.3776832695314e-01
6.6914473970839e-01 9.3023244389313e-01 5.2639886385512e-01
3.3449741425802e-01 9.3294999887668e-01 5.1502945188377e-01
5.0780221531349e-01 7.0363471032519e-01 5.0346292468729e-01
1.5104667101834e-01 7.0631485709242e-01 4.9210111792222e-01
7.9667087654059e-01 7.0899938092827e-01 4.8073900451738e-01
4.4450989203216e-01 7.1168833492748e-01 4.6937742788058e-01
9.4283760854934e-02 7.1438087366975e-01 4.5801693991893e-01
7.4569257771558e-01 7.1707601964376e-01 4.4665769986148e-01
3.9846301986691e-01 7.1977295910858e-01 4.3529961270028e-01
5.2363329253042e-02 7.2247109504194e-01 4.2394247386295e-01
7.0720335170888e-01 7.2517001391796e-01 4.1258605956463e-01
3.6282917068523e-01 7.2786943652401e-01 4.0123016976182e-01
1.9116659856184e-02 7.3056917596165e-01 3.8987464261440e-01
6.7596558085887e-01 7.3326910706305e-01 3.7851935489517e-01
3.3329467335676e-01 7.3596914557020e-01 3.6716421673489e-01
9.9103773346242e-01 7.3866923438417e-01 3.5580916490164e-01
6.4914053505683e-01 7.4136933459192e-01 3.4445415644811e-01
3.0755842677436e-01 7.4406941962105e-01 3.3309916336915e-01
9.6625445603014e-01 7.4676947141554e-01 3.2174416836820e-01
6.2519789951757e-01 7.4946947791335e-01 3.1038916161491e-01
2.8436310616500e-01 7.5216943136543e-01 2.9903413831290e-01
9.4372858053863e-01 7.5486932720223e-01 2.8767909689846e-01
6.0327625190868e-01 7.5756916326072e-01 2.7632403771705e-01
2.6299088735162e-01 7.6026893925142e-01 2.6496896205490e-01
9.2285961719290e-01 7.6296865638840e-01 2.5361387143140e-01
5.8287154856655e-01 7.6566831713157e-01 2.4225876708115e-01
2.4301744848986e-01 7.6836792500867e-01 2.3090364957315e-01
9.0328948209599e-01 7.7106748449475e-01 2.1954851852842e-01
5.6368099488544e-01 7.7376700093488e-01 2.0819337240822e-01
2.2418633031096e-01 7.7646648049984e-01 1.9683820835263e-01
8.8480067589283e-01 7.7916593016800e-01 1.8548302205546e-01
5.4551993251511e-01 7.8186535772821e-01 1.7412780766574e-01
2.0634060268456e-01 7.8456477179966e-01 1.6277255770937e-01
8.6725969441992e-01 7.8726418186561e-01 1.5141726302734e-01
5.2827463813954e-01 7.8996359831770e-01 1.4006191272854e-01
1.8938321447164e-01 7.9266303250804e-01 1.2870649415716e-01
8.5058349135803e-01 7.9536249680580e-01 1.1735099287565e-01
5.1187376918052e-01 7.9806200465519e-01 1.0599539266532e-01
1.7325253292896e-01 8.0076157063091e-01 9.4639675547235e-02
8.3471841066214e-01 8.0346121048749e-01 8.3283821826813e-02
4.9627013769791e-01 8.0616094119817e-01 7.1927810165491e-02
1.5790652611246e-01 8.0886078097920e-01 6.0571617683039e-02
8.1962643923592e-01 8.1156074929515e-01 4.9215220093762e-02
4.8142877090533e-01 8.1426086684116e-01 3.7858591879308e-02
1.4331242927938e-01 8.1696115549815e-01 2.6501706500023e-02
8.0527632503493e-01 8.1966163825788e-01 1.5144536645680e-02
4.6731936375557e-01 8.2236233911501e-01 3.7870545251262e-03
1.2848459022470e-01 8.2506597164779e-01 9.9242605950531e-01
7.9066996492267e-01 8.2776726283133e-01 9.8106778013708e-01
4.5293093242269e-01 8.3046884687850e-01 9.6970910713375e-01
1.1526637705789e-01 8.3317075002413e-01 9.5835001435618e-01
7.7767519326770e-01 8.3587299845987e-01 9.4699047708934e-01
4.4015629652379e-01 8.3857561803891e-01 9.3563047246697e-01
1.0270863569089e-01 8.4127863396350e-01 9.2426997988846e-01
7.6533120623541e-01 8.4398207046222e-01 9.1290898141695e-01
4.2802306369513e-01 8.4668595046507e-01 9.0154746214764e-01
9.0783336849319e-02 8.4939029528470e-01 8.9018541053522e-01
7.5361124008215e-01 8.5209512431221e-01 8.7882281867099e-01
4.1650608451103e-01 8.5480045473604e-01 8.6745968250087e-01
7.9467287551538e-02 8.5750630129148e-01 8.5609600197749e-01
7.4249438070667e-01 8.6021267604813e-01 8.4473178114139e-01
4.0558701549298e-01 8.6291958824112e-01 8.3336702812836e-01
6.8744967542021e-02 8.6562704415121e-01 8.2200175510204e-01
7.3196813903410e-01 8.6833504703725e-01 8.1063597811302e-01
3.9525655972492e-01 8.7104359712341e-01 7.9926971688749e-01
5.8610386906330e-02 8.7375269164206e-01 7.8790299455033e-01
7.2202990469495e-01 8.7646232493196e-01 7.7653583728888e-01
3.8551552306061e-01 8.7917248858991e-01 7.6516827396492e-01
4.9067776988170e-02 8.8188317167289e-01 7.5380033568328e-01
7.1268732610936e-01 8.8459436094611e-01 7.4243205532611e-01
3.7637495504555e-01 8.8730604117142e-01 7.3106346706222e-01
4.0131574570730e-02 8.9001819542885e-01 7.1969460584091e-01
7.0395822353890e-01 8.9273080546302e-01 7.0832550687975e-01
3.6785607132853e-01 8.9544385204467e-01 6.9695620515524e-01
3.1826420344591e-02 8.9815731533620e-01 6.8558673490503e-01
6.9587070789520e-01 9.0087117524904e-01 6.7421712914972e-01
3.5999050653535e-01 9.0358541177907e-01 6.6284741924180e-01
2.4187521755964e-02 9.0630000530542e-01 6.5147763444895e-01
6.8846358570591e-01 9.0901493683673e-01 6.4010780157864e-01
3.5282064548299e-01 9.1173018818829e-01 6.2873794465114e-01
1.7260744433735e-02 9.1444574207265e-01 6.1736808462868e-01
6.8178599488154e-01 9.1716158208635e-01 6.0599823920949e-01
3.4639854078020e-01 9.1987769257564e-01 5.9462842269721e-01
1.1100508991028e-02 9.2259405836552e-01 5.8325864595812e-01
6.7589394823250e-01 9.2531066433860e-01 5.7188891648148e-01
3.4078075527918e-01 9.2802749485446e-01 5.6051923856047e-01
5.7625887121248e-03 9.3074453300599e-01 5.4914961361364e-01
6.7084077145407e-01 9.3346175971759e-01 5.3778004066770e-01
3.3601618444911e-01 9.3617915270122e-01 5.2641051702135e-01
1.2891551193474e-03 9.3889668530030e-01 5.1504103910615e-01
1.7541961350670e-01 7.0959008944014e-01 5.0345941821717e-01
8.1604812794788e-01 7.1226705348364e-01 4.9209733757066e-01
4.5980299649239e-01 7.1494990103914e-01 4.8073491411401e-01
1.0639707885470e-01 7.1763842619179e-01 4.6937336343258e-01
7.5538732562936e-01 7.2033122562466e-01 4.5801332168434e-01
4.0633303737869e-01 7.2302691207279e-01 4.4665482629112e-01
5.8857060172418e-02 7.2572445340840e-01 4.3529761467459e-01
7.1265577139751e-01 7.2842315946636e-01 4.2394134854331e-01
3.6749037775636e-01 7.3112258790873e-01 4.1258572112066e-01
2.3174179202788e-02 7.3382245867814e-01 4.0123049141533e-01
6.7956088365117e-01 7.3652259385014e-01 3.8987548483178e-01
3.3653518991235e-01 7.3922287905616e-01 3.7852058191832e-01
9.9400555714779e-01 7.4192323962873e-01 3.6716570506727e-01
6.5189874969001e-01 7.4462362601931e-01 3.5581080681302e-01
3.1015573677238e-01 7.4732400487044e-01 3.4445586064015e-01
9.6872860192426e-01 7.5002435350028e-01 3.3310085416257e-01
6.2757819927366e-01 7.5272465644861e-01 3.2174578422283e-01
2.8667236269768e-01 7.5542490327829e-01 3.1039065344042e-01
9.4598452602128e-01 7.5812508715014e-01 2.9903546781197e-01
6.0549265080194e-01 7.6082520388195e-01 2.8768023505840e-01
2.6517838596567e-01 7.6352525131570e-01 2.7632496349435e-01
9.2502640354391e-01 7.6622522888536e-01 2.6496966125921e-01
5.8502386919039e-01 7.6892513731821e-01 2.5361433579614e-01
2.4516001662518e-01 7.7162497842767e-01 2.4225899349971e-01
9.0542580279642e-01 7.7432475497095e-01 2.3090363947679e-01
5.6581362616918e-01 7.7702447055427e-01 2.1954827738269e-01
2.2631709471536e-01 7.7972412957431e-01 2.0819290930608e-01
8.8693083328730e-01 7.8242373718847e-01 1.9683753568508e-01
5.4765032239915e-01 7.8512329930879e-01 1.8548215524245e-01
2.0847176221688e-01 7.8782282261564e-01 1.7412676493251e-01
8.6939195691871e-01 7.9052231458843e-01 1.6277135989508e-01
5.3040821563924e-01 7.9322178355069e-01 1.5141593341441e-01
1.9151826703010e-01 7.9592123872725e-01 1.4006047688272e-01
8.5272018511544e-01 7.9862069031079e-01 1.2870497976927e-01
5.1401232463290e-01 8.0132014953523e-01 1.1734942959722e-01
1.7539326446109e-01 8.0401962875276e-01 1.0599381193123e-01
8.3686175806474e-01 8.0671914151091e-01 9.4638110379701e-02
4.9841669015521e-01 8.0941870262588e-01 8.3282306615790e-02
1.6005703897775e-01 8.1211832824768e-01 7.1926380422113e-02
8.2178184380596e-01 8.1481803591252e-01 6.0570309763695e-02
4.8359017735317e-01 8.1751784457745e-01 4.9214070893958e-02
1.4548112290430e-01 8.2021777463252e-01 3.7857638497953e-02
8.0745375603099e-01 8.2291784788546e-01 2.6500985876377e-02
4.6950713078158e-01 8.2561808751469e-01 1.5144085172905e-02
1.3164027023625e-01 8.2831851798676e-01 3.7869076460139e-03
7.9292144223433e-01 8.3102178451720e-01 9.9242633258595e-01
4.5519817775030e-01 8.3372275493439e-01 9.8106842251391e-01
1.1755131068497e-01 8.3642399427721e-01 9.6971014890668e-01
7.7997972730439e-01 8.3912553054914e-01 9.5835148304134e-01
4.4248229421896e-01 8.4182739203965e-01 9.4699239725578e-01
1.0505786995475e-01 8.4452960701447e-01 9.3563286541232e-01
7.6770531845551e-01 8.4723220337963e-01 9.2427286336332e-01
4.3042352388341e-01 8.4993520832677e-01 9.1291236940660e-01
9.3211406059014e-02 8.5263864796810e-01 9.0155136471779e-01
7.5606793588336e-01 8.5534254697033e-01 8.9018983374695e-01
4.1899215012044e-01 8.5804692819738e-01 8.7882776456713e-01
8.1983164985334e-02 8.6075181237166e-01 8.6746514916413e-01
7.4504018808081e-01 8.6345721776328e-01 8.5610198365793e-01
4.0816252834595e-01 8.6616315991610e-01 8.4473826844864e-01
7.1349603818965e-02 8.6886965141834e-01 8.3337400828185e-01
7.3460094716205e-01 8.7157670172417e-01 8.2200921223076e-01
3.9791620904119e-01 8.7428431703157e-01 8.1064389359505e-01
6.1295159586256e-02 8.7699250021986e-01 7.9927806971866e-01
7.2473768826916e-01 8.7970125084907e-01 7.8791176173083e-01
3.8824380262095e-01 8.8241056522137e-01 7.7654499421686e-01
5.1813626255670e-02 8.8512043650343e-01 7.6517779482638e-01
7.1544739667612e-01 8.8783085490695e-01 7.5381019382863e-01
3.7914546329930e-01 8.9054180792299e-01 7.4244222362473e-01
4.2908286059682e-02 8.9325328060430e-01 7.3107391822802e-01
7.0673643482523e-01 8.9596525588831e-01 7.1970531272353e-01
3.7063058970127e-01 8.9867771495196e-01 7.0833644271774e-01
3.4591542104855e-02 9.0139063758816e-01 6.9696734378984e-01
6.9862019627729e-01 9.0410400259233e-01 6.8559805095477e-01
3.6271757067392e-01 9.0681778814586e-01 6.7422859814812e-01
2.6884798441051e-02 9.0953197218269e-01 6.6285901774228e-01
6.9112312597282e-01 9.1224653272351e-01 6.5148934010237e-01
3.5543390835070e-01 9.1496144816180e-01 6.4011959319016e-01
1.9818600320170e-02 9.1767669748497e-01 6.2874980222373e-01
6.8427874137071e-01 9.2039226041404e-01 6.1737998940074e-01
3.4881593347280e-01 9.2310811744518e-01 6.0601017369340e-01
1.3431810108460e-02 9.2582424977773e-01 5.9464037072425e-01
6.7812799542914e-01 9.2854063911463e-01 5.8327059273290e-01
3.4290605270470e-01 9.3125726732416e-01 5.7190084864553e-01
7.7674217071340e-03 9.3397411595577e-01 5.6053114426052e-01
6.7271334531320e-01 9.3669116560834e-01 5.4916148256476e-01
3.3774478641170e-01 9.3940839515678e-01 5.3779186419555e-01
2.8623371472425e-03 9.4212578085228e-01 5.2642228806188e-01
6.6806612358097e-01 9.4484329532318e-01 5.1505275213523e-01
8.4338321440877e-01 7.1554568852049e-01 5.0345638171259e-01
4.8050403438080e-01 7.1821833629064e-01 4.9209400858388e-01
1.2190720329690e-01 7.2089920690639e-01 4.8073131084154e-01
7.6706187025985e-01 7.2358747568011e-01 4.6937008297612e-01
4.1523634339540e-01 7.2628079183464e-01 4.5801095758679e-01
6.5767043853126e-02 7.2897720683579e-01 4.4665368712918e-01
7.1812761952784e-01 7.3167545402833e-01 4.3529773974130e-01
3.7191919273715e-01 7.3437477477202e-01 4.2394261074919e-01
2.6842750765380e-02 7.3707472425048e-01 4.1258791774130e-01
6.8267338352499e-01 7.3977503983385e-01 4.0123340175634e-01
3.3924039012876e-01 7.4247556313900e-01 3.8987890067929e-01
9.9641279576950e-01 7.4517619592262e-01 3.7852432073139e-01
6.5408898227128e-01 7.4787687535484e-01 3.6716961361015e-01
3.1218926387087e-01 7.5057756005003e-01 3.5581476005927e-01
9.7065053387518e-01 7.5327822203977e-01 3.4445975867853e-01
6.2942236550819e-01 7.5597884205586e-01 3.3310461853205e-01
2.8846414001562e-01 7.5867940668588e-01 3.2174935436270e-01
9.4774290710168e-01 7.6137990660926e-01 3.1039398354283e-01
6.0723177290589e-01 7.6408033547135e-01 2.9903852416182e-01
2.6690867209979e-01 7.6678068914375e-01 2.8768299384888e-01
9.2675542265772e-01 7.6948096522530e-01 2.7632740906661e-01
5.8675699079001e-01 7.7218116269770e-01 2.6497178470222e-01
2.4690091366140e-01 7.7488128168411e-01 2.5361613384408e-01
9.0717684167080e-01 7.7758132327896e-01 2.4226046767041e-01
5.6757617211892e-01 7.8028128942949e-01 2.3090479540338e-01
2.2809175330155e-01 7.8298118285629e-01 2.1954912429804e-01
8.8871764329113e-01 7.8568100700498e-01 2.0819345964724e-01
5.4944891149355e-01 7.8838076602371e-01 1.9683780479041e-01
2.1028147389267e-01 7.9108046476276e-01 1.8548216111934e-01
8.7121195500412e-01 7.9378010879385e-01 1.7412652807703e-01
5.3223757114923e-01 7.9647970444688e-01 1.6277090314823e-01
1.9335603087016e-01 7.9917925886242e-01 1.5141528184186e-01
8.5456544923868e-01 8.0187878005802e-01 1.4005965766664e-01
5.1586427353582e-01 8.0457827700626e-01 1.2870402210257e-01
1.7725121835010e-01 8.0727775972225e-01 1.1734836457141e-01
8.3872520859559e-01 8.0997723935763e-01 1.0599267241042e-01
5.0028532931594e-01 8.1267672829783e-01 9.4636930854002e-02
1.6193078143463e-01 8.1537624025866e-01 8.3281123028566e-02
8.2366084285072e-01 8.1807579037778e-01 7.1925229966419e-02
4.8547483447017e-01 8.2077539529618e-01 6.0569230644549e-02
1.4737209091395e-01 8.2347507322428e-01 4.9213102054369e-02
8.0935193575640e-01 8.2617484398708e-01 3.7856819308179e-02
4.7141366122571e-01 8.2887472904267e-01 2.6500355787554e-02
1.3355651234170e-01 8.3157475146855e-01 1.5143683337994e-02
7.9577967547704e-01 8.3427493591091e-01 3.7867725129075e-03
4.5717714872059e-01 8.3697785763230e-01 9.9242658431206e-01
1.1954514818459e-01 8.3967852763507e-01 9.8106901232821e-01
7.8199039894321e-01 8.4237944022451e-01 9.6971111052993e-01
4.4451182466548e-01 8.4508062493946e-01 9.5835284774251e-01
1.0710829611154e-01 8.4778211196880e-01 9.4699419343796e-01
7.6977864303922e-01 8.5048393183437e-01 9.3563511823142e-01
4.3252166861485e-01 8.5318611503756e-01 9.2427559439156e-01
9.5336165673718e-02 8.5588869167704e-01 9.1291559635176e-01
7.5822093411352e-01 8.5859169104634e-01 9.0155510120760e-01
4.2117479867486e-01 8.6129514122135e-01 8.9019408918600e-01
8.4196626370858e-02 8.6399906864880e-01 8.7883254407127e-01
7.4728534287563e-01 8.6670349774686e-01 8.6747045357449e-01
4.1043994726780e-01 8.6940845052937e-01 8.5610780963397e-01
7.3659524645671e-02 8.7211394626416e-01 8.4474460863668e-01
7.3694325627885e-01 8.7482000117544e-01 8.3338085155303e-01
4.0029042712560e-01 8.7752662819866e-01 8.2201654398018e-01
6.3700430717238e-02 8.8023383679485e-01 8.1065169609177e-01
7.2717277157618e-01 8.8294163282958e-01 7.9928632249508e-01
3.9070706548273e-01 8.8565001852000e-01 7.8792044199915e-01
5.4303038025955e-02 8.8835899245148e-01 7.7655407729994e-01
7.1796052195696e-01 8.9106854966332e-01 7.6518725459082e-01
3.8167945390163e-01 8.9377868180153e-01 7.5382000310833e-01
4.5459870980324e-02 8.9648937733441e-01 7.4245235462465e-01
7.0930190782126e-01 8.9920062182531e-01 7.3108434289920e-01
3.7320579434568e-01 9.0191239825495e-01 7.1971600310222e-01
3.7171854547970e-02 9.0462468738414e-01 7.0834737122381e-01
7.0120050630142e-01 9.0733746814619e-01 6.9697848348122e-01
3.6529226199910e-01 9.1005071805668e-01 6.8560937573736e-01
2.9447729600988e-02 9.1276441362701e-01 6.7424008294251e-01
6.9366761341756e-01 9.1547853076686e-01 6.6287063861045e-01
3.5795271373111e-01 9.1819304515965e-01 6.5150107433956e-01
2.2303924155615e-02 9.2090793259466e-01 6.4013141938806e-01
6.8672222548110e-01 9.2362316923885e-01 6.2876170031242e-01
3.5120867465511e-01 9.2633873183176e-01 6.1739194067667e-01
1.5764387539379e-02 9.2905459778730e-01 6.0602216084067e-01
6.8039051415860e-01 9.3177074518784e-01 5.9465237783499e-01
3.4508820534645e-01 9.3448715265778e-01 5.8328260533066e-01
9.8585700026820e-03 9.3720379910673e-01 5.7191285371264e-01
6.7470262261389e-01 9.3992066333684e-01 5.6054313026669e-01
3.3962122126394e-01 9.4263772351358e-01 5.4917343948949e-01
4.6149970600242e-03 9.4535495650658e-01 5.3780378353227e-01
6.6968427670974e-01 9.4807233711500e-01 5.2643416278628e-01
3.3482900086291e-01 9.5078983720206e-01 5.1506457661587e-01
5.1192236094135e-01 7.2150180554000e-01 5.0345381177766e-01
1.4409296684809e-01 7.2416833006886e-01 4.9209112047171e-01
7.8246909222861e-01 7.2684694910627e-01 4.8072829655151e-01
4.2598860981668e-01 7.2953534125298e-01 4.6936802216438e-01
7.3381237918637e-02 7.3222953781500e-01 4.5801062033202e-01
7.2362352371053e-01 7.3492688182578e-01 4.4665525819925e-01
3.7597551105337e-01 7.3762591223452e-01 4.3530102463223e-01
2.9914714138237e-02 7.4032586145006e-01 4.2394725905902e-01
6.8507032052001e-01 7.4302632151293e-01 4.1259355630510e-01
3.4117532684569e-01 7.4572706736176e-01 4.0123969501340e-01
9.9803421472149e-01 7.4842796859517e-01 3.8988557020244e-01
6.5550141167423e-01 7.5112894557403e-01 3.7853114562854e-01
3.1346683844775e-01 7.5382994714330e-01 3.6717642348472e-01
9.7184607368736e-01 7.5653093896303e-01 3.5582142608071e-01
6.3057355101214e-01 7.5923189715855e-01 3.4446618510617e-01
2.8959777514760e-01 7.6193280471647e-01 3.3311073550130e-01
9.4887790362316e-01 7.6463364934652e-01 3.2175511206616e-01
6.0838126684041e-01 7.6733442215506e-01 3.1039934767343e-01
2.6808154297307e-01 7.7003511678574e-01 2.9904347240625e-01
9.2795739657604e-01 7.7273572884041e-01 2.8768751321917e-01
5.8799145011502e-01 7.7543625547531e-01 2.7633149388516e-01
2.4816949761278e-01 7.7813669511218e-01 2.6497543508996e-01
9.0847989649572e-01 7.8083704722819e-01 2.5361935459323e-01
5.6891309206366e-01 7.8353731220276e-01 2.4226326741001e-01
2.2946124168215e-01 7.8623749120733e-01 2.3090718598719e-01
8.9011791467331e-01 7.8893758612954e-01 2.1955112036086e-01
5.5087785017260e-01 7.9163759952587e-01 2.0819507828824e-01
2.1173675973146e-01 7.9433753459909e-01 1.9683906535145e-01
8.7269116471852e-01 7.9703739519781e-01 1.8548308503327e-01
5.3373826097323e-01 7.9973718583639e-01 1.7412713876618e-01
1.9487580494669e-01 8.0243691173365e-01 1.6277122595705e-01
8.5610201690083e-01 8.0513657886894e-01 1.5141534399044e-01
5.1741549775188e-01 8.0783619405432e-01 1.4005948821397e-01
1.7881515692506e-01 8.1053576502112e-01 1.2870365190966e-01
8.4030014919543e-01 8.1323530051878e-01 1.1734782625584e-01
5.0186981897156e-01 8.1593481042364e-01 1.0599200028444e-01
1.6352365086317e-01 8.1863430585446e-01 9.4636160839388e-02
8.2526122568685e-01 8.2133379929107e-01 8.3280292541948e-02
4.8708218131786e-01 8.2403330469158e-01 7.1924377769865e-02
1.4898617800452e-01 8.2673283760342e-01 6.0568396657012e-02
8.1097286792815e-01 8.2943241526213e-01 4.9212327120870e-02
4.7304186892297e-01 8.3213205667209e-01 3.7856144924967e-02
1.3519274236520e-01 8.3483178266257e-01 2.6499823783118e-02
7.9742497530191e-01 8.3753161591251e-01 1.5143335511688e-02
4.5973796691583e-01 8.4023158093776e-01 3.7866502349499e-03
1.2125195687085e-01 8.4293418143193e-01 9.9242681248953e-01
7.8371096512492e-01 8.4563457386160e-01 9.8106954408276e-01
4.4624811442465e-01 8.4833518020037e-01 9.6971198325634e-01
1.0886240602762e-01 8.5103603124645e-01 9.5835409656969e-01
7.7155275161941e-01 8.5373715884387e-01 9.4699585076447e-01
4.3431798513354e-01 8.5643859556676e-01 9.3563721328527e-01
9.7156877716577e-02 8.5914037435646e-01 9.2427815282838e-01
7.6006815519229e-01 8.6184252811805e-01 9.1291863990488e-01
4.2305051727617e-01 8.6454508928515e-01 9.0155864740237e-01
8.6102657720987e-02 8.6724808936361e-01 8.9019815112869e-01
7.4922328454546e-01 8.6995155846594e-01 8.7883713032038e-01
4.1241113951498e-01 8.7265552484946e-01 8.6747556809936e-01
7.5665016109335e-02 8.7536001447120e-01 8.5611345186222e-01
7.3898377532362e-01 8.7806505057249e-01 8.4475077358888e-01
4.0236635879965e-01 8.8077065330543e-01 8.3338753005954e-01
6.5811798965429e-02 8.8347683941194e-01 8.2202372297241e-01
7.2931922605833e-01 8.8618362196478e-01 8.1065935895740e-01
3.9288787210823e-01 8.8889101017776e-01 7.9929444948511e-01
5.6517072146237e-02 8.9159900929038e-01 7.8792901067306e-01
7.2020626306904e-01 8.9430762052986e-01 7.7656306299479e-01
3.8395498071542e-01 8.9701684115154e-01 7.6519663090002e-01
4.7762855791086e-02 8.9972666455600e-01 7.5382974235629e-01
7.1162960930277e-01 9.0243708047965e-01 7.4246242832479e-01
3.7555504812985e-01 9.0514807525298e-01 7.3109472218415e-01
3.9539061270806e-02 9.0785963211905e-01 7.1972665911719e-01
7.0358161715704e-01 9.1057173160264e-01 7.0835827547601e-01
3.6768276223361e-01 9.1328435191875e-01 6.9698960814086e-01
3.1842620777759e-02 9.1599746940762e-01 6.8562069388801e-01
6.9606139567366e-01 9.1871105898159e-01 6.7425156878100e-01
3.6033936960189e-01 9.2142509456820e-01 6.6288226759895e-01
2.4676905852714e-02 9.2413954953276e-01 6.5151282331411e-01
6.8907444775142e-01 9.2685439706297e-01 6.4014326662984e-01
3.5353251551038e-01 9.2956961049816e-01 6.2877362558884e-01
1.8051699213862e-02 9.3228516358602e-01 6.1740392526020e-01
6.8263264661422e-01 9.3500103065049e-01 6.0603418751280e-01
3.4727604448571e-01 9.3771718665642e-01 5.9466443088217e-01
1.1982592456277e-02 9.4043360715862e-01 5.8329467053710e-01
6.7675296852575e-01 9.4315026812646e-01 5.7192491835234e-01
3.4158778588424e-01 9.4586714563906e-01 5.6055518309379e-01
6.4875411650896e-03 9.4858421545146e-01 5.4918547072222e-01
6.7145255489091e-01 9.5130145243825e-01 5.3781578482113e-01
3.3648290557800e-01 9.5401882992878e-01 5.2644612715312e-01
1.5783597173257e-03 9.5673631895607e-01 5.1507649834622e-01
1.8152600144212e-01 7.2745907011052e-01 5.0345169024633e-01
8.0618155245092e-01 7.3011633792355e-01 4.9208864946140e-01
4.4057536460258e-01 7.3279265603531e-01 4.8072621678314e-01
8.2342539181746e-02 7.3548190626617e-01 4.6936826950523e-01
7.2914731495124e-01 7.3817743079244e-01 4.5801392364831e-01
3.7937982744245e-01 7.4087585824678e-01 4.4666131656875e-01
3.2001778358303e-02 7.4357568812456e-01 4.3530915934459e-01
6.8633872071814e-01 7.4627624113893e-01 4.2395677805969e-01
3.4194142456984e-01 7.4897718760648e-01 4.1260388381066e-01
9.9850226858265e-01 7.5167835262668e-01 4.0125039220516e-01
6.5580503240697e-01 7.5437963462526e-01 3.8989631761626e-01
3.1369412983478e-01 7.5708096967585e-01 3.7854171717233e-01
9.7205525700161e-01 7.5978231487672e-01 3.6718666262068e-01
6.3080289565549e-01 7.6248364004916e-01 3.5583122685563e-01
2.8987203982482e-01 7.6518492328039e-01 3.4447547792807e-01
9.4921259112312e-01 7.6788614835397e-01 3.3311947678247e-01
6.0878548097316e-01 7.7058730316693e-01 3.2176327679202e-01
2.6855993532513e-01 7.7328837869638e-01 3.1040692411109e-01
9.2851151076178e-01 7.7598936829125e-01 2.9905045835275e-01
5.8862066124555e-01 7.7869026716794e-01 2.8769391335001e-01
2.4887167620311e-01 7.8139107204077e-01 2.7633731788799e-01
9.0925188257829e-01 7.8409178084609e-01 2.6498069635904e-01
5.6975103724614e-01 7.8679239253442e-01 2.5362406932551e-01
2.3036085852275e-01 7.8949290691418e-01 2.4226745398994e-01
8.9107466053835e-01 7.9219332453638e-01 2.3091086457939e-01
5.5188706451412e-01 7.9489364661290e-01 2.1955431265282e-01
2.1279376810511e-01 7.9759387496360e-01 2.0819780734095e-01
8.7379135897863e-01 8.0029401198893e-01 1.9684135552721e-01
5.3487716236219e-01 8.0299406066575e-01 1.8548496197756e-01
1.9604911486654e-01 8.0569402456485e-01 1.7412862942613e-01
8.5730565876695e-01 8.0839390788878e-01 1.6277235862268e-01
5.1864565231483e-01 8.1109371552916e-01 1.5141614834745e-01
1.8006829269282e-01 8.1379345314219e-01 1.4005999539869e-01
8.4157304901849e-01 8.1649312724117e-01 1.2870389455790e-01
5.0315960341349e-01 8.1919274530449e-01 1.1734783853806e-01
1.6482779863540e-01 8.2189231589675e-01 1.0599181792034e-01
8.2657759115432e-01 8.2459184880063e-01 9.4635821085320e-02
4.8840900886782e-01 8.2729135515585e-01 8.3279834145089e-02
1.5032211290638e-01 8.2999084760112e-01 7.1923840883394e-02
8.1231696319392e-01 8.3269034041420e-01 6.0567822711374e-02
4.7439358760589e-01 8.3538984964407e-01 4.9211758646982e-02
1.3655195471005e-01 8.3808939322893e-01 3.7855625326403e-02
7.9879195018666e-01 8.4078899109271e-01 2.6499397055831e-02
4.6111335710263e-01 8.4348866521278e-01 1.5143045911545e-02
1.2351584025779e-01 8.4618843965101e-01 3.7865418954987e-03
7.8514639602621e-01 8.4889074489308e-01 9.9242701495155e-01
4.4769597078348e-01 8.5159088513111e-01 9.8107001229035e-01
1.1032459604569e-01 8.5429120837001e-01 9.6971275826701e-01
7.7303139565032e-01 8.5699174634675e-01 9.5835521742283e-01
4.3581536401739e-01 8.5969253226645e-01 9.4699735396897e-01
9.8675377339100e-02 8.6239360049877e-01 9.3563913231999e-01
7.6161020860379e-01 8.6509498621543e-01 9.2428051767171e-01
4.2461854585802e-01 8.6779672497426e-01 9.1292147660959e-01
8.7699012961199e-02 8.7049885225820e-01 9.0156197773117e-01
7.5085019196749e-01 8.7320140297979e-01 8.9020199226433e-01
4.1407064619201e-01 8.7590441096389e-01 8.7884149466155e-01
7.7358942993156e-02 8.7860790842284e-01 8.6748046315061e-01
7.4071367533446e-01 8.8131192543904e-01 8.5611888022259e-01
4.0413348128013e-01 8.8401648947029e-01 8.4475673303980e-01
6.7617060722657e-02 8.8672162489252e-01 8.3339401374906e-01
7.3116318883016e-01 8.8942735259364e-01 8.2203071968873e-01
3.9477072592239e-01 8.9213368963046e-01 8.1066685348184e-01
5.8438623720523e-02 8.9484064895862e-01 7.9930242301150e-01
7.2216592815098e-01 8.9754823924307e-01 7.8793744127900e-01
3.8595177909823e-01 9.0025646475423e-01 7.7657192614872e-01
4.9795407681836e-02 9.0296532535209e-01 7.6520589998760e-01
7.1369613176208e-01 9.0567481655837e-01 7.5383938921006e-01
3.7765335044977e-01 9.0838492971379e-01 7.4247242374175e-01
4.1666538398910e-02 9.1109565221547e-01 7.3110503641767e-01
7.0573524059698e-01 9.1380696782692e-01 7.1973726233158e-01
3.6985906823678e-01 9.1651885705092e-01 7.0836913815436e-01
3.4037696064196e-02 9.1923129755348e-01 6.9700070143964e-01
6.9827086135915e-01 9.2194426462517e-01 6.8563198993437e-01
3.6255836443636e-01 9.2465773166432e-01 6.7426304091175e-01
2.6900070267953e-02 9.2737167066526e-01 6.6289389054262e-01
6.9129591055062e-01 9.3008605269370e-01 6.5152457332001e-01
3.5574588528824e-01 9.3280084833066e-01 6.4015512154997e-01
2.0250062756857e-02 9.3551602806622e-01 6.2878556491993e-01
6.8480857658371e-01 9.3823156262523e-01 6.1741593015414e-01
3.4942161862133e-01 9.4094742320787e-01 6.0604624076382e-01
1.4089426346122e-02 9.4366358163011e-01 5.9467651689840e-01
6.7881226366700e-01 9.4638001035189e-01 5.8330677530287e-01
3.4359039429747e-01 9.4909668238408e-01 5.7193702938518e-01
8.4240472232340e-03 9.5181357106985e-01 5.6056728939736e-01
6.7331337427923e-01 9.5453064974083e-01 5.4919756273258e-01
3.3825840041930e-01 9.5724789125489e-01 5.3782785434030e-01
3.2589678955021e-03 9.5996526742855e-01 5.2645816725963e-01
6.6831467622751e-01 9.6268274838499e-01 5.1508850326896e-01
8.5345213016179e-01 7.3341905175595e-01 5.0344995498712e-01
4.6533197879665e-01 7.3606093272642e-01 4.9208657484719e-01
9.4457650286669e-02 7.3873577354990e-01 4.8072639528848e-01
7.3470205197834e-01 7.4142709196940e-01 4.6937375114896e-01
3.8149612916284e-01 7.4412432093219e-01 4.5802432712611e-01
3.2295933920388e-02 7.4682384687923e-01 4.4667507559939e-01
6.8568008771265e-01 7.4952442286174e-01 4.3532481459208e-01
3.4081416166272e-01 7.5222554928741e-01 4.2397326925522e-01
9.9718292942687e-01 7.5492698714631e-01 4.1262050966437e-01
6.5445382951472e-01 7.5762860287085e-01 4.0126671045269e-01
3.1240392318769e-01 7.6033031281129e-01 3.8991205909508e-01
9.7087887838245e-01 7.6303206049187e-01 3.7855672483957e-01
6.2976890107318e-01 7.6573380594124e-01 3.6720085105986e-01
2.8899405011035e-01 7.6843552003763e-01 3.5584455606697e-01
9.4849496389851e-01 7.7113718119702e-01 3.4448793651534e-01
6.0822682390762e-01 7.7383877327995e-01 3.3313107119088e-01
2.6815531782448e-01 7.7654028419179e-01 3.2177402442108e-01
9.2825387420191e-01 7.7924170490726e-01 3.1041684891281e-01
5.8850172674148e-01 7.8194302876952e-01 2.9905958803206e-01
2.4888253257739e-01 7.8464425097471e-01 2.8770227760434e-01
9.0938336828882e-01 7.8734536818677e-01 2.7634494732522e-01
5.6999398833651e-01 7.9004637824635e-01 2.6498762186081e-01
2.3070626893320e-01 7.9274727995006e-01 2.5363032170409e-01
8.9151378496506e-01 7.9544807288369e-01 2.4227306383907e-01
5.5241148370247e-01 7.9814875729855e-01 2.3091586225325e-01
2.1339542979622e-01 8.0084933402304e-01 2.1955872832898e-01
8.7446260335490e-01 8.0354980440444e-01 2.0820167113768e-01
5.3561073793220e-01 8.0625017027728e-01 1.9684469765458e-01
1.9683818877347e-01 8.0895043395609e-01 1.8548781290820e-01
8.5814382416932e-01 8.1165059825066e-01 1.7413102007510e-01
5.1952693456224e-01 8.1435066650310e-01 1.6277432052856e-01
1.8098715536363e-01 8.1705064264559e-01 1.5141771384829e-01
8.4252440041089e-01 8.1975053127831e-01 1.4006119779708e-01
5.0413880372547e-01 8.2245033776655e-01 1.2870476826995e-01
1.6583066779427e-01 8.2515006835583e-01 1.1734841922109e-01
8.2760041703558e-01 8.2784973030368e-01 1.0599214257405e-01
4.8944855546254e-01 8.3054933202552e-01 9.4635928120997e-02
1.5137562784509e-01 8.3324888325212e-01 8.3279763417301e-02
8.1338218391222e-01 8.3594839519453e-01 7.1923633678708e-02
4.7546874534128e-01 8.3864788071209e-01 6.0567521688718e-02
1.3763577545615e-01 8.4134735447762e-01 4.9211407724796e-02
7.9988365170444e-01 8.4404683313335e-01 3.7855269512518e-02
4.6221264110568e-01 8.4674633542994e-01 2.6499082217152e-02
1.2462287895619e-01 8.4944588234045e-01 1.5142818482258e-02
7.8711435113831e-01 8.5214549714058e-01 3.7864485244804e-03
4.4886132916462e-01 8.5484753545020e-01 9.9242718960910e-01
1.1150081188032e-01 8.5754745142206e-01 9.8107041158729e-01
7.7422025504342e-01 8.6024751741772e-01 9.6971342683177e-01
4.3701895481600e-01 8.6294776574685e-01 9.5835619818266e-01
9.9896035320244e-02 8.6564823062896e-01 9.4699868759445e-01
7.6285045852848e-01 8.6834894791446e-01 9.3564085666801e-01
4.2588103869549e-01 8.7104995473473e-01 9.2428266724491e-01
8.8986460837743e-02 8.7375128908494e-01 9.1292408205107e-01
7.5216530256527e-01 8.7645298934683e-01 9.0156506537326e-01
4.1541605839454e-01 8.7915509376137e-01 8.9020558374880e-01
7.8737165533783e-02 8.8185763986453e-01 8.7884560664682e-01
7.4212703004929e-01 8.8456066390115e-01 8.6748510711810e-01
4.0558405230409e-01 8.8726420023378e-01 8.5612406239069e-01
6.9106650613801e-02 8.8996828076424e-01 8.4476245438962e-01
7.3269328218878e-01 8.9267293438542e-01 8.3340027016132e-01
3.9634246061877e-01 8.9537818648015e-01 8.2203750218681e-01
6.0052769393910e-02 8.9808405848249e-01 8.1067414857180e-01
7.2382287122619e-01 9.0079056751454e-01 7.9931021310610e-01
3.8765151321806e-01 9.0349772610935e-01 7.8794570519016e-01
5.1537528197845e-02 9.0620554202765e-01 7.7658063963058e-01
7.1547983278351e-01 9.0891401817295e-01 7.6521503631150e-01
3.7947742292958e-01 9.1162315260679e-01 7.5384891975248e-01
4.3529367839818e-02 9.1433293866250e-01 7.4248231856702e-01
7.0763480318135e-01 9.1704336515324e-01 7.3111526483857e-01
3.7179292450849e-01 9.1975441666713e-01 7.1974779343317e-01
3.6002981696887e-02 9.2246607393966e-01 7.0837994126889e-01
7.0026427500674e-01 9.2517831429115e-01 6.9701174656324e-01
3.6457615314990e-01 9.2789111211463e-01 6.8564324807957e-01
2.8938013444664e-02 9.3060443939780e-01 6.7427448439285e-01
6.9334930382660e-01 9.3331826626068e-01 6.6290549319413e-01
3.5780952616571e-01 9.3603256148952e-01 6.5153631065129e-01
2.2318240045985e-02 9.3874729304707e-01 6.4016697084163e-01
6.8687506591666e-01 9.4146242853862e-01 6.2879750526952e-01
3.5147968635017e-01 9.4417793561451e-01 6.1742794247979e-01
1.6131844056697e-02 9.4689378229079e-01 6.0605830777529e-01
6.8083133532563e-01 9.4960993717202e-01 5.9468862304448e-01
3.4557799769780e-01 9.5232636956328e-01 5.8331890670295e-01
1.0371690920197e-02 9.5504304946212e-01 5.7194917375114e-01
6.7521227059294e-01 9.5775994742584e-01 5.6057943594876e-01
3.4009955437324e-01 9.6047703431456e-01 5.4920970210552e-01
5.0332811351363e-03 9.6319428091639e-01 5.3783997848626e-01
6.7001306406962e-01 9.6591165746757e-01 5.2647026932729e-01
3.3503833931327e-01 9.6862913308672e-01 5.1510057745867e-01
5.3218358559973e-01 7.3938676316890e-01 5.0344843381458e-01
1.1748520880738e-01 7.4199890049133e-01 4.9208529604884e-01
7.4029004220918e-01 7.4467585900381e-01 4.8073475163643e-01
3.8052307495783e-01 7.4737051983943e-01 4.6939275783382e-01
2.8817770363668e-02 7.5006944708192e-01 4.5804912127175e-01
6.8134842531180e-01 7.5276998304130e-01 4.4670199465788e-01
3.3634697718040e-01 7.5547133019988e-01 4.3535183208026e-01
9.9290053766616e-01 7.5817313596636e-01 4.2399937421356e-01
6.5049521450096e-01 7.6087520540641e-01 4.1264523877823e-01
3.0882150163591e-01 7.6357741778815e-01 4.0128988407025e-01
9.6768272184905e-01 7.6627969356460e-01 3.8993364165713e-01
6.2694838237343e-01 7.6898197828807e-01 3.7857675200033e-01
2.8652880746805e-01 7.7168423372362e-01 3.6721939159222e-01
9.4636057248720e-01 7.7438643254403e-01 3.5586169194906e-01
6.0639774872543e-01 7.7708855499331e-01 3.4450375262316e-01
2.6660643167181e-01 7.7979058670527e-01 3.3314565012318e-01
9.2696120434995e-01 7.8249251723216e-01 3.2178744408667e-01
5.8744278340550e-01 7.8519433902587e-01 3.1042918160552e-01
2.4803641143895e-01 7.8789604671643e-01 2.9907090029830e-01
9.0873073350535e-01 7.9059763659095e-01 2.8771263052114e-01
5.6951699559008e-01 7.9329910621136e-01 2.7635439697773e-01
2.3038846194336e-01 7.9600045413066e-01 2.6499621990336e-01
8.9133998410807e-01 7.9870167968101e-01 2.5363811594222e-01
5.5236767692682e-01 8.0140278281601e-01 2.4228009879989e-01
2.1346867116302e-01 8.0410376399502e-01 2.3092217972812e-01
8.7464092173640e-01 8.0680462410167e-01 2.1956436788218e-01
5.3588305680538e-01 8.0950536439116e-01 2.0820667057913e-01
1.9719425714844e-01 8.1220598646298e-01 1.9684909347786e-01
8.5857415820238e-01 8.1490649225684e-01 1.8549164069535e-01
5.2002276914655e-01 8.1760688407058e-01 1.7413431487037e-01
1.8154040486498e-01 8.2030716459951e-01 1.6277711718277e-01
8.4312762765875e-01 8.2300733699654e-01 1.5142004733457e-01
5.0478519634408e-01 8.2570740495327e-01 1.4006310349824e-01
1.6651402094206e-01 8.2840737280145e-01 1.2870628223666e-01
8.2831512160270e-01 8.3110724563446e-01 1.1734957839907e-01
4.9018959074780e-01 8.3380702944790e-01 1.0599298499765e-01
1.5213855769490e-01 8.3650673129774e-01 9.4636493069479e-02
8.1416315525560e-01 8.3920635947390e-01 8.3280091529585e-02
4.7626448800166e-01 8.4190592368601e-01 7.1923767021573e-02
1.3844360206850e-01 8.4460543525744e-01 6.0567503773146e-02
8.0070145652434e-01 8.4730490732216e-01 4.9211283465086e-02
4.6303889647535e-01 8.5000435501808e-01 3.7855085123073e-02
1.2545662820220e-01 8.5270379566933e-01 2.6498885042641e-02
7.8795519672798e-01 8.5540324894875e-01 1.5142656758060e-02
4.5053496629710e-01 8.5810273701113e-01 3.7863710661177e-03
1.1239800455113e-01 8.6080453893940e-01 9.9242733451090e-01
7.7512649174116e-01 8.6350426106019e-01 9.8107073687100e-01
4.3793582729089e-01 8.6620409838224e-01 9.6971398050813e-01
1.0082553395016e-01 8.6890408337725e-01 9.5835702695350e-01
7.6379491764086e-01 8.7160425087840e-01 9.4699983626164e-01
4.2684307525841e-01 8.7430463784019e-01 9.3564236752349e-01
8.9968907699889e-02 8.7700528301544e-01 9.2428457946106e-01
7.5317113778510e-01 8.7970622655075e-01 9.1292643109251e-01
4.1644833248784e-01 8.8240750950571e-01 9.0156788245189e-01
7.9798928655185e-02 8.8510917330435e-01 8.9020889534368e-01
7.4322126118546e-01 8.8781125913170e-01 8.7884943410880e-01
4.0671359248091e-01 8.9051380729096e-01 8.6748946637629e-01
7.0274141905962e-02 8.9321685653958e-01 8.5612896377394e-01
7.3390111398000e-01 8.9592044342428e-01 8.4476790257148e-01
3.9759272411790e-01 8.9862460163558e-01 8.3340626423168e-01
6.1347220900960e-02 9.0132936140228e-01 8.2204403584799e-01
7.2516290410313e-01 9.0403474894498e-01 8.1068121045139e-01
3.8903813799521e-01 9.0674078600574e-01 7.9931778717452e-01
5.2971359779912e-02 9.0944748946816e-01 7.8795377126642e-01
7.1696108334585e-01 9.1215487107900e-01 7.7658917395720e-01
3.8100589884430e-01 9.1486293727894e-01 7.6522401217731e-01
4.5104468860998e-02 9.1757168914648e-01 7.5385830814137e-01
7.0925552215688e-01 9.2028112245533e-01 7.4249208881088e-01
3.7345784606959e-01 9.2299122784210e-01 7.3112538525411e-01
3.7710278691330e-02 9.2570199107800e-01 7.1975823192399e-01
7.0201170186619e-01 9.2841339343458e-01 7.0839066587734e-01
3.6636103588419e-01 9.3112541213119e-01 6.9702272595959e-01
3.0757236502309e-02 9.3383802084859e-01 6.8565445197962e-01
6.9519929461150e-01 9.3655119029109e-01 6.7428588389894e-01
3.5968623851649e-01 9.3926488877750e-01 6.6291706105813e-01
2.4217138430077e-02 9.4197908283955e-01 6.5154802146160e-01
6.8879111243444e-01 9.4469373780567e-01 6.4017880113937e-01
3.5340733285941e-01 9.4740881834777e-01 6.2880943360150e-01
1.8065031799020e-02 9.5012428896944e-01 6.1743994939784e-01
6.8276350435609e-01 9.5284011441528e-01 6.0607037579235e-01
3.4750210818642e-01 9.5555625998388e-01 5.9470073655802e-01
1.2280258019240e-02 9.5827269172994e-01 5.8333105189560e-01
6.7709741405755e-01 9.6098937654545e-01 5.7196133847651e-01
3.4195306350389e-01 9.6370628211456e-01 5.6059160960824e-01
6.8466948974627e-03 9.6642337674218e-01 5.4922187551868e-01
6.7177776546343e-01 9.6914062906235e-01 5.3785214375410e-01
3.3674566223575e-01 9.7185800763856e-01 5.2648241968415e-01
1.7496582852374e-03 9.7457548047421e-01 5.1511270710548e-01
2.5157313711604e-01 7.4537012245938e-01 5.0346032662951e-01
7.4591285845276e-01 7.4792372775989e-01 4.9209693235298e-01
3.6870590385577e-01 7.5061103967945e-01 4.8078157414334e-01
1.5236531359209e-02 7.5330917813882e-01 4.6944742982579e-01
6.6877653386060e-01 7.5601014818112e-01 4.5810142433525e-01
3.2526715602853e-01 7.5871232612187e-01 4.4674951035218e-01
9.8327097164624e-01 7.6141507598168e-01 4.3539441978227e-01
6.4215509793651e-01 7.6411809585491e-01 4.2403749138630e-01
3.0159870491280e-01 7.6682122527432e-01 4.1267944376599e-01
9.6142232271979e-01 7.6952437190730e-01 4.0132069365485e-01
6.2151803297154e-01 7.7222747945685e-01 3.8996149832570e-01
2.8181729133058e-01 7.7493051211728e-01 3.7860202488134e-01
9.4227468638223e-01 7.7763344643290e-01 3.6724238649460e-01
6.0285913543929e-01 7.8033626675406e-01 3.5588266254661e-01
2.6354882822731e-01 7.8303896256293e-01 3.4452291040149e-01
9.2432818108720e-01 7.8574152682692e-01 3.3316317259318e-01
5.8518592705628e-01 7.8844395494411e-01 3.2180348137559e-01
2.4611387633773e-01 7.9114624404396e-01 3.1044386169467e-01
9.0710608747052e-01 7.9384839250914e-01 2.9908433318219e-01
5.6815829834065e-01 7.9655039963974e-01 2.8772491152357e-01
2.2926752625615e-01 7.9925226541207e-01 2.7636560941343e-01
8.9043178074479e-01 8.0195399030264e-01 2.6500643723242e-01
5.5164985313198e-01 8.0465557515870e-01 2.5364740353041e-01
2.1292115939678e-01 8.0735702110348e-01 2.4228851537171e-01
8.7424562059050e-01 8.1005832946881e-01 2.3092977857915e-01
5.3562357009086e-01 8.1275950175029e-01 2.1957119790184e-01
1.9705568022852e-01 8.1546053958245e-01 2.0821277712335e-01
8.5854290299988e-01 8.1816144473231e-01 1.9685451912185e-01
5.2008642105751e-01 8.2086221911083e-01 1.8549642589005e-01
1.8168760619080e-01 8.2356286480239e-01 1.7413849852041e-01
8.4334798322600e-01 8.2626338411247e-01 1.6278073715946e-01
5.0506919778863e-01 8.2896377963453e-01 1.5142314093395e-01
1.6685298674633e-01 8.3166405433643e-01 1.4006570785122e-01
8.2870115043275e-01 8.3436421166730e-01 1.2870843467575e-01
4.9061552597260e-01 8.3706425568507e-01 1.1735131678402e-01
1.5259796120683e-01 8.3976419120469e-01 1.0599434800036e-01
8.1465028886884e-01 8.4246402396656e-01 9.4637520416971e-02
4.7677430080008e-01 8.4516376082371e-01 8.3280824202209e-02
1.3897172212228e-01 8.4786340994568e-01 7.1924247402351e-02
8.0124418540846e-01 8.5056298103570e-01 6.0567775743318e-02
4.6359320501886e-01 8.5326248555676e-01 4.9211392440129e-02
1.2602015188944e-01 8.5596193696055e-01 3.7855078023277e-02
7.8852622917853e-01 8.5866135091220e-01 2.6498810192507e-02
4.5111244928596e-01 8.6136074550186e-01 1.5142563709595e-02
1.1377961285234e-01 8.6406014143336e-01 3.7863103424651e-03
7.7575810624402e-01 8.6676173957520e-01 9.9242744791229e-01
4.3857442211084e-01 8.6946130062541e-01 9.8107098346080e-01
1.0147242626141e-01 8.7216094050052e-01 9.6971441137866e-01
7.6445192635785e-01 8.7486069138196e-01 9.5835769235967e-01
4.2751246712012e-01 8.7756058826243e-01 9.4700078500521e-01
9.0653335147815e-02 8.8026066875981e-01 9.3564364629812e-01
7.5387356957451e-01 8.8296097283749e-01 9.2428623217797e-01
4.1717197844985e-01 8.8566154242914e-01 9.1292849820931e-01
8.0547160450814e-02 8.8836242097013e-01 9.0157040032946e-01
7.4399753122109e-01 8.9106365284205e-01 8.9021189565652e-01
4.0752135335300e-01 8.9376528274162e-01 8.7885294333359e-01
7.1116768785670e-02 8.9646735498935e-01 8.6749350538128e-01
7.3478183222152e-01 8.9916991279702e-01 8.5613354752808e-01
3.9851454408139e-01 9.0187299751602e-01 8.4477303998728e-01
6.2312881540320e-02 9.0457664789015e-01 8.3341195814994e-01
7.2617482631405e-01 9.0728089933700e-01 8.2205028316619e-01
3.9009838809523e-01 9.0998578328161e-01 8.1068800239106e-01
5.4081622851450e-02 9.1269132656392e-01 7.9932510967679e-01
7.1812264556877e-01 9.1539755093900e-01 7.8796160549974e-01
3.8221963742613e-01 9.1810447268556e-01 7.7659749691664e-01
4.6370847782491e-02 9.2081210233411e-01 7.6523279735204e-01
7.1057459171846e-01 9.2352044452208e-01 7.5386752622515e-01
3.7482924416683e-01 9.2622949797876e-01 7.4250170842988e-01
3.9133231868952e-02 9.2893925563877e-01 7.3113537368730e-01
7.0348502448836e-01 9.3164970487875e-01 7.1976855579342e-01
3.6788312619191e-01 9.3436082786794e-01 7.0840129178824e-01
3.2326068871866e-02 9.3707260202002e-01 6.9703362107395e-01
6.9681240794036e-01 9.3978500052993e-01 6.8566558451104e-01
3.6134072129752e-01 9.4249799297678e-01 6.7429722352067e-01
2.5909611683213e-02 9.4521154597134e-01 6.6292857922082e-01
6.9051771220050e-01 9.4792562382464e-01 6.5155969162139e-01
3.5516369438284e-01 9.5064018921311e-01 6.4019059890077e-01
1.9846277802126e-02 9.5335520381524e-01 6.2882133678286e-01
6.8456423991638e-01 9.5607062889540e-01 6.1745193802965e-01
3.4931642466110e-01 9.5878642581200e-01 6.0608243206012e-01
1.4101748219980e-02 9.6150255642998e-01 5.9471284470226e-01
6.7891920047817e-01 9.6421898342123e-01 5.8334319808080e-01
3.4376784086521e-01 9.6693567044109e-01 5.7197351063985e-01
8.6467876242765e-03 9.6965258217445e-01 5.6060379729628e-01
6.7355519997993e-01 9.7236968425068e-01 5.4923406971753e-01
3.3849225318849e-01 9.7508694303293e-01 5.3786433671518e-01
3.4571070111784e-03 9.7780432529331e-01 5.2649460474412e-01
6.6844886871057e-01 9.8052179779189e-01 5.1512487849578e-01
2.5157135070527e-01 7.5117069604925e-01 5.0346029715251e-01
2.4589911395727e-01 7.5380732933527e-01 4.9238644904195e-01
9.5963022916888e-01 7.5652157695255e-01 4.8095865763848e-01
6.3271541041683e-01 7.5923377217884e-01 4.6956726660131e-01
2.9900768058799e-01 7.6194203040904e-01 4.5819115853995e-01
9.6299023703555e-01 7.6464870454288e-01 4.4682084052617e-01
6.2592153798995e-01 7.6735453756647e-01 4.3545334247511e-01
2.8829110060705e-01 7.7005982783731e-01 4.2408746825426e-01
9.5032894108715e-01 7.7276471767564e-01 4.1272264771974e-01
6.1215759906838e-01 7.7546928384903e-01 4.0135857565227e-01
2.7384856818378e-01 7.7817357183145e-01 3.8999507429858e-01
9.3544658827036e-01 7.8087761068199e-01 3.7863203334458e-01
5.9698129654917e-01 7.8358142020551e-01 3.6726938086594e-01
2.5847329779446e-01 7.8628501468240e-01 3.5590706810205e-01
9.1993754092364e-01 7.8898840493570e-01 3.4454506094945e-01
5.8138529988158e-01 7.9169159953471e-01 3.3318333495419e-01
2.4282538806078e-01 7.9439460552356e-01 3.2182187223195e-01
9.0426493088642e-01 7.9709742887607e-01 3.1046065950142e-01
5.6570987298491e-01 7.9980007478651e-01 2.9909968678673e-01
2.2716532014113e-01 8.0250254785889e-01 2.8773894653524e-01
8.8863577516521e-01 8.0520485223214e-01 2.7637843300068e-01
5.5012530371881e-01 8.0790699166433e-01 2.6501814179937e-01
2.1163765272507e-01 8.1060896959080e-01 2.5365806958097e-01
8.7317633592202e-01 8.1331078916640e-01 2.4229821377593e-01
5.3474469613751e-01 8.1601245329882e-01 2.3093857239356e-01
1.9634595070699e-01 8.1871396467849e-01 2.1957914385327e-01
8.5798322440879e-01 8.2141532580900e-01 2.0821992683599e-01
5.1965957293654e-01 8.2411653904168e-01 1.9686092014651e-01
1.8137799901442e-01 8.2681760661727e-01 1.8550212257952e-01
8.4314146263413e-01 8.2951853071735e-01 1.7414353278381e-01
5.0495288645711e-01 8.3221931352802e-01 1.6278514912026e-01
1.6681515712028e-01 8.3491995731823e-01 1.5142696951006e-01
8.2873112296986e-01 8.3762046453466e-01 1.4006899127045e-01
4.9070358859988e-01 8.4032083791519e-01 1.2871121093621e-01
1.5273530647417e-01 8.4302108062241e-01 1.1735362406556e-01
8.1482896585126e-01 8.4572119639825e-01 1.0599622503043e-01
4.7698717920540e-01 8.4842118974044e-01 9.4639006791769e-02
1.3921246633788e-01 8.5112106610035e-01 8.3281960662042e-02
8.0150723639956e-01 8.5382083210145e-01 7.1925076058099e-02
4.6387376809391e-01 8.5652049577595e-01 6.0568340249444e-02
1.2631418839865e-01 8.5922006681622e-01 4.9211738108397e-02
7.8883045022940e-01 8.6191955683615e-01 3.7855251870499e-02
4.5142430956610e-01 8.6461897963561e-01 2.6498860915270e-02
1.1409730266651e-01 8.6731835145997e-01 1.5142541579106e-02
7.7685072409267e-01 8.7001769124443e-01 3.7862670135378e-03
4.3894380195693e-01 8.7271911996743e-01 9.9242752835180e-01
1.0184647328815e-01 8.7541855488810e-01 9.8107114728008e-01
7.6483159538457e-01 8.7811803106848e-01 9.6971471232492e-01
4.2789932249297e-01 8.8081757991125e-01 9.5835818389473e-01
9.1049499554984e-02 8.8351723606415e-01 9.4700151967924e-01
7.5428166308895e-01 8.8621703730393e-01 9.3564467506161e-01
4.1759504836421e-01 8.8891702431806e-01 9.2428760365098e-01
8.0988603061776e-02 8.9161724037781e-01 9.1293025793150e-01
7.4446100728854e-01 8.9431773090116e-01 9.0157259001759e-01
4.0801069945267e-01 8.9701854290848e-01 8.9021455249860e-01
7.1635907143740e-02 8.9971972437986e-01 8.7885609934929e-01
7.3533468182387e-01 9.0242132352780e-01 8.6749718687724e-01
3.9910493585890e-01 9.0512338800454e-01 8.5613777467347e-01
6.2944480232314e-02 9.0782596406722e-01 8.4477782653057e-01
7.2685106121376e-01 9.1052909572720e-01 8.3341731129134e-01
3.9082239431144e-01 9.1323282391160e-01 8.2205620359318e-01
5.4856194026502e-02 9.1593718566546e-01 8.1069448447679e-01
7.1895019823420e-01 9.1864221342148e-01 7.9933214183334e-01
3.8310218641917e-01 9.2134793436190e-01 7.8796917067125e-01
4.7309991455122e-02 9.2405436989334e-01 7.7660557319156e-01
7.1157150510448e-01 9.2676153525112e-01 7.6524135866885e-01
3.7588467787826e-01 9.2946943924452e-01 7.5387654314306e-01
4.0247514301150e-02 9.3217808414959e-01 7.4251114893474e-01
7.0465806493753e-01 9.3488746575066e-01 7.3114520400286e-01
3.6911441672371e-01 9.3759757352709e-01 7.1977874117009e-01
3.3614683204920e-02 9.4030839097702e-01 7.0841179724407e-01
6.9815699618635e-01 9.4301989606529e-01 6.9704441206667e-01
3.6273950008327e-01 9.4573206177891e-01 6.8567662752435e-01
2.7360349912978e-02 9.4844485676975e-01 6.7430848655329e-01
6.9201771343895e-01 9.5115824606101e-01 6.6294003217166e-01
3.5670977749214e-01 9.5387219179151e-01 6.5157130656979e-01
2.1434757997106e-02 9.5658665397024e-01 6.4020235028513e-01
6.8619091276623e-01 9.5930159121297e-01 6.2883320148538e-01
3.5097655573728e-01 9.6201696143298e-01 6.1746389537813e-01
1.5790071053177e-02 9.6473272245982e-01 6.0609446376026e-01
6.8062992524723e-01 9.6744883256274e-01 5.9472493471487e-01
3.4549467581642e-01 9.7016525085957e-01 5.8335533245859e-01
1.0382974640295e-02 9.7288193759687e-01 5.7198567733725e-01
6.7529356502310e-01 9.7559885429316e-01 5.6061598596377e-01
3.4022527157525e-01 9.7831596374312e-01 5.4924627148889e-01
5.1769826756366e-03 9.8103322988765e-01 5.3787654399283e-01
6.7014762582659e-01 9.8375061756087e-01 5.2650681098413e-01
3.3513613678801e-01 9.8646809213159e-01 5.1513707799038e-01
9.7096116690474e-01 7.5716597728563e-01 5.0344834539472e-01
7.4588549953415e-01 7.5998047720322e-01 4.9209687280987e-01
5.2082162388596e-01 7.6252086518685e-01 4.8095862757365e-01
2.3455261878299e-01 7.6517082503061e-01 4.6965941160900e-01
9.2012000351874e-01 7.6787405530902e-01 4.5828097226480e-01
5.9482687057350e-01 7.7058313743568e-01 4.4689742917499e-01
2.6448151861383e-01 7.7329187293203e-01 4.3551803672339e-01
9.3142713701548e-01 7.7599963401899e-01 4.2414271180950e-01
5.9676781371550e-01 7.7870653337363e-01 4.1277045597755e-01
2.6108654701183e-01 7.8141274210201e-01 4.0140045285620e-01
9.2471871930162e-01 7.8411839546287e-01 3.9003213091022e-01
5.8787054696148e-01 7.8682359081118e-01 3.7866509833455e-01
2.5067576805072e-01 7.8952839754309e-01 3.6729908369914e-01
9.1322500595660e-01 7.9223286581066e-01 3.5593389535713e-01
5.7558198611834e-01 7.9493703263669e-01 3.4456939519545e-01
2.3779297823420e-01 7.9764092599432e-01 3.3320548172148e-01
8.9989254297690e-01 8.0034456751091e-01 3.2184207902307e-01
5.6190716360460e-01 8.0304797427469e-01 3.1047912942735e-01
2.2385761626194e-01 8.0575116005773e-01 2.9911658852082e-01
8.8576056094276e-01 8.0845413615616e-01 2.8775442170319e-01
5.4762963574952e-01 8.1115691197578e-01 2.7639260175638e-01
2.0947622580354e-01 8.1385949544620e-01 2.6503110709736e-01
8.7131001378576e-01 8.1656189331836e-01 2.5366992049919e-01
5.3313938065817e-01 8.1926411138199e-01 2.4230902813711e-01
1.9497170151881e-01 8.2196615462844e-01 2.3094841886289e-01
8.5681356668447e-01 8.2466802737656e-01 2.1958808364056e-01
5.1867094852189e-01 8.2736973337457e-01 2.0822801509647e-01
1.8054932825170e-01 8.3007127588763e-01 1.9686820714998e-01
8.4245379272771e-01 8.3277265777855e-01 1.8550865470010e-01
5.0438910831584e-01 8.3547388158765e-01 1.7414935334950e-01
1.6635977700226e-01 8.3817494961682e-01 1.6279029915211e-01
8.2837007845973e-01 8.4087586402205e-01 1.5143148837326e-01
4.9042410080582e-01 8.4357662691810e-01 1.4007291725423e-01
1.5252576207377e-01 8.4627724049867e-01 1.2871458177446e-01
8.1467882390443e-01 8.4897770717477e-01 1.1735647740673e-01
4.7688689860118e-01 8.5167802973371e-01 1.0599859886187e-01
1.3915345043204e-01 8.5437821152052e-01 9.4640939820900e-02
8.0148179188954e-01 8.5707825664271e-01 8.3283492654312e-02
4.6387507551196e-01 8.5977817019874e-01 7.1926248129478e-02
1.2633628181696e-01 8.6247795852918e-01 6.0569195109142e-02
7.8886820389230e-01 8.6517762948848e-01 4.9212320245848e-02
4.5147342921977e-01 8.6787719273349e-01 3.7855607679239e-02
1.1415431937160e-01 8.7057666002338e-01 2.6499038745371e-02
7.7691298830518e-01 8.7327604552339e-01 1.5142591709416e-02
4.3975128008295e-01 8.7597536610298e-01 3.7862415352142e-03
1.0195778662563e-01 8.7867666118586e-01 9.9242757473347e-01
7.6494502100435e-01 8.8137600678373e-01 9.8107122505574e-01
4.2801535832424e-01 8.8407535532833e-01 9.6971487733348e-01
9.1169363440934e-02 8.8677473692714e-01 9.5835849231975e-01
7.5440724660246e-01 8.8947418533427e-01 9.4700202742979e-01
4.1772885941198e-01 8.9217373792048e-01 9.3564543706727e-01
8.1133697319084e-02 8.9487343553494e-01 9.2428867308795e-01
7.4462090919831e-01 8.9757332224783e-01 9.1293168540268e-01
4.0818931423353e-01 9.0027344496704e-01 9.0157442271432e-01
7.1837425924369e-02 9.0297385292727e-01 8.9021683337596e-01
7.3556348259366e-01 9.0567459705657e-01 8.7885886634915e-01
3.9936548333656e-01 9.0837572923100e-01 8.6750047223367e-01
6.3241227954080e-02 9.1107730143550e-01 8.5614160433342e-01
7.2718835909768e-01 9.1377936485424e-01 8.4478221971852e-01
3.9120440465457e-01 9.1648196891885e-01 8.3342228024059e-01
5.5286818353761e-02 9.1918516034632e-01 8.2206175345869e-01
7.1943301668389e-01 9.2188898219976e-01 8.1070061343538e-01
3.8364041048416e-01 9.2459347300521e-01 7.9933884136822e-01
4.7906429978714e-02 9.2729866595548e-01 7.8797642602839e-01
7.1222854253899e-01 9.3000458822854e-01 7.7661336398780e-01
3.7660426303260e-01 9.3271126044342e-01 7.6524965962479e-01
4.1031157199279e-02 9.3541869627064e-01 7.5388532490915e-01
7.0550683903292e-01 9.3812690220870e-01 7.4252037897609e-01
3.7002896359546e-01 9.4083587753148e-01 7.3115484750765e-01
3.4595217003858e-02 9.4354561440590e-01 7.1978876194731e-01
6.9920330548958e-01 9.4625609817294e-01 7.0842215857925e-01
3.6385094541410e-01 9.4896730778012e-01 6.9705507750821e-01
2.8535855938284e-02 9.5167921634825e-01 6.8568756157821e-01
6.9325575570014e-01 9.5439179185084e-01 6.7431965526963e-01
3.5800836438590e-01 9.5710499788066e-01 6.6295140361329e-01
2.2791409523697e-02 9.5981879447476e-01 6.5158285115830e-01
6.8760263823700e-01 9.6253313896678e-01 6.4021404102684e-01
3.5243983312138e-01 9.6524798683444e-01 6.2884501408421e-01
1.7300830906782e-02 9.6796329251000e-01 6.1747580824716e-01
6.8218354218593e-01 9.7067901012335e-01 6.0610645794693e-01
3.4708597026559e-01 9.7339509415003e-01 5.9473699375674e-01
1.2006227152117e-02 9.7611149994132e-01 5.8336744218727e-01
6.7694254482132e-01 9.7882818411897e-01 5.7199782564733e-01
3.4189328023100e-01 9.8154510482388e-01 5.6062816256163e-01
6.8569129859394e-03 9.8426222181499e-01 5.4925846763347e-01
6.7183203511646e-01 9.8697949642223e-01 5.3788875223674e-01
3.3681733303886e-01 9.8969689136417e-01 5.2651902491959e-01
1.8115623760373e-03 9.9241437044791e-01 5.1514929200078e-01
6.4969315171375e-01 7.6313219629984e-01 5.0344980765781e-01
3.7431353939906e-01 7.6591697003251e-01 4.9208517698490e-01
1.1174620480320e-01 7.6860851568306e-01 4.8078148395997e-01
8.3638995267858e-01 7.7120005307799e-01 4.6956720589877e-01
5.3763850785629e-01 7.7384602607145e-01 4.5828094162839e-01
2.2320577667711e-01 7.7653421574403e-01 4.4693258339499e-01
9.0017861240489e-01 7.7923649170528e-01 4.3556288548126e-01
5.7224947691909e-01 7.8194271867062e-01 4.2418813117062e-01
2.4132964142589e-01 7.8464977806913e-01 4.1281339376097e-01
9.0847498441611e-01 7.8735671875663e-01 4.0144005152839e-01
5.7430762031448e-01 7.9006326736842e-01 3.9006832795644e-01
2.3921487732078e-01 7.9276936901769e-01 3.7869810668987e-01
9.0344937343435e-01 7.9547503824816e-01 3.6732919377362e-01
5.6718248449408e-01 7.9818030869819e-01 3.5596139979575e-01
2.3053446544418e-01 8.0088521603310e-01 3.4459456212711e-01
8.9359221543575e-01 8.0358979255671e-01 3.3322854744741e-01
5.5642017993630e-01 8.0629406597324e-01 3.2186324835028e-01
2.1906727641822e-01 8.0899805956869e-01 3.1049857875179e-01
8.8157142909510e-01 8.1170179280446e-01 2.9913446962693e-01
5.4396261825269e-01 8.1440528195879e-01 2.8777086546490e-01
2.0626497961677e-01 8.1710854069585e-01 2.7640772144817e-01
8.6849828027730e-01 8.1981158053482e-01 2.6504500124707e-01
5.3067897585016e-01 8.2251441122409e-01 2.5368267530452e-01
1.9282098039633e-01 8.2521704103614e-01 2.4232071949956e-01
8.5493623551069e-01 8.2791947699957e-01 2.3095911409947e-01
5.1703513650820e-01 8.3062172508350e-01 2.1959784292973e-01
1.7912685525044e-01 8.3332379034706e-01 2.0823689270776e-01
8.4121958705158e-01 8.3602567706451e-01 1.9687625249850e-01
5.0332074098518e-01 8.3872738883482e-01 1.8551591326009e-01
1.6543708737725e-01 8.4142892868293e-01 1.7415586745460e-01
8.2757487243659e-01 8.4413029915900e-01 1.6279610870441e-01
4.8973990727893e-01 8.4683150244104e-01 1.5143663147888e-01
1.5193763668452e-01 8.4953254044567e-01 1.4007743079905e-01
8.1417319154953e-01 8.5223341495138e-01 1.2871850195040e-01
4.7645142799187e-01 8.5493412773802e-01 1.1735984019597e-01
1.3877695534322e-01 8.5763468074593e-01 1.0600144048378e-01
8.0115415472954e-01 8.6033507625760e-01 9.4643297143954e-02
4.6358718956180e-01 8.6303531710405e-01 8.3285403572937e-02
1.2608000899280e-01 8.6573540689747e-01 7.1927751903583e-02
7.8863634522174e-01 8.6843535029055e-01 6.0570332661977e-02
4.5125970542783e-01 8.7113515326179e-01 4.9213134413835e-02
1.1395335907676e-01 8.7383482342443e-01 3.7856143405674e-02
7.7672032135755e-01 8.7653437035516e-01 2.6499343208417e-02
4.3956333356475e-01 8.7923380593622e-01 1.5142714373785e-02
1.0248484133082e-01 8.8193314470252e-01 3.7862341164844e-03
7.6480331993818e-01 8.8463434287927e-01 9.9242758641224e-01
4.2787298833167e-01 8.8733363743503e-01 9.8107121452932e-01
9.1026265524846e-02 8.9003289639339e-01 9.6971490182660e-01
7.5426419206838e-01 8.9273214803329e-01 9.5835861010301e-01
4.1758741117783e-01 8.9543142463214e-01 9.4700229722837e-01
8.0996158847019e-02 8.9813076253647e-01 9.3564591735940e-01
7.4449026041908e-01 9.0083020212264e-01 9.2428942130648e-01
4.0806913459255e-01 9.0352978763113e-01 9.1293275706003e-01
7.1731805513965e-02 9.0622956686189e-01 9.0157587047814e-01
7.3547692318825e-01 9.0892959072318e-01 8.9021870612847e-01
3.9930279194424e-01 9.1162991263302e-01 8.7886120826817e-01
6.3207406160037e-02 9.1433058777989e-01 8.6750332193564e-01
7.2718849192676e-01 9.1703167225701e-01 8.5614499411900e-01
3.9124355286126e-01 9.1973322209261e-01 8.4478617495838e-01
5.5369917921961e-02 9.2243529220516e-01 8.3342681893391e-01
7.1956478888041e-01 9.2513793531829e-01 8.2206688598852e-01
3.8382528508934e-01 9.2784120087349e-01 8.1070634253573e-01
4.8148483375300e-02 9.3054513398010e-01 7.9934516230629e-01
7.1253145126397e-01 9.3324977444121e-01 7.8798332699478e-01
3.7697127228702e-01 9.3595515589093e-01 7.7662082667649e-01
4.1465062773575e-02 9.3866130507397e-01 7.6525765997604e-01
7.0600998023542e-01 9.4136824129192e-01 7.5389383398107e-01
3.7060322414769e-01 9.4407597603421e-01 7.4252936390591e-01
3.5242030540720e-02 9.4678451280395e-01 7.3116427252139e-01
6.9992366229834e-01 9.4949384714178e-01 7.1979858937662e-01
3.6464539735873e-01 9.5220396684362e-01 7.0843234984651e-01
2.9404517109363e-02 9.5491485236156e-01 6.9706559404501e-01
6.9419829714841e-01 9.5762647737080e-01 6.8569836564805e-01
3.5902400226233e-01 9.6033880947998e-01 6.7433071067219e-01
2.3878886970553e-02 9.6305181105717e-01 6.6296267625520e-01
6.8876020236009e-01 9.6576544013961e-01 6.5159430948256e-01
3.5366520923278e-01 9.6847965139222e-01 6.4022565630066e-01
1.8591196882357e-02 9.7119439707776e-01 6.2885676055160e-01
6.8353550617678e-01 9.7390962800161e-01 6.1748766315844e-01
3.4849555510431e-01 9.7662529439505e-01 6.0611840148180e-01
1.3468864602147e-02 9.7934134670430e-01 5.9474900886064e-01
6.7845308235504e-01 9.8205773625751e-01 5.8337951434197e-01
3.4344600233899e-01 9.8477441578793e-01 5.7200994259649e-01
8.4455781795261e-03 9.8749133979942e-01 5.6064031401041e-01
6.7344992886244e-01 9.9020846476815e-01 5.4927064493816e-01
3.3845733592968e-01 9.9292574918299e-01 5.3790094809661e-01
3.4662319557647e-03 9.9564315343487e-01 5.2653123307887e-01
6.6847518077209e-01 9.9836063957271e-01 5.1516150696415e-01
3.2162007450190e-01 7.6909047109152e-01 5.0345148406326e-01
2.6467419783696e-02 7.7185368789747e-01 4.9208639630699e-01
7.4016257766295e-01 7.7459054824556e-01 4.8073460136797e-01
4.5386920832697e-01 7.7724451374359e-01 4.6944730844302e-01
1.5875107729841e-01 7.7986789500503e-01 4.5819106664187e-01
8.5158480376027e-01 7.8252047902832e-01 4.4689736734563e-01
5.3488587648140e-01 7.8520162959236e-01 4.3556285428962e-01
2.1185860069950e-01 7.8789750353372e-01 4.2420596032215e-01
8.8469988295245e-01 7.9059979874485e-01 4.1283926188871e-01
5.5477028499410e-01 7.9330469709750e-01 4.0146892241404e-01
2.2291897213263e-01 7.9601053028146e-01 3.9009769072754e-01
8.8969284464853e-01 7.9871657110027e-01 3.7872673328745e-01
5.5545634797783e-01 8.0142250340755e-01 3.6735650784667e-01
2.2046019967765e-01 8.0412819398144e-01 3.5598715861980e-01
8.8488189948678e-01 8.0683359245041e-01 3.4461869419408e-01
5.4885038210627e-01 8.0953868616732e-01 3.3325106808305e-01
2.1246149304255e-01 8.1224347933980e-01 3.2188421520645e-01
8.7578798640425e-01 8.1494798320408e-01 3.1051806812397e-01
5.3888615441535e-01 8.1765221136867e-01 2.9915256386955e-01
2.0180032835726e-01 8.2035617763213e-01 2.8778764641995e-01
8.6456600020964e-01 8.2305989500171e-01 2.7642326717043e-01
5.2721202981459e-01 8.2576337529703e-01 2.6505938454639e-01
1.8976223293167e-01 8.2846662903574e-01 2.5369596328678e-01
8.5223654204662e-01 8.3116966545053e-01 2.4233297364823e-01
5.1465186710062e-01 8.3387249256282e-01 2.3097039064014e-01
1.7702274199635e-01 8.3657511727711e-01 2.1960819333262e-01
8.3936181583391e-01 8.3927754547927e-01 2.0824636424750e-01
5.0168022998644e-01 8.4197978213258e-01 1.9688488882750e-01
1.6398791008750e-01 8.4468183137030e-01 1.8552375497278e-01
8.2629379375325e-01 8.4738369658615e-01 1.7416295263218e-01
4.8860600912493e-01 8.5008538052586e-01 1.6280247343628e-01
1.5093201527193e-01 8.5278688538293e-01 1.5144231036071e-01
8.1327871260737e-01 8.5548821290259e-01 1.4008245740877e-01
4.7565252938245e-01 8.5818936449786e-01 1.2872290930429e-01
1.3805948880470e-01 8.6089034138128e-01 1.1736366118635e-01
8.0050526020828e-01 8.6359114471641e-01 1.0600470829911e-01
4.6299519688057e-01 8.6629177579213e-01 9.4646045671153e-02
1.2553436254381e-01 8.6899223622317e-01 8.3287667780050e-02
7.8812754805054e-01 8.7169252817909e-01 7.1929568199395e-02
4.5077927954015e-01 8.7439265464363e-01 6.0571739227356e-02
1.1349381909776e-01 8.7709261970509e-01 4.9214171497672e-02
7.7627515883558e-01 8.7979242887693e-01 3.7856853576387e-02
4.3912700926966e-01 8.8249208944640e-01 2.6499771550194e-02
1.0205278287970e-01 8.8519161084652e-01 1.5142908615200e-02
7.6505557380616e-01 8.8789100504466e-01 3.7862446780198e-03
4.2748425607993e-01 8.9059214345447e-01 9.9242756327867e-01
9.0633880839579e-02 8.9329142623042e-01 9.8107111467186e-01
7.5386743523906e-01 8.9599063522179e-01 9.6971478300697e-01
4.1718650009694e-01 8.9868979634266e-01 9.5835853188928e-01
8.0592220412560e-02 9.0138893978949e-01 9.4700232045481e-01
7.4408528883244e-01 9.0408810022463e-01 9.3564610345294e-01
4.0766593511563e-01 9.0678731685318e-01 9.2428983148777e-01
7.1333923025004e-02 9.0948663337132e-01 9.1293345143491e-01
7.3508855579610e-01 9.1218609776675e-01 9.0157690704399e-01
3.9892869093185e-01 9.1488576195666e-01 8.9022013972799e-01
6.2852764552060e-02 9.1758568125512e-01 8.7886308953044e-01
7.2685882491973e-01 9.2028591366981e-01 8.6750569624954e-01
3.9094457411681e-01 9.2298651903721e-01 8.5614790068687e-01
5.5107416210874e-02 9.2568755801504e-01 8.4478964597734e-01
7.1934450970573e-01 9.2838909096005e-01 8.3343087894816e-01
3.8365282167170e-01 9.3109117672715e-01 8.2207155144881e-01
4.8029180757533e-02 9.3379387143237e-01 8.1071162159180e-01
7.1247032633270e-01 9.3649722722534e-01 7.9935105484596e-01
3.7697295130476e-01 9.3920129111833e-01 7.8798982492973e-01
4.1533736682287e-02 9.4190610391675e-01 7.7662791446172e-01
7.0614937666435e-01 9.4461169929137e-01 7.6526531533759e-01
3.7081659386348e-01 9.4731810302670e-01 7.5390202881639e-01
3.5532145131932e-02 9.5002533247157e-01 7.4253806531382e-01
7.0029281927476e-01 9.5273339620953e-01 7.3117344391394e-01
3.6509542861989e-01 9.5544229395784e-01 7.1980819162359e-01
2.9936796909424e-02 9.5815201669501e-01 7.0844234240481e-01
6.9481374623009e-01 9.6086254700832e-01 6.9707593602890e-01
3.5972308569241e-01 9.6357385964505e-01 6.8570901680230e-01
2.4661604296099e-02 9.6628592224408e-01 6.7434163221779e-01
6.8962606991061e-01 9.6899869621791e-01 6.6297383158559e-01
3.5461323560190e-01 9.7171213774977e-01 6.5160566469782e-01
1.9619853716881e-02 9.7442619886626e-01 6.4023718057570e-01
6.8464269727633e-01 9.7714082854300e-01 6.2886842634336e-01
3.4967858741127e-01 9.7985597379981e-01 6.1749944626425e-01
1.4724424889267e-02 9.8257158074283e-01 6.0613028096723e-01
6.7977722328555e-01 9.8528759551409e-01 5.9476096687952e-01
3.4483414110596e-01 9.8800396511447e-01 5.8339153587351e-01
9.8925101855643e-03 9.9072063807295e-01 5.7202201512484e-01
6.7494985795806e-01 9.9343756494369e-01 5.6065242717049e-01
3.4000392168193e-01 9.9615469862182e-01 5.4928279014851e-01
5.0526533604692e-03 9.9887199447872e-01 5.3791311819587e-01
6.7090107299168e-01 1.5855675378202e-03 5.2654782461068e-01
3.3593494643954e-01 4.3030572418747e-03 5.1517811900754e-01
9.9122477378282e-01 7.7504564215670e-01 5.0345354681880e-01
6.8561875666971e-01 7.7779623661520e-01 4.9208841151181e-01
3.8599573641734e-01 7.8053901906818e-01 4.8072618499067e-01
8.8583292387849e-02 7.8323787353600e-01 4.6939257581542e-01
7.8898271705351e-01 7.8588954140756e-01 4.5810127120897e-01
4.8342180023941e-01 7.8853153099725e-01 4.4682071688977e-01
1.7058320767707e-01 7.9119112799671e-01 4.3551794315961e-01
8.5146784090253e-01 7.9387014885596e-01 4.2418806824717e-01
5.2766992877146e-01 7.9656189858378e-01 4.1283923015823e-01
2.0051110392576e-01 7.9926069149719e-01 4.0147953623401e-01
8.7093106817656e-01 8.0196312157232e-01 3.9011424228544e-01
5.3957702527106e-01 8.0466734044770e-01 3.7874633563817e-01
2.0689755361637e-01 8.0737237138878e-01 3.6737742915898e-01
8.7320919422325e-01 8.1007769988579e-01 3.5600836954273e-01
5.3873998874534e-01 8.1278305462964e-01 3.4463958815837e-01
2.0365762730552e-01 8.1548829331380e-01 3.3327129334888e-01
8.6808783707728e-01 8.1819334290277e-01 3.2190357442280e-01
5.3212660519553e-01 8.2089816795360e-01 3.1053645790991e-01
1.9584845939749e-01 8.2360275352647e-01 2.9916993812372e-01
8.5931218438196e-01 8.2630709580941e-01 2.8780399381504e-01
5.2256483912377e-01 8.2901119689882e-01 2.7643859722766e-01
1.8564462768921e-01 8.3171506185995e-01 2.6507371896610e-01
8.4858298277122e-01 8.3441869705948e-01 2.5370933053972e-01
5.1140609962811e-01 8.3712210921809e-01 2.4234540561394e-01
1.7413608036835e-01 8.3982530487583e-01 2.3098192054416e-01
8.3679179793613e-01 8.4252829009670e-01 2.1961885451606e-01
4.9938955568653e-01 8.4523107031336e-01 2.0825618947441e-01
1.6194359594895e-01 8.4793365025519e-01 1.9689390994282e-01
8.2446649563673e-01 8.5063603392729e-01 1.8553200279035e-01
4.8696947634753e-01 8.5333822462228e-01 1.7417045697531e-01
1.4946264895843e-01 8.5604022495519e-01 1.6280926328041e-01
8.1195520743631e-01 8.5874203691693e-01 1.5144841404527e-01
4.7445558278848e-01 8.6144366194503e-01 1.4008790289685e-01
1.3697156532074e-01 8.6414510101208e-01 1.2872772447557e-01
7.9951040134707e-01 8.6684635473395e-01 1.1736787415358e-01
4.6207886899770e-01 8.6954742350009e-01 1.0600834774075e-01
1.2468333665823e-01 8.7224830762908e-01 9.4649141174206e-02
7.8732980674070e-01 8.7494900755255e-01 8.3290250187251e-02
4.5002394687015e-01 8.7764952403034e-01 7.1931669954752e-02
1.1277111011795e-01 8.8034985839964e-01 6.0573394712801e-02
7.7557634559278e-01 8.8305001285992e-01 4.9215417352311e-02
4.3844440048893e-01 8.8574999079447e-01 3.7857728988085e-02
1.0137971457228e-01 8.8844979712794e-01 2.6500318507379e-02
7.6438640804553e-01 8.9114943871726e-01 1.5143172104877e-02
4.2746826376529e-01 8.9384892477101e-01 3.7862728143592e-03
9.0005064042568e-02 8.9655004031863e-01 9.9242750583895e-01
7.5323181324209e-01 8.9924935096682e-01 9.8107092589276e-01
4.1654258719403e-01 9.0194855065086e-01 9.6971452020242e-01
7.9939569216675e-02 9.0464766239739e-01 9.5835825498207e-01
7.4342447431346e-01 9.0734671371359e-01 9.4700209151229e-01
4.0699852542348e-01 9.1004573689193e-01 9.3564598606943e-01
7.0662434754204e-02 9.1274476921767e-01 9.2428989001437e-01
7.3441639207216e-01 9.1544385305173e-01 9.1293375006609e-01
3.9826006164229e-01 9.1814303576293e-01 9.0157750877988e-01
6.2192589165927e-02 9.2084236948665e-01 8.9022110524192e-01
7.2621261956781e-01 9.2354191069324e-01 8.7886447598099e-01
3.9031832583894e-01 9.2624171955729e-01 8.6750755608512e-01
5.4507448394346e-02 9.2894185912894e-01 8.5615028049686e-01
7.1877734361214e-01 9.3164239431992e-01 8.4479258544670e-01
3.8312503946250e-01 9.3434339072858e-01 8.3343440997085e-01
4.7547295495891e-02 9.3704491333942e-01 8.2207569745003e-01
7.1204066402382e-01 9.3974702514184e-01 8.1071639709899e-01
3.7660154915684e-01 9.4244978572005e-01 7.9935646533541e-01
4.1226260501766e-02 9.4515324986934e-01 7.8799586696042e-01
7.0591105876660e-01 9.4785746629421e-01 7.7663457609117e-01
3.7065219124114e-01 9.5056247644043e-01 7.6527257679903e-01
3.5445916046362e-02 9.5326831350708e-01 7.5390986342221e-01
7.0028851508330e-01 9.5597500167573e-01 7.4254644053909e-01
3.6517629665454e-01 9.5868255558436e-01 7.3118232260605e-01
3.0105589674301e-02 9.6139098006268e-01 7.1981753328006e-01
6.9507273212936e-01 9.6410027013458e-01 7.0845210446112e-01
3.6007405692951e-01 9.6681041128331e-01 6.9708607510146e-01
2.5105878446423e-02 9.6952137996486e-01 6.8571948983756e-01
6.9016448286705e-01 9.7223314434588e-01 6.7435239750698e-01
3.5524612500019e-01 9.7494566523457e-01 6.6298484961463e-01
2.0347033420278e-02 9.7765889716539e-01 6.5161689881249e-01
6.8546342488633e-01 9.8037278959309e-01 6.4024859745365e-01
3.5059152793344e-01 9.8308728814703e-01 6.2887999627500e-01
1.5727614506906e-02 9.8580233589478e-01 6.1751114325427e-01
6.8086803760311e-01 9.8851787456452e-01 6.0614208267665e-01
3.4600927219073e-01 9.9123384567835e-01 5.9477285443407e-01
1.1147956281143e-02 9.9395019155450e-01 5.8340349356783e-01
6.7628092893236e-01 9.9666685614441e-01 5.7203403005316e-01
3.4140526220474e-01 9.9938378568033e-01 5.6066448881342e-01
7.3186798965179e-03 2.0971156810278e-03 5.4929925890812e-01
6.7242004744850e-01 4.8144128014405e-03 5.3792963186260e-01
3.3750542904012e-01 7.5318327728783e-03 5.2655997233004e-01
2.5729568711091e-03 1.0249333412665e-02 5.1519028888172e-01
6.5976524382621e-01 7.8099921687665e-01 5.0345605807820e-01
3.4770851299605e-01 7.8374180160816e-01 4.9209082320187e-01
3.9879044566643e-02 7.8648234373854e-01 4.8072594653400e-01
7.3440519410785e-01 7.8920033723178e-01 4.6937350856183e-01
4.2894222010447e-01 7.9188257519019e-01 4.5804890697303e-01
1.2114548615682e-01 7.9453926967976e-01 4.4674932495337e-01
8.0914366311523e-01 7.9719318815418e-01 4.3545318657268e-01
4.9229053007659e-01 7.9985868385696e-01 4.2414258598493e-01
1.7104039858161e-01 8.0253781880858e-01 4.1281329858069e-01
8.4625203471594e-01 8.0522733155974e-01 4.0146885842938e-01
5.1874343787865e-01 8.0792354714133e-01 3.9011421003259e-01
1.8916329944777e-01 8.1062378367802e-01 3.7875330308198e-01
8.5799663967521e-01 8.1332633330683e-01 3.6738879973712e-01
5.2560135687238e-01 8.1603016192285e-01 3.5602237260060e-01
1.9224278255426e-01 8.1873465397262e-01 3.4465503846384e-01
8.5811989800384e-01 8.2143944439779e-01 3.3328740387532e-01
5.2338400398208e-01 8.2414431662805e-01 3.2191982479182e-01
1.8815180485045e-01 8.2684914231072e-01 3.1055250557099e-01
8.5251459493640e-01 8.2955384586193e-01 2.9918555927031e-01
5.1654476645566e-01 8.3225838350086e-01 2.8781904420505e-01
1.8030047439250e-01 8.3496273072430e-01 2.7645298613737e-01
8.4382902262073e-01 8.3766687473446e-01 2.6508739178663e-01
5.0716935284313e-01 8.4037080980525e-01 2.5372225707927e-01
1.7035389636997e-01 8.4307453441378e-01 2.4235757219126e-01
8.3340996779111e-01 8.4577804944578e-01 2.3099332462153e-01
4.9636082527087e-01 8.4848135706324e-01 2.1962950104848e-01
1.5922648537499e-01 8.5118445998566e-01 2.0826608842978e-01
8.2202435510194e-01 8.5388736103332e-01 1.9690307462872e-01
4.8476972628958e-01 8.5659006283946e-01 1.8554044874256e-01
1.4747616529146e-01 8.5929256767373e-01 1.7417820124146e-01
8.1015582210594e-01 8.6199487734136e-01 1.6281632398537e-01
4.7281967689184e-01 8.6469699313639e-01 1.5145481015991e-01
1.3547773727304e-01 8.6739891583610e-01 1.4009365415598e-01
7.9813919651487e-01 8.7010064572949e-01 1.2873285140707e-01
4.6081256020033e-01 8.7280218267661e-01 1.1737239819169e-01
1.2350574720278e-01 8.7550352619777e-01 1.0601229140390e-01
7.8622616937576e-01 8.7820467559358e-01 9.4652528292421e-02
4.4898079334214e-01 8.8090563009784e-01 8.3293106167138e-02
1.1177618698065e-01 8.8360638906569e-01 7.1934022071437e-02
7.7461855262006e-01 8.8630695219990e-01 6.0575272418609e-02
4.3751374851879e-01 8.8900731981786e-01 4.9216852591476e-02
1.0046729990310e-01 8.9170749316139e-01 3.7858756505628e-02
7.6348440064017e-01 8.9440747475006e-01 2.6500976138668e-02
4.2656990652161e-01 8.9710726877773e-01 1.5143501030405e-02
8.9728321116568e-02 8.9980688154927e-01 3.7863177622528e-03
7.5237083706539e-01 9.0250801018600e-01 9.9242741528512e-01
4.1567152591777e-01 9.0520738806271e-01 9.8107065023111e-01
7.9056067098200e-02 9.0790661950333e-01 9.6971411519364e-01
7.4252730429608e-01 9.1060572415659e-01 9.5835777981776e-01
4.0608758809912e-01 9.1330472624837e-01 9.4700160845119e-01
6.9738744787054e-02 9.1600365501068e-01 9.3564555990584e-01
7.3348204878656e-01 9.1870254502963e-01 9.2428958737056e-01
3.9731820106826e-01 9.2140143648003e-01 9.1293363850919e-01
6.1247315713455e-02 9.2410037521411e-01 9.0157765577432e-01
7.2526891665251e-01 9.2679941267292e-01 8.9022157696039e-01
3.8938194615505e-01 9.2949860559372e-01 8.7886533600964e-01
5.3584786030315e-02 9.3219801549373e-01 8.6750886407089e-01
7.1787529169563e-01 9.3489770792089e-01 8.5615209079493e-01
3.8225083834648e-01 9.3759775147472e-01 8.4479494583309e-01
4.6708377492103e-02 9.4029821661457e-01 8.3343736048796e-01
7.1124450121026e-01 9.4299917428669e-01 8.2207926945006e-01
3.7585551073378e-01 9.4570069441506e-01 8.1072061254231e-01
4.0537485513642e-02 9.4840284431181e-01 7.9936133638802e-01
7.0528634878666e-01 9.5110568707060e-01 7.8800139591737e-01
3.7009792594845e-01 9.5380928000961e-01 7.7664075563379e-01
3.4967992685284e-02 9.5651367322964e-01 7.6527939057353e-01
6.9989231079310e-01 9.5921890834768e-01 7.5391728690848e-01
3.6486665080823e-01 9.6192501745755e-01 7.4255444216238e-01
2.9886801865667e-02 9.6463202235802e-01 7.3119086503218e-01
6.9494857044698e-01 9.6733993407621e-01 7.1982657482710e-01
3.6004777074916e-01 9.7004875270091e-01 7.0846160055773e-01
2.5180210184681e-02 9.7275846752707e-01 6.9709597972384e-01
6.9034167393145e-01 9.7546905750077e-01 6.8572975686269e-01
3.5552791244739e-01 9.7818049194178e-01 6.7436298192870e-01
2.0734635448889e-02 9.8089273151065e-01 6.6299570858060e-01
6.8595751506780e-01 9.8360572937793e-01 6.5162799245288e-01
3.5119219983161e-01 9.8631943254495e-01 6.4025988948611e-01
1.6434339867291e-02 9.8903378326011e-01 6.2889145438377e-01
6.8167962244871e-01 9.9174872047091e-01 6.1752273925379e-01
3.4692381582698e-01 9.9446418125102e-01 6.0615379248060e-01
1.2162818334794e-02 9.9718010214480e-01 5.9478465785895e-01
6.7817981474375e-01 9.9989267354135e-01 5.8341966690227e-01
3.4340003555805e-01 2.6093078719700e-03 5.7205028977375e-01
8.6038910252496e-03 5.3262220315010e-03 5.6068082183733e-01
6.7378826729141e-01 8.0433606205199e-03 5.4931128535964e-01
3.3895037835525e-01 1.0760671728476e-02 5.3794169768132e-01
4.0877755406194e-03 1.3478107068778e-02 5.2657207179161e-01
6.6919834390519e-01 1.6195621907752e-02 5.1520241703033e-01
3.2773043299349e-01 7.8695180668598e-01 5.0345903602978e-01
1.1298876391699e-02 7.8968893503113e-01 4.9209365210521e-01
6.9798659505862e-01 7.9242599916814e-01 4.8072796645136e-01
3.8676583674321e-01 7.9515103334830e-01 4.6936796643857e-01
7.6264847400572e-02 7.9785252474432e-01 4.5802405173304e-01
7.6506506376574e-01 8.0052915819118e-01 4.4670174756358e-01
4.5179494169312e-01 8.0319160263468e-01 4.3539420159698e-01
1.3542714861102e-01 8.0585376425158e-01 4.2408727957322e-01
8.1560268286440e-01 8.0852403238852e-01 4.1277029738096e-01
4.9254767087209e-01 8.1120421231906e-01 4.0143992358140e-01
1.6675575307068e-01 8.1389272194311e-01 3.9009759398017e-01
8.3874968091158e-01 8.1658722671379e-01 3.7874627062530e-01
5.0898195840532e-01 8.1928572911868e-01 3.6738876697845e-01
1.7781520037417e-01 8.2198678494677e-01 3.5602725116342e-01
8.4553195956613e-01 8.2468942084205e-01 3.4466325088814e-01
5.1235102711063e-01 8.2739300049935e-01 3.3329779658009e-01
1.7844243495868e-01 8.3009711300933e-01 3.2193156739869e-01
8.4393939203975e-01 8.3280149401878e-01 3.1056500485361e-01
5.0894730371612e-01 8.3550597353619e-01 2.9919839294462e-01
1.7355046912451e-01 8.3821044226390e-01 2.8783191229525e-01
8.3781704022400e-01 8.4091483005974e-01 2.7646567601354e-01
5.0180270816166e-01 8.4361909214891e-01 2.6509975326316e-01
1.6555346297734e-01 8.4632320024962e-01 2.5373418472855e-01
8.2910767706596e-01 8.4902713681653e-01 2.4236899277368e-01
4.9249769216492e-01 8.5173089127286e-01 2.3100418813079e-01
1.5575103898356e-01 8.5443445752110e-01 2.1963977431427e-01
8.1889138265177e-01 8.5713783228347e-01 2.0827575053640e-01
4.8193926168600e-01 8.5984101398656e-01 1.9691211363072e-01
1.4491267003718e-01 8.6254400200734e-01 1.8554885931317e-01
8.0782751879515e-01 8.6524679616236e-01 1.7418598299759e-01
4.7069800474917e-01 8.6794939636381e-01 1.6282348030797e-01
1.3353690617826e-01 8.7065180239275e-01 1.5146134738096e-01
7.9635582123499e-01 8.7335401375755e-01 1.4009958102011e-01
4.5916536057428e-01 8.7605602961714e-01 1.2873817874176e-01
1.2197530310657e-01 8.7875784875656e-01 1.1737713873812e-01
7.8479472166576e-01 8.8145946960766e-01 1.0601645977329e-01
4.4763208379784e-01 8.8416089031145e-01 9.4656141021583e-02
1.1049533166591e-01 8.8686210882130e-01 8.3296181852926e-02
7.7339194414175e-01 8.8956312304768e-01 7.1936581567462e-02
4.3632898344550e-01 8.9226393104661e-01 6.0577339079919e-02
9.9313128160642e-02 8.9496453125421e-01 4.9218452553599e-02
7.6235069405666e-01 8.9766492277013e-01 3.7859918984165e-02
4.2544764387767e-01 9.0036510569199e-01 2.6501733734521e-02
8.8609587086093e-02 9.0306508150201e-01 1.5143890023988e-02
7.5184177047862e-01 9.0576485350525e-01 3.7863783777784e-03
4.1458729012501e-01 9.0846602944656e-01 9.9242729355042e-01
7.7958389951324e-02 9.1116551284532e-01 9.8107029151672e-01
7.4141286035933e-01 9.1386481677465e-01 9.6971357250520e-01
4.0495425679866e-01 9.1656395706685e-01 9.5835711040609e-01
6.8585626113383e-02 9.1926295411350e-01 9.4700087357208e-01
7.3230946805957e-01 9.2196183341570e-01 9.3564482440480e-01
3.9612769854665e-01 9.2466062607575e-01 9.2428891907454e-01
6.0041617197443e-02 9.2735936919517e-01 9.1293310741491e-01
7.2405188193537e-01 9.3005810614042e-01 9.0157733303351e-01
3.8815849324988e-01 9.3275688663699e-01 8.9022153367503e-01
5.2360790515921e-02 9.3545576665446e-01 8.7886564186552e-01
7.1665746221768e-01 9.3815480805062e-01 8.6750958585292e-01
3.8104657114911e-01 9.4085407795214e-01 8.5615329084287e-01
4.5525594664140e-02 9.4355364786173e-01 8.4479668050819e-01
7.1009147890075e-01 9.4625359249799e-01 8.3343967872917e-01
3.7474070471459e-01 9.4895398839109e-01 8.2208221150055e-01
3.9469361964290e-02 9.5165491227533e-01 8.1072420892267e-01
7.0427322793769e-01 9.5435643933555e-01 7.9936560718111e-01
3.6914784522290e-01 9.5705864137686e-01 7.8800635041273e-01
3.4088594303600e-02 9.5976158499557e-01 7.7664639235742e-01
6.9909075662820e-01 9.6246532983132e-01 7.6528569770471e-01
3.6414956483742e-01 9.6516992697794e-01 7.5392424306096e-01
2.9260238142051e-02 9.6787541762229e-01 7.4256201748508e-01
6.9441800237826e-01 9.7058183196857e-01 7.3119902256634e-01
3.5961809585882e-01 9.7328918849113e-01 7.1983527204437e-01
2.4855763606276e-02 9.7599749354263e-01 7.0847079099653e-01
6.9012624373283e-01 9.7870674132827e-01 6.9710561464073e-01
3.5542475049408e-01 9.8141691424095e-01 6.8573978681992e-01
2.0746458805552e-02 9.8412798353739e-01 6.7437335824838e-01
6.8608649474576e-01 9.8683991032166e-01 6.6300638460846e-01
3.5143993580508e-01 9.8955264679017e-01 6.5163892459005e-01
1.6801823461079e-02 9.9226613768200e-01 6.4027103796360e-01
6.8216718919781e-01 9.9498032186987e-01 6.2890278377123e-01
3.4753109350468e-01 9.9769513402179e-01 6.1753421870987e-01
1.3661001260377e-02 4.0682884831165e-04 6.0616960914379e-01
6.7901160272238e-01 3.1226662292368e-03 5.9480060606761e-01
3.4434649012439e-01 5.8389286031707e-03 5.8343143368551e-01
9.6613957889260e-03 8.5555493856134e-03 5.7206212870692e-01
6.7495236485727e-01 1.1272464772828e-02 5.6069272176108e-01
3.4021580031290e-01 1.3989614510031e-02 5.4932323756165e-01
5.4484953213801e-03 1.6706942391159e-02 5.3795369531469e-01
6.7064765094923e-01 1.9424396496435e-02 5.2658410932346e-01
3.3581087857060e-01 2.2141929187987e-02 5.1521448973979e-01
9.9534968423486e-01 7.9290370615143e-01 5.0346248409172e-01
6.7575647398428e-01 7.9563691620544e-01 4.9209692201691e-01
3.5855001778547e-01 7.9837075495423e-01 4.8073092101302e-01
4.3121147067465e-02 8.0109787408937e-01 4.6936765872971e-01
7.2861489426863e-01 8.0380984724831e-01 4.5801358726127e-01
4.1411864404876e-01 8.0650224278615e-01 4.4667476689888e-01
9.8719885828332e-02 8.0917796596943e-01 4.3535155169024e-01
7.8156396627602e-01 8.1184550341794e-01 4.2403723991576e-01
4.6204224036439e-01 8.1451368711387e-01 4.1272242576267e-01
1.3993666758211e-01 8.1718781885742e-01 4.0140026099155e-01
8.1536754319041e-01 8.1986937926928e-01 3.9006816674805e-01
4.8863418339811e-01 8.2255763047287e-01 3.7872660328405e-01
1.6008125446209e-01 8.2525109408308e-01 3.6737733089414e-01
8.3002914658425e-01 8.2794831939738e-01 3.5602230659267e-01
4.9875015446827e-01 8.3064814101628e-01 3.4466321764025e-01
1.6646681983099e-01 8.3334970086219e-01 3.3330136939936e-01
8.3335829159033e-01 8.3605239319858e-01 3.2193771798596e-01
4.9956833876879e-01 8.3875579909300e-01 3.1057294318439e-01
1.6521268489857e-01 8.4145963149811e-01 2.9920752016848e-01
8.3038505032180e-01 8.4416369442958e-01 2.8784177764891e-01
4.9516192008021e-01 8.4686785418097e-01 2.7647594136298e-01
1.5960624094451e-01 8.4957201951233e-01 2.6511016535032e-01
8.2377027275047e-01 8.5227612815029e-01 2.5374455400369e-01
4.8769778916523e-01 8.5498013760856e-01 2.4237917746750e-01
1.5142578300176e-01 8.5768401893177e-01 2.3101408234953e-01
8.1498579495481e-01 8.6038775241068e-01 2.1964929916724e-01
4.7840495544416e-01 8.6309132462865e-01 2.0828484752903e-01
1.4170680686872e-01 8.6579472641153e-01 1.9692073974454e-01
8.0491195677714e-01 8.6849795139479e-01 1.8555698334288e-01
4.6803859996096e-01 8.7120099501596e-01 1.7419358282763e-01
1.3110293819042e-01 8.7390385380369e-01 1.6283054089497e-01
7.9411951939898e-01 8.7660652487655e-01 1.5146785927035e-01
4.5710151295269e-01 8.7930900559326e-01 1.4010553927044e-01
1.2006093375108e-01 8.8201129331509e-01 1.2874358216352e-01
7.8300882496216e-01 8.8471338525453e-01 1.1738198937800e-01
4.4595540694885e-01 8.8741527839345e-01 1.0602076259245e-01
1.0891019822112e-01 8.9011696946051e-01 9.4659903729351e-02
7.7188211291623e-01 8.9281845496207e-01 8.3299414866208e-02
4.3487953827649e-01 8.9551973126432e-01 7.1939298072652e-02
9.7910394790774e-02 8.9822079472651e-01 6.0579555178027e-02
7.6098218104436e-01 9.0092164188686e-01 4.9220187471716e-02
4.2410200484850e-01 9.0362226970374e-01 3.7861195337328e-02
8.7276601872650e-02 9.0632267585490e-01 2.6502577821671e-02
7.5051234276540e-01 9.0902285909728e-01 1.5144332140481e-02
4.1381522961508e-01 9.1172281968911e-01 3.7864531247499e-03
7.6660744864462e-02 9.1442407459059e-01 9.9242714334454e-01
7.4009839974699e-01 9.1712369991201e-01 9.8106985548764e-01
4.0361859369120e-01 9.1982311590868e-01 9.6971289963869e-01
6.7225636704372e-02 9.2252233422918e-01 9.5835625470861e-01
7.3092332526413e-01 9.2522137094137e-01 9.4699989397332e-01
3.9471489723528e-01 9.2792024719311e-01 9.3564378448729e-01
5.8602986091177e-02 9.3061898984354e-01 9.2428788660207e-01
7.2258957720696e-01 9.3331763202825e-01 9.1293215363724e-01
3.8667596929636e-01 9.3601621361586e-01 9.0157653175444e-01
5.0862744221111e-02 9.3871478150970e-01 8.9022096008388e-01
7.1514974834158e-01 9.4141338974698e-01 8.7886537114803e-01
3.7953608823815e-01 9.4411209935039e-01 8.6750969161628e-01
4.4020142975028e-02 9.4681097789433e-01 8.5615384340855e-01
7.0859959409830e-01 9.4951009875941e-01 8.4479774514447e-01
3.7327147330500e-01 9.5220954006528e-01 8.3344131390986e-01
3.8032226286348e-02 9.5490938329177e-01 8.2208446728500e-01
7.0287778978660e-01 9.5760971161989e-01 8.1072712555396e-01
3.6780367760729e-01 9.6031060804600e-01 7.9936921399181e-01
3.2805070312896e-02 9.6301215334153e-01 7.8801066511195e-01
6.9787690291876e-01 9.6571442394523e-01 7.7665142075022e-01
3.6301394496917e-01 9.6841748988307e-01 7.6529143386641e-01
2.8210868167526e-02 9.7112141281246e-01 7.5393066995894e-01
6.9346229803828e-01 9.7382624428156e-01 7.4256910801104e-01
3.5876284783785e-01 9.7653202428312e-01 7.3120674091619e-01
2.4107134885841e-02 9.7923878016591e-01 7.1984357536297e-01
6.8948978142581e-01 9.8194652594818e-01 7.0847963119183e-01
3.5490540381719e-01 9.8465526205748e-01 6.9711494026656e-01
2.0348595109165e-02 9.8736497550099e-01 6.8574954492897e-01
6.8581390670910e-01 9.9007564045185e-01 6.7438349612543e-01
3.5129583485834e-01 9.9278721921890e-01 6.6301685130763e-01
1.6788816966862e-02 9.9549966355160e-01 6.5164967221763e-01
6.8228724161187e-01 9.9821291621765e-01 6.4028202266758e-01
3.4853933053471e-01 9.2332181664993e-04 6.2891808075576e-01
1.4036751127978e-02 3.6379604900180e-03 6.1754971584649e-01
6.7952268067846e-01 6.3532022499202e-03 6.0618106188558e-01
3.4499171939048e-01 9.0689724437262e-03 5.9481217207624e-01
1.0438681578398e-02 1.1785196633757e-02 5.8344309371102e-01
6.7585866888780e-01 1.4501802125739e-02 5.7207386758973e-01
3.4124713845245e-01 1.7218719238199e-02 5.6070452780270e-01
6.5999609338475e-03 1.9935882277101e-02 5.4933510184862e-01
6.7191346441020e-01 2.2653230199906e-02 5.3796561104093e-01
3.3718446144138e-01 2.5370706972757e-02 5.2659607114714e-01
2.4102581439928e-03 2.8088261641953e-02 5.1522649320028e-01
6.6274475995616e-01 7.9885507549529e-01 5.0346639959418e-01
3.4075988233604e-01 8.0158535918520e-01 4.9210064344940e-01
2.0656014091445e-02 8.0431648502140e-01 4.8073446470204e-01
7.0204951873098e-01 8.0704370684384e-01 4.6936965930726e-01
3.8438245054377e-01 8.0976107204495e-01 4.5801022307372e-01
6.7036081440418e-02 8.1246401945032e-01 4.4666094637360e-01
7.4938796242102e-01 8.1515192028357e-01 4.3532447209776e-01
4.3081957442732e-01 8.1782861044231e-01 4.2399906004282e-01
1.1077338972526e-01 8.2050041389580e-01 4.1267915852660e-01
7.8886639857204e-01 8.2317318519346e-01 4.0135831993696e-01
4.6495710166894e-01 8.2585047954090e-01 3.9003190529665e-01
1.3911268717483e-01 8.2853349090203e-01 3.7869791174063e-01
8.1152288010191e-01 8.3122191554320e-01 3.6735634410917e-01
4.8242161736095e-01 8.3391481706483e-01 3.5600823754921e-01
1.5203953231291e-01 8.3661115327682e-01 3.4465493873135e-01
8.2058271067326e-01 8.3931000739377e-01 3.3329772961038e-01
4.8822678859619e-01 8.4201065160452e-01 3.2193768426553e-01
1.5511817096210e-01 8.4471253751914e-01 3.1057564563858e-01
8.2137783142280e-01 8.4741526327873e-01 2.9921225148400e-01
4.8710558914658e-01 8.5011853870310e-01 2.8784797409126e-01
1.5238402963290e-01 8.5282215578161e-01 2.7648315874963e-01
8.1728182133106e-01 8.5552596586268e-01 2.6511805591547e-01
4.8185642015504e-01 8.5822986278232e-01 2.5375284635085e-01
1.4615623804569e-01 8.6093377065990e-01 2.4238766008365e-01
8.1022237013644e-01 8.6363763516287e-01 2.3102259043292e-01
4.7408996864313e-01 8.6634141727784e-01 2.1965770427576e-01
1.3778933768958e-01 8.6904508886740e-01 2.0829304952359e-01
8.0134680877874e-01 8.7174862949177e-01 1.9692866055477e-01
4.6478544390257e-01 8.7445202412534e-01 1.8556456216029e-01
1.2812560289287e-01 8.7715526150795e-01 1.7420077240986e-01
7.9138540340258e-01 8.7985833294903e-01 1.6283730473307e-01
4.5458109551926e-01 8.8256123145731e-01 1.5147416942743e-01
1.1772736807800e-01 8.8526395110732e-01 1.4011137474462e-01
7.8083759993489e-01 8.8796648658122e-01 1.2874892766314e-01
4.4392406652151e-01 8.9066883284299e-01 1.1738683442364e-01
1.0699810972048e-01 8.9337098491622e-01 1.0602510088081e-01
7.7007027732473e-01 8.9607293774583e-01 9.4663732709324e-02
4.3315043695027e-01 8.9877468613151e-01 8.3302735488911e-02
9.6247868175249e-02 9.0147622472575e-01 7.1942114685632e-02
7.5937133581103e-01 9.0417754809302e-01 6.0581875539575e-02
4.2252914652563e-01 9.0687865082973e-01 4.9222022865176e-02
8.5729190502827e-02 9.0957952774634e-01 3.7862560767333e-02
7.4897896940770e-01 9.1228017411439e-01 2.6503492274954e-02
4.1228561162727e-01 9.1498058598192e-01 1.5144818893210e-02
7.5655875556030e-02 9.1768076056035e-01 3.7865400766272e-03
7.3859810193607e-01 9.2038212267909e-01 9.9242696816393e-01
4.0209815279150e-01 9.2308192356219e-01 9.8106934985133e-01
6.5679523736117e-02 9.2578148916451e-01 9.6971210722651e-01
7.2934731157683e-01 9.2848082667314e-01 9.5835522492462e-01
3.9310610613674e-01 9.3117994743032e-01 9.4699868200457e-01
5.6959939782891e-02 9.3387886768651e-01 9.3564245120240e-01
7.2091223344337e-01 9.3657760935807e-01 9.2428649825148e-01
3.8496574186897e-01 9.3927620075399e-01 9.1293078132028e-01
4.9122501394307e-02 9.4197467722778e-01 9.0157525062536e-01
7.1338378385322e-01 9.4467308170327e-01 8.9021984828070e-01
3.7775006051943e-01 9.4737146501828e-01 8.7886450844399e-01
4.2220979880141e-02 9.5006988602837e-01 8.6750915781368e-01
7.0679536262966e-01 9.5276841141630e-01 8.5615371651799e-01
3.7147121643903e-01 9.5546711516188e-01 8.4479809941613e-01
3.6245771816609e-02 9.5816607764170e-01 8.3344221780562e-01
7.0111553878705e-01 9.6086538434930e-01 8.2208598149784e-01
3.6607638232048e-01 9.6356512425131e-01 8.1072930119045e-01
3.1123612639196e-02 9.6626538782231e-01 7.9937209103369e-01
6.9625208495912e-01 9.6896626482782e-01 7.8801427126163e-01
3.6145630212709e-01 9.7166784194781e-01 7.7665577074318e-01
2.6730513194550e-02 9.7437020034923e-01 7.6529652930329e-01
6.9206880133480e-01 9.7707341332461e-01 7.5393649967373e-01
3.5746515566911e-01 9.7977754411250e-01 7.4257564895438e-01
2.2913523399394e-02 9.8248264400617e-01 7.3121395949714e-01
6.8840784093506e-01 9.8518875084054e-01 7.1985142916321e-01
3.5394204516371e-01 9.8789588792543e-01 7.0848807094459e-01
1.9510068340050e-02 9.9060406346932e-01 6.9712391198096e-01
6.8510582196665e-01 9.9331327051230e-01 6.8575899203807e-01
3.5072317627926e-01 9.9602348736293e-01 6.7439336154295e-01
1.6355942381461e-02 9.9873467851034e-01 6.6302707929264e-01
6.8272956472276e-01 1.4433084428268e-03 6.5166420577146e-01
3.4838024811956e-01 4.1562554736123e-03 6.4029686032072e-01
1.4027270688084e-02 6.8700045071136e-03 6.2892906237077e-01
6.7966428251308e-01 9.5844812837648e-03 6.1756087933391e-01
3.4528500120787e-01 1.2299607099578e-02 6.0619237517396e-01
1.0883287932320e-02 1.5015300999954e-02 5.9482360877042e-01
6.7645323254131e-01 1.7731481844201e-02 5.8345463270105e-01
3.4198924137584e-01 2.0448070152546e-02 5.7208549246398e-01
7.4861208736378e-03 2.3164989664985e-02 5.6071622613157e-01
6.7293915063324e-01 2.5882168563284e-02 5.4934686440314e-01
3.3834414069950e-01 2.8599540331829e-02 5.3797743100341e-01
3.6974694477068e-03 3.1317044257273e-02 5.2660794335992e-01
6.6899610028412e-01 3.4034625588863e-02 5.1523841348637e-01
3.2998627315203e-01 8.0480601033063e-01 5.0347077676899e-01
6.1289354782541e-03 8.0753404238867e-01 4.9210481842696e-01
6.8379017928508e-01 8.1026294178626e-01 4.8073849568919e-01
3.6271619342232e-01 8.1298950346417e-01 4.6937287968566e-01
4.2529070925519e-02 8.1570950887876e-01 4.5801049960224e-01
7.2279395813587e-01 8.1841908285740e-01 4.4665482664335e-01
4.0306769638135e-01 8.2111633938174e-01 4.3530875486875e-01
8.2907231221365e-02 8.2380233167859e-01 4.2397289249593e-01
7.6187802165934e-01 8.2648066882448e-01 4.1264489035700e-01
4.3960268172628e-01 8.2915600966811e-01 4.0132037417822e-01
1.1582812943254e-01 8.3183239077044e-01 3.8999478435803e-01
7.9045776858154e-01 8.3451230888976e-01 3.7866483850648e-01
4.6353048466094e-01 8.3719672662937e-01 3.6732896461932e-01
1.3517112546167e-01 8.3988556616207e-01 3.5598696068540e-01
8.0554272779295e-01 8.4257823559985e-01 3.4463942197480e-01
4.7481413414858e-01 8.4527398926760e-01 3.3328726995827e-01
1.4314283688026e-01 8.4797211582982e-01 3.2193146624859e-01
8.1066809668328e-01 8.5067201221783e-01 3.1057287528634e-01
4.7750969142012e-01 8.5337319756614e-01 2.9921221730778e-01
1.4376926692978e-01 8.5607530139052e-01 2.8785006697380e-01
8.0953263358147e-01 8.5877804379374e-01 2.7648687103462e-01
4.7487220786268e-01 8.6148121562770e-01 2.6512297130947e-01
1.3984925981084e-01 8.6418466150123e-01 2.5375862678906e-01
8.0451585380564e-01 8.6688826623461e-01 2.4239403257024e-01
4.6891647239308e-01 8.6959194445906e-01 2.3102933516463e-01
1.3308935349770e-01 8.7229563279106e-01 2.1966464448398e-01
7.9706758350936e-01 8.7499928400582e-01 2.0830004305032e-01
4.6087998854749e-01 8.7770286271942e-01 1.9693559299710e-01
1.2455186137695e-01 8.8040634219180e-01 1.8557134135876e-01
7.8810555542620e-01 8.8310970195679e-01 1.7420732405369e-01
4.5156097157634e-01 8.8581292606027e-01 1.6284356887821e-01
1.1493595835867e-01 8.8851600174682e-01 1.5148009775352e-01
7.7824664201948e-01 8.9121891847829e-01 1.4011692840840e-01
4.4150769952063e-01 8.9392166720052e-01 1.2875407563305e-01
1.0473258482637e-01 8.9662423979800e-01 1.1739155220460e-01
7.6793371665797e-01 8.9932662869372e-01 1.0602936955709e-01
4.3112263416773e-01 9.0202882656440e-01 9.4667538249206e-02
9.4310125600995e-02 9.0473082615063e-01 8.3306068267078e-02
7.5750633390552e-01 9.0743262014883e-01 7.1944969188493e-02
4.2072084235397e-01 9.1013420117723e-01 6.0584250225870e-02
8.3962742523808e-02 9.1283556181224e-01 4.9223920159100e-02
7.4724068639850e-01 9.1553669469474e-01 3.7863987164620e-02
4.1056292389074e-01 9.1823759270816e-01 2.6504458543953e-02
7.3937326730191e-02 9.2093824923170e-01 1.5145340353086e-02
7.3737139939764e-01 9.2363865847312e-01 3.7866369336018e-03
4.0040680139378e-01 9.2634015184610e-01 9.9242677227330e-01
6.3964816776957e-02 9.2904015829106e-01 9.8106878427833e-01
7.2760251241299e-01 9.3173990807308e-01 9.6971120908624e-01
3.9132579191721e-01 9.3443940374437e-01 9.5835403765485e-01
5.5140070310048e-02 9.3713865162945e-01 9.4699725558635e-01
7.1905021989940e-01 9.3983766264484e-01 9.3564084224461e-01
3.8306050958035e-01 9.4253645316085e-01 9.2428476989270e-01
4.7174540082535e-02 9.4523504587348e-01 9.1292900290143e-01
7.1139517836707e-01 9.4793347064430e-01 9.0157349709015e-01
3.7572449502307e-01 9.5063176525553e-01 8.9021819926948e-01
4.0163709040781e-02 9.5332997601825e-01 8.7886304706069e-01
7.0471314463075e-01 9.5602815816561e-01 8.6750796906550e-01
3.6937220469396e-01 9.5872637596057e-01 8.5615288545257e-01
3.4139365020115e-02 9.6142470245210e-01 8.4479770900976e-01
6.9901219223403e-01 9.6412321882547e-01 8.3344234668593e-01
3.6398738692508e-01 9.6682201331130e-01 8.2208670161141e-01
2.9060851375829e-02 9.6952117964525e-01 8.1073067554670e-01
6.9422777904305e-01 9.7222081510178e-01 7.9937417166680e-01
3.5948276071392e-01 9.7492101815985e-01 7.8801709754965e-01
2.4819900383730e-02 9.7762188589140e-01 7.7665936820682e-01
6.9023293269491e-01 9.8032351119018e-01 7.6530090897677e-01
3.5571533348264e-01 9.8302597997728e-01 7.5394165809991e-01
2.1260415708715e-02 9.8572936852658e-01 7.4258156881003e-01
6.8686140479133e-01 9.8843374104895e-01 7.3122061080779e-01
3.5251148985892e-01 9.9113914765912e-01 7.1985877102447e-01
1.8203850446571e-02 9.9384562282570e-01 7.0849605363378e-01
6.8393166121876e-01 9.9655318437630e-01 6.9713247932069e-01
3.4968808003642e-01 9.9926183309860e-01 6.8576808386382e-01
1.6172090285773e-02 1.9681858267816e-03 6.7440677395490e-01
6.8197187827499e-01 4.6789008703072e-03 6.6304094304971e-01
3.4777929764498e-01 7.3906102172228e-03 6.5167446441018e-01
1.3587259524008e-02 1.0103254151512e-02 6.4030741030721e-01
6.7938862196549e-01 1.2816762202853e-02 6.2893985504532e-01
3.4517623378247e-01 1.5531055570387e-02 6.1757187223172e-01
1.0943000884008e-02 1.8246049683003e-02 6.0620353234732e-01
6.7668197169778e-01 2.0961656768581e-02 5.9483490072469e-01
3.4238643576277e-01 2.3677788308008e-02 5.8346603600483e-01
8.0500278732296e-03 2.6394357261746e-02 5.7209698910809e-01
6.7366682944587e-01 2.9111279976723e-02 5.6072780271786e-01
3.3923145910690e-01 3.1828477707563e-02 5.4935851124295e-01
4.7391457335259e-03 3.4545877716120e-02 5.3798914119774e-01
6.7018577909247e-01 3.7263413943666e-02 5.2661971192042e-01
3.3556793547771e-01 3.9981027277920e-02 5.1525023654081e-01
9.9711798030102e-01 8.1075657220509e-01 5.0347560794521e-01
6.7175491706178e-01 8.1348283133155e-01 4.9210944400515e-01
3.4764239374228e-01 8.1620990923878e-01 4.8074297999544e-01
2.4605514747446e-02 8.1893555745288e-01 4.6937688423406e-01
7.0238006120090e-01 8.2165673869350e-01 4.5801280314090e-01
3.8065225303937e-01 8.2437035713833e-01 4.4665319436878e-01
5.9095625803889e-02 8.2707427871215e-01 4.3530055831998e-01
7.3738418589458e-01 8.2976816024631e-01 4.2395633884581e-01
4.1519167661744e-01 8.3245364590637e-01 4.1262009818414e-01
9.2204733398728e-02 8.3513380355256e-01 4.0128950094398e-01
7.6815976049193e-01 8.3781208975984e-01 3.8996114415869e-01
4.4288269293736e-01 8.4049138510052e-01 3.7863170872707e-01
1.1630493009070e-01 8.4317350893864e-01 3.6729878920625e-01
7.8844955964165e-01 8.4585924222561e-01 3.5596113598750e-01
4.5940141957206e-01 8.4854862411450e-01 3.4461846161530e-01
1.2927689547047e-01 8.5124128365242e-01 3.3327109252918e-01
7.9820165116740e-01 8.5393668798395e-01 3.2191965624556e-01
4.6629732399882e-01 8.5663428916870e-01 3.1056486907987e-01
1.3367502040099e-01 8.5933359457633e-01 2.9920741765100e-01
8.0043303363906e-01 8.6203419123098e-01 2.8784790529843e-01
4.6665683476549e-01 8.6473574657606e-01 2.7648683641943e-01
1.3242012091530e-01 8.6743799917015e-01 2.6512462005594e-01
7.9778624621101e-01 8.7014074641623e-01 2.5376158164097e-01
4.6280969507185e-01 8.7284383261730e-01 2.4239797699579e-01
1.2753744487508e-01 8.7554713862931e-01 2.3103400575929e-01
7.9201016237695e-01 8.7825057340532e-01 2.1966982311551e-01
4.5626322563561e-01 8.8095406730420e-01 2.0830554953520e-01
1.2032758404119e-01 8.8365756689198e-01 1.9694127863492e-01
7.8423047660124e-01 8.8636103094378e-01 1.8557708340620e-01
4.4799602983180e-01 8.8906442738455e-01 1.7421302109920e-01
1.1164575504387e-01 8.9176773095247e-01 1.6284913702554e-01
7.7519896225833e-01 8.9447092141413e-01 1.5148546750680e-01
4.3867310526027e-01 8.9717398219945e-01 1.4012204215306e-01
1.0208406977512e-01 8.9987689935702e-01 1.2875888561787e-01
7.6544641454054e-01 9.0257966075492e-01 1.1739601894291e-01
4.2877357318007e-01 9.0528225547219e-01 1.0603346057918e-01
9.2078023229100e-02 9.0798467334073e-01 9.4671227149935e-02
7.5537142737824e-01 9.1068690460898e-01 8.3309334003570e-02
4.1866475093993e-01 9.1338893970767e-01 7.1947795591608e-02
8.1968358668213e-02 9.1609076910465e-01 6.0586625696231e-02
7.4529209333844e-01 9.1879238324136e-01 4.9225837523865e-02
4.0864533789337e-01 9.2149377254761e-01 3.7865443674993e-02
7.2037062468224e-02 9.2419492753496e-01 2.6505455995716e-02
7.3547585720547e-01 9.2689583897125e-01 1.5145885313273e-02
3.9896995145485e-01 9.2959649814107e-01 3.7867410558150e-03
6.2094772462573e-02 9.3229814181512e-01 9.9242656065562e-01
7.2570608499165e-01 9.3499837933224e-01 9.8106817031952e-01
3.8939498938784e-01 9.3769834397607e-01 9.6971022215870e-01
5.3168202592307e-02 9.4039803360582e-01 9.5835271391573e-01
7.1703198296278e-01 9.4309744936417e-01 9.4699563835678e-01
3.8099207472883e-01 9.4579659653170e-01 9.3563898228769e-01
4.5053642722611e-02 9.4849548546159e-01 9.2428272553786e-01
7.0922119946741e-01 9.5119413256926e-01 9.1292683995902e-01
3.7349852750456e-01 9.5389256133984e-01 9.0157128849924e-01
3.7888600702006e-02 9.5659080330290e-01 8.9021602442598e-01
7.0239350912887e-01 9.5928889891121e-01 8.7886099078292e-01
3.6701439277315e-01 9.6198689824790e-01 8.6750612017185e-01
3.1751389774836e-02 9.6468486147856e-01 8.5615133495069e-01
6.9660360478138e-01 9.6738285896209e-01 8.4479654792847e-01
3.6156910236449e-01 9.7008097093953e-01 8.3344166361729e-01
2.6644929079500e-02 9.7277928673527e-01 8.2208658006776e-01
6.9182716571034e-01 9.7547790343015e-01 8.1073119127110e-01
3.5711100865704e-01 9.7817692400049e-01 7.9937539005993e-01
2.2490876558345e-02 9.8087645495819e-01 7.8801907138694e-01
6.8796053547735e-01 9.8357660357002e-01 7.7666213581309e-01
3.5351323374834e-01 9.8627747477491e-01 7.6530449300096e-01
1.9141836296087e-02 9.8897916795012e-01 7.5394606499097e-01
6.8483894818728e-01 9.9168177369679e-01 7.4258678904193e-01
3.5059701846545e-01 9.9438537082071e-01 7.3122661984254e-01
1.6408417841658e-02 9.9709002367385e-01 7.1986553094459e-01
6.8226548724351e-01 9.9979577999978e-01 7.0850351532616e-01
3.4883690590686e-01 2.4994395831788e-03 6.9714428549945e-01
1.4770077325756e-02 5.2074229169508e-03 6.8578052681732e-01
6.8072571417576e-01 7.9165434314780e-03 6.7441592706113e-01
3.4669604528411e-01 1.0626774041722e-02 6.6305054544984e-01
1.2673221182459e-02 1.3338070370927e-02 6.5168445148014e-01
6.7864930757129e-01 1.6050372606448e-02 6.4031772161111e-01
3.4461630086671e-01 1.8763607850039e-02 6.2895043588681e-01
1.0566170486179e-02 2.1477692834413e-02 6.1758267468945e-01
6.7649092878272e-01 2.4192536860211e-02 6.0621451579565e-01
3.4238272568559e-01 2.6908044798174e-02 5.9484603187722e-01
8.2339616934145e-03 2.9624120001945e-02 5.8347728854767e-01
6.7403741036495e-01 3.2340666988658e-02 5.7210834301046e-01
3.3978634018462e-01 3.5057593766883e-02 5.6073924331858e-01
5.4746257565067e-03 3.7774813722798e-02 5.4937002821191e-01
6.7109683959378e-01 4.0492247012337e-02 5.3800072746323e-01
3.3664831802595e-01 4.3209821445365e-02 5.2663136263826e-01
2.1251975341681e-03 4.5927472883551e-02 5.1526194816173e-01
6.6416843335169e-01 8.1670680341187e-01 5.0348088409800e-01
3.3756838387815e-01 8.1943164089685e-01 4.9211451416784e-01
1.2015266569537e-02 8.2215722247233e-01 4.8074790339028e-01
6.8738439169724e-01 8.2488191872747e-01 4.6938148480597e-01
3.6348584517534e-01 8.2760349969904e-01 4.5801636100627e-01
4.0088316680250e-02 8.3031954055334e-01 4.4665427250478e-01
7.1694541699431e-01 8.3302805014057e-01 4.3529721176007e-01
3.9380993917465e-01 8.3572808777307e-01 4.2394675754685e-01
7.0434779751521e-02 8.3842010063159e-01 4.1260340941660e-01
7.4657385992036e-01 8.4110582190210e-01 4.0126626381081e-01
4.2199638668960e-01 8.4378776260682e-01 3.8993322338653e-01
9.6513170183348e-02 8.4646852287458e-01 3.7860163558609e-01
7.7000045169073e-01 8.4915022030391e-01 3.6726902113503e-01
4.4240796263610e-01 8.5183422118305e-01 3.5593356576443e-01
1.1374965700133e-01 8.5452116465528e-01 3.4459426323133e-01
7.8408482756238e-01 8.5721114853479e-01 3.3325080042775e-01
4.5349839305216e-01 8.5990394495021e-01 3.2190333853626e-01
1.2208537884185e-01 8.6259917342155e-01 3.1055230196626e-01
7.8994076650249e-01 8.6529641340281e-01 2.9919822211941e-01
4.5715383924499e-01 8.6799526595433e-01 2.8784164008561e-01
1.2380561195924e-01 8.7069538123570e-01 2.7648305491523e-01
7.8996811828430e-01 8.7339646616487e-01 2.6512290165556e-01
4.5570469686951e-01 8.7609828199702e-01 2.5376154660369e-01
1.2107074611942e-01 8.7880063762328e-01 2.4239929139307e-01
7.8611464618112e-01 8.8150338170146e-01 2.3103638087934e-01
4.5087869017199e-01 8.8420639511711e-01 2.1967301205175e-01
1.1539994971277e-01 8.8690958438356e-01 2.0830934258026e-01
7.7971104532347e-01 8.8961287613588e-01 1.9694549839759e-01
4.4384081575793e-01 8.9231621266382e-01 1.8558158013740e-01
1.0781489159378e-01 8.9501954834718e-01 1.7421766845833e-01
7.7165618304197e-01 8.9772284683858e-01 1.6285382837344e-01
4.3538529319530e-01 9.0042607884842e-01 1.5149011273285e-01
9.9020867551521e-02 9.0312922040773e-01 1.4012656500458e-01
7.6257988955506e-01 9.0583225150694e-01 1.2876322148306e-01
4.2607793057922e-01 9.0853515502999e-01 1.1740011303430e-01
8.9529361450904e-02 9.1123791592085e-01 1.0603726646647e-01
7.5294753140449e-01 9.1394052053502e-01 9.4674705596558e-02
4.1634491927611e-01 9.1664295614014e-01 8.3312452067546e-02
7.9733260816982e-02 9.1934521054001e-01 7.1950525957471e-02
7.4312365520341e-01 9.2204727180385e-01 6.0588946210691e-02
4.0652665313477e-01 9.2474912808935e-01 4.9227730912613e-02
6.9952328323209e-02 9.2745076755276e-01 3.7866897421848e-02
7.3341033367631e-01 9.3015217834416e-01 2.6506462367902e-02
3.9690994304694e-01 9.3285334868892e-01 1.5146441517926e-02
6.0460079067915e-02 9.3555426705981e-01 3.7868495127828e-03
7.2367037467966e-01 9.3825607440891e-01 9.9242633892937e-01
3.8733012159883e-01 9.4095656323089e-01 9.8106752124215e-01
5.1064902812473e-02 9.4365676863381e-01 9.6970916631767e-01
7.1488224088505e-01 9.4635668384607e-01 9.5835127898254e-01
3.7878924058864e-01 9.4905630480500e-01 9.4699385961171e-01
4.2792534513218e-02 9.5175563100935e-01 9.3563690308728e-01
7.0689821595688e-01 9.5445466648853e-01 9.2428039767075e-01
3.7111176052875e-01 9.5715342087206e-01 9.1292432382749e-01
3.5437938746690e-02 9.5985191053052e-01 9.0156865305846e-01
6.9988072292488e-01 9.6255015974445e-01 8.9021334681108e-01
3.6444319275487e-01 9.6524820184111e-01 8.7885835555823e-01
2.9127445075635e-02 9.6794608022234e-01 8.6750361814895e-01
6.9393451424117e-01 9.7064384919128e-01 8.5614906154324e-01
3.5886431002613e-01 9.7334157447512e-01 8.4479460104323e-01
2.3915579946093e-02 9.7603933333769e-01 8.3344014112079e-01
6.8908590201435e-01 9.7873721418268e-01 8.2208557691434e-01
3.5437171219892e-01 9.8143531556816e-01 8.1073079642490e-01
1.9768368200307e-02 9.8413374458593e-01 7.9937568338524e-01
6.8527024783639e-01 9.8683261460409e-01 7.8802012071043e-01
3.5087087667567e-01 9.8953204242423e-01 7.7666399437172e-01
1.6563076129697e-02 9.9223214495906e-01 7.6530719747661e-01
6.8233912058582e-01 9.9493303558657e-01 7.5394963429662e-01
3.4819089086109e-01 9.9763482037397e-01 7.4259122396739e-01
1.4744324163349e-02 3.3457637142893e-04 7.3123536213143e-01
6.8073112121378e-01 3.0383626488298e-03 7.1987515427097e-01
3.4676802122208e-01 5.7432842413256e-03 7.0851397043429e-01
1.2846413874105e-02 8.4493849955074e-03 6.9715181140878e-01
6.7895767720952e-01 1.1156687765985e-02 6.8578869900316e-01
3.4509316475527e-01 1.3865193802906e-02 6.7442467424292e-01
1.1244179145473e-02 1.6574883006723e-02 6.6305979471647e-01
6.7740194499134e-01 1.9285715021512e-02 6.5169413126677e-01
3.4355759376975e-01 2.1997631093846e-02 6.4032776425731e-01
9.7021716563539e-03 2.4710556584664e-02 6.2896077965763e-01
6.7582667780387e-01 2.7424403987984e-02 6.1759326519178e-01
3.4192213605783e-01 3.0139076284208e-02 6.0622530677358e-01
7.9796981766173e-03 3.2854470439583e-02 5.9485698541803e-01
6.7399077201683e-01 3.5570480859567e-02 5.8348837477071e-01
3.3994716454142e-01 3.8287002614153e-02 5.7211953934101e-01
5.8412274118692e-03 4.1003934277889e-02 5.6075053346598e-01
6.7166599257511e-01 4.3721180264369e-02 5.4938140097512e-01
3.3741528657799e-01 4.6438652580481e-02 5.3801217547879e-01
3.0838149928380e-03 4.9156271974224e-02 5.2664288116805e-01
6.6866721178670e-01 5.1873968495831e-02 5.1527353399406e-01
3.3115705082905e-01 8.2265673475874e-01 5.0348659513781e-01
3.5229258183037e-03 8.2538041559069e-01 4.9212002084106e-01
6.7677749243808e-01 8.2810476003398e-01 4.8075325695287e-01
3.5082925784887e-01 8.3082855413942e-01 4.6938659312081e-01
2.5538357641405e-02 8.3355012776012e-01 4.5802078762669e-01
7.0073107567296e-01 8.3626758680784e-01 4.4665708525009e-01
3.7621811360267e-01 8.3897920375057e-01 4.3529702521418e-01
5.1807440286049e-02 8.4168385120044e-01 4.2394204711740e-01
7.2730771050792e-01 8.4438132317491e-01 4.1259301916472e-01
4.0252709909897e-01 8.4707241983123e-01 4.0124988220405e-01
7.7276719686119e-02 8.4975875584135e-01 3.8991157686612e-01
7.5138347411784e-01 8.5244235997420e-01 3.7857629816141e-01
4.2470832203571e-01 8.5512521929593e-01 3.6724196164856e-01
9.7160800740485e-02 8.5780893086156e-01 3.5590667283663e-01
7.6870314060219e-01 8.6049454671400e-01 3.4456903008327e-01
4.3934387741427e-01 8.6318259493556e-01 3.3322821304591e-01
1.0912550375331e-01 8.6587320029627e-01 3.2188391205787e-01
7.7811123323203e-01 8.6856622840705e-01 3.1053618654124e-01
4.4637402267528e-01 8.7126140810751e-01 2.9918532019324e-01
1.1398886508725e-01 8.7395841720696e-01 2.8783170600617e-01
7.8102806800817e-01 8.7665693432166e-01 2.7647576834289e-01
4.4755875776800e-01 8.7935666577618e-01 2.6511791663001e-01
1.1364184728745e-01 8.8205735655807e-01 2.5375852168841e-01
7.7933187686563e-01 8.8475879215072e-01 2.4239790651466e-01
4.4467732728380e-01 8.8746079573605e-01 2.3103634543694e-01
1.0972115576291e-01 8.9016322345077e-01 2.1967406762210e-01
7.7450140962265e-01 8.9286595917004e-01 2.0831126247623e-01
4.3905183871336e-01 8.9556890955300e-01 1.9694808554088e-01
1.0340246740116e-01 8.9827199966260e-01 1.8558466415310e-01
7.6758010960358e-01 9.0097516924673e-01 1.7422110253285e-01
4.3160882274882e-01 9.0367836965636e-01 1.6285748618298e-01
9.5510302850988e-02 9.0638156132940e-01 1.5149388560390e-01
7.5930422577640e-01 9.0908471175465e-01 1.4013035938040e-01
4.2300854077747e-01 9.1178779383305e-01 1.2876695671895e-01
8.6639722386478e-02 9.1449078456259e-01 1.1740371951649e-01
7.5021298629765e-01 9.1719366398529e-01 1.0604068403580e-01
4.1374247419932e-01 9.1989641434627e-01 9.4677882251887e-02
7.7241411795166e-02 9.2259901942566e-01 8.3315342923361e-02
7.4072224354745e-01 9.2530146401327e-01 7.1953092431527e-02
4.0419674701737e-01 9.2800373350400e-01 6.0591155420292e-02
6.7676129079989e-02 9.3070581359847e-01 4.9229555261267e-02
7.3117110575356e-01 9.3340769009940e-01 3.7868314360853e-02
3.9469196690135e-01 9.3610934879875e-01 2.6507454319715e-02
5.8248626635078e-02 9.3881077545519e-01 1.5146995949604e-02
7.2185065986986e-01 9.4151195586500e-01 3.7869591481483e-03
3.8514233298052e-01 9.4421393402983e-01 9.9242611323414e-01
4.8845499086909e-02 9.4691468842549e-01 9.8106685178388e-01
7.1262066022698e-01 9.4961515488257e-01 9.6970806404598e-01
3.7647612253815e-01 9.5231532218012e-01 9.5834976204787e-01
4.0419824500658e-02 9.5501518117242e-01 9.4699195401261e-01
7.0445928648683e-01 9.5771472560082e-01 9.3563464331145e-01
3.6860154497001e-01 9.6041395305399e-01 9.2427782727678e-01
3.2853072018465e-02 9.6311286607077e-01 9.1292149590331e-01
6.9721968151211e-01 9.6581147336807e-01 9.0156563048149e-01
3.6170642458364e-01 9.6850979116157e-01 8.9021020222722e-01
2.6317478070170e-02 9.7120784452804e-01 8.7885517099380e-01
6.9105603118737e-01 9.7390566873767e-01 8.6750048417371e-01
3.5592417705933e-01 9.7660331046293e-01 8.5614607591750e-01
2.0922816942629e-02 9.7930082875065e-01 8.4479186680851e-01
6.8605158570262e-01 9.8199829563011e-01 8.3343776413342e-01
3.5130880703777e-01 9.8469579622529e-01 8.2208366287239e-01
1.6691485845116e-02 9.8739342824880e-01 8.1072944749809e-01
6.8219534282092e-01 9.9009130078116e-01 7.9937499461680e-01
3.4781489293529e-01 9.9278953228232e-01 7.8802017641214e-01
1.3543565090371e-02 9.9548824784143e-01 7.7666486476726e-01
6.7937385561070e-01 9.9818757573904e-01 7.6530893585702e-01
3.4589403644369e-01 8.8480224915464e-04 7.5395553083855e-01
1.1911977262897e-02 3.5856687309271e-03 7.4259810863387e-01
6.7800506928361e-01 6.2875123210168e-03 7.3123976796011e-01
3.4416383272433e-01 8.9904321327418e-03 7.1988044689721e-01
1.0378658284771e-02 1.1694509739042e-02 7.0852010707848e-01
6.7663988354893e-01 1.4399806056107e-02 6.9715873477737e-01
3.4293782833911e-01 1.7106359083896e-02 6.8579634070189e-01
9.2627950744293e-03 1.9814182627329e-02 6.7443295853302e-01
6.7560504547788e-01 2.2523266057612e-02 6.6306864233792e-01
3.4195476851565e-01 2.5233575109690e-02 6.5170346306744e-01
8.3020556990910e-03 2.7945053652263e-02 6.4033750440316e-01
6.7463689864426e-01 3.0657626313045e-02 6.2897085825165e-01
3.4094922025648e-01 3.3371201795192e-02 6.1760362019121e-01
7.2289456293486e-03 3.6085676683012e-02 6.0623588516125e-01
6.7346611254437e-01 3.8800939509036e-02 5.9486774364689e-01
3.3965101531010e-01 4.1516874843582e-02 5.8349927855615e-01
5.7743705614962e-03 4.4233367175099e-02 5.7213056291843e-01
6.7182749033315e-01 4.6950304375740e-02 5.6076165845744e-01
3.3780244666306e-01 4.9667580590158e-02 5.4939261501827e-01
3.6922131901111e-03 5.2385098441595e-02 5.3802347076387e-01
6.6949077228849e-01 5.5102770511110e-02 5.2665425300828e-01
3.3519318055718e-01 5.7820520105313e-02 5.1528497952526e-01
9.9809749550657e-01 8.2860638990430e-01 5.0349273008580e-01
6.6958636291941e-01 8.3132911857200e-01 4.9212595446214e-01
3.4183848552801e-01 8.3405243318291e-01 4.8075903252631e-01
1.4785056765422e-02 8.3677540753690e-01 4.6939216330065e-01
6.8832083829083e-01 8.3949676683985e-01 4.5802588104149e-01
3.6231317004770e-01 8.4221502435406e-01 4.4666108817809e-01
3.6614528795040e-02 8.4492872752894e-01 4.3529896197279e-01
7.1107307438998e-01 8.4763675674257e-01 4.2394072299289e-01
3.8553733544033e-01 8.5033858754668e-01 4.1258731804448e-01
5.9855942968112e-02 8.5303443133692e-01 4.0123912183178e-01
7.3387736409928e-01 8.5572520504719e-01 3.8989577159653e-01
4.0745458779230e-01 8.5841233495263e-01 3.7855620661342e-01
8.0455671240162e-02 8.6109745739802e-01 3.6721890177632e-01
7.5277629855347e-01 8.6378211529315e-01 3.5588220174259e-01
4.2434879098805e-01 8.6646754005460e-01 3.4454462974379e-01
9.5144194111200e-02 8.6915455905168e-01 3.3320508068555e-01
7.6516777523513e-01 8.7184361276933e-01 3.2186287804025e-01
4.3445069386018e-01 8.7453483560373e-01 3.1051772908076e-01
1.0304088809861e-01 8.7722815464997e-01 2.9916963087303e-01
7.7099519027956e-01 8.7992337750807e-01 2.8781876925724e-01
4.3837346762351e-01 8.8262025761595e-01 2.7646543386365e-01
1.0523475975131e-01 8.8531853687929e-01 2.6510995647800e-01
7.7163501498485e-01 8.8801797025502e-01 2.5375267122033e-01
4.3762595723083e-01 8.9071833783920e-01 2.4239389163031e-01
1.0325468336410e-01 8.9341944913708e-01 2.3103389944325e-01
7.6856369642662e-01 8.9612114288038e-01 2.1967294077737e-01
4.3359117633957e-01 8.9882328458463e-01 2.0831122664573e-01
9.8371363087176e-02 9.0152576317188e-01 1.9694893574318e-01
7.6293497780665e-01 9.0422848740353e-01 1.8558621823228e-01
4.2730963988774e-01 9.0693138250667e-01 1.7422319978083e-01
9.1520258380102e-02 9.0963438716372e-01 1.6285998544389e-01
7.5558938800312e-01 9.1233745091684e-01 1.5149666319259e-01
4.1953754688601e-01 9.1504053197653e-01 1.4013330701432e-01
8.3383496847120e-02 9.1774359539608e-01 1.2876997957692e-01
7.4714448883047e-01 9.2044661156336e-01 1.1740673448364e-01
4.1083647684547e-01 9.2314955496137e-01 1.0604361816084e-01
7.4474303878213e-02 9.2585240315353e-01 9.4680671423731e-02
7.3807186304327e-01 9.2855513595620e-01 8.3317930763213e-02
4.0164223688512e-01 9.3125773476764e-01 7.1955429391735e-02
6.5197817310518e-02 9.3396018202961e-01 6.0593198079281e-02
7.2875040818585e-01 9.3666246080361e-01 4.9231265804025e-02
3.9231131219219e-01 9.3936455444966e-01 3.7869660235435e-02
5.5891403106091e-02 9.4206644640043e-01 2.6508408061509e-02
7.1950118427178e-01 9.4476812002811e-01 1.5147535166172e-02
3.8315083365222e-01 9.4746955860559e-01 3.7870666579126e-03
4.6519669733114e-02 9.5017170808722e-01 9.9242589008699e-01
7.1026113072330e-01 9.5287273581291e-01 9.8106617782897e-01
3.7407106220230e-01 9.5557347731625e-01 9.6970693998008e-01
3.7958517976899e-02 9.5827391722001e-01 9.5834819569137e-01
7.0193224173555e-01 9.6097404156083e-01 9.4698996104816e-01
3.6600062332831e-01 9.6367383852206e-01 9.3563224807028e-01
3.0171637671862e-02 9.6637329933971e-01 9.2427506352577e-01
6.9445276601150e-01 9.6907241938675e-01 9.1291840757184e-01
3.5885089995240e-01 9.7177119943169e-01 9.0156227225244e-01
2.3372229668759e-02 9.7446964705343e-01 8.9020663990216e-01
6.8802211902698e-01 9.7716777817615e-01 8.7885148153101e-01
3.5280497198970e-01 9.7986561866461e-01 8.6749675529266e-01
1.7724096503657e-02 9.8256320589383e-01 8.5614240516724e-01
6.8278157393229e-01 9.8526059017884e-01 8.4478836001526e-01
3.4797814369263e-01 9.8795783592487e-01 8.3343453317571e-01
1.3313113729557e-02 9.9065502233898e-01 8.2208082278311e-01
6.7878430740682e-01 9.9335224353886e-01 8.1072711295838e-01
3.4438805595375e-01 9.9604960790644e-01 7.9937327598301e-01
1.0119242574130e-02 9.9874723656864e-01 7.8801917549301e-01
6.7652649214495e-01 1.4426132393394e-03 7.7666770472013e-01
3.4250263749232e-01 4.1411024224959e-03 7.6531273344140e-01
8.5832519754067e-03 6.8402687959563e-03 7.5395708066495e-01
6.7475883493225e-01 9.5402514298381e-03 7.4260062343290e-01
3.4101928964674e-01 1.2241182233915e-02 7.3124325566121e-01
7.3541221195655e-03 1.4943180809896e-02 7.1988489279410e-01
6.7375260730871e-01 1.7646349619811e-02 7.0852547538953e-01
3.4020391038821e-01 2.0350769789401e-02 6.9716497134666e-01
6.6971577663404e-03 2.3056497817627e-02 6.8580337660620e-01
6.7322146072036e-01 2.5763563394400e-02 6.7444071428918e-01
3.3976590217979e-01 2.8471968448206e-02 6.6307703237041e-01
6.3195031364225e-03 3.1181687462084e-02 6.5171240009751e-01
6.7287118839922e-01 3.3892669016437e-02 6.4034690345412e-01
3.3940977141804e-01 3.6604838443758e-02 6.2898064002349e-01
5.9239746733832e-03 3.9318101415432e-02 6.1761371363293e-01
6.7240249619483e-01 4.2032348226579e-02 6.0624622915307e-01
3.3883412511075e-01 4.4747458504659e-02 5.9487828778816e-01
5.2079010292980e-03 4.7463306045734e-02 5.8350998313068e-01
6.7151330487449e-01 5.0179763483326e-02 5.7214139816953e-01
3.3774046373745e-01 5.2896706521097e-02 5.6077260334545e-01
3.8803501151425e-03 5.5614017510975e-02 5.4940365565085e-01
6.6992495677669e-01 5.8331588227389e-02 5.3803459868461e-01
3.3586743201211e-01 6.1049321767036e-02 5.2666546350536e-01
1.7021653139545e-03 6.3767133581374e-02 5.1529627008597e-01
6.6499965653344e-01 8.3455578790239e-01 5.0349927719603e-01
3.3573569046709e-01 8.3727772523610e-01 4.9213230432221e-01
7.1338151565407e-03 8.4000017702143e-01 4.8076522131377e-01
6.7914117502777e-01 8.4272242188099e-01 4.6939816811077e-01
3.5167706662456e-01 8.4544346957600e-01 4.5803152940914e-01
2.4638630917928e-02 8.4816214336644e-01 4.4666596920238e-01
6.9790948473799e-01 8.5087723775226e-01 4.3530238789422e-01
3.7136796240167e-01 8.5358772345779e-01 4.2394178661727e-01
4.4891992145221e-02 8.5629294864466e-01 4.1258505907961e-01
7.1836001235142e-01 8.5899278077120e-01 4.0123276559530e-01
3.9165015475709e-01 8.6168764764267e-01 3.8988496058187e-01
6.4641160769948e-02 8.6437846307822e-01 3.7854113473773e-01
7.3721725807560e-01 8.6706645583430e-01 3.6720029644175e-01
4.0927640251468e-01 8.6975295068997e-01 3.5586116576289e-01
8.0738909825025e-02 8.7243916402068e-01 3.4452241324765e-01
7.5155314830408e-01 8.7512606376841e-01 3.3318286741795e-01
4.2169656230762e-01 8.7781431248158e-01 3.2184164167452e-01
9.1172432810467e-02 8.8050428095521e-01 3.1049817214583e-01
7.6000410547971e-01 8.8319610381539e-01 2.9915218854583e-01
4.2822857277560e-01 8.8588974874081e-01 2.8780365029791e-01
9.5890726494738e-02 8.8858508036494e-01 2.7645267493590e-01
7.6303888279048e-01 8.9128191024161e-01 2.6509947487102e-01
4.2972165446688e-01 8.9398003150853e-01 2.5374430889918e-01
9.5985971499212e-02 8.9667924052563e-01 2.4238744872961e-01
7.6187596540074e-01 8.9937934886609e-01 2.3102915800846e-01
4.2743244944313e-01 9.0208018881965e-01 2.1966968058908e-01
9.2692781763795e-02 9.0478161487362e-01 2.0830923509992e-01
7.5769095789967e-01 9.0748350290302e-01 1.9694801350738e-01
4.2245782926953e-01 9.1018574819927e-01 1.8558618203078e-01
8.7021381354749e-02 9.1288826302955e-01 1.7422388330918e-01
7.5140703113514e-01 9.1559097412479e-01 1.6286123912191e-01
4.1563792036684e-01 9.1829382030709e-01 1.5149835324217e-01
7.9735192108965e-02 9.2099675035353e-01 1.4013531418668e-01
7.4371824454520e-01 9.2369972112812e-01 1.2877219774024e-01
4.0760496007284e-01 9.2640269597796e-01 1.1740906920411e-01
7.1411909839230e-02 9.2910564337232e-01 1.0604598535807e-01
7.3515453505197e-01 9.3180853575699e-01 9.4682996147649e-02
3.9884730689045e-01 9.3451134859501e-01 8.3320146117912e-02
6.2503866967259e-02 9.3721405956758e-01 7.1957475618584e-02
7.2613715018776e-01 9.3991664791264e-01 6.0595021804061e-02
3.8975949163244e-01 9.4261909388310e-01 4.9232819448821e-02
5.3382718796590e-02 9.4532137831140e-01 3.7870901594320e-02
7.1701823019661e-01 9.4802348227152e-01 2.6509300038589e-02
3.8067706101100e-01 9.5072538683407e-01 1.5148045674713e-02
4.4369936075770e-02 9.5342707291453e-01 3.7871686794236e-03
7.0781163695282e-01 9.5612938734905e-01 9.9242567621437e-01
3.7158620121661e-01 9.5883068926999e-01 9.8106551601602e-01
3.5425231839785e-02 9.6153171296300e-01 9.6970582033348e-01
6.9933848162181e-01 9.6423243928574e-01 9.5834661516851e-01
3.6333545588420e-01 9.6693284985439e-01 9.4698792424937e-01
2.7425378068831e-02 9.6963292766299e-01 9.3562976813177e-01
6.9161713668337e-01 9.7233265789126e-01 9.2427216307615e-01
3.5591921242999e-01 9.7503202891700e-01 9.1291511969984e-01
2.0339584818625e-02 9.7773103354288e-01 9.0155864141559e-01
6.8488561837355e-01 9.8042967043670e-01 8.9020272268698e-01
3.4956392975545e-01 9.8312794576733e-01 8.7884734715616e-01
1.4380238775695e-02 9.8582587499588e-01 8.6749248573066e-01
6.7933920835055e-01 9.8852348475281e-01 8.5613809475209e-01
3.4444428081666e-01 9.9122081469707e-01 8.4478411440680e-01
9.6975204691067e-03 9.9391791921744e-01 8.3343046757255e-01
6.7509947431432e-01 9.9661486880188e-01 8.2207705931810e-01
3.4064906435588e-01 9.9931175087597e-01 8.1072377727558e-01
6.8535235870925e-03 2.0062323475217e-03 7.9937328622232e-01
6.7270000130685e-01 4.7032339728023e-03 7.8801994305216e-01
3.3868065222785e-01 7.4005302267531e-03 7.7666630259453e-01
4.7878616666414e-03 1.0098267366919e-02 7.6531220923176e-01
6.7101267901530e-01 1.2796598779884e-02 7.5395750887967e-01
3.3734504270199e-01 1.5495679891789e-02 7.4260205550418e-01
3.7740294228326e-03 1.8195662397748e-02 7.3124571771582e-01
6.7028810332475e-01 2.0896688168874e-02 7.1988838490945e-01
3.3687534068188e-01 2.3598883249966e-02 7.0852997242550e-01
3.5236095853637e-03 2.6302352369592e-02 6.9717042528462e-01
6.7022069176720e-01 2.9007174347987e-02 6.8580972017851e-01
3.3695434363223e-01 3.1713398715128e-02 6.7444786556487e-01
3.7123041051109e-03 3.4421043754432e-02 6.6308489988992e-01
6.7048226611873e-01 3.7130096080299e-02 6.5172088812507e-01
3.3725183483868e-01 3.9840511750456e-02 6.4035591693909e-01
4.0084979404532e-03 4.2552218813638e-02 6.2899008892248e-01
6.7073963104676e-01 4.5265121102920e-02 6.1762351633379e-01
3.3743255517732e-01 4.7979103007711e-02 6.0625631484670e-01
4.0746538568604e-03 5.0694034896783e-02 5.9488859774424e-01
6.7065354700863e-01 5.3409778826537e-02 5.8352047093622e-01
3.3715730882139e-01 5.6126194159071e-02 5.7215202907520e-01
3.5747091913435e-03 5.8843142738448e-02 5.6078335292550e-01
6.6989545437840e-01 6.1560493330627e-02 5.4941450801282e-01
3.3611040266674e-01 6.4278125117130e-02 5.3804554446527e-01
2.2117347099161e-03 6.6995930133145e-02 5.2667649786324e-01
6.6819306478607e-01 6.9713814642964e-02 5.1530739085575e-01
3.3187086527733e-01 8.4050494475685e-01 5.0350622405110e-01
1.9540348102453e-03 8.4322621929244e-01 4.9213905878968e-01
6.7261645238905e-01 8.4594794396340e-01 4.8077181351926e-01
3.4381675895850e-01 8.4866954670384e-01 4.6940458862411e-01
1.5492012125507e-02 8.5139024588295e-01 4.5803766656960e-01
6.8756134346526e-01 8.5410910098602e-01 4.4667154320048e-01
3.5993192121539e-01 8.5682511544620e-01 4.3530691059514e-01
3.2505226961697e-02 8.5953737780520e-01 4.2394457777671e-01
7.0518153045878e-01 8.6224521190589e-01 4.1258533541396e-01
3.7786156671669e-01 8.6494830050631e-01 4.0122979249833e-01
5.0446171709943e-02 8.6764675137290e-01 3.8987822767019e-01
7.2283591815275e-01 8.7034108797575e-01 3.7853049918665e-01
3.9493279278157e-01 8.7303216507946e-01 3.6718604339034e-01
6.6644671493171e-02 8.7572102929788e-01 3.5584396467749e-01
7.3789159645594e-01 8.7840876005446e-01 3.4450318968880e-01
4.0861173932012e-01 8.8109632997787e-01 3.3316263871312e-01
7.8765059108385e-02 8.8378451284390e-01 3.2182136799020e-01
7.4833376174726e-01 8.8647384758905e-01 3.1047865539276e-01
4.1731992574235e-01 8.8916464905750e-01 2.9913402635312e-01
8.5741407147864e-02 8.9185704715664e-01 2.8778723444528e-01
7.5362722810739e-01 8.9455103634007e-01 2.7643821707518e-01
4.2101332726816e-01 8.9724652275726e-01 2.6508704396407e-01
8.7939120371353e-02 8.9994336268196e-01 2.5373386972825e-01
7.5444497831274e-01 9.0264139055191e-01 2.4237889576639e-01
4.2057053200469e-01 9.0534043760214e-01 2.3102234249248e-01
8.6353633701825e-02 9.0804034311115e-01 2.1966443075018e-01
7.5182979687587e-01 9.1074096036945e-01 2.0830537043851e-01
4.1703196260876e-01 9.1344215914865e-01 1.9694535435292e-01
8.1990475912792e-02 9.1614382600656e-01 1.8558455555975e-01
7.4673318861796e-01 9.1884586335800e-01 1.7422312702248e-01
4.1128563221822e-01 9.2154818792143e-01 1.6286120256658e-01
7.5671223849831e-02 9.2425072892084e-01 1.5149889856483e-01
7.3991148241723e-01 9.2695342626589e-01 1.4013631594316e-01
4.0402624118994e-01 9.2965622883142e-01 1.2877354225999e-01
6.8033849250635e-02 9.3235909289441e-01 1.1741065371510e-01
7.3195135799674e-01 9.3506198074848e-01 1.0604771701078e-01
3.9579469122648e-01 9.3776485949535e-01 9.4684791029752e-02
5.9578798692850e-02 9.4046770000159e-01 8.3321928319217e-02
7.2331779373452e-01 9.4317047600472e-01 7.1959176384143e-02
3.8702507591990e-01 9.4587316335213e-01 6.0596578799400e-02
5.0713439718057e-02 9.4857573935756e-01 4.9234176152498e-02
7.1439517013142e-01 9.5127818226281e-01 3.7872006827843e-02
3.7808212604729e-01 9.5398047079551e-01 2.6510107640952e-02
4.1785811823525e-02 9.5668258381764e-01 1.5148514327466e-02
7.0551743733910e-01 9.5938450006321e-01 3.7872618876713e-03
3.6902765192388e-01 9.6208696619701e-01 9.9242547836576e-01
3.2830098939063e-02 9.6478853610459e-01 9.8106488329058e-01
6.9669254357693e-01 9.6748984192481e-01 9.6970473221718e-01
3.6062537775606e-01 9.7019086122065e-01 9.5834505753737e-01
2.4638811175050e-02 9.7289157169802e-01 9.4698589017385e-01
6.8874284841259e-01 9.7559195168340e-01 9.3562725882985e-01
3.5294724695973e-01 9.7829198079479e-01 9.2426918899065e-01
1.7261454211506e-02 9.8099164083177e-01 9.1291170166067e-01
6.8169452163460e-01 9.8369091690754e-01 9.0155481182871e-01
3.4625499484260e-01 9.8638979883933e-01 8.9019852666916e-01
1.0950779253859e-02 9.8908828280049e-01 8.7884284350942e-01
6.7578898247730e-01 9.9178637321837e-01 8.6748774762487e-01
3.4077573662976e-01 9.9448408487337e-01 8.5613320996233e-01
5.9160064245449e-03 9.9718144511822e-01 8.4477918493891e-01
6.7166199069075e-01 9.9987636802428e-01 8.3342804804653e-01
3.3713119759385e-01 2.5730859156613e-03 8.2207492992310e-01
2.7597495745970e-03 5.2696329304231e-03 8.1072206984797e-01
6.6854611448861e-01 7.9661059602013e-03 7.9936934333631e-01
3.3448683231078e-01 1.0662621440741e-02 7.8801660759368e-01
5.7651044033436e-04 1.3359314287165e-02 7.7666370475064e-01
6.6680789757592e-01 1.6056335502726e-02 7.6531046661830e-01
3.3317203685382e-01 1.8753848389148e-02 7.5395672083498e-01
9.9965848904265e-01 2.1452023425116e-02 7.4260229808641e-01
6.6625560658230e-01 2.4151032033543e-02 7.3124703990504e-01
3.3295083121792e-01 2.6851039603359e-02 7.1989080642340e-01
9.9973098390375e-01 2.9552198242273e-02 7.0853348340082e-01
6.6658251686007e-01 3.2254639793463e-02 6.9717498787753e-01
3.3349170438038e-01 3.4958469642918e-02 6.8581527193271e-01
4.4476017272787e-04 3.7663761779348e-02 6.7445432421211e-01
6.6742788261936e-01 4.0370555459521e-02 6.6309216911642e-01
3.3442724274358e-01 4.3078853697411e-02 6.5172886376833e-01
1.4289403163261e-03 4.5788623653374e-02 6.4036449307649e-01
6.6841895940494e-01 4.8499798861953e-02 6.2899916336861e-01
3.3538315467392e-01 5.1212283111182e-02 6.1763299516405e-01
2.3072938890688e-03 5.3925955676185e-02 6.0626611569456e-01
6.6917717144710e-01 5.6640677520077e-02 5.9489865176264e-01
3.3597879436085e-01 5.9356298012311e-02 5.8353072345319e-01
2.6986285439227e-03 6.2072661687726e-02 5.7216243909528e-01
6.6932388206476e-01 6.4789614586140e-02 5.6079389171863e-01
3.3584279534058e-01 6.7507009774908e-02 5.4942515708364e-01
2.2449070444751e-03 7.0224711759711e-02 5.3805629320505e-01
6.6852126858718e-01 7.2942599617483e-02 5.2668734115875e-01
3.3466458808019e-01 7.5660568820446e-02 5.1531832687417e-01
9.9871666953458e-01 8.4645387440304e-01 5.0351355764418e-01
6.6822874527266e-01 8.4917459027265e-01 4.9214620546881e-01
3.3825130029058e-01 8.5189569908010e-01 4.8077879833747e-01
8.7514434259084e-03 8.5461673989789e-01 4.6941140954307e-01
6.7967933974539e-01 8.5733708668629e-01 4.5804425016414e-01
3.5097059650948e-01 8.6005598002910e-01 4.4667769579785e-01
2.2550414802396e-02 8.6277259528791e-01 4.3531228237148e-01
6.9433830942634e-01 8.6548614054169e-01 4.2394866012974e-01
3.6625194841735e-01 8.6819596741240e-01 4.1258750601619e-01
3.8209149313085e-02 8.7090167213018e-01 4.0122940833468e-01
7.1012808998874e-01 8.7360316484828e-01 3.8987474866883e-01
3.8192678812314e-01 8.7630069137462e-01 3.7852361050574e-01
5.3523312212660e-02 8.7899480129895e-01 3.6717573985451e-01
7.2483777818895e-01 8.8168626833608e-01 3.5583057046404e-01
3.9579618944473e-01 8.8437598027367e-01 3.4448730799050e-01
6.6335157380665e-02 8.8706482334975e-01 3.3314505007815e-01
7.3640605380880e-01 8.8975358551347e-01 3.2180291040832e-01
4.0597739778475e-01 8.9244289458924e-01 3.1046011819470e-01
7.5035016096073e-02 8.9513319501622e-01 2.9911607744223e-01
7.4358027355736e-01 8.9782475630946e-01 2.8777038516681e-01
4.1162710669914e-01 9.0051770122567e-01 2.7642281818989e-01
7.9198649843655e-02 9.0321204177865e-01 2.6507330182487e-01
7.4632405071488e-01 9.0590771448531e-01 2.5372187228375e-01
4.1303580779099e-01 9.0860461011903e-01 2.4236864081487e-01
7.9367738297710e-02 9.1130259635768e-01 2.3101376370299e-01
7.4535354410927e-01 9.1400153362156e-01 2.1965741940161e-01
4.1102587557010e-01 9.1670128527778e-01 2.0829979239312e-01
7.6415776100207e-02 9.1940172360455e-01 1.9694106262373e-01
7.4155240003795e-01 9.2210273278271e-01 1.8558139918569e-01
4.0646291629391e-01 9.2480420992805e-01 1.7422095703847e-01
7.1172531917607e-02 9.2750606491119e-01 1.6285987578905e-01
7.3570458847803e-01 9.3020821948452e-01 1.5149827979337e-01
4.0008069896696e-01 9.3291060605985e-01 1.4013627905123e-01
6.4320903878853e-02 9.3561316635385e-01 1.2877397053283e-01
7.2844383286426e-01 9.3831585003140e-01 1.1741143969886e-01
3.9246686364879e-01 9.4101861341934e-01 1.0604876206315e-01
5.6406273428179e-02 9.4372141832653e-01 9.4686004707067e-02
7.2027738021993e-01 9.4642423098383e-01 8.3323227694586e-02
3.8409467306528e-01 9.4912702110481e-01 7.1960485362210e-02
4.7871930810781e-02 9.5182976106125e-01 6.0597827473251e-02
7.1162232964739e-01 9.5453242516487e-01 4.9235300234603e-02
3.7535853977613e-01 9.5723498904646e-01 3.7872947178014e-02
3.9092811587304e-02 9.5993742912479e-01 2.6510809908739e-02
7.0283705163931e-01 9.6263972216025e-01 1.5148928723245e-02
3.6660288853652e-01 9.6534184489103e-01 3.7873430949524e-03
3.0177239196720e-02 9.6804444276827e-01 9.9242530311648e-01
6.9400235295184e-01 9.7074626741121e-01 9.8106429642022e-01
3.5788255482524e-01 9.7344784794728e-01 9.6970370288244e-01
2.1828806718801e-02 9.7614915917221e-01 9.5834356065360e-01
6.8585195372435e-01 9.7885017547975e-01 9.4698390719006e-01
3.4996271433548e-01 9.8155087117776e-01 9.3562477869107e-01
1.4171652071059e-02 9.8425122099212e-01 9.2426620927810e-01
6.7848912589304e-01 9.8695120079123e-01 9.1290822988676e-01
3.4292521668950e-01 9.8965078856506e-01 9.0155086684589e-01
7.4896272476008e-03 9.9234996569091e-01 8.9019414012820e-01
6.7219155368781e-01 9.9504871851070e-01 8.7883806127387e-01
3.3703952741661e-01 9.9774704023042e-01 8.6748263101579e-01
2.4365371898946e-03 4.4305394914697e-04 8.5612999080286e-01
6.6761215474813e-01 3.1404433736240e-03 8.4477590364203e-01
3.3295330250923e-01 5.8374477894985e-03 8.3342237376817e-01
9.9846377082614e-01 8.5341120347069e-03 8.2206933241612e-01
6.6414558695809e-01 1.1230500233852e-02 8.1071668792296e-01
3.2999881031600e-01 1.3926697779232e-02 7.9936432493497e-01
9.9602137533582e-01 1.6622812518228e-02 7.8801210492721e-01
6.6220900344095e-01 1.9318974811550e-02 7.7665986835151e-01
3.2855520213368e-01 2.2015336157352e-02 7.6530743862241e-01
9.9505136185525e-01 2.4712066163009e-02 7.5395462797710e-01
6.6168695021674e-01 2.7409347789907e-02 7.4260124501842e-01
3.2844978996185e-01 3.0107370983888e-02 7.3124710350065e-01
9.9532639402435e-01 3.2806325008297e-02 7.1989203169080e-01
6.6230232125830e-01 3.5506389982239e-02 7.0853588148214e-01
3.2936251254039e-01 3.8207728258524e-02 6.9717853638948e-01
9.9649157052702e-01 4.0910476328996e-02 6.8581991763422e-01
6.6367395722916e-01 4.3614737912216e-02 6.7445998771905e-01
3.3089409979315e-01 4.6320578770433e-02 6.6309875116405e-01
9.9813641317804e-01 4.9028023643014e-02 6.5173625238001e-01
6.6538526502972e-01 5.1737055498583e-02 6.4037257094461e-01
3.3262491957534e-01 5.4447617120767e-02 6.2900781478780e-01
9.9983950148045e-01 5.7159614866512e-02 6.1764211196242e-01
6.6701301650537e-01 5.9872924279163e-02 6.0627560176506e-01
3.3412945428705e-01 6.2587397106469e-02 5.9490842598057e-01
1.1729819760592e-03 6.5302869174402e-02 5.8354072095450e-01
6.6812821916881e-01 6.8019168512957e-02 5.7217261106087e-01
3.3498056836291e-01 7.0736123131876e-02 5.6080420394349e-01
1.7165642952092e-03 7.3453567909157e-02 5.4943558769226e-01
6.6832420195046e-01 7.6171350178443e-02 5.3806682990006e-01
3.3479320691702e-01 7.8889333765981e-02 5.2669797836287e-01
1.1152215589269e-03 8.1607401406881e-02 5.1532906305753e-01
6.6554134255252e-01 8.5240258934421e-01 5.0352126445315e-01
3.3455015669843e-01 8.5512283189285e-01 4.9215373132143e-01
4.0116595218639e-03 8.5784341680169e-01 4.8078616405317e-01
6.7389931775008e-01 8.6056396739771e-01 4.6941861702464e-01
3.4417313906624e-01 8.6328397555239e-01 4.5805125049981e-01
1.4781249746386e-02 8.6600282183617e-01 4.4668435285654e-01
6.8566281199154e-01 8.6871982139885e-01 4.3531834269237e-01
3.5675155536224e-01 8.7143429256156e-01 4.2395374253189e-01
2.7978947371561e-02 8.7414563833485e-01 4.1259112215617e-01
6.9927620842169e-01 8.7685342648380e-01 4.0123101807866e-01
3.7057494508457e-01 8.7955745286999e-01 3.8987384355467e-01
4.1806862945871e-02 8.8225777549157e-01 3.7851980815481e-01
7.1290347956356e-01 8.8495471190948e-01 3.6716886581483e-01
3.8379675808954e-01 8.8764879964320e-01 3.5582070491060e-01
5.4421122763133e-02 8.9034072671068e-01 3.4447478402515e-01
7.2471663026122e-01 8.9303124611594e-01 3.3313040518212e-01
3.9463250206766e-01 8.9572109138677e-01 3.2178680658393e-01
6.4130042891863e-02 8.9841090855278e-01 3.1044325329469e-01
7.3318421951237e-01 9.0110121378678e-01 2.9909910807106e-01
4.0178366516757e-01 9.0379237806796e-01 2.8775387323816e-01
6.9929340338760e-02 9.0648463386514e-01 2.7640720378488e-01
7.3763234410227e-01 9.0917809577812e-01 2.6505889822061e-01
4.0491140510414e-01 9.1187278718120e-01 2.5370887607191e-01
7.1790465339189e-02 9.1456866689672e-01 2.4235715008649e-01
7.3829660312848e-01 9.1726565239802e-01 2.3100379887872e-01
4.0445839354555e-01 9.1996363811828e-01 2.1964894324208e-01
7.0304703509670e-02 9.2266250881050e-01 2.0829272738405e-01
7.3586386470880e-01 9.2536214861714e-01 1.9693530508638e-01
4.0116314793341e-01 9.2806244675894e-01 1.8557683015193e-01
6.6228463300731e-02 9.3076330073799e-01 1.7421745027255e-01
7.3108422144407e-01 9.3346461781431e-01 1.6285730346208e-01
3.9575330428067e-01 9.3616631534622e-01 1.5149651631731e-01
6.0257106909068e-02 9.3886832042565e-01 1.4013520352205e-01
7.2461562305339e-01 9.4157056910920e-01 1.2877346815530e-01
3.8884755493997e-01 9.4427300544604e-01 1.1741140248766e-01
5.2970434771043e-02 9.4697558043135e-01 1.0604908904667e-01
7.1700074945000e-01 9.4967825096398e-01 9.4686601804099e-02
3.8095406332963e-01 9.5238097885327e-01 8.3324007389511e-02
4.4845135859040e-02 9.5508372989808e-01 7.1961366270929e-02
7.0868803238351e-01 9.5778647304769e-01 6.0598733867092e-02
3.7249622721697e-01 9.6048917964612e-01 4.9236161571570e-02
3.6282698614427e-02 9.6319182275784e-01 3.7873697678310e-02
7.0006001552738e-01 9.6589437657085e-01 2.6511388203092e-02
3.6384041610297e-01 9.6859681587408e-01 1.5149277597275e-02
2.7635877834500e-02 9.7129911560713e-01 3.7874093497042e-03
6.9126998143407e-01 9.7400181897169e-01 9.9242515666791e-01
3.5511257770694e-01 9.7670387830963e-01 9.8106377149192e-01
1.9004943758610e-02 9.7940571888871e-01 9.6970275891624e-01
6.8295853520370e-01 9.8210731329798e-01 9.5834216207282e-01
3.4698475415496e-01 9.8480863327563e-01 9.4698202410645e-01
1.1094953466633e-02 9.8750964985386e-01 9.3562238782588e-01
6.7530042371737e-01 9.9021033367625e-01 9.2426329510244e-01
3.3961235989050e-01 9.9291065552460e-01 9.1290478597670e-01
4.0418043105672e-03 9.9561058709685e-01 9.0154689743236e-01
6.6859956216676e-01 9.9831010208088e-01 8.9018966179974e-01
3.3363447793095e-01 1.0075636288714e-03 8.7883495466475e-01
9.9849420001383e-01 3.7060877679046e-03 8.6747919936186e-01
6.6351176587739e-01 6.4041463397277e-03 8.5612414190781e-01
3.2869552953494e-01 9.1017389756669e-03 8.4476977218378e-01
9.9405268330137e-01 1.1798878184884e-02 8.3341606188397e-01
6.5958898032164e-01 1.4495592503690e-02 8.2206296128558e-01
3.2530844578421e-01 1.7191929692432e-02 8.1071039635514e-01
9.9121309558455e-01 1.9887959712699e-02 7.9935826657470e-01
6.5730268648122e-01 2.2583777135755e-02 7.8800644393295e-01
3.2357452527558e-01 2.5279502570608e-02 7.7665477354322e-01
9.9002336466957e-01 2.7975282678992e-02 7.6530307629529e-01
6.5664140878487e-01 3.0671288386824e-02 7.5395115380450e-01
3.2341844114288e-01 3.3367711022073e-02 7.4259879568458e-01
9.9034207278557e-01 3.6064756307360e-02 7.3124578886290e-01
6.5739809037150e-01 3.8762636391214e-02 7.1989192832342e-01
3.2457086716431e-01 4.1461560374262e-02 7.0853702837119e-01
9.9184378811557e-01 4.4161724023749e-02 6.9718093332953e-01
6.5919963734941e-01 4.6863299523263e-02 6.8582352655778e-01
3.2662090396610e-01 4.9566426144095e-02 6.7446473682677e-01
9.9408997923008e-01 5.2271202647099e-02 6.6310454138535e-01
6.6158924140051e-01 5.4977682051447e-02 6.5174296543688e-01
3.2910104861996e-01 5.7685869175829e-02 6.4038007815080e-01
9.9660768005771e-01 6.0395721105657e-02 6.2901598569943e-01
6.6409127659123e-01 6.3107150493950e-02 6.1765082209051e-01
3.3153383231515e-01 6.5820031379566e-02 6.0628473874080e-01
9.9891727709374e-01 6.8534207012921e-02 5.9491789379674e-01
6.6622367060408e-01 7.1249499026037e-02 5.8355044215897e-01
3.3343550417248e-01 7.3965717185448e-02 5.7218252701724e-01
5.3608352559609e-04 7.6682668941080e-02 5.6081427347057e-01
6.6750994844189e-01 7.9400168044589e-02 5.4944578452531e-01
3.3434327768878e-01 8.2118041655659e-02 5.3807713947003e-01
1.0242305354938e-03 8.4836135564416e-02 5.2670839436820e-01
6.6754318785334e-01 8.7554317396653e-02 5.1533958422117e-01
3.3234822710068e-01 8.5835110107233e-01 5.0352933051018e-01
9.1076297438926e-04 8.6107094094390e-01 4.9216162276688e-01
6.6987687697237e-01 8.6379107856589e-01 4.8079389816650e-01
3.3922489154914e-01 8.6651120218001e-01 4.6942619765994e-01
8.9223224062446e-03 8.6923089432714e-01 4.5805864471458e-01
6.7892693368521e-01 8.7194964478310e-01 4.4669146356932e-01
3.4918880786212e-01 8.7466688189314e-01 4.3532498415787e-01
1.9652986871121e-02 8.7738202110088e-01 4.2395962754105e-01
6.9026203253945e-01 8.8009452499594e-01 4.1259586929304e-01
3.6095789464789e-01 8.8280396573819e-01 4.0123418358135e-01
3.1682779334832e-02 8.8551007937293e-01 3.8987497609230e-01
7.0237915592676e-01 8.8821280252110e-01 3.7851851786210e-01
3.7298942621854e-01 8.9091228471212e-01 3.6716489336398e-01
4.3455931587727e-02 8.9360887362016e-01 3.5581397435660e-01
7.1372182195893e-01 8.9630307519427e-01 3.4446542605998e-01
3.8373292468662e-01 8.9899549547598e-01 3.3311874503360e-01
5.3440306053348e-02 9.0168677459258e-01 3.2177332059531e-01
7.2280291367036e-01 9.0437752462244e-01 3.1042850631352e-01
3.9178964783100e-01 9.0706828108206e-01 2.9908368701948e-01
6.0380418308651e-02 9.0975947338583e-01 2.8773833008214e-01
7.2856607066319e-01 9.1245141455724e-01 2.7639201557799e-01
3.9634735880232e-01 9.1514430653575e-01 2.6504444589321e-01
6.3733303405204e-02 9.1783825555810e-01 2.5369543929195e-01
7.3073929843988e-01 9.2053329216231e-01 2.4234491349728e-01
3.9738525636245e-01 9.2322939160751e-01 2.3099286488680e-01
6.3693975487700e-02 9.2592649210274e-01 2.1963934744978e-01
7.2968981345167e-01 9.2862450964265e-01 2.0828445400764e-01
3.9539767852020e-01 9.3132334923103e-01 1.9692830083383e-01
6.0842308172408e-02 9.3402291283549e-01 1.8557101588006e-01
7.2604778568097e-01 9.3672310465819e-01 1.7421273028896e-01
3.9103724200988e-01 9.3942383434945e-01 1.6285357264227e-01
5.5832695175190e-02 9.4212501872767e-01 1.5149366534673e-01
7.2045498740627e-01 9.4482658246583e-01 1.4013312261043e-01
3.8492378915833e-01 9.4752845809662e-01 1.2877204955316e-01
4.9257646859572e-02 9.5023058559275e-01 1.1741054209262e-01
7.1347405771061e-01 9.5293291170242e-01 1.0604868733727e-01
3.7758955984505e-01 9.5563538916131e-01 9.4686564290993e-02
4.1619829878959e-02 9.5833797586003e-01 8.3324244732101e-02
7.0557978256118e-01 9.6104063401605e-01 7.1961794173010e-02
3.6948366912074e-01 9.6374332937869e-01 6.0599272838064e-02
3.3345172191898e-02 9.6644603048289e-01 4.9236736619046e-02
6.9717749604088e-01 9.6914870795856e-01 3.7874237982673e-02
3.6099345133916e-01 9.7185133389818e-01 2.6511826813736e-02
2.4805534013913e-02 9.7455388128294e-01 1.5149551182886e-02
6.8862599781426e-01 9.7725632346731e-01 3.7874580303175e-03
3.5231548087886e-01 9.7995910037213e-01 9.9242504465357e-01
1.6170427690085e-02 9.8266136805010e-01 9.8106332341301e-01
6.8006945608394e-01 9.8536344706145e-01 9.6970192542359e-01
3.4402443412712e-01 9.8806530835848e-01 9.5834089790629e-01
8.0472357149158e-03 9.9076692170784e-01 9.4698028870191e-01
6.7214975902244e-01 9.9346825566501e-01 9.3562014614812e-01
3.3634392975672e-01 9.9616927769792e-01 9.2426051872408e-01
6.4168598962013e-04 9.9886995449763e-01 9.1290145441644e-01
6.6533279802074e-01 1.5689200756160e-03 9.0154452671091e-01
3.2988876276061e-01 4.2687066130714e-03 8.9018683830603e-01
9.9458355404701e-01 6.9680522088934e-03 8.7882984022474e-01
6.5942849327090e-01 9.6669290298212e-03 8.6747356472095e-01
3.2443441160500e-01 1.2365314616625e-02 8.5611803440640e-01
9.8961143297140e-01 1.5063194407403e-02 8.4476325898705e-01
6.5496871247914e-01 1.7760564755313e-02 8.3340923158473e-01
3.2051413369773e-01 2.0457436390645e-02 8.2205592479785e-01
9.8625397364838e-01 2.3153838201575e-02 8.1070328675711e-01
6.5219255133317e-01 2.5849821110693e-02 7.9935123754838e-01
3.1833188335192e-01 2.8545461712051e-02 7.8799966649033e-01
9.8467137742051e-01 3.1240865221688e-02 7.7664843084196e-01
6.5120759951000e-01 3.3936167207937e-02 7.6529735654183e-01
3.1793415059050e-01 3.6631533538143e-02 7.5394624150906e-01
9.8484168250462e-01 3.9327158037554e-02 7.4259486183877e-01
6.5191806830997e-01 4.2023257525346e-02 7.3124298089822e-01
3.1914872149685e-01 4.4720064169921e-02 7.1989036090730e-01
9.8651703425236e-01 4.7417815457595e-02 7.0853677613896e-01
6.5400488291914e-01 5.0116742433690e-02 6.9718202650071e-01
3.2159313516528e-01 5.2817057176509e-02 6.8582595005255e-01
9.8926209293000e-01 5.5518940636549e-02 6.7446843304365e-01
6.5699181947675e-01 5.8222531980465e-02 6.6310941631392e-01
3.2476232533664e-01 6.0927920428662e-02 6.5174889735600e-01
9.9255362055048e-01 6.3635140307589e-02 6.4038692787487e-01
6.6034567166557e-01 6.6344169708594e-02 6.2902360721827e-01
3.2811832404490e-01 6.9054932803912e-02 6.1765907250000e-01
9.9585125803333e-01 7.1767305549361e-02 6.0629348655204e-01
6.6352403955077e-01 7.4481124219566e-02 5.9492702499802e-01
3.3111630359314e-01 7.7196195987694e-02 5.8355986373915e-01
9.9860807808587e-01 7.9912310595875e-02 5.7219216799281e-01
6.6598022292128e-01 8.2629252089856e-02 5.6082408374721e-01
3.3321493257036e-01 8.5346809634739e-02 5.4945573212976e-01
2.9623640288902e-04 8.8064786593703e-02 5.3808720678896e-01
6.6721043130974e-01 9.0783007317190e-02 5.2671857402248e-01
3.3394639486040e-01 9.3501321408737e-02 5.1534987510759e-01
9.9913997374845e-01 8.6429942035092e-01 5.0353774146777e-01
6.6730465070629e-01 8.6701891651860e-01 4.9216986576839e-01
3.3583074096379e-01 8.6973867112368e-01 4.8080198751598e-01
4.7003324178959e-03 8.7245842314321e-01 4.6943413800445e-01
6.7388675018654e-01 8.7517782577309e-01 4.5806641364509e-01
3.4335526899288e-01 8.7789645478733e-01 4.4669899089264e-01
1.3064595974271e-02 8.8061383022108e-01 4.3533213218595e-01
6.8296885062853e-01 8.8332945106141e-01 4.2396617835572e-01
3.5301960622721e-01 8.8604283965942e-01 4.1260152503178e-01
2.3167538414825e-02 8.8875359003638e-01 4.0123858389344e-01
6.9336338173100e-01 8.9146141291947e-01 3.8987773327949e-01
3.6355817212149e-01 8.9416617048606e-01 3.7851926570148e-01
3.3703041963394e-02 8.9686789521154e-01 3.6716334140316e-01
7.0374901526275e-01 8.9956678952483e-01 3.5580995684615e-01
3.7364725067853e-01 9.0226320587605e-01 3.4445893474625e-01
4.3349998886523e-02 9.0495761002610e-01 3.3310993825833e-01
7.1281225232927e-01 9.0765053336249e-01 3.2176250687804e-01
3.8199378806243e-01 9.1034252202268e-01 3.1041610695243e-01
5.0861149383891e-02 9.1303409080889e-01 2.9907018690100e-01
7.1938913053745e-01 9.1572568811652e-01 2.8772422728365e-01
3.8756149116235e-01 9.1841767499441e-01 2.7637777849725e-01
5.5370847262273e-02 9.2111031813817e-01 2.6503048289426e-01
7.2281787313448e-01 9.2380379413064e-01 2.5368208195030e-01
3.8991005192632e-01 9.2649820107696e-01 2.4233241167613e-01
5.6660227548382e-02 9.2919357382719e-01 2.3098139047201e-01
7.2308516542829e-01 9.3188989977794e-01 2.1962900337873e-01
3.8920425954864e-01 9.3458713329936e-01 2.0827528575602e-01
5.5038455429013e-02 9.3728520779592e-01 1.9692030832503e-01
7.2060940696285e-01 9.3998404511846e-01 1.8556416455763e-01
3.8593885234854e-01 9.4268356248126e-01 1.7420696070828e-01
5.1048177850121e-02 9.4538367725311e-01 1.6284880836219e-01
7.1595813289309e-01 9.4808431005725e-01 1.5148981916072e-01
3.8068866144371e-01 9.5078538659516e-01 1.4013010129299e-01
4.5258819301150e-02 9.5348683854953e-01 1.2876975735202e-01
7.0968675265600e-01 9.5618860384976e-01 1.1740888320378e-01
3.7398971881795e-01 9.5889062651525e-01 1.0604756758128e-01
3.8184134827406e-02 9.6159285623341e-01 9.4685892178879e-02
7.0228564356843e-01 9.6429524778300e-01 8.3323932077260e-02
3.6630919002432e-01 9.6699776037827e-01 7.1961756375456e-02
3.0269102579038e-02 9.6970035698346e-01 6.0599428941263e-02
6.9417917591214e-01 9.7240300362906e-01 4.9237009218472e-02
3.5805275319383e-01 9.7510566874874e-01 3.7874553048520e-02
2.1902806074939e-02 9.7780832254771e-01 2.6512113477691e-02
6.8574201147163e-01 9.8051093640820e-01 1.5149741529861e-02
3.4958282443400e-01 9.8321348233539e-01 3.7874869299863e-03
1.3323332366135e-02 9.8591629594274e-01 9.9242497195924e-01
6.7718556346640e-01 9.8861873997513e-01 9.8106296543715e-01
3.4108586289002e-01 9.9132102942427e-01 9.6970122523171e-01
5.0364269044239e-03 9.9402313416341e-01 9.5833980167968e-01
6.6904951157897e-01 9.9672502266878e-01 9.4697874622102e-01
3.3313746488862e-01 9.9942666183461e-01 9.3561811149904e-01
9.9752646094899e-01 2.1269821051356e-03 9.2425913665685e-01
6.6181793541854e-01 4.8279114302771e-03 9.1289962462877e-01
3.2622185319764e-01 7.5284847300780e-03 9.0154069402797e-01
9.9075094706362e-01 1.0228663721436e-02 8.9018239851654e-01
6.5541793828340e-01 1.2928410278336e-02 8.7882478935043e-01
3.2023542964159e-01 1.5627687703374e-02 8.6746791338350e-01
9.8521575539568e-01 1.8326462477549e-02 8.5611181048610e-01
6.5037078190147e-01 2.1024706563256e-02 8.4475651033590e-01
3.1571165406645e-01 2.3722400317302e-02 8.3340202858235e-01
9.8124848570720e-01 2.6419536032490e-02 8.2204836245675e-01
6.4698999675330e-01 2.9116122061423e-02 8.1069548600189e-01
3.1294310732841e-01 3.1812187379630e-02 7.9934334522909e-01
9.7911250793636e-01 3.4507786316786e-02 7.8799185366714e-01
6.4550023549085e-01 3.7203003032974e-02 7.7664088892775e-01
3.1210529502623e-01 3.9897955162694e-02 7.6529029103733e-01
9.7892337390064e-01 4.2592795927774e-02 7.5393986332748e-01
6.4594669580171e-01 4.5287713978068e-02 7.4258937658103e-01
3.1316405281328e-01 4.7982930302859e-02 7.3123857685681e-01
9.8056103378483e-01 5.0678691796697e-02 7.1988719696021e-01
6.4812043793705e-01 5.3375261455328e-02 7.0853497094032e-01
3.1582282947624e-01 5.6072905666667e-02 6.9718165038802e-01
9.8364716039258e-01 5.8771879552565e-02 6.8582702083041e-01
6.5157137352936e-01 6.1472411697183e-02 6.7447091630023e-01
3.1957290262283e-01 6.4174689777229e-02 6.6311323028194e-01
9.8762901106613e-01 6.6878848552000e-02 6.5175392168264e-01
6.5571695120584e-01 6.9584961403654e-02 6.4039301514809e-01
3.2381397097709e-01 7.2293036211326e-02 6.2903059579711e-01
9.9189723304965e-01 7.5003015877826e-02 6.1766679914265e-01
6.5994373360202e-01 7.7714783368391e-02 6.0630179750376e-01
3.2793030784698e-01 8.0428170703663e-02 5.9493578453751e-01
9.9583378749834e-01 8.3142970994298e-02 5.8356895961686e-01
6.6363133701436e-01 8.5858952334614e-02 5.7220151365730e-01
3.3130095046628e-01 8.8575872221098e-02 5.6083361767645e-01
9.9882205083672e-01 9.1293491165296e-02 5.4946541490390e-01
6.6617610850728e-01 9.4011584348406e-02 5.3809701671842e-01
3.3334719109108e-01 9.6729950497642e-02 5.2672850216839e-01
3.2237185986805e-04 9.9448417591073e-02 5.1535992042047e-01
6.6591870944252e-01 8.7024755740691e-01 5.0354648266213e-01
3.3372710231254e-01 8.7296675945972e-01 4.9217844591026e-01
1.8603609850503e-03 8.7568618530255e-01 4.8081041839130e-01
6.7030353361329e-01 8.7840561407283e-01 4.6944242437769e-01
3.3903449917185e-01 8.8112475469557e-01 4.5807454011517e-01
8.0244517868348e-03 8.8384325129785e-01 4.4670690602397e-01
6.7723897283818e-01 8.8656069839160e-01 4.3533973274723e-01
3.4663952121753e-01 8.8927666592412e-01 4.2397329769178e-01
1.6185049792277e-02 8.9199073230825e-01 4.1260792991498e-01
6.8583341696499e-01 8.9470252170646e-01 4.0124398329432e-01
3.5554233521013e-01 8.9741174068155e-01 3.8988180039444e-01
2.5269765963148e-02 9.0011820910333e-01 3.7852167169777e-01
6.9497387278134e-01 9.0282188086306e-01 3.6716379635097e-01
3.6461280686571e-01 9.0552285127292e-01 3.5580825096134e-01
3.4144660630146e-02 9.0822134980331e-01 3.4445497210137e-01
7.0352786339836e-01 9.1091771888195e-01 3.3310375606338e-01
3.7272213113064e-01 9.1361238162962e-01 3.2175427632119e-01
4.1689879530116e-02 9.1630580323944e-01 3.1040611572835e-01
7.1039784237760e-01 9.1899845169787e-01 2.9905880763499e-01
3.7881856394548e-01 9.2169076330358e-01 2.8771187871803e-01
4.6931473497021e-02 9.2438311698594e-01 2.7636488679737e-01
7.1472337220732e-01 9.2707581923888e-01 2.6501744894824e-01
3.8218831208106e-01 9.2976909928745e-01 2.5366925797561e-01
4.9326969073238e-02 9.3246311249897e-01 2.4232008785082e-01
7.1614567952522e-01 9.3515794930457e-01 2.3096979039817e-01
3.8265531928811e-01 9.3785364692686e-01 2.1961828619742e-01
4.8870175775499e-02 9.4055020173152e-01 2.0826555253559e-01
7.1480691660132e-01 9.4324758072839e-01 1.9691161064664e-01
3.8048371173751e-01 9.4594573141419e-01 1.8555651373904e-01
4.5919528816446e-02 9.4864458966389e-01 1.7420033664087e-01
7.1113359616888e-01 9.5134408571290e-01 1.6284316738308e-01
3.7614501423493e-01 9.5404414845622e-01 1.5148510070892e-01
4.0972490269778e-02 9.5674470836296e-01 1.4012623331169e-01
7.0563417081668e-01 9.5944569931005e-01 1.2876666052310e-01
3.7014754884307e-01 9.6214705960684e-01 1.1740647416115e-01
3.4529426078903e-02 9.6484873243662e-01 1.0604576126951e-01
6.9879591495699e-01 9.6755066589288e-01 9.4684603519875e-02
3.6296246810202e-01 9.7025281274479e-01 8.3323077092338e-02
2.7043926323763e-02 9.7295513003015e-01 7.1961252889390e-02
6.9105457849139e-01 9.7565757854562e-01 6.0599196974423e-02
3.5500822854063e-01 9.7836012228187e-01 4.9236971153995e-02
1.8918266272196e-02 9.8106272783570e-01 3.7874633645405e-02
6.8279773962120e-01 9.8376536381965e-01 2.6512239787711e-02
3.4665942679260e-01 9.8646800028174e-01 1.5149842766290e-02
1.0515907128767e-02 9.8917060814337e-01 3.7874943290623e-03
6.7430310787197e-01 9.9187341769155e-01 9.9242494256476e-01
3.3816762966064e-01 9.9457600133471e-01 9.8106270873515e-01
2.0639307475160e-03 9.9727846761191e-01 9.6970067815052e-01
6.6613310289120e-01 9.9998017754162e-01 9.5833960093212e-01
3.3014770856312e-01 2.6822027488142e-03 9.4697826324103e-01
9.9423164563232e-01 5.3840157032091e-03 9.3561728862109e-01
6.5839779961351e-01 8.0855803916912e-03 9.2425673460784e-01
3.2265926568311e-01 1.0786858636902e-02 9.1289666058030e-01
9.8702935500643e-01 1.3487810139289e-02 9.0153712743745e-01
6.5152158006475e-01 1.6188392733434e-02 8.9017819693849e-01
3.1614961305333e-01 1.8888562926927e-02 8.7881993062220e-01
9.8092721024842e-01 2.1588276804837e-02 8.6746238821008e-01
6.4586809419004e-01 2.4287491396217e-02 8.5610562539289e-01
3.1098578485384e-01 2.6986166607901e-02 8.4474969090566e-01
9.7629337107797e-01 2.9684267830065e-02 8.3339462281993e-01
6.4180321490015e-01 3.2381769300344e-02 8.2204044403098e-01
3.0752658480395e-01 3.5078658269808e-02 8.1068715700400e-01
9.7347321985337e-01 3.7774939934716e-02 7.9933473797235e-01
6.3965083582166e-01 4.0470642974264e-02 7.8798313095706e-01
3.0606459668107e-01 4.3165825364358e-02 7.7663224219020e-01
9.7271658923868e-01 4.5860579930347e-02 7.6528193574776e-01
6.3960535289436e-01 4.8555038886490e-02 7.5393203137928e-01
3.0672552653045e-01 5.1249376437375e-02 7.4258230558896e-01
9.7406767543028e-01 5.3943808455473e-02 7.3123249689441e-01
6.4161834825180e-01 5.6638588370205e-02 7.1988231580467e-01
3.0936038544681e-01 5.9333998751557e-02 7.0853145941282e-01
9.7727345899325e-01 6.2030338632473e-02 6.9717962967637e-01
6.4533477761531e-01 6.4727907300866e-02 6.8582655363253e-01
3.1351985476911e-01 6.7426985954180e-02 6.7447200318769e-01
9.8180322153340e-01 7.0127819081221e-02 6.6311581192317e-01
6.5015898081218e-01 7.2830597598026e-02 6.5175788665654e-01
3.1856114150104e-01 7.5535445588233e-02 6.4039821223097e-01
9.8698373082392e-01 7.8242412047811e-02 6.2903684899884e-01
6.5540074348305e-01 8.0951468426019e-02 6.1767392349776e-01
3.2378603075646e-01 8.3662512103955e-02 6.0630961369862e-01
9.9211324899017e-01 8.6375375337135e-02 5.9494413081479e-01
6.6035597063059e-01 8.9089838650356e-02 5.8357769994886e-01
3.2848801646229e-01 9.1805647241872e-02 5.7221054181370e-01
9.9648400729036e-01 9.4522528672947e-02 5.6084285742365e-01
6.6432007376225e-01 9.7240210043982e-02 5.4947481706706e-01
3.3197462078766e-01 9.9958433030643e-02 5.3810655414131e-01
9.9942902875563e-01 1.0267696556296e-01 5.2673816368950e-01
6.6666818889243e-01 1.0539560950133e-01 5.1536970486444e-01
3.3268615905801e-01 8.7619552206150e-01 5.0355553917368e-01
1.7431224170939e-04 8.7891447195667e-01 4.9218734846842e-01
6.6795536981288e-01 8.8163361509138e-01 4.8081917663433e-01
3.3601673615686e-01 8.8435276275427e-01 4.6945104279705e-01
4.3399025184750e-03 8.8707166831283e-01 4.5808300799128e-01
6.7290077772211e-01 8.8979003070913e-01 4.4671518515695e-01
3.4167047006616e-01 8.9250750520261e-01 4.3534774485969e-01
1.0616414552635e-02 8.9522372163530e-01 4.2398091425298e-01
6.7970364892602e-01 8.9793830915225e-01 4.1261496750517e-01
3.4889599946532e-01 9.0065092498920e-01 4.0125020723265e-01
1.8156966763698e-02 9.0336128399253e-01 3.8988693815118e-01
6.8745020001725e-01 9.0606918517803e-01 3.7852543579571e-01
3.5673958513529e-01 9.0877453188903e-01 3.6716591440129e-01
2.5989101094603e-02 9.1147734285825e-01 3.5580849862719e-01
6.9516267653293e-01 9.1417775256633e-01 3.4445320360482e-01
3.6422427857292e-01 9.1687600061626e-01 3.3309992675901e-01
3.3138388284392e-02 9.1957241130819e-01 3.2174845306635e-01
7.0187088807401e-01 9.2226736602653e-01 3.1039847313675e-01
3.7039024949214e-01 9.2496127214558e-01 2.9904961121313e-01
3.8668811651892e-02 9.2765453259717e-01 2.8770145848404e-01
7.0668391115851e-01 9.3034751984161e-01 2.7635360648386e-01
3.7441866697236e-01 9.3304055683384e-01 2.6500567595687e-01
4.1862314312477e-02 9.3573390603489e-01 2.5365733810041e-01
7.0901008395133e-01 9.3842776604081e-01 2.4230832701290e-01
3.7586270481924e-01 9.4112227434557e-01 2.3095844387764e-01
4.2425652096600e-02 9.4381751426700e-01 2.1960755454385e-01
7.0870827010322e-01 9.4651352408452e-01 2.0825558263397e-01
3.7472288020676e-01 9.4921030678535e-01 1.9690250023639e-01
4.0483952498020e-02 9.5190783929760e-01 1.8554831785329e-01
7.0600738739079e-01 9.5460608055701e-01 1.7419307476898e-01
3.7130992623531e-01 9.5730497812893e-01 1.6283683052889e-01
3.6408690911505e-02 9.6000447336529e-01 1.5147965784143e-01
7.0132084086189e-01 9.6270450522813e-01 1.4012163695504e-01
3.6606333050935e-01 9.6540501298236e-01 1.2876285140543e-01
3.0652748577930e-02 9.6810593797802e-01 1.1740338494984e-01
6.9510522184111e-01 9.7080722472903e-01 1.0604331948002e-01
3.5943636804035e-01 9.7350882146653e-01 9.4682733711740e-02
2.3661285347014e-02 9.7621068031252e-01 8.3321702471477e-02
6.8779456831906e-01 9.7891275718714e-01 7.1960296430049e-02
3.5185034259420e-01 9.8161501153542e-01 6.0598582162041e-02
1.5842312476886e-02 9.8431740593603e-01 4.9236622436726e-02
6.7978381710939e-01 9.8701990563679e-01 3.7874476668606e-02
3.4368789210221e-01 9.8972247804767e-01 2.6512201473906e-02
7.5673343842737e-03 9.9242509221211e-01 1.5149851293363e-02
6.7143476866826e-01 9.9512771827008e-01 3.7874790519854e-03
3.3526435166020e-01 9.9783048017475e-01 9.9242495941385e-01
9.9918267576969e-01 5.3288974277852e-04 9.8106287505507e-01
6.6308805128114e-01 3.2353762787297e-03 9.6970074722630e-01
3.2703115910409e-01 5.9377560327596e-03 9.5833880646302e-01
9.9102474409101e-01 8.6399968064954e-03 9.4697710791817e-01
6.5508175525578e-01 1.1342064506303e-02 9.3561570888431e-01
3.1921539464030e-01 1.4043922761137e-02 9.2425466916272e-01
9.8343915735194e-01 1.6745532619458e-02 9.1289405127867e-01
6.4776685944367e-01 1.9446852360158e-02 9.0153392048836e-01
3.1221264923165e-01 2.2147837465365e-02 8.9017434450717e-01
9.7679099633861e-01 2.4848440818842e-02 8.7881539287191e-01
6.4151665124009e-01 2.7548613212282e-02 8.6745713583094e-01
3.0640456645413e-01 3.0248304262166e-02 8.5609964263916e-01
9.7146976892437e-01 3.2947463859982e-02 8.4474297912313e-01
6.3672717191183e-01 3.5646044294508e-02 8.3338720438196e-01
3.0219131432838e-01 3.8344003190228e-02 8.2203236651054e-01
9.6787601664093e-01 4.1041307391001e-02 8.1067849728703e-01
6.3379394619168e-01 4.3737937870127e-02 7.9932560586878e-01
2.9995609207208e-01 4.6433895651605e-02 7.8797367170532e-01
9.6637116144550e-01 4.9129208568877e-02 7.7662263710941e-01
6.3304492565558e-01 5.1823938460224e-02 7.6527240021886e-01
2.9997956441577e-01 5.4518188114992e-02 7.5392280939705e-01
9.6717307654979e-01 5.7212106981138e-02 7.4257366038484e-01
6.3461884037645e-01 5.9905894396229e-02 7.3122469762463e-01
3.0230540836512e-01 6.2599799014854e-02 7.1987562100246e-01
9.7021660209926e-01 6.5294113282874e-02 7.0852609870475e-01
6.3833193200768e-01 6.7989162318651e-02 6.9717578595825e-01
3.0662730676580e-01 7.0685287375704e-02 6.8582434825452e-01
9.7507593390659e-01 7.3382825033950e-02 6.7447148652836e-01
6.4364926619783e-01 7.6082084160690e-02 6.6311696100255e-01
3.1231783643583e-01 7.8783323252657e-02 6.5176061029510e-01
9.8105185512774e-01 8.1486730863669e-02 6.4040236299052e-01
6.4982151472591e-01 8.4192411436918e-02 6.2904224005468e-01
3.1859703080324e-01 8.6900378127418e-02 6.1768034792276e-01
9.8734852862389e-01 8.9610553303725e-02 6.0631686348093e-01
6.5604592942455e-01 9.2322776514554e-02 5.9495201324254e-01
3.2465899072671e-01 9.5036818887442e-02 5.8358604965674e-01
9.9315760915015e-01 9.7752402244070e-02 5.7221922764013e-01
6.6151241614208e-01 1.0046922072574e-01 5.6085178411053e-01
3.2969561032107e-01 1.0318696250300e-01 5.4948392259326e-01
9.9768190095169e-01 1.0590532927144e-01 5.3811580399249e-01
6.6544940525166e-01 1.0862405172470e-01 5.2674754356215e-01
3.3298035414575e-01 1.1134289995771e-01 5.1537921319090e-01
9.9944373454160e-01 8.8214332381949e-01 5.0356489588471e-01
6.6664318042532e-01 8.8486205724451e-01 4.9219655847557e-01
3.3410734495110e-01 8.8758095695397e-01 4.8082824772934e-01
1.8255291290699e-03 8.9029986023670e-01 4.6945997897839e-01
6.6978212092978e-01 8.9301855625777e-01 4.5809180166205e-01
3.3795682137680e-01 8.9573678829098e-01 4.4672380754687e-01
6.3252193234915e-03 8.9845426138897e-01 4.3535613593442e-01
6.7485964867719e-01 9.0117065582282e-01 4.2398897401016e-01
3.4353019506591e-01 9.0388564570622e-01 4.1262255089905e-01
1.2305680048925e-02 9.0659892127196e-01 4.0125712483373e-01
6.8115445790191e-01 9.0931021252124e-01 3.8989296396868e-01
3.5004491577994e-01 9.1201931158691e-01 3.7853032255258e-01
1.8945657932241e-02 9.1472609119000e-01 3.6716941515750e-01
6.8782545116958e-01 9.1743051696853e-01 3.5581039226655e-01
3.5665308189186e-01 9.2013265212273e-01 3.4445332062813e-01
2.5397302107426e-02 9.2283265367008e-01 3.3309817134394e-01
6.9402701474109e-01 9.2553076058207e-01 3.2174481767715e-01
3.6251177541180e-01 9.2822727510368e-01 3.1039304314305e-01
3.0822591323688e-02 9.3092253949074e-01 2.9904255880371e-01
6.9893290803514e-01 9.3361691103220e-01 2.8769302718095e-01
3.6681961908987e-01 9.3631073836179e-01 2.7634408921074e-01
3.4463926558114e-02 9.3900434163541e-01 2.6499539044938e-01
7.0185191942501e-01 9.4169799826382e-01 2.5364660332764e-01
3.6897480352841e-01 9.4439193479787e-01 2.4229744339981e-01
3.5828787565367e-02 9.4708632455581e-01 2.3094767887355e-01
7.0241468484966e-01 9.4978128987650e-01 2.1959713387196e-01
3.6873732033428e-01 9.5247690755721e-01 2.0824568665068e-01
3.4804837460142e-02 9.5517321604791e-01 1.9689326432093e-01
7.0062798668201e-01 9.5787022320960e-01 1.8553983559410e-01
3.6621945594861e-01 9.6056791377727e-01 1.7418540280554e-01
3.1593280420419e-02 9.6326625600022e-01 1.6282999412678e-01
6.9676434893577e-01 9.6596520720804e-01 1.5147365653497e-01
3.6174801013616e-01 9.6866471824847e-01 1.4011644982789e-01
2.6559771813825e-02 9.7136473686739e-01 1.2875844176990e-01
6.9121508204900e-01 9.7406521016618e-01 1.1739970432375e-01
3.5572917884603e-01 9.7676608629496e-01 1.0604031085182e-01
2.0116995124359e-02 9.7946731553814e-01 9.4680334139713e-02
6.8439310717855e-01 9.8216885093204e-01 8.3319845091438e-02
3.4857171646940e-01 9.8487064853230e-01 7.1958911956967e-02
1.2666656454517e-02 9.8757266742601e-01 6.0597599971254e-02
6.7669141789089e-01 9.9027486956182e-01 4.9235971304736e-02
3.4065918754238e-01 9.9297721945350e-01 3.7874085245053e-02
4.5828964649503e-03 9.9567968379737e-01 2.6511998547372e-02
6.6847527503038e-01 9.9838223103221e-01 1.5149765905363e-02
3.3236806154688e-01 1.0847539828000e-03 3.7875294655919e-03
9.9621632849445e-01 3.7874538029183e-03 9.9242507453788e-01
6.6008971951695e-01 6.4900708448014e-03 9.8106271948616e-01
3.2398156650164e-01 9.1926524820823e-03 9.6970042772311e-01
9.8790453828530e-01 1.1895168340535e-02 9.5833825280134e-01
6.5187148775751e-01 1.4597586710989e-02 9.4697625001131e-01
3.1589551561530e-01 1.7299874032975e-02 9.3561447699681e-01
9.7999003097553e-01 2.0001994400541e-02 9.2425299430834e-01
6.4416880687694e-01 2.2703909108195e-02 9.1289186585909e-01
3.0844602794528e-01 2.5405576263621e-02 9.0153115923678e-01
9.7283632647469e-01 2.8106950505717e-02 8.9017094580838e-01
6.3735480185693e-01 3.0807982882069e-02 8.7881130053562e-01
3.0201701663590e-01 3.3508620959840e-02 8.6745230139596e-01
9.6683896047899e-01 3.6208809268157e-02 8.5609402827775e-01
6.3183697110390e-01 3.8908490197907e-02 8.4473656119261e-01
2.9702759885698e-01 4.1607605514141e-02 8.3337997762640e-01
9.6242739957033e-01 4.4306098662722e-02 8.2202434884115e-01
6.2805263915714e-01 4.7003918068193e-02 8.1066973495611e-01
2.9391889411423e-01 4.9701021611486e-02 7.9931617869382e-01
9.6004053603688e-01 5.2397382425592e-02 7.8796369779842e-01
6.2643009702810e-01 5.5092996032080e-02 7.7661227634087e-01
2.9309752802080e-01 5.7787888638331e-02 7.6526185543343e-01
9.6004938420071e-01 6.0482126110586e-02 7.5391232427770e-01
6.2728799961162e-01 6.3175822741972e-02 7.4256351291966e-01
2.9481074206272e-01 6.5869148500649e-02 7.3121518847715e-01
9.6260946107370e-01 6.8562333080607e-02 7.1986705677750e-01
6.3067024420498e-01 7.1255664945287e-02 7.0851877110314e-01
2.9897356981200e-01 7.3949483818848e-02 6.9716994894556e-01
9.6749488292906e-01 7.6644165839378e-02 6.8582019631439e-01
6.3620553445371e-01 7.9340101786919e-02 6.7446913747016e-01
3.0507393549346e-01 8.2037670190040e-02 6.6311644639306e-01
9.7406672036321e-01 8.4737208319219e-02 6.5176187538426e-01
6.4314970995350e-01 8.7438984732051e-02 6.4040527628854e-01
3.1228852994922e-01 9.0143176960635e-02 6.2904661096004e-01
9.8144884899675e-01 9.2849857189255e-02 6.1768594945525e-01
6.5059632486156e-01 9.5558987605417e-02 6.0632345652240e-01
3.1969644128855e-01 9.8270425800489e-02 5.9495936878600e-01
9.8871445365777e-01 1.0098393934958e-01 5.8359396628223e-01
6.5761562615756e-01 1.0369922762187e-01 5.7222754255203e-01
3.2636584853647e-01 1.0641594804064e-01 5.6086037733336e-01
9.9493259756036e-01 1.0913374353659e-01 5.4949271508468e-01
6.6328609769098e-01 1.1185226794652e-01 5.3812475128040e-01
3.3140047275047e-01 1.1457120666313e-01 5.2675662691308e-01
9.9925468210973e-01 1.1729029085111e-01 5.1538843024981e-01
6.6619260129085e-01 8.8809097192997e-01 5.0357453753380e-01
3.3313115952486e-01 8.9080951937498e-01 4.9220606078156e-01
3.0938227932163e-04 8.9352820930739e-01 4.8083761688374e-01
6.6771811290292e-01 8.9624690023184e-01 4.6946921836941e-01
3.3534408566272e-01 8.9896541041062e-01 4.5810090575684e-01
3.1700798583186e-03 9.0168351925730e-01 4.4673275434468e-01
6.7117534532910e-01 9.0440097285455e-01 4.3536487885321e-01
3.3933625791616e-01 9.0711749392649e-01 4.2399743453377e-01
7.6271143554610e-03 9.0983279596380e-01 4.1263061360359e-01
6.7602094308405e-01 9.1254660051873e-01 4.0126463640255e-01
3.4449020365454e-01 9.1525865610896e-01 3.8989973743156e-01
1.3007277642734e-02 9.1796875681899e-01 3.7853614727582e-01
6.8154471166225e-01 9.2067675861972e-01 3.6717407217723e-01
3.5007524251643e-01 9.2338259162378e-01 3.5581367355998e-01
1.8571695763710e-02 9.2608626689466e-01 3.4445505004152e-01
6.8700688459884e-01 9.2878787697355e-01 3.3309822432925e-01
3.5535363594727e-01 9.3148758993751e-01 3.2174313689763e-01
2.3585035032074e-02 9.3418563751916e-01 3.1038964749079e-01
6.9167491841694e-01 9.3688229853445e-01 2.9903754439840e-01
3.5959857610206e-01 9.3957787946662e-01 2.8768656029217e-01
2.7333565511209e-02 9.4227269440060e-01 2.7633639243251e-01
6.9486051001110e-01 9.4496704648569e-01 2.6498672449679e-01
3.6216375838326e-01 9.4766121270930e-01 2.5363724727441e-01
2.9231814659352e-02 9.5035543308905e-01 2.4228767598962e-01
6.9605749681239e-01 9.5304990461149e-01 2.3093776287141e-01
3.6263783239329e-01 9.5574477955027e-01 2.1958730453726e-01
2.8973737091570e-02 9.5844016731593e-01 2.0823614456684e-01
6.9506954265564e-01 9.6113613876990e-01 1.9688417217708e-01
3.6093244423794e-01 9.6383273194350e-01 1.8553131814313e-01
2.6571928642018e-02 9.6652995826513e-01 1.7417754909462e-01
6.9199922996019e-01 9.6922780863189e-01 1.6282286114316e-01
3.5722684269130e-01 9.7192625890046e-01 1.5146727355368e-01
2.2268107883240e-02 9.7462527457563e-01 1.4011082292659e-01
6.8713687668364e-01 9.7732481462703e-01 1.2875355814851e-01
3.5184724815793e-01 9.8002483446488e-01 1.1739553621102e-01
1.6413373868939e-02 9.8272528816291e-01 1.0603681889071e-01
6.8084931974215e-01 9.8542613004136e-01 9.4677470219951e-02
3.4516897539006e-01 9.8812731572756e-01 8.3317554647025e-02
9.3860016950773e-03 9.9082880280286e-01 7.1957135774244e-02
6.7351380857243e-01 9.9353055113064e-01 6.0596275567155e-02
3.3756555613666e-01 9.9623252294342e-01 4.9235033938808e-02
1.5541737806896e-03 9.9893468275160e-01 3.7873468627401e-02
6.6558554968538e-01 1.6365762544415e-03 2.6512119131329e-02
3.2946810748646e-01 4.3391313324515e-03 1.5149938401352e-02
9.9332518372752e-01 7.0417808424309e-03 3.7875939082830e-03
6.5713018884950e-01 9.7445570969364e-03 9.9242505944179e-01
3.2099194715813e-01 1.2447184233045e-02 9.8106268109317e-01
9.8486536773410e-01 1.5149816834729e-02 9.6970029488123e-01
6.4876310970266e-01 1.7852425270972e-02 9.5833795345689e-01
3.1269804574393e-01 2.0554978774257e-02 9.4697571117648e-01
9.7668333199067e-01 2.3257444879010e-02 9.3561362480488e-01
6.4073247787229e-01 2.5959788853034e-02 9.2425175420476e-01
3.0485941446607e-01 2.8661973133225e-02 9.1289016298530e-01
9.6907855931075e-01 3.1363956783449e-02 9.0152891907320e-01
6.3340487476416e-01 3.4065695002172e-02 8.9016809515472e-01
2.9785391578297e-01 3.6767138721133e-02 8.7880776891859e-01
9.6244186139618e-01 3.9468234355028e-02 8.6744802300485e-01
6.2718552206929e-01 4.2168923786636e-02 8.5608894453478e-01
2.9210231257284e-01 4.4869144702927e-02 8.4473062406142e-01
9.5721017690727e-01 4.7568831434989e-02 8.3337315374230e-01
6.2252744846057e-01 5.0267916496555e-02 8.2201662450015e-01
2.8807262528964e-01 5.2966333057317e-02 8.1066112191338e-01
9.5386403799574e-01 5.5664018618502e-02 7.9930672058244e-01
6.1991938740481e-01 5.8360920162435e-02 7.8795347677414e-01
2.8625513307941e-01 6.1057000999414e-02 7.7660141928604e-01
9.5288572403931e-01 6.3752249400168e-02 7.6525053873294e-01
6.1982268249364e-01 6.6446688842956e-02 7.5390077586568e-01
2.8707358143053e-01 6.9140389293549e-02 7.4255201009015e-01
9.5464099628343e-01 7.1833478380575e-02 7.3120405000116e-01
6.2252155324798e-01 7.4526150700257e-02 7.1985662833345e-01
2.9070522927129e-01 7.7218672946720e-02 7.0850940400959e-01
9.5917506316035e-01 7.9911382364453e-02 6.9716197363088e-01
6.2790739639442e-01 8.2604676421697e-02 6.8581389357906e-01
2.9687267018294e-01 8.5298992755533e-02 6.7446471188885e-01
9.6603667789506e-01 8.7994780230125e-02 6.6311400660817e-01
6.3536204854925e-01 9.0692463955917e-02 6.5176142521840e-01
3.0480966781598e-01 9.3392408729134e-02 6.4040671864392e-01
9.7433976511851e-01 9.6094886039190e-02 6.2904976392076e-01
6.4391251140431e-01 9.8800049365326e-02 6.1769057162213e-01
3.1348814483237e-01 1.0150792114707e-01 6.0632927703413e-01
9.8302680927613e-01 1.0421839299700e-01 5.9496611702060e-01
6.5248839261036e-01 1.0693123883692e-01 5.8360139683728e-01
3.2183265175494e-01 1.0964613893177e-01 5.7223545248337e-01
9.9101980938864e-01 1.1236271140680e-01 5.6086861440472e-01
6.6001163960046e-01 1.1508054691355e-01 5.4950117754611e-01
3.2877288708074e-01 1.1779924185805e-01 5.3813338108981e-01
9.9727274847455e-01 1.2051842616971e-01 5.2676539908195e-01
6.6548612239316e-01 1.2323778290662e-01 5.1539734104784e-01
3.3293372826655e-01 8.9403847542710e-01 5.0358444876672e-01
9.9963614032592e-01 8.9675686303942e-01 4.9221584010919e-01
6.6655577725935e-01 8.9947537212048e-01 4.8084726910025e-01
3.3368474838366e-01 9.0219387862606e-01 4.6947874619772e-01
1.0117060687486e-03 9.0491222466522e-01 4.5811030500089e-01
6.6852194989054e-01 9.0763021933322e-01 4.4674200789003e-01
3.3619774353131e-01 9.1034764270459e-01 4.3537395011314e-01
4.0188186278961e-03 9.1306425328950e-01 4.2400626127813e-01
6.7196300362116e-01 9.1577979881633e-01 4.1263910334607e-01
3.4000688873522e-01 9.1849402971216e-01 4.0127266456898e-01
8.1264299187986e-03 9.2120671420980e-01 3.8990714933451e-01
6.7629740968422e-01 9.2391765370766e-01 3.7854276448213e-01
3.4449570907103e-01 9.2662669689524e-01 3.6717970323277e-01
1.2697393565139e-02 9.2933375123580e-01 3.5581812832770e-01
6.8087866396754e-01 9.3203879063284e-01 3.4445815623066e-01
3.4901575822908e-01 9.3474185845732e-01 3.3309984426908e-01
1.7084903032805e-02 9.3744306554446e-01 3.2174318235627e-01
6.8506240112871e-01 9.4014258326173e-01 3.1038809045111e-01
3.5292490539185e-01 9.4284063227080e-01 2.9903442221213e-01
2.0649882400586e-02 9.4553746810042e-01 2.8768197447719e-01
6.8821621971715e-01 9.4823336502567e-01 2.7633050139069e-01
3.5560489475173e-01 9.5092859992277e-01 2.6497973137577e-01
2.2799607979550e-02 9.5362343768002e-01 2.5362938485079e-01
6.8978729156963e-01 9.5631811940484e-01 2.4227919068015e-01
3.5655843140403e-01 9.5901285415555e-01 2.3092889978130e-01
2.3107176610659e-02 9.6170781436662e-01 2.1957829495031e-01
6.8943124585174e-01 9.6440313465029e-01 2.0822719664607e-01
3.5553166601598e-01 9.6709891332530e-01 1.9687546504296e-01
2.1412393214177e-02 9.6979521587232e-01 1.8552299903896e-01
6.8707986862787e-01 9.7249207952105e-01 1.7416973307460e-01
3.5254255587932e-01 9.7518951828743e-01 1.6281563261366e-01
1.7810496046620e-02 9.7788752794440e-01 1.5146068901859e-01
6.8289490463237e-01 9.8058609058298e-01 1.4010491438216e-01
3.4780782401870e-01 9.8328517857174e-01 1.2874833669809e-01
1.2561836388965e-02 9.8598475784003e-01 1.1739099559624e-01
6.7716983607212e-01 9.8868479049120e-01 1.0603293874529e-01
3.4164485653856e-01 9.9138523680072e-01 9.4674218939168e-02
5.9999453686196e-03 9.9408605667914e-01 8.3314891828737e-02
6.7024807990213e-01 9.9678721068720e-01 7.1955014230334e-02
3.3440211649549e-01 9.9948866068732e-01 6.0594642930500e-02
9.9863835498707e-01 2.1896211586382e-03 4.9234694356081e-02
6.6262524092189e-01 4.8916678849325e-03 3.7873375300076e-02
3.2655560895984e-01 7.5939084877283e-03 2.6511724424954e-02
9.9044179220269e-01 1.0296311670496e-02 1.5149793676695e-02
6.5429606159504e-01 1.2998847478740e-02 3.7876338587998e-03
3.1805352455391e-01 1.5701655484623e-02 9.9242509057167e-01
9.8189892322621e-01 1.8404248290847e-02 9.8106275988718e-01
6.4574913277484e-01 2.1106886688671e-02 9.6970035117893e-01
3.0961672201437e-01 2.3809542114986e-02 9.5833791530734e-01
9.7351449807389e-01 2.6512185116131e-02 9.4697550472003e-01
6.3745557996243e-01 2.9214784776019e-02 9.3561317419200e-01
3.0145347494283e-01 3.1917308112127e-02 9.2425098159682e-01
9.6552215677420e-01 3.4619719444877e-02 9.1288898869009e-01
6.2967614446356e-01 3.7321979749986e-02 9.0152726187909e-01
2.9393057949955e-01 4.0024046010356e-02 8.9016587294088e-01
9.5830129855387e-01 4.2725870594660e-02 8.7880489963518e-01
6.2280489723459e-01 4.5427400705375e-02 8.6744442613481e-01
2.8745877854825e-01 4.8128577960934e-02 8.5608454316695e-01
9.5228117715593e-01 5.0829338206575e-02 8.4472534771892e-01
6.1729114718765e-01 5.3529611687877e-02 8.3336694211529e-01
2.8250849727279e-01 5.6229323770502e-02 8.2200943221817e-01
9.4795365166837e-01 5.8928396448352e-02 8.1065292444661e-01
6.1364741133559e-01 6.1626750945197e-02 7.9929752126410e-01
2.7961058445075e-01 6.4324311770421e-02 7.8794331476643e-01
9.4586345382696e-01 6.7021012616322e-02 7.7659037804772e-01
6.1242505175958e-01 6.9716804446650e-02 7.6523875417759e-01
2.7931222460620e-01 7.2411665973228e-02 7.5388844294459e-01
9.4653849418116e-01 7.5105616388721e-02 7.4253938606248e-01
6.1411276392633e-01 7.7798729667675e-02 7.3119145231434e-01
2.8203797396944e-01 8.0491148959873e-02 7.1984442505215e-01
9.5030987184490e-01 8.3183098676917e-02 7.0849799536173e-01
6.1891611518088e-01 8.5874891061617e-02 6.9715176465549e-01
2.8783593016818e-01 8.8566923723400e-02 6.8580525998423e-01
9.5704048723464e-01 9.1259665247053e-02 6.7445796358487e-01
6.2649401464579e-01 9.3953627763507e-02 6.6310935514810e-01
2.9615548056258e-01 9.6649328089044e-02 6.5175896168634e-01
9.6598050167083e-01 9.9347241956332e-02 6.4040640693477e-01
6.3592306184109e-01 1.0204775798362e-01 6.2905145117118e-01
3.0593669682012e-01 1.0475113860033e-01 6.1769401378257e-01
9.7597500650116e-01 1.0745749403721e-01 6.0633417442455e-01
6.4599161836292e-01 1.1016677317949e-01 5.9497215305080e-01
3.1593994039428e-01 1.1287877229195e-01 5.8360827314521e-01
9.8577312416860e-01 1.1559315992259e-01 5.7224291526974e-01
6.5544457639281e-01 1.1830951398526e-01 5.6087646915611e-01
3.2490914253777e-01 1.2102736533453e-01 5.4950929199604e-01
9.9412482412086e-01 1.2374624137610e-01 5.3814167854891e-01
6.6305468567275e-01 1.2646570369735e-01 5.2677384568751e-01
3.3166853488375e-01 1.2918537537756e-01 5.1540593081328e-01
9.9966792629729e-01 8.9998584315719e-01 5.0359461418339e-01
6.6615636462841e-01 9.0270409342980e-01 4.9222588110543e-01
3.3284178340226e-01 9.0542244660141e-01 4.8085718924153e-01
9.9971733993314e-01 9.0814079308516e-01 4.6948854752435e-01
6.6677326811755e-01 9.1085899468061e-01 4.5811998414912e-01
3.3399694762845e-01 9.1357688503041e-01 4.4675155128261e-01
1.3731410592036e-03 9.1629427252160e-01 4.3538332863794e-01
6.6888438313577e-01 9.1901094589864e-01 4.2401542512384e-01
3.3651147939649e-01 9.2172668254538e-01 4.1264797785443e-01
4.2340502345817e-03 9.2444125903691e-01 4.0128114803099e-01
6.7203104658667e-01 9.2715446322507e-01 3.8991511356137e-01
3.3988116996964e-01 9.2986610686936e-01 3.7855005871289e-01
7.7631518336928e-03 9.3257603769904e-01 3.6718616152843e-01
6.7565588054569e-01 9.3528414980439e-01 3.5582358010355e-01
3.4353840073870e-01 9.3799039138645e-01 3.4446243906920e-01
1.1389840361606e-02 9.4069476911735e-01 3.3310281771542e-01
6.7918933812981e-01 9.4339734865198e-01 3.2174474110839e-01
3.4691604407216e-01 9.4609825116884e-01 3.1038817529050e-01
1.4549248489419e-02 9.4879764618894e-01 2.9903302722355e-01
6.8206866403031e-01 9.5149574129902e-01 2.8767914957768e-01
3.4945484915712e-01 9.5419276974088e-01 2.7632634985760e-01
1.6689727916375e-02 9.5688897706110e-01 2.6497440280064e-01
6.8375713865329e-01 9.5958460808658e-01 2.5362306458871e-01
3.5064333788225e-01 9.6227989537563e-01 2.4227208727243e-01
1.7337395128624e-02 9.6497505001160e-01 2.3092123193405e-01
6.8383143605779e-01 9.6767025521931e-01 2.1957027946747e-01
3.5012071792298e-01 9.7036566287942e-01 2.0821903833314e-01
1.6203546499958e-02 9.7306139267203e-01 1.9686734913509e-01
6.8208106233103e-01 9.7575753334821e-01 1.8551508627420e-01
3.4775693417725e-01 9.7845414552259e-01 1.7416215720090e-01
1.3236999538327e-02 9.8115126538351e-01 1.6280849991385e-01
6.7852888802251e-01 9.8384890879751e-01 1.5145407935266e-01
3.4364165584479e-01 9.8654707540356e-01 1.4009888325118e-01
8.5854511355944e-03 9.8924575241892e-01 1.2874291789634e-01
6.7337122170422e-01 9.9194491799237e-01 1.1738620410596e-01
3.3801047032665e-01 9.9464454403108e-01 1.0602877362102e-01
2.5150578190596e-03 9.9734459849189e-01 9.4670666011830e-02
6.6714339410482e-01 4.3880986938059e-05 8.3313263422203e-02
3.3139904212341e-01 2.7447937361541e-03 7.1953820766373e-02
9.9555630214919e-01 5.4460327297075e-03 6.0593841613291e-02
6.5962732325845e-01 8.1475638467118e-03 4.9233376391492e-02
3.2362420197560e-01 1.0849354121374e-02 3.7872475379272e-02
9.8755897966257e-01 1.3551371916213e-02 2.6511188259695e-02
6.5144365635732e-01 1.6253586931320e-02 1.5149563929278e-02
3.1529021806391e-01 1.8955970084016e-02 3.7876505157147e-03
9.7899585596140e-01 2.1658767286703e-02 9.9242516613973e-01
6.4282021931840e-01 2.4361282555073e-02 9.8106295284223e-01
3.0664253941659e-01 2.7063881863067e-02 9.6970059416015e-01
9.7047521634684e-01 2.9766537992034e-02 9.5833813836381e-01
6.3433090951386e-01 3.2469223097369e-02 9.4697563513684e-01
2.9822261243978e-01 3.5171908159295e-02 9.3561313630352e-01
9.6216373294811e-01 3.7874562380984e-02 9.2425069662616e-01
6.2616817822139e-01 4.0577152535114e-02 9.1288837466088e-01
2.9025044400102e-01 4.3279642261522e-02 9.0152623365634e-01
9.5442570676288e-01 4.5981991322575e-02 8.9016434247190e-01
6.1870991698830e-01 4.8684154829843e-02 8.7880277647919e-01
2.8311989055827e-01 5.1386082466971e-02 8.6744161839264e-01
9.4767339369807e-01 5.4087717750882e-02 8.5608095894738e-01
6.1238921462546e-01 5.6788997398746e-02 8.4472089730660e-01
2.7728721191311e-01 5.9489850904001e-02 8.3336154103098e-01
9.4238832536454e-01 6.2190200473939e-02 8.2200300538047e-01
6.0771452977095e-01 6.4889961546109e-02 8.1064541164227e-01
2.7328870526959e-01 6.7589044181524e-02 7.9928888409459e-01
9.3913439052616e-01 7.0287355725398e-02 7.8793354513436e-01
6.0527537764645e-01 7.2984805218333e-02 7.7657950804437e-01
2.7173510272592e-01 7.5681310105548e-02 7.6522686689551e-01
9.3853578699703e-01 7.8376805780195e-02 7.5387568324008e-01
6.0569729620652e-01 8.1071258333944e-02 7.4252596964350e-01
2.7323571705471e-01 8.3764680476408e-02 7.3117767081859e-01
9.4116170531437e-01 8.6457149830397e-02 7.1983064421928e-01
6.0947874115736e-01 8.9148827681118e-02 7.0848464334094e-01
2.7818152135407e-01 9.1839974880466e-02 6.9713930834936e-01
9.4725479500598e-01 9.4530960360846e-02 6.8579416940915e-01
6.1667296230422e-01 9.7222257233685e-02 6.7444866740553e-01
2.8640065841169e-01 9.9914422438701e-02 6.6310219402470e-01
9.5639432513879e-01 1.0260805872269e-01 6.5175414848625e-01
6.2660448527898e-01 1.0530376191986e-01 6.4040400283596e-01
2.9697819529995e-01 1.0800206077205e-01 6.2905136369342e-01
9.6746108913165e-01 1.1070335924028e-01 6.1769601763264e-01
6.3799859856123e-01 1.1340789137512e-01 6.0633795055164e-01
3.0853628816806e-01 1.1611569643178e-01 5.9497733735296e-01
9.7901962516031e-01 1.1882661803597e-01 5.8361450490468e-01
6.4939374905946e-01 1.2154032697477e-01 5.7224987662747e-01
3.1960380618283e-01 1.2425636335482e-01 5.6088391003402e-01
9.8959616929681e-01 1.2697419085624e-01 5.4951703880596e-01
6.5932048039265e-01 1.2969325405292e-01 5.3814962873130e-01
3.2873210211961e-01 1.3241302979148e-01 5.2678195269474e-01
9.9779439064758e-01 1.3513306564810e-01 5.1541418506841e-01
6.6639587768914e-01 9.0593308379623e-01 5.0360501838059e-01
3.3269035825311e-01 9.0865121612841e-01 4.9223616838805e-01
9.9916342707116e-01 9.1136943495217e-01 4.8086736208786e-01
6.6580911578367e-01 9.1408764273398e-01 4.6949860729744e-01
3.3261897269754e-01 9.1680571764297e-01 4.5812992796503e-01
9.9958210733725e-01 9.1952351375438e-01 4.4676136812426e-01
6.6668536848604e-01 9.2224086316146e-01 4.3539299501129e-01
3.3391364614886e-01 9.2495758022537e-01 4.2402490073898e-01
1.2502680208193e-03 9.2767346797283e-01 4.1265720196426e-01
6.6867744407258e-01 9.3038832639135e-01 4.0129003713142e-01
3.3617670349238e-01 9.3310196211535e-01 3.8992356111730e-01
3.7292698062731e-03 9.3581419879163e-01 3.7855793741169e-01
6.7131633323186e-01 9.3852488729044e-01 3.6719332829081e-01
3.3891920180409e-01 9.4123391490521e-01 3.5582988373396e-01
6.5193396323563e-03 9.4394121275063e-01 3.4446773005176e-01
6.7409832573155e-01 9.4664676070853e-01 3.3310695930989e-01
3.4163778455560e-01 9.4935058946325e-01 3.2174762062905e-01
9.1193457199418e-03 9.5205277940002e-01 3.1038971431236e-01
6.7652468402896e-01 9.5475345640115e-01 2.9903318949739e-01
3.4383567322566e-01 9.5745278485312e-01 2.8767794566684e-01
1.1034661785197e-02 9.6015095844792e-01 2.7632383791769e-01
6.7810485224140e-01 9.6284818958762e-01 2.6497068544249e-01
3.4503074303992e-01 9.6554469833684e-01 2.5361828229731e-01
1.1798578667931e-02 9.6824070188177e-01 2.4226640928954e-01
6.7839675221846e-01 9.7093640533447e-01 2.3091484576486e-01
3.4481611391175e-01 9.7363199449141e-01 2.1956338020704e-01
1.1050156062851e-02 9.7632763086293e-01 2.0821181884743e-01
6.7709506483034e-01 9.7902344899447e-01 1.9685999184041e-01
3.4294964726051e-01 9.8171955585438e-01 1.8550775691905e-01
8.6151554888965e-03 9.8441603189833e-01 1.7415500073840e-01
6.7409503718366e-01 9.8711293334507e-01 1.6280163830928e-01
3.3939464278437e-01 9.8981029520099e-01 1.5144761101630e-01
4.5209172316541e-03 9.9250813462788e-01 1.4009288371832e-01
6.6948209848576e-01 9.9520645433477e-01 1.2873744137313e-01
3.3428743882105e-01 9.9790524576805e-01 1.1738128554046e-01
9.9925564298950e-01 6.0303515525394e-04 1.0602610171500e-01
6.6376491443022e-01 3.3028124538786e-03 9.4668461229307e-02
3.2814978247004e-01 6.0029966115532e-03 8.3310177377654e-02
9.9242147578554e-01 8.7035583358787e-03 7.1951287963899e-02
6.5659138198417e-01 1.1404467182147e-02 6.0591834287795e-02
3.2067097899967e-01 1.4105692446816e-02 4.9231859751773e-02
9.8467179249735e-01 1.6807203782749e-02 3.7871408832003e-02
6.4860537541811e-01 1.9508971583042e-02 2.6510526368855e-02
3.1248330619844e-01 2.2210967175032e-02 1.5149257132321e-02
9.7631720271337e-01 2.4913162859614e-02 3.7876456216523e-03
6.3996672718972e-01 2.7615909991088e-02 9.9242528322524e-01
3.0376542651960e-01 3.0318306950023e-02 9.8106325399655e-01
9.6755527949605e-01 3.3020823852336e-02 9.6970101651765e-01
6.3134844135122e-01 3.5723435059631e-02 9.5833861577083e-01
2.9515734482685e-01 3.8426114562499e-02 9.4697609795425e-01
9.5899477393930e-01 4.1128835483232e-02 9.3561351115056e-01
6.2287394537429e-01 4.3831569510043e-02 9.2425090609688e-01
2.8680859735254e-01 4.6534286260983e-02 9.1288833705101e-01
9.5081308576595e-01 4.9236952575200e-02 9.0152586274723e-01
6.1490248718010e-01 5.1939531730164e-02 8.9016354743101e-01
2.7909270787471e-01 5.4641982586848e-02 8.7880146195446e-01
9.4340059735963e-01 5.7344258671474e-02 8.6743968490078e-01
6.0784406363403e-01 6.0046307213920e-02 8.5607830368568e-01
2.7244218567364e-01 6.2748068181370e-02 8.4471741555329e-01
9.3721531600332e-01 6.5449473374143e-02 8.3335712834035e-01
6.0218516244040e-01 6.8150445692768e-02 8.2199756082168e-01
2.6737483283952e-01 7.0850898745634e-02 8.1063884236893e-01
9.3280881959179e-01 7.3550737049729e-02 7.9928111155085e-01
5.9851289153152e-01 7.6249857186136e-02 7.8792451317974e-01
2.6451384997305e-01 7.8948150405684e-02 7.7656919317712e-01
9.3083909388091e-01 8.1645507326264e-02 7.6521529052105e-01
5.9751592926406e-01 8.4341825489629e-02 7.5386292550608e-01
2.6457055483568e-01 8.7037020586940e-02 7.4251218369045e-01
9.3202666806636e-01 8.9731042008850e-02 7.3116309536626e-01
5.9990367422459e-01 9.2423892871585e-02 7.1981561133000e-01
2.6821455715780e-01 9.5115653646230e-02 7.0846957727360e-01
9.3696358837791e-01 9.7806506884403e-02 6.9712471119060e-01
6.0614419476463e-01 1.0049675844587e-01 6.8578059034621e-01
2.7573742730738e-01 1.0318684867879e-01 6.7443665561728e-01
9.4571149546528e-01 1.0587734623761e-01 6.6309224002011e-01
6.1602267593154e-01 1.0856891883731e-01 6.5174662394730e-01
2.8661755493208e-01 1.1126227978562e-01 6.4039911228555e-01
9.5743612184384e-01 1.1395811571647e-01 6.2904912043624e-01
6.2841490610747e-01 1.1665700720488e-01 6.1769625091631e-01
2.9948934143845e-01 1.1935935720691e-01 6.0634034261786e-01
9.7059490582344e-01 1.2206534120743e-01 5.9498148123834e-01
6.4166717063515e-01 1.2477488828016e-01 5.8361996930341e-01
3.1264141656732e-01 1.2748769588310e-01 5.7225626391497e-01
9.8345268250302e-01 1.3020327486120e-01 5.6089089704326e-01
6.5403690502490e-01 1.3292101579729e-01 5.4952439558170e-01
3.2433328191407e-01 1.3564026424533e-01 5.3815721644120e-01
9.9428740814438e-01 1.3836039136709e-01 5.2678970647728e-01
6.6385436840131e-01 1.4108084870990e-01 5.1542208971076e-01
3.3311815934779e-01 9.1188020586118e-01 5.0361564599010e-01
9.9923687896782e-01 9.1459823701943e-01 4.9224668658728e-01
6.6551736393112e-01 9.1731634017380e-01 4.8087777238805e-01
3.3195438050739e-01 9.2003442789629e-01 4.6950891040330e-01
9.9854057752261e-01 9.2275239204892e-01 4.5814012122496e-01
6.6526651341356e-01 9.2547010381688e-01 4.4677144236677e-01
3.3212079007617e-01 9.2818741524396e-01 4.3540293098007e-01
9.9909028796467e-01 9.3090416246543e-01 4.2403466547822e-01
6.6616048206913e-01 9.3362017067556e-01 4.1266674562303e-01
3.3331580508732e-01 9.3633526067610e-01 4.0129929072298e-01
5.4001565230041e-04 9.3904925665736e-01 3.8993243576032e-01
6.6781652850261e-01 9.4176199469869e-01 3.7856632548420e-01
3.3512867098412e-01 9.4447333136474e-01 3.6720110674953e-01
2.4598451786453e-03 9.4718315173375e-01 3.5583691961004e-01
6.6979359432048e-01 9.4989137622110e-01 3.4447388785895e-01
3.3711359192929e-01 9.5259796564638e-01 3.3311210982851e-01
4.4035880917046e-03 9.5530292411989e-01 3.2175165029623e-01
6.7164735623633e-01 9.5800629948423e-01 3.1039253429295e-01
3.3882868374031e-01 9.6070818123049e-01 2.9903474346510e-01
5.9314407806240e-03 9.6340869601035e-01 2.8767821541455e-01
6.7293974582594e-01 9.6610800107259e-01 2.7632284614205e-01
3.3983822638521e-01 9.6880627614489e-01 2.6496849538715e-01
6.6123542501895e-03 9.7150371442988e-01 2.5361499433587e-01
6.7324881955097e-01 9.7420051345913e-01 2.4226215491142e-01
3.3973590042986e-01 9.7689686653099e-01 2.3090977972099e-01
6.0637861034426e-03 9.7959295534646e-01 2.1955767172615e-01
6.7222481952824e-01 9.8228894427266e-01 2.0820564283194e-01
3.3821363919212e-01 9.8498497644193e-01 1.9685352081507e-01
4.0272143795939e-03 9.8768117167704e-01 1.8550115428257e-01
6.6966478154027e-01 9.9037762605504e-01 1.7414841561640e-01
3.3512769905539e-01 9.9307441280380e-01 1.6279520207372e-01
4.1924294052158e-04 9.9577158417134e-01 1.5144143535552e-01
6.6554436713716e-01 9.9846917390972e-01 1.4008706002473e-01
3.3087686285274e-01 1.1654688231951e-03 1.2873402659735e-01
9.9567535532211e-01 3.8640301933366e-03 1.1737824115155e-01
6.6032971978375e-01 6.5630300413350e-03 1.0602179050687e-01
3.2484918709388e-01 9.2624555336747e-03 9.4664684392648e-02
9.8924354602443e-01 1.1962288166431e-02 8.3306941977344e-02
6.5352299755127e-01 1.4662505525189e-02 7.1948589577070e-02
3.1769803995012e-01 1.7363082706493e-02 6.0589658709894e-02
9.8177938266936e-01 2.0063993425321e-02 4.9230184503115e-02
6.4577788622275e-01 2.2765210842568e-02 3.7870204439275e-02
3.0970452508922e-01 2.5466708149816e-02 2.6509757414281e-02
9.7357037067323e-01 2.8168458947785e-02 1.5148883075461e-02
6.3738659164515e-01 3.0870437451543e-02 3.7876214063649e-03
3.0097571006951e-01 3.3573099573684e-02 9.9242543787135e-01
9.6474416283001e-01 3.6275341015756e-02 9.8106365465127e-01
6.2849706934442e-01 3.8977734983559e-02 9.6970160633921e-01
2.9224626628119e-01 4.1680257622506e-02 9.5833933404079e-01
9.5600388402591e-01 4.4382884926605e-02 9.4697687988814e-01
6.1978241736327e-01 4.7085592317361e-02 9.3561428761001e-01
2.8359480570404e-01 4.9788354142721e-02 9.2425160322712e-01
9.4745452295829e-01 5.2491143092975e-02 9.1288887587163e-01
6.1137567727163e-01 5.5193929528147e-02 9.0152615873112e-01
2.7537312084563e-01 5.7896680710060e-02 8.9016351011723e-01
9.3946256991193e-01 6.0599359932285e-02 8.7880099465553e-01
6.0366073455251e-01 6.3301925543451e-02 8.6743868458646e-01
2.6798545734816e-01 6.6004329865068e-02 8.5607666115374e-01
9.3245585863712e-01 6.8706518015873e-02 8.4471501603525e-01
5.9709248424751e-01 7.1408426673595e-02 8.3335385273798e-01
2.6191744861082e-01 7.4109982835720e-02 8.2199328782793e-01
9.2695456174154e-01 7.6811102688837e-02 8.1063345179290e-01
5.9222942213539e-01 7.9511690768278e-02 7.9927448923309e-01
2.5776944857819e-01 8.2211639694416e-02 7.8791655793662e-01
9.2360381161020e-01 8.4910830916777e-02 7.7655982622118e-01
5.8976320977096e-01 8.7609137085924e-02 7.6520446771950e-01
2.5627941758331e-01 9.0306426897069e-02 7.5385065258484e-01
9.2318451445048e-01 9.3002573473231e-02 7.4249853396717e-01
5.9050969287834e-01 9.5697467495432e-02 7.3114822869542e-01
2.5828355279841e-01 9.8391036183889e-02 7.1979979161348e-01
9.2652983440792e-01 1.0108326864011e-01 7.0845318424000e-01
5.9526464558684e-01 1.0377424667393e-01 6.9710824060677e-01
2.6449341323353e-01 1.0646417785093e-01 6.8576463626360e-01
9.3420801124723e-01 1.0915342427421e-01 6.7442186987453e-01
6.0438471582885e-01 1.1184251748110e-01 6.6307926897822e-01
2.7498367758816e-01 1.1453214858358e-01 6.5173603001681e-01
9.4595034392268e-01 1.1722312548325e-01 6.4039129579286e-01
6.1721869484374e-01 1.1991629651356e-01 6.2904426168594e-01
2.8871547040427e-01 1.2261245037317e-01 6.1769428948665e-01
9.6036414151130e-01 1.2531221136468e-01 6.0634100092201e-01
6.3208752279324e-01 1.2801595237969e-01 5.9498432621202e-01
3.0380864574422e-01 1.3072374436397e-01 5.8362449540641e-01
9.7545044213111e-01 1.3343535226012e-01 5.7226197635391e-01
6.4693542730298e-01 1.3615027709898e-01 5.6089737677951e-01
3.1818657304574e-01 1.3886783450548e-01 5.4953133525997e-01
9.8912991997727e-01 1.4158725282598e-01 5.3816442578719e-01
6.5969854771994e-01 1.4430777078793e-01 5.2679709386411e-01
3.2983680154163e-01 1.4702871646250e-01 5.1542963110520e-01
9.9983526100047e-01 9.1782721771711e-01 5.0362648171201e-01
6.6579487562022e-01 9.2054516221779e-01 4.9225742038255e-01
3.3190076658225e-01 9.2326316591102e-01 4.8088840490407e-01
9.9814832141037e-01 9.2598114988308e-01 4.6951944171323e-01
6.6453111674951e-01 9.2869901751420e-01 4.5815054873564e-01
3.3104093290391e-01 9.3141665439723e-01 4.4678175822515e-01
9.9766785434014e-01 9.3413392944836e-01 4.3541311913215e-01
6.6440045317536e-01 9.3685069737448e-01 4.2404469863847e-01
3.3122604152290e-01 9.3956680254723e-01 4.1267658250467e-01
9.9813096815298e-01 9.4228208419274e-01 4.0130887393819e-01
6.6510092761328e-01 9.4499638265816e-01 3.8994169080657e-01
3.3212124774993e-01 9.4770954638388e-01 3.7857516117877e-01
9.9917712549342e-01 9.5042143911555e-01 3.6720941735747e-01
6.6625379047343e-01 9.5313194684223e-01 3.5584458874389e-01
3.3333658979103e-01 9.5584098395175e-01 3.4448079402897e-01
4.1100244370008e-04 9.5854849814246e-01 3.3311813331443e-01
6.6746260548227e-01 9.6125447371513e-01 3.2175668082026e-01
3.3447702333112e-01 9.6395893297828e-01 3.1039647882667e-01
1.4398950263425e-03 9.6666193563058e-01 2.9903753342364e-01
6.6833689079009e-01 9.6936357612883e-01 2.8767981250163e-01
3.3515379999420e-01 9.7206397920527e-01 2.7632324622001e-01
1.8766989689862e-03 9.7476329385132e-01 2.6496772995104e-01
6.6849219193313e-01 9.7746168622021e-01 2.5361312944491e-01
3.3498770454248e-01 9.8015933199568e-01 2.4225928773163e-01
1.3517998432910e-03 9.8285640881036e-01 2.3090603310552e-01
6.6757448247878e-01 9.8555308926410e-01 2.1955318745774e-01
3.3364745919103e-01 9.8824953499365e-01 2.0820057424342e-01
9.9956433096365e-01 9.9094589209773e-01 1.9684802548335e-01
6.6532070255874e-01 9.9364228805332e-01 1.8549538737887e-01
3.3091420634648e-01 9.9633883009744e-01 1.7414252432420e-01
9.9634444712231e-01 9.9903560491888e-01 1.6278932129687e-01
6.6203540136344e-01 1.7306900506130e-03 1.5143796618310e-01
3.2713197776749e-01 4.4282008862927e-03 1.4008372432297e-01
9.9207437992808e-01 7.1260945170954e-03 1.2872892190804e-01
6.5686851320038e-01 9.8243895350891e-03 1.1737352444056e-01
3.2152135330089e-01 1.2523093024159e-02 1.0601751312659e-01
9.8604075186128e-01 1.5222202831497e-02 9.4660882465100e-02
6.5043526193299e-01 1.7921709643144e-02 8.3303637875178e-02
3.1471398759846e-01 2.0621598805783e-02 7.1945793483484e-02
9.7888645941067e-01 2.3321851870401e-02 6.0587370148363e-02
6.4296253560686e-01 2.6022447857391e-02 4.9228393760588e-02
3.0695232787175e-01 2.8723364258263e-02 3.7868893833733e-02
9.7086614975625e-01 3.1424577798011e-02 2.6508902378792e-02
6.3471448557338e-01 3.4126064985942e-02 1.5148453046408e-02
2.9850797758328e-01 3.6827802483037e-02 3.7875805143724e-03
9.6203239929110e-01 3.9530349881487e-02 9.9242562521522e-01
6.2576607272461e-01 4.2232403161120e-02 9.8106414365856e-01
2.8947765335028e-01 4.4934637557323e-02 9.6970234750696e-01
9.5317858831066e-01 4.7637031178982e-02 9.5834027351360e-01
6.1688063327486e-01 5.0339562146593e-02 9.4697795930471e-01
2.8059591827829e-01 5.3042208265687e-02 9.3561544382061e-01
9.4433702468798e-01 5.5744946612718e-02 9.2425276789750e-01
6.0811707357667e-01 5.8447753030383e-02 9.1288997499162e-01
2.7194982602796e-01 6.1150601525749e-02 9.0152711205092e-01
9.3584979607006e-01 6.3853463561394e-02 8.9016423054171e-01
5.9983237701866e-01 6.6556307227243e-02 8.7880138763885e-01
2.6391398194992e-01 6.9259096279442e-02 8.6743864758618e-01
9.2811219874651e-01 7.1961789033140e-02 8.5607608322844e-01
5.9244595953862e-01 7.4664337099851e-02 8.4471377770396e-01
2.5693572321020e-01 7.7366683969240e-02 8.3335182626651e-01
9.2160366767295e-01 8.0068763452996e-02 8.2199033816915e-01
5.8647388540780e-01 8.2770498040009e-02 8.1062943848724e-01
2.5157257073865e-01 8.5471797264562e-02 7.9926926967174e-01
9.1692817962300e-01 8.8172556272996e-02 7.8790999249779e-01
5.8257153138751e-01 9.0872654902547e-02 7.7655178589371e-01
2.4853580566741e-01 9.3571957773979e-02 7.6519484489285e-01
9.1485636592444e-01 9.6270316160405e-02 7.5383937564617e-01
5.8157031349078e-01 9.8967572728795e-02 7.4248558609434e-01
2.4871564597751e-01 1.0166357062532e-01 7.3113367061069e-01
9.1632986938426e-01 1.0435816868789e-01 7.1978378687286e-01
5.8444791165923e-01 1.0705126459535e-01 7.0843602373010e-01
2.5309923579541e-01 1.0974282711283e-01 6.9709036038257e-01
9.2230419028890e-01 1.1243293673607e-01 6.8574662028390e-01
5.9206989203784e-01 1.1512183051271e-01 6.7440442796958e-01
2.6238629046955e-01 1.1780994175491e-01 6.6306318260289e-01
9.3322339040007e-01 1.2049792020966e-01 6.5172206577850e-01
6.0453068035552e-01 1.2318661615529e-01 6.4038009889467e-01
2.7623937955174e-01 1.2587701655981e-01 6.2903625385069e-01
9.4826716302012e-01 1.2857013423062e-01 6.1768960131757e-01
6.2052395104289e-01 1.3126686796423e-01 6.0633946157363e-01
2.9291683949705e-01 1.3396786491211e-01 5.9498551518581e-01
9.6535276849420e-01 1.3667341869560e-01 5.8362784068608e-01
6.3773888065337e-01 1.3938342754081e-01 5.7226686955276e-01
3.0998191471285e-01 1.4209742091392e-01 5.6090327424273e-01
9.8198856365641e-01 1.4481464666701e-01 5.4953782283344e-01
6.5366814306539e-01 1.4753419715658e-01 5.3817123936866e-01
3.2493752104506e-01 1.5025514469566e-01 5.2680410214716e-01
9.9572690712864e-01 1.5297665676227e-01 5.1543679619005e-01
6.6654759967372e-01 9.2377412758181e-01 5.0363751034293e-01
3.3236345578785e-01 9.2649199801170e-01 4.9226835453402e-01
9.9831123581921e-01 9.2920991632765e-01 4.8089924444939e-01
6.6438685575452e-01 9.3192781082015e-01 4.6953018612532e-01
3.3058467953314e-01 9.3464559460763e-01 4.5816119535793e-01
9.9689752455253e-01 9.3736316547827e-01 4.4679230013137e-01
6.6331673677276e-01 9.4008040668492e-01 4.3542354268735e-01
3.2983233406608e-01 9.4279718883114e-01 4.2405498095256e-01
9.9643320802191e-01 9.4551337290310e-01 4.1268668904182e-01
6.6310736627183e-01 9.4822881439685e-01 4.0131875659556e-01
3.2984219122651e-01 9.5094336837749e-01 3.8995128679107e-01
9.9662468843668e-01 9.5365689520229e-01 3.7858439298111e-01
6.6344169952450e-01 9.5636926655863e-01 3.6721819404796e-01
3.3028006079765e-01 9.5908037142062e-01 3.5585280869869e-01
9.9712669830633e-01 9.6179012151867e-01 3.4448834905209e-01
6.6396866151529e-01 9.6449845594200e-01 3.3312491396329e-01
3.3079310884360e-01 9.6720534454785e-01 3.2176258259103e-01
9.9758726704242e-01 9.6991078992725e-01 3.1040140872626e-01
6.6433839116292e-01 9.7261482776864e-01 2.9904141636782e-01
3.3103375185984e-01 9.7531752556581e-01 2.8768259694916e-01
9.9766067198189e-01 9.7801897973063e-01 2.7632490849290e-01
6.6420662566874e-01 9.8071931128964e-01 2.6496827680463e-01
3.3065940208077e-01 9.8341866045661e-01 2.5361259862886e-01
9.9700732446166e-01 9.8611718046663e-01 2.4225774650119e-01
6.6323950557178e-01 9.8881503111637e-01 2.3090357486763e-01
3.2934611435048e-01 9.9151237246750e-01 2.1954992693145e-01
9.9531862691686e-01 9.9420935913124e-01 2.0819664164624e-01
6.6115003768630e-01 9.9690613546693e-01 1.9684356030767e-01
3.2683501254159e-01 9.9960283191061e-01 1.8549053229401e-01
9.9284396990683e-01 2.2973314108183e-03 1.7413997818281e-01
6.5821483346394e-01 4.9942748987127e-03 1.6278656588720e-01
3.2343366234393e-01 7.6914295135034e-03 1.5143284058414e-01
9.8850200219425e-01 1.0388856586091e-02 1.4007871678288e-01
6.5342288489706e-01 1.3086601522543e-02 1.2872412737975e-01
3.1820066579167e-01 1.5784695514577e-02 1.1736902262912e-01
9.8284085038624e-01 1.8483157472983e-02 1.0601336857779e-01
6.4734992196477e-01 2.1181996020820e-02 9.4657145182007e-02
3.1173517887514e-01 2.3881211422936e-02 8.3300344291323e-02
9.7600458763148e-01 2.6580797366767e-02 7.1942967643657e-02
6.4016665558565e-01 2.9280742542946e-02 6.0585024975153e-02
3.0423032498207e-01 3.1981032001012e-02 4.9226532312963e-02
9.6820488875607e-01 3.4681648275383e-02 3.7867510489599e-02
6.3209992744382e-01 3.7382572290074e-02 2.6507983895316e-02
2.9592526597501e-01 4.0083784058767e-02 1.5147979469252e-02
9.5969094883631e-01 4.2785263200683e-02 3.7875259201184e-03
6.2314639695343e-01 4.5487672100935e-02 9.9242583964623e-01
2.8684081661959e-01 4.8189509970331e-02 9.8106470778774e-01
9.5050679793026e-01 5.0891553006320e-02 9.6970322023400e-01
6.1415533082306e-01 5.3593781444186e-02 9.5834140902251e-01
2.7779772749215e-01 5.6296175621738e-02 9.4697930696890e-01
9.4144568194327e-01 5.8998715759201e-02 9.3561694795832e-01
6.0511134192111e-01 6.1701381647286e-02 9.2425436738542e-01
2.6880739361407e-01 6.4404152241476e-02 9.1289160274583e-01
9.3254715986757e-01 6.7107005156288e-02 9.0152869438661e-01
5.9634471294109e-01 6.9809916049000e-02 8.9016568642769e-01
2.6021500311024e-01 7.2512857878133e-02 8.7880262787025e-01
9.2417400461631e-01 7.5215800017968e-02 8.6743957391109e-01
5.8823888055189e-01 7.7918707207052e-02 8.5607658748247e-01
2.5242816816263e-01 8.0621538306693e-02 8.4471374103451e-01
9.1676198560663e-01 8.3324244846256e-02 8.3335111856804e-01
5.8126226022822e-01 8.6026769337903e-02 8.2198881790758e-01
2.4595297653124e-01 8.8729043358189e-02 8.1062695316939e-01
9.1086043874873e-01 9.1430985423645e-02 7.9926565732067e-01
5.7601353740442e-01 9.4132498741733e-02 7.8790508463007e-01
2.4144400038471e-01 9.6833469011605e-02 7.7654541265901e-01
9.0718659518356e-01 9.9533762601104e-02 7.6518684321644e-01
5.7327922811950e-01 1.0223322566366e-01 7.5382960137385e-01
2.3976285631430e-01 1.0493168511024e-01 7.4247393119563e-01
9.0668108779153e-01 1.0762895283728e-01 7.3112008629822e-01
5.7407929607203e-01 1.1032483521329e-01 7.1976831279657e-01
2.4200302754451e-01 1.1301915043195e-01 7.0841882186557e-01
9.1049545655397e-01 1.1571175664302e-01 6.9707174952043e-01
5.7959369026556e-01 1.1840259317152e-01 6.8572710310017e-01
2.4932391002373e-01 1.2109173469706e-01 6.7438469830522e-01
9.1969572079040e-01 1.2377945302600e-01 6.6304409805239e-01
5.9069665160709e-01 1.2646627296692e-01 6.5170457369304e-01
2.6228831068231e-01 1.2915300013379e-01 6.4036511583879e-01
9.3440582483056e-01 1.3184069488340e-01 6.2902451857443e-01
6.0696143394301e-01 1.3453057494085e-01 6.1768154098292e-01
2.7985147303125e-01 1.3722385167519e-01 6.0633511686986e-01
9.5296428565244e-01 1.3992153319176e-01 5.9498455376100e-01
6.2618613894863e-01 1.4262424584778e-01 5.8362965596905e-01
2.9940356797897e-01 1.4533212361736e-01 5.7227073081797e-01
9.7250300630504e-01 1.4804479426285e-01 5.6090847913740e-01
6.4537041652439e-01 1.5076146339116e-01 5.4954380962568e-01
3.1789363444934e-01 1.5348107168439e-01 5.3817763673832e-01
9.8996828389938e-01 1.5620248253556e-01 5.2681071900156e-01
6.6150568979810e-01 1.5892465210649e-01 5.1544357260184e-01
3.3325553127958e-01 9.2972094352925e-01 5.0364871679912e-01
9.9894185997355e-01 9.3243875081653e-01 4.9227947390884e-01
6.6474672999206e-01 9.3515659600644e-01 4.8091027592115e-01
3.3066650923068e-01 9.3787441350746e-01 4.6954112860051e-01
9.9669623375242e-01 9.4059212470912e-01 4.5817204603217e-01
6.6282960669983e-01 9.4330963777250e-01 4.4680305271234e-01
3.2905905385754e-01 9.4602684818784e-01 4.3543418536231e-01
9.9537583553392e-01 9.4874364021544e-01 4.2406549424315e-01
6.6177020800464e-01 9.5145988926405e-01 4.1269704374547e-01
3.2823162138000e-01 9.5417546518248e-01 4.0132891205491e-01
9.9474893562454e-01 9.5689023635344e-01 3.8996118975069e-01
6.6131063371437e-01 9.5960407439591e-01 3.7859397727705e-01
3.2790501134796e-01 9.6231685921438e-01 3.6722738132793e-01
9.9452032642180e-01 9.6502848408934e-01 3.5586151029144e-01
6.6114489815168e-01 9.6773886048686e-01 3.4449646900092e-01
3.2776715417759e-01 9.7044792227586e-01 3.3313235312119e-01
9.9437563278779e-01 9.7315562907502e-01 3.2176924355590e-01
6.6095895501806e-01 9.7586196850343e-01 3.1040720128646e-01
3.2750578653072e-01 9.7856695717378e-01 2.9904626304495e-01
9.9400481099024e-01 9.8127064034285e-01 2.8768643817821e-01
6.6044473482754e-01 9.8397309021790e-01 2.7632770698083e-01
3.2681433812390e-01 9.8667440300737e-01 2.6497002066286e-01
9.9310257871131e-01 9.8937469489452e-01 2.5361330298239e-01
6.5929874777500e-01 9.9207409719465e-01 2.4225745342581e-01
3.2539266677178e-01 9.9477275102077e-01 2.3090235167857e-01
9.9137490879285e-01 9.9747080181702e-01 2.1954786301464e-01
6.5777008301608e-01 1.6586319518233e-04 2.0819674512149e-01
3.2349357859580e-01 2.8632104773902e-03 1.9684296588353e-01
9.8908363405398e-01 5.5603696178433e-03 1.8548936516948e-01
6.5453581993095e-01 8.2574575461465e-03 1.7413580740777e-01
3.1984725322831e-01 1.0954576748422e-02 1.6278216833353e-01
9.8501660796731e-01 1.3651814265477e-02 1.5142833767081e-01
6.5004407430114e-01 1.6349241546831e-02 1.4007422074120e-01
3.1493127509232e-01 1.9046915006538e-02 1.2871973909271e-01
9.7968115101681e-01 2.1744877105295e-02 1.1736483030284e-01
6.4429782577727e-01 2.4443157783825e-02 1.0600944713998e-01
3.0878646225522e-01 2.7141776089140e-02 9.4653556272786e-02
9.7315311885204e-01 2.9840741861666e-02 8.3297136702737e-02
6.3740461328745e-01 3.2540057381466e-02 7.1940178070193e-02
3.0154839908555e-01 3.5239718901626e-02 6.0582678953186e-02
9.6559245811924e-01 3.7939718023292e-02 4.9224645247252e-02
6.2954521103666e-01 4.0640042888412e-02 3.7866088686362e-02
2.9341544620730e-01 4.3340679182614e-02 2.6507025541441e-02
9.5721226699427e-01 4.6041610952049e-02 1.5147475514625e-02
6.2094505663992e-01 4.8742821245202e-02 3.7874608341532e-03
2.8432728557603e-01 5.1445074330374e-02 9.9242607498589e-01
9.4797804885250e-01 5.4146675589087e-02 9.8106533215439e-01
6.1159427402001e-01 5.6848501104612e-02 9.6970420171611e-01
2.7518643020804e-01 5.9550533389957e-02 9.5834271073892e-01
9.3876532149320e-01 6.2252755070511e-02 9.4698088704801e-01
6.0234213905807e-01 6.4955148775909e-02 9.3561875935772e-01
2.6592852663235e-01 6.7657696938529e-02 9.2425635755307e-01
9.2953665970480e-01 7.0360381498266e-02 9.1289371312373e-01
5.9317933937080e-01 7.3063183508860e-02 9.0153085975857e-01
2.5687010207117e-01 7.5766082636329e-02 8.9016783411422e-01
9.2062334687550e-01 7.8469056534980e-02 8.7880467677747e-01
5.8445448235808e-01 8.1172080080930e-02 8.6744143344697e-01
2.4838009548379e-01 8.3875124437274e-02 8.5607815636544e-01
9.1241814522137e-01 8.6578155919116e-02 8.4471490603805e-01
5.7658818375624e-01 8.9281134621318e-02 8.3335175327418e-01
2.4091160804474e-01 9.1984012768229e-02 8.2198878158623e-01
9.0541194380627e-01 9.4686732745151e-02 8.1062608996624e-01
5.7011516249669e-01 9.7389224779993e-02 7.9926379603135e-01
2.3505002871098e-01 1.0009140426750e-01 7.8790203947221e-01
9.0024846980565e-01 1.0279316877946e-01 7.7654098563714e-01
5.6574594972881e-01 1.0549439490124e-01 7.6518082891367e-01
2.3158181273196e-01 1.0819493520687e-01 7.5382179529382e-01
8.9779953649419e-01 1.1089461597364e-01 7.4246414308482e-01
5.6444679405976e-01 1.1359323669734e-01 7.3110816010897e-01
2.3157516583082e-01 1.1629057315575e-01 7.1975415490348e-01
8.9923926566503e-01 1.1898638670421e-01 7.0840243845233e-01
5.6749495767185e-01 1.2168044357210e-01 6.9705329213513e-01
2.3639627358711e-01 1.2437254877927e-01 6.8570691754951e-01
9.0599066492755e-01 1.2706259899033e-01 6.7436336595761e-01
5.7631245841616e-01 1.2975065555282e-01 6.6302245119188e-01
2.4737496162419e-01 1.3243703113915e-01 6.5168366144568e-01
9.1916260848299e-01 1.3512237010848e-01 6.4034610114241e-01
5.9162551934811e-01 1.3780768760088e-01 6.2900850637950e-01
2.6467908590966e-01 1.4049432594205e-01 6.1766937190872e-01
9.3820981170434e-01 1.4318380245766e-01 6.0632719223434e-01
6.1208574461306e-01 1.4587756331492e-01 5.9498076187746e-01
2.8616719774986e-01 1.4857670521711e-01 5.8362943406155e-01
9.6031342407402e-01 1.5128175058163e-01 5.7227323961109e-01
6.3438404334907e-01 1.5399254688158e-01 5.6091282258815e-01
3.0823810412127e-01 1.5670831834129e-01 5.4954922308405e-01
9.8173551061469e-01 1.5942784991200e-01 5.3818359147658e-01
6.5474369722012e-01 1.6214974474717e-01 5.2681693221434e-01
3.2714834921067e-01 1.6487267777818e-01 5.1544994882522e-01
9.9995935994495e-01 9.3566767349265e-01 5.0366008613424e-01
6.6552944088428e-01 9.3838542713793e-01 4.9229076350197e-01
3.3120550838226e-01 9.4110320986874e-01 4.8092148432641e-01
9.9698431860119e-01 9.4382096130465e-01 4.6955225419291e-01
6.6286148470126e-01 9.4653860988944e-01 4.5818308580226e-01
3.2883147086273e-01 9.4925607264708e-01 4.4681400078205e-01
9.9488763264293e-01 9.5197325555934e-01 4.3544503128323e-01
6.6102230397213e-01 9.5469005466248e-01 4.2407622118451e-01
3.2722692616360e-01 9.5740635791470e-01 4.1270762672349e-01
9.9349220930131e-01 9.6012204782660e-01 4.0133931639031e-01
6.5980831213718e-01 9.6283700478018e-01 3.8997136995911e-01
3.2616502403485e-01 9.6555111089632e-01 3.7860387659263e-01
9.9255193218290e-01 9.6826425425471e-01 3.6723693203073e-01
6.5895855952402e-01 9.7097633323031e-01 3.5587063495508e-01
3.2537446338691e-01 9.7368726069100e-01 3.4450508270245e-01
9.9178929099614e-01 9.7639696780244e-01 3.3314036656810e-01
6.5819279488353e-01 9.7910540720605e-01 3.2177656699231e-01
3.2457481762509e-01 9.8181255537064e-01 3.1041374895299e-01
9.9092526025076e-01 9.8451841396455e-01 2.9905195789224e-01
6.5723405134149e-01 9.8722301014977e-01 2.8769121648319e-01
3.2349113382248e-01 9.8992639576140e-01 2.7633152249516e-01
9.8968648378196e-01 9.9262864540332e-01 2.6497284794174e-01
6.5581017070950e-01 9.9532985356050e-01 2.5361513960060e-01
3.2185246213404e-01 9.9803013089726e-01 2.4225832088421e-01
9.8838029959581e-01 7.2686612582989e-04 2.3090542846067e-01
6.5422184302481e-01 3.4257255534533e-03 2.1955000446510e-01
3.1995543184142e-01 6.1240410925462e-03 2.0819513333906e-01
9.8557380629554e-01 8.8219435323338e-03 1.9684068405627e-01
6.5107077964100e-01 1.1519558073523e-02 1.8548652408812e-01
3.1644137901765e-01 1.4217001132948e-02 1.7413252397337e-01
9.8168194483929e-01 1.6914377938198e-02 1.6277856124003e-01
6.4679018389235e-01 1.9611780945614e-02 1.5142452351086e-01
3.1176517627318e-01 2.2309289046324e-02 1.4007031071713e-01
9.7660734044243e-01 2.5006967471217e-02 1.2871583643054e-01
6.4131836363422e-01 2.7704868270615e-02 1.1736102839070e-01
3.0590110648990e-01 3.0403031228422e-02 1.0600582835141e-01
9.7035949119951e-01 3.3101485070732e-02 9.4650191392763e-02
6.3469838188758e-01 3.5800248841485e-02 8.3294084850452e-02
2.9892346479705e-01 3.8499333337703e-02 7.1937487003945e-02
9.6304113431851e-01 4.1198742520156e-02 6.0580385646068e-02
6.2705838933945e-01 4.3898474838542e-02 4.9222776633219e-02
2.9098274293114e-01 4.6598524431184e-02 3.7864662491234e-02
9.5482214715774e-01 4.9298882176656e-02 2.6506051129676e-02
6.1858493382953e-01 5.1999536588271e-02 1.5146954698281e-02
2.8227977133314e-01 5.4700474552016e-02 3.7873886043020e-03
9.4558458956693e-01 5.7402561270440e-02 9.9242632468171e-01
6.0918738372809e-01 6.0103911212232e-02 9.8106600069445e-01
2.7274982717420e-01 6.2805499257638e-02 9.6970526687123e-01
9.3628180972371e-01 6.5507310339729e-02 9.5834414516020e-01
5.9979356758228e-01 6.8209329430637e-02 9.4698265832751e-01
2.6329572732658e-01 7.0911541544760e-02 9.3562082993054e-01
9.2679936373681e-01 7.3613931649299e-02 9.2425868443584e-01
5.9031607202945e-01 7.6316484483531e-02 9.1289624748384e-01
2.5385805543731e-01 7.9019184284173e-02 9.0153354630759e-01
9.1743822952212e-01 8.1722014409629e-02 8.9017061031852e-01
5.8107034508616e-01 8.4424956850664e-02 8.7880747187897e-01
2.4476913207582e-01 8.7127991608996e-02 8.6744416728889e-01
9.0855046743841e-01 8.9831095918347e-02 8.5608073802277e-01
5.7243157049281e-01 9.2534243274462e-02 8.4471723226179e-01
2.3643122997525e-01 9.5237402231551e-02 8.3335370677564e-01
9.0057006745813e-01 9.7940534912879e-02 8.2199022921623e-01
5.6487084218916e-01 1.0064359517360e-01 8.1062688089046e-01
2.2935880232618e-01 1.0334652634658e-01 7.9926376007527e-01
8.9406208663695e-01 1.0604925850069e-01 7.8790098592017e-01
5.5901217826768e-01 1.0875170515347e-01 7.7653870293464e-01
2.2424440692613e-01 1.1145375942009e-01 7.6517708596117e-01
8.8979848572171e-01 1.1415528967078e-01 7.5381634535329e-01
5.5571905064833e-01 1.1685613495066e-01 7.4245673176025e-01
2.2205613920012e-01 1.1955610074907e-01 7.3109853939068e-01
8.8886549263276e-01 1.2225495628503e-01 7.1974210579232e-01
5.5620848502668e-01 1.2495243541520e-01 7.0838780495482e-01
2.2415136418442e-01 1.2764824467960e-01 6.9703602891286e-01
8.9276333809293e-01 1.3034208385169e-01 6.8568715127144e-01
5.6211289197640e-01 1.3303368622660e-01 6.7434146510671e-01
2.3226167888806e-01 1.3572288643176e-01 6.6299908959073e-01
9.0325559932948e-01 1.3840971999647e-01 6.5165984793800e-01
5.7511355392129e-01 1.4109454724131e-01 6.4032313754412e-01
2.4781594354039e-01 1.4377817176285e-01 6.2898784063141e-01
9.2129678019910e-01 1.4646189663667e-01 6.1765234702766e-01
5.9544372491753e-01 1.4914744961655e-01 6.0631475156939e-01
2.7010775434406e-01 1.5183673875897e-01 5.9497322294249e-01
9.4511889249922e-01 1.5453147573335e-01 5.8362643762732e-01
6.2030043408531e-01 1.5723278369757e-01 5.7227390445341e-01
2.9547566065204e-01 1.5994093140295e-01 5.6091603699288e-01
9.7046775094335e-01 1.6265528815265e-01 5.4955394816430e-01
6.4509969122972e-01 1.6537450930579e-01 5.3818906552476e-01
3.1920099143252e-01 1.6809688067756e-01 5.2682272899355e-01
9.9262190837165e-01 1.7082069917851e-01 5.1545591437563e-01
6.6665934556232e-01 9.4161432526775e-01 5.0367160355196e-01
3.3212564669810e-01 9.4433203354290e-01 4.9230220845174e-01
9.9768608554048e-01 9.4704976311011e-01 4.8093285480230e-01
6.6333775322125e-01 9.4976745803802e-01 4.6956354807407e-01
3.2907676110388e-01 9.5248505280924e-01 4.5819429983689e-01
9.9489823109454e-01 9.5520247205295e-01 4.4682512934113e-01
6.6079632401415e-01 9.5791963078406e-01 4.3545606492941e-01
3.2676430672274e-01 9.6063643523131e-01 4.2408714513735e-01
9.9279465488080e-01 9.6335278430203e-01 4.1271841933713e-01
6.5887918428152e-01 9.6606857168613e-01 4.0134994778864e-01
3.2500920021583e-01 9.6878368855029e-01 3.8998180099031e-01
9.9117565198208e-01 9.7149802672150e-01 3.7861405826621e-01
6.5736927894098e-01 9.7421148221264e-01 3.6724680558176e-01
3.2358073571121e-01 9.7692395890811e-01 3.5588013264409e-01
9.8980068713548e-01 9.7963537220734e-01 3.4451412940601e-01
6.5601986812063e-01 9.8234565241986e-01 3.3314888215027e-01
3.2222910870788e-01 9.8505474771590e-01 3.2178446937622e-01
9.8841932995891e-01 9.8776262645975e-01 3.1042095774127e-01
6.5458152065811e-01 9.9046927878536e-01 2.9905839832163e-01
3.2070670776757e-01 9.9317471731332e-01 2.8769682344965e-01
9.8678593459725e-01 9.9587897695424e-01 2.7633624435749e-01
6.5281025961642e-01 9.9858211379382e-01 2.6497664980940e-01
3.1938673179859e-01 1.2812830144697e-03 2.5362135279458e-01
9.8526528556494e-01 3.9824794927291e-03 2.4226353244452e-01
6.5106253970408e-01 6.6828269935311e-03 2.3090652890850e-01
3.1677026579177e-01 9.3824396118526e-03 2.1955024953923e-01
9.8238073696290e-01 1.2081436780317e-02 2.0819458839523e-01
6.4788689315514e-01 1.4779939347383e-02 1.9683943037249e-01
3.1328250220238e-01 1.7478065918516e-02 1.8548465551194e-01
9.7856230375036e-01 2.0175929646108e-02 1.7413014319422e-01
6.4372212529655e-01 2.2873635604398e-02 1.6277577596812e-01
3.0875896310993e-01 2.5571278827377e-02 1.5142144281697e-01
9.7367102472243e-01 2.8268943028698e-02 1.4006704173720e-01
6.3845773348126e-01 3.0966699970144e-02 1.2871248157596e-01
3.0311969882339e-01 3.3664609404418e-02 1.1735768313999e-01
9.6765865818751e-01 3.6362719491550e-02 1.0600257963987e-01
6.3207739773103e-01 3.9061067576027e-02 9.4647116569097e-02
2.9637965935127e-01 4.1759681211969e-02 8.3291251135808e-02
9.6057004111246e-01 4.4458579333128e-02 7.1934951369200e-02
6.2465389729050e-01 4.7157773479837e-02 6.0578195015384e-02
2.8863724309689e-01 4.9857269013054e-02 4.9220968323356e-02
9.5252666792633e-01 5.2557066263855e-02 3.7863264803672e-02
6.1632925982876e-01 5.5257161583382e-02 2.6505084023642e-02
2.8005254292614e-01 5.7957548272367e-02 1.5146430485163e-02
9.4370442871641e-01 6.0658217380462e-02 3.7873126160005e-03
6.0692772606888e-01 6.3360134040470e-02 9.9242658200704e-01
2.7047835016120e-01 6.6061224689991e-02 9.8106669666320e-01
9.3398313541550e-01 6.8762561898811e-02 9.6970638913530e-01
5.9745134013018e-01 7.1464133153405e-02 9.5834567619804e-01
2.6089257877284e-01 7.4165925827212e-02 9.4698457558581e-01
9.2431685733659e-01 7.6867927289047e-02 9.3562310581795e-01
5.8773462275802e-01 7.9570124921413e-02 9.2426128615869e-01
2.5115682700568e-01 8.2272506051640e-02 9.1289913671323e-01
9.1459500683609e-01 8.4975057795392e-02 9.0153667866548e-01
5.7806138066650e-01 8.7677766808105e-02 8.9017393466821e-01
2.4156896453754e-01 9.0380618935073e-02 8.7881092940323e-01
9.0513170974207e-01 9.3083598745162e-02 8.6744769033670e-01
5.6876466539141e-01 9.5786688926079e-02 8.5608424870486e-01
2.3248416998667e-01 9.8489869510659e-02 8.4472064077658e-01
8.9630807696922e-01 1.0119311689310e-01 8.3335690945261e-01
5.6025602023016e-01 1.0389640258149e-01 8.2199310627694e-01
2.2434972663307e-01 1.0659969161777e-01 8.1062929395170e-01
8.8861338365013e-01 1.0930294057928e-01 7.9926554946361e-01
5.5307407103392e-01 1.1200609505744e-01 7.8790196794099e-01
2.1776226564443e-01 1.1470908649387e-01 7.7653866736097e-01
8.8271242736842e-01 1.1741182824695e-01 7.6517579420044e-01
5.4796367010880e-01 1.2011421077944e-01 7.5381353004797e-01
2.1356051258545e-01 1.2281609592532e-01 7.4245209902295e-01
8.7955368481229e-01 1.2551731036281e-01 7.3109177550952e-01
5.4600093027090e-01 1.2821763877758e-01 7.1973289109049e-01
2.1296767934046e-01 1.3091681788846e-01 7.0837583848980e-01
8.8052735965533e-01 1.3361453374234e-01 6.9702106858915e-01
5.4876093392861e-01 1.3631042672943e-01 6.8566907400172e-01
2.1775500407712e-01 1.3900411180055e-01 6.7432034940027e-01
8.8759752277427e-01 1.4169522505264e-01 6.6297531588596e-01
5.5836995323659e-01 1.4438351039417e-01 6.5163419730445e-01
2.3013497102936e-01 1.4706895673129e-01 6.4029684696645e-01
9.0292009036622e-01 1.4975197890030e-01 6.2896255213883e-01
5.7670034267781e-01 1.5243359737144e-01 6.1762989269549e-01
2.5138650682182e-01 1.5511552106954e-01 6.0629677609211e-01
9.2682629010470e-01 1.5780001499667e-01 5.9496075513683e-01
6.0282066560931e-01 1.6048949589305e-01 5.8361960733120e-01
2.7914771269338e-01 1.6318594639972e-01 5.7227196417419e-01
9.5558070266781e-01 1.6589037183918e-01 5.6091768607246e-01
6.3189341052183e-01 1.6860253016032e-01 5.4955779253793e-01
3.0785890092721e-01 1.7132104246119e-01 5.3819399791717e-01
9.8325483996676e-01 1.7404382654575e-01 5.2682809433825e-01
6.5788158082323e-01 1.7676866791258e-01 5.1546146002065e-01
3.3335570992398e-01 9.4756090651689e-01 5.0368325441330e-01
9.9873000749128e-01 9.5027857663765e-01 4.9231379405007e-01
6.6418719428236e-01 9.5299626114939e-01 4.8094437263055e-01
3.2972465144856e-01 9.5571390792545e-01 4.6957499555131e-01
9.9533892268168e-01 9.5843145663527e-01 4.5820567344665e-01
6.6102570161491e-01 9.6114883846048e-01 4.4683642357937e-01
3.2677985060288e-01 9.6386597622611e-01 4.3546727109604e-01
9.9259545180325e-01 9.6658278501679e-01 4.2409825003283e-01
6.5846588852044e-01 9.6929917332133e-01 4.1272940395367e-01
3.2438395164359e-01 9.7201504471468e-01 4.0136078610872e-01
9.9034196316789e-01 9.7473030004587e-01 3.8999245902127e-01
6.5633190671391e-01 9.7744484005955e-01 3.7862449344288e-01
3.2234555406345e-01 9.8015856834049e-01 3.6725696665937e-01
9.8837457726309e-01 9.8287139444062e-01 3.5588996017861e-01
6.5441063782382e-01 9.8558323702847e-01 3.4452355688160e-01
3.2044544774212e-01 9.8829402689378e-01 3.3315783776530e-01
9.8647080104465e-01 9.9100370964425e-01 3.2179287844503e-01
6.5247857874963e-01 9.9371224794605e-01 3.1042874561304e-01
3.1846073393822e-01 9.9641962318228e-01 2.9906549366506e-01
9.8440926650082e-01 9.9912583643248e-01 2.8770316170803e-01
6.5096809684231e-01 1.8278191063694e-03 2.7634531229780e-01
3.1681699661391e-01 4.5318479711615e-03 2.6498480063585e-01
9.8260823822920e-01 7.2348368615309e-03 2.5362521204012e-01
6.4833392790762e-01 9.9368604555382e-03 2.4226650840846e-01
3.1398629528893e-01 1.2638007004712e-02 2.3090863274319e-01
9.7955779094107e-01 1.5338375308610e-02 2.1955151112932e-01
6.4504120813734e-01 1.8038071402612e-02 2.0819505545653e-01
3.1042982034640e-01 2.0737205144495e-02 1.9683916669624e-01
9.7571752387939e-01 2.3435886886948e-02 1.8548373850799e-01
6.4089897458272e-01 2.6134224408345e-02 1.7412866093605e-01
3.0596970833669e-01 2.8832320244309e-02 1.6277382396665e-01
9.7092623716985e-01 3.1530269521793e-02 1.5141912074848e-01
6.3576611561984e-01 3.4228158351136e-02 1.4006445032609e-01
3.0048797508063e-01 3.6926062785618e-02 1.2870971979205e-01
9.6509152682611e-01 3.9624048317682e-02 1.1735484582058e-01
6.2957753684177e-01 4.2322169849864e-02 1.0599975559540e-01
2.9394777733011e-01 4.5020472058311e-02 9.4644387185400e-02
9.5820496072464e-01 4.7718990057438e-02 8.3288689448488e-02
6.2235266231708e-01 5.0417750274451e-02 7.1932621559012e-02
2.8639523731358e-01 5.3116771449651e-02 6.0576152257350e-02
9.5033773746435e-01 5.5816065690300e-02 4.9219258915381e-02
6.1418583152987e-01 5.8515639519865e-02 3.7861926500687e-02
2.7794573290268e-01 6.1215494878802e-02 2.6504146508478e-02
9.4162413680708e-01 6.3915630046392e-02 1.5145915915486e-02
6.0522816871996e-01 6.6616040464546e-02 3.7872361959989e-03
2.6836594695664e-01 6.9317790123053e-02 9.9242684025819e-01
9.3186033637864e-01 7.2018620263949e-02 9.8106740313793e-01
5.9530375600511e-01 7.4719700013127e-02 9.6970754127996e-01
2.5870478796593e-01 7.7421019532277e-02 9.5834726631978e-01
9.2207237815272e-01 8.0122568649383e-02 9.4698659106705e-01
5.8541586280855e-01 8.2824337061531e-02 9.3562552920173e-01
2.4874500903099e-01 8.5526314451836e-02 9.2426409509110e-01
9.1207006985825e-01 8.8228490524355e-02 9.1290230372863e-01
5.7540185496569e-01 9.0930854958465e-02 9.0154017079840e-01
2.3875181844092e-01 9.3633397280947e-02 8.9017771287296e-01
9.0213216563672e-01 9.6336106650100e-02 8.7881494775818e-01
5.6555598173749e-01 9.9038971541195e-02 8.6745189500590e-01
2.2903738543503e-01 1.0174197931638e-01 8.5608857662443e-01
8.9259171202007e-01 1.0444511565427e-01 8.4472501802845e-01
5.5623573128653e-01 1.0714836380435e-01 8.3336124928480e-01
2.1998790694942e-01 1.0985170361847e-01 8.2199730672840e-01
8.8386870582286e-01 1.1255511029518e-01 8.1063323504541e-01
5.4790096680591e-01 1.1525855275139e-01 7.9926908994720e-01
2.1211034175034e-01 1.1796199150976e-01 7.8790494158997e-01
8.7652582241702e-01 1.2066537595825e-01 7.7654087892731e-01
5.4118036965309e-01 1.2336864080255e-01 7.6517701521183e-01
2.0611166194977e-01 1.2607170149725e-01 7.5381349487376e-01
8.7136297924900e-01 1.2877444842042e-01 7.4245050197281e-01
5.3698423144998e-01 1.3147673957643e-01 7.3108827029390e-01
2.0303312417591e-01 1.3417839173150e-01 7.1972709484380e-01
8.6957641713617e-01 1.3687917021764e-01 7.0836734383718e-01
5.3669115544309e-01 1.3957877839069e-01 6.9700946896350e-01
2.0446561278319e-01 1.4227684925221e-01 6.8565400938217e-01
8.7299943555124e-01 1.4497294457848e-01 6.7430158099828e-01
5.4240207376269e-01 1.4766657165826e-01 6.6295283683270e-01
2.1278802559840e-01 1.5035723454654e-01 6.5160837764791e-01
8.8426687781027e-01 1.5304454355750e-01 6.4026858885245e-01
5.5692617950961e-01 1.5572840599185e-01 6.2893339147201e-01
2.3080703440995e-01 1.5840929655276e-01 6.1760194045391e-01
9.0587714479171e-01 1.6108853889300e-01 6.0627239356879e-01
5.8201266729531e-01 1.6376843104763e-01 5.9494196781222e-01
2.5900209485042e-01 1.6645200226418e-01 5.8360747182712e-01
9.3657450218201e-01 1.6914232167210e-01 5.7226624122976e-01
6.1443555404364e-01 1.7184157624579e-01 5.6091704280776e-01
2.9228867681645e-01 1.7455035271806e-01 5.4956041993634e-01
9.6983719312623e-01 1.7726748170272e-01 5.3819828162849e-01
6.4678645647593e-01 1.7999050455047e-01 5.2683300723844e-01
3.2286502289343e-01 1.8271651593063e-01 5.1546657802934e-01
4.8641717671535e-05 9.5350742477409e-01 5.0369502423906e-01
6.6534212419456e-01 9.5622506305130e-01 4.9232550574751e-01
3.3070775559031e-01 9.5894270958881e-01 4.8095602324635e-01
9.9614316890624e-01 9.6166031551634e-01 4.6958658208030e-01
6.6164528487379e-01 9.6437782497147e-01 4.5821719209659e-01
3.2721029696937e-01 9.6709517480316e-01 4.4684786887850e-01
9.9283368237566e-01 9.6981229461720e-01 4.3547863486857e-01
6.5851023994534e-01 9.7252910722320e-01 4.2410952028942e-01
3.2423415390188e-01 9.7524552952223e-01 4.1274056376573e-01
9.8999907952127e-01 9.7796147384786e-01 4.0137181255503e-01
6.5579824470983e-01 9.8067684974165e-01 3.9000332230881e-01
3.2162455958996e-01 9.8339156611123e-01 3.7863515630926e-01
9.8747072525143e-01 9.8610553368822e-01 3.6726738415873e-01
6.5332933294012e-01 9.8881866767719e-01 3.5590007993678e-01
3.1919294619930e-01 9.9153089046921e-01 3.4453331987936e-01
9.8505416073133e-01 9.9424213428432e-01 3.3316717967327e-01
6.5090563972649e-01 9.9695234360798e-01 3.2180173149231e-01
3.1674012571221e-01 9.9966147729493e-01 3.1043704092764e-01
9.8323461491200e-01 2.3662679206530e-03 2.9907688003493e-01
6.4900607146563e-01 5.0732434368041e-03 2.8771380243854e-01
3.1473897057205e-01 7.7791220221872e-03 2.7635160875985e-01
9.8042621814616e-01 1.0483930980126e-02 2.6499031140836e-01
6.4606072649823e-01 1.3187714445529e-02 2.5362990515269e-01
3.1163543108647e-01 1.5890532024407e-02 2.4227036676451e-01
9.7714333234733e-01 1.8592456881739e-02 2.3091165546110e-01
6.4257756362040e-01 2.1293573371719e-02 2.1955371412668e-01
3.0793148286433e-01 2.3993974332573e-02 2.0819647123778e-01
9.7319878286298e-01 2.6693758188660e-02 1.9683984336772e-01
6.3837361234670e-01 2.9393026012627e-02 1.8548373810597e-01
3.0345069918644e-01 3.2091878697990e-02 1.7412805720365e-01
9.6842546667578e-01 3.4790414377621e-02 1.6277269975053e-01
6.3329413484132e-01 3.7488726197753e-02 1.5141756520072e-01
2.9805380049543e-01 4.0186900523676e-02 1.4006255609231e-01
9.6270249204569e-01 4.2885015616385e-02 1.2870758034531e-01
6.2723919754261e-01 4.5583140783510e-02 1.1735255306695e-01
2.9166386675224e-01 4.8281335976628e-02 1.0599739783795e-01
9.5597738993531e-01 5.0979651783087e-02 9.4642047492848e-02
6.2018155735959e-01 5.3678129745029e-02 8.3286444438759e-02
2.8427900432954e-01 5.6376802931114e-02 7.1930540578216e-02
9.4827314673977e-01 5.9075696686531e-02 6.0574296913498e-02
6.1216811194961e-01 6.1774829492476e-02 4.9217682912754e-02
2.7596866926923e-01 6.4474213875445e-02 3.7860675714630e-02
9.3968016367105e-01 6.7173857317730e-02 2.6503259239902e-02
6.0330845560764e-01 6.9873763131883e-02 1.5145423267836e-02
2.6685986911464e-01 7.2573931272621e-02 3.7871625233750e-03
9.2990827445850e-01 7.5275523432817e-02 9.9242709294086e-01
5.9334252694462e-01 7.7976098437704e-02 9.8106810350287e-01
2.5672106323133e-01 8.0676920801758e-02 9.6970869621705e-01
9.2005175341599e-01 8.3377983467626e-02 9.5834887769267e-01
5.8334283618568e-01 8.6079278769814e-02 9.4698865598573e-01
2.4660293679469e-01 8.8780798717457e-02 9.3562804019110e-01
9.0984109844738e-01 9.1482535196893e-02 9.2426704013906e-01
5.7306682782146e-01 9.4184480098167e-02 9.1290566619718e-01
2.3629015577853e-01 9.6886625368234e-02 9.0154392917966e-01
8.9952171471534e-01 9.9588962991371e-02 8.9018184036926e-01
5.6277283452166e-01 1.0229148489444e-01 8.7881941166085e-01
2.2605565974260e-01 1.0499418277073e-01 8.6745665584160e-01
8.8938329132196e-01 1.0769704781113e-01 8.5609358703062e-01
5.5276995725915e-01 1.1040007032495e-01 8.4473022131089e-01
2.1623121769384e-01 1.1310323922405e-01 8.3336657759991e-01
8.7978421139389e-01 1.1580654133281e-01 8.2200267882302e-01
5.4344795243566e-01 1.1850996047115e-01 8.1063855347645e-01
2.0724368811202e-01 1.2121347623771e-01 7.9927423769688e-01
8.7119533187113e-01 1.2391706239286e-01 7.8790977799347e-01
5.3532998845288e-01 1.2662068470514e-01 7.7654523484652e-01
1.9967859237454e-01 1.2932429807675e-01 7.6518068743735e-01
8.6427668539844e-01 1.3202784270358e-01 7.5381623984184e-01
5.2916536313453e-01 1.3473123895166e-01 7.4245202908953e-01
1.9439242428807e-01 1.3743438055435e-01 7.3108823553614e-01
8.6001375550633e-01 1.4013712567340e-01 7.1972509597129e-01
5.2609497439475e-01 1.4283928538137e-01 7.0836291967484e-01
1.9271332090184e-01 1.4554060934066e-01 6.9700210703127e-01
8.5995970890534e-01 1.4824076914806e-01 6.8564316890552e-01
5.2794068047871e-01 1.5093934151898e-01 6.7428674206408e-01
1.9677966752207e-01 1.5363579716039e-01 6.6293359039049e-01
8.6661634671388e-01 1.5632950828104e-01 6.5158457221788e-01
5.3760187528603e-01 1.5901979960589e-01 6.4024054066049e-01
2.0988654245377e-01 1.6170608319367e-01 6.2890213158001e-01
8.8359575554011e-01 1.6438812508713e-01 6.1756940206376e-01
5.5879270382082e-01 1.6706645972974e-01 6.0624135208447e-01
2.3543478645649e-01 1.6974284797466e-01 5.9491553194816e-01
9.1334455156032e-01 1.7242047281849e-01 5.8358813892165e-01
5.9221987249684e-01 1.7510347230164e-01 5.7225495142315e-01
2.7168392333804e-01 1.7779571511578e-01 5.6091288032884e-01
9.5133853641435e-01 1.8049934697627e-01 5.4956121964751e-01
6.3078529699480e-01 1.8321395302775e-01 5.3820171414142e-01
3.0962582917153e-01 1.8593682632657e-01 5.2683743163771e-01
9.8748289059670e-01 1.8866414660425e-01 5.1547126237399e-01
6.6673830059649e-01 9.5945388745129e-01 5.0370689870731e-01
3.3196165959408e-01 9.6217149942473e-01 4.9233732915332e-01
9.9724685407804e-01 9.6488911418351e-01 4.8096779224179e-01
6.6259173627334e-01 9.6760668564419e-01 4.6959829327185e-01
3.2799355733570e-01 9.7032416180316e-01 4.5822884141380e-01
9.9344895029867e-01 9.7304148442951e-01 4.4685945081328e-01
6.5895393451855e-01 9.7575858904073e-01 4.3549014160358e-01
3.2450394274978e-01 9.7847540521258e-01 4.2412094075139e-01
9.9009387014879e-01 9.8119185725837e-01 4.1275188265709e-01
6.5571814250867e-01 9.8390786529237e-01 4.0138300943306e-01
3.2137079911523e-01 9.8662334666703e-01 3.9001437079360e-01
9.8704558406156e-01 9.8933821774738e-01 3.7864602350725e-01
6.5273603891647e-01 9.9205239596051e-01 3.6727803038684e-01
3.1843558950169e-01 9.9476580203618e-01 3.5591045882139e-01
9.8413762026864e-01 9.9747836233818e-01 3.4454337888569e-01
6.5055479356813e-01 1.8658966661559e-04 3.3318078168155e-01
3.1623556674268e-01 2.8973148383911e-03 3.2181484550282e-01
9.8189905347272e-01 5.6070315607081e-03 3.1044960136844e-01
6.4753884404293e-01 8.3157100264677e-03 2.9908510564565e-01
3.1314858276330e-01 1.1023335305392e-02 2.8772140481051e-01
9.7872194727194e-01 1.3729908058546e-02 2.7635853320337e-01
6.4425262869912e-01 1.6435444731595e-02 2.6499651119376e-01
3.0973432009831e-01 1.9139977208635e-02 2.5363534386855e-01
9.7516071962703e-01 2.1843551934031e-02 2.4227502033016e-01
6.4052555323006e-01 2.4546228536563e-02 2.3091551365658e-01
3.0582261925254e-01 2.7248078016978e-02 2.1955678153369e-01
9.7104585478578e-01 2.9949180584898e-02 2.0819876752697e-01
6.3618942095556e-01 3.2649623251629e-02 1.9684140291652e-01
3.0124780213635e-01 3.5349497299243e-02 1.8548460898171e-01
9.6621591248807e-01 3.8048895751478e-02 1.7412829959381e-01
6.3108920244752e-01 4.0747910967312e-02 1.6277238395928e-01
2.9586375791456e-01 4.3446632463911e-02 1.5141676935502e-01
9.6053638577901e-01 4.6145145053500e-02 1.4006136370975e-01
6.2510468095170e-01 4.8843527351283e-02 1.2870607790968e-01
2.8956707193990e-01 5.1541850682018e-02 1.1735082773896e-01
9.5392284397246e-01 5.4240178384627e-02 1.0599553540128e-01
6.1817214049312e-01 5.6938565489887e-02 9.4640130604070e-02
2.8231594532187e-01 5.9637058727671e-02 8.3284551217856e-02
9.4635604883626e-01 6.2335696808268e-02 7.1928743547844e-02
6.1029500212227e-01 6.5034510916762e-02 6.0572662271485e-02
2.7413606322947e-01 6.7733525359481e-02 4.9216270105214e-02
9.3788313951603e-01 7.0432758305885e-02 3.7859537266036e-02
6.0154072968568e-01 7.3132222576428e-02 2.6502440791065e-02
2.6511386859952e-01 7.5831926435655e-02 1.5144963771765e-02
9.2860807737748e-01 7.8531874358868e-02 3.7870945513019e-03
5.9156343337125e-01 8.1233324499933e-02 9.9242733393819e-01
2.5493383498319e-01 8.3933655980639e-02 9.8106878189649e-01
9.1824418004733e-01 8.6634227494565e-02 9.6970982775590e-01
5.8150160061055e-01 8.9335034849057e-02 9.5835047328151e-01
2.4471359883007e-01 9.2036072933126e-02 9.4699072199634e-01
9.0788805676127e-01 9.4737336063226e-02 9.3563057869858e-01
5.7103325869726e-01 9.7438818255887e-02 9.2427004906518e-01
2.3415792686345e-01 1.0014051343085e-01 9.1290913934308e-01
8.9727127152345e-01 1.0284241554795e-01 9.0154785612954e-01
5.6038305691028e-01 1.0554451867990e-01 8.9018620624411e-01
2.2350368486660e-01 1.0824681702105e-01 8.7882419669750e-01
8.8664429867801e-01 1.1094930482990e-01 8.6746183476639e-01
5.4981691033928e-01 1.1365197629922e-01 8.5609912818909e-01
2.1303455544369e-01 1.1635482534298e-01 8.4473608550471e-01
8.7631148108176e-01 1.1905784528284e-01 8.3337271656930e-01
5.3966337364843e-01 1.2176102840826e-01 8.2200903329700e-01
2.0310763538003e-01 1.2446436537192e-01 8.1064505069410e-01
8.6666372089675e-01 1.2716784436604e-01 7.9928078828113e-01
5.3035354816927e-01 1.2987145000122e-01 7.8791627203508e-01
1.9420200235706e-01 1.3257516177729e-01 7.7655153703441e-01
8.5823755611646e-01 1.3527895198924e-01 7.6518663105780e-01
5.2249303650530e-01 1.3798278284756e-01 7.5382161947865e-01
1.8700657673457e-01 1.4068660250326e-01 7.4245659191577e-01
8.5182280075458e-01 1.4339033954843e-01 7.3109167124740e-01
5.1699429944635e-01 1.4609389540892e-01 7.1972702576019e-01
1.8258346699057e-01 1.4879713386446e-01 7.0836288535042e-01
8.4866476934775e-01 1.5149986676394e-01 6.9699956271837e-01
5.1532750056323e-01 1.5420183497001e-01 6.8563748015834e-01
1.8267901707259e-01 1.5690268397861e-01 6.7427720131644e-01
8.5084826142347e-01 1.5960193523422e-01 6.6291946408292e-01
5.1998896611964e-01 1.6229895843138e-01 6.5156520357534e-01
1.9028103344130e-01 1.6499295994435e-01 6.4021553972476e-01
8.6192689132662e-01 1.6768302223425e-01 6.2887167902520e-01
5.3513698449654e-01 1.7036826100945e-01 6.1753464782427e-01
2.1009610084109e-01 1.7304819746547e-01 6.0620476469999e-01
8.8690464960326e-01 1.7572341062718e-01 5.9488085937921e-01
5.6550502080338e-01 1.7839631592134e-01 5.8355957545581e-01
2.4563289465538e-01 1.8107148035039e-01 5.7223554883836e-01
9.2684228818685e-01 1.8375467603529e-01 5.6090313650018e-01
6.0859583056459e-01 1.8645063219956e-01 5.4955904794978e-01
2.9034010720487e-01 1.8916079548878e-01 5.3820388735998e-01
9.7152159909369e-01 1.9188270990456e-01 5.2684129405396e-01
6.5160262214630e-01 1.9461142099914e-01 5.1547550856782e-01
3.3342482048836e-01 9.6540030184589e-01 5.0371886364650e-01
9.9858833100349e-01 9.6811789240385e-01 4.9234925003087e-01
6.6380371798288e-01 9.7083548081900e-01 4.8097966536441e-01
3.2906902480898e-01 9.7355302338989e-01 4.6961011489346e-01
9.9438179360877e-01 9.7627047145263e-01 4.5824060719004e-01
6.5973904629402e-01 9.7898777106333e-01 4.4687115514990e-01
3.2513728342970e-01 9.8170486291546e-01 4.3550177691243e-01
9.9057250210959e-01 9.8442168253670e-01 4.2413249664121e-01
6.5604023255396e-01 9.8713816079720e-01 4.1276334510030e-01
3.2153559153347e-01 9.8985422474470e-01 4.0139435996300e-01
9.8705334914737e-01 9.9256979876240e-01 3.9002558579676e-01
6.5258800415035e-01 9.9528480602399e-01 3.7865707368110e-01
3.1813386213258e-01 9.9799917019949e-01 3.6728888043345e-01
9.8442837513390e-01 7.0928280163893e-04 3.5592511750079e-01
6.4997360741771e-01 3.4221808244795e-03 3.4455770687500e-01
3.1551236088213e-01 6.1342311765333e-03 3.3319080421229e-01
9.8103881065010e-01 8.8453790431304e-03 3.2182447414643e-01
6.4654718968686e-01 1.1555579727458e-02 3.1045877808336e-01
3.1203179258655e-01 1.4264800177427e-02 2.9909377186928e-01
9.7748696657108e-01 1.6973020190536e-02 2.8772950351992e-01
6.4290709447255e-01 1.9680233237323e-02 2.7636601112911e-01
3.0828657551904e-01 2.2386446860057e-02 2.6500332107063e-01
9.7361981010736e-01 2.5091682621462e-02 2.5364144659615e-01
6.3890119438157e-01 2.7795975598035e-02 2.4228038691332e-01
3.0412512938393e-01 3.0499373433630e-02 2.3092012680357e-01
9.6928604792952e-01 3.3201934990587e-02 2.1956063680858e-01
6.3437846036492e-01 3.5903728656809e-02 2.0820187398041e-01
2.9939701824709e-01 3.8604830386049e-02 1.9684378315496e-01
9.6433659297479e-01 4.1305321563514e-02 1.8548629867523e-01
6.2919236476593e-01 4.4005286797960e-02 1.7412934646258e-01
2.9395991628706e-01 4.6704811743293e-02 1.6277284631466e-01
9.5863532481759e-01 4.9403981046917e-02 1.5141671429887e-01
6.2321524708236e-01 5.2102876509033e-02 1.4006086511218e-01
2.8769699173414e-01 5.4801575518251e-02 1.2870521429028e-01
9.5207857575623e-01 5.7500149806405e-02 1.1734968016982e-01
6.1635876258768e-01 6.0198664541822e-02 1.0599418553424e-01
2.8053708134643e-01 6.2897177757950e-02 9.4638658902294e-02
9.4461382795970e-01 6.5595740095201e-02 8.3283035446447e-02
6.0859005017813e-01 6.8294394819399e-02 7.1927257552841e-02
2.7246751927867e-01 7.0993178070876e-02 6.0571275053779e-02
9.3624869173822e-01 7.3692119293999e-02 4.9215045177072e-02
5.9993666431525e-01 7.6391241796866e-02 3.7858532263997e-02
2.6353512587192e-01 7.9090563394369e-02 2.6501707309831e-02
9.2704830898050e-01 8.1790097093423e-02 1.5144547379038e-02
5.9048094395922e-01 8.4489851786148e-02 3.7870349422972e-03
2.5333978309442e-01 8.7191180753916e-02 9.9242755765436e-01
9.1664283284042e-01 8.9891286057113e-02 9.8106942360389e-01
5.7988190687708e-01 9.2591619310116e-02 9.6971091128285e-01
2.4306338381831e-01 9.5292179240228e-02 9.5835201785368e-01
9.0619401015766e-01 9.7992963330060e-02 9.4699274256414e-01
5.6928090252254e-01 1.0069396820801e-01 9.3563308622056e-01
2.3233156131403e-01 1.0339518997389e-01 9.2427305072853e-01
8.9535389659822e-01 1.0609662446222e-01 9.1291263870345e-01
5.5835626730537e-01 1.0879826744609e-01 9.0155185315240e-01
2.2134753508842e-01 1.1150011478449e-01 8.9019069722827e-01
8.8433713460328e-01 1.1420216251490e-01 8.7882917405854e-01
5.4733516251665e-01 1.1690440689164e-01 8.6746728665269e-01
2.1035248824900e-01 1.1960684436814e-01 8.5610503789601e-01
8.7340089035933e-01 1.2230947151812e-01 8.4474243064290e-01
5.3649322362980e-01 1.2501228488644e-01 8.3337946792968e-01
1.9964362338562e-01 1.2771528075423e-01 8.2201615333803e-01
8.6286775549493e-01 1.3041845479457e-01 8.1065249155440e-01
5.2618312297325e-01 1.3312180158306e-01 7.9928848919185e-01
1.8960944336143e-01 1.3582531391037e-01 7.8792415597033e-01
8.5316911531487e-01 1.3852898181912e-01 7.7655950639326e-01
5.1688779848658e-01 1.4123279125117e-01 7.6519456211893e-01
1.8079513827827e-01 1.4393672213779e-01 7.5382935531081e-01
8.4492567698358e-01 1.4664074568676e-01 7.4246393337396e-01
5.0932000601849e-01 1.4934482050426e-01 7.3109836565772e-01
1.7402623117725e-01 1.5204888701978e-01 7.1973275294563e-01
8.3910184486126e-01 1.5475285943810e-01 7.0836724087512e-01
5.0461612579885e-01 1.5745661410342e-01 6.9700203883123e-01
1.7065321483212e-01 1.6015997272292e-01 6.8563744628411e-01
8.3731603396098e-01 1.6286267842573e-01 6.7427388878526e-01
5.0473119407139e-01 1.6556436239383e-01 6.6291196533253e-01
1.7305489527955e-01 1.6826449960201e-01 6.5155250613725e-01
8.4247937267219e-01 1.7096235616503e-01 6.4019663174783e-01
5.1323826116643e-01 1.7365694288491e-01 6.2884578444956e-01
1.8560655510885e-01 1.7634701971301e-01 6.1750166025465e-01
8.5988562512045e-01 1.7903125729551e-01 6.0616589737702e-01
5.3635668605726e-01 1.8170874820394e-01 5.9483931003159e-01
2.1518572819181e-01 1.8438006258236e-01 5.8352057242682e-01
8.9629228303362e-01 1.8704864236423e-01 5.7220491300379e-01
5.7926334645782e-01 1.8972132395246e-01 5.6088447623813e-01
2.6341466735623e-01 1.9240629461087e-01 5.4955172310780e-01
9.4795595685069e-01 1.9510882907170e-01 5.3820393180680e-01
6.3208759169859e-01 1.9782813668932e-01 5.2684442491015e-01
3.1501907366226e-01 2.0055813700779e-01 5.1547931194068e-01
1.0831227285229e-04 9.7134667514954e-01 5.0373090502441e-01
6.6522190431763e-01 9.7406424863695e-01 4.9236125428868e-01
3.3037770288402e-01 9.7678181549554e-01 4.8099162851081e-01
9.9557391822026e-01 9.7949933405431e-01 4.6962203286576e-01
6.6080834993593e-01 9.8221675854473e-01 4.5825247537947e-01
3.2607836617123e-01 9.8493403877171e-01 4.4688296784087e-01
9.9138089754456e-01 9.8765111998057e-01 4.3551352664555e-01
6.5671244709017e-01 9.9036794295880e-01 4.2414417352045e-01
3.2206911621787e-01 9.9308444440180e-01 4.1277493607623e-01
9.8744664541378e-01 9.9580055755836e-01 4.0140584813470e-01
6.5284046709342e-01 9.9851621315590e-01 3.9003694978401e-01
3.1900934973643e-01 1.2277099804830e-03 3.7867244719045e-01
9.8441651917447e-01 3.9422702704515e-03 3.6730403613073e-01
6.4982491482553e-01 6.6561650944375e-03 3.5593596600328e-01
3.1522936379259e-01 9.3693289831868e-03 3.4456829606529e-01
9.8062468215289e-01 1.2081701599900e-02 3.3320108726564e-01
6.4600571045575e-01 1.4793229456918e-02 3.2183440045860e-01
3.1136733500167e-01 1.7503867522657e-02 3.1046829444920e-01
9.7670449519484e-01 2.0213580646643e-02 2.9910282395461e-01
6.4201217892395e-01 2.2922344737893e-02 2.8773803757515e-01
3.0728540937179e-01 2.5630147641957e-02 2.7637397587271e-01
9.7251922771894e-01 2.8336989674127e-02 2.6501066965358e-01
6.3770867677083e-01 3.1042883780151e-02 2.5364813854607e-01
3.0284879053909e-01 3.3747855311143e-02 2.4228638995143e-01
9.6793459424505e-01 3.6451941415855e-02 2.3092541842897e-01
6.3296111814854e-01 3.9155190070788e-02 2.1956520555437e-01
2.9792342715596e-01 4.1857658786025e-02 2.0820572026409e-01
9.6281666649186e-01 4.4559413041149e-02 1.9684691967058e-01
6.2763612201858e-01 4.7260524519874e-02 1.8548875030504e-01
2.9237729225380e-01 4.9961069222526e-02 1.7413114971847e-01
9.5703596794207e-01 5.2661125541142e-02 1.6277404835133e-01
6.2160831430849e-01 5.5360772381738e-02 1.5141737156783e-01
2.8609095092665e-01 5.8060087412022e-02 1.4006104174620e-01
9.5048102445810e-01 6.0759145501151e-02 1.2870498031912e-01
6.1477627029166e-01 6.3458017402196e-02 1.1734910967036e-01
2.7897506019978e-01 6.6156768709522e-02 1.0599335481136e-01
9.4307643438309e-01 6.8855459104309e-02 9.4637644783285e-02
6.0708011753720e-01 7.1554141883674e-02 8.3281913753385e-02
2.7098651971794e-01 7.4252863753857e-02 7.1926101796978e-02
9.3479672370593e-01 7.6951664856569e-02 6.0570155375588e-02
5.9851246122981e-01 7.9650578990294e-02 4.9214027537320e-02
2.6213608079026e-01 8.2349633984967e-02 3.7857677876059e-02
9.2567050995913e-01 8.5048852188457e-02 2.6501072292141e-02
5.8911921495646e-01 8.7748251025955e-02 1.5144182598983e-02
2.5248616009076e-01 9.0447843597872e-02 3.7869860187975e-03
9.1524525271642e-01 9.3149076889501e-02 9.9242775912908e-01
5.7847768453838e-01 9.5848978467814e-02 9.8107001538054e-01
2.4164265111475e-01 9.8549091555759e-02 9.6971192433782e-01
9.0474577331392e-01 1.0124941782108e-01 9.5835347885235e-01
5.6779303360389e-01 1.0394995736399e-01 9.4699467418101e-01
2.3079077162240e-01 1.0665070913373e-01 9.3563550744604e-01
8.9374568970251e-01 1.0935167128976e-01 9.2427597714324e-01
5.5666486927135e-01 1.1205284150823e-01 9.1291608270275e-01
2.1955579916421e-01 1.1475421723593e-01 9.0155582410458e-01
8.8242641712321e-01 1.1745579589411e-01 8.9019520154956e-01
5.4528516608812e-01 1.2015757503503e-01 8.7883421518594e-01
2.0814106735426e-01 1.2285955245319e-01 8.6747286489257e-01
8.7100381329289e-01 1.2556172625241e-01 8.5611115012087e-01
5.3388388314095e-01 1.2826409486788e-01 8.4474906980044e-01
1.9679268642094e-01 1.3096665704018e-01 8.3338662231775e-01
8.5974273992464e-01 1.3366941173425e-01 8.2202380558318e-01
5.2274788598761e-01 1.3637235799161e-01 8.1066061721109e-01
1.8582356214329e-01 1.3907549469634e-01 7.9929705485117e-01
8.4898713537642e-01 1.4177882022472e-01 7.8793311672875e-01
5.1225831838139e-01 1.4448233193242e-01 7.7656880248109e-01
1.7565969087287e-01 1.4718602540875e-01 7.6520411441967e-01
8.3921735665910e-01 1.4988989339108e-01 7.5383905941348e-01
5.0296177767529e-01 1.5259392417606e-01 7.4247365168544e-01
1.6692884062162e-01 1.5529809927694e-01 7.3110791696105e-01
8.3116123184872e-01 1.5800238994047e-01 7.1974189863090e-01
4.9571022387533e-01 1.6070675192358e-01 7.0837566692361e-01
1.6063801527027e-01 1.6341111759726e-01 6.9700933258576e-01
8.2602081790113e-01 1.6611538392921e-01 6.8564306729403e-01
4.9195295425796e-01 1.6881939411691e-01 6.7427713403357e-01
1.5855231005282e-01 1.7152290952866e-01 6.6291193192527e-01
8.2596756251457e-01 1.7422556722251e-01 6.5154806108298e-01
4.9438759549776e-01 1.7692681724677e-01 6.4018641260161e-01
1.6405318308451e-01 1.7962583549673e-01 6.2882828247451e-01
8.3526972419960e-01 1.8232141929756e-01 6.1747548504380e-01
5.0841594338000e-01 1.8501191313057e-01 6.0613037673121e-01
1.8393390392824e-01 1.8769532053082e-01 5.9479555494348e-01
8.6226781151281e-01 1.9036997187027e-01 5.8347277619625e-01
5.4370575438122e-01 1.9303627807859e-01 5.7216066760312e-01
2.2813037972931e-01 1.9569938815891e-01 5.6085212210606e-01
9.1485654916444e-01 1.9837007072987e-01 5.4953512212358e-01
6.0278480721499e-01 2.0105995867803e-01 5.3819990188463e-01
2.9070274054650e-01 2.0377332815530e-01 5.2684639327007e-01
9.7739783092846e-01 2.0650400012762e-01 5.1548265972729e-01
6.6678886592412e-01 9.7729301445815e-01 5.0374300893330e-01
3.3186218922372e-01 9.8001057477532e-01 4.9237332796750e-01
9.9696827852455e-01 9.8272812431825e-01 4.8100366771610e-01
6.6210548979338e-01 9.8544562313865e-01 4.6963403325423e-01
3.2727185164606e-01 9.8816302798105e-01 4.5826443209167e-01
9.9246504257885e-01 9.9088029194041e-01 4.4689487501633e-01
6.5768238042166e-01 9.9359736428360e-01 4.3552537687587e-01
3.2292082525811e-01 9.9631419046932e-01 4.2415595725551e-01
9.8817699609308e-01 9.9903071239314e-01 4.1278664100801e-01
6.5422741426685e-01 1.7431592116420e-03 4.0142170916222e-01
3.1950400744388e-01 4.4589123413405e-03 3.9005266784403e-01
9.8478634316545e-01 7.1741736612911e-03 3.7868383595486e-01
6.5007006768336e-01 9.8888780387173e-03 3.6731526033671e-01
3.1535071252506e-01 1.2602961461269e-02 3.5594699220802e-01
9.8062374480704e-01 1.5316362491903e-02 3.4457908614050e-01
6.4588461038287e-01 1.8029023783398e-02 3.3321159877340e-01
3.1112876704263e-01 2.0740893589482e-02 3.2184458730532e-01
9.7635170612385e-01 2.3451927208446e-02 3.1047810781800e-01
6.4154896222697e-01 2.6162088297238e-02 2.9911221349909e-01
3.0671611210070e-01 2.8871349999092e-02 2.8774695283915e-01
9.7184876502478e-01 3.1579695835084e-02 2.7638236788379e-01
6.3694254803020e-01 3.4287120319211e-02 2.6501849262271e-01
3.0199308995235e-01 3.6993629267231e-02 2.5365535159403e-01
9.6699600854165e-01 3.9699239781390e-02 2.4229295877497e-01
6.3194690463316e-01 4.2403979906083e-02 2.3093131681792e-01
2.9684136672660e-01 4.5107887963248e-02 2.1957041667488e-01
9.6167498831737e-01 4.7811011590436e-02 2.0821023763394e-01
6.2644339905564e-01 5.0513406518472e-02 1.9685074776950e-01
2.9114230943034e-01 5.3215135138422e-02 1.8549190478484e-01
9.5576756732878e-01 5.5916264918230e-02 1.7413365720314e-01
6.2031522365724e-01 5.8616866736749e-02 1.6277594584365e-01
2.8478160334766e-01 6.1317013206033e-02 1.5141870550401e-01
9.4916337760467e-01 6.4016777051283e-02 1.4006186676063e-01
6.1345763319991e-01 6.6716229611556e-02 1.2870535779635e-01
2.7766193497535e-01 6.9415439514016e-02 1.1734910616921e-01
9.4177437840500e-01 7.2114471561076e-02 1.0599304044621e-01
6.0579362998276e-01 7.4813385854606e-02 9.4637091641506e-02
2.6971895423062e-01 7.7512237166115e-02 8.3281194416184e-02
9.3355022714175e-01 8.0211074547584e-02 7.1925288015882e-02
5.9728793678568e-01 8.2909941165768e-02 6.0569316940210e-02
2.6093317254153e-01 8.5608874333831e-02 4.9213231353685e-02
9.2448760495080e-01 8.8307905708392e-02 3.7856987260023e-02
5.8795345849187e-01 9.1007061617406e-02 2.6500546470176e-02
2.5133347969079e-01 9.3706363484321e-02 1.5143876399364e-02
9.1463090293581e-01 9.6405828316076e-02 3.7869497299401e-03
5.7728728215568e-01 9.9106995293765e-02 9.9242793412025e-01
2.4044608322529e-01 1.0180671998553e-01 9.8107054569821e-01
9.0353436076748e-01 1.0450663585388e-01 9.6971284706977e-01
5.6655698469074e-01 1.0720674748743e-01 9.5835482710771e-01
2.2951917605132e-01 1.0990705760622e-01 9.4699647738110e-01
8.9242649693301e-01 1.1260756748652e-01 9.3563779162992e-01
5.5528484865397e-01 1.1530827734052e-01 9.2427876526953e-01
2.1810047925090e-01 1.1800918664969e-01 9.1291939493149e-01
8.8088000125325e-01 1.2071029445231e-01 9.0155967804390e-01
5.4363042091548e-01 1.2341159958693e-01 8.9019961245287e-01
2.0635918033423e-01 1.2611310089429e-01 8.7883919608467e-01
8.6907421425371e-01 1.2881479738033e-01 8.6747842664779e-01
5.3178402388120e-01 1.3151668834287e-01 8.5611730137359e-01
1.9449777072726e-01 1.3421877346388e-01 8.4475581679535e-01
8.5722539440066e-01 1.3692105286804e-01 8.3339396856680e-01
5.1997775949439e-01 1.3962352714661e-01 8.2203175132416e-01
1.8276683829447e-01 1.4232619734320e-01 8.1066915860005e-01
8.4560593816322e-01 1.4502906489393e-01 7.9930618280449e-01
5.0850998529138e-01 1.4773213150900e-01 7.8794281529865e-01
1.7149588035535e-01 1.5043539897414e-01 7.7657904660313e-01
8.3458294686114e-01 1.5313886883687e-01 7.6521486680701e-01
4.9779350018699e-01 1.5584254192228e-01 7.5385026628304e-01
1.6115357540659e-01 1.5854641759001e-01 7.4248523687420e-01
8.2469386614580e-01 1.6125049259182e-01 7.3111977381356e-01
4.8845094687675e-01 1.6395475930373e-01 7.1975387879256e-01
1.5246888000209e-01 1.6665920296623e-01 7.0838756484037e-01
8.1680135106954e-01 1.6936379733188e-01 6.9702086407843e-01
4.8151453683979e-01 1.7206849772609e-01 6.8565384006694e-01
1.4669100107474e-01 1.7477322986061e-01 6.7428660752069e-01
8.1243504492636e-01 1.7747787161190e-01 6.6291936387232e-01
4.7888012856925e-01 1.8018222309337e-01 6.5155243980503e-01
1.4619923839242e-01 1.8288595732212e-01 6.4018637967801e-01
8.1461936815846e-01 1.8558853939282e-01 6.2882206717297e-01
4.8444141073678e-01 1.8828909755301e-01 6.1746091276194e-01
1.5606591898827e-01 1.9098623241380e-01 6.0610510326789e-01
8.3002071398546e-01 1.9367779004060e-01 5.9475783428812e-01
5.0697063629713e-01 1.9636079109539e-01 5.8342318900567e-01
1.8764843140970e-01 1.9903218577514e-01 5.7210473767579e-01
8.7258579503493e-01 2.0169182330102e-01 5.6080144644534e-01
5.6160094010310e-01 2.0434791782145e-01 5.4950206747072e-01
2.5346711636435e-01 2.0701848494515e-01 5.3818730125127e-01
9.4631210925368e-01 2.0971929326902e-01 5.2684600049675e-01
6.3815675907187e-01 2.1244859730724e-01 5.1548549762208e-01
3.3346655219656e-01 9.8323932678290e-01 5.0375516157191e-01
9.9850903539647e-01 9.8595687747695e-01 4.9238545722387e-01
6.6357501825510e-01 9.8867441349231e-01 4.8101576913947e-01
3.2866298394742e-01 9.9139189633160e-01 4.6964610225682e-01
9.9377116587586e-01 9.9410928492183e-01 4.5827646358044e-01
6.5889752280648e-01 9.9682653525594e-01 4.4690686297154e-01
3.2403972406491e-01 9.9954360017156e-01 4.3553731388037e-01
9.8998830072725e-01 2.2566576208510e-03 4.2417215552516e-01
6.5515128570551e-01 4.9732172083409e-03 4.1280274505691e-01
3.2032126777086e-01 7.6894334959133e-03 4.0143345139722e-01
9.8549483927425e-01 1.0405248172886e-02 3.9006430727556e-01
6.5066839542422e-01 1.3120601544569e-02 3.7869535056811e-01
3.1583817699931e-01 1.5835433490864e-02 3.6732662396995e-01
9.8100031473569e-01 1.8549684582963e-02 3.5595817441204e-01
6.4615087197655e-01 2.1263297318225e-02 3.4459005222651e-01
3.1128588241219e-01 2.3976217425890e-02 3.3322231007699e-01
9.7640138031585e-01 2.6688395191483e-02 3.2185500168399e-01
6.4149342153327e-01 2.9399786745851e-02 3.1048818038755e-01
3.0655809452604e-01 3.2110355265683e-02 2.9912189760001e-01
9.7159152188856e-01 3.4820072035591e-02 2.8775620120992e-01
6.3658985383793e-01 3.7528917327185e-02 2.7639113400344e-01
3.0154925610092e-01 4.0236881057492e-02 2.6502673217179e-01
9.6646589529919e-01 4.2943963197265e-02 2.5366302397239e-01
6.3133592529223e-01 4.5650173908968e-02 2.4230002860652e-01
2.9615547794356e-01 4.8355533404415e-02 2.3093775536853e-01
9.6092066142690e-01 5.1060071522879e-02 2.1957620311030e-01
6.2562756852073e-01 5.3763827041980e-02 2.0821536005025e-01
2.9027229641904e-01 5.6466846745164e-02 1.9685520393921e-01
9.5485097850618e-01 5.9169184280759e-02 1.8549570257737e-01
6.1935982741660e-01 6.1870898857481e-02 1.7413681465743e-01
2.8379518764315e-01 6.4572053829128e-02 1.6277849089151e-01
9.4815359507932e-01 6.7272715226185e-02 1.5142067536445e-01
6.1243184027060e-01 6.9972950293576e-02 1.4006330704460e-01
2.7662703185991e-01 7.2672826091472e-02 1.2870632137737e-01
9.4073665675966e-01 7.5372408210163e-02 1.1734965188614e-01
6.0475863393871e-01 7.8071759640883e-02 1.0599323170946e-01
2.6869135931974e-01 8.0770939833194e-02 9.4636995013476e-02
9.3253374005332e-01 8.3470003957091e-02 8.3280878230683e-02
5.9628521727913e-01 8.6169002375584e-02 7.1924821091583e-02
2.5994577731023e-01 8.8867980322128e-02 6.0568767430751e-02
9.2351595191107e-01 9.1566977767847e-02 4.9212665763506e-02
5.8699680893168e-01 9.4266029456326e-02 3.7856469641813e-02
2.5038993498388e-01 9.6965165079232e-02 2.6500137807850e-02
9.1369741209740e-01 9.9664409563870e-02 1.5143634170564e-02
5.7692179039053e-01 1.0236378344382e-01 3.7869276344796e-03
2.3947277193015e-01 1.0506491651250e-01 9.9242807915401e-01
9.0255519645439e-01 1.0776449476431e-01 9.8107100490904e-01
5.6556446535786e-01 1.1046424047526e-01 9.6971366256058e-01
2.2850471508361e-01 1.1316416109143e-01 9.5835603736722e-01
8.9138041866017e-01 1.1586426192696e-01 9.4699811752575e-01
5.5419637097541e-01 1.1856454658065e-01 9.3563989368494e-01
2.1695768062689e-01 1.2126501731845e-01 9.2428135847189e-01
8.7966976930707e-01 1.2396567541841e-01 9.1292250603932e-01
5.4233837968922e-01 1.2666652147687e-01 9.0156333163928e-01
2.0496959286593e-01 1.2936755567622e-01 8.9020383122558e-01
8.6756985656799e-01 1.3206877801593e-01 8.7884400108568e-01
5.3014602567017e-01 1.3477018850920e-01 8.6748383750060e-01
1.9270541689213e-01 1.3747178734814e-01 8.5612333643055e-01
8.5525588015558e-01 1.4017357504047e-01 8.4476249322320e-01
5.1780588980302e-01 1.4287555252041e-01 8.3340130234119e-01
1.8036465987830e-01 1.4557772123630e-01 8.2203975710598e-01
8.4294228899793e-01 1.4828008321601e-01 8.1067784945585e-01
5.0554994212488e-01 1.5098264111046e-01 7.9931556971837e-01
1.6820007896629e-01 1.5368539821267e-01 7.8795290640056e-01
8.3090674200320e-01 1.5638835844678e-01 7.7658984600648e-01
4.9368592168911e-01 1.5909152631582e-01 7.6522637290135e-01
1.5655602266377e-01 1.6179490678804e-01 7.5386246925810e-01
8.1953846372363e-01 1.6449850508733e-01 7.4249811514883e-01
4.8265845698519e-01 1.6720232632910e-01 7.3113328888987e-01
1.4594603003377e-01 1.6990637490227e-01 7.1976796782555e-01
8.0943738175387e-01 1.7261065342848e-01 7.0840212986787e-01
4.7317670253637e-01 1.7531516100751e-01 6.9703575633620e-01
1.3721864990486e-01 1.7801989024198e-01 6.8566883703862e-01
8.0163176303098e-01 1.8072482214389e-01 6.7430137923907e-01
4.6650324327345e-01 1.8342991730776e-01 6.6293342341007e-01
1.3194575418541e-01 1.8613510039643e-01 6.5156507093324e-01
7.9810725427345e-01 1.8884023245739e-01 6.4019653298822e-01
4.6518544747076e-01 1.9154506080471e-01 6.2882821712626e-01
1.3344932023826e-01 1.9424912732129e-01 6.1746088033864e-01
8.0327146402457e-01 1.9695160069145e-01 6.0609589558416e-01
4.7517593810426e-01 1.9965097749310e-01 5.9473569328595e-01
1.4990406532867e-01 2.0234459944635e-01 5.8338439158952e-01
8.2848045746452e-01 2.0502811473723e-01 5.7204827727138e-01
5.1217634654553e-01 2.0769593931779e-01 5.6073445941014e-01
2.0205460091201e-01 2.1034626412280e-01 5.4944347015985e-01
8.9780689088733e-01 2.1299341129616e-01 5.3815598967758e-01
5.9700526271311e-01 2.1566956125126e-01 5.2683959109467e-01
2.9619252847638e-01 2.1839146584955e-01 5.1548757838660e-01
1.4142391085864e-04 9.8918561906216e-01 5.0376734922472e-01
6.6516232954436e-01 9.9190316341272e-01 4.9239762831066e-01
3.3019759072702e-01 9.9462068932247e-01 4.8102791904631e-01
9.9524580155926e-01 9.9733815950227e-01 4.6965822618773e-01
6.6110945724444e-01 5.1706484086969e-05 4.5829294233064e-01
3.2617694227754e-01 2.7689570619652e-03 4.4692329110125e-01
9.9125185688996e-01 5.4860293974809e-03 4.3555368160931e-01
6.5633196031916e-01 8.2028768662375e-03 4.2418412981292e-01
3.2141478050792e-01 1.0919450243240e-02 4.1281465595567e-01
9.8649762501797e-01 1.3635697933984e-02 4.0144528491400e-01
6.5157760116166e-01 1.6351566394633e-02 3.9007604637102e-01
3.1665164369958e-01 1.9067000737793e-02 3.7870697479328e-01
9.8171654782830e-01 2.1781945510194e-02 3.6733810919358e-01
6.4676900475812e-01 2.4496345618967e-02 3.5596949267102e-01
3.1180563700784e-01 2.7210147374621e-02 3.4460117172942e-01
9.7682303065163e-01 2.9923299612095e-02 3.3323319538566e-01
6.4181776213808e-01 3.2635754846579e-02 3.2186561409073e-01
3.0678641792851e-01 3.5347470418523e-02 3.1049847849600e-01
9.7172560600861e-01 3.8058409582178e-02 2.9913183810684e-01
6.3663195923208e-01 4.0768542494045e-02 2.8776573987274e-01
3.0150213136355e-01 4.3477847061422e-02 2.7640022676848e-01
9.6633278751150e-01 4.6186309616392e-02 2.6503533642367e-01
6.3112059129582e-01 4.8893925386960e-02 2.5367109985838e-01
2.9586219151847e-01 5.1600698744250e-02 2.4230754037989e-01
9.6055421125651e-01 5.4306643212698e-02 2.3094467269058e-01
6.2519324216166e-01 5.7011781238811e-02 2.1958250224908e-01
2.8977584634219e-01 5.9716143723277e-02 2.0822102491623e-01
9.5429856755932e-01 6.2419769330647e-02 1.9686022690455e-01
6.1875795265504e-01 6.5122703600288e-02 1.8550008503581e-01
2.8315058321910e-01 6.7824997891073e-02 1.7414056729555e-01
9.4747311659158e-01 7.0526708199929e-02 1.6278163365862e-01
6.1172233447326e-01 7.3227893900078e-02 1.5142323714538e-01
2.7589519676022e-01 7.5928616448058e-02 1.4006532505732e-01
9.3998889779327e-01 7.8628938108974e-02 1.2870784033234e-01
6.0400092204951e-01 8.1328920746728e-02 1.1735072295629e-01
2.6792909640706e-01 8.4028624720473e-02 1.0599391136789e-01
9.3177163645610e-01 8.6728107920629e-02 9.4637343798832e-02
5.9552718485831e-01 8.9427424968313e-02 8.3280959499291e-02
2.5919484040503e-01 9.2126626591757e-02 7.1924699809686e-02
9.2277417711827e-01 9.4825759183245e-02 6.0568509052235e-02
5.8626525341008e-01 9.7524864530934e-02 4.9212335228055e-02
2.4966861190875e-01 1.0022397971232e-01 3.7856130517449e-02
9.1298527103709e-01 1.0292313713049e-01 2.6499851591068e-02
5.7621670977016e-01 1.0562236467055e-01 1.5143459747557e-02
2.3936484720486e-01 1.0832168595208e-01 3.7869208988753e-03
9.0180804354966e-01 1.1102281973402e-01 9.9242819154326e-01
5.6481162157476e-01 1.1372228479938e-01 9.8107138532815e-01
2.2773982666259e-01 1.1642189075675e-01 9.6971435701535e-01
8.9059608921254e-01 1.1912164780077e-01 9.5835708863719e-01
5.5338415777272e-01 1.2182156378088e-01 9.4699956534187e-01
2.1610808082767e-01 1.2452164459549e-01 9.3564177496589e-01
8.7877219337011e-01 1.2722189456332e-01 9.2428370760335e-01
5.4138110919607e-01 1.2992231676637e-01 9.1292535518237e-01
2.0393971982509e-01 1.3262291336150e-01 9.0156671105591e-01
8.6645320097031e-01 1.3532368585926e-01 8.9020776961124e-01
5.2892702757954e-01 1.3802463537042e-01 8.7884852589998e-01
1.9136699864749e-01 1.4072576282161e-01 8.6748897528851e-01
8.5377927327970e-01 1.4342706914237e-01 8.5612911312660e-01
5.1617041988946e-01 1.4612855542658e-01 8.4476893443099e-01
1.7854748096488e-01 1.4883022307161e-01 8.3340843357880e-01
8.4091805659721e-01 1.5153207389851e-01 8.2204760400525e-01
5.0329041098115e-01 1.5423411025699e-01 8.1068643789871e-01
1.6567360747738e-01 1.5693633511834e-01 7.9932492588569e-01
8.2807767970593e-01 1.5963875215951e-01 7.8796305669779e-01
4.9051384871692e-01 1.6234136584058e-01 7.7660081681238e-01
1.5299479986347e-01 1.6504418147690e-01 7.6523819005973e-01
8.1553503802278e-01 1.6774720530493e-01 7.5387515719117e-01
4.7815134695115e-01 1.7045044453715e-01 7.4251169540802e-01
1.4086338884949e-01 1.7315390739460e-01 7.3114777786137e-01
8.0369449526546e-01 1.7585760309344e-01 7.1978337315350e-01
4.6667272281837e-01 1.7856154173948e-01 7.0841844491167e-01
1.2983228100908e-01 1.8126573404285e-01 6.9705295158194e-01
7.9321549132787e-01 1.8397019068649e-01 6.8568684673871e-01
4.5687551832558e-01 1.8667492103095e-01 6.7432008049229e-01
1.2088024368738e-01 1.8937993054171e-01 6.6295260313833e-01
7.8531786767567e-01 1.9208521572946e-01 6.5158437331060e-01
4.5030517960976e-01 1.9479075416590e-01 6.4021537516266e-01
1.1600005210117e-01 1.9749648453410e-01 6.2884565377541e-01
7.8262079067573e-01 2.0020226600372e-01 6.1747538778508e-01
4.5047689872535e-01 2.0290779359879e-01 6.0610503893682e-01
1.2001928352059e-01 2.0561241791174e-01 5.9473566137951e-01
7.9192386322727e-01 2.0831475623458e-01 5.8336953612269e-01
4.6723020002009e-01 2.1101187215647e-01 5.7201140590947e-01
1.4754921396259e-01 2.1369777292697e-01 5.6067041902105e-01
8.3521751281352e-01 2.1636225764066e-01 5.4936057374859e-01
5.3260063211817e-01 2.1899869307972e-01 5.3808791203815e-01
2.3909046908127e-01 2.2163550887380e-01 5.2681521092852e-01
9.4916260272302e-01 2.2433285947783e-01 5.1548765869120e-01
6.6681351688360e-01 9.9513189817406e-01 5.0377955823900e-01
3.3182199320694e-01 9.9784943927480e-01 4.9240982755524e-01
9.9764445297631e-01 5.6310800096683e-04 4.8104451499036e-01
6.6266144665165e-01 3.2805741907412e-03 4.6967479624360e-01
3.2768085642979e-01 5.9979464736404e-03 4.5830509258678e-01
9.9270113079424e-01 8.7151856789056e-03 4.4693541214751e-01
6.5772055896803e-01 1.1432250693646e-02 4.3556576588587e-01
3.2273725570932e-01 1.4149098281331e-02 4.2419616818075e-01
9.8774915498447e-01 1.6865683046436e-02 4.1282663730995e-01
6.5275401268260e-01 1.9581957560510e-02 4.0145719579910e-01
3.1774941783422e-01 2.2297872657120e-02 3.9008787061683e-01
9.8273281112770e-01 2.5013377894628e-02 3.7871869319670e-01
6.4770150894406e-01 2.7728422177019e-02 3.6734969927134e-01
3.1265273071319e-01 3.0442954514671e-02 3.5598092851102e-01
9.7758362717686e-01 3.3156924899460e-02 3.4461242396616e-01
6.4249130715562e-01 3.5870285262505e-02 3.3324423132226e-01
3.0737286065828e-01 3.8582990478456e-02 3.2187639798416e-01
9.7222537662525e-01 4.1294999377700e-02 3.1050897201525e-01
6.3704595421573e-01 4.4006275727249e-02 2.9914200096510e-01
3.0183170727186e-01 4.6716789142171e-02 2.8777553062517e-01
9.6657976235122e-01 4.9426515892088e-02 2.7640960375750e-01
6.3128725143550e-01 5.2135439571148e-02 2.6504425884419e-01
2.9595130103322e-01 5.4843551604747e-02 2.5367952890654e-01
9.6056901983778e-01 5.7550851571976e-02 2.4231544044188e-01
6.2513748734053e-01 6.0257347329093e-02 2.3095201252254e-01
2.8965374580929e-01 6.2963054926219e-02 2.1958925609633e-01
9.5411479782576e-01 6.5667998316852e-02 2.0822717352013e-01
6.1851761115025e-01 6.8372208867504e-02 1.9686575834872e-01
2.8285913208764e-01 7.1075724682631e-02 1.8550499538991e-01
9.4713630782034e-01 7.3778589767649e-02 1.7414486102452e-01
6.1134611741846e-01 7.6480853059820e-02 1.6278532377737e-01
2.7548561050567e-01 7.9182567362572e-02 1.5142634511294e-01
9.3955195192188e-01 8.1883788222938e-02 1.4006788041865e-01
6.0354247023812e-01 8.4584572793780e-02 1.2870988012977e-01
2.6745470768903e-01 8.7284978722084e-02 1.1735229094478e-01
9.3128646901223e-01 8.9985063101731e-02 1.0599505707712e-01
5.9503586681819e-01 9.2684881524199e-02 9.4638121491045e-02
2.5870136143303e-01 9.5384487253724e-02 8.3281427073659e-02
9.2228179361478e-01 9.8083930545545e-02 7.1924917702455e-02
5.8577640908750e-01 1.0078325811737e-01 6.0568539177061e-02
2.4918487440932e-01 1.0348251277598e-01 4.9212239993961e-02
9.1250728423744e-01 1.0618173319344e-01 3.7855971953325e-02
5.7574416053350e-01 1.0888095382145e-01 2.6499690596595e-02
2.3889644463779e-01 1.1158020492759e-01 1.5143355481657e-02
9.0196548341730e-01 1.1427951273440e-01 3.7869303088860e-03
5.6429881185882e-01 1.1698068326992e-01 9.9242826937727e-01
2.2722135285421e-01 1.1968007041450e-01 9.8107168124011e-01
8.9006670118313e-01 1.2237956957820e-01 9.6971491982429e-01
5.5283761743326e-01 1.2507919355013e-01 9.5835796433956e-01
2.1553715841760e-01 1.2777895261981e-01 9.4700079721409e-01
8.7816865657706e-01 1.3047885493722e-01 9.3564340373991e-01
5.4073570241094e-01 1.3317890686114e-01 9.2428577170136e-01
2.0324213082846e-01 1.3587911328854e-01 9.1292789099820e-01
8.6569201224263e-01 1.3857947796023e-01 9.0156975327248e-01
5.2808964919897e-01 1.4128000373969e-01 8.9021135154537e-01
1.9043957935515e-01 1.4398069286413e-01 8.7885267986755e-01
8.5274658571388e-01 1.4668154716769e-01 8.6749373298480e-01
5.1501571517222e-01 1.4938256827821e-01 8.5613450601828e-01
1.7725230670125e-01 1.5208375778964e-01 8.4477499415701e-01
8.3946203083311e-01 1.5478511741286e-01 8.3341519235888e-01
5.0165094263897e-01 1.5748664910819e-01 8.2205509505436e-01
1.6382555107678e-01 1.6018835520337e-01 8.1069469584587e-01
8.2599290853887e-01 1.6289023850112e-01 7.9933398719387e-01
4.8816072573224e-01 1.6559230238068e-01 7.8797296007872e-01
1.5033751882285e-01 1.6829455089841e-01 7.7661160362474e-01
8.1253279828095e-01 1.7099698889276e-01 7.6524990467016e-01
4.7475731239759e-01 1.7369962209964e-01 7.5388784726239e-01
1.3702336349004e-01 1.7640245728503e-01 7.4252541205343e-01
7.9934522213065e-01 1.7910550240211e-01 7.3116257556427e-01
4.6173967550855e-01 1.8180876678062e-01 7.1979930927990e-01
1.2422676217231e-01 1.8451226135557e-01 7.0843557852972e-01
7.8683077002536e-01 1.8721599893910e-01 6.9707134110249e-01
4.4958161277503e-01 1.8991999453000e-01 6.8570654554774e-01
1.1251676101870e-01 1.9262426563256e-01 6.7434112913937e-01
7.7568400352240e-01 1.9532883250137e-01 6.6297501555588e-01
4.3914548049009e-01 1.9803371810273e-01 6.5160811254249e-01
1.0298371690946e-01 2.0073894728723e-01 6.4024031035175e-01
7.6731089311913e-01 2.0344454396585e-01 6.2887148306983e-01
4.3228352756170e-01 2.0615052336061e-01 6.1750149819402e-01
9.8126543324384e-02 2.0885687201677e-01 6.0613024809140e-01
7.6517428239934e-01 2.1156349656338e-01 5.9475773857997e-01
4.3394354622382e-01 2.1427008934897e-01 5.8338432830868e-01
1.0527013203119e-01 2.1697576230676e-01 5.7201137453638e-01
7.8057657886399e-01 2.1967801035579e-01 5.6064298015996e-01
4.6240778637644e-01 2.2236979594651e-01 5.4929047920796e-01
1.5533355150902e-01 2.2503322668298e-01 5.3798032384526e-01
8.6548149908482e-01 2.2764686168545e-01 5.2673767625433e-01
5.9125923056138e-01 2.3027944398501e-01 5.1547810602989e-01
3.3429247687117e-01 1.0743159008155e-03 5.0379619171412e-01
9.9929779940487e-01 3.7918564927807e-03 4.9242645836913e-01
6.6429921564848e-01 6.5093732377620e-03 4.8105672481905e-01
3.2929550003143e-01 9.2268306241439e-03 4.6968699529314e-01
9.9428538936041e-01 1.1944192540432e-02 4.5831727518975e-01
6.5926754988264e-01 1.4661421843428e-02 4.4694757178553e-01
3.2424054956871e-01 1.7378480007708e-02 4.3557789488618e-01
9.8920283714851e-01 2.0095326882889e-02 4.2420825739753e-01
6.5415272890623e-01 2.2811920580065e-02 4.1283867579628e-01
3.1908840363638e-01 2.5528217502653e-02 4.0146917048027e-01
9.8400790553113e-01 2.8244172529882e-02 3.9009976597988e-01
6.4890915415983e-01 3.0959739353717e-02 3.7873049101482e-01
3.1378996016771e-01 3.3674870962357e-02 3.6736137838454e-01
9.7864804491101e-01 3.6389520256187e-02 3.5599246468524e-01
6.4348106199909e-01 3.9103640775444e-02 3.4462378985293e-01
3.0828661865515e-01 4.1817187513423e-02 3.3325539653849e-01
9.7306229494247e-01 4.4530117784864e-02 3.2188732932778e-01
6.3780565922062e-01 4.7242392116683e-02 3.1051963382711e-01
3.0251427866466e-01 4.9953975127146e-02 2.9915235564051e-01
9.6718572425540e-01 5.2664836360112e-02 2.8778553927143e-01
6.3181757027368e-01 5.5374951042738e-02 2.7641922698531e-01
2.9640738894932e-01 5.8084300737992e-02 2.6505345767318e-01
9.6095274146743e-01 6.0792873867012e-02 2.5368826575757e-01
6.2545116697259e-01 6.3500666080916e-02 2.4232368018196e-01
2.8990017149544e-01 6.6207680466654e-02 2.3095972352330e-01
9.5429721883358e-01 6.8913927577046e-02 2.1959641126325e-01
6.1863972533907e-01 7.1619425281096e-02 2.0823375124865e-01
2.8292506030673e-01 7.4324198436974e-02 1.9687174336448e-01
9.4715055324278e-01 7.7028278396506e-02 1.8551037943427e-01
6.1131350875946e-01 7.9731702356547e-02 1.7414964335308e-01
2.7541122923586e-01 8.2434512578736e-02 1.6278951144778e-01
9.3944104476250e-01 8.5136755504630e-02 1.5142995304916e-01
6.0340034930437e-01 8.7838480797591e-02 1.4007093125028e-01
2.6728664152686e-01 9.0539740345677e-02 1.2871240381705e-01
9.3109756837390e-01 9.3240587260844e-02 1.1735432421082e-01
5.9483096929559e-01 9.5941074908940e-02 1.0599664267826e-01
2.5848491900517e-01 9.8641256002103e-02 9.4639307363286e-02
9.2205776679371e-01 1.0134118178068e-01 8.3282265397091e-02
5.8554817072073e-01 1.0404090130586e-01 7.1925463926730e-02
2.4895512539414e-01 1.0674046087760e-01 6.0568851049156e-02
9.1227798250824e-01 1.0943990358526e-01 4.9212376625319e-02
5.7551646378005e-01 1.1213926899181e-01 3.7855992957484e-02
2.3867066637045e-01 1.1483859294643e-01 2.6499655321347e-02
9.0174106126667e-01 1.1753790751515e-01 1.5143322352376e-02
5.6472848541306e-01 1.2023724101603e-01 3.7869562924518e-03
2.2695016996620e-01 1.2293848501491e-01 9.9242831148653e-01
8.8978976124236e-01 1.2563783075510e-01 9.8107188883765e-01
5.5255071549368e-01 1.2833725787339e-01 9.6971534350844e-01
2.1523516716534e-01 1.3103678155672e-01 9.5835865229770e-01
8.7784552202505e-01 1.3373641440167e-01 9.4700179524460e-01
5.4038443521436e-01 1.3643616673154e-01 9.3564475535599e-01
2.0285479067815e-01 1.3913604691224e-01 9.2428751830494e-01
8.6525968279342e-01 1.4183606165859e-01 9.1293007211318e-01
5.2760240092579e-01 1.4453621632463e-01 9.0157240682777e-01
1.8988641757263e-01 1.4723651517354e-01 8.9021451419985e-01
8.5211538071552e-01 1.4993696162437e-01 8.7885638737121e-01
5.1429311100874e-01 1.5263755847446e-01 8.6749802057209e-01
1.7642360448419e-01 1.5533830809737e-01 8.5613940883186e-01
8.3851104156423e-01 1.5803921261725e-01 8.4478054770234e-01
5.0055980335742e-01 1.6074027406153e-01 8.3342143299189e-01
1.6257449648329e-01 1.6344149449409e-01 8.2206206050694e-01
8.2455998805888e-01 1.6614287613195e-01 8.1070242579607e-01
4.8652145301743e-01 1.6884442144877e-01 7.9934252388981e-01
1.4846443667485e-01 1.7154613326906e-01 7.8798234902794e-01
8.1039493649302e-01 1.7424801485744e-01 7.7662189436311e-01
4.7231950843390e-01 1.7695007000817e-01 7.6526115162691e-01
1.3424540533774e-01 1.7965230314073e-01 7.5390011074019e-01
7.9618075767715e-01 1.8235471940917e-01 7.4253875934362e-01
4.5813481127401e-01 1.8505732483429e-01 7.3117708221633e-01
1.2011824281467e-01 1.8776012647082e-01 7.1981506053870e-01
7.8214358338206e-01 1.9046313262587e-01 7.0845267093845e-01
4.4422579458885e-01 1.9316635315086e-01 6.9708988423400e-01
1.0638306436336e-01 1.9586979983778e-01 6.8572666375231e-01
7.6863792540647e-01 1.9857348696385e-01 6.7436296304268e-01
4.3101885842491e-01 2.0127743204689e-01 6.6299872272552e-01
9.3562642145878e-02 2.0398165689975e-01 6.5163386609029e-01
7.5631788655221e-01 2.0668618910493e-01 6.4026829287530e-01
4.1935050162242e-01 2.0939106406130e-01 6.2890187041045e-01
8.2752449853440e-02 2.1209632774099e-01 6.1753442101757e-01
7.4665630989303e-01 2.1480204007756e-01 6.0616570447315e-01
4.1126064197120e-01 2.1750827790282e-01 5.9479539546713e-01
7.6876630546593e-02 2.2021513237070e-01 5.8342306246631e-01
7.4401964022090e-01 2.2292268004134e-01 5.7204818316329e-01
4.1360382430402e-01 2.2563084017957e-01 5.6067035682335e-01
8.7398302056061e-02 2.2833871756606e-01 5.4929044838466e-01
7.6922962401567e-01 2.3104136647969e-01 5.3791622088528e-01
4.6846496441861e-01 2.3371328636522e-01 5.2658815311400e-01
2.0517976129137e-01 2.3628357796638e-01 5.1540208069110e-01
9.5839324395291e-04 7.0205910865576e-03 5.0380840034265e-01
6.6597008482599e-01 9.7381325917185e-03 4.9243867292552e-01
3.3096879935987e-01 1.2455646317508e-02 4.8106894046004e-01
9.9595338372309e-01 1.5173097354286e-02 4.6969920681767e-01
6.6092271505689e-01 1.7890450817219e-02 4.5832947678694e-01
3.2587566424434e-01 2.0607671378816e-02 4.4695975677946e-01
9.9081106336808e-01 2.3324722869767e-02 4.3559005547333e-01
6.5572767882500e-01 2.6041567974360e-02 4.2422038437612e-01
3.2062419128344e-01 2.8758168041688e-02 4.1285075829039e-01
9.8549918310754e-01 3.1474483027977e-02 4.0148119566572e-01
6.5035113328553e-01 3.4190471578912e-02 3.9011171882275e-01
3.1517841932999e-01 3.6906091254088e-02 3.7874235403704e-01
9.7997932511070e-01 3.9621298888914e-02 3.6737313147334e-01
6.4475205317768e-01 4.2336051082841e-02 3.5600408496508e-01
3.0949473986373e-01 4.5050304796922e-02 3.4463525163835e-01
9.7420547134603e-01 4.7764018038801e-02 3.3326667138511e-01
6.3888229890148e-01 5.0477150609415e-02 3.2189838619590e-01
3.0352325180426e-01 5.3189664883210e-02 3.1053043936829e-01
9.6812634666476e-01 5.5901526592441e-02 2.9916287461240e-01
6.3268959246173e-01 5.8612705586213e-02 2.8779573507989e-01
2.9721099103338e-01 6.1323176536104e-02 2.7642906234675e-01
9.6168853331899e-01 6.4032919562359e-02 2.6506289538303e-01
6.2612019213473e-01 6.6741920757578e-02 2.5369726954452e-01
2.9050391268301e-01 6.9450172588390e-02 2.4233221562173e-01
9.5483760229778e-01 7.2157674159619e-02 2.3096775898079e-01
6.1911912109560e-01 7.4864431329944e-02 2.1960391882813e-01
2.8334627521863e-01 7.7570456672849e-02 2.0824070762752e-01
9.4751681422302e-01 8.0275769281794e-02 1.9687813069272e-01
6.1162843389301e-01 8.2980394423871e-02 1.8551618597231e-01
2.7567878537380e-01 8.5684363051654e-02 1.7415486403627e-01
9.3966549104590e-01 8.8387711188291e-02 1.6279414826502e-01
6.0358616705176e-01 9.1090479205827e-02 1.5143401523330e-01
2.6743845187665e-01 9.3792711021044e-02 1.4007443527262e-01
9.3122003992433e-01 9.6494453236351e-02 1.2871537318799e-01
5.9492871865299e-01 9.9195754255207e-02 1.1735678909828e-01
2.5856240758052e-01 1.0189666340202e-01 1.0599863936466e-01
9.2211919734705e-01 1.0459723007529e-01 9.4640877568918e-02
5.8559738704319e-01 1.0729750296002e-01 8.3283455503028e-02
2.4899551816310e-01 1.0999752932151e-01 7.1926324133378e-02
9.1231240380178e-01 1.1269735439735e-01 6.0569434506965e-02
5.7554715205425e-01 1.1539702089914e-01 4.9212738572384e-02
2.3869918295544e-01 1.1809656862924e-01 3.7856189895073e-02
9.0176823868806e-01 1.2079603421274e-01 2.6499744253375e-02
5.6475438715012e-01 1.2349545093964e-01 1.5143360108928e-02
2.2765801928967e-01 1.2619484870837e-01 3.7869989513528e-03
8.8976658673400e-01 1.2889620287078e-01 9.9242831738730e-01
5.5252158698398e-01 1.3159554426715e-01 9.8107200610421e-01
2.1519685486382e-01 1.3429493515077e-01 9.6971562356581e-01
8.7779394824610e-01 1.3699439287148e-01 9.5835914457269e-01
5.4031466903327e-01 1.3969393216287e-01 9.4700254710641e-01
2.0276104078480e-01 1.4239356541018e-01 9.3564581214238e-01
8.6513528623578e-01 1.4509330292679e-01 9.2428892342473e-01
5.2743980543490e-01 1.4779315322995e-01 9.1293186721572e-01
1.8967715512721e-01 1.5049312330837e-01 9.0157463203099e-01
8.5185002991090e-01 1.5319321887596e-01 8.9021720836984e-01
5.1396124561497e-01 1.5589344460756e-01 8.7885958844765e-01
1.7601372528318e-01 1.5859380435423e-01 8.6750176593621e-01
8.3801048811231e-01 1.6129430133653e-01 8.5614373571528e-01
4.9995464168557e-01 1.6399493831556e-01 8.4478549363780e-01
1.6184937786911e-01 1.6669571774222e-01 8.3342703630939e-01
8.2369797280620e-01 1.6939664188587e-01 8.2206836088157e-01
4.8550379155948e-01 1.7209771294398e-01 8.1070946485720e-01
1.4727029812759e-01 1.7479893313502e-01 7.9935034590528e-01
8.0900107181760e-01 1.7750030477684e-01 7.8799100168135e-01
4.7069983131625e-01 1.8020183035344e-01 7.7663142964843e-01
1.3237046831275e-01 1.8290351257314e-01 7.6527162689212e-01
7.9401709324736e-01 1.8560535442154e-01 7.5391158992128e-01
4.5564409679056e-01 1.8830735921345e-01 7.4255131444369e-01
1.1725623214909e-01 1.9100953064853e-01 7.3119079510184e-01
7.7885872548816e-01 1.9371187287663e-01 7.1983002514911e-01
4.4045742503957e-01 1.9641439058089e-01 7.0846899603856e-01
1.0205900447141e-01 1.9911708908937e-01 6.9710769688407e-01
7.6367124389797e-01 2.0181997453059e-01 6.8574611373526e-01
4.2530342436458e-01 2.0452305405612e-01 6.7438422857682e-01
8.6966892054804e-02 2.0722633616507e-01 6.6302201791446e-01
7.4867588291708e-01 2.0992983118635e-01 6.5165945072686e-01
4.1044875850075e-01 2.1263355200995e-01 6.4029648542144e-01
7.2309912630320e-02 2.1533751522259e-01 6.2893306517765e-01
7.3429281435741e-01 2.1804174292230e-01 6.1756911058919e-01
3.9644506178655e-01 2.2074626571847e-01 6.0620450759907e-01
5.8837184101003e-02 2.2345112789917e-01 5.9483908684286e-01
7.2157888080766e-01 2.2615639677495e-01 5.8347258644303e-01
3.8485119753886e-01 2.2886218056929e-01 5.7210458086618e-01
4.8976333334430e-02 2.3156866495892e-01 5.6073433503706e-01
7.1458833214165e-01 2.3427619246803e-01 5.4936048128983e-01
3.8312557250874e-01 2.3698543401911e-01 5.3798026276349e-01
5.8647552495776e-02 2.3969750629369e-01 5.2658812285683e-01
6.6762100610820e-01 1.2966875136255e-02 5.0382058761419e-01
3.3264816653416e-01 1.5684420557021e-02 4.9245087293712e-01
9.9765320741997e-01 1.8401934003558e-02 4.8108114837833e-01
6.6263503801837e-01 2.1119380991691e-02 4.6971141738035e-01
3.2759266621071e-01 2.3836727687936e-02 4.5834168406543e-01
9.9252515777783e-01 2.6553940399066e-02 4.4697195394962e-01
6.5743159853251e-01 2.9270985119047e-02 4.3560223458838e-01
3.2231106117699e-01 3.1987827160085e-02 4.2423253613862e-01
9.8716257827027e-01 3.4704430889266e-02 4.1286287182325e-01
6.5198512214784e-01 3.7420759586412e-02 4.0149325828594e-01
3.1677759207019e-01 4.0136775432557e-02 3.9012371582543e-01
9.8153880832852e-01 4.2852439632159e-02 3.7875426850004e-01
6.4626751254399e-01 4.5567712665958e-02 3.6738494409622e-01
3.1096237299436e-01 4.8282554665522e-02 3.5601577395710e-01
9.7562199351812e-01 5.0996925895315e-02 3.4464679267122e-01
6.4024492439715e-01 5.3710787323669e-02 3.3327803762538e-01
3.0482967361432e-01 5.6424101260549e-02 3.2190954843025e-01
9.6937471701682e-01 5.9136832037646e-02 3.1054136623171e-01
6.3387850617340e-01 6.1848946705040e-02 2.9917353292542e-01
2.9833947306995e-01 6.4560415718455e-02 2.8780609029632e-01
9.6275603120675e-01 6.7271213591931e-02 2.7643907910797e-01
6.2712657310777e-01 6.9981319492301e-02 2.6507253816940e-01
2.9144946468564e-01 7.2690717754197e-02 2.5370650340908e-01
9.5572303729113e-01 7.5399398297118e-02 2.4234100698607e-01
6.1994557858174e-01 7.8107356929380e-02 2.3097607646850e-01
2.8411532354455e-01 8.0814595527397e-02 2.1961173410769e-01
9.4823044709142e-01 8.3521122082673e-02 2.0824799623395e-01
6.1228905960242e-01 8.6226950613046e-02 1.9688487279639e-01
2.7628920663082e-01 8.8932100939181e-02 1.8552236706397e-01
9.4022887371427e-01 9.1636598331781e-02 1.7416047549962e-01
6.0410599688382e-01 9.4340473039570e-02 1.6279918781244e-01
2.6791847905439e-01 9.7043759712474e-02 1.5143848718586e-01
9.3166421205198e-01 9.9746496738357e-02 1.4007835067259e-01
5.9534110361882e-01 1.0244872551503e-01 1.2871874973999e-01
2.5894710837464e-01 1.0515048968160e-01 1.1735965094332e-01
9.2248026142719e-01 1.0785183433454e-01 1.0600101669937e-01
5.8593871314223e-01 1.1055280525374e-01 9.4642806129196e-02
2.4932076351292e-01 1.1325344816274e-01 8.3284975937221e-02
9.1262489461160e-01 1.1595380804442e-01 7.1927481293999e-02
5.7584979975325e-01 1.1865392853029e-01 6.0570276691935e-02
2.3899440823007e-01 1.2135385137677e-01 4.9213316746888e-02
9.0205790476499e-01 1.2405361603740e-01 3.7856556923365e-02
5.6503974314968e-01 1.2675325933504e-01 2.6499954166625e-02
2.2793965385289e-01 1.2945281523331e-01 1.5143467430933e-02
8.9075764568105e-01 1.3215231470302e-01 3.7870580990203e-03
5.5275191621939e-01 1.3485381512334e-01 9.9242828721148e-01
2.1542094466063e-01 1.3755318914540e-01 9.8107203265289e-01
8.7800946486523e-01 1.4025258000214e-01 9.6971575823824e-01
5.4051851199785e-01 1.4295200693914e-01 9.5835943717738e-01
2.0294933620448e-01 1.4565148662250e-01 9.4700304572380e-01
8.6530338060689e-01 1.4835103335422e-01 9.3564656307267e-01
5.2758225790615e-01 1.5105065930455e-01 9.2428997121355e-01
1.8978772621313e-01 1.5375037475185e-01 9.1293325475026e-01
8.5192166462256e-01 1.5645018832174e-01 9.0157640069916e-01
5.1398604893660e-01 1.5915010721910e-01 8.9021939827580e-01
1.7598292783167e-01 1.6185013744752e-01 8.7886223867844e-01
8.3791439965127e-01 1.6455028401283e-01 8.6750491487531e-01
4.9978258990227e-01 1.6725055110771e-01 8.5614742140128e-01
1.6158962943051e-01 1.6995094227615e-01 8.4478975416801e-01
8.2333763315490e-01 1.7265146055707e-01 8.3343191029078e-01
4.8502867914191e-01 1.7535210860685e-01 8.2207388793425e-01
1.4666478770205e-01 1.7805288880156e-01 8.1071568617860e-01
8.0824790007701e-01 1.8075380331950e-01 7.9935730490713e-01
4.6977985615319e-01 1.8345485420527e-01 7.8799874471608e-01
1.3126237046864e-01 1.8615604341638e-01 7.7664000684758e-01
7.9269700555828e-01 1.8885737285356e-01 7.6528109314671e-01
4.5408514137755e-01 1.9155884437569e-01 7.5392200604446e-01
1.1542793911630e-01 1.9426045980004e-01 7.4256274856920e-01
7.7672629710019e-01 1.9696222088811e-01 7.3120332439084e-01
4.3798079557731e-01 1.9966412931654e-01 7.1984373790435e-01
9.9191625849772e-02 2.0236618663208e-01 7.0848399436245e-01
7.6035849718010e-01 2.0506839418783e-01 6.9712410007265e-01
4.2148051176249e-01 2.0777075305637e-01 6.8576406268186e-01
8.2555993083401e-02 2.1047326391226e-01 6.7440389158422e-01
7.4358224494668e-01 2.1317592687160e-01 6.6304359850808e-01
4.0455520502609e-01 2.1587874126856e-01 6.5168319837166e-01
6.5468933749685e-02 2.1858170533545e-01 6.4032271055418e-01
7.2631483824634e-01 2.2128481572946e-01 6.2896216083139e-01
3.8708045488173e-01 2.2398806680592e-01 6.1760158441202e-01
4.7747465707455e-02 2.2669144945474e-01 6.0624103087584e-01
7.0828831954225e-01 2.2939494914731e-01 5.9488057255626e-01
3.6866015977789e-01 2.3209854247366e-01 5.8352031952672e-01
2.8793171449216e-02 2.3480219059065e-01 5.7216044814780e-01
6.8856628752548e-01 2.3750582579186e-01 5.6080125994151e-01
3.4775075708343e-01 2.4020932101366e-01 5.4944331609909e-01
3.3428028625668e-01 1.8913174816182e-02 5.0383273987602e-01
9.9933209629696e-01 2.1630727278805e-02 4.9246304478197e-01
6.6435252306184e-01 2.4348243201139e-02 4.8109333502599e-01
3.2934052923937e-01 2.7065688354373e-02 4.6972361354435e-01
9.9429524612616e-01 2.9783029798221e-02 4.5835388372569e-01
6.5921592793760e-01 3.2500235318881e-02 4.4698415014471e-01
3.2410190795434e-01 3.5217272915182e-02 4.3561441921968e-01
9.8895255860571e-01 3.7934110360850e-02 4.2424469978054e-01
6.5376725710319e-01 4.0650714865110e-02 4.1287500353692e-01
3.1854535768966e-01 4.3367052847722e-02 4.0150534543722e-01
9.8328617100370e-01 4.6083089838358e-02 3.9013574391114e-01
6.4798895051306e-01 4.8798790504130e-02 3.7876622099032e-01
3.1265288547999e-01 5.1514118803226e-02 3.6739680230274e-01
9.7727709951546e-01 5.4229038257251e-02 3.5602751693950e-01
6.4186065347865e-01 5.6943512330105e-02 3.4465839719461e-01
3.0640255130040e-01 5.9657504897201e-02 3.3328947818203e-01
9.7090174725835e-01 6.2370980785702e-02 3.2192079733711e-01
6.3535715330533e-01 6.5083906364206e-02 3.1055239381353e-01
2.9976764523891e-01 6.7796250159065e-02 2.9918430779013e-01
9.6413206677767e-01 7.0507983474180e-02 2.8781657970548e-01
6.2844923095356e-01 7.3219080991574e-02 2.7644924944090e-01
2.9271791860721e-01 7.5929521331308e-02 2.6508235547514e-01
9.5693687415366e-01 7.8639287551070e-02 2.5371593403533e-01
6.2110479913846e-01 8.1348367568121e-02 2.4235001827025e-01
2.8522034440033e-01 8.4056754488916e-02 2.3098463747151e-01
9.4928210187535e-01 8.6764446834727e-02 2.1961981636789e-01
6.1328859720257e-01 8.9471448654808e-02 2.0825557451620e-01
2.7723828431463e-01 9.2177769522095e-02 1.9689192580928e-01
9.4112954311795e-01 9.4883424410069e-02 1.8552887811851e-01
6.0496068119289e-01 9.7588433453187e-02 1.7416643308353e-01
2.6872994018982e-01 1.0029282159711e-01 1.6280458605683e-01
9.3243550728183e-01 1.0299661814879e-01 1.5144332620518e-01
5.9607553168495e-01 1.0569985623990e-01 1.4008263676372e-01
2.5964814590001e-01 1.0840257222056e-01 1.2872249543270e-01
9.2315149099637e-01 1.1110480500236e-01 1.1736287490097e-01
5.8658374497160e-01 1.1380659537198e-01 1.0600374347552e-01
2.4994315300734e-01 1.1650798529697e-01 9.4645065792394e-02
9.1322805831216e-01 1.1920901724521e-01 8.3286803581878e-02
5.7643693220802e-01 1.2190973353816e-01 7.1928916459652e-02
2.3956840217173e-01 1.2461017575566e-01 6.0571362716267e-02
9.0262127668041e-01 1.2731038420680e-01 4.9214100079214e-02
5.6559456591099e-01 1.3001039747782e-01 3.7857086425057e-02
2.2848749758810e-01 1.3271025206390e-01 2.6500280423324e-02
8.9129952753906e-01 1.3540998208768e-01 1.5143642098563e-02
5.5403034477709e-01 1.3810961910363e-01 3.7871333019703e-03
2.1590941798364e-01 1.4081130076478e-01 9.9242822162657e-01
8.7849133971076e-01 1.4351074373778e-01 9.8107196953513e-01
5.4099226508974e-01 1.4621017058112e-01 9.6971574822043e-01
2.0341276314944e-01 1.4890960214358e-01 9.5835952969797e-01
8.6575358773843e-01 1.5160905678921e-01 9.4700328881884e-01
5.2801565652624e-01 1.5430855055913e-01 9.3564700324857e-01
1.9020002755930e-01 1.5700809735731e-01 9.2429065339580e-01
8.5230787386936e-01 1.5970770915109e-01 9.1293422230220e-01
5.1434045653918e-01 1.6240739617791e-01 9.0157769550467e-01
1.7629909651727e-01 1.6510716715122e-01 8.9022106087973e-01
8.3818514534486e-01 1.6780702945983e-01 8.7886430847931e-01
4.9999995481722e-01 1.7050698935580e-01 8.6750743036582e-01
1.6174484544835e-01 1.7320705212762e-01 8.5615042045325e-01
8.2342107344063e-01 1.7590722225597e-01 8.4479327436022e-01
4.8502979567456e-01 1.7860750355049e-01 8.3343598927972e-01
1.4657203201786e-01 1.8130789926657e-01 8.2207856386995e-01
8.0804862399644e-01 1.8400841220161e-01 8.1072099817009e-01
4.6946018855060e-01 1.8670904477062e-01 7.9936329354470e-01
1.3080706519093e-01 1.8940979906127e-01 7.8800545266086e-01
7.9208925433054e-01 1.9211067686829e-01 7.7664747950285e-01
4.5330634384409e-01 1.9481167970711e-01 7.6528937943028e-01
1.1445741990617e-01 1.9751280880620e-01 7.5393115928767e-01
7.7554095676604e-01 2.0021406507687e-01 7.4257282757634e-01
4.3655467813123e-01 2.0291544905830e-01 7.3121439470380e-01
9.7495379968050e-02 2.0561696083437e-01 7.1985587333164e-01
7.5835870032194e-01 2.0831859991645e-01 7.0849727885224e-01
4.1913881548045e-01 2.1102036508362e-01 6.9713863003740e-01
7.9828032246075e-02 2.1372225416729e-01 6.8577994992143e-01
7.4041623124320e-01 2.1642426376039e-01 6.7442126701106e-01
4.0089009260260e-01 2.1912638882112e-01 6.6306261695935e-01
6.1231996950579e-02 2.2182862212498e-01 6.5170404491261e-01
7.2141843026922e-01 2.2453095349244e-01 6.4034560885282e-01
3.8141760992388e-01 2.2723336867597e-01 6.2898738444497e-01
4.1185849827866e-02 2.2993584771482e-01 6.1762947220916e-01
7.0066181089982e-01 2.3263836243219e-01 6.0627200836415e-01
3.5975705638370e-01 2.3534087250013e-01 5.9491518159151e-01
1.8339832969920e-02 2.3804331900755e-01 5.8355925949815e-01
6.7620570528203e-01 2.4074561342721e-01 5.7220463098088e-01
9.3618746920044e-04 2.4859496895173e-02 5.0384484341260e-01
6.6602195917970e-01 2.7577059729248e-02 4.9247517478744e-01
3.3106691090766e-01 3.0294580971695e-02 4.8110548681677e-01
9.9607005088150e-01 3.3012026481933e-02 4.6973578184691e-01
6.6103062872038e-01 3.5729364068783e-02 4.5836606245476e-01
3.2594808387241e-01 3.8446562866599e-02 4.4699633221343e-01
9.9082199493246e-01 4.1163592751948e-02 4.3562659637144e-01
6.5565203223890e-01 4.3880423830726e-02 4.2425686243425e-01
3.2043791562991e-01 4.6597026019155e-02 4.1288714064029e-01
9.8517937866260e-01 4.9313368735470e-02 4.0151744432591e-01
6.4987614000943e-01 5.2029420712666e-02 3.9014779017522e-01
3.1452788218415e-01 5.4745149936557e-02 3.7877819835248e-01
9.7913423724654e-01 5.7460523707741e-02 3.6740869251584e-01
6.4369477871681e-01 6.0175508821012e-02 3.5603929971288e-01
3.0820901861402e-01 6.2890071851356e-02 3.4467005015999e-01
9.7267640833051e-01 6.5604179532051e-02 3.3330097691021e-01
6.3709634196823e-01 6.8317799207525e-02 3.2193211541605e-01
3.0146816079195e-01 7.1030899341634e-02 3.1056350299696e-01
9.6579115758501e-01 7.3743450060846e-02 2.9919517822355e-01
6.3006457991178e-01 7.6455423711400e-02 2.8782718023349e-01
2.9428763157269e-01 7.9166795409876e-02 2.7645954799626e-01
9.5845947185941e-01 8.1877543567544e-02 2.6509231954615e-01
6.2257921255133e-01 8.4587650370339e-02 2.5372553120440e-01
2.8664591291490e-01 8.7297102198223e-02 2.4235921681230e-01
9.5065857324943e-01 9.0005889969901e-02 2.3099340699753e-01
6.1461612774801e-01 9.2714009401342e-02 2.1962812849567e-01
2.7851743759371e-01 9.5421461169233e-02 2.0826340354753e-01
9.4236128528097e-01 9.8128250973382e-02 1.9689924939147e-01
6.0614637113515e-01 1.0083438949505e-01 1.8553567786710e-01
2.6987131290375e-01 1.0353989225141e-01 1.7417269514328e-01
9.3353464911864e-01 1.0624477934947e-01 1.6281030157975e-01
5.9713484669318e-01 1.0894907514617e-01 1.5144849172692e-01
2.6067031294153e-01 1.1165280782428e-01 1.4008725446332e-01
9.2413941190923e-01 1.1435600889688e-01 1.2872657326548e-01
5.8754048460945e-01 1.1705871265554e-01 1.1736642659969e-01
2.5087187248960e-01 1.1976095557918e-01 1.0600678842066e-01
9.1413194322976e-01 1.2246277572213e-01 9.4647628758083e-02
5.7731911781325e-01 1.2516421209969e-01 8.3288914369413e-02
2.4043189772145e-01 1.2786530408978e-01 7.1930609434919e-02
9.0346889109308e-01 1.3056609086712e-01 6.0572676271021e-02
5.6642883674938e-01 1.3326661088485e-01 4.9215076038290e-02
2.2931062511175e-01 1.3596690141535e-01 3.7857769422546e-02
8.9211331521218e-01 1.3866699815906e-01 2.6500717270319e-02
5.5483614720293e-01 1.4136693492666e-01 1.5143881163562e-02
2.1747854999000e-01 1.4406674339649e-01 3.7872239225537e-03
8.7924178683705e-01 1.4676863975740e-01 9.9242812175089e-01
5.4173569507696e-01 1.4946818689703e-01 9.8107181903181e-01
2.0414837365615e-01 1.5216768503698e-01 9.6971559633219e-01
8.6647998325270e-01 1.5486715631893e-01 9.5835942485348e-01
5.2873083871112e-01 1.5756662054573e-01 9.4700327836374e-01
1.9090138953226e-01 1.6026609529076e-01 9.3564713325009e-01
8.5299219710038e-01 1.6296559603670e-01 9.2429096851840e-01
5.1500390901964e-01 1.6566513633491e-01 9.1293476575014e-01
1.7693723085415e-01 1.6836472797722e-01 9.0157850902641e-01
8.3879289545874e-01 1.7106438117283e-01 8.9022218482868e-01
5.0057162995537e-01 1.7376410472430e-01 8.7886578192559e-01
1.6227412025500e-01 1.7646390619709e-01 8.6750929125377e-01
8.2390097284473e-01 1.7916379207867e-01 8.5615270580041e-01
4.8545267335236e-01 1.8186376792397e-01 8.4479602049414e-01
1.4692954115754e-01 1.8456383848443e-01 8.3343923211022e-01
8.0833167902802e-01 1.8726400781930e-01 8.2208233919544e-01
4.6965891640313e-01 1.8996427938764e-01 8.1072534201802e-01
1.3091074449930e-01 1.9266465612028e-01 7.9936824254750e-01
7.9208624083518e-01 1.9536514047121e-01 7.8801104447029e-01
4.5318398001619e-01 1.9806573444759e-01 7.7665375324704e-01
1.1420192660201e-01 2.0076643961779e-01 7.6529637621949e-01
7.7513730449642e-01 2.0346725709621e-01 7.5393892277638e-01
4.3598643538543e-01 2.0616818750322e-01 7.4258140459080e-01
9.6744536064613e-02 2.0886923089733e-01 7.3122383594544e-01
7.5740546067494e-01 2.1157038667562e-01 7.1986623416768e-01
4.1796136834697e-01 2.1427165343617e-01 7.0850862020392e-01
7.8402288652144e-02 2.1697302879366e-01 6.9715101937314e-01
7.3871554517354e-01 2.1967450913486e-01 6.8579346235376e-01
3.9888497914043e-01 2.2237608929492e-01 6.7443598647693e-01
5.8889886602570e-02 2.2507776212609e-01 6.6307863742550e-01
7.1870353752712e-01 2.2777951791615e-01 6.5172147147052e-01
3.7829107205313e-01 2.3048134359130e-01 6.4036455841719e-01
3.7606447311701e-02 2.3318322159987e-01 6.2900798547051e-01
6.9658789930818e-01 2.3588512830522e-01 6.1765186224213e-01
3.5515101426634e-01 2.3858703158623e-01 6.0629632702551e-01
1.3177823920762e-02 2.4128888707147e-01 5.9494155404471e-01
6.6758864558305e-01 3.0805848154978e-02 5.0385688442269e-01
3.3271787506996e-01 3.3523424976251e-02 4.9248724920639e-01
9.9779661994629e-01 3.6240954548444e-02 4.8111759010166e-01
6.6282392707799e-01 3.8958402651700e-02 4.6974790877414e-01
3.2779916158151e-01 4.1675737711280e-02 4.5837820689989e-01
9.9272194828708e-01 4.4392930099687e-02 4.4700848697602e-01
6.5759211317241e-01 4.7109951473134e-02 4.3563875303197e-01
3.2240962754116e-01 4.9826774175038e-02 4.2426901123213e-01
9.8717455714382e-01 5.2543370731726e-02 4.1289927036450e-01
6.5188701780558e-01 5.5259713458133e-02 4.0152954221337e-01
3.1654713848964e-01 5.7975774184313e-02 3.9015984181556e-01
9.8115503212552e-01 6.0691524107216e-02 3.7879018760122e-01
6.4571077400256e-01 6.3406933766558e-02 3.6742060142084e-01
3.1021438708845e-01 6.6121973138718e-02 3.5605110846154e-01
9.7466583329850e-01 6.8836611838509e-02 3.4468173705613e-01
6.3906500951645e-01 7.1550819415349e-02 3.3331251839009e-01
3.0341174705609e-01 7.4264565727830e-02 3.2194348611333e-01
9.6770581324760e-01 7.6977821378909e-02 3.1057467586500e-01
6.3194691392358e-01 7.9690558192920e-02 2.9920612472214e-01
2.9613469575219e-01 8.2402749715267e-02 2.8783787058443e-01
9.6026874759989e-01 8.5114371715880e-02 2.7646995150976e-01
6.2434860038232e-01 8.7825402678407e-02 2.6510240501660e-01
2.8837372515656e-01 9.0535824258257e-02 2.5373526737085e-01
9.5234352949902e-01 9.3245621694325e-02 2.4236857287564e-01
6.1625735247963e-01 9.5954784161005e-02 2.3100235318309e-01
2.8011445876700e-01 9.8663305049289e-02 2.1963663664701e-01
9.4391403256698e-01 1.0137118216792e-01 2.0827144773493e-01
6.0765517219750e-01 1.0407841785805e-01 1.9690680651649e-01
2.7133688613260e-01 1.0678501901730e-01 1.8554272824362e-01
9.3495809130559e-01 1.0949099703180e-01 1.7417922303512e-01
5.9851761435212e-01 1.1219636761752e-01 1.6281629567568e-01
2.6201419630601e-01 1.1490115057485e-01 1.5145394553535e-01
9.2544650104816e-01 1.1760536946325e-01 1.4009216661189e-01
5.8881312756749e-01 1.2030905120524e-01 1.2873094769382e-01
2.5211262584100e-01 1.2301222563145e-01 1.1737027263826e-01
9.1534351589668e-01 1.2571492498034e-01 1.0601012075280e-01
5.7850430940639e-01 1.2841718336764e-01 9.4650467267722e-02
2.4159353298118e-01 1.3111903624146e-01 8.3291283881180e-02
9.0460975222154e-01 1.3382051983897e-01 7.1932539358111e-02
5.6755159551524e-01 1.3652167066011e-01 6.0574200162285e-02
2.3041777657918e-01 1.3922252497232e-01 4.9216231100664e-02
8.9320711480316e-01 1.4192311835850e-01 3.7858595960147e-02
5.5591855256487e-01 1.4462348531790e-01 2.6501258118533e-02
2.1855116883467e-01 1.4732365892683e-01 1.5144181114058e-02
8.8110418856019e-01 1.5002367056350e-01 3.7873291610553e-03
5.4275119036921e-01 1.5272581323811e-01 9.9242798906829e-01
2.0515638636235e-01 1.5542549827402e-01 9.8107158443846e-01
8.6748034087652e-01 1.5812510189377e-01 9.6971530717370e-01
5.2972286389268e-01 1.6082464721135e-01 9.5835912802165e-01
1.9188388898842e-01 1.6352415519096e-01 9.4700301997786e-01
8.5396345564649e-01 1.6622364470681e-01 9.3564695840285e-01
5.1596168771352e-01 1.6892313263604e-01 9.2429092108579e-01
1.7787876825682e-01 1.7162263397675e-01 9.1293488826283e-01
8.3971491100638e-01 1.7432216198366e-01 9.0157884259889e-01
5.0147032847419e-01 1.7702172831413e-01 8.9022276914148e-01
1.6314519671801e-01 1.7972134317837e-01 8.7886665525535e-01
8.2473961655966e-01 1.8242101548836e-01 8.6751049054581e-01
4.8625357087964e-01 1.8512075300064e-01 8.5615426677840e-01
1.4768687738395e-01 1.8782056244956e-01 8.4479797780183e-01
8.0903913596864e-01 1.9052044966751e-01 8.3344161948044e-01
4.7030966947915e-01 1.9322041969026e-01 8.2208518964205e-01
1.3149745625556e-01 1.9592047684528e-01 8.1072868804670e-01
7.9260105234416e-01 1.9862062482190e-01 7.9937211638153e-01
4.5361850060061e-01 2.0132086672217e-01 7.8801547828736e-01
1.1454722305435e-01 2.0402120509140e-01 7.7665877942254e-01
7.7538389176995e-01 2.0672164192753e-01 7.6530202757079e-01
4.3612427191369e-01 2.0942217866789e-01 7.5394523280044e-01
9.6763028651384e-02 2.1212281615159e-01 7.4258840768399e-01
7.5729348662340e-01 2.1482355455512e-01 7.3123156758899e-01
4.1770732671178e-01 2.1752439329707e-01 7.1987473105305e-01
7.7994199092301e-02 2.2022533090675e-01 7.0851792025850e-01
7.3814122333255e-01 2.2292636484835e-01 6.9716116162418e-01
3.9813233427719e-01 2.2562749128858e-01 6.8580448653306e-01
5.7947414635505e-02 2.2832870478980e-01 6.7444793221195e-01
7.1756112829925e-01 2.3102999790062e-01 6.6309154277063e-01
3.7694132712360e-01 2.3373136060001e-01 6.5173537038248e-01
3.6046839221297e-02 2.3643277952144e-01 6.4037947653106e-01
6.9482434332793e-01 2.3913423682967e-01 6.2902393312169e-01
3.5320386434929e-01 2.4183570851406e-01 6.1766882299158e-01
3.3423757720703e-01 3.6752235399107e-02 5.0386884899857e-01
9.9942000088034e-01 3.9469830196986e-02 4.9249925419534e-01
6.6454198811799e-01 4.2187371352942e-02 4.8112963114601e-01
3.2960261849450e-01 4.4904824396691e-02 4.6975998073715e-01
9.9460137217886e-01 4.7622158246454e-02 4.5839030364316e-01
6.5953806308356e-01 5.0339344422512e-02 4.4702060119649e-01
3.2441277110585e-01 5.3056356290277e-02 4.3565087614222e-01
9.8922577670373e-01 5.5773168369311e-02 4.2428113326961e-01
6.5397750038401e-01 5.8489755735839e-02 4.1291137991835e-01
3.1866844896086e-01 6.1206093537732e-02 4.0154162636087e-01
9.8329916973203e-01 6.3922156633220e-02 3.9017188606416e-01
6.4787021306320e-01 6.6637919357752e-02 3.7880217583574e-01
3.1238210329686e-01 6.9353355417646e-02 3.6743251585881e-01
9.7683531743295e-01 7.2068437904339e-02 3.5606292962184e-01
6.4123027066961e-01 7.4783139419183e-02 3.4469344374852e-01
3.0556730765001e-01 7.7497432295706e-02 3.3332408773388e-01
9.6984669812855e-01 8.0211288904048e-02 3.2195489359381e-01
6.3406863574119e-01 8.2924682020776e-02 3.1058589543586e-01
2.9823323862762e-01 8.5637585246468e-02 2.9921712896065e-01
9.6234055079322e-01 8.8349973453217e-02 2.8784863090463e-01
6.2639054329907e-01 9.1061823244489e-02 2.7648043843640e-01
2.9038311460993e-01 9.3773113410548e-02 2.6511258852000e-01
9.5431808969354e-01 9.6483825363734e-02 2.5374511726016e-01
6.1819521773112e-01 9.9193943539304e-02 2.4237805924481e-01
2.8201416854961e-01 1.0190345574921e-01 2.3101144690126e-01
9.4577452810650e-01 1.0461235347791e-01 2.1964530988228e-01
6.0947579353377e-01 1.0732063211138e-01 2.0827967449809e-01
2.7311736836981e-01 1.1002829109234e-01 1.9691456320957e-01
9.3669855867112e-01 1.1273533399701e-01 1.8554999419663e-01
6.0021857069655e-01 1.1544176853090e-01 1.7418598101386e-01
2.6367651079900e-01 1.1814760644326e-01 1.6282253234350e-01
9.2707138804696e-01 1.2085286336238e-01 1.5145965185273e-01
5.9040211994151e-01 1.2355755855607e-01 1.4009733815933e-01
2.5366754140430e-01 1.2626171462404e-01 1.2873558490622e-01
9.1686641700339e-01 1.2896535713099e-01 1.1737438094190e-01
5.7999745617167e-01 1.3166851419111e-01 1.0601371060000e-01
2.4305933097212e-01 1.3437121601606e-01 9.4653554068101e-02
9.0605069578995e-01 1.3707349443987e-01 8.3293887832622e-02
5.6897020819558e-01 1.3977538243446e-01 7.1934685184588e-02
2.3181655013246e-01 1.4247691362939e-01 6.0575916769040e-02
8.9458844854510e-01 1.4517812184899e-01 4.9217551160307e-02
5.5728469457477e-01 1.4787904067859e-01 3.7859555445254e-02
2.1990416050914e-01 1.5057970306982e-01 2.6501895797557e-02
8.8244581376993e-01 1.5328014099309e-01 1.5144538027689e-02
5.4490872734812e-01 1.5598038514267e-01 3.7874480955626e-03
2.0643931251584e-01 1.5868280366526e-01 9.9242782534577e-01
8.6875528373900e-01 1.6138265855103e-01 9.8107126985407e-01
5.3099019028391e-01 1.6408240036879e-01 9.6971488678084e-01
1.9314354780095e-01 1.6678205288128e-01 9.5835864675669e-01
8.5521496556245e-01 1.6948163792493e-01 9.4700252230391e-01
5.1720413078050e-01 1.7218117542427e-01 9.3564648800789e-01
1.7911078870237e-01 1.7488068344156e-01 9.2429052063634e-01
8.4093471861247e-01 1.7758017825470e-01 9.1293459921223e-01
5.0267570582703e-01 1.8027967445662e-01 9.0157870504784e-01
1.6433350968886e-01 1.8297918506948e-01 8.9022282174712e-01
8.2590782745759e-01 1.8567872166763e-01 8.7886693518410e-01
4.8739825384308e-01 1.8837829450382e-01 8.6751103346474e-01
1.4880423574585e-01 1.9107791263376e-01 8.5615510687927e-01
8.1012502154266e-01 1.9377758403499e-01 8.4479914785177e-01
4.7135960398040e-01 1.9647731571665e-01 8.3344315089286e-01
1.3250665540482e-01 1.9917711381746e-01 8.2208711256124e-01
7.9356445363311e-01 2.0187698368950e-01 8.1073103143903e-01
4.5453079625632e-01 2.0457692996632e-01 7.9937490812564e-01
1.1540290048998e-01 2.0727695661375e-01 7.8801874525431e-01
7.7617728482878e-01 2.0997706696216e-01 7.7666254753558e-01
4.3684962763055e-01 2.1267726371908e-01 7.6530632183128e-01
9.7414596252306e-02 2.1537754896045e-01 7.5395007726271e-01
7.5786563834192e-01 2.1807792409885e-01 7.4259382535604e-01
4.1819472414134e-01 2.2077838982578e-01 7.3123758022716e-01
7.8392024875491e-02 2.2347894602409e-01 7.1988135880649e-01
7.3844550703478e-01 2.2617959164444e-01 7.0852518110078e-01
3.9834041493589e-01 2.2888032453668e-01 6.9716907048293e-01
5.8058603351018e-02 2.3158114122157e-01 6.8581305398934e-01
7.1757766667215e-01 2.3428203658016e-01 6.7445716258405e-01
3.7686978860385e-01 2.3698300342319e-01 6.6310143131326e-01
3.5900202927245e-02 2.3968403187747e-01 6.5174589921022e-01
8.8287803593818e-04 4.2698665459831e-02 5.0388072310800e-01
6.6612853350227e-01 4.5416282691113e-02 4.9251117579502e-01
3.3130344865496e-01 4.8133839012537e-02 4.8114159610898e-01
9.9640673084117e-01 5.0851299525277e-02 4.6977198405007e-01
6.6143797741706e-01 5.3568633524378e-02 4.5840233917766e-01
3.2639719895049e-01 5.6285813606210e-02 4.4703266155612e-01
9.9128474065491e-01 5.9002814802028e-02 4.3566295256511e-01
6.5610120588098e-01 6.1719613778957e-02 4.2429321556864e-01
3.2084738470101e-01 6.4436188138946e-02 4.1292345644384e-01
9.8552418978002e-01 6.7152515836172e-02 4.0155368397473e-01
6.5013260090984e-01 6.9868574724507e-02 3.9018391011886e-01
3.1467361884079e-01 7.2584342239172e-02 3.7881415015525e-01
9.7914822841134e-01 7.5299795210576e-02 3.6744442272248e-01
6.4355737046402e-01 7.8014909803483e-02 3.5607474975506e-01
3.0790192165097e-01 8.0729661570951e-02 3.4470515632601e-01
9.7218268097149e-01 8.3444025609749e-02 3.3333567040315e-01
6.3640036173682e-01 8.6157976802072e-02 3.2196632252656e-01
3.0055558761470e-01 8.8871490127185e-02 3.1059714541517e-01
9.6464889145160e-01 9.1584541026079e-02 2.9922817351083e-01
6.2868071569113e-01 9.4297105802165e-02 2.8785944246918e-01
2.9265141338376e-01 9.7009162041436e-02 2.7649098860778e-01
9.5656124900080e-01 9.9720689036298e-02 2.6512284832261e-01
6.2041039850601e-01 1.0243166819831e-01 2.5375505748537e-01
2.8419894838532e-01 1.0514208344639e-01 2.4238765083473e-01
9.4792689357267e-01 1.0785192155850e-01 2.3102066137497e-01
6.1159413442467e-01 1.1056117247642e-01 2.1965411979615e-01
2.7520047307690e-01 1.1326982955483e-01 2.0828805392953e-01
9.3874560965123e-01 1.1597788974787e-01 1.9692248825184e-01
6.0222913887004e-01 1.1868535372791e-01 1.8555744345069e-01
2.6565054766792e-01 1.2139222593332e-01 1.7419293606274e-01
9.2900921437314e-01 1.2409851454395e-01 1.6282897819401e-01
5.9230440996422e-01 1.2680423138481e-01 1.5146557733004e-01
2.5553530179682e-01 1.2950939176072e-01 1.4010273624071e-01
9.1870096005140e-01 1.3221401422648e-01 1.2874045298229e-01
5.8180036698457e-01 1.3491812029900e-01 1.1737872099576e-01
2.4483242888668e-01 1.3762173411962e-01 1.0601752929783e-01
9.0779599047038e-01 1.4032488207633e-01 9.4656862757760e-02
5.7068985124922e-01 1.4302759239683e-01 8.3296702450629e-02
2.3351278332561e-01 1.4572989472392e-01 7.1937026074965e-02
8.9626354990054e-01 1.4843181968528e-01 6.0577808421097e-02
5.5894092375036e-01 1.5113339846927e-01 4.9219021875150e-02
2.2154370489160e-01 1.5383466241782e-01 3.7860636943186e-02
8.8407073667081e-01 1.5653564264626e-01 2.6502622780154e-02
5.4652091957087e-01 1.5923636969833e-01 1.5144947709205e-02
2.0889322210850e-01 1.6193687324299e-01 3.7875797184341e-03
8.7030740142083e-01 1.6463959491227e-01 9.9242763255691e-01
5.3253379882663e-01 1.6733964961584e-01 9.8107087998166e-01
1.9467946517862e-01 1.7003956062802e-01 9.6971434229446e-01
8.5674367592951e-01 1.7273935204031e-01 9.5835799031998e-01
5.1872577121085e-01 1.7543904626696e-01 9.4700179639288e-01
1.8062514223069e-01 1.7813866401924e-01 9.3564573457284e-01
8.4244121326606e-01 1.8083822431536e-01 9.2428978081009e-01
5.0417341927001e-01 1.8353774452049e-01 9.1293391306536e-01
1.6582117908656e-01 1.8623724041086e-01 9.0157811138975e-01
8.2738386420589e-01 1.8893672625614e-01 8.9022235797033e-01
4.8886076289699e-01 1.9163621491423e-01 8.7886663714783e-01
1.5025103942313e-01 1.9433571793332e-01 8.6751093541297e-01
8.1155368787622e-01 1.9703524565621e-01 8.5615524138749e-01
4.7276747995285e-01 1.9973480732286e-01 8.4479954579591e-01
1.3389090573276e-01 2.0243441116735e-01 8.3344384143345e-01
7.9492210619591e-01 2.0513406450629e-01 8.2208812313484e-01
4.5585879581373e-01 2.0783377381603e-01 8.1073238774830e-01
1.1669817304776e-01 2.1053354479665e-01 7.9937663411820e-01
7.7743681595503e-01 2.1323338242077e-01 7.8802086307904e-01
4.3807055928666e-01 2.1593329096560e-01 7.7666507746252e-01
9.8594348415526e-02 2.1863327402653e-01 7.6530928211851e-01
7.5900206404995e-01 2.2133333451031e-01 7.5395348394895e-01
4.1928630986636e-01 2.2403347460525e-01 7.4259769195177e-01
7.9438152750747e-02 2.2673369572485e-01 7.3124191726878e-01
7.3944680203559e-01 2.2943399841944e-01 7.1988617322649e-01
3.9929920960600e-01 2.3213438224767e-01 7.0853047535154e-01
5.8979566518197e-02 2.3483484559496e-01 6.9717484133052e-01
7.1846866308216e-01 2.3753538541833e-01 6.8581929086573e-01
3.7774306710108e-01 2.4023599688468e-01 6.7446384534922e-01
6.6752442070769e-01 4.8645145202987e-02 5.0389249257946e-01
3.3284371361095e-01 5.1362789893545e-02 4.9252299991393e-01
9.9808153838837e-01 5.4080365378701e-02 4.8115347102563e-01
6.6323702627730e-01 5.6797836142561e-02 4.6978390491080e-01
3.2830989683433e-01 5.9515171746744e-02 4.5841429988612e-01
9.9330036919279e-01 6.2232345810477e-02 4.4704465462877e-01
6.5820907069842e-01 6.4949335014954e-02 4.3567496905622e-01
3.2303694747756e-01 6.7666118179525e-02 4.2430524504192e-01
9.8778518052779e-01 7.0382675443998e-02 4.1293548697177e-01
6.5245510991217e-01 7.3098987579300e-02 4.0156570215110e-01
3.1704816867530e-01 7.5815035438390e-02 3.9019590107487e-01
9.8156582724434e-01 7.8530799550878e-02 3.7882609757436e-01
6.4600954836101e-01 8.1246259858188e-02 3.6745630885289e-01
3.1038075202582e-01 8.3961395581158e-02 3.5608655542245e-01
9.7468078951850e-01 8.6676185208353e-02 3.4471686095143e-01
6.3891092528063e-01 8.9390606590981e-02 3.3334725203255e-01
3.0307232529089e-01 9.2104637128729e-02 3.2197775787957e-01
9.6716605051415e-01 9.4818254030065e-02 3.1060840996043e-01
6.3119305404547e-01 9.7531434630291e-02 2.9923924157502e-01
2.9515418068070e-01 1.0024415675086e-01 2.8787028738544e-01
9.5905016781065e-01 1.0295639908403e-01 2.7650158290786e-01
6.2288164673952e-01 1.0566814158788e-01 2.6513316397513e-01
2.8664914375339e-01 1.0837936587756e-01 2.5376506618051e-01
9.5035308049899e-01 1.1109005560024e-01 2.4239732431327e-01
6.1399377345967e-01 1.1380019678224e-01 2.3102997179763e-01
2.7757143252626e-01 1.1650977813851e-01 2.1966304014678e-01
9.4108615884180e-01 1.1921879133601e-01 2.0829655844384e-01
6.0453794224535e-01 1.2192723120406e-01 1.9693055286143e-01
2.6792665874432e-01 1.2463509588647e-01 1.8556504623103e-01
9.3125206850344e-01 1.2734238693157e-01 1.7420005767240e-01
5.9451381485233e-01 1.3004910931823e-01 1.6283560229219e-01
2.5771142478227e-01 1.3275527141732e-01 1.5147169095917e-01
9.2084431133158e-01 1.3546088489000e-01 1.4010833016184e-01
5.8391177815383e-01 1.3816596452575e-01 1.2874552195176e-01
2.4691302643185e-01 1.4087052802479e-01 1.1738326397352e-01
9.0984716415226e-01 1.4357459573109e-01 1.0602154957999e-01
5.7271321760110e-01 1.4627819032347e-01 9.4660368028580e-02
2.3551014479088e-01 1.4898133647375e-01 8.3299704751811e-02
8.9823685039309e-01 1.5168406048133e-01 7.1939541693442e-02
5.6089220163720e-01 1.5438638989465e-01 6.0579857699193e-02
2.2347504455247e-01 1.5708835312964e-01 4.9220628949847e-02
8.8598421987941e-01 1.5978997909523e-01 3.7861829423554e-02
5.4841857796120e-01 1.6249129683536e-01 2.6503431373838e-02
2.1077699194430e-01 1.6519233519546e-01 1.5145405810142e-02
8.7305836866430e-01 1.6789312252067e-01 3.7877229685712e-03
5.3435631994252e-01 1.7059617231314e-01 9.9242741281302e-01
1.9649293482732e-01 1.7329645468015e-01 9.8107041994618e-01
8.5854927890629e-01 1.7599656397955e-01 9.6971368165435e-01
5.2052443860045e-01 1.7869652432079e-01 9.5835716924039e-01
1.8241753745462e-01 1.8139635840236e-01 9.4700085512119e-01
8.4422772458693e-01 1.8409608745155e-01 9.3564471307592e-01
5.0595415871145e-01 1.8679573119915e-01 9.2428871844840e-01
1.6759598765698e-01 1.8949530788465e-01 9.1293284830606e-01
8.2915232329247e-01 1.9219483428698e-01 9.0157708155879e-01
4.9062221172567e-01 1.9489432577570e-01 8.9022139904216e-01
1.5200459856479e-01 1.9759379637754e-01 8.7886578356988e-01
8.1329828892098e-01 2.0029325885327e-01 8.6751021996063e-01
4.7450190167970e-01 2.0299272478028e-01 8.5615469504453e-01
1.3561381737758e-01 2.0569220463664e-01 8.4479919765406e-01
7.9663211878213e-01 2.0839170788287e-01 8.3344371860407e-01
4.5755452297397e-01 2.1109124303809e-01 8.2208825066460e-01
1.1837830336360e-01 2.1379081774769e-01 8.1073278852982e-01
7.7910019961561e-01 2.1649043883995e-01 7.9937732878520e-01
4.3971631287757e-01 2.1919011236942e-01 7.8802186987400e-01
1.0022198298169e-01 2.2188984364465e-01 7.7666641206276e-01
7.6061164335369e-01 2.2458963723818e-01 7.6531095740373e-01
4.2087864815760e-01 2.2728949697581e-01 7.5395550968971e-01
8.1015064631511e-02 2.2998942590146e-01 7.4260007439359e-01
7.4101142149732e-01 2.3268942621257e-01 7.3124465858052e-01
4.0085640157416e-01 2.3538949915822e-01 7.1988927077370e-01
6.0536463035011e-02 2.3808964488837e-01 7.0853392074620e-01
7.2003536875577e-01 2.4078986223604e-01 6.9717861919680e-01
3.3416205212663e-01 5.4591681530252e-02 5.0390414309103e-01
9.9956583045439e-01 5.7309359386678e-02 4.9253471231545e-01
6.6487690830480e-01 6.0026958546289e-02 4.8116524179255e-01
3.3009443808653e-01 6.2744442673570e-02 4.6979572938477e-01
9.9521826986786e-01 6.5461781491288e-02 4.5842617202223e-01
6.6024884822855e-01 6.8178949607295e-02 4.4705656685865e-01
3.2518710563987e-01 7.0895925365626e-02 4.3568691223622e-01
9.9003435783596e-01 7.3612689777275e-02 4.2431720845808e-01
6.5479220544165e-01 7.6329225569598e-02 4.1294745837766e-01
3.1946244488719e-01 7.9045516378582e-02 4.0157766782028e-01
9.8404699048564e-01 8.1761546095983e-02 3.9020784585531e-01
6.4854780854322e-01 8.4477298373638e-02 3.7883800493754e-01
3.1296686355093e-01 8.7192756280027e-02 3.6746816093525e-01
9.7730607587377e-01 8.9907902098965e-02 3.5609833306030e-01
6.4156728990049e-01 9.2622717256905e-02 3.4472854371356e-01
3.0575225132349e-01 9.5337182363182e-02 3.3335881825672e-01
9.6986259206058e-01 9.8051277346473e-02 3.2198918471986e-01
6.3389982128458e-01 1.0076498167033e-01 3.1061967345314e-01
2.9786532107094e-01 1.0347827461087e-01 2.9925031672992e-01
9.6176034528635e-01 1.0619113558019e-01 2.8788114830892e-01
6.2558602050615e-01 1.0890354448004e-01 2.7651220296249e-01
2.8934334794707e-01 1.1161548207108e-01 2.6514351597906e-01
9.5303320562153e-01 1.1432693034456e-01 2.5377512264796e-01
6.1665635014579e-01 1.1703787288407e-01 2.4240705773549e-01
2.8021341785624e-01 1.1974829520686e-01 2.3103935496161e-01
9.4370492509469e-01 1.2245818507507e-01 2.1967204648713e-01
6.0713126770648e-01 1.2516753276887e-01 2.0830516242124e-01
2.7049271994818e-01 1.2787633131490e-01 1.9693873035955e-01
9.3378943311784e-01 1.3058457666456e-01 1.8557277496233e-01
5.9702143429877e-01 1.3329226781830e-01 1.7420731758221e-01
2.6018862564523e-01 1.3599940689328e-01 1.6284237594961e-01
9.2329078463591e-01 1.3870599913340e-01 1.5147796392341e-01
5.8632756568202e-01 1.4141205286183e-01 1.4011409131245e-01
2.4929850340427e-01 1.4411757937795e-01 1.2875076377211e-01
9.1220301779449e-01 1.4682259280174e-01 1.1738798277809e-01
5.7504042135876e-01 1.4952710987025e-01 1.0602574567760e-01
2.3780992820913e-01 1.5223114969194e-01 9.4664045815615e-02
9.0051066493852e-01 1.5493473346574e-01 8.3302872732125e-02
5.6314168298648e-01 1.5763788417297e-01 7.1942212423809e-02
2.2570197209102e-01 1.6034062625032e-01 6.0582047661998e-02
8.8819047432896e-01 1.6304298525305e-01 4.9222358357085e-02
5.5060609818084e-01 1.6574498751709e-01 3.7863121958503e-02
2.1294773201756e-01 1.6844665982871e-01 2.6504313878712e-02
8.7521425639618e-01 1.7114802910961e-01 1.5145907929474e-02
5.3740455456972e-01 1.7384912212431e-01 3.7878767590621e-03
1.9858659730392e-01 1.7655252266666e-01 9.9242716830315e-01
8.6063333833910e-01 1.7925305834734e-01 9.8106989513359e-01
5.2260036721397e-01 1.8195339300816e-01 9.6971291332537e-01
1.8448662898871e-01 1.8465355047961e-01 9.5835619491599e-01
8.4629107995650e-01 1.8735355345682e-01 9.4699971265721e-01
5.0801267804094e-01 1.9005342341094e-01 9.3564344028628e-01
1.6965036887576e-01 1.9275318053354e-01 9.2428735275622e-01
8.3120306740682e-01 1.9545284371065e-01 9.1293142642558e-01
4.9266963485563e-01 1.9815243052280e-01 9.0157563920957e-01
1.5404885085422e-01 2.0085195726652e-01 8.9021997069518e-01
8.1533938050262e-01 2.0355143899303e-01 8.7886440222385e-01
4.7653973600860e-01 2.0625088955961e-01 8.6750891694539e-01
1.3764823244279e-01 2.0895032168922e-01 8.5615349984719e-01
7.9866293697605e-01 2.1164974703429e-01 8.4479813776247e-01
4.5958161075634e-01 2.1434917624097e-01 8.3344281936113e-01
1.2040164231872e-01 2.1704861901015e-01 8.2208753512601e-01
7.8111997109703e-01 2.1974808415232e-01 8.1073227731659e-01
4.4173299920039e-01 2.2244757963319e-01 7.9937703992129e-01
1.0223648911330e-01 2.2514711260730e-01 7.8802181859791e-01
7.6262544434485e-01 2.2784668943673e-01 7.7666661060046e-01
4.2289396925163e-01 2.3054631569179e-01 7.6531141468801e-01
8.3035103237640e-02 2.3324599612969e-01 7.5395623100897e-01
7.4304062322202e-01 2.3594573464620e-01 7.4260106095006e-01
4.0290080656427e-01 2.3864553419302e-01 7.3124590693464e-01
6.2604144416415e-02 2.4134539665050e-01 7.1989077214806e-01
7.9559015417495e-04 6.0538281378545e-02 5.0391566016349e-01
6.6629522779968e-01 6.3255998912095e-02 4.9254629860881e-01
3.3169033672425e-01 6.5973626873985e-02 4.8117689415721e-01
9.9698008912310e-01 6.8691127888561e-02 4.6980744339254e-01
6.6216447775405e-01 7.1408471738534e-02 4.5843794169552e-01
3.2724419533740e-01 7.4125634006641e-02 4.4706838454091e-01
9.9222050963865e-01 7.6842594743935e-02 4.3569876856544e-01
6.5709514083586e-01 7.9559337229902e-02 4.2432909240810e-01
3.2187014625940e-01 8.2275846869015e-02 4.1295935733778e-01
9.8654781605892e-01 8.4992110250183e-02 4.0158956769027e-01
6.5113058194427e-01 8.7708114380909e-02 3.9021973113992e-01
3.1562093995002e-01 9.0423846096681e-02 3.7884985883113e-01
9.8002138721745e-01 9.3139291638074e-02 3.6747996539235e-01
6.4433437207592e-01 9.5854436382647e-02 3.5611006885277e-01
3.0856225621147e-01 9.8569264715430e-02 3.4474019047782e-01
9.7270728740564e-01 1.0128376002012e-01 3.3337035453714e-01
6.3677158117558e-01 1.0399790477246e-01 3.2200058801539e-01
3.0075710961439e-01 1.0671168071755e-01 3.1063092027475e-01
9.6466569579155e-01 1.0942506911339e-01 2.9926138267636e-01
6.2849901220044e-01 1.1213805102406e-01 2.8789200816725e-01
2.9225858191503e-01 1.1485060764699e-01 2.7652283083887e-01
9.5594578132252e-01 1.1756272066016e-01 2.6515388546378e-01
6.1956184352047e-01 1.2027437257635e-01 2.5378520701678e-01
2.8310786169262e-01 1.2298554709273e-01 2.4241683018786e-01
9.4658479199785e-01 1.2569622942573e-01 2.3104878889423e-01
6.0999345571284e-01 1.2840640662215e-01 2.1968111579965e-01
2.7333454055350e-01 1.3111606783883e-01 2.0831384184907e-01
9.3660860125797e-01 1.3382520458448e-01 1.9694699582727e-01
5.9981605963985e-01 1.3653381091854e-01 1.8558060394979e-01
2.6295720441200e-01 1.3924188360291e-01 1.7421468949418e-01
9.2603219113737e-01 1.4194942220390e-01 1.6284927247948e-01
5.8904104268333e-01 1.4465642914269e-01 1.5148436940035e-01
2.5198365054276e-01 1.4736290969402e-01 1.4011999302204e-01
9.1485977734026e-01 1.5006887193383e-01 1.2875615224038e-01
5.7766906077064e-01 1.5277432663795e-01 1.1739285200993e-01
2.4041101912557e-01 1.5547928713505e-01 1.0603009334145e-01
9.0308505845658e-01 1.5818376911823e-01 9.4667873368275e-02
5.6569048130976e-01 1.6088779042072e-01 8.3306185478952e-02
2.2822649685249e-01 1.6359137076187e-01 7.1945019511964e-02
8.9069223210592e-01 1.6629453147062e-01 6.0584362006486e-02
5.5308674390219e-01 1.6899729519383e-01 4.9224196501576e-02
2.1540903110995e-01 1.7169968559729e-01 3.7864503875046e-02
8.7765804661750e-01 1.7440172706692e-01 2.6505262712320e-02
5.3983270853245e-01 1.7710344441762e-01 1.5146449695157e-02
2.0193191005033e-01 1.7980486261627e-01 3.7880400000407e-03
8.6299848057586e-01 1.8250863420615e-01 9.9242690124367e-01
5.2495535145373e-01 1.8520944663592e-01 9.8106931105339e-01
1.8683314121433e-01 1.8791003165658e-01 9.6971204606000e-01
8.4863069739139e-01 1.9061041254015e-01 9.5835507926484e-01
5.1034685472485e-01 1.9331061170123e-01 9.4699838398866e-01
1.7198042734829e-01 1.9601065058627e-01 9.3564193415671e-01
8.3353019686819e-01 1.9871054959267e-01 9.2428570454819e-01
4.9499489610743e-01 2.0141032801611e-01 9.1292967100927e-01
1.5637318829275e-01 2.0411000402276e-01 9.0157381062975e-01
8.1766364144834e-01 2.0680959464338e-01 8.9021810188463e-01
4.7886469771556e-01 2.0950911578528e-01 8.7886252474261e-01
1.3997463724678e-01 2.1220858225831e-01 8.6750706074395e-01
8.0099153621799e-01 2.1490800781096e-01 8.5615169305038e-01
4.6191321836593e-01 2.1760740517253e-01 8.4479640646979e-01
1.2273719927664e-01 2.2030678609779e-01 8.3344118745805e-01
7.8346062242717e-01 2.2300616141038e-01 8.2208602409996e-01
4.4408018570194e-01 2.2570554104170e-01 8.1073090607036e-01
1.0459205675822e-01 2.2840493406187e-01 7.9937582457540e-01
7.6499177518428e-01 2.3110434869947e-01 7.8802077227291e-01
4.2527413885878e-01 2.3380379234649e-01 7.7666574316866e-01
8.5433071249362e-02 2.3650327154440e-01 7.6531073248344e-01
7.4546146554540e-01 2.3920279194634e-01 7.5395573648299e-01
4.0535100045159e-01 2.4190235824867e-01 7.4260075225929e-01
6.6742481953889e-01 6.6484951716193e-02 5.0392702915789e-01
3.3303231126109e-01 6.9202716381850e-02 4.9255774424456e-01
9.9852274558201e-01 7.1920379006262e-02 4.8118841371171e-01
6.6389531468087e-01 7.4637900930852e-02 4.6981903270157e-01
3.2915017080691e-01 7.7355251901238e-02 4.5844959486006e-01
9.9428828443159e-01 8.0072408484372e-02 4.4708009380559e-01
6.5931129725550e-01 8.2789352517605e-02 4.3571052432075e-01
3.2422137810100e-01 8.5506069667212e-02 4.2434088327283e-01
9.8902108778525e-01 8.8222548147751e-02 4.1297117028492e-01
6.5371325713824e-01 9.0938777631138e-02 4.0160138818852e-01
3.1830088059458e-01 9.3654748356526e-02 3.9023154329107e-01
9.8278702634491e-01 9.6370450438865e-02 3.7886164549158e-01
6.4717476291266e-01 9.9085873365006e-02 3.6749170827356e-01
3.1146710121485e-01 1.0180100566057e-01 3.5612174860031e-01
9.7566695062941e-01 1.0451583470772e-01 3.4475178673281e-01
6.3977708728136e-01 1.0723034669294e-01 3.3338184598593e-01
3.0380013262449e-01 1.0994452666373e-01 3.2201195243510e-01
9.6773854039107e-01 1.1265835867430e-01 3.1064213458268e-01
6.3159459007279e-01 1.1537182600140e-01 2.9927242299099e-01
2.9537038525138e-01 1.1808491141298e-01 2.8790284988799e-01
9.5906785529447e-01 1.2079759747407e-01 2.7653344875064e-01
6.2268875915428e-01 1.2350986687579e-01 2.6516425387083e-01
2.8623469023847e-01 1.2622170277481e-01 2.5379529990888e-01
9.4970708155510e-01 1.2893308913233e-01 2.4242662144010e-01
6.1310721055696e-01 1.3164401104260e-01 2.3105825250003e-01
2.7643620331925e-01 1.3435445504280e-01 2.1969022613479e-01
9.3969503787138e-01 1.3706440939686e-01 2.0832257396308e-01
6.0288454666616e-01 1.3977386434718e-01 1.9695532575693e-01
2.6600541830239e-01 1.4248281232934e-01 1.8558850904814e-01
9.2905819871939e-01 1.4519124814582e-01 1.7422214876755e-01
5.9204329215203e-01 1.4789916909586e-01 1.6285626692389e-01
2.5496096217292e-01 1.5060657505953e-01 1.5149088232845e-01
9.1781133315512e-01 1.5331346853534e-01 1.4012601037117e-01
5.8059439246718e-01 1.5601985463126e-01 1.2876166285282e-01
2.4330999366430e-01 1.5872574101068e-01 1.1739784787650e-01
9.0595786087138e-01 1.6143113779521e-01 1.0603456980084e-01
5.6853759446778e-01 1.6413605742783e-01 9.4671829255335e-02
2.3104867808874e-01 1.6684051450021e-01 8.3309623216766e-02
8.9349048685796e-01 1.6954452554946e-01 7.1947945144329e-02
5.5586229666705e-01 1.7224810882979e-01 6.0586785169074e-02
2.1816329422636e-01 1.7495128406546e-01 4.9226130332160e-02
8.8039258753221e-01 1.7765407219152e-01 3.7865964864995e-02
5.4254921633248e-01 1.8035649508899e-01 2.6506270503412e-02
2.0463216212757e-01 1.8305857532113e-01 1.5147026827306e-02
8.6664035721924e-01 1.8576033587693e-01 3.7882116168436e-03
5.2759198488971e-01 1.8846449654181e-01 9.9242661383725e-01
1.8945906295181e-01 1.9116560696552e-01 9.8106867322505e-01
8.5124773970108e-01 1.9386646525973e-01 9.6971108869946e-01
5.1295680496423e-01 1.9656709387822e-01 9.5835383442854e-01
1.7458501374180e-01 1.9926751469189e-01 9.4699688451723e-01
8.3613107999308e-01 2.0196774886163e-01 9.3564021329793e-01
4.9759366659456e-01 2.0466781673762e-01 9.2428379559146e-01
1.5897137119298e-01 2.0736773778419e-01 9.1292760693608e-01
8.2026270768442e-01 2.1006753052797e-01 9.0157162378342e-01
4.8146608304358e-01 2.1276721252711e-01 8.9021582366127e-01
1.4257976919789e-01 2.1546680035818e-01 8.7886018530381e-01
8.0360186958719e-01 2.1816630961775e-01 8.6750468875050e-01
4.6453027996824e-01 2.2086575493483e-01 8.5614931541508e-01
1.2536264291019e-01 2.2356514999072e-01 8.4479404812634e-01
7.8609629527992e-01 2.2626450754257e-01 8.3343887114205e-01
4.4672820782801e-01 2.2896383944715e-01 8.2208377013702e-01
1.0725491575147e-01 2.3166315668105e-01 8.1072873216565e-01
7.6767243881874e-01 2.3436246935399e-01 7.9937374559837e-01
4.2797618928325e-01 2.3706178671106e-01 7.8801880003000e-01
8.8160865368728e-02 2.3976111712002e-01 7.7666388615669e-01
3.3404948692114e-01 7.2431699535502e-02 5.0393823527790e-01
9.9977755729917e-01 7.5149519889543e-02 4.9256903451459e-01
6.6537522043648e-01 7.7867223897450e-02 4.8119978589118e-01
3.3084169062558e-01 8.0584771347865e-02 4.6983048292262e-01
9.9617730203369e-01 8.3302131857089e-02 4.5846111730755e-01
6.6138334084774e-01 8.6019283012627e-02 4.4709168060512e-01
3.2646187148803e-01 8.8736208557975e-02 4.3572216557497e-01
9.9141556672601e-01 9.1452896711561e-02 4.2435256719143e-01
6.5624754903308e-01 9.4169338679318e-02 4.1298288336339e-01
3.2096124803376e-01 9.6885527392229e-02 4.0161311540125e-01
9.8556027677442e-01 9.9601456478408e-02 3.9024326827531e-01
6.5004832776488e-01 1.0231711946376e-01 3.7887335070801e-01
3.1442908843049e-01 1.0503250918505e-01 3.6750337513711e-01
9.7870617469493e-01 1.0774761739352e-01 3.5613335758090e-01
6.4288308083815e-01 1.1046243452424e-01 3.4476331742982e-01
3.0696314346799e-01 1.1317694960637e-01 3.3339327718306e-01
9.7094951734070e-01 1.1589115028998e-01 3.2202326214375e-01
6.3484516080630e-01 1.1860502296702e-01 3.1065330008248e-01
2.9865282879200e-01 1.2131855296605e-01 2.9928342087606e-01
9.6237507143562e-01 1.2403172480256e-01 2.8791365612661e-01
6.2601423671405e-01 1.2674452246862e-01 2.7654403876495e-01
2.8957247566176e-01 1.2945692974780e-01 2.6517460264172e-01
9.5305174902617e-01 1.3216893054314e-01 2.5380538210991e-01
6.1645383445257e-01 1.3488050920713e-01 2.4243641160226e-01
2.7978033352378e-01 1.3759165086481e-01 2.3106772520791e-01
9.4303267819314e-01 1.4030234172179e-01 2.1969935625290e-01
6.0621213634125e-01 1.4301256935058e-01 2.0833133688958e-01
2.6931981635339e-01 1.4572232294950e-01 1.9696369770057e-01
9.3235667075301e-01 1.4843159356954e-01 1.8559646732314e-01
5.9532349903787e-01 1.5114037430539e-01 1.7422967209978e-01
2.5822094994545e-01 1.5384866044789e-01 1.6286333576112e-01
9.2104952342671e-01 1.5655644959583e-01 1.5149747914664e-01
5.8380957262954e-01 1.5926374172604e-01 1.4013211996858e-01
2.4650130619054e-01 1.6197053922144e-01 1.2876727262346e-01
9.0912479110530e-01 1.6467684685770e-01 1.1740294805495e-01
5.7167995639887e-01 1.6738267174989e-01 1.0603915367081e-01
2.3416659775192e-01 1.7008802326137e-01 9.4675893315266e-02
8.9658438316043e-01 1.7279291287809e-01 8.3313167296977e-02
5.5893285961991e-01 1.7549735405207e-01 7.1950972471479e-02
2.2121146073793e-01 1.7820136201865e-01 6.0589302375324e-02
8.8341951509189e-01 1.8090495359252e-01 4.9228147408047e-02
5.4555625507124e-01 1.8360814694809e-01 3.7867495056781e-02
2.0762082587492e-01 1.8631096138981e-01 2.6507330157268e-02
8.6961229428180e-01 1.8901341711837e-01 1.5147635184259e-02
5.3152965677349e-01 1.9171553499824e-01 3.7883905637152e-03
1.9236694148597e-01 1.9442010058236e-01 9.9242630824123e-01
8.5414435762221e-01 1.9712152811254e-01 9.8106798708805e-01
5.1584407978048e-01 1.9982268053904e-01 9.6971005001315e-01
1.7746487584180e-01 2.0252357925900e-01 9.5835247252948e-01
8.3900545889324e-01 2.0522424535279e-01 9.4699522972228e-01
5.0046448239903e-01 2.0792469944610e-01 9.3563829653821e-01
1.6184053187531e-01 2.1062496159467e-01 9.2428164805068e-01
8.2313211275194e-01 2.1332505119118e-01 9.1292525969847e-01
4.8433763411758e-01 2.1602498689326e-01 9.0156910749508e-01
1.4545538804211e-01 2.1872478657082e-01 8.9021316820783e-01
8.0648352415188e-01 2.2142446727021e-01 8.7885741950599e-01
4.6742001909441e-01 2.2412404519270e-01 8.6750184007796e-01
1.2826264046651e-01 2.2682353568392e-01 8.5614640971805e-01
7.8900890469119e-01 2.2952295323126e-01 8.4479110938323e-01
4.4965602821010e-01 2.3222231146559e-01 8.3343592122059e-01
1.1020087120544e-01 2.3492162316387e-01 8.2208082856558e-01
7.7063987287339e-01 2.3762090024899e-01 8.1072581591082e-01
4.3096897703315e-01 2.4032015378284e-01 7.9937086884435e-01
6.6929469877062e-04 7.8378531841322e-02 5.0394926357718e-01
6.6653152426463e-01 8.1096417721559e-02 4.9258015455722e-01
3.3224903500473e-01 8.3814170838722e-02 4.8121099597732e-01
9.9782106787062e-01 8.6531749126303e-02 4.6984177951131e-01
6.6324816831929e-01 8.9249121985423e-02 4.5847249466500e-01
3.2853198643877e-01 9.1966268093200e-02 4.4710313070559e-01
9.9367507044869e-01 9.4683173267171e-02 4.3573367817842e-01
6.5868066569581e-01 9.7399828497900e-02 4.2436413003018e-01
3.2355252798487e-01 1.0011622821884e-01 4.1299448238207e-01
9.8829475693437e-01 1.0283236884704e-01 4.0162473500845e-01
6.5291165233329e-01 1.0554824760160e-01 3.9025489157873e-01
3.1740759433023e-01 1.0826386158828e-01 3.7888495971655e-01
9.8178694672488e-01 1.1097920712737e-01 3.6751495092287e-01
6.4605398159415e-01 1.1369427929607e-01 3.5614488040081e-01
3.1021282287425e-01 1.1640907165445e-01 3.4477476681167e-01
9.7426740623745e-01 1.1912357612465e-01 3.3340463198347e-01
6.3822145255243e-01 1.2183778299521e-01 3.2203450058738e-01
3.0207845232513e-01 1.2455168102480e-01 3.1066439979215e-01
9.6584165872221e-01 1.2726525762305e-01 2.9929435890318e-01
6.2951408703939e-01 1.2997849908877e-01 2.8792440899036e-01
2.9309851876185e-01 1.3269139088905e-01 2.7655458250757e-01
9.5659750865383e-01 1.3540391796465e-01 2.6518491290578e-01
6.2001339359657e-01 1.3811606504965e-01 2.5381543424174e-01
2.8334830216030e-01 1.4082781699493e-01 2.4244618078434e-01
9.4660416414227e-01 1.4353915908676e-01 2.3107718662119e-01
6.0978271952556e-01 1.4625007735307e-01 2.1970848526861e-01
2.7288552651093e-01 1.4896055885114e-01 2.0834010928860e-01
9.3591396844531e-01 1.5167059193165e-01 1.9697208991713e-01
5.9886925961429e-01 1.5438016647456e-01 1.8560445670814e-01
2.6175244998293e-01 1.5708927409378e-01 1.7423723719850e-01
9.2456442905777e-01 1.5979790830750e-01 1.6287045659864e-01
5.8730592910515e-01 1.6250606467262e-01 1.5150413751407e-01
2.4997752799658e-01 1.6521374088185e-01 1.4013829970259e-01
9.1257965196287e-01 1.6792093682287e-01 1.2877295987139e-01
5.7511257852699e-01 1.7062765459999e-01 1.1740813151802e-01
2.3757643985371e-01 1.7333389851877e-01 1.0604382481816e-01
8.9997122670485e-01 1.7603967503550e-01 9.4680046562186e-02
5.6229679312641e-01 1.7874499267362e-01 8.3316800141726e-02
2.2455286192220e-01 1.8144986190994e-01 7.1954085585864e-02
8.8673903089049e-01 1.8415429503436e-01 6.0591899645808e-02
5.4885477972288e-01 1.8685830598693e-01 4.9230235925406e-02
2.1089947738838e-01 1.8956191017697e-01 3.7869085053827e-02
8.7287238975835e-01 1.9226512428881e-01 2.6508434895714e-02
5.3477268716811e-01 1.9496796607928e-01 1.5148270793183e-02
1.9659945156597e-01 1.9767045417174e-01 3.7885758334251e-03
8.5732304290485e-01 2.0037543844241e-01 9.9242598654432e-01
5.1901096294055e-01 2.0307720014229e-01 9.8106725793412e-01
1.8062190500047e-01 2.0577866556431e-01 9.6970893857491e-01
8.4215464678744e-01 2.0847985483249e-01 9.5835100547994e-01
5.0360789332687e-01 2.1118078800780e-01 9.4699343489250e-01
1.6498027330765e-01 2.1388148494478e-01 9.3563620256749e-01
8.2627033228345e-01 2.1658196516658e-01 9.2427928403519e-01
4.8747652239474e-01 2.1928224775850e-01 9.1292265484352e-01
1.4859718828304e-01 2.2198235127985e-01 9.0156629077564e-01
8.0963054887056e-01 2.2468229369273e-01 8.9021016804082e-01
4.7057467466685e-01 2.2738209230618e-01 8.7885426343675e-01
1.3142746023727e-01 2.3008176373336e-01 8.6749855448261e-01
7.9218659142180e-01 2.3278132385940e-01 8.5614301952216e-01
4.5284950682471e-01 2.3548078781673e-01 8.4478763779656e-01
1.1341335300318e-01 2.3818016996493e-01 8.3343238948677e-01
7.7387493266197e-01 2.4087948387165e-01 8.2207725572500e-01
6.6728389347665e-01 8.4325455635160e-02 5.0396009897200e-01
3.3329486597133e-01 8.7043418369010e-02 4.9259108936743e-01
9.9914568126080e-01 8.9761229489090e-02 4.8122202910725e-01
6.6483561458685e-01 9.2478844732686e-02 4.6985290777491e-01
3.3036546075683e-01 9.5196233209015e-02 4.5848371239740e-01
9.9573729459993e-01 9.7913374794546e-02 4.4711442968169e-01
6.6095422427896e-01 1.0063025760686e-01 4.3574504774235e-01
3.2602015248829e-01 1.0334687569320e-01 4.2437555735050e-01
9.9093955623493e-01 1.0606322701397e-01 4.1300595276397e-01
6.5571729182404e-01 1.0877931175662e-01 4.0163623221250e-01
3.2035842823942e-01 1.1149513098210e-01 3.9026639811297e-01
9.8486810947689e-01 1.1421068558446e-01 3.7889645708316e-01
6.4925144454065e-01 1.1692597553168e-01 3.6752641981178e-01
3.1351342264615e-01 1.1964099934955e-01 3.5615630083115e-01
9.7765885054383e-01 1.2235575381018e-01 3.4478611822689e-01
6.4169230864494e-01 1.2507023378790e-01 3.3341589330964e-01
3.0561812266568e-01 1.2778443224923e-01 3.2204565026534e-01
9.6944034771090e-01 1.3049834034773e-01 3.1067541579452e-01
6.3316276201822e-01 1.3321194759855e-01 2.9930521874691e-01
2.9678886792771e-01 1.3592524211168e-01 2.8793508975415e-01
9.6032189799590e-01 1.3863821086623e-01 2.7656506086230e-01
6.2376482451758e-01 1.4135084001115e-01 2.6519516516437e-01
2.8712037104105e-01 1.4406311518033e-01 2.5382543643334e-01
9.5039102475739e-01 1.4677502181201e-01 2.4245590875601e-01
6.1357904891013e-01 1.4948654546433e-01 2.3108661616872e-01
2.7668649460659e-01 1.5219767212020e-01 2.1971759229650e-01
9.3971521161753e-01 1.5490838847571e-01 2.0834886999803e-01
6.0266685792868e-01 1.5761868220760e-01 1.9698048101905e-01
2.6554290795551e-01 1.6032854221594e-01 1.8561245565801e-01
9.2834465945451e-01 1.6303795883885e-01 1.7424482244762e-01
5.9107323925850e-01 1.6574692403708e-01 1.6287760785658e-01
2.5372960803339e-01 1.6845543154644e-01 1.5151083601585e-01
9.1631456429868e-01 1.7116347699700e-01 1.4014452847382e-01
5.7882874797664e-01 1.7387105799828e-01 1.2877870398438e-01
2.4127264373582e-01 1.7657817419045e-01 1.1741337833176e-01
9.0364658437676e-01 1.7928482726187e-01 1.0604856419527e-01
5.6595075447271e-01 1.8199102093396e-01 9.4684271056456e-02
2.2818519443005e-01 1.8469676091519e-01 8.3320505150391e-02
8.9034980507344e-01 1.8740205482603e-01 7.1957269461749e-02
5.5244435279479e-01 1.9010691209776e-01 6.0594563765435e-02
2.1446847523504e-01 1.9281134384827e-01 4.9232384710390e-02
8.7642168739786e-01 1.9551536273830e-01 3.7870725944190e-02
5.3830338802779e-01 1.9821898281220e-01 2.6509578275062e-02
2.0011286602508e-01 2.0092221932730e-01 1.5148929867035e-02
8.6184930661789e-01 2.0362508857601e-01 3.7887664632350e-03
5.2245988480543e-01 2.0633050334106e-01 9.9242565075124e-01
1.8405849253156e-01 2.0903261432421e-01 9.8106649085985e-01
8.4558083035898e-01 2.1173440969014e-01 9.6970776267324e-01
5.0702571373019e-01 2.1443590809512e-01 9.5834944483992e-01
1.6839186959149e-01 2.1713712837068e-01 9.4699151492159e-01
8.2967793369112e-01 2.1983808937953e-01 9.3563394966202e-01
4.9088244510477e-01 2.2253880988554e-01 9.2427672524404e-01
1.5200383763769e-01 2.2523930843856e-01 9.1291981753047e-01
8.1304042776121e-01 2.2793960327398e-01 9.0156320228519e-01
4.7399039874360e-01 2.3063971222667e-01 8.9020685537151e-01
1.3485178062873e-01 2.3333965265800e-01 8.7885075292543e-01
7.9562242569767e-01 2.3603944139428e-01 8.6749487150201e-01
4.5629997901521e-01 2.3873909467457e-01 8.5613918819342e-01
1.1688184361306e-01 2.4143862810532e-01 8.4478368071773e-01
3.3389287273729e-01 9.0272477894318e-02 5.0397072625899e-01
6.8348429366949e-05 9.2990530541150e-02 4.9260182381231e-01
6.6606690643997e-01 9.5708409911939e-02 4.8123287028798e-01
3.3188786793957e-01 9.8426069160923e-02 4.6986385288491e-01
9.9753232614918e-01 1.0114347704228e-01 4.5849475581528e-01
6.6300285731361e-01 1.0386061479323e-01 4.4712556291504e-01
3.2830322432435e-01 1.0657747312885e-01 4.3575625962322e-01
9.9343809174553e-01 1.0929404951499e-01 4.2438683437475e-01
6.5841276522135e-01 1.1201034581258e-01 4.1301727948978e-01
3.2323296292751e-01 1.1472636632899e-01 4.0164759165704e-01
9.8790462236958e-01 1.1744211627174e-01 3.9027777210828e-01
6.5243374259086e-01 1.2015760057382e-01 3.7890782657049e-01
3.1682625967069e-01 1.2287282304598e-01 3.6753776506722e-01
9.8108795210113e-01 1.2558778580670e-01 3.5616760162474e-01
6.4522437201165e-01 1.2830248894055e-01 3.4479735392374e-01
3.0924079806919e-01 1.3101693033994e-01 3.3342704292451e-01
9.7314220604434e-01 1.3373110569043e-01 3.2205669248361e-01
6.3693325337616e-01 1.3644500856594e-01 3.1068632897253e-01
3.0061827449634e-01 1.3915863060563e-01 2.9931598090349e-01
9.6420128412787e-01 1.4187196174962e-01 2.8794567856412e-01
6.2768598621765e-01 1.4458499051491e-01 2.7657545366058e-01
2.9107578657676e-01 1.4729770429649e-01 2.6520533896805e-01
9.5437380767541e-01 1.5001008968188e-01 2.5383536798696e-01
6.1758290437175e-01 1.5272213276918e-01 2.4246557460364e-01
2.8070567964296e-01 1.5543381948137e-01 2.3109599275495e-01
9.4374449963818e-01 1.5814513587023e-01 2.1972665609687e-01
6.0670150758756e-01 1.6085606840520e-01 2.0835759767785e-01
2.6957863628406e-01 1.6356660424311e-01 1.9698884961876e-01
9.3237761900682e-01 1.6627673147535e-01 1.8562044280159e-01
5.9509999887938e-01 1.6898643935016e-01 1.7425240656998e-01
2.5774713675489e-01 1.7169571846777e-01 1.6288476844491e-01
9.2032021779496e-01 1.7440456094705e-01 1.5151755385917e-01
5.8282025696065e-01 1.7711296056236e-01 1.4015078591449e-01
2.4524810366493e-01 1.7982091285009e-01 1.2878448516526e-01
9.0760444584681e-01 1.8252841518465e-01 1.1741866943216e-01
5.6988981372076e-01 1.8523546682392e-01 1.0605335364935e-01
2.3210458343182e-01 1.8794206892501e-01 9.4688549747706e-02
8.9424898081081e-01 1.9064822453121e-01 8.3324266576500e-02
5.5632308537642e-01 1.9335393853180e-01 7.1960509864730e-02
2.1832683467484e-01 1.9605921759657e-01 6.0597282222825e-02
8.8026002898687e-01 1.9876407008756e-01 4.9234583184286e-02
5.4212233636831e-01 2.0146850595065e-01 3.7872409286039e-02
2.0391329792707e-01 2.0417253659035e-01 2.6510754185273e-02
8.6563233318011e-01 2.0687617473087e-01 1.5149608809742e-02
5.2727874528009e-01 2.0957943426729e-01 3.7889615376713e-03
1.8777700405665e-01 2.1228528949693e-01 9.9242530277413e-01
8.4928648600767e-01 2.1498776303597e-01 9.8106569073753e-01
5.1072038374959e-01 2.1768990347367e-01 9.6970653025224e-01
1.7207757473241e-01 2.2039172782507e-01 9.5834780171919e-01
8.3335683410699e-01 2.2309325350111e-01 9.4698948416280e-01
4.9455683277853e-01 2.2579449816812e-01 9.3563155548312e-01
1.5567613305110e-01 2.2849547961721e-01 9.2427399270167e-01
8.1671318151236e-01 2.3119621564441e-01 9.1291677219658e-01
4.7766629881215e-01 2.3389672394218e-01 9.0155986992292e-01
1.3853366598933e-01 2.3659702200208e-01 8.9020326161484e-01
7.9931330699705e-01 2.3929712702791e-01 8.7884692296647e-01
4.6000306706450e-01 2.4199705585817e-01 8.6749082978923e-01
4.9574925484109e-04 9.6219605545405e-02 5.0398113013822e-01
6.6685287056641e-01 9.8937763181330e-02 4.9261234265204e-01
3.3301475870113e-01 1.0165572261909e-01 4.8124350441663e-01
9.9898079767914e-01 1.0437343398904e-01 4.6987459989538e-01
6.6475244228850e-01 1.0709086564765e-01 4.5850561008724e-01
3.3033286688015e-01 1.0980800042085e-01 4.4713651559537e-01
9.9572660714183e-01 1.1252483200773e-01 4.3576729890640e-01
6.6093921839379e-01 1.1524136174877e-01 4.2439794594707e-01
3.2597696618186e-01 1.1795759586646e-01 4.1302844703194e-01
9.9084655796059e-01 1.2067354321256e-01 4.0165879733179e-01
6.5555491909396e-01 1.2338921350584e-01 3.9028899698810e-01
3.2010901240868e-01 1.2610461601358e-01 3.7891905098291e-01
9.8451569797639e-01 1.2881975861594e-01 3.6754896885186e-01
6.4878162839930e-01 1.3153464718817e-01 3.5617876430723e-01
3.1291317431690e-01 1.3424928523875e-01 3.4480845481805e-01
9.7691637486260e-01 1.3696367374813e-01 3.3343806117850e-01
6.4079690815109e-01 1.3967781116074e-01 3.2206760708361e-01
3.0456007740913e-01 1.4239169349118e-01 3.1069711872203e-01
9.6821080896149e-01 1.4510531451309e-01 2.9932662438966e-01
6.3175365888202e-01 1.4781866600552e-01 2.8795615412408e-01
2.9519282567859e-01 1.5053173803733e-01 2.7658573935717e-01
9.5853216688148e-01 1.5324451927424e-01 2.6521541258288e-01
6.2177521784211e-01 1.5595699729682e-01 2.5384520703644e-01
2.8492521142489e-01 1.5866915892034e-01 2.4247515638212e-01
9.4798509759354e-01 1.6138099050939e-01 2.3110529440708e-01
6.1095756216319e-01 1.6409247828183e-01 2.1973565472006e-01
2.7384504421428e-01 1.6680360859793e-01 2.0836627045430e-01
9.3664975185325e-01 1.6951436823123e-01 1.9699717397514e-01
5.9937367615934e-01 1.7222474461861e-01 1.8562839659369e-01
2.6201860328407e-01 1.7493472608754e-01 1.7425996828839e-01
9.2458612477002e-01 1.7764430205888e-01 1.6289191743726e-01
5.8707764623350e-01 1.8035346322407e-01 1.5152427056369e-01
2.4949439461117e-01 1.8306220169576e-01 1.4015705209916e-01
9.1183742420783e-01 1.8577051113148e-01 1.2879028416644e-01
5.7410762180034e-01 1.8847838682985e-01 1.1742398638662e-01
2.3630571105512e-01 1.9118582579962e-01 1.0605817571332e-01
8.9843225650284e-01 1.9389282680174e-01 9.4692866296958e-02
5.6048766728881e-01 1.9659939036520e-01 8.3328069381745e-02
2.2247220087927e-01 1.9930551877769e-01 7.1963793237237e-02
8.8438596685845e-01 2.0201121605248e-01 6.0600043125728e-02
5.4622893089815e-01 2.0471648787328e-01 4.9236821306016e-02
2.0800091892506e-01 2.0742134151905e-01 3.7874127073206e-02
8.6970162145285e-01 2.1012578577132e-01 2.6511956833416e-02
5.3133059798853e-01 2.1282983080660e-01 1.5150304211432e-02
1.9288728136859e-01 2.1553348807663e-01 3.7891601885756e-03
8.5327391287339e-01 2.1823979202360e-01 9.9242494442979e-01
5.1469444491644e-01 2.2094263966174e-01 9.8106486220174e-01
1.7604004355252e-01 2.2364513857971e-01 9.6970524887957e-01
8.3730966908925e-01 2.2634730399796e-01 9.5834608671849e-01
4.9850216768057e-01 2.2904915173416e-01 9.4698735633573e-01
1.5961626994752e-01 2.3175069807013e-01 9.3562903694254e-01
8.2065058754094e-01 2.3445195962460e-01 9.2427110657521e-01
4.8160360730440e-01 2.3715295323277e-01 9.1291354232060e-01
1.4247368269522e-01 2.3985369583364e-01 9.0155632053204e-01
6.6709195261737e-01 1.0216684543142e-01 5.0399129524149e-01
3.3364949003229e-01 1.0488512548695e-01 4.9262263056602e-01
9.9999164375090e-01 1.0760317862503e-01 4.8125391630664e-01
6.6611788461965e-01 1.1032095144787e-01 4.6988513376723e-01
3.3203011038941e-01 1.1303841190221e-01 4.5851626025714e-01
9.9773221575835e-01 1.1575554471764e-01 4.4714727272321e-01
6.6322965661536e-01 1.1847234707600e-01 4.3577815038697e-01
3.2852903822049e-01 1.2118882476358e-01 4.2440887648582e-01
9.9363774651527e-01 1.2390498892999e-01 4.1303943927407e-01
6.5856363252142e-01 1.2662085348248e-01 4.0166983245676e-01
3.2331475265451e-01 1.2933643308233e-01 3.9030005521798e-01
9.8789916286749e-01 1.3205174167428e-01 3.7893011198157e-01
6.5232476156956e-01 1.3476679146576e-01 3.6756001201187e-01
3.1659917480175e-01 1.3748159227096e-01 3.5618976893383e-01
9.8072967673899e-01 1.4019615114161e-01 3.4481940022630e-01
6.4472313884311e-01 1.4291047221663e-01 3.3344892672289e-01
3.0858600161752e-01 1.4562455673439e-01 3.2207837213925e-01
9.7232426370499e-01 1.4833840316232e-01 3.1070776263558e-01
6.3594348389457e-01 1.5105200740839e-01 2.9933712641564e-01
2.9944879238579e-01 1.5376536308737e-01 2.8796649335988e-01
9.6284490835880e-01 1.5647846182090e-01 2.7659589468752e-01
6.2613616150683e-01 1.5919129355638e-01 2.6522536264213e-01
2.8932651569972e-01 1.6190384689277e-01 2.5385493019462e-01
9.5241959337485e-01 1.6461610940515e-01 2.4248463075912e-01
6.1541869960481e-01 1.6732806796175e-01 2.3111449791730e-01
2.7832684508029e-01 1.7003970902869e-01 2.1974456514838e-01
9.4114676748380e-01 1.7275101895937e-01 2.0837486556298e-01
6.0388095092321e-01 1.7546198426564e-01 1.9700543164014e-01
2.6653164325101e-01 1.7817259186916e-01 1.8563629496759e-01
9.2910087122202e-01 1.8088282933146e-01 1.7426748598669e-01
5.9159045354286e-01 1.8359268506156e-01 1.6289903374356e-01
2.5400201194434e-01 1.8630214850048e-01 1.5153096564905e-01
9.1633698046607e-01 1.8901121028212e-01 1.4016330725028e-01
5.7859661318205e-01 1.9171986236993e-01 1.2879608201688e-01
2.4078199061953e-01 1.9442809816948e-01 1.1742931114527e-01
9.0289402513152e-01 1.9713591261675e-01 1.0606301338401e-01
5.6493346547795e-01 1.9984330224237e-01 9.4697204883548e-02
2.2690090085280e-01 2.0255026521229e-01 8.3331899072868e-02
8.8879676456614e-01 2.0525680134565e-01 7.1967106565674e-02
5.5062133755254e-01 2.0796291211048e-01 6.0602835097723e-02
2.1237475183307e-01 2.1066860059884e-01 4.9239089496639e-02
8.7405699400786e-01 2.1337387148255e-01 3.7875871684687e-02
5.3566790880402e-01 2.1607873095138e-01 2.6513180714311e-02
1.9720720264932e-01 2.1878318663576e-01 1.5151012835436e-02
8.5867444718966e-01 2.2148724751602e-01 3.7893615928722e-03
5.1895012887326e-01 2.2419400682926e-01 9.9242457744207e-01
1.8028186464321e-01 2.2689723848915e-01 9.8106400964960e-01
8.4153927220004e-01 2.2960010767877e-01 9.6970392573756e-01
5.0272141238854e-01 2.3230262768942e-01 9.5834430990477e-01
1.6382722191621e-01 2.3500481259045e-01 9.4698514447902e-01
8.2485551236914e-01 2.3770667710716e-01 9.3562641012625e-01
4.8580496750072e-01 2.4040823650045e-01 9.2426808606367e-01
3.3368080700503e-01 1.0811420427132e-01 5.0400120616558e-01
4.5945554917499e-04 1.1083262693533e-01 4.9263267218450e-01
6.6700039548282e-01 1.1355078951482e-01 4.8126409071995e-01
3.3330321799646e-01 1.1626863450532e-01 4.6989543939848e-01
9.9937036907383e-01 1.1898612947750e-01 4.5852669126525e-01
6.6520661893563e-01 1.2170326149420e-01 4.4715781911247e-01
3.3081852836753e-01 1.2442003186152e-01 4.3578879854413e-01
9.9621394971932e-01 1.2713645152520e-01 4.2441960992202e-01
6.6140158591679e-01 1.2985253725271e-01 4.1305023940867e-01
3.2639061853375e-01 1.3256830861837e-01 4.0168067933731e-01
9.9119040680909e-01 1.3528378573153e-01 3.9031092811887e-01
6.5581025347322e-01 1.3799898760788e-01 3.7894098985910e-01
3.2025922986257e-01 1.4071393107040e-01 3.6757087381750e-01
9.8454605136800e-01 1.4342863006934e-01 3.5620059380107e-01
6.4867899412744e-01 1.4614309532270e-01 3.4483016755412e-01
3.1266584450596e-01 1.4885733419424e-01 3.3345961618006e-01
9.7651387392070e-01 1.5157135074227e-01 3.2208896361363e-01
6.4022983271054e-01 1.5428514588705e-01 3.1071823614951e-01
3.0381995786806e-01 1.5699871765714e-01 2.9934746202574e-01
9.6728999046474e-01 1.5971206148526e-01 2.8797667105590e-01
6.3064519947578e-01 1.6242517053225e-01 2.7660589430204e-01
2.9389040944446e-01 1.6513803602356e-01 2.6523516377965e-01
9.5703003002678e-01 1.6785064758752e-01 2.5386451218669e-01
6.2006808594336e-01 1.7056299358765e-01 2.4249397264934e-01
2.8300824625481e-01 1.7327506144391e-01 2.3112357847881e-01
9.4585385218805e-01 1.7598683793923e-01 2.1975336293458e-01
6.0860794298844e-01 1.7869830950900e-01 2.0838335899108e-01
2.7127327947029e-01 1.8140946251188e-01 1.9701359910592e-01
9.3385236509497e-01 1.8412028348094e-01 1.8564411498892e-01
5.9634746453127e-01 1.8683075935439e-01 1.7427493737243e-01
2.5876061975019e-01 1.8954087768554e-01 1.6290609578386e-01
9.2109366378303e-01 1.9225062683161e-01 1.5153761832241e-01
5.8334823232846e-01 1.9495999612135e-01 1.4016953144234e-01
2.4552577343528e-01 1.9766897600128e-01 1.2880185974555e-01
9.0762755551284e-01 2.0037755816055e-01 1.1743462578640e-01
5.6965467393419e-01 2.0308573563464e-01 1.0606784989214e-01
2.3160805649627e-01 2.0579350288779e-01 9.4701550001745e-02
8.9348846799151e-01 2.0850085587470e-01 8.3335741526392e-02
5.5529651412315e-01 2.1120779208178e-01 7.1970437234087e-02
2.1703264496780e-01 2.1391431054857e-01 6.0605647160812e-02
8.7869715815134e-01 2.1662041187004e-01 4.9241378549997e-02
5.4029020186129e-01 2.1932609818083e-01 3.7877635821531e-02
2.0181177777221e-01 2.2203137312251e-01 2.6514420570939e-02
8.6326174391051e-01 2.2473624179528e-01 1.5151731598631e-02
5.2463981743542e-01 2.2744071069571e-01 3.7895649684682e-03
1.8480520100753e-01 2.3014793052342e-01 9.9242420344828e-01
8.4604826220138e-01 2.3285155460869e-01 9.8106313725219e-01
5.0721754563981e-01 2.3555480434255e-01 9.6970256763399e-01
1.6831222745907e-01 2.3825769096983e-01 9.5834248081531e-01
8.2933135146699e-01 2.4096022667313e-01 9.4698286094198e-01
2.6150806812070e-04 1.1406168861065e-01 5.0401084751042e-01
6.6728424777883e-01 1.1678027731820e-01 4.9264245212520e-01
3.3404436471795e-01 1.1949856753036e-01 4.8127401240535e-01
5.4161699320194e-04 1.2221649697083e-01 4.6990550166041e-01
6.6677913568008e-01 1.2493403293559e-01 4.5853688797237e-01
3.3276276456827e-01 1.2765116540292e-01 4.4716813939000e-01
9.9850040184999e-01 1.3036790062766e-01 4.3579922750398e-01
6.6400139205172e-01 1.3308425560564e-01 4.2443012961629e-01
3.2927599652896e-01 1.3580025356389e-01 4.1306082980286e-01
9.9433495437386e-01 1.3851592047197e-01 4.0169131917775e-01
6.5918913379730e-01 1.4123128247519e-01 3.9032159563125e-01
3.2384926672057e-01 1.4394636410596e-01 3.7895166325957e-01
9.8832575555961e-01 1.4666118711934e-01 3.6758153164564e-01
6.5262853995213e-01 1.4937576980915e-01 3.5621121509970e-01
3.1676701152387e-01 1.5209012668073e-01 3.4484073192686e-01
9.8074996599092e-01 1.5480426837916e-01 3.3347010375879e-01
6.4458558345563e-01 1.5751820179427e-01 3.2209935496488e-01
3.0828142936597e-01 1.6023193028246e-01 3.1072851214519e-01
9.7184447010314e-01 1.6294545396163e-01 2.9935760369884e-01
6.3528109846355e-01 1.6565877004763e-01 2.8798665945755e-01
2.9859716538679e-01 1.6837187321027e-01 2.7661571037228e-01
9.6179801516241e-01 1.7108475593391e-01 2.6524478824104e-01
6.2488852204866e-01 1.7379740887248e-01 2.5387392546692e-01
2.8787312678646e-01 1.7650982119267e-01 2.4250315483701e-01
9.5075587191961e-01 1.7922198090119e-01 2.3113250931426e-01
6.1354043516402e-01 1.8193387515380e-01 2.1976202183659e-01
2.7623016032549e-01 1.8464549054488e-01 2.0839172511862e-01
9.3882808546440e-01 1.8735681337704e-01 1.9702165145337e-01
6.0133696816017e-01 1.9006782991053e-01 1.8565183251226e-01
2.6375930784865e-01 1.9277852659278e-01 1.7428229914291e-01
9.2609736529839e-01 1.9548889026814e-01 1.6291308116551e-01
5.8835317936360e-01 1.9819890836813e-01 1.5154420716874e-01
2.5052858120537e-01 2.0090856908267e-01 1.4017570430753e-01
9.1262520621190e-01 2.0361786151230e-01 1.2880759810478e-01
5.7464450387424e-01 2.0632677580186e-01 1.1743991226011e-01
2.3658774588843e-01 2.0903530325560e-01 1.0607266846839e-01
8.9845603275881e-01 2.1174343643425e-01 9.4705886251433e-02
5.6025029917085e-01 2.1445116923385e-01 8.3339582805612e-02
2.2197131838731e-01 2.1715849694696e-01 7.1973772868641e-02
8.8361970589811e-01 2.1986541630632e-01 6.0608468607956e-02
5.4519592252439e-01 2.2257192551154e-01 4.9243679533138e-02
2.0670027714074e-01 2.2527802423920e-01 3.7879412434182e-02
8.6813292913857e-01 2.2798371363720e-01 2.6515671346950e-02
5.2949389070861e-01 2.3068899630403e-01 1.5152457546538e-02
1.9078302897409e-01 2.3339387625410e-01 3.7897695686623e-03
8.5083873137253e-01 2.3610156033304e-01 9.9242382400898e-01
5.1199319990538e-01 2.3880558381889e-01 9.8106224897533e-01
1.7307437868750e-01 2.4150922294097e-01 9.6970118102894e-01
6.6683309331737e-01 1.2000930476124e-01 5.0402020392178e-01
3.3412563139654e-01 1.2272808678837e-01 4.9265195503511e-01
1.1275315719586e-03 1.2544652568120e-01 4.8128366614308e-01
6.6783878352001e-01 1.2816455362613e-01 4.6991530543927e-01
3.3426338253714e-01 1.3088213784582e-01 4.5854683518518e-01
4.0850040119484e-04 1.3359927202078e-01 4.4717821798767e-01
6.6628366707169e-01 1.3631596841535e-01 4.3580942098307e-01
3.3190002532023e-01 1.3903225118737e-01 4.2444041824279e-01
9.9726969246883e-01 1.4174815104708e-01 4.1307119181779e-01
6.6240524122101e-01 1.4446370122264e-01 4.0170173183676e-01
3.2731929640806e-01 1.4717893457360e-01 3.9033203601180e-01
9.9202423566671e-01 1.4989388164765e-01 3.7896210882468e-01
6.5653197809819e-01 1.5260856947255e-01 3.6759196058586e-01
3.2085384421818e-01 1.5532302089703e-01 3.5622160649084e-01
9.8500047164437e-01 1.5803725432536e-01 3.4485106574612e-01
6.4898177300939e-01 1.6075128372331e-01 3.3348036079995e-01
3.1280692490446e-01 1.6346511880282e-01 3.2210951668860e-01
9.7648437889372e-01 1.6617876531763e-01 3.1073856049401e-01
6.4002188761393e-01 1.6889222542211e-01 2.9936752090022e-01
3.0342654062777e-01 1.7160549806000e-01 2.8799642783276e-01
9.6670480603310e-01 1.7431857936134e-01 2.7662531216399e-01
6.2986257488000e-01 1.7703146303333e-01 2.6525420546875e-01
2.9290520625672e-01 1.7974414073663e-01 2.5388313981608e-01
9.5583757152251e-01 1.8245660244212e-01 2.4251214758524e-01
6.1866409663039e-01 1.8516883676578e-01 2.3114126129645e-01
2.8138880183287e-01 1.8788083128080e-01 2.1977051344880e-01
9.4401533832522e-01 1.9059257280698e-01 2.0839993635990e-01
6.0654702157810e-01 1.9330404767820e-01 1.9702956200349e-01
2.6898686125869e-01 1.9601524198869e-01 1.8565942184258e-01
9.3133758775198e-01 1.9872614181931e-01 1.7428954665719e-01
5.9360167537802e-01 2.0143673344468e-01 1.6291996636662e-01
2.5578136246507e-01 2.0414700352217e-01 1.5155070984707e-01
9.1787866848643e-01 2.0685693926357e-01 1.4018180474631e-01
5.7989540850345e-01 2.0956652859007e-01 1.2881327729732e-01
2.4183320518045e-01 2.1227576027108e-01 1.1744515213371e-01
9.0369349865217e-01 2.1498462404740e-01 1.0607745210954e-01
5.6547755452846e-01 2.1769311073897e-01 9.4710198126845e-02
2.2718647031934e-01 2.2040121233742e-01 8.3343408973804e-02
8.8882118055200e-01 2.2310892208366e-01 7.1977101176779e-02
5.5038246083471e-01 2.2581623453056e-01 6.0611288869159e-02
2.1187093109765e-01 2.2852314559099e-01 4.9245983679732e-02
8.7328705821113e-01 2.3122965257122e-01 3.7881194642988e-02
5.3463115814599e-01 2.3393575419018e-01 2.6516928133299e-02
1.9590339780154e-01 2.3664145058470e-01 1.5153187824444e-02
8.5710379658360e-01 2.3934674330141e-01 3.7899746753962e-03
5.1705039294163e-01 2.4205489402990e-01 9.9242344062038e-01
3.3339440381961e-01 1.2595705872718e-01 5.0402926013826e-01
9.8572208461732e-04 1.2867606592346e-01 4.9266116563700e-01
6.6825464900518e-01 1.3139467788823e-01 4.8129303679579e-01
3.3520149581949e-01 1.3411282039002e-01 4.6992483568317e-01
1.8313584094209e-03 1.3683046092702e-01 4.5855651767994e-01
6.6815306586951e-01 1.3954759794524e-01 4.4718803912043e-01
3.3417815502284e-01 1.4226425108634e-01 4.3581936220115e-01
9.9991995111600e-01 1.4498045305948e-01 4.2445045762404e-01
6.6539279553533e-01 1.4769624330163e-01 4.1308130556119e-01
3.3061143132404e-01 1.5041166331842e-01 4.0171189550107e-01
9.9559053783163e-01 1.5312675345950e-01 3.9034222543731e-01
6.6034439545269e-01 1.5584155083739e-01 3.7897230074086e-01
3.2488665757046e-01 1.5855608811022e-01 3.6760213294493e-01
9.8923020705443e-01 1.6127039288726e-01 3.5623173858263e-01
6.5338707705632e-01 1.6398448756416e-01 3.4486113815147e-01
3.1736841911144e-01 1.6669838944046e-01 3.3349035523471e-01
9.8118450490792e-01 1.6941211101206e-01 3.2211941578164e-01
6.4484475113424e-01 1.7212566036298e-01 3.1074834753360e-01
3.0835775939162e-01 1.7483904160477e-01 2.9937717957489e-01
9.7173136523532e-01 1.7755225533024e-01 2.8800594198526e-01
6.3497269203079e-01 1.8026529906009e-01 2.7663466557157e-01
2.9808820654551e-01 1.8297816767036e-01 2.6526338165802e-01
9.6108377412204e-01 1.8569085379395e-01 2.5389212191784e-01
6.2396471195916e-01 1.8840334819343e-01 2.4252091823168e-01
2.8673583952709e-01 1.9111564010479e-01 2.3114980256154e-01
9.4940152550382e-01 1.9382771755280e-01 2.1977880683127e-01
6.1196573088103e-01 1.9653956763985e-01 2.0840796280717e-01
2.7443204807680e-01 1.9925117681013e-01 1.9703730197413e-01
9.3680373602842e-01 2.0196253109118e-01 1.8566685540434e-01
5.9908375133767e-01 2.0467361631487e-01 1.7429665361707e-01
2.6127477561293e-01 2.0738441831961e-01 1.6292672642909e-01
9.2337923920517e-01 2.1009492313527e-01 1.5155710279615e-01
5.8539934157340e-01 2.1280511715235e-01 1.4018781064686e-01
2.4733706854229e-01 2.1551498727649e-01 1.2881887671079e-01
9.0919420673380e-01 2.1822452106906e-01 1.1745032634308e-01
5.7097235546578e-01 2.2093370687480e-01 1.0608218334841e-01
2.3267293641523e-01 2.2364253393680e-01 9.4714469807241e-02
8.9429720134272e-01 2.2635099249910e-01 8.3347205907437e-02
5.5584623816702e-01 2.2905907389728e-01 7.1980409784632e-02
2.1732097566539e-01 2.3176677063691e-01 6.0614097374409e-02
8.7872218705676e-01 2.3447407645982e-01 4.9248282279427e-02
5.4005049269975e-01 2.3718098639829e-01 3.7882975654320e-02
2.0130636210848e-01 2.3988749681682e-01 2.6518186110880e-02
9.9994403408544e-01 1.3190495611314e-01 5.0403800104213e-01
6.6786707364239e-01 1.3462422581367e-01 4.9267006878072e-01
3.3543142802013e-01 1.3734303917169e-01 4.8130210936617e-01
2.6378559974820e-03 1.4006131452745e-01 4.6993407745308e-01
6.6949286887958e-01 1.4277902022128e-01 4.5856592022010e-01
3.3600738305097e-01 1.4549616090456e-01 4.4719758673999e-01
2.1954236704752e-03 1.4821276536529e-01 4.3582903374595e-01
6.6807298363468e-01 1.5092887660234e-01 4.2446022849280e-01
3.3365708577392e-01 1.5364454428747e-01 4.1309114954393e-01
9.9896505534616e-01 1.5635981940012e-01 4.0172178624460e-01
6.6401398510673e-01 1.5907475065818e-01 3.9035213748161e-01
3.2882036341377e-01 1.6178938233385e-01 3.7898221015304e-01
9.9339983277132e-01 1.6450375307905e-01 3.6761201761784e-01
6.5776704820041e-01 1.6721789545058e-01 3.5624157827762e-01
3.2193560917169e-01 1.6993183589582e-01 3.4487091436157e-01
9.8591804384607e-01 1.7264559502380e-01 3.3350005093339e-01
6.4972582912989e-01 1.7535918803856e-01 3.2212901510936e-01
3.1336943414780e-01 1.7807262525195e-01 3.1075783546116e-01
9.7685837805290e-01 1.8078591262221e-01 2.9938654157176e-01
6.4020129567194e-01 1.8349905228534e-01 2.8801516371211e-01
3.0340600642652e-01 1.8621204306047e-01 2.7664373260931e-01
9.6647958340537e-01 1.8892488091948e-01 2.6527227928077e-01
6.2942842049952e-01 1.9163755941729e-01 2.5390083491347e-01
2.9225829625326e-01 1.9435007008260e-01 2.4252943077120e-01
9.5497443360505e-01 1.9706240277111e-01 2.3115809811686e-01
6.1758155505644e-01 1.9977454598407e-01 2.1978686813962e-01
2.8008393305837e-01 2.0248648715562e-01 2.0841577187977e-01
9.4248543557677e-01 2.0519821291239e-01 1.9704484014610e-01
6.0478956691672e-01 2.0790970930863e-01 1.8567410342246e-01
2.6699950396504e-01 2.1062096203979e-01 1.7430359176163e-01
9.2911812806518e-01 2.1333195663731e-01 1.6293333466587e-01
5.9114805277567e-01 2.1604267864694e-01 1.5156336095422e-01
2.5309164778853e-01 2.1875311379242e-01 1.4019369861773e-01
9.1495105930107e-01 2.2146324812632e-01 1.2882437466411e-01
5.7672822714544e-01 2.2417306816922e-01 1.1745541495412e-01
2.3842489898653e-01 2.2688256103831e-01 1.0608684403208e-01
9.0004264190079e-01 2.2959171456604e-01 9.4718684953533e-02
5.6158285164693e-01 2.3230051740944e-01 8.3350959113159e-02
2.2304675993311e-01 2.3500895915014e-01 7.1983686076170e-02
8.8443543997541e-01 2.3771703038529e-01 6.0616883416612e-02
5.4574981062756e-01 2.4042472280915e-01 4.9250566565858e-02
6.6648026569000e-01 1.3785300200927e-01 5.0404641171384e-01
3.3477279259208e-01 1.4057257818302e-01 4.9267864949896e-01
2.6647791943224e-03 1.4329162589891e-01 4.8131086906159e-01
6.7015760960924e-01 1.4601005491623e-01 4.6994301597659e-01
3.3725963444175e-01 1.4872783530644e-01 4.5857502756054e-01
3.9844241474495e-03 1.5144497988319e-01 4.4720684444813e-01
6.7034911522053e-01 1.5416152887734e-01 4.3583841736431e-01
3.3637298475751e-01 1.5687753775522e-01 4.2446971014585e-01
2.0763080213415e-03 1.5959306824855e-01 4.1310070019852e-01
6.6747949783081e-01 1.6230818220567e-01 4.0173137742704e-01
3.3260249338609e-01 1.6502293769362e-01 3.9036174241822e-01
9.9746435345147e-01 1.6773738677252e-01 3.7899180439940e-01
6.6208300774787e-01 1.7045157444024e-01 3.6762157928275e-01
3.2647512538684e-01 1.7316553835135e-01 3.5625108795376e-01
9.9065606647774e-01 1.7587930901718e-01 3.4488035486260e-01
6.5463989052324e-01 1.7859291028112e-01 3.3350940691837e-01
3.1843940185287e-01 1.8130635993140e-01 3.2213827265544e-01
9.8206621776887e-01 1.8401967036266e-01 3.1076698162811e-01
6.4553084928898e-01 1.8673284923294e-01 2.9939556398753e-01
3.0884278752040e-01 1.8944590008568e-01 2.8802405019799e-01
9.7201059099109e-01 1.9215882292189e-01 2.7665247085526e-01
6.3504197089648e-01 1.9487161471691e-01 2.6528085657632e-01
2.9794387235913e-01 1.9758426988208e-01 2.5390923793551e-01
9.6072255058184e-01 2.0029678067435e-01 2.4253764542820e-01
6.2338364130323e-01 2.0300913755902e-01 2.3116610944706e-01
2.8593222531548e-01 2.0572132953062e-01 2.1979466026037e-01
9.4837288703262e-01 2.0843334439765e-01 2.0842332798433e-01
6.1070976724265e-01 2.1114516903588e-01 1.9705214254429e-01
2.7294661026716e-01 2.1385678961512e-01 1.8568113362117e-01
9.3508680580584e-01 2.1656819180329e-01 1.7431033058122e-01
5.9713342577423e-01 2.1927936095137e-01 1.6293976238828e-01
2.5908925645896e-01 2.2199028226238e-01 1.5156945749861e-01
9.2095682632360e-01 2.2470094094682e-01 1.4019944373923e-01
5.8273842979985e-01 2.2741132236674e-01 1.2882974817112e-01
2.4443614739997e-01 2.3012141217030e-01 1.1746039693938e-01
9.0605186248386e-01 2.3283119641791e-01 1.0609141511277e-01
5.6758727501145e-01 2.3554066170126e-01 9.4722826515200e-02
2.2904391260670e-01 2.3824979525568e-01 8.3354653552534e-02
8.9042313925327e-01 2.4095858506633e-01 7.1986917037691e-02
3.3300097813007e-01 1.4380120084424e-01 5.0405447748958e-01
1.7066909319572e-03 1.4652113555796e-01 4.9268689306763e-01
6.6996313147413e-01 1.4924045611358e-01 4.8131930136632e-01
3.3777256273526e-01 1.5195906238873e-01 4.6995163670135e-01
5.1457522012247e-03 1.5467692755523e-01 4.5858382442643e-01
6.7209967946393e-01 1.5739407526158e-01 4.4721579534445e-01
3.3865540556283e-01 1.6011056017476e-01 4.3584749363980e-01
4.8362806532204e-03 1.6282645295951e-01 4.2447887993776e-01
6.7066654876535e-01 1.6554182960910e-01 4.1310993119877e-01
3.3617033224795e-01 1.6825676445282e-01 4.0174063886677e-01
1.3709400122911e-03 1.7097132598272e-01 3.9037100628386e-01
6.6629043159197e-01 1.7368557468764e-01 3.7900104600581e-01
3.3094937206935e-01 1.7639956222722e-01 3.6763077736486e-01
9.9536672281768e-01 1.7911333144343e-01 3.5626022443097e-01
6.5955982471350e-01 1.8182691685514e-01 3.4488941440447e-01
3.2354444139320e-01 1.8454034539846e-01 3.3351837640972e-01
9.8733483921803e-01 1.8725363726239e-01 3.2214714063027e-01
6.5094388768486e-01 1.8996680672911e-01 3.1077573771897e-01
3.1438316927612e-01 1.9267986296873e-01 2.9940419841900e-01
9.7766309152195e-01 1.9539281076410e-01 2.8803255334030e-01
6.4079299669356e-01 1.9810565115667e-01 2.7666083284568e-01
3.0378126635964e-01 2.0081838201392e-01 2.6528906700994e-01
9.6663541925204e-01 2.0353099852356e-01 2.5391728562415e-01
6.2936220168484e-01 2.0624349362210e-01 2.4254551822378e-01
2.9196767028027e-01 2.0895585836598e-01 2.3117379412514e-01
9.5445726707163e-01 2.1166808225352e-01 2.1980214245903e-01
6.1683588724045e-01 2.1438015350507e-01 2.0843059219366e-01
2.7910793984934e-01 2.1709205930827e-01 1.9705917214182e-01
9.4127740198249e-01 2.1980378603426e-01 1.8568791094867e-01
6.0334786672496e-01 2.2251531942990e-01 1.7431683705873e-01
2.6532258541092e-01 2.2522664479058e-01 1.6294597866095e-01
9.2720450456146e-01 2.2793774711722e-01 1.5157536361232e-01
5.8899629791759e-01 2.3064861126065e-01 1.4020501934061e-01
2.5070039395924e-01 2.3335922205610e-01 1.2883497272777e-01
9.1231899928593e-01 2.3606956444978e-01 1.1746524997577e-01
5.7385411822277e-01 2.3877962361957e-01 1.0609587645706e-01
2.3530756900400e-01 2.4148938509081e-01 9.4726876552416e-02
9.9950352727287e-01 1.4974955619379e-01 5.0406218402189e-01
6.6867349268361e-01 1.5246991150318e-01 4.9269478507105e-01
3.3733685852613e-01 1.5518954997674e-01 4.8132739212220e-01
5.4971327416566e-03 1.5790836017172e-01 4.6995992534319e-01
6.7316829735082e-01 1.6062632044987e-01 4.5859229544708e-01
3.4037175872942e-01 1.6334346896677e-01 4.4722442176855e-01
7.1335740243052e-03 1.6605987874310e-01 4.3585624149607e-01
6.7348218307176e-01 1.6877563906749e-01 4.2448771253653e-01
3.3944670189299e-01 1.7149084283658e-01 4.1311881249204e-01
5.0557301883502e-03 1.7420557869934e-01 4.0174953569616e-01
6.7033657960204e-01 1.7691992672254e-01 3.9037988961342e-01
3.3531482115447e-01 1.7963395644324e-01 3.7900989135793e-01
1.4060990134620e-05 1.8234772642662e-01 3.6763956469734e-01
6.6445587136221e-01 1.8506128469964e-01 3.5626893766353e-01
3.2865982193140e-01 1.8777466963902e-01 3.4489804075697e-01
9.9264357206671e-01 1.9048791104752e-01 3.3352690566729e-01
6.5642299707726e-01 1.9320103126023e-01 3.2215556441217e-01
3.2001233036203e-01 1.9591404619428e-01 3.1078404879755e-01
9.8342430989899e-01 1.9862696629998e-01 2.9941239011427e-01
6.4667032192740e-01 2.0133979739835e-01 2.8804061900135e-01
3.0976053765009e-01 2.0405254140558e-01 2.7666876542102e-01
9.7270404072984e-01 2.0676519695256e-01 2.6529685870390e-01
6.3550894459551e-01 2.0947775991113e-01 2.5392492763336e-01
2.9818249934085e-01 2.1219022383997e-01 2.4255300054689e-01
9.6073118845338e-01 2.1490258036219e-01 2.3118110543855e-01
6.2316081586216e-01 2.1761481948593e-01 2.1980927005184e-01
2.8547658391587e-01 2.2032692987775e-01 2.0843752195559e-01
9.4768316294728e-01 2.2303889909746e-01 1.9706588859833e-01
6.0978475307685e-01 2.2575071380140e-01 1.8569439733843e-01
2.7178513888109e-01 2.2846235992069e-01 1.7432307544858e-01
9.3368773751077e-01 2.3117382281932e-01 1.6295195009432e-01
5.9549564080046e-01 2.3388508743678e-01 1.5158104828688e-01
2.5721165186851e-01 2.3659613841873e-01 1.4021039681125e-01
9.1883831666827e-01 2.3930696023901e-01 1.2884002213085e-01
5.8037795091820e-01 2.4201753731542e-01 1.1746995027050e-01
6.6598457677883e-01 1.5569807052558e-01 5.0406951734290e-01
3.3567911767458e-01 1.5841892095448e-01 4.9270231147240e-01
4.7988572979564e-03 1.6113893036303e-01 4.8133512761888e-01
6.7334908504661e-01 1.6385797445422e-01 4.6996786791946e-01
3.4134811563004e-01 1.6657603996064e-01 4.5860042501107e-01
8.8231712825029e-03 1.6929318462571e-01 4.4723270487132e-01
6.7580673160568e-01 1.7200950497189e-01 4.3586463743013e-01
3.4233364614177e-01 1.7472511320361e-01 4.2449617882265e-01
8.4390475198772e-03 1.7744012226536e-01 4.1312730891381e-01
6.7415696327175e-01 1.8015463718370e-01 4.0175802676782e-01
3.3951947406914e-01 1.8286875078176e-01 3.9038834572285e-01
4.5562677139747e-03 1.8558254219041e-01 3.7901828894130e-01
6.6929446270578e-01 1.8829607700294e-01 3.6764788578694e-01
3.3375860531113e-01 1.9100940829698e-01 3.5627716908418e-01
9.9797077147947e-01 1.9372257803512e-01 3.4490617316919e-01
6.6195072682676e-01 1.9643561855692e-01 3.3353493258845e-01
3.2571611407445e-01 1.9914855400637e-01 3.2216348129445e-01
9.8928264868627e-01 2.0186140162040e-01 3.1079185220504e-01
6.5266431112957e-01 2.0457417285186e-01 2.9942007701694e-01
3.1587352923266e-01 2.0728687432677e-01 2.8804818618878e-01
9.7892134732929e-01 2.0999950864875e-01 2.7667620903022e-01
6.4181758087556e-01 2.1271207506927e-01 2.6530417385131e-01
3.0457095641442e-01 2.1542457004323e-01 2.5393210813974e-01
9.6718923743456e-01 2.1813698768880e-01 2.4256003874372e-01
6.2967933701759e-01 2.2084932016810e-01 2.3118799204558e-01
2.9204741831344e-01 2.2356155800330e-01 2.1981599411656e-01
9.5429898391198e-01 2.2627369034035e-01 2.0844407084679e-01
6.1643895513941e-01 2.2898570517068e-01 1.9707224804714e-01
2.7847174223465e-01 2.3169758951929e-01 1.8570055152115e-01
9.4040130627315e-01 2.3440932960647e-01 1.7432900710660e-01
6.0223121361513e-01 2.3712091098912e-01 1.6295764068689e-01
2.6396468357029e-01 2.3983231868651e-01 1.5158647817292e-01
3.3243985982405e-01 1.6164674485028e-01 5.0407646393053e-01
2.7310784446778e-03 1.6436818069344e-01 4.9270945869104e-01
6.7236534589777e-01 1.6708862367474e-01 4.8134249469489e-01
3.4135053282620e-01 1.6980793512298e-01 4.6997545074965e-01
9.7108803187014e-03 1.7252611499463e-01 4.5860819698481e-01
6.7748134898424e-01 1.7524324770512e-01 4.4724062390795e-01
3.4470276003014e-01 1.7795946006729e-01 4.3587265432366e-01
1.1418099949980e-02 1.8067489256219e-01 4.2450424425034e-01
6.7766997556045e-01 1.8338968187779e-01 4.1313537819192e-01
3.4349902145853e-01 1.8610395165657e-01 4.0176606242443e-01
8.9430148507873e-03 1.8881780861747e-01 3.9039631837007e-01
6.7403647603548e-01 1.9153134187927e-01 3.7902617700413e-01
3.3881058091053e-01 1.9424462399900e-01 3.6765567456681e-01
3.2932608555697e-03 1.9695771278991e-01 3.5628484950973e-01
6.6750940576203e-01 1.9967065337366e-01 3.4491374046793e-01
3.3148111599034e-01 2.0238348017420e-01 3.3354238501923e-01
9.9522796989493e-01 2.0509621871564e-01 3.2217081901428e-01
6.5876728741419e-01 2.0780888717464e-01 3.1079907630052e-01
3.2211437910467e-01 2.1052149768535e-01 2.9942718870455e-01
9.8528277550982e-01 2.1323405741784e-01 2.8805518617391e-01
6.4828443505831e-01 2.1594656945949e-01 2.7668309700820e-01
3.1112993058324e-01 2.1865903353093e-01 2.6531094813158e-01
9.7382861557208e-01 2.2137144656533e-01 2.5393876537500e-01
6.3638877174156e-01 2.2408380317682e-01 2.4256657374746e-01
2.9881773970024e-01 2.2679609603949e-01 2.3119439768411e-01
9.6112203445020e-01 2.2950831619491e-01 2.1982226126381e-01
6.2330744737333e-01 2.3222045330271e-01 2.0845018839247e-01
2.8537913619601e-01 2.3493249584636e-01 1.9707820295068e-01
9.4734170426027e-01 2.3764443130346e-01 1.8570632890561e-01
6.0919927026558e-01 2.4035624628889e-01 1.7433459038760e-01
9.9886383502915e-01 1.6759557823115e-01 5.0408301077792e-01
6.6983904656998e-01 1.7031771003393e-01 4.9271621368878e-01
3.4005698385939e-01 1.7303866096584e-01 4.8134948085031e-01
9.5293149120283e-03 1.7575827671595e-01 4.6998266038919e-01
6.7828852108153e-01 1.7847657791452e-01 4.5861559419208e-01
3.4638000261201e-01 1.8119368560873e-01 4.4724815507205e-01
1.3855534067778e-02 1.8390976587516e-01 4.3588025958771e-01
6.8076851660967e-01 1.8662499412479e-01 4.2451186638997e-01
3.4717089562933e-01 1.8933953504708e-01 4.1314296805351e-01
1.3111377960364e-02 1.9205353322716e-01 4.0177358136386e-01
6.7863455855934e-01 1.9476711025580e-01 3.9040373856008e-01
3.4378063325977e-01 1.9748036537184e-01 3.7903348045192e-01
8.5854616532901e-03 2.0019337776014e-01 3.6766285148967e-01
6.7308082134074e-01 2.0290620941718e-01 3.5629189650317e-01
3.3729475295071e-01 2.0561890800890e-01 3.4492065872560e-01
1.2519354513221e-03 2.0833150945289e-01 3.3354917873936e-01
6.6497405742347e-01 2.1104404012904e-01 3.2217749404737e-01
3.2848016627987e-01 2.1375651871189e-01 3.1080563904547e-01
9.9178698722269e-01 2.1646895765898e-01 2.9943364523578e-01
6.5490920940009e-01 2.1918136440425e-01 2.8806154157073e-01
3.1785973983299e-01 2.2189374230758e-01 2.7668935485494e-01
9.8064992725899e-01 2.2460609140672e-01 2.6531711015861e-01
6.4328975871663e-01 2.2731840901159e-01 2.5394483121416e-01
3.0578803186339e-01 2.3003069017368e-01 2.4257254078001e-01
9.6815250591815e-01 2.3274292805717e-01 2.3120026096329e-01
6.3039003388367e-01 2.3545511423292e-01 2.1982801349810e-01
2.9250667841444e-01 2.3816723891219e-01 2.0845581997873e-01
9.5450781339747e-01 2.4087929113330e-01 1.9708370204927e-01
6.6524917743707e-01 1.7354456707356e-01 5.0408914546772e-01
3.3701567914045e-01 1.7626753183184e-01 4.9272256406987e-01
7.9004769389625e-03 1.7898907951318e-01 4.8135607436991e-01
6.7792092810586e-01 1.8170903965710e-01 4.6998948342917e-01
3.4712118392026e-01 1.8442746511290e-01 4.5862259746948e-01
1.5560942474206e-02 1.8714452768189e-01 4.4725526956102e-01
6.8330652853498e-01 1.8986044457146e-01 4.3588741224016e-01
3.5042478450904e-01 1.9257543428457e-01 4.2451899121227e-01
1.6979388017306e-02 1.9528969426448e-01 4.1315001200548e-01
6.8302893904979e-01 1.9800339227479e-01 4.0178050621586e-01
3.4862620174254e-01 2.0071666540346e-01 3.9041052017658e-01
1.3818034521900e-02 2.0342962272884e-01 3.7904010672811e-01
6.7864569366040e-01 2.0614234935336e-01 3.6766931978495e-01
3.4314531341000e-01 2.0885491060633e-01 3.5629821107642e-01
7.3484472331026e-03 2.1156735586641e-01 3.4492682843224e-01
6.7128260662846e-01 2.1427972180793e-01 3.3355521509473e-01
3.3497176554606e-01 2.1699203505161e-01 3.2218340967091e-01
9.9843681664486e-01 2.1970431427779e-01 3.1081144645505e-01
6.6169597575738e-01 2.2241657188664e-01 2.9943935594231e-01
3.2476513619313e-01 2.2512881529071e-01 2.8806716541948e-01
9.8765817699342e-01 2.2784104791611e-01 2.7669489956377e-01
6.5038723017713e-01 2.3055326997605e-01 2.6532258101018e-01
3.1296291211587e-01 2.3326547906777e-01 2.5395023086704e-01
9.7539452386376e-01 2.3597767063332e-01 2.4257786917114e-01
6.3769022477439e-01 2.3868983831537e-01 2.3120551528054e-01
2.9985718319248e-01 2.4140197423221e-01 2.1983318820719e-01
3.3158600430297e-01 1.7949370406142e-01 5.0409485625395e-01
4.2778562146633e-03 1.8221767400116e-01 4.9272849820273e-01
6.7593092848194e-01 1.8493992504264e-01 4.8136226444836e-01
3.4657128950109e-01 1.8766027184668e-01 4.6999590602655e-01
1.6259962933153e-02 1.9037881759779e-01 4.5862918395492e-01
6.8507655432321e-01 1.9309580504255e-01 4.4726193033125e-01
3.5310695334373e-01 1.9581151816848e-01 4.3589405825157e-01
2.0435494158489e-02 1.9852622838331e-01 4.2452554742092e-01
6.8714066771455e-01 2.0124017091032e-01 4.1315642314439e-01
3.5329322678885e-01 2.0395353852235e-01 4.0178673728528e-01
1.8955724936112e-02 2.0666648380351e-01 3.9041655401415e-01
6.8418282819212e-01 2.0937912478433e-01 3.7904594037393e-01
3.4902199008824e-01 2.1209155128460e-01 3.6767496067911e-01
1.3514255665482e-02 2.1480383076266e-01 3.5630367362040e-01
6.7769507006411e-01 2.1751601324596e-01 3.4493213112589e-01
3.4159503172650e-01 2.2022813529035e-01 3.3356037828077e-01
5.2405660981208e-03 2.2294022307095e-01 3.2218845383205e-01
6.6865451459193e-01 2.2565229475473e-01 3.1081639097474e-01
3.3185664282566e-01 2.2836436230395e-01 2.9944421823581e-01
9.9486407611069e-01 2.3107643283974e-01 2.8807196035030e-01
6.5769167136819e-01 2.3378850967052e-01 2.7669963907458e-01
3.2035233447027e-01 2.3650059306713e-01 2.6532727391166e-01
9.8285729120651e-01 2.3921268084724e-01 2.5395488274132e-01
6.4521631906083e-01 2.4192476881605e-01 2.4258248235753e-01
9.9786065925162e-01 1.8544297650678e-01 5.0410013215780e-01
6.7164858890826e-01 1.8816817184722e-01 4.9273400537847e-01
3.4419537382720e-01 1.9089125492816e-01 4.8136804128964e-01
1.5540770636634e-02 1.9361203068170e-01 4.7000191288370e-01
6.8577077494822e-01 1.9633068148757e-01 4.5863532395541e-01
3.5499321406269e-01 1.9904755011392e-01 4.4726808657545e-01
2.3320621349542e-02 2.0176300778978e-01 4.3590012303983e-01
6.9086027739335e-01 2.0447739021602e-01 4.2453143773171e-01
3.5770945545076e-01 2.0719097520413e-01 4.1316208503271e-01
2.3953853468147e-02 2.0990398144905e-01 4.0179214369732e-01
6.8966773953651e-01 2.1261657602193e-01 3.9042169965726e-01
3.5491489662059e-01 2.1532888418007e-01 3.7905083590595e-01
1.9749855692919e-02 2.1804099865894e-01 3.6767962737630e-01
6.8421915619485e-01 2.2075298745600e-01 3.5630813898138e-01
3.4836251420775e-01 2.2346489996315e-01 3.4493642548694e-01
1.2213853694857e-02 2.2617677163736e-01 3.3356453234043e-01
6.7580219340389e-01 2.2888862749020e-01 3.2219249692045e-01
3.3915239891595e-01 2.3160048466758e-01 3.1082034989906e-01
2.2858157514097e-03 2.3431235434709e-01 2.9944811655322e-01
6.6522080074582e-01 2.3702424313098e-01 2.8807581794848e-01
3.2797316789476e-01 2.3973615406924e-01 2.7670347196713e-01
6.6405373015024e-01 1.9139236368326e-01 5.0410496308972e-01
3.3916006357925e-01 1.9411907176562e-01 4.9273907603440e-01
1.2758250055637e-02 1.9684314285607e-01 4.8137339609692e-01
6.8491023159089e-01 1.9956438554888e-01 4.7000748507353e-01
3.5573996141761e-01 2.0228310820722e-01 4.5864097511219e-01
2.5396068403463e-02 2.0499979568529e-01 4.4727366415560e-01
6.9402784388246e-01 2.0771493271306e-01 4.3590549920492e-01
3.6177289395010e-01 2.1042893167430e-01 4.2453652534305e-01
2.8752376283133e-02 2.1314211661217e-01 4.1316683821291e-01
6.9507035633793e-01 2.1585473135730e-01 4.0179655090054e-01
3.6081507045324e-01 2.1856695496277e-01 3.9042577452476e-01
2.6060881837984e-02 2.2127891711152e-01 3.7905460863144e-01
6.9087033415329e-01 2.2399071097894e-01 3.6768313764864e-01
3.5529604519502e-01 2.2670240314455e-01 3.5631143069301e-01
1.9382352538087e-02 2.2941404092010e-01 3.4493954300983e-01
6.8316670137461e-01 2.3212565764932e-01 3.3356751804950e-01
3.4668079715290e-01 2.3483727649774e-01 3.2219538964622e-01
9.9515560146608e-03 2.3754891315049e-01 3.1082318404047e-01
6.7300188659185e-01 2.4026057773155e-01 2.9945092164162e-01
3.3013667143652e-01 1.9734183231429e-01 5.0410934002487e-01
6.8586967259411e-03 2.0007043731602e-01 4.9274370209655e-01
6.8171014937928e-01 2.0279568571595e-01 4.8137832071326e-01
3.5479029217792e-01 2.0551742071371e-01 4.7001259536128e-01
2.6282559256092e-02 2.0823615399425e-01 4.5864607126515e-01
6.9639583492616e-01 2.1095257330674e-01 4.4727854869712e-01
3.6533078085304e-01 2.1366730933954e-01 4.3591002621592e-01
3.3265328132693e-02 2.1638086295603e-01 4.2454061281276e-01
7.0035103431374e-01 2.1909360525763e-01 4.1317046028987e-01
3.6671446456646e-01 2.2180580163151e-01 4.0179972317420e-01
3.2460397108047e-02 2.2451763857054e-01 3.9042853933055e-01
6.9767534876377e-01 2.2722924613649e-01 3.7905702310758e-01
3.6243081139729e-01 2.2994071482512e-01 3.6768526505361e-01
2.6786010656315e-02 2.3265210758821e-01 3.5631333457537e-01
6.9079017724232e-01 2.3536346818590e-01 3.4494128356803e-01
3.5448438645538e-01 2.3807482692164e-01 3.3356915004943e-01
1.7903037990482e-02 2.4078620456587e-01 3.2219696138712e-01
9.9606573301239e-01 2.0329132851504e-01 5.0411325529231e-01
6.7481393794012e-01 2.0602235962338e-01 4.9274787754880e-01
3.5118238351305e-01 2.0874901387340e-01 4.8138280621780e-01
2.5336015223596e-02 2.1147123819779e-01 4.7001719786900e-01
6.9755479535438e-01 2.1418987804426e-01 4.5865050057273e-01
3.6813863484350e-01 2.1690591095156e-01 4.4728255507748e-01
3.7360862083067e-02 2.1962015068800e-01 4.3591345636869e-01
7.0545324077401e-01 2.2233319430324e-01 4.2454340899628e-01
3.7260593072655e-01 2.2504545535051e-01 4.1317263672266e-01
3.8972854135097e-02 2.2775721306982e-01 4.0180133956890e-01
7.0467814297077e-01 2.3046865438695e-01 3.9042967935273e-01
3.6982204124167e-01 2.3317990449704e-01 3.7905777927895e-01
3.4485814929467e-02 2.3589104768542e-01 3.6768572919445e-01
6.9873567356671e-01 2.3860214107833e-01 3.5631359230882e-01
3.6262584891734e-01 2.4131322361606e-01 3.4494141154704e-01
6.6177030825800e-01 2.0924076243242e-01 5.0411670312388e-01
3.4313450994857e-01 2.1197497610826e-01 4.9275159944668e-01
2.1372349730861e-02 2.1470330645738e-01 4.8138683838004e-01
6.9677097353383e-01 2.1742595942207e-01 4.7002120431606e-01
3.6977343640652e-01 2.2014433834666e-01 4.5865406107423e-01
4.0820443430403e-02 2.2285983052357e-01 4.4728537121021e-01
7.1028915417964e-01 2.2557346810797e-01 4.3591539727984e-01
3.7848319691123e-01 2.2828594129239e-01 4.2454447755206e-01
4.5641370566851e-02 2.3099769244215e-01 4.1317291878104e-01
7.1195038083675e-01 2.3370900164201e-01 4.0180096193562e-01
3.7755652545060e-01 2.3642004684376e-01 3.9042878158894e-01
4.2575111241556e-02 2.3913094244333e-01 3.7905649717026e-01
7.0709768469209e-01 2.4184176313552e-01 3.6768418626489e-01
3.2712864113667e-01 2.1518997650581e-01 5.0411968084974e-01
1.2000585091929e-02 2.1792850632000e-01 4.9275486981447e-01
6.9259017403144e-01 2.2065881344882e-01 4.8139038322091e-01
3.6942826449509e-01 2.2338172260736e-01 4.7002442655596e-01
4.3246553999888e-02 2.2609958460022e-01 4.5865636719794e-01
7.1470862703918e-01 2.2881434798902e-01 4.4728645266865e-01
3.8434084136114e-01 2.3152727945642e-01 4.3591521480759e-01
5.2542080291208e-02 2.3423913767004e-01 4.2454315826622e-01
7.1961136218814e-01 2.3695036747371e-01 4.1317066543359e-01
3.8577360278254e-01 2.3966123152465e-01 4.0179799524855e-01
9.9191139506193e-01 2.2113867274217e-01 5.0412219188807e-01
6.8173359317437e-01 2.2388332568385e-01 4.9275769916460e-01
3.6535026747536e-01 2.2661588425218e-01 4.8139333891115e-01
4.3823550873087e-02 2.2933866987387e-01 4.7002642895782e-01
7.1842373070330e-01 2.3205565127035e-01 4.5865664626039e-01
3.9017426543210e-01 2.3476948543562e-01 4.4728482365148e-01
5.9815116840732e-02 2.3748163344792e-01 4.3591187224240e-01
7.2786877638031e-01 2.4019286292583e-01 4.2453845258260e-01
6.5562962803065e-01 2.2708622151038e-01 5.0412425442720e-01
3.5296691342927e-01 2.2984013107531e-01 4.9276011129213e-01
4.0565935081913e-02 2.3257499045166e-01 4.8139535993383e-01
7.2079159734539e-01 2.3529691784655e-01 4.7002612303606e-01
3.9597966709834e-01 2.3801257111110e-01 4.5865328455561e-01
6.7740014450948e-02 2.4072532164011e-01 4.4727870711356e-01
3.1702535212619e-01 2.3303105326726e-01 5.0412593062091e-01
2.7138364583282e-02 2.3580034960338e-01 4.9276212692510e-01
7.2000555331716e-01 2.3853668617161e-01 4.8139512255920e-01
4.0175401662234e-01 2.4125654525640e-01 4.7002058266265e-01
9.7161573518282e-01 2.3896815530093e-01 5.0412739267691e-01
7.0830699004793e-01 2.4176719523495e-01 4.9276334666603e-01
"""